# Homework 3 Solutions

**Name**:

**ID**:

> **Due Date**
>
> Thursday, 10/03/23, 9:00pm

## Overview

### Load Environment

The following code loads the environment and makes sure all needed
packages are installed. This should be at the start of most Julia
scripts.

In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

In [1]:
using Random
using CSV
using DataFrames
using Plots
using LaTeXStrings
using Distributions

## Problems (Total: 50/60 Points)

### Problem 1 (30 points)

The first step is to write function(s) to implement the dissolved oxygen
simulation. With two releases, we can turn this into a two-box model,
with the first box from the initial waste release ($x=0\ \text{km}$) to
the second release ($x=15\ \text{km}$), and the second from the second
release to the end of the domain ($x=50\ \text{km}$). As a result, our
lives will be easiest if we write a function to simulate each box with
appropriate initial conditions, which we can then call for each river
segment. An example of how this might look is below. Note that we need
to compute B and N as well to get the appropriate initial conditions at
the transition between boxes (and this might also help with debugging).

``` julia
# mix_concentration: function to compute initial conditions by mixing inflow and new waste stream concentrations
# inputs:
#   - arguments ending in "_in" are inflow, those ending in "_st" are from the stream
#   - V is the volume (L/d), Q is the relevant concentration (mg/L); these should be Floats
# outputs:
#   - mixed concentration (a Float) in mg/L
function mix_concentration(V_in, Q_in, V_st, Q_st)
    Q_mix = ((V_in * Q_in) + (V_st * Q_st)) / (V_in + V_st)
    return Q_mix
end

# dissolved_oxygen: function to simulate dissolved oxygen concentrations for a given segment
# inputs:
#   - x: vector or range of downstream distances to simulate over
#   - C₀, B₀, N₀: initial conditions for DO, CBOD, and NBOD, respectively (mg/L)
#   - U: velocity of river (km/d)
#   - Cₛ = saturation oxygen concentration (mg/L)
#   - ka, kc, kn: reaeration, CBOD decay, and NBOD decay rates, respectively (d^{-1})
function dissolved_oxygen(x, C₀, B₀, N₀, U, Cₛ, ka, kc, kn)

    # initialize vectors for C, B, and N
    # the zeros function lets us define a vector of the appropriate length with values set to zero
    C = zeros(length(x))
    B = zeros(length(x))
    N = zeros(length(x))

    # compute α values for the simulation
    α₁ = exp.(-ka * x / U)
    α₂ = (kc / (ka - kc)) * (exp.(-kc * x / U) .- α₁)
    α₃ = (kn / (ka - kn)) * (exp.(-kn * x / U) .- α₁)

    # loop over values in x to calculate B, N, and C
    for (i, d) in pairs(x)
        B[i] = B₀ * exp(-kb * x[i] / U)
        N[i] = N₀ * exp(-kc * x[i] / U)
        C[i] = Cₛ * (1 - α₁[i]) + (C₀ * α₁[i]) - (B₀ * α₂[i]) - (N₀ * α₃[i])
    end

    return (C, B, N)
end
```

    dissolved_oxygen (generic function with 1 method)

1.  These will be vectors due to the broadcasting of `exp` and `-` over
    the `x` vector. We could also have computed the `α` values in the
    loop below for each value of `x`.
2.  `pairs(x)` lets us directly iterate over indices (`i`) and values
    (`d`) in the vector `x`, rather than only iterating over indices and
    needing to look up the values `d=x[i]`.
3.  While we don’t need `B` and `N` for this solution, returning this
    tuple can be useful for debugging.

Next, let’s simulate the concentrations. Hopefully this is intuitive,
but one critical thing is that we need to compute the initial segment
from $x=0$ to $15\ \text{km}$, not just to 14, as $x=15$ is the inflow
for the initial condition of the segment after the second waste stream.

> **Note**
>
> I initially wrote this out as a script to debug, but then reformulated
> it as a function with an optional parameters for treatment of the two
> waste streams to solve Problems 2 and 3, which means I didn’t have to
> copy and paste everything, possibly introducing new bugs.

``` julia
# do_simulate: function to simulate dissolved oxygen concentrations over the entire river
# inputs:
#   - inflow: tuple with inflow properties: (Volume, DO, CBOD, NBOD)
#   - waste1: tuple with waste stream 1 properties: (Volume, DO, CBOD, NBOD)
#   - waste2: tuple with waste stream 2 properties: (Volume, DO, CBOD, NBOD)
#   - U: velocity of river (km/d)
#   - Cₛ = saturation oxygen concentration (mg/L)
#   - ka, kc, kn: reaeration, CBOD decay, and NBOD decay rates, respectively (d^{-1})
function do_simulate(inflow, waste1, waste2, U, Cₛ, ka, kb, kn)
    # set up ranges for each box/segment
    x₁ = 0:1:15
    x₂ = (15:1:50) .- 15

    V_inflow, C_inflow, B_inflow, N_inflow = inflow
    V_ws1, C_ws1, B_ws1, N_ws1 = waste1
    V_ws2, C_ws2, B_ws2, N_ws2 = waste2

    # initialize storage for final C, B, and N
    # need to store d=1 so the total length should be d+1
    C = zeros(51)
    B = zeros(51)
    N = zeros(51)

    # compute initial conditions for first segment
    C₀ = mix_concentration(V_inflow, C_inflow, V_ws1, C_ws1)
    B₀ = mix_concentration(V_inflow, B_inflow, V_ws1, B_ws1)
    N₀ = mix_concentration(V_inflow, N_inflow, V_ws1, N_ws1)

    # conduct first segment simulation
    (C₁, B₁, N₁) = dissolved_oxygen(x₁, C₀, B₀, N₀, U, Cₛ, ka, kb, kn)
    C[1:15] = C₁[1:end-1]
    B[1:15] = B₁[1:end-1]
    N[1:15] = N₁[1:end-1]

    # compute initial conditions for second segment
    C₀₁ = mix_concentration(V_inflow + V_ws1, C₁[end], V_ws2, C_ws2)
    B₀₁ = mix_concentration(V_inflow + V_ws1, B₁[end], V_ws2, B_ws2)
    N₀₁ = mix_concentration(V_inflow + V_ws1, N₁[end], V_ws2, N_ws2)

    # conduct second segment simulation
    (C₂, B₂, N₂) = dissolved_oxygen(x₂, C₀₁, B₀₁, N₀₁, U, Cₛ, ka, kb, kn)
    C[16:end] = C₂
    B[16:end] = B₂
    N[16:end] = N₂

    return (C, B, N)
end

# set variables for river dynamics
U = 6
Cₛ = 10
ka = 0.55
kb = 0.35
kn = 0.25

# set initial parameters
C_inflow = 7.5 # DO concentration
B_inflow = 5.0 # CBOD
N_inflow = 5.0 # NBOD
V_inflow = 100 * 1_000 # volume converted to L
inflow = (V_inflow, C_inflow, B_inflow, N_inflow)

# set waste stream parameters
C_ws1 = 5.0
B_ws1 = 50.0
N_ws1 = 35.0
V_ws1 = 10 * 1_000
waste1 = (V_ws1, C_ws1, B_ws1, N_ws1)

C_ws2 = 4.0
B_ws2 = 45.0
N_ws2 = 35.0
V_ws2 = 15 * 1_000
waste2 = (V_ws2, C_ws2, B_ws2, N_ws2)

C, B, N = do_simulate(inflow, waste1, waste2, U, Cₛ, ka, kb, kn)
```

    ([7.2727272727272725, 6.718366940001292, 6.252736478441488, 5.865982487427475, 5.549224625451668, 5.294464471079079, 5.094502688374758, 4.942863751725384, 4.8337275512230455, 4.761867260163488  …  6.13037601820527, 6.272760855101071, 6.412340434703638, 6.548872787980461, 6.6821623236453025, 6.812054438906417, 6.938430673374777, 7.061204354693571, 7.1803166891679835, 7.295733254966803], [9.090909090909092, 8.575776817031748, 8.089834281709308, 7.63142746153825, 7.198996057607107, 6.791068184640297, 6.4062553610792135, 6.043247783048131, 5.700809866118693, 5.377776039698319  …  1.916804331604075, 1.8081894764731885, 1.7057292332453498, 1.6090748536058688, 1.5178973509064155, 1.4318863803790853, 1.350749182802631, 1.274209588025372, 1.2020070749530312, 1.1338958848019383], [7.7272727272727275, 7.289410294476985, 6.876359139452911, 6.486713342307512, 6.11914664896604, 5.772407956944252, 5.4453170569173315, 5.136760615590911, 4.845688386200889, 4.571109633743571  …  1.5437012460963393, 1.4562281094565608, 1.3737115987526298, 1.2958708490057462, 1.2224409103247795, 1.1531718460849871, 1.087827882208027, 1.0261866046476376, 0.9680382023494304, 0.9131847531080266])

1.  The colon syntax sets up the range using the syntax
    `initial_value:stepsize:end_value`. In general a stepsize of 1 is
    implicit, but I’ve made it explicit here for illustration.
2.  This starts at 0 because we care about the distance from the initial
    condition, not the “absolute” distance.
3.  Tuples (including multiple outputs from functions) can be unpacked
    into multiple variables this way to make the subsequent code more
    readable (versus just relying on indexing).
4.  We don’t need to store the last value because that occurs at the
    point of mixing with the next waste stream. We will use it to
    compute the mixed concentration at that point.
5.  This was my mistake; I meant for the inflow/waste stream data to not
    include the last thousands digits, *e.g.* to be 100
    m<sup>3</sup>/day. But this should wash out in this problem when you
    compute the mixed concentrations.

Now we can plot the dissolved oxygen concentration, shown in
<a href="#fig-do-untreated" class="quarto-xref">Figure 1</a>.

``` julia
# create plot axis with labels, etc
p = plot(; xlabel="Distance Downriver (km)", ylabel="Dissolved Oxygen Concentration (mg/L)", legend=:top)
plot!(p, 0:50, C, label="Simulated DO")
hline!([4], label="Regulatory Standard")
```

<?xml version="1.0" encoding="utf-8"?>
<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="672" height="480" viewBox="0 0 2688 1920">
<defs>
  <clipPath id="clip350">
    <rect x="0" y="0" width="2688" height="1920"/>
  </clipPath>
</defs>
<path clip-path="url(#clip350)" d="M0 1920 L2688 1920 L2688 -4.26326e-14 L0 -4.26326e-14  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<defs>
  <clipPath id="clip351">
    <rect x="537" y="0" width="1883" height="1883"/>
  </clipPath>
</defs>
<path clip-path="url(#clip350)" d="M181.719 1734.12 L2640.76 1734.12 L2640.76 47.2441 L181.719 47.2441  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<defs>
  <clipPath id="clip352">
    <rect x="181" y="47" width="2460" height="1688"/>
  </clipPath>
</defs>
<polyline clip-path="url(#clip352)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="251.315,1734.12 251.315,47.2441 "/>
<polyline clip-path="url(#clip352)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="715.284,1734.12 715.284,47.2441 "/>
<polyline clip-path="url(#clip352)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="1179.25,1734.12 1179.25,47.2441 "/>
<polyline clip-path="url(#clip352)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="1643.22,1734.12 1643.22,47.2441 "/>
<polyline clip-path="url(#clip352)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="2107.19,1734.12 2107.19,47.2441 "/>
<polyline clip-path="url(#clip352)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="2571.16,1734.12 2571.16,47.2441 "/>
<polyline clip-path="url(#clip350)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="181.719,1734.12 2640.76,1734.12 "/>
<polyline clip-path="url(#clip350)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="251.315,1734.12 251.315,1715.22 "/>
<polyline clip-path="url(#clip350)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="715.284,1734.12 715.284,1715.22 "/>
<polyline clip-path="url(#clip350)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1179.25,1734.12 1179.25,1715.22 "/>
<polyline clip-path="url(#clip350)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1643.22,1734.12 1643.22,1715.22 "/>
<polyline clip-path="url(#clip350)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2107.19,1734.12 2107.19,1715.22 "/>
<polyline clip-path="url(#clip350)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2571.16,1734.12 2571.16,1715.22 "/>
<path clip-path="url(#clip350)" d="M251.315 1767.34 Q247.704 1767.34 245.875 1770.9 Q244.07 1774.45 244.07 1781.58 Q244.07 1788.68 245.875 1792.25 Q247.704 1795.79 251.315 1795.79 Q254.949 1795.79 256.755 1792.25 Q258.583 1788.68 258.583 1781.58 Q258.583 1774.45 256.755 1770.9 Q254.949 1767.34 251.315 1767.34 M251.315 1763.64 Q257.125 1763.64 260.181 1768.24 Q263.259 1772.83 263.259 1781.58 Q263.259 1790.3 260.181 1794.91 Q257.125 1799.49 251.315 1799.49 Q245.505 1799.49 242.426 1794.91 Q239.37 1790.3 239.37 1781.58 Q239.37 1772.83 242.426 1768.24 Q245.505 1763.64 251.315 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M689.972 1794.89 L697.61 1794.89 L697.61 1768.52 L689.3 1770.19 L689.3 1765.93 L697.564 1764.26 L702.24 1764.26 L702.24 1794.89 L709.879 1794.89 L709.879 1798.82 L689.972 1798.82 L689.972 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M729.323 1767.34 Q725.712 1767.34 723.883 1770.9 Q722.078 1774.45 722.078 1781.58 Q722.078 1788.68 723.883 1792.25 Q725.712 1795.79 729.323 1795.79 Q732.958 1795.79 734.763 1792.25 Q736.592 1788.68 736.592 1781.58 Q736.592 1774.45 734.763 1770.9 Q732.958 1767.34 729.323 1767.34 M729.323 1763.64 Q735.133 1763.64 738.189 1768.24 Q741.268 1772.83 741.268 1781.58 Q741.268 1790.3 738.189 1794.91 Q735.133 1799.49 729.323 1799.49 Q723.513 1799.49 720.434 1794.91 Q717.379 1790.3 717.379 1781.58 Q717.379 1772.83 720.434 1768.24 Q723.513 1763.64 729.323 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M1158.03 1794.89 L1174.35 1794.89 L1174.35 1798.82 L1152.4 1798.82 L1152.4 1794.89 Q1155.06 1792.13 1159.65 1787.5 Q1164.25 1782.85 1165.43 1781.51 Q1167.68 1778.98 1168.56 1777.25 Q1169.46 1775.49 1169.46 1773.8 Q1169.46 1771.04 1167.52 1769.31 Q1165.6 1767.57 1162.49 1767.57 Q1160.29 1767.57 1157.84 1768.34 Q1155.41 1769.1 1152.63 1770.65 L1152.63 1765.93 Q1155.46 1764.79 1157.91 1764.21 Q1160.36 1763.64 1162.4 1763.64 Q1167.77 1763.64 1170.97 1766.32 Q1174.16 1769.01 1174.16 1773.5 Q1174.16 1775.63 1173.35 1777.55 Q1172.56 1779.45 1170.46 1782.04 Q1169.88 1782.71 1166.78 1785.93 Q1163.67 1789.12 1158.03 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M1194.16 1767.34 Q1190.55 1767.34 1188.72 1770.9 Q1186.92 1774.45 1186.92 1781.58 Q1186.92 1788.68 1188.72 1792.25 Q1190.55 1795.79 1194.16 1795.79 Q1197.79 1795.79 1199.6 1792.25 Q1201.43 1788.68 1201.43 1781.58 Q1201.43 1774.45 1199.6 1770.9 Q1197.79 1767.34 1194.16 1767.34 M1194.16 1763.64 Q1199.97 1763.64 1203.03 1768.24 Q1206.1 1772.83 1206.1 1781.58 Q1206.1 1790.3 1203.03 1794.91 Q1199.97 1799.49 1194.16 1799.49 Q1188.35 1799.49 1185.27 1794.91 Q1182.22 1790.3 1182.22 1781.58 Q1182.22 1772.83 1185.27 1768.24 Q1188.35 1763.64 1194.16 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M1632.06 1780.19 Q1635.42 1780.9 1637.3 1783.17 Q1639.19 1785.44 1639.19 1788.77 Q1639.19 1793.89 1635.68 1796.69 Q1632.16 1799.49 1625.68 1799.49 Q1623.5 1799.49 1621.19 1799.05 Q1618.89 1798.64 1616.44 1797.78 L1616.44 1793.27 Q1618.38 1794.4 1620.7 1794.98 Q1623.01 1795.56 1625.54 1795.56 Q1629.94 1795.56 1632.23 1793.82 Q1634.54 1792.08 1634.54 1788.77 Q1634.54 1785.72 1632.39 1784.01 Q1630.26 1782.27 1626.44 1782.27 L1622.41 1782.27 L1622.41 1778.43 L1626.63 1778.43 Q1630.07 1778.43 1631.9 1777.06 Q1633.73 1775.67 1633.73 1773.08 Q1633.73 1770.42 1631.83 1769.01 Q1629.96 1767.57 1626.44 1767.57 Q1624.52 1767.57 1622.32 1767.99 Q1620.12 1768.4 1617.48 1769.28 L1617.48 1765.12 Q1620.14 1764.38 1622.46 1764.01 Q1624.8 1763.64 1626.86 1763.64 Q1632.18 1763.64 1635.28 1766.07 Q1638.38 1768.47 1638.38 1772.59 Q1638.38 1775.46 1636.74 1777.46 Q1635.1 1779.42 1632.06 1780.19 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M1658.06 1767.34 Q1654.45 1767.34 1652.62 1770.9 Q1650.81 1774.45 1650.81 1781.58 Q1650.81 1788.68 1652.62 1792.25 Q1654.45 1795.79 1658.06 1795.79 Q1661.69 1795.79 1663.5 1792.25 Q1665.33 1788.68 1665.33 1781.58 Q1665.33 1774.45 1663.5 1770.9 Q1661.69 1767.34 1658.06 1767.34 M1658.06 1763.64 Q1663.87 1763.64 1666.93 1768.24 Q1670 1772.83 1670 1781.58 Q1670 1790.3 1666.93 1794.91 Q1663.87 1799.49 1658.06 1799.49 Q1652.25 1799.49 1649.17 1794.91 Q1646.12 1790.3 1646.12 1781.58 Q1646.12 1772.83 1649.17 1768.24 Q1652.25 1763.64 1658.06 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M2095.36 1768.34 L2083.56 1786.78 L2095.36 1786.78 L2095.36 1768.34 M2094.14 1764.26 L2100.02 1764.26 L2100.02 1786.78 L2104.95 1786.78 L2104.95 1790.67 L2100.02 1790.67 L2100.02 1798.82 L2095.36 1798.82 L2095.36 1790.67 L2079.76 1790.67 L2079.76 1786.16 L2094.14 1764.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M2122.68 1767.34 Q2119.07 1767.34 2117.24 1770.9 Q2115.43 1774.45 2115.43 1781.58 Q2115.43 1788.68 2117.24 1792.25 Q2119.07 1795.79 2122.68 1795.79 Q2126.31 1795.79 2128.12 1792.25 Q2129.95 1788.68 2129.95 1781.58 Q2129.95 1774.45 2128.12 1770.9 Q2126.31 1767.34 2122.68 1767.34 M2122.68 1763.64 Q2128.49 1763.64 2131.54 1768.24 Q2134.62 1772.83 2134.62 1781.58 Q2134.62 1790.3 2131.54 1794.91 Q2128.49 1799.49 2122.68 1799.49 Q2116.87 1799.49 2113.79 1794.91 Q2110.73 1790.3 2110.73 1781.58 Q2110.73 1772.83 2113.79 1768.24 Q2116.87 1763.64 2122.68 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M2545.86 1764.26 L2564.22 1764.26 L2564.22 1768.2 L2550.14 1768.2 L2550.14 1776.67 Q2551.16 1776.32 2552.18 1776.16 Q2553.2 1775.97 2554.22 1775.97 Q2560 1775.97 2563.38 1779.15 Q2566.76 1782.32 2566.76 1787.73 Q2566.76 1793.31 2563.29 1796.41 Q2559.82 1799.49 2553.5 1799.49 Q2551.32 1799.49 2549.05 1799.12 Q2546.81 1798.75 2544.4 1798.01 L2544.4 1793.31 Q2546.48 1794.45 2548.71 1795 Q2550.93 1795.56 2553.41 1795.56 Q2557.41 1795.56 2559.75 1793.45 Q2562.09 1791.34 2562.09 1787.73 Q2562.09 1784.12 2559.75 1782.02 Q2557.41 1779.91 2553.41 1779.91 Q2551.53 1779.91 2549.66 1780.33 Q2547.8 1780.74 2545.86 1781.62 L2545.86 1764.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M2585.98 1767.34 Q2582.36 1767.34 2580.54 1770.9 Q2578.73 1774.45 2578.73 1781.58 Q2578.73 1788.68 2580.54 1792.25 Q2582.36 1795.79 2585.98 1795.79 Q2589.61 1795.79 2591.42 1792.25 Q2593.24 1788.68 2593.24 1781.58 Q2593.24 1774.45 2591.42 1770.9 Q2589.61 1767.34 2585.98 1767.34 M2585.98 1763.64 Q2591.79 1763.64 2594.84 1768.24 Q2597.92 1772.83 2597.92 1781.58 Q2597.92 1790.3 2594.84 1794.91 Q2591.79 1799.49 2585.98 1799.49 Q2580.17 1799.49 2577.09 1794.91 Q2574.03 1790.3 2574.03 1781.58 Q2574.03 1772.83 2577.09 1768.24 Q2580.17 1763.64 2585.98 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M1016.83 1841.06 L1016.83 1878.02 L1024.6 1878.02 Q1034.44 1878.02 1038.99 1873.56 Q1043.57 1869.11 1043.57 1859.49 Q1043.57 1849.94 1038.99 1845.52 Q1034.44 1841.06 1024.6 1841.06 L1016.83 1841.06 M1010.41 1835.78 L1023.61 1835.78 Q1037.43 1835.78 1043.89 1841.54 Q1050.35 1847.27 1050.35 1859.49 Q1050.35 1871.78 1043.86 1877.54 Q1037.36 1883.3 1023.61 1883.3 L1010.41 1883.3 L1010.41 1835.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M1060.34 1847.65 L1066.2 1847.65 L1066.2 1883.3 L1060.34 1883.3 L1060.34 1847.65 M1060.34 1833.78 L1066.2 1833.78 L1066.2 1841.19 L1060.34 1841.19 L1060.34 1833.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M1101.18 1848.7 L1101.18 1854.24 Q1098.7 1852.97 1096.02 1852.33 Q1093.35 1851.7 1090.49 1851.7 Q1086.13 1851.7 1083.93 1853.03 Q1081.76 1854.37 1081.76 1857.04 Q1081.76 1859.08 1083.32 1860.26 Q1084.88 1861.4 1089.59 1862.45 L1091.6 1862.9 Q1097.84 1864.24 1100.45 1866.69 Q1103.09 1869.11 1103.09 1873.47 Q1103.09 1878.43 1099.14 1881.33 Q1095.23 1884.22 1088.35 1884.22 Q1085.49 1884.22 1082.37 1883.65 Q1079.28 1883.11 1075.84 1882 L1075.84 1875.95 Q1079.09 1877.64 1082.24 1878.49 Q1085.39 1879.32 1088.48 1879.32 Q1092.62 1879.32 1094.85 1877.92 Q1097.07 1876.49 1097.07 1873.91 Q1097.07 1871.52 1095.45 1870.25 Q1093.86 1868.98 1088.42 1867.8 L1086.38 1867.32 Q1080.94 1866.18 1078.52 1863.82 Q1076.1 1861.43 1076.1 1857.3 Q1076.1 1852.27 1079.66 1849.53 Q1083.23 1846.79 1089.79 1846.79 Q1093.03 1846.79 1095.9 1847.27 Q1098.76 1847.75 1101.18 1848.7 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M1118.21 1837.53 L1118.21 1847.65 L1130.27 1847.65 L1130.27 1852.2 L1118.21 1852.2 L1118.21 1871.56 Q1118.21 1875.92 1119.39 1877.16 Q1120.6 1878.4 1124.26 1878.4 L1130.27 1878.4 L1130.27 1883.3 L1124.26 1883.3 Q1117.48 1883.3 1114.9 1880.79 Q1112.32 1878.24 1112.32 1871.56 L1112.32 1852.2 L1108.02 1852.2 L1108.02 1847.65 L1112.32 1847.65 L1112.32 1837.53 L1118.21 1837.53 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M1154.17 1865.38 Q1147.08 1865.38 1144.34 1867 Q1141.6 1868.63 1141.6 1872.54 Q1141.6 1875.66 1143.64 1877.51 Q1145.71 1879.32 1149.24 1879.32 Q1154.11 1879.32 1157.04 1875.88 Q1160 1872.42 1160 1866.69 L1160 1865.38 L1154.17 1865.38 M1165.86 1862.96 L1165.86 1883.3 L1160 1883.3 L1160 1877.89 Q1157.99 1881.14 1155 1882.7 Q1152.01 1884.22 1147.68 1884.22 Q1142.21 1884.22 1138.96 1881.17 Q1135.75 1878.08 1135.75 1872.92 Q1135.75 1866.91 1139.76 1863.85 Q1143.8 1860.8 1151.79 1860.8 L1160 1860.8 L1160 1860.23 Q1160 1856.18 1157.33 1853.99 Q1154.68 1851.76 1149.88 1851.76 Q1146.82 1851.76 1143.93 1852.49 Q1141.03 1853.22 1138.36 1854.69 L1138.36 1849.28 Q1141.57 1848.03 1144.59 1847.43 Q1147.62 1846.79 1150.48 1846.79 Q1158.22 1846.79 1162.04 1850.8 Q1165.86 1854.81 1165.86 1862.96 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M1207.55 1861.78 L1207.55 1883.3 L1201.69 1883.3 L1201.69 1861.98 Q1201.69 1856.92 1199.72 1854.4 Q1197.75 1851.89 1193.8 1851.89 Q1189.06 1851.89 1186.32 1854.91 Q1183.58 1857.93 1183.58 1863.15 L1183.58 1883.3 L1177.7 1883.3 L1177.7 1847.65 L1183.58 1847.65 L1183.58 1853.19 Q1185.68 1849.98 1188.52 1848.39 Q1191.38 1846.79 1195.11 1846.79 Q1201.25 1846.79 1204.4 1850.61 Q1207.55 1854.4 1207.55 1861.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M1244.89 1849.02 L1244.89 1854.5 Q1242.4 1853.13 1239.89 1852.46 Q1237.41 1851.76 1234.86 1851.76 Q1229.16 1851.76 1226.01 1855.39 Q1222.86 1858.98 1222.86 1865.51 Q1222.86 1872.03 1226.01 1875.66 Q1229.16 1879.26 1234.86 1879.26 Q1237.41 1879.26 1239.89 1878.59 Q1242.4 1877.89 1244.89 1876.52 L1244.89 1881.93 Q1242.44 1883.08 1239.79 1883.65 Q1237.18 1884.22 1234.22 1884.22 Q1226.17 1884.22 1221.43 1879.16 Q1216.69 1874.1 1216.69 1865.51 Q1216.69 1856.79 1221.46 1851.79 Q1226.27 1846.79 1234.61 1846.79 Q1237.31 1846.79 1239.89 1847.37 Q1242.47 1847.91 1244.89 1849.02 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M1285.56 1864.01 L1285.56 1866.88 L1258.64 1866.88 Q1259.02 1872.92 1262.26 1876.11 Q1265.54 1879.26 1271.37 1879.26 Q1274.74 1879.26 1277.89 1878.43 Q1281.07 1877.6 1284.19 1875.95 L1284.19 1881.49 Q1281.04 1882.82 1277.73 1883.52 Q1274.42 1884.22 1271.02 1884.22 Q1262.49 1884.22 1257.49 1879.26 Q1252.52 1874.29 1252.52 1865.83 Q1252.52 1857.07 1257.24 1851.95 Q1261.98 1846.79 1270 1846.79 Q1277.19 1846.79 1281.36 1851.44 Q1285.56 1856.06 1285.56 1864.01 M1279.71 1862.29 Q1279.64 1857.49 1277 1854.62 Q1274.39 1851.76 1270.06 1851.76 Q1265.16 1851.76 1262.2 1854.53 Q1259.27 1857.3 1258.83 1862.33 L1279.71 1862.29 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M1322.58 1841.06 L1322.58 1878.02 L1330.35 1878.02 Q1340.18 1878.02 1344.73 1873.56 Q1349.32 1869.11 1349.32 1859.49 Q1349.32 1849.94 1344.73 1845.52 Q1340.18 1841.06 1330.35 1841.06 L1322.58 1841.06 M1316.15 1835.78 L1329.36 1835.78 Q1343.17 1835.78 1349.63 1841.54 Q1356.09 1847.27 1356.09 1859.49 Q1356.09 1871.78 1349.6 1877.54 Q1343.11 1883.3 1329.36 1883.3 L1316.15 1883.3 L1316.15 1835.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M1379.9 1851.76 Q1375.19 1851.76 1372.45 1855.45 Q1369.72 1859.11 1369.72 1865.51 Q1369.72 1871.91 1372.42 1875.6 Q1375.16 1879.26 1379.9 1879.26 Q1384.58 1879.26 1387.32 1875.57 Q1390.06 1871.87 1390.06 1865.51 Q1390.06 1859.17 1387.32 1855.48 Q1384.58 1851.76 1379.9 1851.76 M1379.9 1846.79 Q1387.54 1846.79 1391.9 1851.76 Q1396.26 1856.72 1396.26 1865.51 Q1396.26 1874.26 1391.9 1879.26 Q1387.54 1884.22 1379.9 1884.22 Q1372.23 1884.22 1367.87 1879.26 Q1363.54 1874.26 1363.54 1865.51 Q1363.54 1856.72 1367.87 1851.76 Q1372.23 1846.79 1379.9 1846.79 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M1402.56 1847.65 L1408.42 1847.65 L1415.74 1875.47 L1423.03 1847.65 L1429.94 1847.65 L1437.26 1875.47 L1444.55 1847.65 L1450.4 1847.65 L1441.08 1883.3 L1434.17 1883.3 L1426.5 1854.08 L1418.8 1883.3 L1411.89 1883.3 L1402.56 1847.65 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M1488.92 1861.78 L1488.92 1883.3 L1483.06 1883.3 L1483.06 1861.98 Q1483.06 1856.92 1481.09 1854.4 Q1479.11 1851.89 1475.17 1851.89 Q1470.42 1851.89 1467.69 1854.91 Q1464.95 1857.93 1464.95 1863.15 L1464.95 1883.3 L1459.06 1883.3 L1459.06 1847.65 L1464.95 1847.65 L1464.95 1853.19 Q1467.05 1849.98 1469.88 1848.39 Q1472.75 1846.79 1476.47 1846.79 Q1482.61 1846.79 1485.76 1850.61 Q1488.92 1854.4 1488.92 1861.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M1521.25 1853.13 Q1520.27 1852.55 1519.09 1852.3 Q1517.94 1852.01 1516.54 1852.01 Q1511.58 1852.01 1508.9 1855.26 Q1506.26 1858.47 1506.26 1864.52 L1506.26 1883.3 L1500.37 1883.3 L1500.37 1847.65 L1506.26 1847.65 L1506.26 1853.19 Q1508.11 1849.94 1511.07 1848.39 Q1514.03 1846.79 1518.26 1846.79 Q1518.87 1846.79 1519.6 1846.89 Q1520.33 1846.95 1521.22 1847.11 L1521.25 1853.13 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M1527.4 1847.65 L1533.25 1847.65 L1533.25 1883.3 L1527.4 1883.3 L1527.4 1847.65 M1527.4 1833.78 L1533.25 1833.78 L1533.25 1841.19 L1527.4 1841.19 L1527.4 1833.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M1541.31 1847.65 L1547.51 1847.65 L1558.65 1877.57 L1569.79 1847.65 L1576 1847.65 L1562.63 1883.3 L1554.67 1883.3 L1541.31 1847.65 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M1614.57 1864.01 L1614.57 1866.88 L1587.65 1866.88 Q1588.03 1872.92 1591.28 1876.11 Q1594.55 1879.26 1600.38 1879.26 Q1603.75 1879.26 1606.9 1878.43 Q1610.09 1877.6 1613.21 1875.95 L1613.21 1881.49 Q1610.05 1882.82 1606.74 1883.52 Q1603.43 1884.22 1600.03 1884.22 Q1591.5 1884.22 1586.5 1879.26 Q1581.54 1874.29 1581.54 1865.83 Q1581.54 1857.07 1586.25 1851.95 Q1590.99 1846.79 1599.01 1846.79 Q1606.2 1846.79 1610.37 1851.44 Q1614.57 1856.06 1614.57 1864.01 M1608.72 1862.29 Q1608.65 1857.49 1606.01 1854.62 Q1603.4 1851.76 1599.07 1851.76 Q1594.17 1851.76 1591.21 1854.53 Q1588.28 1857.3 1587.84 1862.33 L1608.72 1862.29 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M1644.84 1853.13 Q1643.86 1852.55 1642.68 1852.3 Q1641.53 1852.01 1640.13 1852.01 Q1635.17 1852.01 1632.49 1855.26 Q1629.85 1858.47 1629.85 1864.52 L1629.85 1883.3 L1623.96 1883.3 L1623.96 1847.65 L1629.85 1847.65 L1629.85 1853.19 Q1631.7 1849.94 1634.66 1848.39 Q1637.62 1846.79 1641.85 1846.79 Q1642.46 1846.79 1643.19 1846.89 Q1643.92 1846.95 1644.81 1847.11 L1644.84 1853.13 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M1685.77 1833.84 Q1681.51 1841.16 1679.44 1848.32 Q1677.37 1855.48 1677.37 1862.84 Q1677.37 1870.19 1679.44 1877.41 Q1681.54 1884.61 1685.77 1891.89 L1680.68 1891.89 Q1675.91 1884.41 1673.52 1877.19 Q1671.17 1869.96 1671.17 1862.84 Q1671.17 1855.74 1673.52 1848.54 Q1675.88 1841.35 1680.68 1833.84 L1685.77 1833.84 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M1696.91 1833.78 L1702.8 1833.78 L1702.8 1863.03 L1720.28 1847.65 L1727.76 1847.65 L1708.85 1864.33 L1728.55 1883.3 L1720.91 1883.3 L1702.8 1865.89 L1702.8 1883.3 L1696.91 1883.3 L1696.91 1833.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M1762.64 1854.5 Q1764.84 1850.55 1767.89 1848.67 Q1770.95 1846.79 1775.09 1846.79 Q1780.66 1846.79 1783.68 1850.71 Q1786.7 1854.59 1786.7 1861.78 L1786.7 1883.3 L1780.81 1883.3 L1780.81 1861.98 Q1780.81 1856.85 1779 1854.37 Q1777.19 1851.89 1773.46 1851.89 Q1768.91 1851.89 1766.27 1854.91 Q1763.63 1857.93 1763.63 1863.15 L1763.63 1883.3 L1757.74 1883.3 L1757.74 1861.98 Q1757.74 1856.82 1755.92 1854.37 Q1754.11 1851.89 1750.32 1851.89 Q1745.84 1851.89 1743.19 1854.94 Q1740.55 1857.97 1740.55 1863.15 L1740.55 1883.3 L1734.66 1883.3 L1734.66 1847.65 L1740.55 1847.65 L1740.55 1853.19 Q1742.56 1849.91 1745.36 1848.35 Q1748.16 1846.79 1752.01 1846.79 Q1755.89 1846.79 1758.6 1848.77 Q1761.34 1850.74 1762.64 1854.5 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M1797.46 1833.84 L1802.55 1833.84 Q1807.33 1841.35 1809.68 1848.54 Q1812.07 1855.74 1812.07 1862.84 Q1812.07 1869.96 1809.68 1877.19 Q1807.33 1884.41 1802.55 1891.89 L1797.46 1891.89 Q1801.69 1884.61 1803.76 1877.41 Q1805.86 1870.19 1805.86 1862.84 Q1805.86 1855.48 1803.76 1848.32 Q1801.69 1841.16 1797.46 1833.84 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><polyline clip-path="url(#clip352)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="181.719,1613.71 2640.76,1613.71 "/>
<polyline clip-path="url(#clip352)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="181.719,1152.9 2640.76,1152.9 "/>
<polyline clip-path="url(#clip352)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="181.719,692.081 2640.76,692.081 "/>
<polyline clip-path="url(#clip352)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="181.719,231.265 2640.76,231.265 "/>
<polyline clip-path="url(#clip350)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="181.719,1734.12 181.719,47.2441 "/>
<polyline clip-path="url(#clip350)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="181.719,1613.71 200.617,1613.71 "/>
<polyline clip-path="url(#clip350)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="181.719,1152.9 200.617,1152.9 "/>
<polyline clip-path="url(#clip350)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="181.719,692.081 200.617,692.081 "/>
<polyline clip-path="url(#clip350)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="181.719,231.265 200.617,231.265 "/>
<path clip-path="url(#clip350)" d="M131.816 1600.51 L120.011 1618.96 L131.816 1618.96 L131.816 1600.51 M130.589 1596.43 L136.469 1596.43 L136.469 1618.96 L141.399 1618.96 L141.399 1622.85 L136.469 1622.85 L136.469 1630.99 L131.816 1630.99 L131.816 1622.85 L116.214 1622.85 L116.214 1618.33 L130.589 1596.43 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M120.497 1135.62 L138.853 1135.62 L138.853 1139.55 L124.779 1139.55 L124.779 1148.03 Q125.798 1147.68 126.816 1147.52 Q127.835 1147.33 128.853 1147.33 Q134.64 1147.33 138.02 1150.5 Q141.399 1153.67 141.399 1159.09 Q141.399 1164.67 137.927 1167.77 Q134.455 1170.85 128.136 1170.85 Q125.96 1170.85 123.691 1170.48 Q121.446 1170.11 119.039 1169.37 L119.039 1164.67 Q121.122 1165.8 123.344 1166.36 Q125.566 1166.91 128.043 1166.91 Q132.048 1166.91 134.386 1164.81 Q136.724 1162.7 136.724 1159.09 Q136.724 1155.48 134.386 1153.37 Q132.048 1151.27 128.043 1151.27 Q126.168 1151.27 124.293 1151.68 Q122.441 1152.1 120.497 1152.98 L120.497 1135.62 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M129.872 690.218 Q126.724 690.218 124.872 692.371 Q123.043 694.523 123.043 698.273 Q123.043 702 124.872 704.176 Q126.724 706.329 129.872 706.329 Q133.02 706.329 134.849 704.176 Q136.7 702 136.7 698.273 Q136.7 694.523 134.849 692.371 Q133.02 690.218 129.872 690.218 M139.154 675.565 L139.154 679.824 Q137.395 678.991 135.589 678.551 Q133.807 678.111 132.048 678.111 Q127.418 678.111 124.964 681.236 Q122.534 684.361 122.187 690.681 Q123.552 688.667 125.613 687.602 Q127.673 686.514 130.15 686.514 Q135.358 686.514 138.367 689.685 Q141.399 692.833 141.399 698.273 Q141.399 703.597 138.251 706.815 Q135.103 710.032 129.872 710.032 Q123.876 710.032 120.705 705.449 Q117.534 700.843 117.534 692.116 Q117.534 683.921 121.423 679.06 Q125.312 674.176 131.863 674.176 Q133.622 674.176 135.404 674.523 Q137.21 674.871 139.154 675.565 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M119.177 213.985 L141.399 213.985 L141.399 215.975 L128.853 248.545 L123.969 248.545 L135.775 217.92 L119.177 217.92 L119.177 213.985 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M19.402 1529.67 L56.3549 1529.67 L56.3549 1521.9 Q56.3549 1512.07 51.8989 1507.52 Q47.4429 1502.93 37.8307 1502.93 Q28.2821 1502.93 23.858 1507.52 Q19.402 1512.07 19.402 1521.9 L19.402 1529.67 M14.1184 1536.1 L14.1184 1522.89 Q14.1184 1509.08 19.8794 1502.62 Q25.6085 1496.15 37.8307 1496.15 Q50.1165 1496.15 55.8775 1502.65 Q61.6384 1509.14 61.6384 1522.89 L61.6384 1536.1 L14.1184 1536.1 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M25.9905 1486.16 L25.9905 1480.3 L61.6384 1480.3 L61.6384 1486.16 L25.9905 1486.16 M12.1132 1486.16 L12.1132 1480.3 L19.5293 1480.3 L19.5293 1486.16 L12.1132 1486.16 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M27.0408 1445.32 L32.579 1445.32 Q31.3059 1447.81 30.6693 1450.48 Q30.0327 1453.15 30.0327 1456.02 Q30.0327 1460.38 31.3695 1462.58 Q32.7063 1464.74 35.3799 1464.74 Q37.4169 1464.74 38.5946 1463.18 Q39.7404 1461.62 40.7908 1456.91 L41.2364 1454.9 Q42.5732 1448.67 45.0239 1446.06 Q47.4429 1443.41 51.8034 1443.41 Q56.7687 1443.41 59.6651 1447.36 Q62.5615 1451.28 62.5615 1458.15 Q62.5615 1461.02 61.9886 1464.14 Q61.4475 1467.22 60.3335 1470.66 L54.2861 1470.66 Q55.973 1467.41 56.8323 1464.26 Q57.6599 1461.11 57.6599 1458.02 Q57.6599 1453.89 56.2594 1451.66 Q54.8271 1449.43 52.249 1449.43 Q49.8619 1449.43 48.5887 1451.05 Q47.3156 1452.65 46.1379 1458.09 L45.6605 1460.12 Q44.5147 1465.57 42.1594 1467.99 Q39.7722 1470.41 35.6345 1470.41 Q30.6056 1470.41 27.8684 1466.84 Q25.1311 1463.28 25.1311 1456.72 Q25.1311 1453.47 25.6085 1450.61 Q26.086 1447.74 27.0408 1445.32 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M27.0408 1411.36 L32.579 1411.36 Q31.3059 1413.85 30.6693 1416.52 Q30.0327 1419.19 30.0327 1422.06 Q30.0327 1426.42 31.3695 1428.61 Q32.7063 1430.78 35.3799 1430.78 Q37.4169 1430.78 38.5946 1429.22 Q39.7404 1427.66 40.7908 1422.95 L41.2364 1420.94 Q42.5732 1414.71 45.0239 1412.1 Q47.4429 1409.45 51.8034 1409.45 Q56.7687 1409.45 59.6651 1413.4 Q62.5615 1417.32 62.5615 1424.19 Q62.5615 1427.05 61.9886 1430.17 Q61.4475 1433.26 60.3335 1436.7 L54.2861 1436.7 Q55.973 1433.45 56.8323 1430.3 Q57.6599 1427.15 57.6599 1424.06 Q57.6599 1419.93 56.2594 1417.7 Q54.8271 1415.47 52.249 1415.47 Q49.8619 1415.47 48.5887 1417.09 Q47.3156 1418.68 46.1379 1424.13 L45.6605 1426.16 Q44.5147 1431.61 42.1594 1434.03 Q39.7722 1436.44 35.6345 1436.44 Q30.6056 1436.44 27.8684 1432.88 Q25.1311 1429.31 25.1311 1422.76 Q25.1311 1419.51 25.6085 1416.65 Q26.086 1413.78 27.0408 1411.36 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M30.0964 1386.31 Q30.0964 1391.03 33.7885 1393.76 Q37.4488 1396.5 43.8463 1396.5 Q50.2438 1396.5 53.9359 1393.79 Q57.5962 1391.06 57.5962 1386.31 Q57.5962 1381.64 53.9041 1378.9 Q50.212 1376.16 43.8463 1376.16 Q37.5124 1376.16 33.8203 1378.9 Q30.0964 1381.64 30.0964 1386.31 M25.1311 1386.31 Q25.1311 1378.68 30.0964 1374.32 Q35.0616 1369.95 43.8463 1369.95 Q52.5991 1369.95 57.5962 1374.32 Q62.5615 1378.68 62.5615 1386.31 Q62.5615 1393.99 57.5962 1398.35 Q52.5991 1402.67 43.8463 1402.67 Q35.0616 1402.67 30.0964 1398.35 Q25.1311 1393.99 25.1311 1386.31 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M12.1132 1360.25 L12.1132 1354.39 L61.6384 1354.39 L61.6384 1360.25 L12.1132 1360.25 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M25.9905 1346.34 L25.9905 1340.13 L55.9093 1328.99 L25.9905 1317.85 L25.9905 1311.64 L61.6384 1325.01 L61.6384 1332.97 L25.9905 1346.34 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M42.3504 1273.07 L45.2149 1273.07 L45.2149 1300 Q51.2623 1299.61 54.4452 1296.37 Q57.5962 1293.09 57.5962 1287.26 Q57.5962 1283.89 56.7687 1280.74 Q55.9411 1277.56 54.2861 1274.44 L59.8242 1274.44 Q61.161 1277.59 61.8612 1280.9 Q62.5615 1284.21 62.5615 1287.61 Q62.5615 1296.14 57.5962 1301.14 Q52.631 1306.11 44.1646 1306.11 Q35.4117 1306.11 30.2873 1301.4 Q25.1311 1296.65 25.1311 1288.63 Q25.1311 1281.44 29.7781 1277.27 Q34.3932 1273.07 42.3504 1273.07 M40.6316 1278.92 Q35.8255 1278.99 32.9609 1281.63 Q30.0964 1284.24 30.0964 1288.57 Q30.0964 1293.47 32.8654 1296.43 Q35.6345 1299.36 40.6634 1299.8 L40.6316 1278.92 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M31.4013 1240 L12.1132 1240 L12.1132 1234.14 L61.6384 1234.14 L61.6384 1240 L56.2912 1240 Q59.4741 1241.84 61.0337 1244.68 Q62.5615 1247.48 62.5615 1251.43 Q62.5615 1257.89 57.4052 1261.96 Q52.249 1266 43.8463 1266 Q35.4436 1266 30.2873 1261.96 Q25.1311 1257.89 25.1311 1251.43 Q25.1311 1247.48 26.6907 1244.68 Q28.2185 1241.84 31.4013 1240 M43.8463 1259.96 Q50.3075 1259.96 53.9996 1257.31 Q57.6599 1254.64 57.6599 1249.99 Q57.6599 1245.35 53.9996 1242.67 Q50.3075 1240 43.8463 1240 Q37.3851 1240 33.7248 1242.67 Q30.0327 1245.35 30.0327 1249.99 Q30.0327 1254.64 33.7248 1257.31 Q37.3851 1259.96 43.8463 1259.96 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M18.479 1181.82 Q18.479 1188.82 23.6988 1192.96 Q28.9187 1197.06 37.9262 1197.06 Q46.9018 1197.06 52.1217 1192.96 Q57.3416 1188.82 57.3416 1181.82 Q57.3416 1174.81 52.1217 1170.74 Q46.9018 1166.63 37.9262 1166.63 Q28.9187 1166.63 23.6988 1170.74 Q18.479 1174.81 18.479 1181.82 M13.2591 1181.82 Q13.2591 1171.82 19.9749 1165.84 Q26.6589 1159.85 37.9262 1159.85 Q49.1617 1159.85 55.8775 1165.84 Q62.5615 1171.82 62.5615 1181.82 Q62.5615 1191.84 55.8775 1197.86 Q49.1935 1203.84 37.9262 1203.84 Q26.6589 1203.84 19.9749 1197.86 Q13.2591 1191.84 13.2591 1181.82 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M25.9905 1120.42 L43.337 1133.31 L61.6384 1119.75 L61.6384 1126.66 L47.6339 1137.03 L61.6384 1147.41 L61.6384 1154.32 L42.9869 1140.47 L25.9905 1153.14 L25.9905 1146.23 L38.6901 1136.78 L25.9905 1127.33 L25.9905 1120.42 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M64.9486 1096.64 Q71.3143 1099.13 73.2559 1101.48 Q75.1974 1103.84 75.1974 1107.78 L75.1974 1112.46 L70.2958 1112.46 L70.2958 1109.02 Q70.2958 1106.61 69.15 1105.27 Q68.0041 1103.93 63.7391 1102.31 L61.0655 1101.26 L25.9905 1115.68 L25.9905 1109.47 L53.8723 1098.33 L25.9905 1087.19 L25.9905 1080.98 L64.9486 1096.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M43.4007 1049.44 Q37.035 1049.44 33.5338 1052.08 Q30.0327 1054.69 30.0327 1059.44 Q30.0327 1064.15 33.5338 1066.79 Q37.035 1069.4 43.4007 1069.4 Q49.7346 1069.4 53.2357 1066.79 Q56.7368 1064.15 56.7368 1059.44 Q56.7368 1054.69 53.2357 1052.08 Q49.7346 1049.44 43.4007 1049.44 M57.2143 1043.58 Q66.3172 1043.58 70.7414 1047.63 Q75.1974 1051.67 75.1974 1060.01 Q75.1974 1063.1 74.72 1065.83 Q74.2744 1068.57 73.3195 1071.15 L67.6222 1071.15 Q69.0227 1068.57 69.6911 1066.06 Q70.3595 1063.54 70.3595 1060.93 Q70.3595 1055.17 67.3358 1052.31 Q64.3439 1049.44 58.2646 1049.44 L55.3682 1049.44 Q58.5192 1051.26 60.0788 1054.09 Q61.6384 1056.92 61.6384 1060.87 Q61.6384 1067.42 56.6414 1071.43 Q51.6443 1075.45 43.4007 1075.45 Q35.1253 1075.45 30.1282 1071.43 Q25.1311 1067.42 25.1311 1060.87 Q25.1311 1056.92 26.6907 1054.09 Q28.2503 1051.26 31.4013 1049.44 L25.9905 1049.44 L25.9905 1043.58 L57.2143 1043.58 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M42.3504 1001.03 L45.2149 1001.03 L45.2149 1027.96 Q51.2623 1027.58 54.4452 1024.33 Q57.5962 1021.05 57.5962 1015.23 Q57.5962 1011.85 56.7687 1008.7 Q55.9411 1005.52 54.2861 1002.4 L59.8242 1002.4 Q61.161 1005.55 61.8612 1008.86 Q62.5615 1012.17 62.5615 1015.58 Q62.5615 1024.11 57.5962 1029.1 Q52.631 1034.07 44.1646 1034.07 Q35.4117 1034.07 30.2873 1029.36 Q25.1311 1024.62 25.1311 1016.59 Q25.1311 1009.4 29.7781 1005.23 Q34.3932 1001.03 42.3504 1001.03 M40.6316 1006.89 Q35.8255 1006.95 32.9609 1009.59 Q30.0964 1012.2 30.0964 1016.53 Q30.0964 1021.43 32.8654 1024.39 Q35.6345 1027.32 40.6634 1027.77 L40.6316 1006.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M40.1224 961.785 L61.6384 961.785 L61.6384 967.642 L40.3133 967.642 Q35.2526 967.642 32.7381 969.615 Q30.2237 971.589 30.2237 975.535 Q30.2237 980.278 33.2474 983.015 Q36.2711 985.752 41.491 985.752 L61.6384 985.752 L61.6384 991.641 L25.9905 991.641 L25.9905 985.752 L31.5287 985.752 Q28.314 983.652 26.7225 980.819 Q25.1311 977.954 25.1311 974.23 Q25.1311 968.088 28.9505 964.937 Q32.7381 961.785 40.1224 961.785 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M17.7787 893.545 L24.5582 893.545 Q21.5345 896.792 20.0386 900.484 Q18.5426 904.144 18.5426 908.282 Q18.5426 916.43 23.5397 920.759 Q28.5049 925.087 37.9262 925.087 Q47.3156 925.087 52.3127 920.759 Q57.2779 916.43 57.2779 908.282 Q57.2779 904.144 55.782 900.484 Q54.2861 896.792 51.2623 893.545 L57.9782 893.545 Q60.2698 896.919 61.4156 900.707 Q62.5615 904.462 62.5615 908.664 Q62.5615 919.454 55.973 925.66 Q49.3526 931.867 37.9262 931.867 Q26.4679 931.867 19.8794 925.66 Q13.2591 919.454 13.2591 908.664 Q13.2591 904.399 14.4049 900.643 Q15.5189 896.855 17.7787 893.545 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M30.0964 870.056 Q30.0964 874.766 33.7885 877.504 Q37.4488 880.241 43.8463 880.241 Q50.2438 880.241 53.9359 877.535 Q57.5962 874.798 57.5962 870.056 Q57.5962 865.377 53.9041 862.64 Q50.212 859.902 43.8463 859.902 Q37.5124 859.902 33.8203 862.64 Q30.0964 865.377 30.0964 870.056 M25.1311 870.056 Q25.1311 862.417 30.0964 858.056 Q35.0616 853.696 43.8463 853.696 Q52.5991 853.696 57.5962 858.056 Q62.5615 862.417 62.5615 870.056 Q62.5615 877.726 57.5962 882.087 Q52.5991 886.416 43.8463 886.416 Q35.0616 886.416 30.0964 882.087 Q25.1311 877.726 25.1311 870.056 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M40.1224 814.356 L61.6384 814.356 L61.6384 820.212 L40.3133 820.212 Q35.2526 820.212 32.7381 822.186 Q30.2237 824.159 30.2237 828.106 Q30.2237 832.848 33.2474 835.585 Q36.2711 838.323 41.491 838.323 L61.6384 838.323 L61.6384 844.211 L25.9905 844.211 L25.9905 838.323 L31.5287 838.323 Q28.314 836.222 26.7225 833.389 Q25.1311 830.525 25.1311 826.801 Q25.1311 820.658 28.9505 817.507 Q32.7381 814.356 40.1224 814.356 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M27.3591 777.021 L32.8336 777.021 Q31.465 779.503 30.7966 782.018 Q30.0964 784.501 30.0964 787.047 Q30.0964 792.744 33.7248 795.895 Q37.3214 799.046 43.8463 799.046 Q50.3711 799.046 53.9996 795.895 Q57.5962 792.744 57.5962 787.047 Q57.5962 784.501 56.9278 782.018 Q56.2276 779.503 54.859 777.021 L60.2698 777.021 Q61.4156 779.472 61.9886 782.113 Q62.5615 784.723 62.5615 787.683 Q62.5615 795.736 57.5007 800.478 Q52.44 805.221 43.8463 805.221 Q35.1253 805.221 30.1282 800.447 Q25.1311 795.641 25.1311 787.301 Q25.1311 784.596 25.704 782.018 Q26.2451 779.44 27.3591 777.021 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M42.3504 736.344 L45.2149 736.344 L45.2149 763.271 Q51.2623 762.889 54.4452 759.642 Q57.5962 756.364 57.5962 750.54 Q57.5962 747.166 56.7687 744.015 Q55.9411 740.832 54.2861 737.713 L59.8242 737.713 Q61.161 740.864 61.8612 744.174 Q62.5615 747.484 62.5615 750.89 Q62.5615 759.42 57.5962 764.417 Q52.631 769.382 44.1646 769.382 Q35.4117 769.382 30.2873 764.671 Q25.1311 759.929 25.1311 751.908 Q25.1311 744.715 29.7781 740.545 Q34.3932 736.344 42.3504 736.344 M40.6316 742.2 Q35.8255 742.264 32.9609 744.906 Q30.0964 747.516 30.0964 751.844 Q30.0964 756.746 32.8654 759.706 Q35.6345 762.634 40.6634 763.08 L40.6316 742.2 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M40.1224 697.099 L61.6384 697.099 L61.6384 702.956 L40.3133 702.956 Q35.2526 702.956 32.7381 704.929 Q30.2237 706.903 30.2237 710.849 Q30.2237 715.592 33.2474 718.329 Q36.2711 721.066 41.491 721.066 L61.6384 721.066 L61.6384 726.955 L25.9905 726.955 L25.9905 721.066 L31.5287 721.066 Q28.314 718.966 26.7225 716.133 Q25.1311 713.268 25.1311 709.544 Q25.1311 703.401 28.9505 700.25 Q32.7381 697.099 40.1224 697.099 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M15.869 679.626 L25.9905 679.626 L25.9905 667.563 L30.542 667.563 L30.542 679.626 L49.8937 679.626 Q54.2542 679.626 55.4955 678.448 Q56.7368 677.238 56.7368 673.578 L56.7368 667.563 L61.6384 667.563 L61.6384 673.578 Q61.6384 680.358 59.124 682.936 Q56.5777 685.514 49.8937 685.514 L30.542 685.514 L30.542 689.811 L25.9905 689.811 L25.9905 685.514 L15.869 685.514 L15.869 679.626 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M31.465 639.203 Q30.8921 640.19 30.6375 641.368 Q30.351 642.513 30.351 643.914 Q30.351 648.879 33.5975 651.553 Q36.8122 654.195 42.8596 654.195 L61.6384 654.195 L61.6384 660.083 L25.9905 660.083 L25.9905 654.195 L31.5287 654.195 Q28.2821 652.348 26.7225 649.388 Q25.1311 646.428 25.1311 642.195 Q25.1311 641.59 25.2266 640.858 Q25.2903 640.126 25.4494 639.235 L31.465 639.203 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M43.719 616.86 Q43.719 623.957 45.3422 626.695 Q46.9655 629.432 50.8804 629.432 Q53.9996 629.432 55.8457 627.395 Q57.6599 625.326 57.6599 621.793 Q57.6599 616.923 54.2224 613.995 Q50.7531 611.035 45.0239 611.035 L43.719 611.035 L43.719 616.86 M41.3 605.179 L61.6384 605.179 L61.6384 611.035 L56.2276 611.035 Q59.4741 613.04 61.0337 616.032 Q62.5615 619.024 62.5615 623.353 Q62.5615 628.827 59.5059 632.074 Q56.4186 635.288 51.2623 635.288 Q45.2467 635.288 42.1912 631.278 Q39.1357 627.236 39.1357 619.247 L39.1357 611.035 L38.5628 611.035 Q34.5205 611.035 32.3244 613.709 Q30.0964 616.35 30.0964 621.157 Q30.0964 624.212 30.8284 627.108 Q31.5605 630.005 33.0246 632.678 L27.6137 632.678 Q26.3724 629.464 25.7677 626.44 Q25.1311 623.416 25.1311 620.552 Q25.1311 612.817 29.1415 608.998 Q33.1519 605.179 41.3 605.179 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M15.869 587.323 L25.9905 587.323 L25.9905 575.26 L30.542 575.26 L30.542 587.323 L49.8937 587.323 Q54.2542 587.323 55.4955 586.145 Q56.7368 584.936 56.7368 581.275 L56.7368 575.26 L61.6384 575.26 L61.6384 581.275 Q61.6384 588.055 59.124 590.633 Q56.5777 593.211 49.8937 593.211 L30.542 593.211 L30.542 597.508 L25.9905 597.508 L25.9905 593.211 L15.869 593.211 L15.869 587.323 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M25.9905 567.557 L25.9905 561.701 L61.6384 561.701 L61.6384 567.557 L25.9905 567.557 M12.1132 567.557 L12.1132 561.701 L19.5293 561.701 L19.5293 567.557 L12.1132 567.557 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M30.0964 535.633 Q30.0964 540.344 33.7885 543.081 Q37.4488 545.818 43.8463 545.818 Q50.2438 545.818 53.9359 543.113 Q57.5962 540.376 57.5962 535.633 Q57.5962 530.954 53.9041 528.217 Q50.212 525.48 43.8463 525.48 Q37.5124 525.48 33.8203 528.217 Q30.0964 530.954 30.0964 535.633 M25.1311 535.633 Q25.1311 527.994 30.0964 523.634 Q35.0616 519.273 43.8463 519.273 Q52.5991 519.273 57.5962 523.634 Q62.5615 527.994 62.5615 535.633 Q62.5615 543.304 57.5962 547.664 Q52.5991 551.993 43.8463 551.993 Q35.0616 551.993 30.0964 547.664 Q25.1311 543.304 25.1311 535.633 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M40.1224 479.933 L61.6384 479.933 L61.6384 485.79 L40.3133 485.79 Q35.2526 485.79 32.7381 487.763 Q30.2237 489.737 30.2237 493.683 Q30.2237 498.426 33.2474 501.163 Q36.2711 503.9 41.491 503.9 L61.6384 503.9 L61.6384 509.788 L25.9905 509.788 L25.9905 503.9 L31.5287 503.9 Q28.314 501.8 26.7225 498.967 Q25.1311 496.102 25.1311 492.378 Q25.1311 486.235 28.9505 483.084 Q32.7381 479.933 40.1224 479.933 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M12.1769 433.464 Q19.4975 437.729 26.6589 439.798 Q33.8203 441.866 41.1727 441.866 Q48.5251 441.866 55.7502 439.798 Q62.9434 437.697 70.2321 433.464 L70.2321 438.556 Q62.7524 443.331 55.5274 445.718 Q48.3023 448.073 41.1727 448.073 Q34.0749 448.073 26.8817 445.718 Q19.6884 443.362 12.1769 438.556 L12.1769 433.464 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M32.8336 394.346 Q28.8869 392.15 27.009 389.095 Q25.1311 386.039 25.1311 381.901 Q25.1311 376.331 29.046 373.308 Q32.9291 370.284 40.1224 370.284 L61.6384 370.284 L61.6384 376.172 L40.3133 376.172 Q35.1889 376.172 32.7063 377.987 Q30.2237 379.801 30.2237 383.525 Q30.2237 388.076 33.2474 390.718 Q36.2711 393.36 41.491 393.36 L61.6384 393.36 L61.6384 399.248 L40.3133 399.248 Q35.1571 399.248 32.7063 401.062 Q30.2237 402.876 30.2237 406.664 Q30.2237 411.152 33.2792 413.794 Q36.3029 416.435 41.491 416.435 L61.6384 416.435 L61.6384 422.324 L25.9905 422.324 L25.9905 416.435 L31.5287 416.435 Q28.2503 414.43 26.6907 411.629 Q25.1311 408.828 25.1311 404.977 Q25.1311 401.094 27.1045 398.389 Q29.0779 395.651 32.8336 394.346 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M43.4007 335.145 Q37.035 335.145 33.5338 337.787 Q30.0327 340.397 30.0327 345.139 Q30.0327 349.85 33.5338 352.492 Q37.035 355.102 43.4007 355.102 Q49.7346 355.102 53.2357 352.492 Q56.7368 349.85 56.7368 345.139 Q56.7368 340.397 53.2357 337.787 Q49.7346 335.145 43.4007 335.145 M57.2143 329.289 Q66.3172 329.289 70.7414 333.331 Q75.1974 337.373 75.1974 345.712 Q75.1974 348.8 74.72 351.537 Q74.2744 354.274 73.3195 356.852 L67.6222 356.852 Q69.0227 354.274 69.6911 351.76 Q70.3595 349.245 70.3595 346.635 Q70.3595 340.874 67.3358 338.01 Q64.3439 335.145 58.2646 335.145 L55.3682 335.145 Q58.5192 336.96 60.0788 339.792 Q61.6384 342.625 61.6384 346.572 Q61.6384 353.128 56.6414 357.139 Q51.6443 361.149 43.4007 361.149 Q35.1253 361.149 30.1282 357.139 Q25.1311 353.128 25.1311 346.572 Q25.1311 342.625 26.6907 339.792 Q28.2503 336.96 31.4013 335.145 L25.9905 335.145 L25.9905 329.289 L57.2143 329.289 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M14.1184 306.818 L14.1184 301.407 L67.6859 317.958 L67.6859 323.369 L14.1184 306.818 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M14.1184 295.01 L14.1184 288.58 L56.2276 288.58 L56.2276 265.441 L61.6384 265.441 L61.6384 295.01 L14.1184 295.01 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M12.1769 259.871 L12.1769 254.778 Q19.6884 250.004 26.8817 247.649 Q34.0749 245.262 41.1727 245.262 Q48.3023 245.262 55.5274 247.649 Q62.7524 250.004 70.2321 254.778 L70.2321 259.871 Q62.9434 255.638 55.7502 253.569 Q48.5251 251.468 41.1727 251.468 Q33.8203 251.468 26.6589 253.569 Q19.4975 255.638 12.1769 259.871 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><polyline clip-path="url(#clip352)" style="stroke:#009af9; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="251.315,105.587 297.712,361.046 344.109,575.616 390.506,753.839 436.903,899.806 483.299,1017.2 529.696,1109.35 576.093,1179.23 622.49,1229.52 668.887,1262.63 715.284,1280.73 761.681,1285.75 808.078,1279.43 854.475,1263.31 900.872,1238.79 947.269,1255.9 993.665,1389.28 1040.06,1493.48 1086.46,1571.95 1132.86,1627.83 1179.25,1663.89 1225.65,1682.66 1272.05,1686.38 1318.44,1677.05 1364.84,1656.48 1411.24,1626.28 1457.63,1587.87 1504.03,1542.54 1550.43,1491.43 1596.83,1435.53 1643.22,1375.74 1689.62,1312.86 1736.02,1247.58 1782.41,1180.52 1828.81,1112.21 1875.21,1043.13 1921.6,973.686 1968,904.241 2014.4,835.102 2060.79,766.537 2107.19,698.772 2153.59,632.002 2199.99,566.388 2246.38,502.068 2292.78,439.151 2339.18,377.729 2385.57,317.873 2431.97,259.637 2478.37,203.061 2524.76,148.172 2571.16,94.9858 "/>
<polyline clip-path="url(#clip352)" style="stroke:#e26f46; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="-2277.32,1613.71 5099.79,1613.71 "/>
<path clip-path="url(#clip350)" d="M1048.66 258.993 L1773.82 258.993 L1773.82 103.473 L1048.66 103.473  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip350)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1048.66,258.993 1773.82,258.993 1773.82,103.473 1048.66,103.473 1048.66,258.993 "/>
<polyline clip-path="url(#clip350)" style="stroke:#009af9; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1075.98,155.313 1239.92,155.313 "/>
<path clip-path="url(#clip350)" d="M1289.48 139.167 L1289.48 143.728 Q1286.82 142.454 1284.46 141.829 Q1282.1 141.204 1279.9 141.204 Q1276.08 141.204 1274 142.686 Q1271.94 144.167 1271.94 146.899 Q1271.94 149.191 1273.3 150.371 Q1274.69 151.528 1278.53 152.246 L1281.36 152.825 Q1286.59 153.82 1289.07 156.343 Q1291.57 158.843 1291.57 163.056 Q1291.57 168.079 1288.19 170.672 Q1284.83 173.264 1278.33 173.264 Q1275.87 173.264 1273.09 172.709 Q1270.34 172.153 1267.38 171.065 L1267.38 166.251 Q1270.22 167.848 1272.96 168.658 Q1275.69 169.468 1278.33 169.468 Q1282.33 169.468 1284.51 167.894 Q1286.68 166.32 1286.68 163.403 Q1286.68 160.857 1285.11 159.422 Q1283.56 157.987 1279.99 157.269 L1277.15 156.714 Q1271.91 155.672 1269.58 153.45 Q1267.24 151.228 1267.24 147.269 Q1267.24 142.686 1270.46 140.047 Q1273.7 137.408 1279.37 137.408 Q1281.8 137.408 1284.32 137.848 Q1286.84 138.288 1289.48 139.167 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M1298.67 146.667 L1302.93 146.667 L1302.93 172.593 L1298.67 172.593 L1298.67 146.667 M1298.67 136.575 L1302.93 136.575 L1302.93 141.968 L1298.67 141.968 L1298.67 136.575 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M1332.03 151.644 Q1333.63 148.774 1335.85 147.408 Q1338.07 146.042 1341.08 146.042 Q1345.13 146.042 1347.33 148.89 Q1349.53 151.714 1349.53 156.945 L1349.53 172.593 L1345.25 172.593 L1345.25 157.084 Q1345.25 153.357 1343.93 151.552 Q1342.61 149.746 1339.9 149.746 Q1336.59 149.746 1334.67 151.945 Q1332.75 154.144 1332.75 157.94 L1332.75 172.593 L1328.47 172.593 L1328.47 157.084 Q1328.47 153.334 1327.15 151.552 Q1325.83 149.746 1323.07 149.746 Q1319.81 149.746 1317.89 151.968 Q1315.97 154.167 1315.97 157.94 L1315.97 172.593 L1311.68 172.593 L1311.68 146.667 L1315.97 146.667 L1315.97 150.695 Q1317.42 148.311 1319.46 147.177 Q1321.5 146.042 1324.3 146.042 Q1327.12 146.042 1329.09 147.478 Q1331.08 148.913 1332.03 151.644 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M1357.59 162.362 L1357.59 146.667 L1361.84 146.667 L1361.84 162.2 Q1361.84 165.88 1363.28 167.732 Q1364.71 169.561 1367.59 169.561 Q1371.03 169.561 1373.03 167.362 Q1375.04 165.163 1375.04 161.366 L1375.04 146.667 L1379.3 146.667 L1379.3 172.593 L1375.04 172.593 L1375.04 168.612 Q1373.49 170.973 1371.43 172.13 Q1369.39 173.264 1366.68 173.264 Q1362.21 173.264 1359.9 170.487 Q1357.59 167.709 1357.59 162.362 M1368.3 146.042 L1368.3 146.042 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M1388.07 136.575 L1392.33 136.575 L1392.33 172.593 L1388.07 172.593 L1388.07 136.575 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M1413.02 159.561 Q1407.86 159.561 1405.87 160.741 Q1403.88 161.922 1403.88 164.769 Q1403.88 167.038 1405.36 168.38 Q1406.87 169.7 1409.44 169.7 Q1412.98 169.7 1415.11 167.2 Q1417.26 164.677 1417.26 160.51 L1417.26 159.561 L1413.02 159.561 M1421.52 157.802 L1421.52 172.593 L1417.26 172.593 L1417.26 168.658 Q1415.8 171.019 1413.63 172.153 Q1411.45 173.264 1408.3 173.264 Q1404.32 173.264 1401.96 171.042 Q1399.62 168.797 1399.62 165.047 Q1399.62 160.672 1402.54 158.45 Q1405.48 156.228 1411.29 156.228 L1417.26 156.228 L1417.26 155.811 Q1417.26 152.871 1415.32 151.274 Q1413.4 149.653 1409.9 149.653 Q1407.68 149.653 1405.57 150.186 Q1403.46 150.718 1401.52 151.783 L1401.52 147.848 Q1403.86 146.945 1406.06 146.505 Q1408.26 146.042 1410.34 146.042 Q1415.96 146.042 1418.74 148.959 Q1421.52 151.876 1421.52 157.802 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M1434.51 139.306 L1434.51 146.667 L1443.28 146.667 L1443.28 149.978 L1434.51 149.978 L1434.51 164.052 Q1434.51 167.223 1435.36 168.126 Q1436.24 169.028 1438.9 169.028 L1443.28 169.028 L1443.28 172.593 L1438.9 172.593 Q1433.97 172.593 1432.1 170.764 Q1430.22 168.913 1430.22 164.052 L1430.22 149.978 L1427.1 149.978 L1427.1 146.667 L1430.22 146.667 L1430.22 139.306 L1434.51 139.306 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M1471.06 158.565 L1471.06 160.649 L1451.47 160.649 Q1451.75 165.047 1454.11 167.362 Q1456.5 169.653 1460.73 169.653 Q1463.19 169.653 1465.48 169.052 Q1467.79 168.45 1470.06 167.246 L1470.06 171.274 Q1467.77 172.246 1465.36 172.755 Q1462.96 173.264 1460.48 173.264 Q1454.27 173.264 1450.64 169.653 Q1447.03 166.042 1447.03 159.885 Q1447.03 153.519 1450.46 149.792 Q1453.9 146.042 1459.74 146.042 Q1464.97 146.042 1468 149.422 Q1471.06 152.778 1471.06 158.565 M1466.8 157.315 Q1466.75 153.82 1464.83 151.737 Q1462.93 149.653 1459.78 149.653 Q1456.22 149.653 1454.07 151.667 Q1451.94 153.681 1451.61 157.339 L1466.8 157.315 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M1495.11 150.603 L1495.11 136.575 L1499.37 136.575 L1499.37 172.593 L1495.11 172.593 L1495.11 168.704 Q1493.77 171.019 1491.7 172.153 Q1489.67 173.264 1486.8 173.264 Q1482.1 173.264 1479.14 169.515 Q1476.2 165.765 1476.2 159.653 Q1476.2 153.542 1479.14 149.792 Q1482.1 146.042 1486.8 146.042 Q1489.67 146.042 1491.7 147.177 Q1493.77 148.288 1495.11 150.603 M1480.59 159.653 Q1480.59 164.352 1482.52 167.038 Q1484.46 169.7 1487.84 169.7 Q1491.22 169.7 1493.16 167.038 Q1495.11 164.352 1495.11 159.653 Q1495.11 154.954 1493.16 152.292 Q1491.22 149.607 1487.84 149.607 Q1484.46 149.607 1482.52 152.292 Q1480.59 154.954 1480.59 159.653 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M1528.07 141.876 L1528.07 168.751 L1533.72 168.751 Q1540.87 168.751 1544.18 165.51 Q1547.51 162.269 1547.51 155.278 Q1547.51 148.334 1544.18 145.116 Q1540.87 141.876 1533.72 141.876 L1528.07 141.876 M1523.39 138.033 L1533 138.033 Q1543.05 138.033 1547.75 142.223 Q1552.45 146.39 1552.45 155.278 Q1552.45 164.214 1547.72 168.403 Q1543 172.593 1533 172.593 L1523.39 172.593 L1523.39 138.033 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M1573.93 141.204 Q1568.83 141.204 1565.82 145.001 Q1562.84 148.797 1562.84 155.348 Q1562.84 161.876 1565.82 165.672 Q1568.83 169.468 1573.93 169.468 Q1579.02 169.468 1581.98 165.672 Q1584.97 161.876 1584.97 155.348 Q1584.97 148.797 1581.98 145.001 Q1579.02 141.204 1573.93 141.204 M1573.93 137.408 Q1581.2 137.408 1585.55 142.292 Q1589.9 147.154 1589.9 155.348 Q1589.9 163.519 1585.55 168.403 Q1581.2 173.264 1573.93 173.264 Q1566.64 173.264 1562.26 168.403 Q1557.91 163.542 1557.91 155.348 Q1557.91 147.154 1562.26 142.292 Q1566.64 137.408 1573.93 137.408 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><polyline clip-path="url(#clip350)" style="stroke:#e26f46; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1075.98,207.153 1239.92,207.153 "/>
<path clip-path="url(#clip350)" d="M1283.63 208.23 Q1285.13 208.739 1286.54 210.405 Q1287.98 212.072 1289.41 214.989 L1294.16 224.433 L1289.14 224.433 L1284.72 215.567 Q1283 212.095 1281.38 210.961 Q1279.78 209.827 1277.01 209.827 L1271.91 209.827 L1271.91 224.433 L1267.24 224.433 L1267.24 189.873 L1277.79 189.873 Q1283.72 189.873 1286.64 192.35 Q1289.55 194.827 1289.55 199.827 Q1289.55 203.091 1288.03 205.243 Q1286.52 207.396 1283.63 208.23 M1271.91 193.716 L1271.91 205.984 L1277.79 205.984 Q1281.17 205.984 1282.89 204.433 Q1284.62 202.859 1284.62 199.827 Q1284.62 196.794 1282.89 195.267 Q1281.17 193.716 1277.79 193.716 L1271.91 193.716 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M1320.04 210.405 L1320.04 212.489 L1300.46 212.489 Q1300.73 216.887 1303.09 219.202 Q1305.48 221.493 1309.72 221.493 Q1312.17 221.493 1314.46 220.892 Q1316.78 220.29 1319.04 219.086 L1319.04 223.114 Q1316.75 224.086 1314.34 224.595 Q1311.94 225.104 1309.46 225.104 Q1303.26 225.104 1299.62 221.493 Q1296.01 217.882 1296.01 211.725 Q1296.01 205.359 1299.44 201.632 Q1302.89 197.882 1308.72 197.882 Q1313.95 197.882 1316.98 201.262 Q1320.04 204.618 1320.04 210.405 M1315.78 209.155 Q1315.73 205.66 1313.81 203.577 Q1311.91 201.493 1308.77 201.493 Q1305.2 201.493 1303.05 203.507 Q1300.92 205.521 1300.59 209.179 L1315.78 209.155 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M1344.09 211.169 Q1344.09 206.54 1342.17 203.993 Q1340.27 201.447 1336.82 201.447 Q1333.4 201.447 1331.47 203.993 Q1329.58 206.54 1329.58 211.169 Q1329.58 215.776 1331.47 218.322 Q1333.4 220.868 1336.82 220.868 Q1340.27 220.868 1342.17 218.322 Q1344.09 215.776 1344.09 211.169 M1348.35 221.216 Q1348.35 227.836 1345.41 231.054 Q1342.47 234.294 1336.4 234.294 Q1334.16 234.294 1332.17 233.947 Q1330.18 233.623 1328.3 232.929 L1328.3 228.785 Q1330.18 229.804 1332.01 230.29 Q1333.84 230.776 1335.73 230.776 Q1339.92 230.776 1342.01 228.577 Q1344.09 226.401 1344.09 221.98 L1344.09 219.873 Q1342.77 222.165 1340.71 223.299 Q1338.65 224.433 1335.78 224.433 Q1331.01 224.433 1328.09 220.799 Q1325.18 217.165 1325.18 211.169 Q1325.18 205.151 1328.09 201.517 Q1331.01 197.882 1335.78 197.882 Q1338.65 197.882 1340.71 199.017 Q1342.77 200.151 1344.09 202.443 L1344.09 198.507 L1348.35 198.507 L1348.35 221.216 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M1356.68 214.202 L1356.68 198.507 L1360.94 198.507 L1360.94 214.04 Q1360.94 217.72 1362.38 219.572 Q1363.81 221.401 1366.68 221.401 Q1370.13 221.401 1372.12 219.202 Q1374.14 217.003 1374.14 213.206 L1374.14 198.507 L1378.4 198.507 L1378.4 224.433 L1374.14 224.433 L1374.14 220.452 Q1372.59 222.813 1370.53 223.97 Q1368.49 225.104 1365.78 225.104 Q1361.31 225.104 1359 222.327 Q1356.68 219.549 1356.68 214.202 M1367.4 197.882 L1367.4 197.882 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M1387.17 188.415 L1391.43 188.415 L1391.43 224.433 L1387.17 224.433 L1387.17 188.415 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M1412.12 211.401 Q1406.96 211.401 1404.97 212.581 Q1402.98 213.762 1402.98 216.609 Q1402.98 218.878 1404.46 220.22 Q1405.96 221.54 1408.53 221.54 Q1412.08 221.54 1414.21 219.04 Q1416.36 216.517 1416.36 212.35 L1416.36 211.401 L1412.12 211.401 M1420.62 209.642 L1420.62 224.433 L1416.36 224.433 L1416.36 220.498 Q1414.9 222.859 1412.72 223.993 Q1410.55 225.104 1407.4 225.104 Q1403.42 225.104 1401.06 222.882 Q1398.72 220.637 1398.72 216.887 Q1398.72 212.512 1401.64 210.29 Q1404.58 208.068 1410.39 208.068 L1416.36 208.068 L1416.36 207.651 Q1416.36 204.711 1414.41 203.114 Q1412.49 201.493 1409 201.493 Q1406.77 201.493 1404.67 202.026 Q1402.56 202.558 1400.62 203.623 L1400.62 199.688 Q1402.96 198.785 1405.15 198.345 Q1407.35 197.882 1409.44 197.882 Q1415.06 197.882 1417.84 200.799 Q1420.62 203.716 1420.62 209.642 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M1433.6 191.146 L1433.6 198.507 L1442.38 198.507 L1442.38 201.818 L1433.6 201.818 L1433.6 215.892 Q1433.6 219.063 1434.46 219.966 Q1435.34 220.868 1438 220.868 L1442.38 220.868 L1442.38 224.433 L1438 224.433 Q1433.07 224.433 1431.2 222.604 Q1429.32 220.753 1429.32 215.892 L1429.32 201.818 L1426.2 201.818 L1426.2 198.507 L1429.32 198.507 L1429.32 191.146 L1433.6 191.146 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M1458.02 201.493 Q1454.6 201.493 1452.61 204.179 Q1450.62 206.841 1450.62 211.493 Q1450.62 216.146 1452.58 218.831 Q1454.58 221.493 1458.02 221.493 Q1461.43 221.493 1463.42 218.808 Q1465.41 216.123 1465.41 211.493 Q1465.41 206.887 1463.42 204.202 Q1461.43 201.493 1458.02 201.493 M1458.02 197.882 Q1463.58 197.882 1466.75 201.493 Q1469.92 205.105 1469.92 211.493 Q1469.92 217.859 1466.75 221.493 Q1463.58 225.104 1458.02 225.104 Q1452.45 225.104 1449.27 221.493 Q1446.13 217.859 1446.13 211.493 Q1446.13 205.105 1449.27 201.493 Q1452.45 197.882 1458.02 197.882 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M1492.01 202.489 Q1491.29 202.072 1490.43 201.887 Q1489.6 201.679 1488.58 201.679 Q1484.97 201.679 1483.02 204.04 Q1481.1 206.378 1481.1 210.776 L1481.1 224.433 L1476.82 224.433 L1476.82 198.507 L1481.1 198.507 L1481.1 202.535 Q1482.45 200.174 1484.6 199.04 Q1486.75 197.882 1489.83 197.882 Q1490.27 197.882 1490.8 197.952 Q1491.33 197.998 1491.98 198.114 L1492.01 202.489 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M1507.26 226.841 Q1505.45 231.47 1503.74 232.882 Q1502.03 234.294 1499.16 234.294 L1495.76 234.294 L1495.76 230.729 L1498.26 230.729 Q1500.02 230.729 1500.99 229.896 Q1501.96 229.063 1503.14 225.961 L1503.9 224.017 L1493.42 198.507 L1497.93 198.507 L1506.03 218.785 L1514.14 198.507 L1518.65 198.507 L1507.26 226.841 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M1560.5 191.007 L1560.5 195.568 Q1557.84 194.294 1555.48 193.669 Q1553.12 193.044 1550.92 193.044 Q1547.1 193.044 1545.01 194.526 Q1542.95 196.007 1542.95 198.739 Q1542.95 201.031 1544.32 202.211 Q1545.71 203.368 1549.55 204.086 L1552.38 204.665 Q1557.61 205.66 1560.08 208.183 Q1562.58 210.683 1562.58 214.896 Q1562.58 219.919 1559.2 222.512 Q1555.85 225.104 1549.34 225.104 Q1546.89 225.104 1544.11 224.549 Q1541.36 223.993 1538.39 222.905 L1538.39 218.091 Q1541.24 219.688 1543.97 220.498 Q1546.7 221.308 1549.34 221.308 Q1553.35 221.308 1555.52 219.734 Q1557.7 218.16 1557.7 215.243 Q1557.7 212.697 1556.13 211.262 Q1554.57 209.827 1551.01 209.109 L1548.16 208.554 Q1542.93 207.512 1540.59 205.29 Q1538.26 203.068 1538.26 199.109 Q1538.26 194.526 1541.47 191.887 Q1544.71 189.248 1550.39 189.248 Q1552.82 189.248 1555.34 189.688 Q1557.86 190.128 1560.5 191.007 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M1573.9 191.146 L1573.9 198.507 L1582.68 198.507 L1582.68 201.818 L1573.9 201.818 L1573.9 215.892 Q1573.9 219.063 1574.76 219.966 Q1575.64 220.868 1578.3 220.868 L1582.68 220.868 L1582.68 224.433 L1578.3 224.433 Q1573.37 224.433 1571.5 222.604 Q1569.62 220.753 1569.62 215.892 L1569.62 201.818 L1566.5 201.818 L1566.5 198.507 L1569.62 198.507 L1569.62 191.146 L1573.9 191.146 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M1600.06 211.401 Q1594.9 211.401 1592.91 212.581 Q1590.92 213.762 1590.92 216.609 Q1590.92 218.878 1592.4 220.22 Q1593.9 221.54 1596.47 221.54 Q1600.01 221.54 1602.14 219.04 Q1604.3 216.517 1604.3 212.35 L1604.3 211.401 L1600.06 211.401 M1608.56 209.642 L1608.56 224.433 L1604.3 224.433 L1604.3 220.498 Q1602.84 222.859 1600.66 223.993 Q1598.49 225.104 1595.34 225.104 Q1591.36 225.104 1589 222.882 Q1586.66 220.637 1586.66 216.887 Q1586.66 212.512 1589.57 210.29 Q1592.51 208.068 1598.32 208.068 L1604.3 208.068 L1604.3 207.651 Q1604.3 204.711 1602.35 203.114 Q1600.43 201.493 1596.94 201.493 Q1594.71 201.493 1592.61 202.026 Q1590.5 202.558 1588.56 203.623 L1588.56 199.688 Q1590.89 198.785 1593.09 198.345 Q1595.29 197.882 1597.38 197.882 Q1603 197.882 1605.78 200.799 Q1608.56 203.716 1608.56 209.642 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M1638.88 208.785 L1638.88 224.433 L1634.62 224.433 L1634.62 208.924 Q1634.62 205.243 1633.19 203.415 Q1631.75 201.586 1628.88 201.586 Q1625.43 201.586 1623.44 203.785 Q1621.45 205.984 1621.45 209.78 L1621.45 224.433 L1617.17 224.433 L1617.17 198.507 L1621.45 198.507 L1621.45 202.535 Q1622.98 200.197 1625.04 199.04 Q1627.12 197.882 1629.83 197.882 Q1634.3 197.882 1636.59 200.66 Q1638.88 203.415 1638.88 208.785 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M1664.44 202.443 L1664.44 188.415 L1668.69 188.415 L1668.69 224.433 L1664.44 224.433 L1664.44 220.544 Q1663.09 222.859 1661.03 223.993 Q1659 225.104 1656.13 225.104 Q1651.43 225.104 1648.46 221.355 Q1645.52 217.605 1645.52 211.493 Q1645.52 205.382 1648.46 201.632 Q1651.43 197.882 1656.13 197.882 Q1659 197.882 1661.03 199.017 Q1663.09 200.128 1664.44 202.443 M1649.92 211.493 Q1649.92 216.192 1651.84 218.878 Q1653.79 221.54 1657.17 221.54 Q1660.55 221.54 1662.49 218.878 Q1664.44 216.192 1664.44 211.493 Q1664.44 206.794 1662.49 204.132 Q1660.55 201.447 1657.17 201.447 Q1653.79 201.447 1651.84 204.132 Q1649.92 206.794 1649.92 211.493 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M1689.25 211.401 Q1684.09 211.401 1682.1 212.581 Q1680.11 213.762 1680.11 216.609 Q1680.11 218.878 1681.59 220.22 Q1683.09 221.54 1685.66 221.54 Q1689.2 221.54 1691.33 219.04 Q1693.49 216.517 1693.49 212.35 L1693.49 211.401 L1689.25 211.401 M1697.75 209.642 L1697.75 224.433 L1693.49 224.433 L1693.49 220.498 Q1692.03 222.859 1689.85 223.993 Q1687.68 225.104 1684.53 225.104 Q1680.55 225.104 1678.19 222.882 Q1675.85 220.637 1675.85 216.887 Q1675.85 212.512 1678.76 210.29 Q1681.7 208.068 1687.51 208.068 L1693.49 208.068 L1693.49 207.651 Q1693.49 204.711 1691.54 203.114 Q1689.62 201.493 1686.13 201.493 Q1683.9 201.493 1681.8 202.026 Q1679.69 202.558 1677.75 203.623 L1677.75 199.688 Q1680.08 198.785 1682.28 198.345 Q1684.48 197.882 1686.57 197.882 Q1692.19 197.882 1694.97 200.799 Q1697.75 203.716 1697.75 209.642 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M1721.54 202.489 Q1720.82 202.072 1719.97 201.887 Q1719.13 201.679 1718.12 201.679 Q1714.5 201.679 1712.56 204.04 Q1710.64 206.378 1710.64 210.776 L1710.64 224.433 L1706.36 224.433 L1706.36 198.507 L1710.64 198.507 L1710.64 202.535 Q1711.98 200.174 1714.13 199.04 Q1716.29 197.882 1719.37 197.882 Q1719.81 197.882 1720.34 197.952 Q1720.87 197.998 1721.52 198.114 L1721.54 202.489 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip350)" d="M1742.24 202.443 L1742.24 188.415 L1746.5 188.415 L1746.5 224.433 L1742.24 224.433 L1742.24 220.544 Q1740.89 222.859 1738.83 223.993 Q1736.8 225.104 1733.93 225.104 Q1729.23 225.104 1726.26 221.355 Q1723.32 217.605 1723.32 211.493 Q1723.32 205.382 1726.26 201.632 Q1729.23 197.882 1733.93 197.882 Q1736.8 197.882 1738.83 199.017 Q1740.89 200.128 1742.24 202.443 M1727.72 211.493 Q1727.72 216.192 1729.64 218.878 Q1731.59 221.54 1734.97 221.54 Q1738.35 221.54 1740.29 218.878 Q1742.24 216.192 1742.24 211.493 Q1742.24 206.794 1740.29 204.132 Q1738.35 201.447 1734.97 201.447 Q1731.59 201.447 1729.64 204.132 Q1727.72 206.794 1727.72 211.493 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /></svg>

Figure 1: Simulated dissolved oxygen concentration for Problem 1 without
treatment.

1.  These lines of code separate the creation of the plot axis (with
    labels, legend positions, etc) using `p = plot(...)` from the
    plotting of the data with `plot!(p, ...)`. You can do this all in a
    single `plot()` call, but this may sometimes make things more
    readable when there are a lot of style arguments for the axes.

We can see that the DO concentration falls below the regulatory standard
of 4 mg/L before 20 km downstream. To find the minimum value, we can use
the `minimum()` function.

``` julia
@show minimum(C);
```

    minimum(C) = 3.8423213306662958

So the minimum value is 3.8 mg/L.

To determine the minimum treatment of the waste streams needed to
maintain compliance, we will write a function which will apply treatment
levels `eff1` and `eff2` and evaluate our function above, returning the
minimum DO concentration.

``` julia
# waste_treat: function to simulate DO with treated discharges; treatments apply to CBOD and NBOD released
# inputs:
#   - eff1: efficiency of treatment for waste stream 1 (as a decimal)
#   - eff2: efficiency of treatment for waste stream 2 (as a decimal)
#   - inflow: tuple with inflow properties: (Volume, DO, CBOD, NBOD)
#   - waste1: tuple with waste stream 1 properties: (Volume, DO, CBOD, NBOD)
#   - waste2: tuple with waste stream 2 properties: (Volume, DO, CBOD, NBOD)
#   - U: velocity of river (km/d)
#   - Cₛ = saturation oxygen concentration (mg/L)
#   - ka, kc, kn: reaeration, CBOD decay, and NBOD decay rates, respectively (d^{-1})
function waste_treat(eff1, eff2, inflow, waste1, waste2, U, Cₛ, ka, kb, kn)
    waste1_treated = (waste1[1], waste1[2], (1 - eff1) * waste1[3], (1 - eff1) * waste1[4])    
    waste2_treated = (waste2[1], waste2[2], (1 - eff2) * waste2[3], (1 - eff2) * waste2[4])    
    C, B, N = do_simulate(inflow, waste1_treated, waste2_treated, U, Cₛ, ka, kb, kn)
    return minimum(C)
end
```

    waste_treat (generic function with 1 method)

Now we can evaluate this function over a range of treatment
efficiencies. There are a number of ways to do this, but we’ll use a
trick which Julia makes easy: broadcasting using an anonymous function.

``` julia
# evaluate treatment efficiencies between 0 and 1
effs = 0:0.01:1
treat1 = (e1 -> waste_treat(e1, 0, inflow, waste1, waste2, U, Cₛ, ka, kb, kn)).(effs)
treat2 = (e2 -> waste_treat(0, e2, inflow, waste1, waste2, U, Cₛ, ka, kb, kn)).(effs)
```

    101-element Vector{Float64}:
     3.8423213306662958
     3.8631158474631597
     3.883910364260024
     3.9047048810568867
     3.925499397853751
     3.9460612716757266
     3.965197323117371
     3.984333374559016
     4.0034694260006605
     4.022605477442305
     ⋮
     4.711700098308851
     4.711700098308851
     4.711700098308851
     4.711700098308851
     4.711700098308851
     4.711700098308851
     4.711700098308851
     4.711700098308851
     4.711700098308851

To find the minimum treatment level which ensures compliance, we can now
find the position of the first value where the minimum value is at least
4 mg/L, and look up the associated efficiency. Julia provides the
`findfirst()` function which lets you find the index of the first value
satisfying a Boolean condition.

``` julia
# find indices and treatment values
idx1 = findfirst(treat1 .>= 4.0)
idx2 = findfirst(treat2 .>= 4.0)
@show effs[idx1];
@show effs[idx2];
```

    effs[idx1] = 0.11
    effs[idx2] = 0.08

So the minimum treatment level for waste stream 1 is 11% and the minimum
treatment level for waste stream 2 is 8%.

There is no “right” answer to the question of which treatment option you
would pick, so long as your solution is thoughtful and justified.

-   For example, one could argue that as waste stream 1 on its own does
    not result in a lack of compliance (which we can see from
    <a href="#fig-do-untreated" class="quarto-xref">Figure 1</a>, as the
    initial “sag” has started to recover prior to waste stream 2), waste
    stream 2 ought to be treated.
-   On the other hand, waste stream 1 has a much more negative impact on
    the inflow dissolved oxygen levels, and waste stream 2 might not
    cause a lack of compliance without that effect, which might suggest
    that waste stream 1 should be treated.
-   Another consideration might be the relative cost of treating each
    waste stream, which we have no information on, or whether these
    waste streams are from municipal, residual, or industrial sources
    (in other words, non-profit vs. profit).

### Problem 2 (20 points)

Loading the data:

``` julia
# Dataset from https://zenodo.org/record/3973015
# The CSV is read into a DataFrame object, and we specify that it is comma delimited
forcings_all = CSV.read("data/ERF_ssp585_1750-2500.csv", DataFrame, delim=",")

# Separate out the individual components
# Get total aerosol forcings
forcing_aerosol_rad = forcings_all[!,"aerosol-radiation_interactions"]
forcing_aerosol_cloud = forcings_all[!,"aerosol-cloud_interactions"]
forcing_aerosol = forcing_aerosol_rad + forcing_aerosol_cloud
# Calculate non-aerosol forcings from the total.
forcing_total = forcings_all[!,"total"]
forcing_non_aerosol = forcing_total - forcing_aerosol
t = Int64.(forcings_all[!,"year"])
idx_2100 = findfirst(t .== 2100) # find the index which corresponds to the year 2100
```

    351

The energy balance model is given by
$$cd \frac{dT}{dt} = F - \lambda T.$$ Discretizing with Forward Euler,
we get $$
\begin{aligned}
cd\frac{T(t + \Delta t) - T(t)}{\Delta t} &= F(t) - \lambda T(t) \\
T(t + \Delta t) &= T(t) + \frac{\Delta t}{cd} \left(F(t) - \lambda T(t)\right).
\end{aligned}
$$

In Julia code (and with the assumed parameter values), this becomes the
following function (replacing
$F(t) = F_\text{nonaerosol} + \alpha F_\text{aerosol}$):

``` julia
# ebm: function to simulate the energy balance model
# inputs:
#   - t: vector of time values.
#   - F_nonaerosol: vector of non-aerosol radiative forcing values
#   - F_aerosol: vector of aerosol radiative forcing values
#   - c, d, λ, α, Δt: model parameters, see below.
function ebm(t, F_nonaerosol, F_aerosol, c, d, λ, α, Δt)
    T = zeros(length(t))
    F = F_nonaerosol + α * F_aerosol
    for s in 1:length(t)-1
        T[s+1] = T[s] + Δt * (F[s] - λ * T[s]) / (c * d)
    end
    return T
end

c = 4.184e6 # specific heat of water, J/K/m^2
d = 86 # ocean mixing depth, m
λ = 2.1 # climate feedback factor
α = 0.8 # aerosol scaling factor
Δt = 31_558_152 # annual time step, s

temps = ebm(t, forcing_non_aerosol, forcing_aerosol, c, d, λ, α, Δt)
```

    751-element Vector{Float64}:
     0.0
     0.022731483349329427
     0.040278289494741326
     0.05240826578450054
     0.05895069636157351
     0.06063584224928685
     0.058278631700672184
     0.036311110702349894
     0.03060499874837628
     0.03778068117354878
     ⋮
     6.0349064449781515
     6.033928949865114
     6.032952263810847
     6.031978595303466
     6.0310066214578395
     6.030034177683138
     6.029061472805756
     6.028090703835366
     6.027123866872981

1.  The `zeros` function initializes a vector with zero values of the
    given length. This is a good way to initialize storage when you
    aren’t worried about distinguishing non-set values from “true”
    zeroes.
2.  While normally `eachindex(t)` is preferred instead of `1:length(t)`,
    in this case we need to iterate only until the second-to-last index
    of `t` as we set the next value `T[s+1]` in the loop.

The simulated temperatures are plotted in
<a href="#fig-temps" class="quarto-xref">Figure 2</a>:

``` julia
p = plot(; xlabel="Year", ylabel="Global Mean Temperature Anomaly (°C)", legend=false)
plot!(p, t, temps)
xlims!(p, (1750, 2100))
```

<?xml version="1.0" encoding="utf-8"?>
<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="672" height="480" viewBox="0 0 2688 1920">
<defs>
  <clipPath id="clip410">
    <rect x="0" y="0" width="2688" height="1920"/>
  </clipPath>
</defs>
<path clip-path="url(#clip410)" d="M0 1920 L2688 1920 L2688 -4.26326e-14 L0 -4.26326e-14  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<defs>
  <clipPath id="clip411">
    <rect x="537" y="0" width="1883" height="1883"/>
  </clipPath>
</defs>
<path clip-path="url(#clip410)" d="M181.719 1734.12 L2640.76 1734.12 L2640.76 47.2441 L181.719 47.2441  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<defs>
  <clipPath id="clip412">
    <rect x="181" y="47" width="2460" height="1688"/>
  </clipPath>
</defs>
<polyline clip-path="url(#clip412)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="181.719,1734.12 181.719,47.2441 "/>
<polyline clip-path="url(#clip412)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="533.01,1734.12 533.01,47.2441 "/>
<polyline clip-path="url(#clip412)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="884.301,1734.12 884.301,47.2441 "/>
<polyline clip-path="url(#clip412)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="1235.59,1734.12 1235.59,47.2441 "/>
<polyline clip-path="url(#clip412)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="1586.88,1734.12 1586.88,47.2441 "/>
<polyline clip-path="url(#clip412)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="1938.17,1734.12 1938.17,47.2441 "/>
<polyline clip-path="url(#clip412)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="2289.46,1734.12 2289.46,47.2441 "/>
<polyline clip-path="url(#clip412)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="2640.76,1734.12 2640.76,47.2441 "/>
<polyline clip-path="url(#clip410)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="181.719,1734.12 2640.76,1734.12 "/>
<polyline clip-path="url(#clip410)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="181.719,1734.12 181.719,1715.22 "/>
<polyline clip-path="url(#clip410)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="533.01,1734.12 533.01,1715.22 "/>
<polyline clip-path="url(#clip410)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="884.301,1734.12 884.301,1715.22 "/>
<polyline clip-path="url(#clip410)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1235.59,1734.12 1235.59,1715.22 "/>
<polyline clip-path="url(#clip410)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1586.88,1734.12 1586.88,1715.22 "/>
<polyline clip-path="url(#clip410)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1938.17,1734.12 1938.17,1715.22 "/>
<polyline clip-path="url(#clip410)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2289.46,1734.12 2289.46,1715.22 "/>
<polyline clip-path="url(#clip410)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2640.76,1734.12 2640.76,1715.22 "/>
<path clip-path="url(#clip410)" d="M126.245 1794.89 L133.884 1794.89 L133.884 1768.52 L125.574 1770.19 L125.574 1765.93 L133.838 1764.26 L138.514 1764.26 L138.514 1794.89 L146.153 1794.89 L146.153 1798.82 L126.245 1798.82 L126.245 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M154.416 1764.26 L176.639 1764.26 L176.639 1766.25 L164.092 1798.82 L159.208 1798.82 L171.014 1768.2 L154.416 1768.2 L154.416 1764.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M185.805 1764.26 L204.162 1764.26 L204.162 1768.2 L190.088 1768.2 L190.088 1776.67 Q191.106 1776.32 192.125 1776.16 Q193.143 1775.97 194.162 1775.97 Q199.949 1775.97 203.328 1779.15 Q206.708 1782.32 206.708 1787.73 Q206.708 1793.31 203.236 1796.41 Q199.763 1799.49 193.444 1799.49 Q191.268 1799.49 189 1799.12 Q186.754 1798.75 184.347 1798.01 L184.347 1793.31 Q186.43 1794.45 188.652 1795 Q190.875 1795.56 193.351 1795.56 Q197.356 1795.56 199.694 1793.45 Q202.032 1791.34 202.032 1787.73 Q202.032 1784.12 199.694 1782.02 Q197.356 1779.91 193.351 1779.91 Q191.476 1779.91 189.601 1780.33 Q187.75 1780.74 185.805 1781.62 L185.805 1764.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M225.921 1767.34 Q222.31 1767.34 220.481 1770.9 Q218.675 1774.45 218.675 1781.58 Q218.675 1788.68 220.481 1792.25 Q222.31 1795.79 225.921 1795.79 Q229.555 1795.79 231.36 1792.25 Q233.189 1788.68 233.189 1781.58 Q233.189 1774.45 231.36 1770.9 Q229.555 1767.34 225.921 1767.34 M225.921 1763.64 Q231.731 1763.64 234.786 1768.24 Q237.865 1772.83 237.865 1781.58 Q237.865 1790.3 234.786 1794.91 Q231.731 1799.49 225.921 1799.49 Q220.11 1799.49 217.032 1794.91 Q213.976 1790.3 213.976 1781.58 Q213.976 1772.83 217.032 1768.24 Q220.11 1763.64 225.921 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M477.536 1794.89 L485.175 1794.89 L485.175 1768.52 L476.865 1770.19 L476.865 1765.93 L485.129 1764.26 L489.805 1764.26 L489.805 1794.89 L497.443 1794.89 L497.443 1798.82 L477.536 1798.82 L477.536 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M516.888 1782.41 Q513.554 1782.41 511.633 1784.19 Q509.735 1785.97 509.735 1789.1 Q509.735 1792.22 511.633 1794.01 Q513.554 1795.79 516.888 1795.79 Q520.221 1795.79 522.142 1794.01 Q524.064 1792.2 524.064 1789.1 Q524.064 1785.97 522.142 1784.19 Q520.244 1782.41 516.888 1782.41 M512.212 1780.42 Q509.203 1779.68 507.513 1777.62 Q505.846 1775.56 505.846 1772.59 Q505.846 1768.45 508.786 1766.04 Q511.749 1763.64 516.888 1763.64 Q522.05 1763.64 524.99 1766.04 Q527.929 1768.45 527.929 1772.59 Q527.929 1775.56 526.24 1777.62 Q524.573 1779.68 521.587 1780.42 Q524.966 1781.21 526.841 1783.5 Q528.74 1785.79 528.74 1789.1 Q528.74 1794.12 525.661 1796.81 Q522.605 1799.49 516.888 1799.49 Q511.17 1799.49 508.092 1796.81 Q505.036 1794.12 505.036 1789.1 Q505.036 1785.79 506.934 1783.5 Q508.832 1781.21 512.212 1780.42 M510.499 1773.03 Q510.499 1775.72 512.166 1777.22 Q513.855 1778.73 516.888 1778.73 Q519.897 1778.73 521.587 1777.22 Q523.3 1775.72 523.3 1773.03 Q523.3 1770.35 521.587 1768.84 Q519.897 1767.34 516.888 1767.34 Q513.855 1767.34 512.166 1768.84 Q510.499 1770.35 510.499 1773.03 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M547.05 1767.34 Q543.439 1767.34 541.61 1770.9 Q539.804 1774.45 539.804 1781.58 Q539.804 1788.68 541.61 1792.25 Q543.439 1795.79 547.05 1795.79 Q550.684 1795.79 552.489 1792.25 Q554.318 1788.68 554.318 1781.58 Q554.318 1774.45 552.489 1770.9 Q550.684 1767.34 547.05 1767.34 M547.05 1763.64 Q552.86 1763.64 555.915 1768.24 Q558.994 1772.83 558.994 1781.58 Q558.994 1790.3 555.915 1794.91 Q552.86 1799.49 547.05 1799.49 Q541.24 1799.49 538.161 1794.91 Q535.105 1790.3 535.105 1781.58 Q535.105 1772.83 538.161 1768.24 Q541.24 1763.64 547.05 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M577.212 1767.34 Q573.6 1767.34 571.772 1770.9 Q569.966 1774.45 569.966 1781.58 Q569.966 1788.68 571.772 1792.25 Q573.6 1795.79 577.212 1795.79 Q580.846 1795.79 582.651 1792.25 Q584.48 1788.68 584.48 1781.58 Q584.48 1774.45 582.651 1770.9 Q580.846 1767.34 577.212 1767.34 M577.212 1763.64 Q583.022 1763.64 586.077 1768.24 Q589.156 1772.83 589.156 1781.58 Q589.156 1790.3 586.077 1794.91 Q583.022 1799.49 577.212 1799.49 Q571.401 1799.49 568.323 1794.91 Q565.267 1790.3 565.267 1781.58 Q565.267 1772.83 568.323 1768.24 Q571.401 1763.64 577.212 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M828.827 1794.89 L836.466 1794.89 L836.466 1768.52 L828.156 1770.19 L828.156 1765.93 L836.42 1764.26 L841.096 1764.26 L841.096 1794.89 L848.734 1794.89 L848.734 1798.82 L828.827 1798.82 L828.827 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M868.179 1782.41 Q864.845 1782.41 862.924 1784.19 Q861.026 1785.97 861.026 1789.1 Q861.026 1792.22 862.924 1794.01 Q864.845 1795.79 868.179 1795.79 Q871.512 1795.79 873.433 1794.01 Q875.355 1792.2 875.355 1789.1 Q875.355 1785.97 873.433 1784.19 Q871.535 1782.41 868.179 1782.41 M863.503 1780.42 Q860.494 1779.68 858.804 1777.62 Q857.137 1775.56 857.137 1772.59 Q857.137 1768.45 860.077 1766.04 Q863.04 1763.64 868.179 1763.64 Q873.341 1763.64 876.281 1766.04 Q879.22 1768.45 879.22 1772.59 Q879.22 1775.56 877.531 1777.62 Q875.864 1779.68 872.878 1780.42 Q876.257 1781.21 878.132 1783.5 Q880.031 1785.79 880.031 1789.1 Q880.031 1794.12 876.952 1796.81 Q873.896 1799.49 868.179 1799.49 Q862.461 1799.49 859.382 1796.81 Q856.327 1794.12 856.327 1789.1 Q856.327 1785.79 858.225 1783.5 Q860.123 1781.21 863.503 1780.42 M861.79 1773.03 Q861.79 1775.72 863.457 1777.22 Q865.146 1778.73 868.179 1778.73 Q871.188 1778.73 872.878 1777.22 Q874.591 1775.72 874.591 1773.03 Q874.591 1770.35 872.878 1768.84 Q871.188 1767.34 868.179 1767.34 Q865.146 1767.34 863.457 1768.84 Q861.79 1770.35 861.79 1773.03 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M888.387 1764.26 L906.743 1764.26 L906.743 1768.2 L892.669 1768.2 L892.669 1776.67 Q893.688 1776.32 894.706 1776.16 Q895.725 1775.97 896.743 1775.97 Q902.53 1775.97 905.91 1779.15 Q909.29 1782.32 909.29 1787.73 Q909.29 1793.31 905.817 1796.41 Q902.345 1799.49 896.026 1799.49 Q893.85 1799.49 891.581 1799.12 Q889.336 1798.75 886.929 1798.01 L886.929 1793.31 Q889.012 1794.45 891.234 1795 Q893.456 1795.56 895.933 1795.56 Q899.938 1795.56 902.276 1793.45 Q904.614 1791.34 904.614 1787.73 Q904.614 1784.12 902.276 1782.02 Q899.938 1779.91 895.933 1779.91 Q894.058 1779.91 892.183 1780.33 Q890.331 1780.74 888.387 1781.62 L888.387 1764.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M928.502 1767.34 Q924.891 1767.34 923.063 1770.9 Q921.257 1774.45 921.257 1781.58 Q921.257 1788.68 923.063 1792.25 Q924.891 1795.79 928.502 1795.79 Q932.137 1795.79 933.942 1792.25 Q935.771 1788.68 935.771 1781.58 Q935.771 1774.45 933.942 1770.9 Q932.137 1767.34 928.502 1767.34 M928.502 1763.64 Q934.313 1763.64 937.368 1768.24 Q940.447 1772.83 940.447 1781.58 Q940.447 1790.3 937.368 1794.91 Q934.313 1799.49 928.502 1799.49 Q922.692 1799.49 919.614 1794.91 Q916.558 1790.3 916.558 1781.58 Q916.558 1772.83 919.614 1768.24 Q922.692 1763.64 928.502 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M1180.12 1794.89 L1187.76 1794.89 L1187.76 1768.52 L1179.45 1770.19 L1179.45 1765.93 L1187.71 1764.26 L1192.39 1764.26 L1192.39 1794.89 L1200.03 1794.89 L1200.03 1798.82 L1180.12 1798.82 L1180.12 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M1209.61 1798.1 L1209.61 1793.84 Q1211.37 1794.68 1213.17 1795.12 Q1214.98 1795.56 1216.72 1795.56 Q1221.34 1795.56 1223.78 1792.46 Q1226.23 1789.33 1226.58 1782.99 Q1225.23 1784.98 1223.17 1786.04 Q1221.11 1787.11 1218.61 1787.11 Q1213.43 1787.11 1210.4 1783.98 Q1207.39 1780.83 1207.39 1775.4 Q1207.39 1770.07 1210.53 1766.85 Q1213.68 1763.64 1218.91 1763.64 Q1224.91 1763.64 1228.06 1768.24 Q1231.23 1772.83 1231.23 1781.58 Q1231.23 1789.75 1227.34 1794.63 Q1223.47 1799.49 1216.92 1799.49 Q1215.16 1799.49 1213.36 1799.15 Q1211.55 1798.8 1209.61 1798.1 M1218.91 1783.45 Q1222.06 1783.45 1223.89 1781.3 Q1225.74 1779.15 1225.74 1775.4 Q1225.74 1771.67 1223.89 1769.52 Q1222.06 1767.34 1218.91 1767.34 Q1215.77 1767.34 1213.91 1769.52 Q1212.09 1771.67 1212.09 1775.4 Q1212.09 1779.15 1213.91 1781.3 Q1215.77 1783.45 1218.91 1783.45 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M1249.63 1767.34 Q1246.02 1767.34 1244.19 1770.9 Q1242.39 1774.45 1242.39 1781.58 Q1242.39 1788.68 1244.19 1792.25 Q1246.02 1795.79 1249.63 1795.79 Q1253.27 1795.79 1255.07 1792.25 Q1256.9 1788.68 1256.9 1781.58 Q1256.9 1774.45 1255.07 1770.9 Q1253.27 1767.34 1249.63 1767.34 M1249.63 1763.64 Q1255.44 1763.64 1258.5 1768.24 Q1261.58 1772.83 1261.58 1781.58 Q1261.58 1790.3 1258.5 1794.91 Q1255.44 1799.49 1249.63 1799.49 Q1243.82 1799.49 1240.74 1794.91 Q1237.69 1790.3 1237.69 1781.58 Q1237.69 1772.83 1240.74 1768.24 Q1243.82 1763.64 1249.63 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M1279.79 1767.34 Q1276.18 1767.34 1274.35 1770.9 Q1272.55 1774.45 1272.55 1781.58 Q1272.55 1788.68 1274.35 1792.25 Q1276.18 1795.79 1279.79 1795.79 Q1283.43 1795.79 1285.23 1792.25 Q1287.06 1788.68 1287.06 1781.58 Q1287.06 1774.45 1285.23 1770.9 Q1283.43 1767.34 1279.79 1767.34 M1279.79 1763.64 Q1285.6 1763.64 1288.66 1768.24 Q1291.74 1772.83 1291.74 1781.58 Q1291.74 1790.3 1288.66 1794.91 Q1285.6 1799.49 1279.79 1799.49 Q1273.98 1799.49 1270.9 1794.91 Q1267.85 1790.3 1267.85 1781.58 Q1267.85 1772.83 1270.9 1768.24 Q1273.98 1763.64 1279.79 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M1531.41 1794.89 L1539.05 1794.89 L1539.05 1768.52 L1530.74 1770.19 L1530.74 1765.93 L1539 1764.26 L1543.68 1764.26 L1543.68 1794.89 L1551.32 1794.89 L1551.32 1798.82 L1531.41 1798.82 L1531.41 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M1560.9 1798.1 L1560.9 1793.84 Q1562.66 1794.68 1564.46 1795.12 Q1566.27 1795.56 1568.01 1795.56 Q1572.64 1795.56 1575.07 1792.46 Q1577.52 1789.33 1577.87 1782.99 Q1576.52 1784.98 1574.46 1786.04 Q1572.4 1787.11 1569.9 1787.11 Q1564.72 1787.11 1561.69 1783.98 Q1558.68 1780.83 1558.68 1775.4 Q1558.68 1770.07 1561.83 1766.85 Q1564.97 1763.64 1570.21 1763.64 Q1576.2 1763.64 1579.35 1768.24 Q1582.52 1772.83 1582.52 1781.58 Q1582.52 1789.75 1578.63 1794.63 Q1574.77 1799.49 1568.21 1799.49 Q1566.46 1799.49 1564.65 1799.15 Q1562.84 1798.8 1560.9 1798.1 M1570.21 1783.45 Q1573.35 1783.45 1575.18 1781.3 Q1577.03 1779.15 1577.03 1775.4 Q1577.03 1771.67 1575.18 1769.52 Q1573.35 1767.34 1570.21 1767.34 Q1567.06 1767.34 1565.21 1769.52 Q1563.38 1771.67 1563.38 1775.4 Q1563.38 1779.15 1565.21 1781.3 Q1567.06 1783.45 1570.21 1783.45 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M1590.97 1764.26 L1609.33 1764.26 L1609.33 1768.2 L1595.25 1768.2 L1595.25 1776.67 Q1596.27 1776.32 1597.29 1776.16 Q1598.31 1775.97 1599.33 1775.97 Q1605.11 1775.97 1608.49 1779.15 Q1611.87 1782.32 1611.87 1787.73 Q1611.87 1793.31 1608.4 1796.41 Q1604.93 1799.49 1598.61 1799.49 Q1596.43 1799.49 1594.16 1799.12 Q1591.92 1798.75 1589.51 1798.01 L1589.51 1793.31 Q1591.59 1794.45 1593.82 1795 Q1596.04 1795.56 1598.52 1795.56 Q1602.52 1795.56 1604.86 1793.45 Q1607.2 1791.34 1607.2 1787.73 Q1607.2 1784.12 1604.86 1782.02 Q1602.52 1779.91 1598.52 1779.91 Q1596.64 1779.91 1594.77 1780.33 Q1592.91 1780.74 1590.97 1781.62 L1590.97 1764.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M1631.08 1767.34 Q1627.47 1767.34 1625.64 1770.9 Q1623.84 1774.45 1623.84 1781.58 Q1623.84 1788.68 1625.64 1792.25 Q1627.47 1795.79 1631.08 1795.79 Q1634.72 1795.79 1636.52 1792.25 Q1638.35 1788.68 1638.35 1781.58 Q1638.35 1774.45 1636.52 1770.9 Q1634.72 1767.34 1631.08 1767.34 M1631.08 1763.64 Q1636.89 1763.64 1639.95 1768.24 Q1643.03 1772.83 1643.03 1781.58 Q1643.03 1790.3 1639.95 1794.91 Q1636.89 1799.49 1631.08 1799.49 Q1625.27 1799.49 1622.2 1794.91 Q1619.14 1790.3 1619.14 1781.58 Q1619.14 1772.83 1622.2 1768.24 Q1625.27 1763.64 1631.08 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M1886.79 1794.89 L1903.1 1794.89 L1903.1 1798.82 L1881.16 1798.82 L1881.16 1794.89 Q1883.82 1792.13 1888.41 1787.5 Q1893.01 1782.85 1894.19 1781.51 Q1896.44 1778.98 1897.32 1777.25 Q1898.22 1775.49 1898.22 1773.8 Q1898.22 1771.04 1896.28 1769.31 Q1894.35 1767.57 1891.25 1767.57 Q1889.05 1767.57 1886.6 1768.34 Q1884.17 1769.1 1881.39 1770.65 L1881.39 1765.93 Q1884.22 1764.79 1886.67 1764.21 Q1889.12 1763.64 1891.16 1763.64 Q1896.53 1763.64 1899.73 1766.32 Q1902.92 1769.01 1902.92 1773.5 Q1902.92 1775.63 1902.11 1777.55 Q1901.32 1779.45 1899.22 1782.04 Q1898.64 1782.71 1895.54 1785.93 Q1892.43 1789.12 1886.79 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M1922.92 1767.34 Q1919.31 1767.34 1917.48 1770.9 Q1915.67 1774.45 1915.67 1781.58 Q1915.67 1788.68 1917.48 1792.25 Q1919.31 1795.79 1922.92 1795.79 Q1926.55 1795.79 1928.36 1792.25 Q1930.19 1788.68 1930.19 1781.58 Q1930.19 1774.45 1928.36 1770.9 Q1926.55 1767.34 1922.92 1767.34 M1922.92 1763.64 Q1928.73 1763.64 1931.79 1768.24 Q1934.86 1772.83 1934.86 1781.58 Q1934.86 1790.3 1931.79 1794.91 Q1928.73 1799.49 1922.92 1799.49 Q1917.11 1799.49 1914.03 1794.91 Q1910.98 1790.3 1910.98 1781.58 Q1910.98 1772.83 1914.03 1768.24 Q1917.11 1763.64 1922.92 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M1953.08 1767.34 Q1949.47 1767.34 1947.64 1770.9 Q1945.84 1774.45 1945.84 1781.58 Q1945.84 1788.68 1947.64 1792.25 Q1949.47 1795.79 1953.08 1795.79 Q1956.72 1795.79 1958.52 1792.25 Q1960.35 1788.68 1960.35 1781.58 Q1960.35 1774.45 1958.52 1770.9 Q1956.72 1767.34 1953.08 1767.34 M1953.08 1763.64 Q1958.89 1763.64 1961.95 1768.24 Q1965.03 1772.83 1965.03 1781.58 Q1965.03 1790.3 1961.95 1794.91 Q1958.89 1799.49 1953.08 1799.49 Q1947.27 1799.49 1944.19 1794.91 Q1941.14 1790.3 1941.14 1781.58 Q1941.14 1772.83 1944.19 1768.24 Q1947.27 1763.64 1953.08 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M1983.24 1767.34 Q1979.63 1767.34 1977.8 1770.9 Q1976 1774.45 1976 1781.58 Q1976 1788.68 1977.8 1792.25 Q1979.63 1795.79 1983.24 1795.79 Q1986.88 1795.79 1988.68 1792.25 Q1990.51 1788.68 1990.51 1781.58 Q1990.51 1774.45 1988.68 1770.9 Q1986.88 1767.34 1983.24 1767.34 M1983.24 1763.64 Q1989.05 1763.64 1992.11 1768.24 Q1995.19 1772.83 1995.19 1781.58 Q1995.19 1790.3 1992.11 1794.91 Q1989.05 1799.49 1983.24 1799.49 Q1977.43 1799.49 1974.35 1794.91 Q1971.3 1790.3 1971.3 1781.58 Q1971.3 1772.83 1974.35 1768.24 Q1977.43 1763.64 1983.24 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M2238.08 1794.89 L2254.4 1794.89 L2254.4 1798.82 L2232.45 1798.82 L2232.45 1794.89 Q2235.11 1792.13 2239.7 1787.5 Q2244.3 1782.85 2245.48 1781.51 Q2247.73 1778.98 2248.61 1777.25 Q2249.51 1775.49 2249.51 1773.8 Q2249.51 1771.04 2247.57 1769.31 Q2245.65 1767.57 2242.54 1767.57 Q2240.34 1767.57 2237.89 1768.34 Q2235.46 1769.1 2232.68 1770.65 L2232.68 1765.93 Q2235.51 1764.79 2237.96 1764.21 Q2240.41 1763.64 2242.45 1763.64 Q2247.82 1763.64 2251.02 1766.32 Q2254.21 1769.01 2254.21 1773.5 Q2254.21 1775.63 2253.4 1777.55 Q2252.61 1779.45 2250.51 1782.04 Q2249.93 1782.71 2246.83 1785.93 Q2243.72 1789.12 2238.08 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M2274.21 1767.34 Q2270.6 1767.34 2268.77 1770.9 Q2266.97 1774.45 2266.97 1781.58 Q2266.97 1788.68 2268.77 1792.25 Q2270.6 1795.79 2274.21 1795.79 Q2277.84 1795.79 2279.65 1792.25 Q2281.48 1788.68 2281.48 1781.58 Q2281.48 1774.45 2279.65 1770.9 Q2277.84 1767.34 2274.21 1767.34 M2274.21 1763.64 Q2280.02 1763.64 2283.08 1768.24 Q2286.15 1772.83 2286.15 1781.58 Q2286.15 1790.3 2283.08 1794.91 Q2280.02 1799.49 2274.21 1799.49 Q2268.4 1799.49 2265.32 1794.91 Q2262.27 1790.3 2262.27 1781.58 Q2262.27 1772.83 2265.32 1768.24 Q2268.4 1763.64 2274.21 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M2294.42 1764.26 L2312.78 1764.26 L2312.78 1768.2 L2298.7 1768.2 L2298.7 1776.67 Q2299.72 1776.32 2300.74 1776.16 Q2301.76 1775.97 2302.78 1775.97 Q2308.56 1775.97 2311.94 1779.15 Q2315.32 1782.32 2315.32 1787.73 Q2315.32 1793.31 2311.85 1796.41 Q2308.38 1799.49 2302.06 1799.49 Q2299.88 1799.49 2297.61 1799.12 Q2295.37 1798.75 2292.96 1798.01 L2292.96 1793.31 Q2295.04 1794.45 2297.27 1795 Q2299.49 1795.56 2301.96 1795.56 Q2305.97 1795.56 2308.31 1793.45 Q2310.65 1791.34 2310.65 1787.73 Q2310.65 1784.12 2308.31 1782.02 Q2305.97 1779.91 2301.96 1779.91 Q2300.09 1779.91 2298.21 1780.33 Q2296.36 1780.74 2294.42 1781.62 L2294.42 1764.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M2334.53 1767.34 Q2330.92 1767.34 2329.09 1770.9 Q2327.29 1774.45 2327.29 1781.58 Q2327.29 1788.68 2329.09 1792.25 Q2330.92 1795.79 2334.53 1795.79 Q2338.17 1795.79 2339.97 1792.25 Q2341.8 1788.68 2341.8 1781.58 Q2341.8 1774.45 2339.97 1770.9 Q2338.17 1767.34 2334.53 1767.34 M2334.53 1763.64 Q2340.34 1763.64 2343.4 1768.24 Q2346.48 1772.83 2346.48 1781.58 Q2346.48 1790.3 2343.4 1794.91 Q2340.34 1799.49 2334.53 1799.49 Q2328.72 1799.49 2325.65 1794.91 Q2322.59 1790.3 2322.59 1781.58 Q2322.59 1772.83 2325.65 1768.24 Q2328.72 1763.64 2334.53 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M2589.37 1794.89 L2605.69 1794.89 L2605.69 1798.82 L2583.74 1798.82 L2583.74 1794.89 Q2586.4 1792.13 2590.99 1787.5 Q2595.59 1782.85 2596.77 1781.51 Q2599.02 1778.98 2599.9 1777.25 Q2600.8 1775.49 2600.8 1773.8 Q2600.8 1771.04 2598.86 1769.31 Q2596.94 1767.57 2593.83 1767.57 Q2591.64 1767.57 2589.18 1768.34 Q2586.75 1769.1 2583.97 1770.65 L2583.97 1765.93 Q2586.8 1764.79 2589.25 1764.21 Q2591.71 1763.64 2593.74 1763.64 Q2599.11 1763.64 2602.31 1766.32 Q2605.5 1769.01 2605.5 1773.5 Q2605.5 1775.63 2604.69 1777.55 Q2603.9 1779.45 2601.8 1782.04 Q2601.22 1782.71 2598.12 1785.93 Q2595.02 1789.12 2589.37 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M2616.31 1794.89 L2623.95 1794.89 L2623.95 1768.52 L2615.64 1770.19 L2615.64 1765.93 L2623.9 1764.26 L2628.58 1764.26 L2628.58 1794.89 L2636.22 1794.89 L2636.22 1798.82 L2616.31 1798.82 L2616.31 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M2655.66 1767.34 Q2652.05 1767.34 2650.22 1770.9 Q2648.42 1774.45 2648.42 1781.58 Q2648.42 1788.68 2650.22 1792.25 Q2652.05 1795.79 2655.66 1795.79 Q2659.3 1795.79 2661.1 1792.25 Q2662.93 1788.68 2662.93 1781.58 Q2662.93 1774.45 2661.1 1770.9 Q2659.3 1767.34 2655.66 1767.34 M2655.66 1763.64 Q2661.47 1763.64 2664.53 1768.24 Q2667.61 1772.83 2667.61 1781.58 Q2667.61 1790.3 2664.53 1794.91 Q2661.47 1799.49 2655.66 1799.49 Q2649.85 1799.49 2646.77 1794.91 Q2643.72 1790.3 2643.72 1781.58 Q2643.72 1772.83 2646.77 1768.24 Q2649.85 1763.64 2655.66 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M2685.83 1767.34 Q2682.21 1767.34 2680.39 1770.9 Q2678.58 1774.45 2678.58 1781.58 Q2678.58 1788.68 2680.39 1792.25 Q2682.21 1795.79 2685.83 1795.79 Q2689.46 1795.79 2691.26 1792.25 Q2693.09 1788.68 2693.09 1781.58 Q2693.09 1774.45 2691.26 1770.9 Q2689.46 1767.34 2685.83 1767.34 M2685.83 1763.64 Q2691.64 1763.64 2694.69 1768.24 Q2697.77 1772.83 2697.77 1781.58 Q2697.77 1790.3 2694.69 1794.91 Q2691.64 1799.49 2685.83 1799.49 Q2680.01 1799.49 2676.94 1794.91 Q2673.88 1790.3 2673.88 1781.58 Q2673.88 1772.83 2676.94 1768.24 Q2680.01 1763.64 2685.83 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M1342.17 1835.78 L1349.08 1835.78 L1362.25 1855.32 L1375.34 1835.78 L1382.24 1835.78 L1365.44 1860.67 L1365.44 1883.3 L1358.98 1883.3 L1358.98 1860.67 L1342.17 1835.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M1410.09 1864.01 L1410.09 1866.88 L1383.16 1866.88 Q1383.55 1872.92 1386.79 1876.11 Q1390.07 1879.26 1395.9 1879.26 Q1399.27 1879.26 1402.42 1878.43 Q1405.6 1877.6 1408.72 1875.95 L1408.72 1881.49 Q1405.57 1882.82 1402.26 1883.52 Q1398.95 1884.22 1395.55 1884.22 Q1387.02 1884.22 1382.02 1879.26 Q1377.05 1874.29 1377.05 1865.83 Q1377.05 1857.07 1381.76 1851.95 Q1386.51 1846.79 1394.53 1846.79 Q1401.72 1846.79 1405.89 1851.44 Q1410.09 1856.06 1410.09 1864.01 M1404.24 1862.29 Q1404.17 1857.49 1401.53 1854.62 Q1398.92 1851.76 1394.59 1851.76 Q1389.69 1851.76 1386.73 1854.53 Q1383.8 1857.3 1383.36 1862.33 L1404.24 1862.29 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M1435.9 1865.38 Q1428.81 1865.38 1426.07 1867 Q1423.33 1868.63 1423.33 1872.54 Q1423.33 1875.66 1425.37 1877.51 Q1427.44 1879.32 1430.97 1879.32 Q1435.84 1879.32 1438.77 1875.88 Q1441.73 1872.42 1441.73 1866.69 L1441.73 1865.38 L1435.9 1865.38 M1447.59 1862.96 L1447.59 1883.3 L1441.73 1883.3 L1441.73 1877.89 Q1439.72 1881.14 1436.73 1882.7 Q1433.74 1884.22 1429.41 1884.22 Q1423.94 1884.22 1420.69 1881.17 Q1417.48 1878.08 1417.48 1872.92 Q1417.48 1866.91 1421.49 1863.85 Q1425.53 1860.8 1433.52 1860.8 L1441.73 1860.8 L1441.73 1860.23 Q1441.73 1856.18 1439.06 1853.99 Q1436.41 1851.76 1431.61 1851.76 Q1428.55 1851.76 1425.66 1852.49 Q1422.76 1853.22 1420.09 1854.69 L1420.09 1849.28 Q1423.3 1848.03 1426.32 1847.43 Q1429.35 1846.79 1432.21 1846.79 Q1439.95 1846.79 1443.77 1850.8 Q1447.59 1854.81 1447.59 1862.96 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M1480.31 1853.13 Q1479.32 1852.55 1478.14 1852.3 Q1477 1852.01 1475.59 1852.01 Q1470.63 1852.01 1467.96 1855.26 Q1465.31 1858.47 1465.31 1864.52 L1465.31 1883.3 L1459.43 1883.3 L1459.43 1847.65 L1465.31 1847.65 L1465.31 1853.19 Q1467.16 1849.94 1470.12 1848.39 Q1473.08 1846.79 1477.31 1846.79 Q1477.92 1846.79 1478.65 1846.89 Q1479.38 1846.95 1480.27 1847.11 L1480.31 1853.13 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><polyline clip-path="url(#clip412)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="181.719,1525.16 2640.76,1525.16 "/>
<polyline clip-path="url(#clip412)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="181.719,1302.48 2640.76,1302.48 "/>
<polyline clip-path="url(#clip412)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="181.719,1079.8 2640.76,1079.8 "/>
<polyline clip-path="url(#clip412)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="181.719,857.114 2640.76,857.114 "/>
<polyline clip-path="url(#clip412)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="181.719,634.431 2640.76,634.431 "/>
<polyline clip-path="url(#clip412)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="181.719,411.748 2640.76,411.748 "/>
<polyline clip-path="url(#clip412)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="181.719,189.065 2640.76,189.065 "/>
<polyline clip-path="url(#clip410)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="181.719,1734.12 181.719,47.2441 "/>
<polyline clip-path="url(#clip410)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="181.719,1525.16 200.617,1525.16 "/>
<polyline clip-path="url(#clip410)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="181.719,1302.48 200.617,1302.48 "/>
<polyline clip-path="url(#clip410)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="181.719,1079.8 200.617,1079.8 "/>
<polyline clip-path="url(#clip410)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="181.719,857.114 200.617,857.114 "/>
<polyline clip-path="url(#clip410)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="181.719,634.431 200.617,634.431 "/>
<polyline clip-path="url(#clip410)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="181.719,411.748 200.617,411.748 "/>
<polyline clip-path="url(#clip410)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="181.719,189.065 200.617,189.065 "/>
<path clip-path="url(#clip410)" d="M129.455 1510.96 Q125.844 1510.96 124.015 1514.53 Q122.21 1518.07 122.21 1525.2 Q122.21 1532.3 124.015 1535.87 Q125.844 1539.41 129.455 1539.41 Q133.089 1539.41 134.895 1535.87 Q136.724 1532.3 136.724 1525.2 Q136.724 1518.07 134.895 1514.53 Q133.089 1510.96 129.455 1510.96 M129.455 1507.26 Q135.265 1507.26 138.321 1511.86 Q141.399 1516.45 141.399 1525.2 Q141.399 1533.92 138.321 1538.53 Q135.265 1543.11 129.455 1543.11 Q123.645 1543.11 120.566 1538.53 Q117.511 1533.92 117.511 1525.2 Q117.511 1516.45 120.566 1511.86 Q123.645 1507.26 129.455 1507.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M121.492 1315.82 L129.131 1315.82 L129.131 1289.46 L120.821 1291.13 L120.821 1286.87 L129.085 1285.2 L133.761 1285.2 L133.761 1315.82 L141.399 1315.82 L141.399 1319.76 L121.492 1319.76 L121.492 1315.82 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M125.08 1093.14 L141.399 1093.14 L141.399 1097.08 L119.455 1097.08 L119.455 1093.14 Q122.117 1090.39 126.7 1085.76 Q131.307 1081.1 132.488 1079.76 Q134.733 1077.24 135.612 1075.5 Q136.515 1073.74 136.515 1072.05 Q136.515 1069.3 134.571 1067.56 Q132.65 1065.83 129.548 1065.83 Q127.349 1065.83 124.895 1066.59 Q122.464 1067.36 119.687 1068.91 L119.687 1064.18 Q122.511 1063.05 124.964 1062.47 Q127.418 1061.89 129.455 1061.89 Q134.825 1061.89 138.02 1064.58 Q141.214 1067.26 141.214 1071.75 Q141.214 1073.88 140.404 1075.8 Q139.617 1077.7 137.511 1080.29 Q136.932 1080.97 133.83 1084.18 Q130.728 1087.38 125.08 1093.14 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M134.27 855.76 Q137.626 856.478 139.501 858.746 Q141.399 861.015 141.399 864.348 Q141.399 869.464 137.881 872.265 Q134.362 875.065 127.881 875.065 Q125.705 875.065 123.39 874.626 Q121.099 874.209 118.645 873.352 L118.645 868.839 Q120.589 869.973 122.904 870.552 Q125.219 871.13 127.742 871.13 Q132.14 871.13 134.432 869.394 Q136.747 867.658 136.747 864.348 Q136.747 861.292 134.594 859.579 Q132.464 857.843 128.645 857.843 L124.617 857.843 L124.617 854.001 L128.83 854.001 Q132.279 854.001 134.108 852.635 Q135.937 851.246 135.937 848.654 Q135.937 845.992 134.038 844.579 Q132.163 843.144 128.645 843.144 Q126.724 843.144 124.525 843.561 Q122.326 843.978 119.687 844.857 L119.687 840.691 Q122.349 839.95 124.663 839.58 Q127.001 839.209 129.062 839.209 Q134.386 839.209 137.487 841.64 Q140.589 844.047 140.589 848.167 Q140.589 851.038 138.946 853.029 Q137.302 854.996 134.27 855.76 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M131.816 621.225 L120.011 639.674 L131.816 639.674 L131.816 621.225 M130.589 617.151 L136.469 617.151 L136.469 639.674 L141.399 639.674 L141.399 643.563 L136.469 643.563 L136.469 651.711 L131.816 651.711 L131.816 643.563 L116.214 643.563 L116.214 639.049 L130.589 617.151 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M120.497 394.468 L138.853 394.468 L138.853 398.403 L124.779 398.403 L124.779 406.876 Q125.798 406.528 126.816 406.366 Q127.835 406.181 128.853 406.181 Q134.64 406.181 138.02 409.353 Q141.399 412.524 141.399 417.94 Q141.399 423.519 137.927 426.621 Q134.455 429.7 128.136 429.7 Q125.96 429.7 123.691 429.329 Q121.446 428.959 119.039 428.218 L119.039 423.519 Q121.122 424.653 123.344 425.209 Q125.566 425.764 128.043 425.764 Q132.048 425.764 134.386 423.658 Q136.724 421.552 136.724 417.94 Q136.724 414.329 134.386 412.223 Q132.048 410.116 128.043 410.116 Q126.168 410.116 124.293 410.533 Q122.441 410.95 120.497 411.829 L120.497 394.468 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M129.872 187.202 Q126.724 187.202 124.872 189.355 Q123.043 191.508 123.043 195.258 Q123.043 198.984 124.872 201.16 Q126.724 203.313 129.872 203.313 Q133.02 203.313 134.849 201.16 Q136.7 198.984 136.7 195.258 Q136.7 191.508 134.849 189.355 Q133.02 187.202 129.872 187.202 M139.154 172.549 L139.154 176.809 Q137.395 175.975 135.589 175.535 Q133.807 175.096 132.048 175.096 Q127.418 175.096 124.964 178.221 Q122.534 181.346 122.187 187.665 Q123.552 185.651 125.613 184.586 Q127.673 183.498 130.15 183.498 Q135.358 183.498 138.367 186.67 Q141.399 189.818 141.399 195.258 Q141.399 200.582 138.251 203.799 Q135.103 207.017 129.872 207.017 Q123.876 207.017 120.705 202.433 Q117.534 197.827 117.534 189.1 Q117.534 180.906 121.423 176.045 Q125.312 171.16 131.863 171.16 Q133.622 171.16 135.404 171.508 Q137.21 171.855 139.154 172.549 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M54.859 1499.02 L42.0957 1499.02 L42.0957 1509.52 L36.8122 1509.52 L36.8122 1492.65 L57.2143 1492.65 Q59.856 1496.38 61.2247 1500.87 Q62.5615 1505.35 62.5615 1510.45 Q62.5615 1521.59 56.0684 1527.89 Q49.5436 1534.16 37.9262 1534.16 Q26.2769 1534.16 19.7839 1527.89 Q13.2591 1521.59 13.2591 1510.45 Q13.2591 1505.8 14.4049 1501.63 Q15.5507 1497.43 17.7787 1493.89 L24.6219 1493.89 Q21.5981 1497.46 20.0704 1501.47 Q18.5426 1505.48 18.5426 1509.9 Q18.5426 1518.63 23.4124 1523.02 Q28.2821 1527.38 37.9262 1527.38 Q47.5384 1527.38 52.4082 1523.02 Q57.2779 1518.63 57.2779 1509.9 Q57.2779 1506.5 56.705 1503.83 Q56.1003 1501.15 54.859 1499.02 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M12.1132 1481.16 L12.1132 1475.31 L61.6384 1475.31 L61.6384 1481.16 L12.1132 1481.16 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M30.0964 1449.24 Q30.0964 1453.95 33.7885 1456.69 Q37.4488 1459.42 43.8463 1459.42 Q50.2438 1459.42 53.9359 1456.72 Q57.5962 1453.98 57.5962 1449.24 Q57.5962 1444.56 53.9041 1441.82 Q50.212 1439.09 43.8463 1439.09 Q37.5124 1439.09 33.8203 1441.82 Q30.0964 1444.56 30.0964 1449.24 M25.1311 1449.24 Q25.1311 1441.6 30.0964 1437.24 Q35.0616 1432.88 43.8463 1432.88 Q52.5991 1432.88 57.5962 1437.24 Q62.5615 1441.6 62.5615 1449.24 Q62.5615 1456.91 57.5962 1461.27 Q52.5991 1465.6 43.8463 1465.6 Q35.0616 1465.6 30.0964 1461.27 Q25.1311 1456.91 25.1311 1449.24 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M43.8463 1397.58 Q37.3851 1397.58 33.7248 1400.26 Q30.0327 1402.9 30.0327 1407.54 Q30.0327 1412.19 33.7248 1414.86 Q37.3851 1417.51 43.8463 1417.51 Q50.3075 1417.51 53.9996 1414.86 Q57.6599 1412.19 57.6599 1407.54 Q57.6599 1402.9 53.9996 1400.26 Q50.3075 1397.58 43.8463 1397.58 M31.4013 1417.51 Q28.2185 1415.66 26.6907 1412.86 Q25.1311 1410.03 25.1311 1406.11 Q25.1311 1399.62 30.2873 1395.58 Q35.4436 1391.5 43.8463 1391.5 Q52.249 1391.5 57.4052 1395.58 Q62.5615 1399.62 62.5615 1406.11 Q62.5615 1410.03 61.0337 1412.86 Q59.4741 1415.66 56.2912 1417.51 L61.6384 1417.51 L61.6384 1423.39 L12.1132 1423.39 L12.1132 1417.51 L31.4013 1417.51 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M43.719 1365.59 Q43.719 1372.69 45.3422 1375.43 Q46.9655 1378.17 50.8804 1378.17 Q53.9996 1378.17 55.8457 1376.13 Q57.6599 1374.06 57.6599 1370.53 Q57.6599 1365.66 54.2224 1362.73 Q50.7531 1359.77 45.0239 1359.77 L43.719 1359.77 L43.719 1365.59 M41.3 1353.91 L61.6384 1353.91 L61.6384 1359.77 L56.2276 1359.77 Q59.4741 1361.77 61.0337 1364.77 Q62.5615 1367.76 62.5615 1372.09 Q62.5615 1377.56 59.5059 1380.81 Q56.4186 1384.02 51.2623 1384.02 Q45.2467 1384.02 42.1912 1380.01 Q39.1357 1375.97 39.1357 1367.98 L39.1357 1359.77 L38.5628 1359.77 Q34.5205 1359.77 32.3244 1362.44 Q30.0964 1365.08 30.0964 1369.89 Q30.0964 1372.95 30.8284 1375.84 Q31.5605 1378.74 33.0246 1381.41 L27.6137 1381.41 Q26.3724 1378.2 25.7677 1375.17 Q25.1311 1372.15 25.1311 1369.29 Q25.1311 1361.55 29.1415 1357.73 Q33.1519 1353.91 41.3 1353.91 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M12.1132 1341.85 L12.1132 1335.99 L61.6384 1335.99 L61.6384 1341.85 L12.1132 1341.85 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M14.1184 1302.76 L14.1184 1293.18 L46.4562 1281.06 L14.1184 1268.87 L14.1184 1259.29 L61.6384 1259.29 L61.6384 1265.56 L19.9112 1265.56 L52.5037 1277.81 L52.5037 1284.27 L19.9112 1296.53 L61.6384 1296.53 L61.6384 1302.76 L14.1184 1302.76 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M42.3504 1216.29 L45.2149 1216.29 L45.2149 1243.21 Q51.2623 1242.83 54.4452 1239.58 Q57.5962 1236.31 57.5962 1230.48 Q57.5962 1227.11 56.7687 1223.96 Q55.9411 1220.77 54.2861 1217.66 L59.8242 1217.66 Q61.161 1220.81 61.8612 1224.12 Q62.5615 1227.43 62.5615 1230.83 Q62.5615 1239.36 57.5962 1244.36 Q52.631 1249.32 44.1646 1249.32 Q35.4117 1249.32 30.2873 1244.61 Q25.1311 1239.87 25.1311 1231.85 Q25.1311 1224.66 29.7781 1220.49 Q34.3932 1216.29 42.3504 1216.29 M40.6316 1222.14 Q35.8255 1222.21 32.9609 1224.85 Q30.0964 1227.46 30.0964 1231.79 Q30.0964 1236.69 32.8654 1239.65 Q35.6345 1242.58 40.6634 1243.02 L40.6316 1222.14 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M43.719 1190.47 Q43.719 1197.57 45.3422 1200.31 Q46.9655 1203.05 50.8804 1203.05 Q53.9996 1203.05 55.8457 1201.01 Q57.6599 1198.94 57.6599 1195.41 Q57.6599 1190.54 54.2224 1187.61 Q50.7531 1184.65 45.0239 1184.65 L43.719 1184.65 L43.719 1190.47 M41.3 1178.79 L61.6384 1178.79 L61.6384 1184.65 L56.2276 1184.65 Q59.4741 1186.65 61.0337 1189.65 Q62.5615 1192.64 62.5615 1196.97 Q62.5615 1202.44 59.5059 1205.69 Q56.4186 1208.9 51.2623 1208.9 Q45.2467 1208.9 42.1912 1204.89 Q39.1357 1200.85 39.1357 1192.86 L39.1357 1184.65 L38.5628 1184.65 Q34.5205 1184.65 32.3244 1187.32 Q30.0964 1189.96 30.0964 1194.77 Q30.0964 1197.83 30.8284 1200.72 Q31.5605 1203.62 33.0246 1206.29 L27.6137 1206.29 Q26.3724 1203.08 25.7677 1200.05 Q25.1311 1197.03 25.1311 1194.17 Q25.1311 1186.43 29.1415 1182.61 Q33.1519 1178.79 41.3 1178.79 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M40.1224 1137.1 L61.6384 1137.1 L61.6384 1142.95 L40.3133 1142.95 Q35.2526 1142.95 32.7381 1144.93 Q30.2237 1146.9 30.2237 1150.85 Q30.2237 1155.59 33.2474 1158.33 Q36.2711 1161.06 41.491 1161.06 L61.6384 1161.06 L61.6384 1166.95 L25.9905 1166.95 L25.9905 1161.06 L31.5287 1161.06 Q28.314 1158.96 26.7225 1156.13 Q25.1311 1153.27 25.1311 1149.54 Q25.1311 1143.4 28.9505 1140.25 Q32.7381 1137.1 40.1224 1137.1 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M14.1184 1111.03 L14.1184 1070.83 L19.5293 1070.83 L19.5293 1087.7 L61.6384 1087.7 L61.6384 1094.16 L19.5293 1094.16 L19.5293 1111.03 L14.1184 1111.03 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M42.3504 1045.46 L45.2149 1045.46 L45.2149 1072.39 Q51.2623 1072.01 54.4452 1068.76 Q57.5962 1065.48 57.5962 1059.66 Q57.5962 1056.28 56.7687 1053.13 Q55.9411 1049.95 54.2861 1046.83 L59.8242 1046.83 Q61.161 1049.98 61.8612 1053.29 Q62.5615 1056.6 62.5615 1060.01 Q62.5615 1068.54 57.5962 1073.54 Q52.631 1078.5 44.1646 1078.5 Q35.4117 1078.5 30.2873 1073.79 Q25.1311 1069.05 25.1311 1061.03 Q25.1311 1053.83 29.7781 1049.66 Q34.3932 1045.46 42.3504 1045.46 M40.6316 1051.32 Q35.8255 1051.38 32.9609 1054.02 Q30.0964 1056.63 30.0964 1060.96 Q30.0964 1065.86 32.8654 1068.82 Q35.6345 1071.75 40.6634 1072.2 L40.6316 1051.32 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M32.8336 1008.1 Q28.8869 1005.9 27.009 1002.84 Q25.1311 999.789 25.1311 995.651 Q25.1311 990.081 29.046 987.057 Q32.9291 984.034 40.1224 984.034 L61.6384 984.034 L61.6384 989.922 L40.3133 989.922 Q35.1889 989.922 32.7063 991.736 Q30.2237 993.55 30.2237 997.274 Q30.2237 1001.83 33.2474 1004.47 Q36.2711 1007.11 41.491 1007.11 L61.6384 1007.11 L61.6384 1013 L40.3133 1013 Q35.1571 1013 32.7063 1014.81 Q30.2237 1016.63 30.2237 1020.41 Q30.2237 1024.9 33.2792 1027.54 Q36.3029 1030.19 41.491 1030.19 L61.6384 1030.19 L61.6384 1036.07 L25.9905 1036.07 L25.9905 1030.19 L31.5287 1030.19 Q28.2503 1028.18 26.6907 1025.38 Q25.1311 1022.58 25.1311 1018.73 Q25.1311 1014.84 27.1045 1012.14 Q29.0779 1009.4 32.8336 1008.1 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M56.2912 966.687 L75.1974 966.687 L75.1974 972.575 L25.9905 972.575 L25.9905 966.687 L31.4013 966.687 Q28.2185 964.841 26.6907 962.04 Q25.1311 959.207 25.1311 955.292 Q25.1311 948.799 30.2873 944.757 Q35.4436 940.683 43.8463 940.683 Q52.249 940.683 57.4052 944.757 Q62.5615 948.799 62.5615 955.292 Q62.5615 959.207 61.0337 962.04 Q59.4741 964.841 56.2912 966.687 M43.8463 946.762 Q37.3851 946.762 33.7248 949.436 Q30.0327 952.078 30.0327 956.725 Q30.0327 961.372 33.7248 964.045 Q37.3851 966.687 43.8463 966.687 Q50.3075 966.687 53.9996 964.045 Q57.6599 961.372 57.6599 956.725 Q57.6599 952.078 53.9996 949.436 Q50.3075 946.762 43.8463 946.762 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M42.3504 900.484 L45.2149 900.484 L45.2149 927.411 Q51.2623 927.029 54.4452 923.782 Q57.5962 920.504 57.5962 914.679 Q57.5962 911.305 56.7687 908.154 Q55.9411 904.972 54.2861 901.852 L59.8242 901.852 Q61.161 905.003 61.8612 908.314 Q62.5615 911.624 62.5615 915.029 Q62.5615 923.559 57.5962 928.556 Q52.631 933.522 44.1646 933.522 Q35.4117 933.522 30.2873 928.811 Q25.1311 924.069 25.1311 916.048 Q25.1311 908.855 29.7781 904.685 Q34.3932 900.484 42.3504 900.484 M40.6316 906.34 Q35.8255 906.404 32.9609 909.046 Q30.0964 911.656 30.0964 915.984 Q30.0964 920.886 32.8654 923.846 Q35.6345 926.774 40.6634 927.22 L40.6316 906.34 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M31.465 870.215 Q30.8921 871.201 30.6375 872.379 Q30.351 873.525 30.351 874.925 Q30.351 879.891 33.5975 882.564 Q36.8122 885.206 42.8596 885.206 L61.6384 885.206 L61.6384 891.094 L25.9905 891.094 L25.9905 885.206 L31.5287 885.206 Q28.2821 883.36 26.7225 880.4 Q25.1311 877.44 25.1311 873.207 Q25.1311 872.602 25.2266 871.87 Q25.2903 871.138 25.4494 870.247 L31.465 870.215 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M43.719 847.871 Q43.719 854.969 45.3422 857.706 Q46.9655 860.443 50.8804 860.443 Q53.9996 860.443 55.8457 858.406 Q57.6599 856.338 57.6599 852.805 Q57.6599 847.935 54.2224 845.007 Q50.7531 842.047 45.0239 842.047 L43.719 842.047 L43.719 847.871 M41.3 836.19 L61.6384 836.19 L61.6384 842.047 L56.2276 842.047 Q59.4741 844.052 61.0337 847.044 Q62.5615 850.036 62.5615 854.364 Q62.5615 859.839 59.5059 863.085 Q56.4186 866.3 51.2623 866.3 Q45.2467 866.3 42.1912 862.289 Q39.1357 858.247 39.1357 850.258 L39.1357 842.047 L38.5628 842.047 Q34.5205 842.047 32.3244 844.72 Q30.0964 847.362 30.0964 852.168 Q30.0964 855.224 30.8284 858.12 Q31.5605 861.016 33.0246 863.69 L27.6137 863.69 Q26.3724 860.475 25.7677 857.452 Q25.1311 854.428 25.1311 851.563 Q25.1311 843.829 29.1415 840.01 Q33.1519 836.19 41.3 836.19 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M15.869 818.334 L25.9905 818.334 L25.9905 806.271 L30.542 806.271 L30.542 818.334 L49.8937 818.334 Q54.2542 818.334 55.4955 817.157 Q56.7368 815.947 56.7368 812.287 L56.7368 806.271 L61.6384 806.271 L61.6384 812.287 Q61.6384 819.066 59.124 821.644 Q56.5777 824.223 49.8937 824.223 L30.542 824.223 L30.542 828.519 L25.9905 828.519 L25.9905 824.223 L15.869 824.223 L15.869 818.334 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M47.5702 799.174 L25.9905 799.174 L25.9905 793.317 L47.3474 793.317 Q52.4082 793.317 54.9545 791.344 Q57.4689 789.37 57.4689 785.424 Q57.4689 780.681 54.4452 777.944 Q51.4215 775.175 46.2016 775.175 L25.9905 775.175 L25.9905 769.318 L61.6384 769.318 L61.6384 775.175 L56.1639 775.175 Q59.4104 777.307 61.0019 780.14 Q62.5615 782.941 62.5615 786.665 Q62.5615 792.808 58.742 795.991 Q54.9226 799.174 47.5702 799.174 M25.1311 784.437 L25.1311 784.437 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M31.465 736.599 Q30.8921 737.585 30.6375 738.763 Q30.351 739.909 30.351 741.309 Q30.351 746.274 33.5975 748.948 Q36.8122 751.59 42.8596 751.59 L61.6384 751.59 L61.6384 757.478 L25.9905 757.478 L25.9905 751.59 L31.5287 751.59 Q28.2821 749.744 26.7225 746.784 Q25.1311 743.824 25.1311 739.59 Q25.1311 738.986 25.2266 738.254 Q25.2903 737.522 25.4494 736.63 L31.465 736.599 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M42.3504 701.396 L45.2149 701.396 L45.2149 728.323 Q51.2623 727.941 54.4452 724.695 Q57.5962 721.416 57.5962 715.592 Q57.5962 712.218 56.7687 709.067 Q55.9411 705.884 54.2861 702.765 L59.8242 702.765 Q61.161 705.916 61.8612 709.226 Q62.5615 712.536 62.5615 715.942 Q62.5615 724.472 57.5962 729.469 Q52.631 734.434 44.1646 734.434 Q35.4117 734.434 30.2873 729.724 Q25.1311 724.981 25.1311 716.96 Q25.1311 709.767 29.7781 705.598 Q34.3932 701.396 42.3504 701.396 M40.6316 707.253 Q35.8255 707.316 32.9609 709.958 Q30.0964 712.568 30.0964 716.897 Q30.0964 721.798 32.8654 724.758 Q35.6345 727.687 40.6634 728.132 L40.6316 707.253 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M20.4523 654.927 L44.1009 663.648 L44.1009 646.174 L20.4523 654.927 M14.1184 658.555 L14.1184 651.266 L61.6384 633.156 L61.6384 639.84 L49.4481 644.169 L49.4481 665.589 L61.6384 669.918 L61.6384 676.697 L14.1184 658.555 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M40.1224 596.84 L61.6384 596.84 L61.6384 602.696 L40.3133 602.696 Q35.2526 602.696 32.7381 604.669 Q30.2237 606.643 30.2237 610.589 Q30.2237 615.332 33.2474 618.069 Q36.2711 620.806 41.491 620.806 L61.6384 620.806 L61.6384 626.695 L25.9905 626.695 L25.9905 620.806 L31.5287 620.806 Q28.314 618.706 26.7225 615.873 Q25.1311 613.008 25.1311 609.284 Q25.1311 603.142 28.9505 599.991 Q32.7381 596.84 40.1224 596.84 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M30.0964 571.345 Q30.0964 576.055 33.7885 578.793 Q37.4488 581.53 43.8463 581.53 Q50.2438 581.53 53.9359 578.825 Q57.5962 576.087 57.5962 571.345 Q57.5962 566.666 53.9041 563.929 Q50.212 561.192 43.8463 561.192 Q37.5124 561.192 33.8203 563.929 Q30.0964 566.666 30.0964 571.345 M25.1311 571.345 Q25.1311 563.706 30.0964 559.346 Q35.0616 554.985 43.8463 554.985 Q52.5991 554.985 57.5962 559.346 Q62.5615 563.706 62.5615 571.345 Q62.5615 579.016 57.5962 583.376 Q52.5991 587.705 43.8463 587.705 Q35.0616 587.705 30.0964 583.376 Q25.1311 579.016 25.1311 571.345 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M32.8336 517.523 Q28.8869 515.327 27.009 512.271 Q25.1311 509.216 25.1311 505.078 Q25.1311 499.508 29.046 496.484 Q32.9291 493.46 40.1224 493.46 L61.6384 493.46 L61.6384 499.349 L40.3133 499.349 Q35.1889 499.349 32.7063 501.163 Q30.2237 502.977 30.2237 506.701 Q30.2237 511.253 33.2474 513.894 Q36.2711 516.536 41.491 516.536 L61.6384 516.536 L61.6384 522.424 L40.3133 522.424 Q35.1571 522.424 32.7063 524.239 Q30.2237 526.053 30.2237 529.84 Q30.2237 534.328 33.2792 536.97 Q36.3029 539.612 41.491 539.612 L61.6384 539.612 L61.6384 545.5 L25.9905 545.5 L25.9905 539.612 L31.5287 539.612 Q28.2503 537.607 26.6907 534.806 Q25.1311 532.005 25.1311 528.154 Q25.1311 524.27 27.1045 521.565 Q29.0779 518.828 32.8336 517.523 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M43.719 465.579 Q43.719 472.676 45.3422 475.414 Q46.9655 478.151 50.8804 478.151 Q53.9996 478.151 55.8457 476.114 Q57.6599 474.045 57.6599 470.512 Q57.6599 465.642 54.2224 462.714 Q50.7531 459.754 45.0239 459.754 L43.719 459.754 L43.719 465.579 M41.3 453.898 L61.6384 453.898 L61.6384 459.754 L56.2276 459.754 Q59.4741 461.759 61.0337 464.751 Q62.5615 467.743 62.5615 472.072 Q62.5615 477.546 59.5059 480.793 Q56.4186 484.007 51.2623 484.007 Q45.2467 484.007 42.1912 479.997 Q39.1357 475.955 39.1357 467.966 L39.1357 459.754 L38.5628 459.754 Q34.5205 459.754 32.3244 462.428 Q30.0964 465.069 30.0964 469.876 Q30.0964 472.931 30.8284 475.827 Q31.5605 478.724 33.0246 481.397 L27.6137 481.397 Q26.3724 478.183 25.7677 475.159 Q25.1311 472.135 25.1311 469.271 Q25.1311 461.536 29.1415 457.717 Q33.1519 453.898 41.3 453.898 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M12.1132 441.835 L12.1132 435.978 L61.6384 435.978 L61.6384 441.835 L12.1132 441.835 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M64.9486 408.892 Q71.3143 411.375 73.2559 413.73 Q75.1974 416.085 75.1974 420.032 L75.1974 424.711 L70.2958 424.711 L70.2958 421.273 Q70.2958 418.854 69.15 417.518 Q68.0041 416.181 63.7391 414.558 L61.0655 413.507 L25.9905 427.925 L25.9905 421.719 L53.8723 410.579 L25.9905 399.439 L25.9905 393.232 L64.9486 408.892 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M12.1769 350.359 Q19.4975 354.624 26.6589 356.693 Q33.8203 358.762 41.1727 358.762 Q48.5251 358.762 55.7502 356.693 Q62.9434 354.593 70.2321 350.359 L70.2321 355.452 Q62.7524 360.226 55.5274 362.613 Q48.3023 364.969 41.1727 364.969 Q34.0749 364.969 26.8817 362.613 Q19.6884 360.258 12.1769 355.452 L12.1769 350.359 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M17.365 328.843 Q17.365 331.39 19.1474 333.14 Q20.8979 334.891 23.4442 334.891 Q25.9587 334.891 27.7092 333.14 Q29.428 331.39 29.428 328.843 Q29.428 326.297 27.7092 324.546 Q25.9587 322.796 23.4442 322.796 Q20.9297 322.796 19.1474 324.578 Q17.365 326.329 17.365 328.843 M13.2591 328.843 Q13.2591 326.806 14.0548 324.928 Q14.8187 323.05 16.2828 321.682 Q17.7151 320.218 19.5293 319.486 Q21.3435 318.754 23.4442 318.754 Q27.6456 318.754 30.5738 321.714 Q33.4702 324.642 33.4702 328.907 Q33.4702 333.204 30.6056 336.068 Q27.7411 338.933 23.4442 338.933 Q19.1792 338.933 16.2191 336.005 Q13.2591 333.076 13.2591 328.843 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M17.7787 270.565 L24.5582 270.565 Q21.5345 273.812 20.0386 277.504 Q18.5426 281.164 18.5426 285.302 Q18.5426 293.45 23.5397 297.779 Q28.5049 302.107 37.9262 302.107 Q47.3156 302.107 52.3127 297.779 Q57.2779 293.45 57.2779 285.302 Q57.2779 281.164 55.782 277.504 Q54.2861 273.812 51.2623 270.565 L57.9782 270.565 Q60.2698 273.939 61.4156 277.727 Q62.5615 281.482 62.5615 285.684 Q62.5615 296.474 55.973 302.68 Q49.3526 308.887 37.9262 308.887 Q26.4679 308.887 19.8794 302.68 Q13.2591 296.474 13.2591 285.684 Q13.2591 281.419 14.4049 277.663 Q15.5189 273.875 17.7787 270.565 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip410)" d="M12.1769 261.812 L12.1769 256.72 Q19.6884 251.946 26.8817 249.59 Q34.0749 247.203 41.1727 247.203 Q48.3023 247.203 55.5274 249.59 Q62.7524 251.946 70.2321 256.72 L70.2321 261.812 Q62.9434 257.579 55.7502 255.51 Q48.5251 253.41 41.1727 253.41 Q33.8203 253.41 26.6589 255.51 Q19.4975 257.579 12.1769 261.812 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><polyline clip-path="url(#clip412)" style="stroke:#009af9; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="181.719,1525.16 188.745,1520.1 195.771,1516.19 202.797,1513.49 209.823,1512.04 216.849,1511.66 223.874,1512.19 230.9,1517.08 237.926,1518.35 244.952,1516.75 251.978,1513.92 259.003,1511.04 266.029,1508.65 273.055,1524.7 280.081,1532.93 287.107,1532.24 294.133,1529.64 301.158,1533.25 308.184,1536.92 315.21,1534.01 322.236,1528.16 329.262,1524.62 336.287,1520.63 343.313,1516.47 350.339,1513.33 357.365,1511.68 364.391,1510.79 371.417,1510.33 378.442,1509.95 385.468,1508.83 392.494,1506.37 399.52,1503.65 406.546,1501.3 413.571,1499.77 420.597,1534.18 427.623,1599.61 434.649,1611.61 441.675,1605.3 448.701,1592.31 455.726,1576.99 462.752,1561.48 469.778,1547.85 476.804,1536.62 483.83,1527.65 490.855,1520.9 497.881,1516.34 504.907,1513.2 511.933,1513.7 518.959,1516.62 525.985,1517.33 533.01,1516.17 540.036,1514.33 547.062,1511.93 554.088,1509.33 561.114,1506.7 568.14,1504.39 575.165,1502.8 582.191,1501.68 589.217,1501.19 596.243,1501.41 603.269,1571.17 610.294,1607.79 617.32,1606.29 624.346,1593.21 631.372,1578.31 638.398,1564.66 645.424,1621.24 652.449,1686.38 659.475,1684.62 666.501,1661.48 673.527,1635.22 680.553,1611.7 687.578,1593.91 694.604,1579.46 701.63,1574.13 708.656,1564.79 715.682,1553.97 722.708,1543.41 729.733,1534.07 736.759,1526.47 743.785,1520.26 750.811,1515.61 757.837,1559.59 764.862,1581.31 771.888,1578.3 778.914,1568.22 785.94,1591.3 792.966,1597.39 799.992,1585.84 807.017,1570.48 814.043,1556.18 821.069,1544.31 828.095,1535.08 835.121,1528.09 842.146,1522.78 849.172,1518.74 856.198,1515.61 863.224,1516.21 870.25,1515.48 877.276,1513.25 884.301,1510.47 891.327,1507.81 898.353,1505.43 905.379,1503.44 912.405,1505.23 919.43,1507.81 926.456,1507.41 933.482,1506.9 940.508,1509.68 947.534,1509.07 954.56,1506.89 961.585,1504.43 968.611,1503.52 975.637,1517.37 982.663,1521.46 989.689,1519.07 996.714,1515.32 1003.74,1511.5 1010.77,1508.33 1017.79,1505.6 1024.82,1502.79 1031.84,1500.54 1038.87,1498.26 1045.9,1496.36 1052.92,1499.19 1059.95,1499.39 1066.97,1499.24 1074,1499.07 1081.02,1498.21 1088.05,1497.3 1095.08,1496.51 1102.1,1496.25 1109.13,1495.67 1116.15,1494.38 1123.18,1503.18 1130.2,1539.03 1137.23,1545.62 1144.26,1540.23 1151.28,1534.85 1158.31,1527.55 1165.33,1520.36 1172.36,1516.72 1179.39,1516.31 1186.41,1511.8 1193.44,1505.58 1200.46,1500.13 1207.49,1495.88 1214.51,1492.75 1221.54,1490.63 1228.57,1489 1235.59,1488.03 1242.62,1487.26 1249.64,1486.66 1256.67,1488.01 1263.7,1504.32 1270.72,1508.41 1277.75,1505.4 1284.77,1500.44 1291.8,1499.13 1298.82,1496.31 1305.85,1492.89 1312.88,1490.43 1319.9,1488.33 1326.93,1495.11 1333.95,1501.89 1340.98,1500.34 1348.01,1495.03 1355.03,1489.57 1362.06,1484.68 1369.08,1480.41 1376.11,1476.71 1383.13,1474.47 1390.16,1471.97 1397.19,1472.12 1404.21,1472.41 1411.24,1470.73 1418.26,1468.89 1425.29,1466.73 1432.32,1464.76 1439.34,1463.82 1446.37,1466.51 1453.39,1465.09 1460.42,1463.11 1467.44,1464.08 1474.47,1462.88 1481.5,1461.44 1488.52,1459.79 1495.55,1457.51 1502.57,1455.51 1509.6,1453.26 1516.62,1451.78 1523.65,1451.04 1530.68,1450.68 1537.7,1450.86 1544.73,1451.19 1551.75,1451.21 1558.78,1449.7 1565.81,1447.79 1572.83,1445.78 1579.86,1444.07 1586.88,1442.75 1593.91,1443.08 1600.93,1443.75 1607.96,1444.37 1614.99,1445.15 1622.01,1445.52 1629.04,1445.56 1636.06,1444.96 1643.09,1443.71 1650.12,1441.76 1657.14,1440.78 1664.17,1440.23 1671.19,1441.98 1678.22,1446.49 1685.24,1465.18 1692.27,1477.78 1699.3,1481.47 1706.32,1475.47 1713.35,1473.44 1720.37,1469.41 1727.4,1463.98 1734.43,1459.25 1741.45,1457.57 1748.48,1454.59 1755.5,1449.26 1762.53,1445.34 1769.55,1447.24 1776.58,1444.32 1783.61,1438.69 1790.63,1431.94 1797.66,1425.03 1804.68,1419.18 1811.71,1413.64 1818.74,1415.07 1825.76,1423.25 1832.79,1422.39 1839.81,1416.82 1846.84,1411.36 1853.86,1405.01 1860.89,1396.82 1867.92,1387.11 1874.94,1378.99 1881.97,1386.26 1888.99,1410.58 1896.02,1413.75 1903.04,1405.15 1910.07,1393.01 1917.1,1380.99 1924.12,1372.96 1931.15,1363.93 1938.17,1352.23 1945.2,1340.92 1952.23,1331.36 1959.25,1324.3 1966.28,1319.58 1973.3,1315.14 1980.33,1312.25 1987.35,1310.11 1994.38,1307.56 2001.41,1303.65 2008.43,1299.88 2015.46,1295.49 2022.48,1290.57 2029.51,1285.74 2036.54,1279.34 2043.56,1274.35 2050.59,1268.41 2057.61,1262.78 2064.64,1256.89 2071.66,1250.71 2078.69,1244.23 2085.72,1237.39 2092.74,1230.07 2099.77,1222.26 2106.79,1214.37 2113.82,1206.65 2120.85,1199.37 2127.87,1192.49 2134.9,1185.92 2141.92,1179.26 2148.95,1172.66 2155.97,1165.91 2163,1159.2 2170.03,1152.27 2177.05,1144.75 2184.08,1136.38 2191.1,1127.96 2198.13,1119.2 2205.16,1110.82 2212.18,1102.39 2219.21,1094.35 2226.23,1086.57 2233.26,1078.93 2240.28,1071.08 2247.31,1062.89 2254.34,1054.2 2261.36,1044.61 2268.39,1034.5 2275.41,1023.88 2282.44,1013.31 2289.46,1003.25 2296.49,993.821 2303.52,984.661 2310.54,975.679 2317.57,966.749 2324.59,957.55 2331.62,947.798 2338.65,937.49 2345.67,927.21 2352.7,916.874 2359.72,906.505 2366.75,896.581 2373.77,886.92 2380.8,877.4 2387.83,867.797 2394.85,858.05 2401.88,848.013 2408.9,837.298 2415.93,826.078 2422.96,814.661 2429.98,803.461 2437.01,792.626 2444.03,782.155 2451.06,771.755 2458.08,761.367 2465.11,751.069 2472.14,740.715 2479.16,730.321 2486.19,719.681 2493.21,708.62 2500.24,697.193 2507.27,685.582 2514.29,674.06 2521.32,662.685 2528.34,651.726 2535.37,640.986 2542.39,630.748 2549.42,620.76 2556.45,610.789 2563.47,600.806 2570.5,590.546 2577.52,579.869 2584.55,569.024 2591.58,557.945 2598.6,547.43 2605.63,537.618 2612.65,528.31 2619.68,519.492 2626.7,510.828 2633.73,502.299 2640.76,493.585 2647.78,484.497 2654.81,475.119 2661.83,466.06 2668.86,457.282 2675.88,448.956 2682.91,441.221 2689.94,433.816 2696.96,426.662 2703.99,419.564 2711.01,412.387 2718.04,404.892 2725.07,396.718 2732.09,388.818 2739.12,381.154 2746.14,374.237 2753.17,367.606 2760.19,361.501 2767.22,355.647 2774.25,349.941 2781.27,344.231 2788.3,338.156 2795.32,331.585 2802.35,324.592 2809.38,318.278 2816.4,312.084 2823.43,306.612 2830.45,301.685 2837.48,296.991 2844.5,292.462 2851.53,287.846 2858.56,282.835 2865.58,276.902 2872.61,271.079 2879.63,265.332 2886.66,260.181 2893.69,255.701 2900.71,251.88 2907.74,248.2 2914.76,244.645 2921.79,241.114 2928.81,237.466 2935.84,233.399 2942.87,228.99 2949.89,224.676 2956.92,220.417 2963.94,216.378 2970.97,212.824 2978,209.36 2985.02,206.182 2992.05,203.141 2999.07,200.152 3006.1,197.149 3013.12,193.801 3020.15,189.855 3027.18,185.595 3034.2,181.634 3041.23,178.003 3048.25,174.881 3055.28,172.327 3062.31,170.166 3069.33,168.161 3076.36,166.065 3083.38,163.576 3090.41,160.423 3097.43,156.761 3104.46,153.593 3111.49,150.519 3118.51,148.296 3125.54,146.588 3132.56,145.176 3139.59,143.87 3146.61,142.518 3153.64,140.888 3160.67,138.634 3167.69,136.133 3174.72,133.478 3181.74,131.009 3188.77,129.032 3195.8,127.603 3202.82,126.45 3209.85,125.558 3216.87,124.767 3223.9,124.045 3230.92,123.291 3237.95,122.38 3244.98,121.15 3252,119.296 3259.03,117.481 3266.05,115.624 3273.08,113.793 3280.11,112.569 3287.13,111.791 3294.16,111.248 3301.18,110.737 3308.21,110.214 3315.23,109.484 3322.26,108.178 3329.29,106.381 3336.31,104.453 3343.34,102.917 3350.36,101.843 3357.39,101.368 3364.42,101.092 3371.44,100.946 3378.47,101.019 3385.49,101.106 3392.52,101.231 3399.54,101.153 3406.57,100.68 3413.6,99.8475 3420.62,98.827 3427.65,97.9569 3434.67,97.2561 3441.7,96.9634 3448.73,96.8752 3455.75,97.2408 3462.78,97.7884 3469.8,98.256 3476.83,98.5953 3483.85,98.5342 3490.88,97.9604 3497.91,97.109 3504.93,95.8558 3511.96,95.1156 3518.98,94.9858 3526.01,95.2983 3533.03,96.0624 3540.06,96.8573 3547.09,97.7547 3554.11,98.3438 3561.14,98.4007 3568.16,98.1934 3575.19,98.0245 3582.22,97.782 3589.24,97.9053 3596.27,98.3045 3603.29,99.1861 3610.32,100.19 3617.34,101.233 3624.37,102.169 3631.4,102.803 3638.42,102.842 3645.45,102.462 3652.47,102.214 3659.5,102.221 3666.53,102.769 3673.55,103.775 3680.58,104.921 3687.6,106.29 3694.63,107.738 3701.65,109.213 3708.68,110.631 3715.71,111.787 3722.73,112.509 3729.76,112.807 3736.78,113.139 3743.81,113.447 3750.84,113.929 3757.86,114.764 3764.89,115.793 3771.91,117.175 3778.94,118.627 3785.96,119.915 3792.99,120.999 3800.02,121.577 3807.04,121.571 3814.07,121.225 3821.09,120.625 3828.12,120.682 3835.15,121.168 3842.17,122.117 3849.2,123.257 3856.22,124.539 3863.25,125.672 3870.27,126.362 3877.3,126.545 3884.33,126.439 3891.35,126.437 3898.38,126.583 3905.4,127.025 3912.43,127.909 3919.45,128.936 3926.48,129.936 3933.51,130.868 3940.53,131.472 3947.56,131.479 3954.58,130.967 3961.61,130.423 3968.64,130.062 3975.66,130.494 3982.69,131.063 3989.71,131.947 3996.74,133.03 4003.76,134.144 4010.79,134.985 4017.82,135.222 4024.84,135.02 4031.87,134.427 4038.89,133.989 4045.92,134.066 4052.95,134.22 4059.97,134.405 4067,134.613 4074.02,134.841 4081.05,135.084 4088.07,135.34 4095.1,135.605 4102.13,135.878 4109.15,136.157 4116.18,136.442 4123.2,136.729 4130.23,137.018 4137.26,137.308 4144.28,137.599 4151.31,137.891 4158.33,138.184 4165.36,138.477 4172.38,138.77 4179.41,139.063 4186.44,139.356 4193.46,139.647 4200.49,139.938 4207.51,140.228 4214.54,140.517 4221.57,140.805 4228.59,141.094 4235.62,141.382 4242.64,141.668 4249.67,141.953 4256.69,142.237 4263.72,142.52 4270.75,142.801 4277.77,143.082 4284.8,143.364 4291.82,143.643 4298.85,143.921 4305.87,144.2 4312.9,144.479 4319.93,144.756 4326.95,145.032 4333.98,145.306 4341,145.58 4348.03,145.852 4355.06,146.123 4362.08,146.392 4369.11,146.66 4376.13,146.928 4383.16,147.195 4390.18,147.461 4397.21,147.726 4404.24,147.99 4411.26,148.254 4418.29,148.516 4425.31,148.778 4432.34,149.039 4439.37,149.299 4446.39,149.559 4453.42,149.818 4460.44,150.077 4467.47,150.335 4474.49,150.593 4481.52,150.849 4488.55,151.105 4495.57,151.361 4502.6,151.616 4509.62,151.87 4516.65,152.124 4523.68,152.378 4530.7,152.632 4537.73,152.884 4544.75,153.137 4551.78,153.388 4558.8,153.639 4565.83,153.89 4572.86,154.139 4579.88,154.389 4586.91,154.639 4593.93,154.888 4600.96,155.136 4607.99,155.384 4615.01,155.632 4622.04,155.88 4629.06,156.128 4636.09,156.375 4643.11,156.62 4650.14,156.866 4657.17,157.112 4664.19,157.358 4671.22,157.603 4678.24,157.847 4685.27,158.091 4692.29,158.336 4699.32,158.579 4706.35,158.822 4713.37,159.064 4720.4,159.307 4727.42,159.55 4734.45,159.792 4741.48,160.033 4748.5,160.274 4755.53,160.513 4762.55,160.754 4769.58,160.994 4776.6,161.232 4783.63,161.471 4790.66,161.71 4797.68,161.949 4804.71,162.188 4811.73,162.426 4818.76,162.664 4825.79,162.902 4832.81,163.139 4839.84,163.377 4846.86,163.613 4853.89,163.849 4860.91,164.086 4867.94,164.322 4874.97,164.557 4881.99,164.793 4889.02,165.028 4896.04,165.262 4903.07,165.497 4910.1,165.731 4917.12,165.965 4924.15,166.199 4931.17,166.432 4938.2,166.665 4945.22,166.897 4952.25,167.13 4959.28,167.363 4966.3,167.595 4973.33,167.826 4980.35,168.057 4987.38,168.288 4994.41,168.519 5001.43,168.749 5008.46,168.98 5015.48,169.211 5022.51,169.442 5029.53,169.672 5036.56,169.902 5043.59,170.132 5050.61,170.361 5057.64,170.591 5064.66,170.819 5071.69,171.048 5078.71,171.276 5085.74,171.504 5092.77,171.731 5099.79,171.958 5106.82,172.184 5113.84,172.411 5120.87,172.637 5127.9,172.863 5134.92,173.089 5141.95,173.314 5148.97,173.54 5156,173.766 5163.02,173.991 5170.05,174.216 5177.08,174.44 5184.1,174.664 5191.13,174.889 5198.15,175.113 5205.18,175.337 5212.21,175.561 5219.23,175.783 5226.26,176.006 5233.28,176.229 5240.31,176.452 5247.33,176.674 5254.36,176.896 5261.39,177.118 5268.41,177.34 5275.44,177.562 5282.46,177.784 5289.49,178.006 5296.52,178.227 5303.54,178.448 5310.57,178.669 5317.59,178.889 5324.62,179.108 5331.64,179.327 5338.67,179.547 5345.7,179.764 5352.72,179.983 5359.75,180.202 5366.77,180.42 5373.8,180.639 5380.83,180.858 5387.85,181.075 5394.88,181.292 5401.9,181.51 5408.93,181.728 5415.95,181.944 5422.98,182.161 5430.01,182.377 5437.03,182.594 5444.06,182.81 5451.08,183.025 "/>
</svg>

Figure 2: Global mean temperature anomalies (in °C, relative to 1750)
from 1750 to 2100 simulated using the energy balance model and RCP 8.5
forcings and parameters given in the problem statement.

1.  This syntax creates an empty plot object with the right axis labels,
    legend settings, etc; this is not necessary but can make the code
    more readable.
2.  `plot!` is different than `plot` because it adds elements to an
    existing plot object (in this case, `p`) rather than creating a new
    one. Since `p` is the last plot object created, we could have just
    written `plot!(t, temps)`, but adding `p` explicitly minimizes the
    chances of accidentally plotting onto the wrong axis.
3.  Similarly, `xlims!` changes the x limits for an existing plot object
    (analogously, there is also `ylims!`).

For the Monte Carlo analysis, we’ll start with 10,000 samples of
$\lambda$ and see how the simulation looks.

``` julia
n_samples = 10_000
λ_dist = LogNormal(log(2.1), log(2) / 4)
λ_samples = rand(λ_dist, n_samples)
h = histogram(λ_samples; xlabel="Climate Feedback Factor (°C/W/m²)", ylabel="Count", legend=false)
vline!(h, [2.1], color=:red, linestyle=:dash) # the original value for context
```

<?xml version="1.0" encoding="utf-8"?>
<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="672" height="480" viewBox="0 0 2688 1920">
<defs>
  <clipPath id="clip470">
    <rect x="0" y="0" width="2688" height="1920"/>
  </clipPath>
</defs>
<path clip-path="url(#clip470)" d="M0 1920 L2688 1920 L2688 -4.26326e-14 L0 -4.26326e-14  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<defs>
  <clipPath id="clip471">
    <rect x="537" y="0" width="1883" height="1883"/>
  </clipPath>
</defs>
<path clip-path="url(#clip470)" d="M241.557 1734.12 L2640.76 1734.12 L2640.76 47.2441 L241.557 47.2441  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<defs>
  <clipPath id="clip472">
    <rect x="241" y="47" width="2400" height="1688"/>
  </clipPath>
</defs>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="334.694,1734.12 334.694,47.2441 "/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="722.927,1734.12 722.927,47.2441 "/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="1111.16,1734.12 1111.16,47.2441 "/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="1499.39,1734.12 1499.39,47.2441 "/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="1887.62,1734.12 1887.62,47.2441 "/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="2275.86,1734.12 2275.86,47.2441 "/>
<polyline clip-path="url(#clip470)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="241.557,1734.12 2640.76,1734.12 "/>
<polyline clip-path="url(#clip470)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="334.694,1734.12 334.694,1715.22 "/>
<polyline clip-path="url(#clip470)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="722.927,1734.12 722.927,1715.22 "/>
<polyline clip-path="url(#clip470)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1111.16,1734.12 1111.16,1715.22 "/>
<polyline clip-path="url(#clip470)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1499.39,1734.12 1499.39,1715.22 "/>
<polyline clip-path="url(#clip470)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1887.62,1734.12 1887.62,1715.22 "/>
<polyline clip-path="url(#clip470)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2275.86,1734.12 2275.86,1715.22 "/>
<path clip-path="url(#clip470)" d="M301.847 1794.89 L309.486 1794.89 L309.486 1768.52 L301.176 1770.19 L301.176 1765.93 L309.44 1764.26 L314.116 1764.26 L314.116 1794.89 L321.754 1794.89 L321.754 1798.82 L301.847 1798.82 L301.847 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M331.199 1792.94 L336.083 1792.94 L336.083 1798.82 L331.199 1798.82 L331.199 1792.94 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M356.268 1767.34 Q352.657 1767.34 350.828 1770.9 Q349.023 1774.45 349.023 1781.58 Q349.023 1788.68 350.828 1792.25 Q352.657 1795.79 356.268 1795.79 Q359.902 1795.79 361.708 1792.25 Q363.537 1788.68 363.537 1781.58 Q363.537 1774.45 361.708 1770.9 Q359.902 1767.34 356.268 1767.34 M356.268 1763.64 Q362.078 1763.64 365.134 1768.24 Q368.212 1772.83 368.212 1781.58 Q368.212 1790.3 365.134 1794.91 Q362.078 1799.49 356.268 1799.49 Q350.458 1799.49 347.379 1794.91 Q344.324 1790.3 344.324 1781.58 Q344.324 1772.83 347.379 1768.24 Q350.458 1763.64 356.268 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M690.577 1794.89 L698.216 1794.89 L698.216 1768.52 L689.906 1770.19 L689.906 1765.93 L698.17 1764.26 L702.846 1764.26 L702.846 1794.89 L710.484 1794.89 L710.484 1798.82 L690.577 1798.82 L690.577 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M719.929 1792.94 L724.813 1792.94 L724.813 1798.82 L719.929 1798.82 L719.929 1792.94 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M735.045 1764.26 L753.401 1764.26 L753.401 1768.2 L739.327 1768.2 L739.327 1776.67 Q740.345 1776.32 741.364 1776.16 Q742.382 1775.97 743.401 1775.97 Q749.188 1775.97 752.568 1779.15 Q755.947 1782.32 755.947 1787.73 Q755.947 1793.31 752.475 1796.41 Q749.003 1799.49 742.683 1799.49 Q740.507 1799.49 738.239 1799.12 Q735.994 1798.75 733.586 1798.01 L733.586 1793.31 Q735.67 1794.45 737.892 1795 Q740.114 1795.56 742.591 1795.56 Q746.595 1795.56 748.933 1793.45 Q751.271 1791.34 751.271 1787.73 Q751.271 1784.12 748.933 1782.02 Q746.595 1779.91 742.591 1779.91 Q740.716 1779.91 738.841 1780.33 Q736.989 1780.74 735.045 1781.62 L735.045 1764.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M1082.4 1794.89 L1098.72 1794.89 L1098.72 1798.82 L1076.77 1798.82 L1076.77 1794.89 Q1079.43 1792.13 1084.02 1787.5 Q1088.62 1782.85 1089.8 1781.51 Q1092.05 1778.98 1092.93 1777.25 Q1093.83 1775.49 1093.83 1773.8 Q1093.83 1771.04 1091.89 1769.31 Q1089.97 1767.57 1086.87 1767.57 Q1084.67 1767.57 1082.21 1768.34 Q1079.78 1769.1 1077 1770.65 L1077 1765.93 Q1079.83 1764.79 1082.28 1764.21 Q1084.74 1763.64 1086.77 1763.64 Q1092.14 1763.64 1095.34 1766.32 Q1098.53 1769.01 1098.53 1773.5 Q1098.53 1775.63 1097.72 1777.55 Q1096.93 1779.45 1094.83 1782.04 Q1094.25 1782.71 1091.15 1785.93 Q1088.05 1789.12 1082.4 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M1108.53 1792.94 L1113.42 1792.94 L1113.42 1798.82 L1108.53 1798.82 L1108.53 1792.94 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M1133.6 1767.34 Q1129.99 1767.34 1128.16 1770.9 Q1126.36 1774.45 1126.36 1781.58 Q1126.36 1788.68 1128.16 1792.25 Q1129.99 1795.79 1133.6 1795.79 Q1137.24 1795.79 1139.04 1792.25 Q1140.87 1788.68 1140.87 1781.58 Q1140.87 1774.45 1139.04 1770.9 Q1137.24 1767.34 1133.6 1767.34 M1133.6 1763.64 Q1139.41 1763.64 1142.47 1768.24 Q1145.55 1772.83 1145.55 1781.58 Q1145.55 1790.3 1142.47 1794.91 Q1139.41 1799.49 1133.6 1799.49 Q1127.79 1799.49 1124.71 1794.91 Q1121.66 1790.3 1121.66 1781.58 Q1121.66 1772.83 1124.71 1768.24 Q1127.79 1763.64 1133.6 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M1471.13 1794.89 L1487.45 1794.89 L1487.45 1798.82 L1465.5 1798.82 L1465.5 1794.89 Q1468.16 1792.13 1472.75 1787.5 Q1477.35 1782.85 1478.54 1781.51 Q1480.78 1778.98 1481.66 1777.25 Q1482.56 1775.49 1482.56 1773.8 Q1482.56 1771.04 1480.62 1769.31 Q1478.7 1767.57 1475.6 1767.57 Q1473.4 1767.57 1470.94 1768.34 Q1468.51 1769.1 1465.73 1770.65 L1465.73 1765.93 Q1468.56 1764.79 1471.01 1764.21 Q1473.47 1763.64 1475.5 1763.64 Q1480.87 1763.64 1484.07 1766.32 Q1487.26 1769.01 1487.26 1773.5 Q1487.26 1775.63 1486.45 1777.55 Q1485.66 1779.45 1483.56 1782.04 Q1482.98 1782.71 1479.88 1785.93 Q1476.78 1789.12 1471.13 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M1497.26 1792.94 L1502.15 1792.94 L1502.15 1798.82 L1497.26 1798.82 L1497.26 1792.94 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M1512.38 1764.26 L1530.73 1764.26 L1530.73 1768.2 L1516.66 1768.2 L1516.66 1776.67 Q1517.68 1776.32 1518.7 1776.16 Q1519.72 1775.97 1520.73 1775.97 Q1526.52 1775.97 1529.9 1779.15 Q1533.28 1782.32 1533.28 1787.73 Q1533.28 1793.31 1529.81 1796.41 Q1526.34 1799.49 1520.02 1799.49 Q1517.84 1799.49 1515.57 1799.12 Q1513.33 1798.75 1510.92 1798.01 L1510.92 1793.31 Q1513 1794.45 1515.22 1795 Q1517.45 1795.56 1519.92 1795.56 Q1523.93 1795.56 1526.27 1793.45 Q1528.6 1791.34 1528.6 1787.73 Q1528.6 1784.12 1526.27 1782.02 Q1523.93 1779.91 1519.92 1779.91 Q1518.05 1779.91 1516.17 1780.33 Q1514.32 1780.74 1512.38 1781.62 L1512.38 1764.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M1868.93 1780.19 Q1872.29 1780.9 1874.16 1783.17 Q1876.06 1785.44 1876.06 1788.77 Q1876.06 1793.89 1872.54 1796.69 Q1869.02 1799.49 1862.54 1799.49 Q1860.37 1799.49 1858.05 1799.05 Q1855.76 1798.64 1853.31 1797.78 L1853.31 1793.27 Q1855.25 1794.4 1857.57 1794.98 Q1859.88 1795.56 1862.4 1795.56 Q1866.8 1795.56 1869.09 1793.82 Q1871.41 1792.08 1871.41 1788.77 Q1871.41 1785.72 1869.26 1784.01 Q1867.13 1782.27 1863.31 1782.27 L1859.28 1782.27 L1859.28 1778.43 L1863.49 1778.43 Q1866.94 1778.43 1868.77 1777.06 Q1870.6 1775.67 1870.6 1773.08 Q1870.6 1770.42 1868.7 1769.01 Q1866.83 1767.57 1863.31 1767.57 Q1861.39 1767.57 1859.19 1767.99 Q1856.99 1768.4 1854.35 1769.28 L1854.35 1765.12 Q1857.01 1764.38 1859.33 1764.01 Q1861.66 1763.64 1863.72 1763.64 Q1869.05 1763.64 1872.15 1766.07 Q1875.25 1768.47 1875.25 1772.59 Q1875.25 1775.46 1873.61 1777.46 Q1871.96 1779.42 1868.93 1780.19 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M1884.93 1792.94 L1889.81 1792.94 L1889.81 1798.82 L1884.93 1798.82 L1884.93 1792.94 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M1910 1767.34 Q1906.39 1767.34 1904.56 1770.9 Q1902.75 1774.45 1902.75 1781.58 Q1902.75 1788.68 1904.56 1792.25 Q1906.39 1795.79 1910 1795.79 Q1913.63 1795.79 1915.44 1792.25 Q1917.26 1788.68 1917.26 1781.58 Q1917.26 1774.45 1915.44 1770.9 Q1913.63 1767.34 1910 1767.34 M1910 1763.64 Q1915.81 1763.64 1918.86 1768.24 Q1921.94 1772.83 1921.94 1781.58 Q1921.94 1790.3 1918.86 1794.91 Q1915.81 1799.49 1910 1799.49 Q1904.19 1799.49 1901.11 1794.91 Q1898.05 1790.3 1898.05 1781.58 Q1898.05 1772.83 1901.11 1768.24 Q1904.19 1763.64 1910 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M2257.66 1780.19 Q2261.02 1780.9 2262.89 1783.17 Q2264.79 1785.44 2264.79 1788.77 Q2264.79 1793.89 2261.27 1796.69 Q2257.75 1799.49 2251.27 1799.49 Q2249.1 1799.49 2246.78 1799.05 Q2244.49 1798.64 2242.04 1797.78 L2242.04 1793.27 Q2243.98 1794.4 2246.3 1794.98 Q2248.61 1795.56 2251.13 1795.56 Q2255.53 1795.56 2257.82 1793.82 Q2260.14 1792.08 2260.14 1788.77 Q2260.14 1785.72 2257.99 1784.01 Q2255.86 1782.27 2252.04 1782.27 L2248.01 1782.27 L2248.01 1778.43 L2252.22 1778.43 Q2255.67 1778.43 2257.5 1777.06 Q2259.33 1775.67 2259.33 1773.08 Q2259.33 1770.42 2257.43 1769.01 Q2255.56 1767.57 2252.04 1767.57 Q2250.12 1767.57 2247.92 1767.99 Q2245.72 1768.4 2243.08 1769.28 L2243.08 1765.12 Q2245.74 1764.38 2248.06 1764.01 Q2250.39 1763.64 2252.45 1763.64 Q2257.78 1763.64 2260.88 1766.07 Q2263.98 1768.47 2263.98 1772.59 Q2263.98 1775.46 2262.34 1777.46 Q2260.69 1779.42 2257.66 1780.19 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M2273.66 1792.94 L2278.54 1792.94 L2278.54 1798.82 L2273.66 1798.82 L2273.66 1792.94 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M2288.77 1764.26 L2307.13 1764.26 L2307.13 1768.2 L2293.06 1768.2 L2293.06 1776.67 Q2294.07 1776.32 2295.09 1776.16 Q2296.11 1775.97 2297.13 1775.97 Q2302.92 1775.97 2306.3 1779.15 Q2309.68 1782.32 2309.68 1787.73 Q2309.68 1793.31 2306.2 1796.41 Q2302.73 1799.49 2296.41 1799.49 Q2294.24 1799.49 2291.97 1799.12 Q2289.72 1798.75 2287.31 1798.01 L2287.31 1793.31 Q2289.4 1794.45 2291.62 1795 Q2293.84 1795.56 2296.32 1795.56 Q2300.32 1795.56 2302.66 1793.45 Q2305 1791.34 2305 1787.73 Q2305 1784.12 2302.66 1782.02 Q2300.32 1779.91 2296.32 1779.91 Q2294.44 1779.91 2292.57 1780.33 Q2290.72 1780.74 2288.77 1781.62 L2288.77 1764.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M909.143 1839.44 L909.143 1846.22 Q905.896 1843.2 902.204 1841.7 Q898.544 1840.21 894.406 1840.21 Q886.258 1840.21 881.929 1845.2 Q877.601 1850.17 877.601 1859.59 Q877.601 1868.98 881.929 1873.98 Q886.258 1878.94 894.406 1878.94 Q898.544 1878.94 902.204 1877.44 Q905.896 1875.95 909.143 1872.92 L909.143 1879.64 Q905.769 1881.93 901.981 1883.08 Q898.225 1884.22 894.024 1884.22 Q883.234 1884.22 877.028 1877.64 Q870.821 1871.02 870.821 1859.59 Q870.821 1848.13 877.028 1841.54 Q883.234 1834.92 894.024 1834.92 Q898.289 1834.92 902.045 1836.07 Q905.832 1837.18 909.143 1839.44 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M918.818 1833.78 L924.675 1833.78 L924.675 1883.3 L918.818 1883.3 L918.818 1833.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M936.929 1847.65 L942.785 1847.65 L942.785 1883.3 L936.929 1883.3 L936.929 1847.65 M936.929 1833.78 L942.785 1833.78 L942.785 1841.19 L936.929 1841.19 L936.929 1833.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M982.794 1854.5 Q984.99 1850.55 988.046 1848.67 Q991.101 1846.79 995.239 1846.79 Q1000.81 1846.79 1003.83 1850.71 Q1006.86 1854.59 1006.86 1861.78 L1006.86 1883.3 L1000.97 1883.3 L1000.97 1861.98 Q1000.97 1856.85 999.154 1854.37 Q997.339 1851.89 993.616 1851.89 Q989.064 1851.89 986.422 1854.91 Q983.78 1857.93 983.78 1863.15 L983.78 1883.3 L977.892 1883.3 L977.892 1861.98 Q977.892 1856.82 976.078 1854.37 Q974.264 1851.89 970.476 1851.89 Q965.988 1851.89 963.347 1854.94 Q960.705 1857.97 960.705 1863.15 L960.705 1883.3 L954.817 1883.3 L954.817 1847.65 L960.705 1847.65 L960.705 1853.19 Q962.71 1849.91 965.511 1848.35 Q968.312 1846.79 972.163 1846.79 Q976.046 1846.79 978.752 1848.77 Q981.489 1850.74 982.794 1854.5 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M1034.74 1865.38 Q1027.64 1865.38 1024.9 1867 Q1022.17 1868.63 1022.17 1872.54 Q1022.17 1875.66 1024.2 1877.51 Q1026.27 1879.32 1029.8 1879.32 Q1034.67 1879.32 1037.6 1875.88 Q1040.56 1872.42 1040.56 1866.69 L1040.56 1865.38 L1034.74 1865.38 M1046.42 1862.96 L1046.42 1883.3 L1040.56 1883.3 L1040.56 1877.89 Q1038.56 1881.14 1035.57 1882.7 Q1032.57 1884.22 1028.24 1884.22 Q1022.77 1884.22 1019.52 1881.17 Q1016.31 1878.08 1016.31 1872.92 Q1016.31 1866.91 1020.32 1863.85 Q1024.36 1860.8 1032.35 1860.8 L1040.56 1860.8 L1040.56 1860.23 Q1040.56 1856.18 1037.89 1853.99 Q1035.25 1851.76 1030.44 1851.76 Q1027.39 1851.76 1024.49 1852.49 Q1021.59 1853.22 1018.92 1854.69 L1018.92 1849.28 Q1022.13 1848.03 1025.16 1847.43 Q1028.18 1846.79 1031.05 1846.79 Q1038.78 1846.79 1042.6 1850.8 Q1046.42 1854.81 1046.42 1862.96 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M1064.27 1837.53 L1064.27 1847.65 L1076.34 1847.65 L1076.34 1852.2 L1064.27 1852.2 L1064.27 1871.56 Q1064.27 1875.92 1065.45 1877.16 Q1066.66 1878.4 1070.32 1878.4 L1076.34 1878.4 L1076.34 1883.3 L1070.32 1883.3 Q1063.54 1883.3 1060.96 1880.79 Q1058.39 1878.24 1058.39 1871.56 L1058.39 1852.2 L1054.09 1852.2 L1054.09 1847.65 L1058.39 1847.65 L1058.39 1837.53 L1064.27 1837.53 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M1114.53 1864.01 L1114.53 1866.88 L1087.61 1866.88 Q1087.99 1872.92 1091.23 1876.11 Q1094.51 1879.26 1100.34 1879.26 Q1103.71 1879.26 1106.86 1878.43 Q1110.04 1877.6 1113.16 1875.95 L1113.16 1881.49 Q1110.01 1882.82 1106.7 1883.52 Q1103.39 1884.22 1099.99 1884.22 Q1091.46 1884.22 1086.46 1879.26 Q1081.49 1874.29 1081.49 1865.83 Q1081.49 1857.07 1086.2 1851.95 Q1090.95 1846.79 1098.97 1846.79 Q1106.16 1846.79 1110.33 1851.44 Q1114.53 1856.06 1114.53 1864.01 M1108.68 1862.29 Q1108.61 1857.49 1105.97 1854.62 Q1103.36 1851.76 1099.03 1851.76 Q1094.13 1851.76 1091.17 1854.53 Q1088.24 1857.3 1087.8 1862.33 L1108.68 1862.29 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M1145.12 1835.78 L1172.43 1835.78 L1172.43 1841.19 L1151.55 1841.19 L1151.55 1855.2 L1170.39 1855.2 L1170.39 1860.61 L1151.55 1860.61 L1151.55 1883.3 L1145.12 1883.3 L1145.12 1835.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M1209.29 1864.01 L1209.29 1866.88 L1182.36 1866.88 Q1182.74 1872.92 1185.99 1876.11 Q1189.27 1879.26 1195.09 1879.26 Q1198.46 1879.26 1201.61 1878.43 Q1204.8 1877.6 1207.92 1875.95 L1207.92 1881.49 Q1204.77 1882.82 1201.46 1883.52 Q1198.15 1884.22 1194.74 1884.22 Q1186.21 1884.22 1181.21 1879.26 Q1176.25 1874.29 1176.25 1865.83 Q1176.25 1857.07 1180.96 1851.95 Q1185.7 1846.79 1193.72 1846.79 Q1200.91 1846.79 1205.08 1851.44 Q1209.29 1856.06 1209.29 1864.01 M1203.43 1862.29 Q1203.37 1857.49 1200.72 1854.62 Q1198.11 1851.76 1193.79 1851.76 Q1188.88 1851.76 1185.92 1854.53 Q1183 1857.3 1182.55 1862.33 L1203.43 1862.29 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M1249.39 1864.01 L1249.39 1866.88 L1222.46 1866.88 Q1222.84 1872.92 1226.09 1876.11 Q1229.37 1879.26 1235.19 1879.26 Q1238.57 1879.26 1241.72 1878.43 Q1244.9 1877.6 1248.02 1875.95 L1248.02 1881.49 Q1244.87 1882.82 1241.56 1883.52 Q1238.25 1884.22 1234.84 1884.22 Q1226.31 1884.22 1221.32 1879.26 Q1216.35 1874.29 1216.35 1865.83 Q1216.35 1857.07 1221.06 1851.95 Q1225.8 1846.79 1233.83 1846.79 Q1241.02 1846.79 1245.19 1851.44 Q1249.39 1856.06 1249.39 1864.01 M1243.53 1862.29 Q1243.47 1857.49 1240.83 1854.62 Q1238.22 1851.76 1233.89 1851.76 Q1228.99 1851.76 1226.03 1854.53 Q1223.1 1857.3 1222.65 1862.33 L1243.53 1862.29 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M1282.46 1853.06 L1282.46 1833.78 L1288.32 1833.78 L1288.32 1883.3 L1282.46 1883.3 L1282.46 1877.95 Q1280.61 1881.14 1277.78 1882.7 Q1274.98 1884.22 1271.03 1884.22 Q1264.57 1884.22 1260.5 1879.07 Q1256.46 1873.91 1256.46 1865.51 Q1256.46 1857.11 1260.5 1851.95 Q1264.57 1846.79 1271.03 1846.79 Q1274.98 1846.79 1277.78 1848.35 Q1280.61 1849.88 1282.46 1853.06 M1262.5 1865.51 Q1262.5 1871.97 1265.14 1875.66 Q1267.82 1879.32 1272.47 1879.32 Q1277.11 1879.32 1279.79 1875.66 Q1282.46 1871.97 1282.46 1865.51 Q1282.46 1859.05 1279.79 1855.39 Q1277.11 1851.7 1272.47 1851.7 Q1267.82 1851.7 1265.14 1855.39 Q1262.5 1859.05 1262.5 1865.51 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M1325.97 1865.51 Q1325.97 1859.05 1323.3 1855.39 Q1320.65 1851.7 1316.01 1851.7 Q1311.36 1851.7 1308.69 1855.39 Q1306.04 1859.05 1306.04 1865.51 Q1306.04 1871.97 1308.69 1875.66 Q1311.36 1879.32 1316.01 1879.32 Q1320.65 1879.32 1323.3 1875.66 Q1325.97 1871.97 1325.97 1865.51 M1306.04 1853.06 Q1307.89 1849.88 1310.69 1848.35 Q1313.52 1846.79 1317.44 1846.79 Q1323.93 1846.79 1327.97 1851.95 Q1332.05 1857.11 1332.05 1865.51 Q1332.05 1873.91 1327.97 1879.07 Q1323.93 1884.22 1317.44 1884.22 Q1313.52 1884.22 1310.69 1882.7 Q1307.89 1881.14 1306.04 1877.95 L1306.04 1883.3 L1300.16 1883.3 L1300.16 1833.78 L1306.04 1833.78 L1306.04 1853.06 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M1357.96 1865.38 Q1350.86 1865.38 1348.12 1867 Q1345.38 1868.63 1345.38 1872.54 Q1345.38 1875.66 1347.42 1877.51 Q1349.49 1879.32 1353.02 1879.32 Q1357.89 1879.32 1360.82 1875.88 Q1363.78 1872.42 1363.78 1866.69 L1363.78 1865.38 L1357.96 1865.38 M1369.64 1862.96 L1369.64 1883.3 L1363.78 1883.3 L1363.78 1877.89 Q1361.78 1881.14 1358.78 1882.7 Q1355.79 1884.22 1351.46 1884.22 Q1345.99 1884.22 1342.74 1881.17 Q1339.53 1878.08 1339.53 1872.92 Q1339.53 1866.91 1343.54 1863.85 Q1347.58 1860.8 1355.57 1860.8 L1363.78 1860.8 L1363.78 1860.23 Q1363.78 1856.18 1361.11 1853.99 Q1358.47 1851.76 1353.66 1851.76 Q1350.6 1851.76 1347.71 1852.49 Q1344.81 1853.22 1342.14 1854.69 L1342.14 1849.28 Q1345.35 1848.03 1348.38 1847.43 Q1351.4 1846.79 1354.26 1846.79 Q1362 1846.79 1365.82 1850.8 Q1369.64 1854.81 1369.64 1862.96 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M1407.35 1849.02 L1407.35 1854.5 Q1404.87 1853.13 1402.36 1852.46 Q1399.87 1851.76 1397.33 1851.76 Q1391.63 1851.76 1388.48 1855.39 Q1385.33 1858.98 1385.33 1865.51 Q1385.33 1872.03 1388.48 1875.66 Q1391.63 1879.26 1397.33 1879.26 Q1399.87 1879.26 1402.36 1878.59 Q1404.87 1877.89 1407.35 1876.52 L1407.35 1881.93 Q1404.9 1883.08 1402.26 1883.65 Q1399.65 1884.22 1396.69 1884.22 Q1388.64 1884.22 1383.9 1879.16 Q1379.15 1874.1 1379.15 1865.51 Q1379.15 1856.79 1383.93 1851.79 Q1388.73 1846.79 1397.07 1846.79 Q1399.78 1846.79 1402.36 1847.37 Q1404.94 1847.91 1407.35 1849.02 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M1417.32 1833.78 L1423.21 1833.78 L1423.21 1863.03 L1440.68 1847.65 L1448.16 1847.65 L1429.25 1864.33 L1448.95 1883.3 L1441.32 1883.3 L1423.21 1865.89 L1423.21 1883.3 L1417.32 1883.3 L1417.32 1833.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M1476.26 1835.78 L1503.57 1835.78 L1503.57 1841.19 L1482.69 1841.19 L1482.69 1855.2 L1501.54 1855.2 L1501.54 1860.61 L1482.69 1860.61 L1482.69 1883.3 L1476.26 1883.3 L1476.26 1835.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M1523.72 1865.38 Q1516.62 1865.38 1513.88 1867 Q1511.15 1868.63 1511.15 1872.54 Q1511.15 1875.66 1513.18 1877.51 Q1515.25 1879.32 1518.79 1879.32 Q1523.66 1879.32 1526.58 1875.88 Q1529.54 1872.42 1529.54 1866.69 L1529.54 1865.38 L1523.72 1865.38 M1535.4 1862.96 L1535.4 1883.3 L1529.54 1883.3 L1529.54 1877.89 Q1527.54 1881.14 1524.55 1882.7 Q1521.56 1884.22 1517.23 1884.22 Q1511.75 1884.22 1508.51 1881.17 Q1505.29 1878.08 1505.29 1872.92 Q1505.29 1866.91 1509.3 1863.85 Q1513.34 1860.8 1521.33 1860.8 L1529.54 1860.8 L1529.54 1860.23 Q1529.54 1856.18 1526.87 1853.99 Q1524.23 1851.76 1519.42 1851.76 Q1516.37 1851.76 1513.47 1852.49 Q1510.57 1853.22 1507.9 1854.69 L1507.9 1849.28 Q1511.12 1848.03 1514.14 1847.43 Q1517.16 1846.79 1520.03 1846.79 Q1527.76 1846.79 1531.58 1850.8 Q1535.4 1854.81 1535.4 1862.96 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M1573.12 1849.02 L1573.12 1854.5 Q1570.63 1853.13 1568.12 1852.46 Q1565.64 1851.76 1563.09 1851.76 Q1557.39 1851.76 1554.24 1855.39 Q1551.09 1858.98 1551.09 1865.51 Q1551.09 1872.03 1554.24 1875.66 Q1557.39 1879.26 1563.09 1879.26 Q1565.64 1879.26 1568.12 1878.59 Q1570.63 1877.89 1573.12 1876.52 L1573.12 1881.93 Q1570.67 1883.08 1568.03 1883.65 Q1565.42 1884.22 1562.46 1884.22 Q1554.4 1884.22 1549.66 1879.16 Q1544.92 1874.1 1544.92 1865.51 Q1544.92 1856.79 1549.69 1851.79 Q1554.5 1846.79 1562.84 1846.79 Q1565.54 1846.79 1568.12 1847.37 Q1570.7 1847.91 1573.12 1849.02 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M1589.1 1837.53 L1589.1 1847.65 L1601.16 1847.65 L1601.16 1852.2 L1589.1 1852.2 L1589.1 1871.56 Q1589.1 1875.92 1590.27 1877.16 Q1591.48 1878.4 1595.14 1878.4 L1601.16 1878.4 L1601.16 1883.3 L1595.14 1883.3 Q1588.36 1883.3 1585.79 1880.79 Q1583.21 1878.24 1583.21 1871.56 L1583.21 1852.2 L1578.91 1852.2 L1578.91 1847.65 L1583.21 1847.65 L1583.21 1837.53 L1589.1 1837.53 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M1622.67 1851.76 Q1617.96 1851.76 1615.23 1855.45 Q1612.49 1859.11 1612.49 1865.51 Q1612.49 1871.91 1615.19 1875.6 Q1617.93 1879.26 1622.67 1879.26 Q1627.35 1879.26 1630.09 1875.57 Q1632.83 1871.87 1632.83 1865.51 Q1632.83 1859.17 1630.09 1855.48 Q1627.35 1851.76 1622.67 1851.76 M1622.67 1846.79 Q1630.31 1846.79 1634.67 1851.76 Q1639.03 1856.72 1639.03 1865.51 Q1639.03 1874.26 1634.67 1879.26 Q1630.31 1884.22 1622.67 1884.22 Q1615 1884.22 1610.64 1879.26 Q1606.31 1874.26 1606.31 1865.51 Q1606.31 1856.72 1610.64 1851.76 Q1615 1846.79 1622.67 1846.79 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M1669.4 1853.13 Q1668.41 1852.55 1667.23 1852.3 Q1666.09 1852.01 1664.69 1852.01 Q1659.72 1852.01 1657.05 1855.26 Q1654.41 1858.47 1654.41 1864.52 L1654.41 1883.3 L1648.52 1883.3 L1648.52 1847.65 L1654.41 1847.65 L1654.41 1853.19 Q1656.25 1849.94 1659.21 1848.39 Q1662.17 1846.79 1666.41 1846.79 Q1667.01 1846.79 1667.74 1846.89 Q1668.48 1846.95 1669.37 1847.11 L1669.4 1853.13 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M1710.33 1833.84 Q1706.07 1841.16 1704 1848.32 Q1701.93 1855.48 1701.93 1862.84 Q1701.93 1870.19 1704 1877.41 Q1706.1 1884.61 1710.33 1891.89 L1705.24 1891.89 Q1700.46 1884.41 1698.08 1877.19 Q1695.72 1869.96 1695.72 1862.84 Q1695.72 1855.74 1698.08 1848.54 Q1700.43 1841.35 1705.24 1833.84 L1710.33 1833.84 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M1731.85 1839.03 Q1729.3 1839.03 1727.55 1840.81 Q1725.8 1842.56 1725.8 1845.11 Q1725.8 1847.62 1727.55 1849.37 Q1729.3 1851.09 1731.85 1851.09 Q1734.39 1851.09 1736.14 1849.37 Q1737.89 1847.62 1737.89 1845.11 Q1737.89 1842.59 1736.11 1840.81 Q1734.36 1839.03 1731.85 1839.03 M1731.85 1834.92 Q1733.88 1834.92 1735.76 1835.72 Q1737.64 1836.48 1739.01 1837.95 Q1740.47 1839.38 1741.2 1841.19 Q1741.94 1843.01 1741.94 1845.11 Q1741.94 1849.31 1738.98 1852.24 Q1736.05 1855.13 1731.78 1855.13 Q1727.49 1855.13 1724.62 1852.27 Q1721.76 1849.4 1721.76 1845.11 Q1721.76 1840.84 1724.69 1837.88 Q1727.61 1834.92 1731.85 1834.92 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M1790.12 1839.44 L1790.12 1846.22 Q1786.88 1843.2 1783.19 1841.7 Q1779.53 1840.21 1775.39 1840.21 Q1767.24 1840.21 1762.91 1845.2 Q1758.58 1850.17 1758.58 1859.59 Q1758.58 1868.98 1762.91 1873.98 Q1767.24 1878.94 1775.39 1878.94 Q1779.53 1878.94 1783.19 1877.44 Q1786.88 1875.95 1790.12 1872.92 L1790.12 1879.64 Q1786.75 1881.93 1782.96 1883.08 Q1779.21 1884.22 1775.01 1884.22 Q1764.22 1884.22 1758.01 1877.64 Q1751.8 1871.02 1751.8 1859.59 Q1751.8 1848.13 1758.01 1841.54 Q1764.22 1834.92 1775.01 1834.92 Q1779.27 1834.92 1783.03 1836.07 Q1786.81 1837.18 1790.12 1839.44 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M1810.21 1835.78 L1815.62 1835.78 L1799.07 1889.35 L1793.66 1889.35 L1810.21 1835.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M1817.78 1835.78 L1824.28 1835.78 L1834.27 1875.95 L1844.23 1835.78 L1851.46 1835.78 L1861.45 1875.95 L1871.41 1835.78 L1877.94 1835.78 L1866 1883.3 L1857.92 1883.3 L1847.89 1842.05 L1837.77 1883.3 L1829.69 1883.3 L1817.78 1835.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M1896.62 1835.78 L1902.03 1835.78 L1885.48 1889.35 L1880.07 1889.35 L1896.62 1835.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M1935.93 1854.5 Q1938.13 1850.55 1941.18 1848.67 Q1944.24 1846.79 1948.38 1846.79 Q1953.95 1846.79 1956.97 1850.71 Q1959.99 1854.59 1959.99 1861.78 L1959.99 1883.3 L1954.11 1883.3 L1954.11 1861.98 Q1954.11 1856.85 1952.29 1854.37 Q1950.48 1851.89 1946.75 1851.89 Q1942.2 1851.89 1939.56 1854.91 Q1936.92 1857.93 1936.92 1863.15 L1936.92 1883.3 L1931.03 1883.3 L1931.03 1861.98 Q1931.03 1856.82 1929.22 1854.37 Q1927.4 1851.89 1923.61 1851.89 Q1919.13 1851.89 1916.48 1854.94 Q1913.84 1857.97 1913.84 1863.15 L1913.84 1883.3 L1907.95 1883.3 L1907.95 1847.65 L1913.84 1847.65 L1913.84 1853.19 Q1915.85 1849.91 1918.65 1848.35 Q1921.45 1846.79 1925.3 1846.79 Q1929.18 1846.79 1931.89 1848.77 Q1934.63 1850.74 1935.93 1854.5 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M1974.06 1858.41 L1987.56 1858.41 L1987.56 1862.04 L1968.52 1862.04 L1968.52 1858.54 Q1969.61 1857.55 1971.61 1855.77 Q1982.56 1846.06 1982.56 1843.07 Q1982.56 1840.97 1980.9 1839.7 Q1979.25 1838.39 1976.54 1838.39 Q1974.89 1838.39 1972.95 1838.96 Q1971.01 1839.5 1968.71 1840.62 L1968.71 1836.7 Q1971.17 1835.81 1973.27 1835.37 Q1975.4 1834.92 1977.21 1834.92 Q1981.83 1834.92 1984.6 1837.02 Q1987.37 1839.12 1987.37 1842.56 Q1987.37 1846.98 1976.83 1856.02 Q1975.05 1857.55 1974.06 1858.41 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M1996.88 1833.84 L2001.98 1833.84 Q2006.75 1841.35 2009.1 1848.54 Q2011.49 1855.74 2011.49 1862.84 Q2011.49 1869.96 2009.1 1877.19 Q2006.75 1884.41 2001.98 1891.89 L1996.88 1891.89 Q2001.12 1884.61 2003.18 1877.41 Q2005.29 1870.19 2005.29 1862.84 Q2005.29 1855.48 2003.18 1848.32 Q2001.12 1841.16 1996.88 1833.84 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="241.557,1734.12 2640.76,1734.12 "/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="241.557,1445.27 2640.76,1445.27 "/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="241.557,1156.42 2640.76,1156.42 "/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="241.557,867.573 2640.76,867.573 "/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="241.557,578.725 2640.76,578.725 "/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="241.557,289.876 2640.76,289.876 "/>
<polyline clip-path="url(#clip470)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="241.557,1734.12 241.557,47.2441 "/>
<polyline clip-path="url(#clip470)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="241.557,1734.12 260.455,1734.12 "/>
<polyline clip-path="url(#clip470)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="241.557,1445.27 260.455,1445.27 "/>
<polyline clip-path="url(#clip470)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="241.557,1156.42 260.455,1156.42 "/>
<polyline clip-path="url(#clip470)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="241.557,867.573 260.455,867.573 "/>
<polyline clip-path="url(#clip470)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="241.557,578.725 260.455,578.725 "/>
<polyline clip-path="url(#clip470)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="241.557,289.876 260.455,289.876 "/>
<path clip-path="url(#clip470)" d="M189.293 1719.92 Q185.682 1719.92 183.853 1723.48 Q182.047 1727.02 182.047 1734.15 Q182.047 1741.26 183.853 1744.82 Q185.682 1748.36 189.293 1748.36 Q192.927 1748.36 194.733 1744.82 Q196.561 1741.26 196.561 1734.15 Q196.561 1727.02 194.733 1723.48 Q192.927 1719.92 189.293 1719.92 M189.293 1716.21 Q195.103 1716.21 198.158 1720.82 Q201.237 1725.4 201.237 1734.15 Q201.237 1742.88 198.158 1747.48 Q195.103 1752.07 189.293 1752.07 Q183.483 1752.07 180.404 1747.48 Q177.348 1742.88 177.348 1734.15 Q177.348 1725.4 180.404 1720.82 Q183.483 1716.21 189.293 1716.21 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M119.779 1458.61 L127.418 1458.61 L127.418 1432.25 L119.108 1433.91 L119.108 1429.66 L127.372 1427.99 L132.048 1427.99 L132.048 1458.61 L139.687 1458.61 L139.687 1462.55 L119.779 1462.55 L119.779 1458.61 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M159.131 1431.07 Q155.52 1431.07 153.691 1434.63 Q151.886 1438.17 151.886 1445.3 Q151.886 1452.41 153.691 1455.97 Q155.52 1459.52 159.131 1459.52 Q162.765 1459.52 164.571 1455.97 Q166.399 1452.41 166.399 1445.3 Q166.399 1438.17 164.571 1434.63 Q162.765 1431.07 159.131 1431.07 M159.131 1427.36 Q164.941 1427.36 167.997 1431.97 Q171.075 1436.55 171.075 1445.3 Q171.075 1454.03 167.997 1458.64 Q164.941 1463.22 159.131 1463.22 Q153.321 1463.22 150.242 1458.64 Q147.186 1454.03 147.186 1445.3 Q147.186 1436.55 150.242 1431.97 Q153.321 1427.36 159.131 1427.36 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M189.293 1431.07 Q185.682 1431.07 183.853 1434.63 Q182.047 1438.17 182.047 1445.3 Q182.047 1452.41 183.853 1455.97 Q185.682 1459.52 189.293 1459.52 Q192.927 1459.52 194.733 1455.97 Q196.561 1452.41 196.561 1445.3 Q196.561 1438.17 194.733 1434.63 Q192.927 1431.07 189.293 1431.07 M189.293 1427.36 Q195.103 1427.36 198.158 1431.97 Q201.237 1436.55 201.237 1445.3 Q201.237 1454.03 198.158 1458.64 Q195.103 1463.22 189.293 1463.22 Q183.483 1463.22 180.404 1458.64 Q177.348 1454.03 177.348 1445.3 Q177.348 1436.55 180.404 1431.97 Q183.483 1427.36 189.293 1427.36 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M122.997 1169.77 L139.316 1169.77 L139.316 1173.7 L117.372 1173.7 L117.372 1169.77 Q120.034 1167.01 124.617 1162.38 Q129.224 1157.73 130.404 1156.39 Q132.65 1153.86 133.529 1152.13 Q134.432 1150.37 134.432 1148.68 Q134.432 1145.92 132.488 1144.19 Q130.566 1142.45 127.464 1142.45 Q125.265 1142.45 122.812 1143.21 Q120.381 1143.98 117.603 1145.53 L117.603 1140.81 Q120.427 1139.67 122.881 1139.09 Q125.335 1138.52 127.372 1138.52 Q132.742 1138.52 135.937 1141.2 Q139.131 1143.89 139.131 1148.38 Q139.131 1150.51 138.321 1152.43 Q137.534 1154.33 135.427 1156.92 Q134.849 1157.59 131.747 1160.81 Q128.645 1164 122.997 1169.77 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M159.131 1142.22 Q155.52 1142.22 153.691 1145.78 Q151.886 1149.33 151.886 1156.46 Q151.886 1163.56 153.691 1167.13 Q155.52 1170.67 159.131 1170.67 Q162.765 1170.67 164.571 1167.13 Q166.399 1163.56 166.399 1156.46 Q166.399 1149.33 164.571 1145.78 Q162.765 1142.22 159.131 1142.22 M159.131 1138.52 Q164.941 1138.52 167.997 1143.12 Q171.075 1147.71 171.075 1156.46 Q171.075 1165.18 167.997 1169.79 Q164.941 1174.37 159.131 1174.37 Q153.321 1174.37 150.242 1169.79 Q147.186 1165.18 147.186 1156.46 Q147.186 1147.71 150.242 1143.12 Q153.321 1138.52 159.131 1138.52 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M189.293 1142.22 Q185.682 1142.22 183.853 1145.78 Q182.047 1149.33 182.047 1156.46 Q182.047 1163.56 183.853 1167.13 Q185.682 1170.67 189.293 1170.67 Q192.927 1170.67 194.733 1167.13 Q196.561 1163.56 196.561 1156.46 Q196.561 1149.33 194.733 1145.78 Q192.927 1142.22 189.293 1142.22 M189.293 1138.52 Q195.103 1138.52 198.158 1143.12 Q201.237 1147.71 201.237 1156.46 Q201.237 1165.18 198.158 1169.79 Q195.103 1174.37 189.293 1174.37 Q183.483 1174.37 180.404 1169.79 Q177.348 1165.18 177.348 1156.46 Q177.348 1147.71 180.404 1143.12 Q183.483 1138.52 189.293 1138.52 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M133.136 866.219 Q136.492 866.936 138.367 869.205 Q140.265 871.473 140.265 874.806 Q140.265 879.922 136.747 882.723 Q133.228 885.524 126.747 885.524 Q124.571 885.524 122.256 885.084 Q119.964 884.668 117.511 883.811 L117.511 879.297 Q119.455 880.431 121.77 881.01 Q124.085 881.589 126.608 881.589 Q131.006 881.589 133.298 879.853 Q135.612 878.117 135.612 874.806 Q135.612 871.751 133.46 870.038 Q131.33 868.302 127.511 868.302 L123.483 868.302 L123.483 864.459 L127.696 864.459 Q131.145 864.459 132.974 863.094 Q134.802 861.705 134.802 859.112 Q134.802 856.45 132.904 855.038 Q131.029 853.603 127.511 853.603 Q125.589 853.603 123.39 854.02 Q121.191 854.436 118.552 855.316 L118.552 851.149 Q121.214 850.408 123.529 850.038 Q125.867 849.668 127.927 849.668 Q133.251 849.668 136.353 852.098 Q139.455 854.506 139.455 858.626 Q139.455 861.496 137.812 863.487 Q136.168 865.455 133.136 866.219 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M159.131 853.371 Q155.52 853.371 153.691 856.936 Q151.886 860.478 151.886 867.607 Q151.886 874.714 153.691 878.279 Q155.52 881.82 159.131 881.82 Q162.765 881.82 164.571 878.279 Q166.399 874.714 166.399 867.607 Q166.399 860.478 164.571 856.936 Q162.765 853.371 159.131 853.371 M159.131 849.668 Q164.941 849.668 167.997 854.274 Q171.075 858.857 171.075 867.607 Q171.075 876.334 167.997 880.941 Q164.941 885.524 159.131 885.524 Q153.321 885.524 150.242 880.941 Q147.186 876.334 147.186 867.607 Q147.186 858.857 150.242 854.274 Q153.321 849.668 159.131 849.668 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M189.293 853.371 Q185.682 853.371 183.853 856.936 Q182.047 860.478 182.047 867.607 Q182.047 874.714 183.853 878.279 Q185.682 881.82 189.293 881.82 Q192.927 881.82 194.733 878.279 Q196.561 874.714 196.561 867.607 Q196.561 860.478 194.733 856.936 Q192.927 853.371 189.293 853.371 M189.293 849.668 Q195.103 849.668 198.158 854.274 Q201.237 858.857 201.237 867.607 Q201.237 876.334 198.158 880.941 Q195.103 885.524 189.293 885.524 Q183.483 885.524 180.404 880.941 Q177.348 876.334 177.348 867.607 Q177.348 858.857 180.404 854.274 Q183.483 849.668 189.293 849.668 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M131.816 565.519 L120.011 583.968 L131.816 583.968 L131.816 565.519 M130.589 561.445 L136.469 561.445 L136.469 583.968 L141.399 583.968 L141.399 587.856 L136.469 587.856 L136.469 596.005 L131.816 596.005 L131.816 587.856 L116.214 587.856 L116.214 583.343 L130.589 561.445 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M159.131 564.523 Q155.52 564.523 153.691 568.088 Q151.886 571.63 151.886 578.759 Q151.886 585.866 153.691 589.431 Q155.52 592.972 159.131 592.972 Q162.765 592.972 164.571 589.431 Q166.399 585.866 166.399 578.759 Q166.399 571.63 164.571 568.088 Q162.765 564.523 159.131 564.523 M159.131 560.82 Q164.941 560.82 167.997 565.426 Q171.075 570.009 171.075 578.759 Q171.075 587.486 167.997 592.093 Q164.941 596.676 159.131 596.676 Q153.321 596.676 150.242 592.093 Q147.186 587.486 147.186 578.759 Q147.186 570.009 150.242 565.426 Q153.321 560.82 159.131 560.82 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M189.293 564.523 Q185.682 564.523 183.853 568.088 Q182.047 571.63 182.047 578.759 Q182.047 585.866 183.853 589.431 Q185.682 592.972 189.293 592.972 Q192.927 592.972 194.733 589.431 Q196.561 585.866 196.561 578.759 Q196.561 571.63 194.733 568.088 Q192.927 564.523 189.293 564.523 M189.293 560.82 Q195.103 560.82 198.158 565.426 Q201.237 570.009 201.237 578.759 Q201.237 587.486 198.158 592.093 Q195.103 596.676 189.293 596.676 Q183.483 596.676 180.404 592.093 Q177.348 587.486 177.348 578.759 Q177.348 570.009 180.404 565.426 Q183.483 560.82 189.293 560.82 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M119.015 272.596 L137.372 272.596 L137.372 276.532 L123.298 276.532 L123.298 285.004 Q124.316 284.657 125.335 284.495 Q126.353 284.309 127.372 284.309 Q133.159 284.309 136.538 287.481 Q139.918 290.652 139.918 296.069 Q139.918 301.647 136.446 304.749 Q132.974 307.828 126.654 307.828 Q124.478 307.828 122.21 307.457 Q119.964 307.087 117.557 306.346 L117.557 301.647 Q119.64 302.782 121.863 303.337 Q124.085 303.893 126.562 303.893 Q130.566 303.893 132.904 301.786 Q135.242 299.68 135.242 296.069 Q135.242 292.458 132.904 290.351 Q130.566 288.245 126.562 288.245 Q124.687 288.245 122.812 288.661 Q120.96 289.078 119.015 289.958 L119.015 272.596 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M159.131 275.675 Q155.52 275.675 153.691 279.24 Q151.886 282.782 151.886 289.911 Q151.886 297.018 153.691 300.582 Q155.52 304.124 159.131 304.124 Q162.765 304.124 164.571 300.582 Q166.399 297.018 166.399 289.911 Q166.399 282.782 164.571 279.24 Q162.765 275.675 159.131 275.675 M159.131 271.971 Q164.941 271.971 167.997 276.578 Q171.075 281.161 171.075 289.911 Q171.075 298.638 167.997 303.244 Q164.941 307.828 159.131 307.828 Q153.321 307.828 150.242 303.244 Q147.186 298.638 147.186 289.911 Q147.186 281.161 150.242 276.578 Q153.321 271.971 159.131 271.971 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M189.293 275.675 Q185.682 275.675 183.853 279.24 Q182.047 282.782 182.047 289.911 Q182.047 297.018 183.853 300.582 Q185.682 304.124 189.293 304.124 Q192.927 304.124 194.733 300.582 Q196.561 297.018 196.561 289.911 Q196.561 282.782 194.733 279.24 Q192.927 275.675 189.293 275.675 M189.293 271.971 Q195.103 271.971 198.158 276.578 Q201.237 281.161 201.237 289.911 Q201.237 298.638 198.158 303.244 Q195.103 307.828 189.293 307.828 Q183.483 307.828 180.404 303.244 Q177.348 298.638 177.348 289.911 Q177.348 281.161 180.404 276.578 Q183.483 271.971 189.293 271.971 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M17.7787 946.54 L24.5582 946.54 Q21.5345 949.786 20.0386 953.478 Q18.5426 957.139 18.5426 961.276 Q18.5426 969.424 23.5397 973.753 Q28.5049 978.082 37.9262 978.082 Q47.3156 978.082 52.3127 973.753 Q57.2779 969.424 57.2779 961.276 Q57.2779 957.139 55.782 953.478 Q54.2861 949.786 51.2623 946.54 L57.9782 946.54 Q60.2698 949.913 61.4156 953.701 Q62.5615 957.457 62.5615 961.658 Q62.5615 972.448 55.973 978.655 Q49.3526 984.861 37.9262 984.861 Q26.4679 984.861 19.8794 978.655 Q13.2591 972.448 13.2591 961.658 Q13.2591 957.393 14.4049 953.637 Q15.5189 949.85 17.7787 946.54 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M30.0964 923.05 Q30.0964 927.761 33.7885 930.498 Q37.4488 933.235 43.8463 933.235 Q50.2438 933.235 53.9359 930.53 Q57.5962 927.793 57.5962 923.05 Q57.5962 918.371 53.9041 915.634 Q50.212 912.897 43.8463 912.897 Q37.5124 912.897 33.8203 915.634 Q30.0964 918.371 30.0964 923.05 M25.1311 923.05 Q25.1311 915.411 30.0964 911.051 Q35.0616 906.69 43.8463 906.69 Q52.5991 906.69 57.5962 911.051 Q62.5615 915.411 62.5615 923.05 Q62.5615 930.721 57.5962 935.081 Q52.5991 939.41 43.8463 939.41 Q35.0616 939.41 30.0964 935.081 Q25.1311 930.721 25.1311 923.05 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M47.5702 897.587 L25.9905 897.587 L25.9905 891.731 L47.3474 891.731 Q52.4082 891.731 54.9545 889.758 Q57.4689 887.784 57.4689 883.837 Q57.4689 879.095 54.4452 876.358 Q51.4215 873.589 46.2016 873.589 L25.9905 873.589 L25.9905 867.732 L61.6384 867.732 L61.6384 873.589 L56.1639 873.589 Q59.4104 875.721 61.0019 878.554 Q62.5615 881.355 62.5615 885.079 Q62.5615 891.222 58.742 894.404 Q54.9226 897.587 47.5702 897.587 M25.1311 882.851 L25.1311 882.851 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M40.1224 826.037 L61.6384 826.037 L61.6384 831.893 L40.3133 831.893 Q35.2526 831.893 32.7381 833.867 Q30.2237 835.84 30.2237 839.787 Q30.2237 844.529 33.2474 847.266 Q36.2711 850.004 41.491 850.004 L61.6384 850.004 L61.6384 855.892 L25.9905 855.892 L25.9905 850.004 L31.5287 850.004 Q28.314 847.903 26.7225 845.07 Q25.1311 842.206 25.1311 838.482 Q25.1311 832.339 28.9505 829.188 Q32.7381 826.037 40.1224 826.037 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip470)" d="M15.869 808.563 L25.9905 808.563 L25.9905 796.5 L30.542 796.5 L30.542 808.563 L49.8937 808.563 Q54.2542 808.563 55.4955 807.385 Q56.7368 806.176 56.7368 802.516 L56.7368 796.5 L61.6384 796.5 L61.6384 802.516 Q61.6384 809.295 59.124 811.873 Q56.5777 814.451 49.8937 814.451 L30.542 814.451 L30.542 818.748 L25.9905 818.748 L25.9905 814.451 L15.869 814.451 L15.869 808.563 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip472)" d="M373.517 1728.34 L373.517 1734.12 L412.341 1734.12 L412.341 1728.34 L373.517 1728.34 L373.517 1728.34  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="373.517,1728.34 373.517,1734.12 412.341,1734.12 412.341,1728.34 373.517,1728.34 "/>
<path clip-path="url(#clip472)" d="M412.341 1728.34 L412.341 1734.12 L451.164 1734.12 L451.164 1728.34 L412.341 1728.34 L412.341 1728.34  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="412.341,1728.34 412.341,1734.12 451.164,1734.12 451.164,1728.34 412.341,1728.34 "/>
<path clip-path="url(#clip472)" d="M451.164 1719.67 L451.164 1734.12 L489.987 1734.12 L489.987 1719.67 L451.164 1719.67 L451.164 1719.67  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="451.164,1719.67 451.164,1734.12 489.987,1734.12 489.987,1719.67 451.164,1719.67 "/>
<path clip-path="url(#clip472)" d="M489.987 1719.67 L489.987 1734.12 L528.81 1734.12 L528.81 1719.67 L489.987 1719.67 L489.987 1719.67  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="489.987,1719.67 489.987,1734.12 528.81,1734.12 528.81,1719.67 489.987,1719.67 "/>
<path clip-path="url(#clip472)" d="M528.81 1702.34 L528.81 1734.12 L567.634 1734.12 L567.634 1702.34 L528.81 1702.34 L528.81 1702.34  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="528.81,1702.34 528.81,1734.12 567.634,1734.12 567.634,1702.34 528.81,1702.34 "/>
<path clip-path="url(#clip472)" d="M567.634 1653.24 L567.634 1734.12 L606.457 1734.12 L606.457 1653.24 L567.634 1653.24 L567.634 1653.24  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="567.634,1653.24 567.634,1734.12 606.457,1734.12 606.457,1653.24 567.634,1653.24 "/>
<path clip-path="url(#clip472)" d="M606.457 1595.47 L606.457 1734.12 L645.28 1734.12 L645.28 1595.47 L606.457 1595.47 L606.457 1595.47  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="606.457,1595.47 606.457,1734.12 645.28,1734.12 645.28,1595.47 606.457,1595.47 "/>
<path clip-path="url(#clip472)" d="M645.28 1581.03 L645.28 1734.12 L684.103 1734.12 L684.103 1581.03 L645.28 1581.03 L645.28 1581.03  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="645.28,1581.03 645.28,1734.12 684.103,1734.12 684.103,1581.03 645.28,1581.03 "/>
<path clip-path="url(#clip472)" d="M684.103 1442.38 L684.103 1734.12 L722.927 1734.12 L722.927 1442.38 L684.103 1442.38 L684.103 1442.38  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="684.103,1442.38 684.103,1734.12 722.927,1734.12 722.927,1442.38 684.103,1442.38 "/>
<path clip-path="url(#clip472)" d="M722.927 1329.73 L722.927 1734.12 L761.75 1734.12 L761.75 1329.73 L722.927 1329.73 L722.927 1329.73  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="722.927,1329.73 722.927,1734.12 761.75,1734.12 761.75,1329.73 722.927,1329.73 "/>
<path clip-path="url(#clip472)" d="M761.75 1231.52 L761.75 1734.12 L800.573 1734.12 L800.573 1231.52 L761.75 1231.52 L761.75 1231.52  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="761.75,1231.52 761.75,1734.12 800.573,1734.12 800.573,1231.52 761.75,1231.52 "/>
<path clip-path="url(#clip472)" d="M800.573 1023.55 L800.573 1734.12 L839.396 1734.12 L839.396 1023.55 L800.573 1023.55 L800.573 1023.55  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="800.573,1023.55 800.573,1734.12 839.396,1734.12 839.396,1023.55 800.573,1023.55 "/>
<path clip-path="url(#clip472)" d="M839.396 913.788 L839.396 1734.12 L878.219 1734.12 L878.219 913.788 L839.396 913.788 L839.396 913.788  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="839.396,913.788 839.396,1734.12 878.219,1734.12 878.219,913.788 839.396,913.788 "/>
<path clip-path="url(#clip472)" d="M878.219 688.487 L878.219 1734.12 L917.043 1734.12 L917.043 688.487 L878.219 688.487 L878.219 688.487  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="878.219,688.487 878.219,1734.12 917.043,1734.12 917.043,688.487 878.219,688.487 "/>
<path clip-path="url(#clip472)" d="M917.043 541.174 L917.043 1734.12 L955.866 1734.12 L955.866 541.174 L917.043 541.174 L917.043 541.174  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="917.043,541.174 917.043,1734.12 955.866,1734.12 955.866,541.174 917.043,541.174 "/>
<path clip-path="url(#clip472)" d="M955.866 549.84 L955.866 1734.12 L994.689 1734.12 L994.689 549.84 L955.866 549.84 L955.866 549.84  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="955.866,549.84 955.866,1734.12 994.689,1734.12 994.689,549.84 955.866,549.84 "/>
<path clip-path="url(#clip472)" d="M994.689 263.88 L994.689 1734.12 L1033.51 1734.12 L1033.51 263.88 L994.689 263.88 L994.689 263.88  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="994.689,263.88 994.689,1734.12 1033.51,1734.12 1033.51,263.88 994.689,263.88 "/>
<path clip-path="url(#clip472)" d="M1033.51 90.5713 L1033.51 1734.12 L1072.34 1734.12 L1072.34 90.5713 L1033.51 90.5713 L1033.51 90.5713  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1033.51,90.5713 1033.51,1734.12 1072.34,1734.12 1072.34,90.5713 1033.51,90.5713 "/>
<path clip-path="url(#clip472)" d="M1072.34 159.895 L1072.34 1734.12 L1111.16 1734.12 L1111.16 159.895 L1072.34 159.895 L1072.34 159.895  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1072.34,159.895 1072.34,1734.12 1111.16,1734.12 1111.16,159.895 1072.34,159.895 "/>
<path clip-path="url(#clip472)" d="M1111.16 269.657 L1111.16 1734.12 L1149.98 1734.12 L1149.98 269.657 L1111.16 269.657 L1111.16 269.657  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1111.16,269.657 1111.16,1734.12 1149.98,1734.12 1149.98,269.657 1111.16,269.657 "/>
<path clip-path="url(#clip472)" d="M1149.98 47.2441 L1149.98 1734.12 L1188.81 1734.12 L1188.81 47.2441 L1149.98 47.2441 L1149.98 47.2441  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1149.98,47.2441 1149.98,1734.12 1188.81,1734.12 1188.81,47.2441 1149.98,47.2441 "/>
<path clip-path="url(#clip472)" d="M1188.81 226.33 L1188.81 1734.12 L1227.63 1734.12 L1227.63 226.33 L1188.81 226.33 L1188.81 226.33  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1188.81,226.33 1188.81,1734.12 1227.63,1734.12 1227.63,226.33 1188.81,226.33 "/>
<path clip-path="url(#clip472)" d="M1227.63 260.992 L1227.63 1734.12 L1266.45 1734.12 L1266.45 260.992 L1227.63 260.992 L1227.63 260.992  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1227.63,260.992 1227.63,1734.12 1266.45,1734.12 1266.45,260.992 1227.63,260.992 "/>
<path clip-path="url(#clip472)" d="M1266.45 252.326 L1266.45 1734.12 L1305.28 1734.12 L1305.28 252.326 L1266.45 252.326 L1266.45 252.326  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1266.45,252.326 1266.45,1734.12 1305.28,1734.12 1305.28,252.326 1266.45,252.326 "/>
<path clip-path="url(#clip472)" d="M1305.28 454.52 L1305.28 1734.12 L1344.1 1734.12 L1344.1 454.52 L1305.28 454.52 L1305.28 454.52  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1305.28,454.52 1305.28,1734.12 1344.1,1734.12 1344.1,454.52 1305.28,454.52 "/>
<path clip-path="url(#clip472)" d="M1344.1 587.39 L1344.1 1734.12 L1382.92 1734.12 L1382.92 587.39 L1344.1 587.39 L1344.1 587.39  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1344.1,587.39 1344.1,1734.12 1382.92,1734.12 1382.92,587.39 1344.1,587.39 "/>
<path clip-path="url(#clip472)" d="M1382.92 691.375 L1382.92 1734.12 L1421.74 1734.12 L1421.74 691.375 L1382.92 691.375 L1382.92 691.375  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1382.92,691.375 1382.92,1734.12 1421.74,1734.12 1421.74,691.375 1382.92,691.375 "/>
<path clip-path="url(#clip472)" d="M1421.74 792.472 L1421.74 1734.12 L1460.57 1734.12 L1460.57 792.472 L1421.74 792.472 L1421.74 792.472  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1421.74,792.472 1421.74,1734.12 1460.57,1734.12 1460.57,792.472 1421.74,792.472 "/>
<path clip-path="url(#clip472)" d="M1460.57 893.569 L1460.57 1734.12 L1499.39 1734.12 L1499.39 893.569 L1460.57 893.569 L1460.57 893.569  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1460.57,893.569 1460.57,1734.12 1499.39,1734.12 1499.39,893.569 1460.57,893.569 "/>
<path clip-path="url(#clip472)" d="M1499.39 1009.11 L1499.39 1734.12 L1538.21 1734.12 L1538.21 1009.11 L1499.39 1009.11 L1499.39 1009.11  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1499.39,1009.11 1499.39,1734.12 1538.21,1734.12 1538.21,1009.11 1499.39,1009.11 "/>
<path clip-path="url(#clip472)" d="M1538.21 1043.77 L1538.21 1734.12 L1577.04 1734.12 L1577.04 1043.77 L1538.21 1043.77 L1538.21 1043.77  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1538.21,1043.77 1538.21,1734.12 1577.04,1734.12 1577.04,1043.77 1538.21,1043.77 "/>
<path clip-path="url(#clip472)" d="M1577.04 1130.42 L1577.04 1734.12 L1615.86 1734.12 L1615.86 1130.42 L1577.04 1130.42 L1577.04 1130.42  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1577.04,1130.42 1577.04,1734.12 1615.86,1734.12 1615.86,1130.42 1577.04,1130.42 "/>
<path clip-path="url(#clip472)" d="M1615.86 1257.52 L1615.86 1734.12 L1654.68 1734.12 L1654.68 1257.52 L1615.86 1257.52 L1615.86 1257.52  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1615.86,1257.52 1615.86,1734.12 1654.68,1734.12 1654.68,1257.52 1615.86,1257.52 "/>
<path clip-path="url(#clip472)" d="M1654.68 1344.17 L1654.68 1734.12 L1693.51 1734.12 L1693.51 1344.17 L1654.68 1344.17 L1654.68 1344.17  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1654.68,1344.17 1654.68,1734.12 1693.51,1734.12 1693.51,1344.17 1654.68,1344.17 "/>
<path clip-path="url(#clip472)" d="M1693.51 1381.72 L1693.51 1734.12 L1732.33 1734.12 L1732.33 1381.72 L1693.51 1381.72 L1693.51 1381.72  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1693.51,1381.72 1693.51,1734.12 1732.33,1734.12 1732.33,1381.72 1693.51,1381.72 "/>
<path clip-path="url(#clip472)" d="M1732.33 1439.49 L1732.33 1734.12 L1771.15 1734.12 L1771.15 1439.49 L1732.33 1439.49 L1732.33 1439.49  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1732.33,1439.49 1732.33,1734.12 1771.15,1734.12 1771.15,1439.49 1732.33,1439.49 "/>
<path clip-path="url(#clip472)" d="M1771.15 1436.6 L1771.15 1734.12 L1809.98 1734.12 L1809.98 1436.6 L1771.15 1436.6 L1771.15 1436.6  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1771.15,1436.6 1771.15,1734.12 1809.98,1734.12 1809.98,1436.6 1771.15,1436.6 "/>
<path clip-path="url(#clip472)" d="M1809.98 1557.92 L1809.98 1734.12 L1848.8 1734.12 L1848.8 1557.92 L1809.98 1557.92 L1809.98 1557.92  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1809.98,1557.92 1809.98,1734.12 1848.8,1734.12 1848.8,1557.92 1809.98,1557.92 "/>
<path clip-path="url(#clip472)" d="M1848.8 1566.59 L1848.8 1734.12 L1887.62 1734.12 L1887.62 1566.59 L1848.8 1566.59 L1848.8 1566.59  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1848.8,1566.59 1848.8,1734.12 1887.62,1734.12 1887.62,1566.59 1848.8,1566.59 "/>
<path clip-path="url(#clip472)" d="M1887.62 1581.03 L1887.62 1734.12 L1926.45 1734.12 L1926.45 1581.03 L1887.62 1581.03 L1887.62 1581.03  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1887.62,1581.03 1887.62,1734.12 1926.45,1734.12 1926.45,1581.03 1887.62,1581.03 "/>
<path clip-path="url(#clip472)" d="M1926.45 1644.57 L1926.45 1734.12 L1965.27 1734.12 L1965.27 1644.57 L1926.45 1644.57 L1926.45 1644.57  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1926.45,1644.57 1926.45,1734.12 1965.27,1734.12 1965.27,1644.57 1926.45,1644.57 "/>
<path clip-path="url(#clip472)" d="M1965.27 1673.46 L1965.27 1734.12 L2004.09 1734.12 L2004.09 1673.46 L1965.27 1673.46 L1965.27 1673.46  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1965.27,1673.46 1965.27,1734.12 2004.09,1734.12 2004.09,1673.46 1965.27,1673.46 "/>
<path clip-path="url(#clip472)" d="M2004.09 1670.57 L2004.09 1734.12 L2042.92 1734.12 L2042.92 1670.57 L2004.09 1670.57 L2004.09 1670.57  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2004.09,1670.57 2004.09,1734.12 2042.92,1734.12 2042.92,1670.57 2004.09,1670.57 "/>
<path clip-path="url(#clip472)" d="M2042.92 1661.9 L2042.92 1734.12 L2081.74 1734.12 L2081.74 1661.9 L2042.92 1661.9 L2042.92 1661.9  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2042.92,1661.9 2042.92,1734.12 2081.74,1734.12 2081.74,1661.9 2042.92,1661.9 "/>
<path clip-path="url(#clip472)" d="M2081.74 1699.46 L2081.74 1734.12 L2120.56 1734.12 L2120.56 1699.46 L2081.74 1699.46 L2081.74 1699.46  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2081.74,1699.46 2081.74,1734.12 2120.56,1734.12 2120.56,1699.46 2081.74,1699.46 "/>
<path clip-path="url(#clip472)" d="M2120.56 1711.01 L2120.56 1734.12 L2159.39 1734.12 L2159.39 1711.01 L2120.56 1711.01 L2120.56 1711.01  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2120.56,1711.01 2120.56,1734.12 2159.39,1734.12 2159.39,1711.01 2120.56,1711.01 "/>
<path clip-path="url(#clip472)" d="M2159.39 1716.79 L2159.39 1734.12 L2198.21 1734.12 L2198.21 1716.79 L2159.39 1716.79 L2159.39 1716.79  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2159.39,1716.79 2159.39,1734.12 2198.21,1734.12 2198.21,1716.79 2159.39,1716.79 "/>
<path clip-path="url(#clip472)" d="M2198.21 1731.23 L2198.21 1734.12 L2237.03 1734.12 L2237.03 1731.23 L2198.21 1731.23 L2198.21 1731.23  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2198.21,1731.23 2198.21,1734.12 2237.03,1734.12 2237.03,1731.23 2198.21,1731.23 "/>
<path clip-path="url(#clip472)" d="M2237.03 1722.56 L2237.03 1734.12 L2275.86 1734.12 L2275.86 1722.56 L2237.03 1722.56 L2237.03 1722.56  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2237.03,1722.56 2237.03,1734.12 2275.86,1734.12 2275.86,1722.56 2237.03,1722.56 "/>
<path clip-path="url(#clip472)" d="M2275.86 1725.45 L2275.86 1734.12 L2314.68 1734.12 L2314.68 1725.45 L2275.86 1725.45 L2275.86 1725.45  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2275.86,1725.45 2275.86,1734.12 2314.68,1734.12 2314.68,1725.45 2275.86,1725.45 "/>
<path clip-path="url(#clip472)" d="M2314.68 1725.45 L2314.68 1734.12 L2353.5 1734.12 L2353.5 1725.45 L2314.68 1725.45 L2314.68 1725.45  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2314.68,1725.45 2314.68,1734.12 2353.5,1734.12 2353.5,1725.45 2314.68,1725.45 "/>
<path clip-path="url(#clip472)" d="M2353.5 1731.23 L2353.5 1734.12 L2392.33 1734.12 L2392.33 1731.23 L2353.5 1731.23 L2353.5 1731.23  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2353.5,1731.23 2353.5,1734.12 2392.33,1734.12 2392.33,1731.23 2353.5,1731.23 "/>
<path clip-path="url(#clip472)" d="M2392.33 1728.34 L2392.33 1734.12 L2431.15 1734.12 L2431.15 1728.34 L2392.33 1728.34 L2392.33 1728.34  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2392.33,1728.34 2392.33,1734.12 2431.15,1734.12 2431.15,1728.34 2392.33,1728.34 "/>
<path clip-path="url(#clip472)" d="M2431.15 1734.12 L2431.15 1734.12 L2469.97 1734.12 L2469.97 1734.12 L2431.15 1734.12 L2431.15 1734.12  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2431.15,1734.12 2431.15,1734.12 2469.97,1734.12 2431.15,1734.12 "/>
<path clip-path="url(#clip472)" d="M2469.97 1728.34 L2469.97 1734.12 L2508.8 1734.12 L2508.8 1728.34 L2469.97 1728.34 L2469.97 1728.34  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip472)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2469.97,1728.34 2469.97,1734.12 2508.8,1734.12 2508.8,1728.34 2469.97,1728.34 "/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="392.929" cy="1728.34" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="431.752" cy="1728.34" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="470.575" cy="1719.67" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="509.399" cy="1719.67" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="548.222" cy="1702.34" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="587.045" cy="1653.24" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="625.868" cy="1595.47" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="664.692" cy="1581.03" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="703.515" cy="1442.38" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="742.338" cy="1329.73" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="781.161" cy="1231.52" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="819.985" cy="1023.55" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="858.808" cy="913.788" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="897.631" cy="688.487" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="936.454" cy="541.174" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="975.278" cy="549.84" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1014.1" cy="263.88" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1052.92" cy="90.5713" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1091.75" cy="159.895" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1130.57" cy="269.657" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1169.39" cy="47.2441" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1208.22" cy="226.33" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1247.04" cy="260.992" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1285.86" cy="252.326" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1324.69" cy="454.52" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1363.51" cy="587.39" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1402.33" cy="691.375" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1441.16" cy="792.472" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1479.98" cy="893.569" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1518.8" cy="1009.11" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1557.63" cy="1043.77" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1596.45" cy="1130.42" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1635.27" cy="1257.52" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1674.1" cy="1344.17" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1712.92" cy="1381.72" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1751.74" cy="1439.49" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1790.57" cy="1436.6" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1829.39" cy="1557.92" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1868.21" cy="1566.59" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1907.04" cy="1581.03" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1945.86" cy="1644.57" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1984.68" cy="1673.46" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="2023.51" cy="1670.57" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="2062.33" cy="1661.9" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="2101.15" cy="1699.46" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="2139.97" cy="1711.01" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="2178.8" cy="1716.79" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="2217.62" cy="1731.23" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="2256.44" cy="1722.56" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="2295.27" cy="1725.45" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="2334.09" cy="1725.45" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="2372.91" cy="1731.23" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="2411.74" cy="1728.34" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="2450.56" cy="1734.12" r="2"/>
<circle clip-path="url(#clip472)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="2489.38" cy="1728.34" r="2"/>
<polyline clip-path="url(#clip472)" style="stroke:#ff0000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" stroke-dasharray="16, 10" points="1188.81,3420.99 1188.81,-1639.63 "/>
</svg>

Figure 3: Samples of the climate feedback factor $\lambda$. The red line
is the deterministic value we used in the original simulation above.

1.  Creating a variable for the number of samples which can then be used
    later makes it easier to change the number of samples down the road
    without possibly creating bugs (if *e.g.* we forget to change the
    value everywhere).

Now, simulate:

``` julia
temp_samples = [ebm(t, forcing_non_aerosol, forcing_aerosol, c, d, λ, α, Δt)[idx_2100] for λ in λ_samples]
```

    10000-element Vector{Float64}:
     4.233693282773828
     5.933893605971489
     4.514426403132338
     4.605895747888149
     5.288967986478737
     5.182479744965509
     4.897900480221148
     4.580485648560467
     4.075054588461983
     4.504148961819632
     ⋮
     3.8867364172204786
     6.203514297510288
     4.1002072449594715
     4.142140456350632
     4.06146799698681
     5.263024402522369
     4.927274625722817
     5.541573154673585
     4.356137199634894

1.  We directly get the values of `λ` from `λ_samples` instead of
    looping over the indices (which is slightly slower and makes the
    line longer).

The expected value after this many simulations is the mean, or 4.69, and
the 95% confidence interval is (4.67, 4.7) (using the formula for the
standard error). The confidence interval around the expected value is
pretty tight, but let’s look at how the estimates of the mean and the
confidence interval evolve over the course of the simulations to see
what an “efficient” sample size might have been:

``` julia
# compute running estimate of the expected value and standard deviation
# first, pre-allocate memory for better efficiency
T_est = zeros(n_samples)
T_se = zeros(n_samples)
for i = 1:n_samples
    T_est[i] = mean(temp_samples[1:i])
    if i > 1
        T_se[i] = std(temp_samples[1:i]) / sqrt(i)
    end    
end

p = plot(; xlabel="Number of Samples", ylabel="Global Temperature Anomaly (°C)", legend=false)
plot!(p, 1:n_samples, T_est, ribbon=1.96 * T_se)
ylims!(p, (4.5, 5.0))
```

<?xml version="1.0" encoding="utf-8"?>
<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="672" height="480" viewBox="0 0 2688 1920">
<defs>
  <clipPath id="clip530">
    <rect x="0" y="0" width="2688" height="1920"/>
  </clipPath>
</defs>
<path clip-path="url(#clip530)" d="M0 1920 L2688 1920 L2688 -4.26326e-14 L0 -4.26326e-14  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<defs>
  <clipPath id="clip531">
    <rect x="537" y="0" width="1883" height="1883"/>
  </clipPath>
</defs>
<path clip-path="url(#clip530)" d="M226.627 1734.12 L2640.76 1734.12 L2640.76 47.2441 L226.627 47.2441  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<defs>
  <clipPath id="clip532">
    <rect x="226" y="47" width="2415" height="1688"/>
  </clipPath>
</defs>
<polyline clip-path="url(#clip532)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="294.723,1734.12 294.723,47.2441 "/>
<polyline clip-path="url(#clip532)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="864.15,1734.12 864.15,47.2441 "/>
<polyline clip-path="url(#clip532)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="1433.58,1734.12 1433.58,47.2441 "/>
<polyline clip-path="url(#clip532)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="2003,1734.12 2003,47.2441 "/>
<polyline clip-path="url(#clip532)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="2572.43,1734.12 2572.43,47.2441 "/>
<polyline clip-path="url(#clip530)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="226.627,1734.12 2640.76,1734.12 "/>
<polyline clip-path="url(#clip530)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="294.723,1734.12 294.723,1715.22 "/>
<polyline clip-path="url(#clip530)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="864.15,1734.12 864.15,1715.22 "/>
<polyline clip-path="url(#clip530)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1433.58,1734.12 1433.58,1715.22 "/>
<polyline clip-path="url(#clip530)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2003,1734.12 2003,1715.22 "/>
<polyline clip-path="url(#clip530)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2572.43,1734.12 2572.43,1715.22 "/>
<path clip-path="url(#clip530)" d="M294.723 1767.34 Q291.112 1767.34 289.284 1770.9 Q287.478 1774.45 287.478 1781.58 Q287.478 1788.68 289.284 1792.25 Q291.112 1795.79 294.723 1795.79 Q298.358 1795.79 300.163 1792.25 Q301.992 1788.68 301.992 1781.58 Q301.992 1774.45 300.163 1770.9 Q298.358 1767.34 294.723 1767.34 M294.723 1763.64 Q300.533 1763.64 303.589 1768.24 Q306.668 1772.83 306.668 1781.58 Q306.668 1790.3 303.589 1794.91 Q300.533 1799.49 294.723 1799.49 Q288.913 1799.49 285.834 1794.91 Q282.779 1790.3 282.779 1781.58 Q282.779 1772.83 285.834 1768.24 Q288.913 1763.64 294.723 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M812.762 1794.89 L829.081 1794.89 L829.081 1798.82 L807.137 1798.82 L807.137 1794.89 Q809.799 1792.13 814.382 1787.5 Q818.989 1782.85 820.169 1781.51 Q822.414 1778.98 823.294 1777.25 Q824.197 1775.49 824.197 1773.8 Q824.197 1771.04 822.252 1769.31 Q820.331 1767.57 817.229 1767.57 Q815.03 1767.57 812.577 1768.34 Q810.146 1769.1 807.368 1770.65 L807.368 1765.93 Q810.192 1764.79 812.646 1764.21 Q815.1 1763.64 817.137 1763.64 Q822.507 1763.64 825.701 1766.32 Q828.896 1769.01 828.896 1773.5 Q828.896 1775.63 828.086 1777.55 Q827.299 1779.45 825.192 1782.04 Q824.614 1782.71 821.512 1785.93 Q818.41 1789.12 812.762 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M838.942 1764.26 L857.299 1764.26 L857.299 1768.2 L843.225 1768.2 L843.225 1776.67 Q844.243 1776.32 845.262 1776.16 Q846.28 1775.97 847.299 1775.97 Q853.086 1775.97 856.465 1779.15 Q859.845 1782.32 859.845 1787.73 Q859.845 1793.31 856.373 1796.41 Q852.9 1799.49 846.581 1799.49 Q844.405 1799.49 842.137 1799.12 Q839.891 1798.75 837.484 1798.01 L837.484 1793.31 Q839.567 1794.45 841.789 1795 Q844.012 1795.56 846.488 1795.56 Q850.493 1795.56 852.831 1793.45 Q855.169 1791.34 855.169 1787.73 Q855.169 1784.12 852.831 1782.02 Q850.493 1779.91 846.488 1779.91 Q844.613 1779.91 842.738 1780.33 Q840.887 1780.74 838.942 1781.62 L838.942 1764.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M879.058 1767.34 Q875.447 1767.34 873.618 1770.9 Q871.812 1774.45 871.812 1781.58 Q871.812 1788.68 873.618 1792.25 Q875.447 1795.79 879.058 1795.79 Q882.692 1795.79 884.497 1792.25 Q886.326 1788.68 886.326 1781.58 Q886.326 1774.45 884.497 1770.9 Q882.692 1767.34 879.058 1767.34 M879.058 1763.64 Q884.868 1763.64 887.923 1768.24 Q891.002 1772.83 891.002 1781.58 Q891.002 1790.3 887.923 1794.91 Q884.868 1799.49 879.058 1799.49 Q873.248 1799.49 870.169 1794.91 Q867.113 1790.3 867.113 1781.58 Q867.113 1772.83 870.169 1768.24 Q873.248 1763.64 879.058 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M909.22 1767.34 Q905.608 1767.34 903.78 1770.9 Q901.974 1774.45 901.974 1781.58 Q901.974 1788.68 903.78 1792.25 Q905.608 1795.79 909.22 1795.79 Q912.854 1795.79 914.659 1792.25 Q916.488 1788.68 916.488 1781.58 Q916.488 1774.45 914.659 1770.9 Q912.854 1767.34 909.22 1767.34 M909.22 1763.64 Q915.03 1763.64 918.085 1768.24 Q921.164 1772.83 921.164 1781.58 Q921.164 1790.3 918.085 1794.91 Q915.03 1799.49 909.22 1799.49 Q903.409 1799.49 900.331 1794.91 Q897.275 1790.3 897.275 1781.58 Q897.275 1772.83 900.331 1768.24 Q903.409 1763.64 909.22 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M1378.11 1764.26 L1396.47 1764.26 L1396.47 1768.2 L1382.4 1768.2 L1382.4 1776.67 Q1383.42 1776.32 1384.43 1776.16 Q1385.45 1775.97 1386.47 1775.97 Q1392.26 1775.97 1395.64 1779.15 Q1399.02 1782.32 1399.02 1787.73 Q1399.02 1793.31 1395.55 1796.41 Q1392.07 1799.49 1385.75 1799.49 Q1383.58 1799.49 1381.31 1799.12 Q1379.06 1798.75 1376.66 1798.01 L1376.66 1793.31 Q1378.74 1794.45 1380.96 1795 Q1383.18 1795.56 1385.66 1795.56 Q1389.67 1795.56 1392 1793.45 Q1394.34 1791.34 1394.34 1787.73 Q1394.34 1784.12 1392 1782.02 Q1389.67 1779.91 1385.66 1779.91 Q1383.79 1779.91 1381.91 1780.33 Q1380.06 1780.74 1378.11 1781.62 L1378.11 1764.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M1418.23 1767.34 Q1414.62 1767.34 1412.79 1770.9 Q1410.98 1774.45 1410.98 1781.58 Q1410.98 1788.68 1412.79 1792.25 Q1414.62 1795.79 1418.23 1795.79 Q1421.86 1795.79 1423.67 1792.25 Q1425.5 1788.68 1425.5 1781.58 Q1425.5 1774.45 1423.67 1770.9 Q1421.86 1767.34 1418.23 1767.34 M1418.23 1763.64 Q1424.04 1763.64 1427.1 1768.24 Q1430.17 1772.83 1430.17 1781.58 Q1430.17 1790.3 1427.1 1794.91 Q1424.04 1799.49 1418.23 1799.49 Q1412.42 1799.49 1409.34 1794.91 Q1406.29 1790.3 1406.29 1781.58 Q1406.29 1772.83 1409.34 1768.24 Q1412.42 1763.64 1418.23 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M1448.39 1767.34 Q1444.78 1767.34 1442.95 1770.9 Q1441.15 1774.45 1441.15 1781.58 Q1441.15 1788.68 1442.95 1792.25 Q1444.78 1795.79 1448.39 1795.79 Q1452.03 1795.79 1453.83 1792.25 Q1455.66 1788.68 1455.66 1781.58 Q1455.66 1774.45 1453.83 1770.9 Q1452.03 1767.34 1448.39 1767.34 M1448.39 1763.64 Q1454.2 1763.64 1457.26 1768.24 Q1460.34 1772.83 1460.34 1781.58 Q1460.34 1790.3 1457.26 1794.91 Q1454.2 1799.49 1448.39 1799.49 Q1442.58 1799.49 1439.5 1794.91 Q1436.45 1790.3 1436.45 1781.58 Q1436.45 1772.83 1439.5 1768.24 Q1442.58 1763.64 1448.39 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M1478.55 1767.34 Q1474.94 1767.34 1473.11 1770.9 Q1471.31 1774.45 1471.31 1781.58 Q1471.31 1788.68 1473.11 1792.25 Q1474.94 1795.79 1478.55 1795.79 Q1482.19 1795.79 1483.99 1792.25 Q1485.82 1788.68 1485.82 1781.58 Q1485.82 1774.45 1483.99 1770.9 Q1482.19 1767.34 1478.55 1767.34 M1478.55 1763.64 Q1484.36 1763.64 1487.42 1768.24 Q1490.5 1772.83 1490.5 1781.58 Q1490.5 1790.3 1487.42 1794.91 Q1484.36 1799.49 1478.55 1799.49 Q1472.74 1799.49 1469.67 1794.91 Q1466.61 1790.3 1466.61 1781.58 Q1466.61 1772.83 1469.67 1768.24 Q1472.74 1763.64 1478.55 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M1946.2 1764.26 L1968.42 1764.26 L1968.42 1766.25 L1955.88 1798.82 L1950.99 1798.82 L1962.8 1768.2 L1946.2 1768.2 L1946.2 1764.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M1977.59 1764.26 L1995.94 1764.26 L1995.94 1768.2 L1981.87 1768.2 L1981.87 1776.67 Q1982.89 1776.32 1983.91 1776.16 Q1984.93 1775.97 1985.94 1775.97 Q1991.73 1775.97 1995.11 1779.15 Q1998.49 1782.32 1998.49 1787.73 Q1998.49 1793.31 1995.02 1796.41 Q1991.55 1799.49 1985.23 1799.49 Q1983.05 1799.49 1980.78 1799.12 Q1978.54 1798.75 1976.13 1798.01 L1976.13 1793.31 Q1978.21 1794.45 1980.44 1795 Q1982.66 1795.56 1985.13 1795.56 Q1989.14 1795.56 1991.48 1793.45 Q1993.81 1791.34 1993.81 1787.73 Q1993.81 1784.12 1991.48 1782.02 Q1989.14 1779.91 1985.13 1779.91 Q1983.26 1779.91 1981.38 1780.33 Q1979.53 1780.74 1977.59 1781.62 L1977.59 1764.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M2017.7 1767.34 Q2014.09 1767.34 2012.26 1770.9 Q2010.46 1774.45 2010.46 1781.58 Q2010.46 1788.68 2012.26 1792.25 Q2014.09 1795.79 2017.7 1795.79 Q2021.34 1795.79 2023.14 1792.25 Q2024.97 1788.68 2024.97 1781.58 Q2024.97 1774.45 2023.14 1770.9 Q2021.34 1767.34 2017.7 1767.34 M2017.7 1763.64 Q2023.51 1763.64 2026.57 1768.24 Q2029.65 1772.83 2029.65 1781.58 Q2029.65 1790.3 2026.57 1794.91 Q2023.51 1799.49 2017.7 1799.49 Q2011.89 1799.49 2008.81 1794.91 Q2005.76 1790.3 2005.76 1781.58 Q2005.76 1772.83 2008.81 1768.24 Q2011.89 1763.64 2017.7 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M2047.87 1767.34 Q2044.25 1767.34 2042.43 1770.9 Q2040.62 1774.45 2040.62 1781.58 Q2040.62 1788.68 2042.43 1792.25 Q2044.25 1795.79 2047.87 1795.79 Q2051.5 1795.79 2053.31 1792.25 Q2055.13 1788.68 2055.13 1781.58 Q2055.13 1774.45 2053.31 1770.9 Q2051.5 1767.34 2047.87 1767.34 M2047.87 1763.64 Q2053.68 1763.64 2056.73 1768.24 Q2059.81 1772.83 2059.81 1781.58 Q2059.81 1790.3 2056.73 1794.91 Q2053.68 1799.49 2047.87 1799.49 Q2042.06 1799.49 2038.98 1794.91 Q2035.92 1790.3 2035.92 1781.58 Q2035.92 1772.83 2038.98 1768.24 Q2042.06 1763.64 2047.87 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M2501.88 1794.89 L2509.52 1794.89 L2509.52 1768.52 L2501.21 1770.19 L2501.21 1765.93 L2509.47 1764.26 L2514.14 1764.26 L2514.14 1794.89 L2521.78 1794.89 L2521.78 1798.82 L2501.88 1798.82 L2501.88 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M2541.23 1767.34 Q2537.62 1767.34 2535.79 1770.9 Q2533.98 1774.45 2533.98 1781.58 Q2533.98 1788.68 2535.79 1792.25 Q2537.62 1795.79 2541.23 1795.79 Q2544.86 1795.79 2546.67 1792.25 Q2548.5 1788.68 2548.5 1781.58 Q2548.5 1774.45 2546.67 1770.9 Q2544.86 1767.34 2541.23 1767.34 M2541.23 1763.64 Q2547.04 1763.64 2550.09 1768.24 Q2553.17 1772.83 2553.17 1781.58 Q2553.17 1790.3 2550.09 1794.91 Q2547.04 1799.49 2541.23 1799.49 Q2535.42 1799.49 2532.34 1794.91 Q2529.28 1790.3 2529.28 1781.58 Q2529.28 1772.83 2532.34 1768.24 Q2535.42 1763.64 2541.23 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M2571.39 1767.34 Q2567.78 1767.34 2565.95 1770.9 Q2564.14 1774.45 2564.14 1781.58 Q2564.14 1788.68 2565.95 1792.25 Q2567.78 1795.79 2571.39 1795.79 Q2575.02 1795.79 2576.83 1792.25 Q2578.66 1788.68 2578.66 1781.58 Q2578.66 1774.45 2576.83 1770.9 Q2575.02 1767.34 2571.39 1767.34 M2571.39 1763.64 Q2577.2 1763.64 2580.26 1768.24 Q2583.33 1772.83 2583.33 1781.58 Q2583.33 1790.3 2580.26 1794.91 Q2577.2 1799.49 2571.39 1799.49 Q2565.58 1799.49 2562.5 1794.91 Q2559.45 1790.3 2559.45 1781.58 Q2559.45 1772.83 2562.5 1768.24 Q2565.58 1763.64 2571.39 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M2601.55 1767.34 Q2597.94 1767.34 2596.11 1770.9 Q2594.31 1774.45 2594.31 1781.58 Q2594.31 1788.68 2596.11 1792.25 Q2597.94 1795.79 2601.55 1795.79 Q2605.19 1795.79 2606.99 1792.25 Q2608.82 1788.68 2608.82 1781.58 Q2608.82 1774.45 2606.99 1770.9 Q2605.19 1767.34 2601.55 1767.34 M2601.55 1763.64 Q2607.36 1763.64 2610.42 1768.24 Q2613.5 1772.83 2613.5 1781.58 Q2613.5 1790.3 2610.42 1794.91 Q2607.36 1799.49 2601.55 1799.49 Q2595.74 1799.49 2592.66 1794.91 Q2589.61 1790.3 2589.61 1781.58 Q2589.61 1772.83 2592.66 1768.24 Q2595.74 1763.64 2601.55 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M2631.71 1767.34 Q2628.1 1767.34 2626.27 1770.9 Q2624.47 1774.45 2624.47 1781.58 Q2624.47 1788.68 2626.27 1792.25 Q2628.1 1795.79 2631.71 1795.79 Q2635.35 1795.79 2637.15 1792.25 Q2638.98 1788.68 2638.98 1781.58 Q2638.98 1774.45 2637.15 1770.9 Q2635.35 1767.34 2631.71 1767.34 M2631.71 1763.64 Q2637.52 1763.64 2640.58 1768.24 Q2643.66 1772.83 2643.66 1781.58 Q2643.66 1790.3 2640.58 1794.91 Q2637.52 1799.49 2631.71 1799.49 Q2625.9 1799.49 2622.82 1794.91 Q2619.77 1790.3 2619.77 1781.58 Q2619.77 1772.83 2622.82 1768.24 Q2625.9 1763.64 2631.71 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M1116.23 1835.78 L1124.89 1835.78 L1145.96 1875.53 L1145.96 1835.78 L1152.2 1835.78 L1152.2 1883.3 L1143.54 1883.3 L1122.47 1843.55 L1122.47 1883.3 L1116.23 1883.3 L1116.23 1835.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M1164.14 1869.23 L1164.14 1847.65 L1169.99 1847.65 L1169.99 1869.01 Q1169.99 1874.07 1171.97 1876.62 Q1173.94 1879.13 1177.89 1879.13 Q1182.63 1879.13 1185.37 1876.11 Q1188.13 1873.08 1188.13 1867.86 L1188.13 1847.65 L1193.99 1847.65 L1193.99 1883.3 L1188.13 1883.3 L1188.13 1877.83 Q1186 1881.07 1183.17 1882.66 Q1180.37 1884.22 1176.64 1884.22 Q1170.5 1884.22 1167.32 1880.4 Q1164.14 1876.59 1164.14 1869.23 M1178.87 1846.79 L1178.87 1846.79 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M1233.81 1854.5 Q1236 1850.55 1239.06 1848.67 Q1242.12 1846.79 1246.25 1846.79 Q1251.82 1846.79 1254.85 1850.71 Q1257.87 1854.59 1257.87 1861.78 L1257.87 1883.3 L1251.98 1883.3 L1251.98 1861.98 Q1251.98 1856.85 1250.17 1854.37 Q1248.35 1851.89 1244.63 1851.89 Q1240.08 1851.89 1237.44 1854.91 Q1234.79 1857.93 1234.79 1863.15 L1234.79 1883.3 L1228.91 1883.3 L1228.91 1861.98 Q1228.91 1856.82 1227.09 1854.37 Q1225.28 1851.89 1221.49 1851.89 Q1217 1851.89 1214.36 1854.94 Q1211.72 1857.97 1211.72 1863.15 L1211.72 1883.3 L1205.83 1883.3 L1205.83 1847.65 L1211.72 1847.65 L1211.72 1853.19 Q1213.72 1849.91 1216.53 1848.35 Q1219.33 1846.79 1223.18 1846.79 Q1227.06 1846.79 1229.77 1848.77 Q1232.5 1850.74 1233.81 1854.5 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M1295.14 1865.51 Q1295.14 1859.05 1292.47 1855.39 Q1289.83 1851.7 1285.18 1851.7 Q1280.53 1851.7 1277.86 1855.39 Q1275.22 1859.05 1275.22 1865.51 Q1275.22 1871.97 1277.86 1875.66 Q1280.53 1879.32 1285.18 1879.32 Q1289.83 1879.32 1292.47 1875.66 Q1295.14 1871.97 1295.14 1865.51 M1275.22 1853.06 Q1277.06 1849.88 1279.86 1848.35 Q1282.7 1846.79 1286.61 1846.79 Q1293.1 1846.79 1297.15 1851.95 Q1301.22 1857.11 1301.22 1865.51 Q1301.22 1873.91 1297.15 1879.07 Q1293.1 1884.22 1286.61 1884.22 Q1282.7 1884.22 1279.86 1882.7 Q1277.06 1881.14 1275.22 1877.95 L1275.22 1883.3 L1269.33 1883.3 L1269.33 1833.78 L1275.22 1833.78 L1275.22 1853.06 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M1341.42 1864.01 L1341.42 1866.88 L1314.49 1866.88 Q1314.88 1872.92 1318.12 1876.11 Q1321.4 1879.26 1327.22 1879.26 Q1330.6 1879.26 1333.75 1878.43 Q1336.93 1877.6 1340.05 1875.95 L1340.05 1881.49 Q1336.9 1882.82 1333.59 1883.52 Q1330.28 1884.22 1326.87 1884.22 Q1318.34 1884.22 1313.35 1879.26 Q1308.38 1874.29 1308.38 1865.83 Q1308.38 1857.07 1313.09 1851.95 Q1317.84 1846.79 1325.86 1846.79 Q1333.05 1846.79 1337.22 1851.44 Q1341.42 1856.06 1341.42 1864.01 M1335.56 1862.29 Q1335.5 1857.49 1332.86 1854.62 Q1330.25 1851.76 1325.92 1851.76 Q1321.02 1851.76 1318.06 1854.53 Q1315.13 1857.3 1314.68 1862.33 L1335.56 1862.29 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M1371.69 1853.13 Q1370.7 1852.55 1369.52 1852.3 Q1368.38 1852.01 1366.98 1852.01 Q1362.01 1852.01 1359.34 1855.26 Q1356.7 1858.47 1356.7 1864.52 L1356.7 1883.3 L1350.81 1883.3 L1350.81 1847.65 L1356.7 1847.65 L1356.7 1853.19 Q1358.54 1849.94 1361.5 1848.39 Q1364.46 1846.79 1368.7 1846.79 Q1369.3 1846.79 1370.03 1846.89 Q1370.77 1846.95 1371.66 1847.11 L1371.69 1853.13 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M1412.37 1851.76 Q1407.66 1851.76 1404.92 1855.45 Q1402.18 1859.11 1402.18 1865.51 Q1402.18 1871.91 1404.89 1875.6 Q1407.62 1879.26 1412.37 1879.26 Q1417.04 1879.26 1419.78 1875.57 Q1422.52 1871.87 1422.52 1865.51 Q1422.52 1859.17 1419.78 1855.48 Q1417.04 1851.76 1412.37 1851.76 M1412.37 1846.79 Q1420.01 1846.79 1424.37 1851.76 Q1428.73 1856.72 1428.73 1865.51 Q1428.73 1874.26 1424.37 1879.26 Q1420.01 1884.22 1412.37 1884.22 Q1404.7 1884.22 1400.33 1879.26 Q1396.01 1874.26 1396.01 1865.51 Q1396.01 1856.72 1400.33 1851.76 Q1404.7 1846.79 1412.37 1846.79 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M1456.48 1833.78 L1456.48 1838.65 L1450.88 1838.65 Q1447.73 1838.65 1446.49 1839.92 Q1445.28 1841.19 1445.28 1844.5 L1445.28 1847.65 L1454.92 1847.65 L1454.92 1852.2 L1445.28 1852.2 L1445.28 1883.3 L1439.39 1883.3 L1439.39 1852.2 L1433.79 1852.2 L1433.79 1847.65 L1439.39 1847.65 L1439.39 1845.17 Q1439.39 1839.22 1442.16 1836.51 Q1444.93 1833.78 1450.94 1833.78 L1456.48 1833.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M1510.84 1837.34 L1510.84 1843.61 Q1507.18 1841.86 1503.94 1841 Q1500.69 1840.14 1497.67 1840.14 Q1492.41 1840.14 1489.55 1842.18 Q1486.72 1844.22 1486.72 1847.97 Q1486.72 1851.12 1488.6 1852.75 Q1490.51 1854.34 1495.79 1855.32 L1499.67 1856.12 Q1506.87 1857.49 1510.27 1860.96 Q1513.71 1864.39 1513.71 1870.19 Q1513.71 1877.09 1509.06 1880.66 Q1504.45 1884.22 1495.5 1884.22 Q1492.13 1884.22 1488.31 1883.46 Q1484.52 1882.7 1480.45 1881.2 L1480.45 1874.58 Q1484.36 1876.78 1488.12 1877.89 Q1491.87 1879 1495.5 1879 Q1501.01 1879 1504 1876.84 Q1506.99 1874.68 1506.99 1870.67 Q1506.99 1867.16 1504.83 1865.19 Q1502.7 1863.22 1497.79 1862.23 L1493.88 1861.47 Q1486.69 1860.03 1483.47 1856.98 Q1480.26 1853.92 1480.26 1848.48 Q1480.26 1842.18 1484.68 1838.55 Q1489.14 1834.92 1496.93 1834.92 Q1500.28 1834.92 1503.75 1835.53 Q1507.22 1836.13 1510.84 1837.34 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M1539.68 1865.38 Q1532.58 1865.38 1529.85 1867 Q1527.11 1868.63 1527.11 1872.54 Q1527.11 1875.66 1529.15 1877.51 Q1531.21 1879.32 1534.75 1879.32 Q1539.62 1879.32 1542.54 1875.88 Q1545.5 1872.42 1545.5 1866.69 L1545.5 1865.38 L1539.68 1865.38 M1551.36 1862.96 L1551.36 1883.3 L1545.5 1883.3 L1545.5 1877.89 Q1543.5 1881.14 1540.51 1882.7 Q1537.52 1884.22 1533.19 1884.22 Q1527.71 1884.22 1524.47 1881.17 Q1521.25 1878.08 1521.25 1872.92 Q1521.25 1866.91 1525.26 1863.85 Q1529.3 1860.8 1537.29 1860.8 L1545.5 1860.8 L1545.5 1860.23 Q1545.5 1856.18 1542.83 1853.99 Q1540.19 1851.76 1535.38 1851.76 Q1532.33 1851.76 1529.43 1852.49 Q1526.54 1853.22 1523.86 1854.69 L1523.86 1849.28 Q1527.08 1848.03 1530.1 1847.43 Q1533.12 1846.79 1535.99 1846.79 Q1543.72 1846.79 1547.54 1850.8 Q1551.36 1854.81 1551.36 1862.96 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M1591.18 1854.5 Q1593.38 1850.55 1596.43 1848.67 Q1599.49 1846.79 1603.62 1846.79 Q1609.19 1846.79 1612.22 1850.71 Q1615.24 1854.59 1615.24 1861.78 L1615.24 1883.3 L1609.35 1883.3 L1609.35 1861.98 Q1609.35 1856.85 1607.54 1854.37 Q1605.72 1851.89 1602 1851.89 Q1597.45 1851.89 1594.81 1854.91 Q1592.17 1857.93 1592.17 1863.15 L1592.17 1883.3 L1586.28 1883.3 L1586.28 1861.98 Q1586.28 1856.82 1584.46 1854.37 Q1582.65 1851.89 1578.86 1851.89 Q1574.37 1851.89 1571.73 1854.94 Q1569.09 1857.97 1569.09 1863.15 L1569.09 1883.3 L1563.2 1883.3 L1563.2 1847.65 L1569.09 1847.65 L1569.09 1853.19 Q1571.1 1849.91 1573.9 1848.35 Q1576.7 1846.79 1580.55 1846.79 Q1584.43 1846.79 1587.14 1848.77 Q1589.87 1850.74 1591.18 1854.5 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M1632.59 1877.95 L1632.59 1896.86 L1626.7 1896.86 L1626.7 1847.65 L1632.59 1847.65 L1632.59 1853.06 Q1634.43 1849.88 1637.23 1848.35 Q1640.07 1846.79 1643.98 1846.79 Q1650.48 1846.79 1654.52 1851.95 Q1658.59 1857.11 1658.59 1865.51 Q1658.59 1873.91 1654.52 1879.07 Q1650.48 1884.22 1643.98 1884.22 Q1640.07 1884.22 1637.23 1882.7 Q1634.43 1881.14 1632.59 1877.95 M1652.51 1865.51 Q1652.51 1859.05 1649.84 1855.39 Q1647.2 1851.7 1642.55 1851.7 Q1637.9 1851.7 1635.23 1855.39 Q1632.59 1859.05 1632.59 1865.51 Q1632.59 1871.97 1635.23 1875.66 Q1637.9 1879.32 1642.55 1879.32 Q1647.2 1879.32 1649.84 1875.66 Q1652.51 1871.97 1652.51 1865.51 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M1668.3 1833.78 L1674.16 1833.78 L1674.16 1883.3 L1668.3 1883.3 L1668.3 1833.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M1716.9 1864.01 L1716.9 1866.88 L1689.97 1866.88 Q1690.36 1872.92 1693.6 1876.11 Q1696.88 1879.26 1702.71 1879.26 Q1706.08 1879.26 1709.23 1878.43 Q1712.41 1877.6 1715.53 1875.95 L1715.53 1881.49 Q1712.38 1882.82 1709.07 1883.52 Q1705.76 1884.22 1702.36 1884.22 Q1693.83 1884.22 1688.83 1879.26 Q1683.86 1874.29 1683.86 1865.83 Q1683.86 1857.07 1688.57 1851.95 Q1693.32 1846.79 1701.34 1846.79 Q1708.53 1846.79 1712.7 1851.44 Q1716.9 1856.06 1716.9 1864.01 M1711.05 1862.29 Q1710.98 1857.49 1708.34 1854.62 Q1705.73 1851.76 1701.4 1851.76 Q1696.5 1851.76 1693.54 1854.53 Q1690.61 1857.3 1690.17 1862.33 L1711.05 1862.29 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M1749.24 1848.7 L1749.24 1854.24 Q1746.76 1852.97 1744.08 1852.33 Q1741.41 1851.7 1738.54 1851.7 Q1734.18 1851.7 1731.99 1853.03 Q1729.82 1854.37 1729.82 1857.04 Q1729.82 1859.08 1731.38 1860.26 Q1732.94 1861.4 1737.65 1862.45 L1739.66 1862.9 Q1745.9 1864.24 1748.51 1866.69 Q1751.15 1869.11 1751.15 1873.47 Q1751.15 1878.43 1747.2 1881.33 Q1743.29 1884.22 1736.41 1884.22 Q1733.55 1884.22 1730.43 1883.65 Q1727.34 1883.11 1723.9 1882 L1723.9 1875.95 Q1727.15 1877.64 1730.3 1878.49 Q1733.45 1879.32 1736.54 1879.32 Q1740.68 1879.32 1742.91 1877.92 Q1745.13 1876.49 1745.13 1873.91 Q1745.13 1871.52 1743.51 1870.25 Q1741.92 1868.98 1736.48 1867.8 L1734.44 1867.32 Q1729 1866.18 1726.58 1863.82 Q1724.16 1861.43 1724.16 1857.3 Q1724.16 1852.27 1727.72 1849.53 Q1731.29 1846.79 1737.84 1846.79 Q1741.09 1846.79 1743.96 1847.27 Q1746.82 1847.75 1749.24 1848.7 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><polyline clip-path="url(#clip532)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="226.627,1734.12 2640.76,1734.12 "/>
<polyline clip-path="url(#clip532)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="226.627,1396.74 2640.76,1396.74 "/>
<polyline clip-path="url(#clip532)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="226.627,1059.37 2640.76,1059.37 "/>
<polyline clip-path="url(#clip532)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="226.627,721.993 2640.76,721.993 "/>
<polyline clip-path="url(#clip532)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="226.627,384.619 2640.76,384.619 "/>
<polyline clip-path="url(#clip532)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="226.627,47.2441 2640.76,47.2441 "/>
<polyline clip-path="url(#clip530)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="226.627,1734.12 226.627,47.2441 "/>
<polyline clip-path="url(#clip530)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="226.627,1734.12 245.524,1734.12 "/>
<polyline clip-path="url(#clip530)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="226.627,1396.74 245.524,1396.74 "/>
<polyline clip-path="url(#clip530)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="226.627,1059.37 245.524,1059.37 "/>
<polyline clip-path="url(#clip530)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="226.627,721.993 245.524,721.993 "/>
<polyline clip-path="url(#clip530)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="226.627,384.619 245.524,384.619 "/>
<polyline clip-path="url(#clip530)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="226.627,47.2441 245.524,47.2441 "/>
<path clip-path="url(#clip530)" d="M132.974 1720.91 L121.168 1739.36 L132.974 1739.36 L132.974 1720.91 M131.747 1716.84 L137.626 1716.84 L137.626 1739.36 L142.557 1739.36 L142.557 1743.25 L137.626 1743.25 L137.626 1751.4 L132.974 1751.4 L132.974 1743.25 L117.372 1743.25 L117.372 1738.74 L131.747 1716.84 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M150.288 1745.52 L155.173 1745.52 L155.173 1751.4 L150.288 1751.4 L150.288 1745.52 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M165.404 1716.84 L183.76 1716.84 L183.76 1720.77 L169.686 1720.77 L169.686 1729.24 Q170.705 1728.9 171.723 1728.74 Q172.742 1728.55 173.76 1728.55 Q179.547 1728.55 182.927 1731.72 Q186.307 1734.89 186.307 1740.31 Q186.307 1745.89 182.834 1748.99 Q179.362 1752.07 173.043 1752.07 Q170.867 1752.07 168.598 1751.7 Q166.353 1751.33 163.946 1750.59 L163.946 1745.89 Q166.029 1747.02 168.251 1747.58 Q170.473 1748.13 172.95 1748.13 Q176.955 1748.13 179.293 1746.03 Q181.631 1743.92 181.631 1740.31 Q181.631 1736.7 179.293 1734.59 Q176.955 1732.49 172.95 1732.49 Q171.075 1732.49 169.2 1732.9 Q167.348 1733.32 165.404 1734.2 L165.404 1716.84 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M131.816 1383.54 L120.011 1401.99 L131.816 1401.99 L131.816 1383.54 M130.589 1379.46 L136.469 1379.46 L136.469 1401.99 L141.399 1401.99 L141.399 1405.87 L136.469 1405.87 L136.469 1414.02 L131.816 1414.02 L131.816 1405.87 L116.214 1405.87 L116.214 1401.36 L130.589 1379.46 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M149.131 1408.14 L154.015 1408.14 L154.015 1414.02 L149.131 1414.02 L149.131 1408.14 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M174.779 1394.88 Q171.631 1394.88 169.779 1397.03 Q167.95 1399.18 167.95 1402.93 Q167.95 1406.66 169.779 1408.84 Q171.631 1410.99 174.779 1410.99 Q177.927 1410.99 179.756 1408.84 Q181.608 1406.66 181.608 1402.93 Q181.608 1399.18 179.756 1397.03 Q177.927 1394.88 174.779 1394.88 M184.061 1380.23 L184.061 1384.49 Q182.302 1383.65 180.497 1383.21 Q178.714 1382.77 176.955 1382.77 Q172.325 1382.77 169.872 1385.9 Q167.441 1389.02 167.094 1395.34 Q168.46 1393.33 170.52 1392.26 Q172.58 1391.18 175.057 1391.18 Q180.265 1391.18 183.274 1394.35 Q186.307 1397.49 186.307 1402.93 Q186.307 1408.26 183.159 1411.48 Q180.01 1414.69 174.779 1414.69 Q168.784 1414.69 165.612 1410.11 Q162.441 1405.5 162.441 1396.78 Q162.441 1388.58 166.33 1383.72 Q170.219 1378.84 176.77 1378.84 Q178.529 1378.84 180.311 1379.18 Q182.117 1379.53 184.061 1380.23 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M132.881 1046.16 L121.076 1064.61 L132.881 1064.61 L132.881 1046.16 M131.654 1042.09 L137.534 1042.09 L137.534 1064.61 L142.464 1064.61 L142.464 1068.5 L137.534 1068.5 L137.534 1076.65 L132.881 1076.65 L132.881 1068.5 L117.279 1068.5 L117.279 1063.99 L131.654 1042.09 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M150.196 1070.77 L155.08 1070.77 L155.08 1076.65 L150.196 1076.65 L150.196 1070.77 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M164.085 1042.09 L186.307 1042.09 L186.307 1044.08 L173.76 1076.65 L168.876 1076.65 L180.682 1046.02 L164.085 1046.02 L164.085 1042.09 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M132.071 708.787 L120.265 727.236 L132.071 727.236 L132.071 708.787 M130.844 704.713 L136.724 704.713 L136.724 727.236 L141.654 727.236 L141.654 731.125 L136.724 731.125 L136.724 739.273 L132.071 739.273 L132.071 731.125 L116.469 731.125 L116.469 726.611 L130.844 704.713 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M149.386 733.394 L154.27 733.394 L154.27 739.273 L149.386 739.273 L149.386 733.394 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M174.455 722.861 Q171.122 722.861 169.2 724.644 Q167.302 726.426 167.302 729.551 Q167.302 732.676 169.2 734.458 Q171.122 736.241 174.455 736.241 Q177.788 736.241 179.709 734.458 Q181.631 732.653 181.631 729.551 Q181.631 726.426 179.709 724.644 Q177.811 722.861 174.455 722.861 M169.779 720.871 Q166.77 720.13 165.08 718.07 Q163.413 716.009 163.413 713.047 Q163.413 708.903 166.353 706.496 Q169.316 704.088 174.455 704.088 Q179.617 704.088 182.557 706.496 Q185.496 708.903 185.496 713.047 Q185.496 716.009 183.807 718.07 Q182.14 720.13 179.154 720.871 Q182.534 721.658 184.409 723.949 Q186.307 726.241 186.307 729.551 Q186.307 734.574 183.228 737.259 Q180.172 739.945 174.455 739.945 Q168.737 739.945 165.659 737.259 Q162.603 734.574 162.603 729.551 Q162.603 726.241 164.501 723.949 Q166.399 721.658 169.779 720.871 M168.066 713.486 Q168.066 716.172 169.733 717.676 Q171.422 719.181 174.455 719.181 Q177.464 719.181 179.154 717.676 Q180.867 716.172 180.867 713.486 Q180.867 710.801 179.154 709.297 Q177.464 707.792 174.455 707.792 Q171.422 707.792 169.733 709.297 Q168.066 710.801 168.066 713.486 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M132.163 371.413 L120.358 389.862 L132.163 389.862 L132.163 371.413 M130.937 367.339 L136.816 367.339 L136.816 389.862 L141.747 389.862 L141.747 393.751 L136.816 393.751 L136.816 401.899 L132.163 401.899 L132.163 393.751 L116.562 393.751 L116.562 389.237 L130.937 367.339 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M149.478 396.019 L154.362 396.019 L154.362 401.899 L149.478 401.899 L149.478 396.019 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M164.686 401.181 L164.686 396.922 Q166.446 397.755 168.251 398.195 Q170.057 398.635 171.793 398.635 Q176.422 398.635 178.853 395.533 Q181.307 392.408 181.654 386.065 Q180.311 388.056 178.251 389.121 Q176.191 390.186 173.691 390.186 Q168.506 390.186 165.473 387.061 Q162.464 383.913 162.464 378.473 Q162.464 373.149 165.612 369.931 Q168.76 366.714 173.992 366.714 Q179.987 366.714 183.135 371.32 Q186.307 375.903 186.307 384.653 Q186.307 392.825 182.418 397.709 Q178.552 402.57 172.001 402.57 Q170.242 402.57 168.436 402.223 Q166.631 401.876 164.686 401.181 M173.992 386.528 Q177.14 386.528 178.969 384.376 Q180.821 382.223 180.821 378.473 Q180.821 374.746 178.969 372.593 Q177.14 370.417 173.992 370.417 Q170.844 370.417 168.992 372.593 Q167.163 374.746 167.163 378.473 Q167.163 382.223 168.992 384.376 Q170.844 386.528 173.992 386.528 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M119.177 29.9641 L137.534 29.9641 L137.534 33.8993 L123.46 33.8993 L123.46 42.3714 Q124.478 42.0242 125.497 41.8622 Q126.515 41.677 127.534 41.677 Q133.321 41.677 136.7 44.8483 Q140.08 48.0196 140.08 53.4362 Q140.08 59.0149 136.608 62.1167 Q133.136 65.1954 126.816 65.1954 Q124.64 65.1954 122.372 64.825 Q120.126 64.4547 117.719 63.7139 L117.719 59.0149 Q119.802 60.1491 122.025 60.7047 Q124.247 61.2602 126.724 61.2602 Q130.728 61.2602 133.066 59.1538 Q135.404 57.0473 135.404 53.4362 Q135.404 49.8251 133.066 47.7186 Q130.728 45.6122 126.724 45.6122 Q124.849 45.6122 122.974 46.0288 Q121.122 46.4455 119.177 47.3251 L119.177 29.9641 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M149.293 58.6445 L154.177 58.6445 L154.177 64.5241 L149.293 64.5241 L149.293 58.6445 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M174.362 33.0428 Q170.751 33.0428 168.922 36.6076 Q167.117 40.1492 167.117 47.2788 Q167.117 54.3853 168.922 57.9501 Q170.751 61.4917 174.362 61.4917 Q177.997 61.4917 179.802 57.9501 Q181.631 54.3853 181.631 47.2788 Q181.631 40.1492 179.802 36.6076 Q177.997 33.0428 174.362 33.0428 M174.362 29.3391 Q180.172 29.3391 183.228 33.9456 Q186.307 38.5289 186.307 47.2788 Q186.307 56.0056 183.228 60.6121 Q180.172 65.1954 174.362 65.1954 Q168.552 65.1954 165.473 60.6121 Q162.418 56.0056 162.418 47.2788 Q162.418 38.5289 165.473 33.9456 Q168.552 29.3391 174.362 29.3391 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M54.859 1399.86 L42.0957 1399.86 L42.0957 1410.36 L36.8122 1410.36 L36.8122 1393.49 L57.2143 1393.49 Q59.856 1397.22 61.2247 1401.7 Q62.5615 1406.19 62.5615 1411.28 Q62.5615 1422.42 56.0684 1428.73 Q49.5436 1435 37.9262 1435 Q26.2769 1435 19.7839 1428.73 Q13.2591 1422.42 13.2591 1411.28 Q13.2591 1406.64 14.4049 1402.47 Q15.5507 1398.27 17.7787 1394.73 L24.6219 1394.73 Q21.5981 1398.3 20.0704 1402.31 Q18.5426 1406.32 18.5426 1410.74 Q18.5426 1419.46 23.4124 1423.86 Q28.2821 1428.22 37.9262 1428.22 Q47.5384 1428.22 52.4082 1423.86 Q57.2779 1419.46 57.2779 1410.74 Q57.2779 1407.34 56.705 1404.66 Q56.1003 1401.99 54.859 1399.86 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M12.1132 1382 L12.1132 1376.15 L61.6384 1376.15 L61.6384 1382 L12.1132 1382 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M30.0964 1350.08 Q30.0964 1354.79 33.7885 1357.53 Q37.4488 1360.26 43.8463 1360.26 Q50.2438 1360.26 53.9359 1357.56 Q57.5962 1354.82 57.5962 1350.08 Q57.5962 1345.4 53.9041 1342.66 Q50.212 1339.92 43.8463 1339.92 Q37.5124 1339.92 33.8203 1342.66 Q30.0964 1345.4 30.0964 1350.08 M25.1311 1350.08 Q25.1311 1342.44 30.0964 1338.08 Q35.0616 1333.72 43.8463 1333.72 Q52.5991 1333.72 57.5962 1338.08 Q62.5615 1342.44 62.5615 1350.08 Q62.5615 1357.75 57.5962 1362.11 Q52.5991 1366.44 43.8463 1366.44 Q35.0616 1366.44 30.0964 1362.11 Q25.1311 1357.75 25.1311 1350.08 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M43.8463 1298.42 Q37.3851 1298.42 33.7248 1301.09 Q30.0327 1303.74 30.0327 1308.38 Q30.0327 1313.03 33.7248 1315.7 Q37.3851 1318.34 43.8463 1318.34 Q50.3075 1318.34 53.9996 1315.7 Q57.6599 1313.03 57.6599 1308.38 Q57.6599 1303.74 53.9996 1301.09 Q50.3075 1298.42 43.8463 1298.42 M31.4013 1318.34 Q28.2185 1316.5 26.6907 1313.7 Q25.1311 1310.86 25.1311 1306.95 Q25.1311 1300.46 30.2873 1296.41 Q35.4436 1292.34 43.8463 1292.34 Q52.249 1292.34 57.4052 1296.41 Q62.5615 1300.46 62.5615 1306.95 Q62.5615 1310.86 61.0337 1313.7 Q59.4741 1316.5 56.2912 1318.34 L61.6384 1318.34 L61.6384 1324.23 L12.1132 1324.23 L12.1132 1318.34 L31.4013 1318.34 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M43.719 1266.43 Q43.719 1273.53 45.3422 1276.27 Q46.9655 1279 50.8804 1279 Q53.9996 1279 55.8457 1276.97 Q57.6599 1274.9 57.6599 1271.37 Q57.6599 1266.5 54.2224 1263.57 Q50.7531 1260.61 45.0239 1260.61 L43.719 1260.61 L43.719 1266.43 M41.3 1254.75 L61.6384 1254.75 L61.6384 1260.61 L56.2276 1260.61 Q59.4741 1262.61 61.0337 1265.6 Q62.5615 1268.6 62.5615 1272.93 Q62.5615 1278.4 59.5059 1281.65 Q56.4186 1284.86 51.2623 1284.86 Q45.2467 1284.86 42.1912 1280.85 Q39.1357 1276.81 39.1357 1268.82 L39.1357 1260.61 L38.5628 1260.61 Q34.5205 1260.61 32.3244 1263.28 Q30.0964 1265.92 30.0964 1270.73 Q30.0964 1273.78 30.8284 1276.68 Q31.5605 1279.58 33.0246 1282.25 L27.6137 1282.25 Q26.3724 1279.04 25.7677 1276.01 Q25.1311 1272.99 25.1311 1270.12 Q25.1311 1262.39 29.1415 1258.57 Q33.1519 1254.75 41.3 1254.75 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M12.1132 1242.69 L12.1132 1236.83 L61.6384 1236.83 L61.6384 1242.69 L12.1132 1242.69 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M14.1184 1210.19 L14.1184 1169.99 L19.5293 1169.99 L19.5293 1186.86 L61.6384 1186.86 L61.6384 1193.32 L19.5293 1193.32 L19.5293 1210.19 L14.1184 1210.19 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M42.3504 1144.62 L45.2149 1144.62 L45.2149 1171.55 Q51.2623 1171.17 54.4452 1167.92 Q57.5962 1164.64 57.5962 1158.82 Q57.5962 1155.45 56.7687 1152.3 Q55.9411 1149.11 54.2861 1145.99 L59.8242 1145.99 Q61.161 1149.14 61.8612 1152.45 Q62.5615 1155.76 62.5615 1159.17 Q62.5615 1167.7 57.5962 1172.7 Q52.631 1177.66 44.1646 1177.66 Q35.4117 1177.66 30.2873 1172.95 Q25.1311 1168.21 25.1311 1160.19 Q25.1311 1153 29.7781 1148.83 Q34.3932 1144.62 42.3504 1144.62 M40.6316 1150.48 Q35.8255 1150.54 32.9609 1153.19 Q30.0964 1155.8 30.0964 1160.12 Q30.0964 1165.03 32.8654 1167.99 Q35.6345 1170.91 40.6634 1171.36 L40.6316 1150.48 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M32.8336 1107.26 Q28.8869 1105.06 27.009 1102.01 Q25.1311 1098.95 25.1311 1094.81 Q25.1311 1089.24 29.046 1086.22 Q32.9291 1083.2 40.1224 1083.2 L61.6384 1083.2 L61.6384 1089.08 L40.3133 1089.08 Q35.1889 1089.08 32.7063 1090.9 Q30.2237 1092.71 30.2237 1096.44 Q30.2237 1100.99 33.2474 1103.63 Q36.2711 1106.27 41.491 1106.27 L61.6384 1106.27 L61.6384 1112.16 L40.3133 1112.16 Q35.1571 1112.16 32.7063 1113.97 Q30.2237 1115.79 30.2237 1119.58 Q30.2237 1124.06 33.2792 1126.71 Q36.3029 1129.35 41.491 1129.35 L61.6384 1129.35 L61.6384 1135.24 L25.9905 1135.24 L25.9905 1129.35 L31.5287 1129.35 Q28.2503 1127.34 26.6907 1124.54 Q25.1311 1121.74 25.1311 1117.89 Q25.1311 1114.01 27.1045 1111.3 Q29.0779 1108.56 32.8336 1107.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M56.2912 1065.85 L75.1974 1065.85 L75.1974 1071.74 L25.9905 1071.74 L25.9905 1065.85 L31.4013 1065.85 Q28.2185 1064 26.6907 1061.2 Q25.1311 1058.37 25.1311 1054.45 Q25.1311 1047.96 30.2873 1043.92 Q35.4436 1039.84 43.8463 1039.84 Q52.249 1039.84 57.4052 1043.92 Q62.5615 1047.96 62.5615 1054.45 Q62.5615 1058.37 61.0337 1061.2 Q59.4741 1064 56.2912 1065.85 M43.8463 1045.92 Q37.3851 1045.92 33.7248 1048.6 Q30.0327 1051.24 30.0327 1055.89 Q30.0327 1060.53 33.7248 1063.21 Q37.3851 1065.85 43.8463 1065.85 Q50.3075 1065.85 53.9996 1063.21 Q57.6599 1060.53 57.6599 1055.89 Q57.6599 1051.24 53.9996 1048.6 Q50.3075 1045.92 43.8463 1045.92 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M42.3504 999.646 L45.2149 999.646 L45.2149 1026.57 Q51.2623 1026.19 54.4452 1022.94 Q57.5962 1019.67 57.5962 1013.84 Q57.5962 1010.47 56.7687 1007.32 Q55.9411 1004.13 54.2861 1001.01 L59.8242 1001.01 Q61.161 1004.17 61.8612 1007.48 Q62.5615 1010.79 62.5615 1014.19 Q62.5615 1022.72 57.5962 1027.72 Q52.631 1032.68 44.1646 1032.68 Q35.4117 1032.68 30.2873 1027.97 Q25.1311 1023.23 25.1311 1015.21 Q25.1311 1008.02 29.7781 1003.85 Q34.3932 999.646 42.3504 999.646 M40.6316 1005.5 Q35.8255 1005.57 32.9609 1008.21 Q30.0964 1010.82 30.0964 1015.15 Q30.0964 1020.05 32.8654 1023.01 Q35.6345 1025.94 40.6634 1026.38 L40.6316 1005.5 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M31.465 969.377 Q30.8921 970.363 30.6375 971.541 Q30.351 972.687 30.351 974.087 Q30.351 979.052 33.5975 981.726 Q36.8122 984.368 42.8596 984.368 L61.6384 984.368 L61.6384 990.256 L25.9905 990.256 L25.9905 984.368 L31.5287 984.368 Q28.2821 982.522 26.7225 979.562 Q25.1311 976.602 25.1311 972.368 Q25.1311 971.764 25.2266 971.032 Q25.2903 970.3 25.4494 969.408 L31.465 969.377 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M43.719 947.033 Q43.719 954.131 45.3422 956.868 Q46.9655 959.605 50.8804 959.605 Q53.9996 959.605 55.8457 957.568 Q57.6599 955.499 57.6599 951.966 Q57.6599 947.097 54.2224 944.168 Q50.7531 941.208 45.0239 941.208 L43.719 941.208 L43.719 947.033 M41.3 935.352 L61.6384 935.352 L61.6384 941.208 L56.2276 941.208 Q59.4741 943.214 61.0337 946.205 Q62.5615 949.197 62.5615 953.526 Q62.5615 959 59.5059 962.247 Q56.4186 965.462 51.2623 965.462 Q45.2467 965.462 42.1912 961.451 Q39.1357 957.409 39.1357 949.42 L39.1357 941.208 L38.5628 941.208 Q34.5205 941.208 32.3244 943.882 Q30.0964 946.524 30.0964 951.33 Q30.0964 954.385 30.8284 957.282 Q31.5605 960.178 33.0246 962.852 L27.6137 962.852 Q26.3724 959.637 25.7677 956.613 Q25.1311 953.59 25.1311 950.725 Q25.1311 942.991 29.1415 939.171 Q33.1519 935.352 41.3 935.352 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M15.869 917.496 L25.9905 917.496 L25.9905 905.433 L30.542 905.433 L30.542 917.496 L49.8937 917.496 Q54.2542 917.496 55.4955 916.318 Q56.7368 915.109 56.7368 911.449 L56.7368 905.433 L61.6384 905.433 L61.6384 911.449 Q61.6384 918.228 59.124 920.806 Q56.5777 923.384 49.8937 923.384 L30.542 923.384 L30.542 927.681 L25.9905 927.681 L25.9905 923.384 L15.869 923.384 L15.869 917.496 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M47.5702 898.335 L25.9905 898.335 L25.9905 892.479 L47.3474 892.479 Q52.4082 892.479 54.9545 890.505 Q57.4689 888.532 57.4689 884.585 Q57.4689 879.843 54.4452 877.106 Q51.4215 874.337 46.2016 874.337 L25.9905 874.337 L25.9905 868.48 L61.6384 868.48 L61.6384 874.337 L56.1639 874.337 Q59.4104 876.469 61.0019 879.302 Q62.5615 882.103 62.5615 885.827 Q62.5615 891.97 58.742 895.152 Q54.9226 898.335 47.5702 898.335 M25.1311 883.599 L25.1311 883.599 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M31.465 835.76 Q30.8921 836.747 30.6375 837.925 Q30.351 839.071 30.351 840.471 Q30.351 845.436 33.5975 848.11 Q36.8122 850.752 42.8596 850.752 L61.6384 850.752 L61.6384 856.64 L25.9905 856.64 L25.9905 850.752 L31.5287 850.752 Q28.2821 848.906 26.7225 845.946 Q25.1311 842.985 25.1311 838.752 Q25.1311 838.148 25.2266 837.415 Q25.2903 836.683 25.4494 835.792 L31.465 835.76 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M42.3504 800.558 L45.2149 800.558 L45.2149 827.485 Q51.2623 827.103 54.4452 823.857 Q57.5962 820.578 57.5962 814.754 Q57.5962 811.38 56.7687 808.229 Q55.9411 805.046 54.2861 801.927 L59.8242 801.927 Q61.161 805.078 61.8612 808.388 Q62.5615 811.698 62.5615 815.104 Q62.5615 823.634 57.5962 828.631 Q52.631 833.596 44.1646 833.596 Q35.4117 833.596 30.2873 828.885 Q25.1311 824.143 25.1311 816.122 Q25.1311 808.929 29.7781 804.759 Q34.3932 800.558 42.3504 800.558 M40.6316 806.414 Q35.8255 806.478 32.9609 809.12 Q30.0964 811.73 30.0964 816.059 Q30.0964 820.96 32.8654 823.92 Q35.6345 826.848 40.6634 827.294 L40.6316 806.414 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M20.4523 754.088 L44.1009 762.809 L44.1009 745.336 L20.4523 754.088 M14.1184 757.717 L14.1184 750.428 L61.6384 732.318 L61.6384 739.002 L49.4481 743.33 L49.4481 764.751 L61.6384 769.08 L61.6384 775.859 L14.1184 757.717 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M40.1224 696.001 L61.6384 696.001 L61.6384 701.858 L40.3133 701.858 Q35.2526 701.858 32.7381 703.831 Q30.2237 705.805 30.2237 709.751 Q30.2237 714.494 33.2474 717.231 Q36.2711 719.968 41.491 719.968 L61.6384 719.968 L61.6384 725.856 L25.9905 725.856 L25.9905 719.968 L31.5287 719.968 Q28.314 717.868 26.7225 715.035 Q25.1311 712.17 25.1311 708.446 Q25.1311 702.303 28.9505 699.152 Q32.7381 696.001 40.1224 696.001 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M30.0964 670.507 Q30.0964 675.217 33.7885 677.955 Q37.4488 680.692 43.8463 680.692 Q50.2438 680.692 53.9359 677.986 Q57.5962 675.249 57.5962 670.507 Q57.5962 665.828 53.9041 663.091 Q50.212 660.353 43.8463 660.353 Q37.5124 660.353 33.8203 663.091 Q30.0964 665.828 30.0964 670.507 M25.1311 670.507 Q25.1311 662.868 30.0964 658.507 Q35.0616 654.147 43.8463 654.147 Q52.5991 654.147 57.5962 658.507 Q62.5615 662.868 62.5615 670.507 Q62.5615 678.177 57.5962 682.538 Q52.5991 686.867 43.8463 686.867 Q35.0616 686.867 30.0964 682.538 Q25.1311 678.177 25.1311 670.507 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M32.8336 616.685 Q28.8869 614.488 27.009 611.433 Q25.1311 608.377 25.1311 604.24 Q25.1311 598.67 29.046 595.646 Q32.9291 592.622 40.1224 592.622 L61.6384 592.622 L61.6384 598.511 L40.3133 598.511 Q35.1889 598.511 32.7063 600.325 Q30.2237 602.139 30.2237 605.863 Q30.2237 610.414 33.2474 613.056 Q36.2711 615.698 41.491 615.698 L61.6384 615.698 L61.6384 621.586 L40.3133 621.586 Q35.1571 621.586 32.7063 623.4 Q30.2237 625.215 30.2237 629.002 Q30.2237 633.49 33.2792 636.132 Q36.3029 638.774 41.491 638.774 L61.6384 638.774 L61.6384 644.662 L25.9905 644.662 L25.9905 638.774 L31.5287 638.774 Q28.2503 636.768 26.6907 633.968 Q25.1311 631.167 25.1311 627.315 Q25.1311 623.432 27.1045 620.727 Q29.0779 617.99 32.8336 616.685 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M43.719 564.74 Q43.719 571.838 45.3422 574.575 Q46.9655 577.313 50.8804 577.313 Q53.9996 577.313 55.8457 575.276 Q57.6599 573.207 57.6599 569.674 Q57.6599 564.804 54.2224 561.876 Q50.7531 558.916 45.0239 558.916 L43.719 558.916 L43.719 564.74 M41.3 553.059 L61.6384 553.059 L61.6384 558.916 L56.2276 558.916 Q59.4741 560.921 61.0337 563.913 Q62.5615 566.905 62.5615 571.233 Q62.5615 576.708 59.5059 579.954 Q56.4186 583.169 51.2623 583.169 Q45.2467 583.169 42.1912 579.159 Q39.1357 575.117 39.1357 567.128 L39.1357 558.916 L38.5628 558.916 Q34.5205 558.916 32.3244 561.589 Q30.0964 564.231 30.0964 569.037 Q30.0964 572.093 30.8284 574.989 Q31.5605 577.886 33.0246 580.559 L27.6137 580.559 Q26.3724 577.345 25.7677 574.321 Q25.1311 571.297 25.1311 568.433 Q25.1311 560.698 29.1415 556.879 Q33.1519 553.059 41.3 553.059 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M12.1132 540.996 L12.1132 535.14 L61.6384 535.14 L61.6384 540.996 L12.1132 540.996 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M64.9486 508.054 Q71.3143 510.536 73.2559 512.892 Q75.1974 515.247 75.1974 519.194 L75.1974 523.873 L70.2958 523.873 L70.2958 520.435 Q70.2958 518.016 69.15 516.679 Q68.0041 515.343 63.7391 513.719 L61.0655 512.669 L25.9905 527.087 L25.9905 520.881 L53.8723 509.741 L25.9905 498.601 L25.9905 492.394 L64.9486 508.054 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M12.1769 449.521 Q19.4975 453.786 26.6589 455.855 Q33.8203 457.924 41.1727 457.924 Q48.5251 457.924 55.7502 455.855 Q62.9434 453.754 70.2321 449.521 L70.2321 454.614 Q62.7524 459.388 55.5274 461.775 Q48.3023 464.13 41.1727 464.13 Q34.0749 464.13 26.8817 461.775 Q19.6884 459.42 12.1769 454.614 L12.1769 449.521 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M17.365 428.005 Q17.365 430.551 19.1474 432.302 Q20.8979 434.052 23.4442 434.052 Q25.9587 434.052 27.7092 432.302 Q29.428 430.551 29.428 428.005 Q29.428 425.459 27.7092 423.708 Q25.9587 421.958 23.4442 421.958 Q20.9297 421.958 19.1474 423.74 Q17.365 425.491 17.365 428.005 M13.2591 428.005 Q13.2591 425.968 14.0548 424.09 Q14.8187 422.212 16.2828 420.844 Q17.7151 419.38 19.5293 418.647 Q21.3435 417.915 23.4442 417.915 Q27.6456 417.915 30.5738 420.875 Q33.4702 423.804 33.4702 428.069 Q33.4702 432.366 30.6056 435.23 Q27.7411 438.095 23.4442 438.095 Q19.1792 438.095 16.2191 435.166 Q13.2591 432.238 13.2591 428.005 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M17.7787 369.727 L24.5582 369.727 Q21.5345 372.974 20.0386 376.666 Q18.5426 380.326 18.5426 384.464 Q18.5426 392.612 23.5397 396.94 Q28.5049 401.269 37.9262 401.269 Q47.3156 401.269 52.3127 396.94 Q57.2779 392.612 57.2779 384.464 Q57.2779 380.326 55.782 376.666 Q54.2861 372.974 51.2623 369.727 L57.9782 369.727 Q60.2698 373.101 61.4156 376.888 Q62.5615 380.644 62.5615 384.846 Q62.5615 395.635 55.973 401.842 Q49.3526 408.049 37.9262 408.049 Q26.4679 408.049 19.8794 401.842 Q13.2591 395.635 13.2591 384.846 Q13.2591 380.581 14.4049 376.825 Q15.5189 373.037 17.7787 369.727 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip530)" d="M12.1769 360.974 L12.1769 355.882 Q19.6884 351.107 26.8817 348.752 Q34.0749 346.365 41.1727 346.365 Q48.3023 346.365 55.5274 348.752 Q62.7524 351.107 70.2321 355.882 L70.2321 360.974 Q62.9434 356.741 55.7502 354.672 Q48.5251 352.571 41.1727 352.571 Q33.8203 352.571 26.6589 354.672 Q19.4975 356.741 12.1769 360.974 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip532)" d="M294.951 2632.57 L295.179 5385.87 L295.407 3884.51 L295.634 3154.02 L295.862 2369.89 L296.09 1871.73 L296.318 1641.24 L296.545 1643.43 L296.773 1958.55 L297.001 1945.61 L297.229 1912.99 L297.457 1897.9 L297.684 1781.91 L297.912 1636.02 L298.14 1518.14 L298.368 1457.97 L298.595 1483.7 L298.823 1348.95 L299.051 1254.84 L299.279 1129.07 L299.506 1291.1 L299.734 1163.7 L299.962 1076.12 L300.19 1106.95 L300.418 1078.67 L300.645 1063.97 L300.873 1018.05 L301.101 1150.59 L301.329 1188.53 L301.556 1327.4 L301.784 1373.99 L302.012 1421.68 L302.24 1367.71 L302.468 1386.99 L302.695 1439.22 L302.923 1427.76 L303.151 1539.92 L303.379 1610.41 L303.606 1529.44 L303.834 1545.08 L304.062 1628.11 L304.29 1556.46 L304.517 1480.83 L304.745 1399.95 L304.973 1479.79 L305.201 1449.93 L305.429 1507.38 L305.656 1432.74 L305.884 1392.89 L306.112 1371.01 L306.34 1311.3 L306.567 1249.01 L306.795 1175.57 L307.023 1206.21 L307.251 1191.15 L307.478 1169.92 L307.706 1211.16 L307.934 1169.39 L308.162 1129.52 L308.39 1110.78 L308.617 1084.05 L308.845 1020.3 L309.073 968.693 L309.301 948.431 L309.528 997.587 L309.756 1023.9 L309.984 1127.44 L310.212 1126.21 L310.439 1094.77 L310.667 1048.77 L310.895 1122.17 L311.123 1171.02 L311.351 1107.08 L311.578 1059.04 L311.806 1027.73 L312.034 1032.68 L312.262 1025.79 L312.489 977.936 L312.717 997.516 L312.945 990.316 L313.173 967.045 L313.401 963.531 L313.628 1031.62 L313.856 1028.17 L314.084 1023.33 L314.312 1043.91 L314.539 1028.75 L314.767 1034.61 L314.995 1044.29 L315.223 1026.22 L315.45 1060.57 L315.678 1088.19 L315.906 1121.97 L316.134 1124.03 L316.362 1193.97 L316.589 1192.16 L316.817 1234.85 L317.045 1248.96 L317.273 1227.49 L317.5 1256.38 L317.728 1240.73 L317.956 1217.86 L318.184 1161.55 L318.411 1145.44 L318.639 1188.78 L318.867 1187.7 L319.095 1192.14 L319.323 1162.5 L319.55 1173.66 L319.778 1157.54 L320.006 1157.45 L320.234 1165.62 L320.461 1178.81 L320.689 1146.73 L320.917 1169.67 L321.145 1138.35 L321.372 1112.42 L321.6 1137.45 L321.828 1122.12 L322.056 1132.78 L322.284 1141.71 L322.511 1156.52 L322.739 1187.53 L322.967 1166.22 L323.195 1136.54 L323.422 1160.22 L323.65 1131.09 L323.878 1153.26 L324.106 1154.62 L324.334 1138 L324.561 1140.58 L324.789 1159.02 L325.017 1150.83 L325.245 1143.91 L325.472 1127.93 L325.7 1129.68 L325.928 1119.27 L326.156 1118.07 L326.383 1111.27 L326.611 1148.79 L326.839 1137.79 L327.067 1155.05 L327.295 1158.99 L327.522 1158.08 L327.75 1179.34 L327.978 1159.68 L328.206 1186.75 L328.433 1181.06 L328.661 1186.32 L328.889 1190.51 L329.117 1163.18 L329.344 1172.97 L329.572 1184.31 L329.8 1193.58 L330.028 1199.54 L330.256 1179.23 L330.483 1156.74 L330.711 1160.61 L330.939 1154.25 L331.167 1161.73 L331.394 1196.26 L331.622 1194.13 L331.85 1158.98 L332.078 1162.05 L332.305 1173.14 L332.533 1165.33 L332.761 1194.96 L332.989 1195.86 L333.217 1197.09 L333.444 1182.05 L333.672 1201.58 L333.9 1227.99 L334.128 1234.67 L334.355 1240.64 L334.583 1233.59 L334.811 1230.19 L335.039 1228.86 L335.267 1195.1 L335.494 1182.24 L335.722 1171.85 L335.95 1185.09 L336.178 1172.86 L336.405 1173.68 L336.633 1174.68 L336.861 1183.71 L337.089 1185.69 L337.316 1155.56 L337.544 1171.01 L337.772 1167.83 L338 1176.11 L338.228 1155.46 L338.455 1155.19 L338.683 1155.08 L338.911 1140.93 L339.139 1156.16 L339.366 1182.98 L339.594 1179.32 L339.822 1173.29 L340.05 1171.72 L340.277 1198.19 L340.505 1195.17 L340.733 1208.61 L340.961 1225.03 L341.189 1261.62 L341.416 1278.19 L341.644 1303.36 L341.872 1309.15 L342.1 1327.39 L342.327 1331.11 L342.555 1348.65 L342.783 1340.93 L343.011 1348.74 L343.238 1354.11 L343.466 1371.26 L343.694 1379.94 L343.922 1374.85 L344.15 1376.19 L344.377 1399.56 L344.605 1394.52 L344.833 1394.72 L345.061 1387.82 L345.288 1368.21 L345.516 1379.83 L345.744 1388.33 L345.972 1371.14 L346.2 1370.2 L346.427 1379.86 L346.655 1370.33 L346.883 1367.35 L347.111 1372.92 L347.338 1379.68 L347.566 1377.34 L347.794 1379.42 L348.022 1366.68 L348.249 1363.76 L348.477 1364.93 L348.705 1376.3 L348.933 1384.31 L349.161 1388.72 L349.388 1391.44 L349.616 1401.36 L349.844 1409.59 L350.072 1411.49 L350.299 1411.16 L350.527 1415.98 L350.755 1428.77 L350.983 1437.74 L351.21 1446.02 L351.438 1450.88 L351.666 1450.08 L351.894 1440.05 L352.122 1431.67 L352.349 1435.58 L352.577 1425.81 L352.805 1410.24 L353.033 1397.91 L353.26 1379.01 L353.488 1367.28 L353.716 1372.76 L353.944 1372.13 L354.171 1384.88 L354.399 1395.07 L354.627 1386.59 L354.855 1377.14 L355.083 1378.69 L355.31 1357.41 L355.538 1358.26 L355.766 1350.73 L355.994 1357.68 L356.221 1352.82 L356.449 1351.93 L356.677 1349.18 L356.905 1345.18 L357.133 1335.73 L357.36 1333.6 L357.588 1323.4 L357.816 1323.26 L358.044 1309.47 L358.271 1326.97 L358.499 1321.91 L358.727 1313.87 L358.955 1313.48 L359.182 1320.88 L359.41 1322.35 L359.638 1340.17 L359.866 1330.68 L360.094 1341.97 L360.321 1339.43 L360.549 1340.52 L360.777 1353.38 L361.005 1348.14 L361.232 1358.87 L361.46 1353.88 L361.688 1355.95 L361.916 1352.09 L362.143 1361.22 L362.371 1364.8 L362.599 1371.3 L362.827 1362.64 L363.055 1353.64 L363.282 1356.36 L363.51 1341.66 L363.738 1342.27 L363.966 1333.93 L364.193 1340.23 L364.421 1345.08 L364.649 1355.63 L364.877 1361.13 L365.104 1367.04 L365.332 1357.88 L365.56 1360.66 L365.788 1356.32 L366.016 1358.54 L366.243 1358.24 L366.471 1354.51 L366.699 1358.53 L366.927 1366.79 L367.154 1365.54 L367.382 1360.82 L367.61 1361.56 L367.838 1355.29 L368.066 1358.58 L368.293 1350.28 L368.521 1349.74 L368.749 1352.83 L368.977 1335.16 L369.204 1332.28 L369.432 1328.69 L369.66 1333.85 L369.888 1340.42 L370.115 1334.88 L370.343 1323.6 L370.571 1319.6 L370.799 1323.68 L371.027 1324.9 L371.254 1321.64 L371.482 1297.48 L371.71 1294.04 L371.938 1290.14 L372.165 1268.61 L372.393 1277.84 L372.621 1284.4 L372.849 1280.92 L373.076 1283.43 L373.304 1266.29 L373.532 1251.58 L373.76 1248.21 L373.988 1243.13 L374.215 1253.2 L374.443 1251.18 L374.671 1251.98 L374.899 1249.11 L375.126 1237.6 L375.354 1227.42 L375.582 1229.33 L375.81 1234.77 L376.037 1233 L376.265 1231.52 L376.493 1208.19 L376.721 1198.67 L376.949 1192.74 L377.176 1193.62 L377.404 1192.82 L377.632 1195.87 L377.86 1198.96 L378.087 1194.33 L378.315 1185.9 L378.543 1185.34 L378.771 1179.81 L378.999 1181.44 L379.226 1185.74 L379.454 1189.6 L379.682 1182.03 L379.91 1190.51 L380.137 1191.77 L380.365 1192.26 L380.593 1191.21 L380.821 1198.28 L381.048 1202.46 L381.276 1211.17 L381.504 1217.41 L381.732 1230.44 L381.96 1234.28 L382.187 1231.33 L382.415 1237.85 L382.643 1246.39 L382.871 1259.65 L383.098 1247.51 L383.326 1230.03 L383.554 1229.33 L383.782 1235.23 L384.009 1242.67 L384.237 1234.06 L384.465 1232.89 L384.693 1240.97 L384.921 1242.09 L385.148 1239.1 L385.376 1232.71 L385.604 1229.67 L385.832 1226.89 L386.059 1217.04 L386.287 1222.83 L386.515 1235.02 L386.743 1241.01 L386.97 1239.41 L387.198 1239.06 L387.426 1233.2 L387.654 1223.64 L387.882 1224.51 L388.109 1231.49 L388.337 1237.02 L388.565 1230.88 L388.793 1233.87 L389.02 1231.3 L389.248 1233 L389.476 1218.27 L389.704 1214.47 L389.932 1218.39 L390.159 1220.03 L390.387 1224.92 L390.615 1229.8 L390.843 1228.09 L391.07 1226.61 L391.298 1229.63 L391.526 1231.33 L391.754 1221.91 L391.981 1223.87 L392.209 1231.58 L392.437 1236.13 L392.665 1246.25 L392.893 1244.44 L393.12 1245.52 L393.348 1237.08 L393.576 1232.57 L393.804 1229.77 L394.031 1235.93 L394.259 1242.21 L394.487 1245.27 L394.715 1256.05 L394.942 1261.88 L395.17 1269.05 L395.398 1270.3 L395.626 1270.9 L395.854 1276.87 L396.081 1270.2 L396.309 1264.94 L396.537 1266.62 L396.765 1266.65 L396.992 1254.18 L397.22 1254.75 L397.448 1249.36 L397.676 1253.42 L397.903 1258.09 L398.131 1262.49 L398.359 1271.57 L398.587 1278.76 L398.815 1272.14 L399.042 1263.59 L399.27 1260.98 L399.498 1271.93 L399.726 1275.49 L399.953 1272.14 L400.181 1280.61 L400.409 1281.94 L400.637 1281.1 L400.865 1289.66 L401.092 1294.57 L401.32 1286.42 L401.548 1285.43 L401.776 1287.91 L402.003 1297.93 L402.231 1294.83 L402.459 1285.66 L402.687 1287.96 L402.914 1295.56 L403.142 1295.01 L403.37 1295.76 L403.598 1296.52 L403.826 1305.22 L404.053 1308.86 L404.281 1310.58 L404.509 1308.38 L404.737 1307.81 L404.964 1310.12 L405.192 1312.03 L405.42 1321 L405.648 1318.18 L405.875 1315.74 L406.103 1320.85 L406.331 1324.07 L406.559 1326.29 L406.787 1334.62 L407.014 1334.17 L407.242 1335.7 L407.47 1336.67 L407.698 1328.29 L407.925 1329.59 L408.153 1340.06 L408.381 1353.78 L408.609 1362.49 L408.836 1365.14 L409.064 1364.45 L409.292 1360.51 L409.52 1365.96 L409.748 1373.31 L409.975 1379.6 L410.203 1378.46 L410.431 1371.38 L410.659 1368.11 L410.886 1357.8 L411.114 1363.52 L411.342 1366.34 L411.57 1371.83 L411.798 1362.6 L412.025 1353.07 L412.253 1353.92 L412.481 1355.7 L412.709 1350.27 L412.936 1344.18 L413.164 1340.69 L413.392 1336.5 L413.62 1335.9 L413.847 1339.25 L414.075 1335.76 L414.303 1330.57 L414.531 1331.73 L414.759 1322.98 L414.986 1320.26 L415.214 1318.96 L415.442 1323.8 L415.67 1318.56 L415.897 1312.36 L416.125 1307.9 L416.353 1308.37 L416.581 1317 L416.808 1322.9 L417.036 1320.32 L417.264 1316.47 L417.492 1323.38 L417.72 1332.63 L417.947 1338.08 L418.175 1342.02 L418.403 1341.93 L418.631 1341.5 L418.858 1337.4 L419.086 1335.86 L419.314 1335.67 L419.542 1330.37 L419.769 1338.01 L419.997 1336.03 L420.225 1337.76 L420.453 1340.46 L420.681 1341.73 L420.908 1344.34 L421.136 1336.13 L421.364 1333.07 L421.592 1323.53 L421.819 1322.15 L422.047 1328.33 L422.275 1333.02 L422.503 1333.22 L422.731 1328.89 L422.958 1334.41 L423.186 1330.74 L423.414 1323.34 L423.642 1329.76 L423.869 1328.19 L424.097 1321.08 L424.325 1326.39 L424.553 1322.44 L424.78 1312.21 L425.008 1316.52 L425.236 1305.94 L425.464 1310.47 L425.692 1296.4 L425.919 1291.14 L426.147 1287.58 L426.375 1281.68 L426.603 1283.97 L426.83 1283.91 L427.058 1290.07 L427.286 1287.86 L427.514 1293.12 L427.741 1291.2 L427.969 1292.98 L428.197 1286.65 L428.425 1287.22 L428.653 1284.87 L428.88 1286.11 L429.108 1286.14 L429.336 1278.87 L429.564 1282.29 L429.791 1279.03 L430.019 1282.85 L430.247 1282.92 L430.475 1274.98 L430.702 1276.27 L430.93 1270.98 L431.158 1270.19 L431.386 1271.17 L431.614 1276.44 L431.841 1281.96 L432.069 1270.4 L432.297 1273.55 L432.525 1274.6 L432.752 1275.9 L432.98 1267.57 L433.208 1268.04 L433.436 1265.03 L433.664 1269.83 L433.891 1273.02 L434.119 1269.34 L434.347 1264.34 L434.575 1260.18 L434.802 1262.44 L435.03 1262.05 L435.258 1256.33 L435.486 1245.79 L435.713 1247.13 L435.941 1249.75 L436.169 1248.06 L436.397 1243.07 L436.625 1239.29 L436.852 1236.94 L437.08 1230.53 L437.308 1223.46 L437.536 1224.56 L437.763 1225.81 L437.991 1222.19 L438.219 1226.51 L438.447 1223.67 L438.674 1226.25 L438.902 1225.28 L439.13 1227.35 L439.358 1237.01 L439.586 1237.08 L439.813 1235.26 L440.041 1233.29 L440.269 1233.62 L440.497 1237.94 L440.724 1233.6 L440.952 1237 L441.18 1241.26 L441.408 1232.25 L441.635 1231.94 L441.863 1237.97 L442.091 1236.37 L442.319 1236.63 L442.547 1235.72 L442.774 1228.12 L443.002 1228.94 L443.23 1226.17 L443.458 1226.53 L443.685 1227.35 L443.913 1229.35 L444.141 1228.88 L444.369 1223.04 L444.597 1226.27 L444.824 1230.5 L445.052 1227.58 L445.28 1227.22 L445.508 1230.55 L445.735 1233.51 L445.963 1233.63 L446.191 1236.41 L446.419 1236.16 L446.646 1237.06 L446.874 1235.59 L447.102 1234.44 L447.33 1230.44 L447.558 1233.3 L447.785 1227.93 L448.013 1232.67 L448.241 1237.93 L448.469 1235.14 L448.696 1240.14 L448.924 1237.41 L449.152 1234.66 L449.38 1233.79 L449.607 1237.51 L449.835 1234.01 L450.063 1230.93 L450.291 1232.5 L450.519 1232.16 L450.746 1228.32 L450.974 1225.83 L451.202 1227.56 L451.43 1224.54 L451.657 1229 L451.885 1232.16 L452.113 1229.57 L452.341 1228.51 L452.568 1227.78 L452.796 1232.8 L453.024 1233.92 L453.252 1237.56 L453.48 1230.78 L453.707 1220.53 L453.935 1223.22 L454.163 1223.07 L454.391 1218.21 L454.618 1215.71 L454.846 1217.7 L455.074 1222.69 L455.302 1221.01 L455.53 1219.68 L455.757 1221.91 L455.985 1221.96 L456.213 1225 L456.441 1227.82 L456.668 1226.15 L456.896 1224.85 L457.124 1227.26 L457.352 1225.91 L457.579 1226.23 L457.807 1232.24 L458.035 1226.58 L458.263 1222.45 L458.491 1222.53 L458.718 1217.98 L458.946 1221.74 L459.174 1226.12 L459.402 1228.42 L459.629 1229.04 L459.857 1231.56 L460.085 1234.35 L460.313 1230.9 L460.54 1229.01 L460.768 1228.96 L460.996 1224.29 L461.224 1230.86 L461.452 1231.57 L461.679 1227.67 L461.907 1221.83 L462.135 1215.56 L462.363 1215.54 L462.59 1221.36 L462.818 1215.28 L463.046 1213.18 L463.274 1214.34 L463.501 1210.41 L463.729 1209.14 L463.957 1209.06 L464.185 1205.23 L464.413 1209.25 L464.64 1210.61 L464.868 1210.86 L465.096 1211.61 L465.324 1210.14 L465.551 1206.18 L465.779 1207.29 L466.007 1203.3 L466.235 1200.55 L466.462 1204.61 L466.69 1198.06 L466.918 1191.46 L467.146 1192.11 L467.374 1189.8 L467.601 1184.44 L467.829 1183.44 L468.057 1180.69 L468.285 1179.19 L468.512 1182.57 L468.74 1176.97 L468.968 1180.75 L469.196 1178.28 L469.424 1182.63 L469.651 1180.96 L469.879 1178.86 L470.107 1176.32 L470.335 1171.24 L470.562 1172.94 L470.79 1176.73 L471.018 1175.57 L471.246 1171.47 L471.473 1177.47 L471.701 1183.15 L471.929 1174.7 L472.157 1179.97 L472.385 1180.93 L472.612 1183.17 L472.84 1183.26 L473.068 1184.56 L473.296 1187.92 L473.523 1180.95 L473.751 1174.2 L473.979 1163.94 L474.207 1166.48 L474.434 1168.37 L474.662 1167.06 L474.89 1167.17 L475.118 1166.12 L475.346 1163.83 L475.573 1159.42 L475.801 1159.96 L476.029 1159.96 L476.257 1163.25 L476.484 1165.14 L476.712 1166.81 L476.94 1166.23 L477.168 1167.25 L477.395 1166.7 L477.623 1165.13 L477.851 1161.14 L478.079 1162.69 L478.307 1157.32 L478.534 1156.95 L478.762 1164.32 L478.99 1170.28 L479.218 1172.47 L479.445 1171.32 L479.673 1168.64 L479.901 1167.91 L480.129 1172.29 L480.357 1172.21 L480.584 1169.09 L480.812 1171.85 L481.04 1173.75 L481.268 1172.95 L481.495 1175.8 L481.723 1175.37 L481.951 1168.22 L482.179 1170.35 L482.406 1174.61 L482.634 1178.55 L482.862 1170.38 L483.09 1172.21 L483.318 1174.88 L483.545 1175.84 L483.773 1175.52 L484.001 1172.39 L484.229 1178.2 L484.456 1177.93 L484.684 1180.38 L484.912 1180.53 L485.14 1182.75 L485.367 1178.82 L485.595 1178.85 L485.823 1181.39 L486.051 1181.36 L486.279 1183.55 L486.506 1187.61 L486.734 1188.92 L486.962 1188.53 L487.19 1185.33 L487.417 1182.99 L487.645 1182.57 L487.873 1182.05 L488.101 1185.1 L488.328 1186.67 L488.556 1188.68 L488.784 1190.18 L489.012 1193.08 L489.24 1193.11 L489.467 1192.01 L489.695 1188.22 L489.923 1192.99 L490.151 1191.45 L490.378 1191.74 L490.606 1188.67 L490.834 1192.39 L491.062 1193.56 L491.29 1190.57 L491.517 1191.09 L491.745 1185.34 L491.973 1185.09 L492.201 1181.84 L492.428 1182.06 L492.656 1185.1 L492.884 1179.59 L493.112 1177.25 L493.339 1173.17 L493.567 1170.36 L493.795 1170.66 L494.023 1173.25 L494.251 1175.63 L494.478 1175.09 L494.706 1176.5 L494.934 1176.56 L495.162 1173.59 L495.389 1172.03 L495.617 1171.03 L495.845 1171.33 L496.073 1172.64 L496.3 1175.07 L496.528 1172.74 L496.756 1173.2 L496.984 1170.3 L497.212 1170.65 L497.439 1170.96 L497.667 1172.82 L497.895 1175.5 L498.123 1176.14 L498.35 1175.22 L498.578 1175.4 L498.806 1172.42 L499.034 1174.23 L499.261 1173.89 L499.489 1176.37 L499.717 1178.22 L499.945 1178.6 L500.173 1176.83 L500.4 1175.24 L500.628 1174.98 L500.856 1172.57 L501.084 1170.24 L501.311 1170.31 L501.539 1172.04 L501.767 1171.84 L501.995 1171.14 L502.223 1173.89 L502.45 1176.32 L502.678 1174.91 L502.906 1171.28 L503.134 1169.38 L503.361 1171.67 L503.589 1172.61 L503.817 1178.12 L504.045 1176.3 L504.272 1173.91 L504.5 1176.18 L504.728 1175.55 L504.956 1176.27 L505.184 1178.16 L505.411 1175.54 L505.639 1175.21 L505.867 1177.46 L506.095 1176.48 L506.322 1170.08 L506.55 1171.07 L506.778 1171.21 L507.006 1171 L507.233 1174.88 L507.461 1176.1 L507.689 1175.59 L507.917 1176.6 L508.145 1170.91 L508.372 1169.4 L508.6 1164.25 L508.828 1165.21 L509.056 1164.68 L509.283 1163.71 L509.511 1164.43 L509.739 1159.35 L509.967 1158.61 L510.194 1158.85 L510.422 1163.58 L510.65 1160.99 L510.878 1164.11 L511.106 1164.18 L511.333 1163.01 L511.561 1163.95 L511.789 1163.51 L512.017 1163.67 L512.244 1164.58 L512.472 1169.04 L512.7 1172.31 L512.928 1173.41 L513.156 1170.63 L513.383 1169.78 L513.611 1170.37 L513.839 1168.87 L514.067 1169 L514.294 1165.82 L514.522 1169.33 L514.75 1166.33 L514.978 1168.4 L515.205 1169.39 L515.433 1169.41 L515.661 1171.09 L515.889 1170.17 L516.117 1170.52 L516.344 1173.73 L516.572 1174.73 L516.8 1174.82 L517.028 1176.07 L517.255 1177.68 L517.483 1168.34 L517.711 1169.78 L517.939 1169.77 L518.166 1170.99 L518.394 1171.92 L518.622 1174.74 L518.85 1174.62 L519.078 1174.23 L519.305 1174 L519.533 1171.5 L519.761 1173.22 L519.989 1176.12 L520.216 1176.62 L520.444 1175.38 L520.672 1173.81 L520.9 1174.16 L521.127 1174.09 L521.355 1179.25 L521.583 1174.5 L521.811 1174.95 L522.039 1174.12 L522.266 1170.69 L522.494 1172.84 L522.722 1172.68 L522.95 1167.38 L523.177 1166.29 L523.405 1166.22 L523.633 1161.38 L523.861 1161.57 L524.089 1160.66 L524.316 1162.19 L524.544 1164 L524.772 1165.65 L525 1165.11 L525.227 1166.05 L525.455 1169.17 L525.683 1170.22 L525.911 1169.16 L526.138 1170.08 L526.366 1167.19 L526.594 1166.08 L526.822 1161.79 L527.05 1157.15 L527.277 1160.21 L527.505 1159.45 L527.733 1162.34 L527.961 1163.07 L528.188 1166.5 L528.416 1168.86 L528.644 1167.15 L528.872 1165.86 L529.099 1164.29 L529.327 1166.76 L529.555 1165.12 L529.783 1166.01 L530.011 1168.3 L530.238 1170.93 L530.466 1169.57 L530.694 1174.83 L530.922 1178.45 L531.149 1180.11 L531.377 1183.43 L531.605 1184.4 L531.833 1185.75 L532.06 1188.42 L532.288 1188.5 L532.516 1190.92 L532.744 1196.74 L532.972 1199.46 L533.199 1200.65 L533.427 1201.49 L533.655 1200.19 L533.883 1201.6 L534.11 1201.92 L534.338 1202.1 L534.566 1201.89 L534.794 1202.9 L535.022 1203.18 L535.249 1204 L535.477 1204.17 L535.705 1201.64 L535.933 1204.07 L536.16 1206.23 L536.388 1204.95 L536.616 1206.34 L536.844 1206.54 L537.071 1211.68 L537.299 1212.35 L537.527 1213.44 L537.755 1217.05 L537.983 1211.35 L538.21 1209.26 L538.438 1211.35 L538.666 1208.78 L538.894 1214.89 L539.121 1212.79 L539.349 1214.97 L539.577 1216.17 L539.805 1217.32 L540.032 1214.1 L540.26 1212.85 L540.488 1217.9 L540.716 1218.61 L540.944 1219.57 L541.171 1218.02 L541.399 1214.27 L541.627 1213.49 L541.855 1212.08 L542.082 1210.03 L542.31 1208.15 L542.538 1210.35 L542.766 1211.32 L542.993 1209.51 L543.221 1211.88 L543.449 1203.83 L543.677 1205.95 L543.905 1206.95 L544.132 1208.85 L544.36 1208.06 L544.588 1209.79 L544.816 1208.47 L545.043 1206.9 L545.271 1209.84 L545.499 1206.36 L545.727 1200.25 L545.955 1201.01 L546.182 1199.94 L546.41 1197.18 L546.638 1193.26 L546.866 1189.11 L547.093 1191.74 L547.321 1196.35 L547.549 1194.83 L547.777 1194.58 L548.004 1193.52 L548.232 1197.07 L548.46 1197.08 L548.688 1197.21 L548.916 1194 L549.143 1194.15 L549.371 1196.25 L549.599 1194.95 L549.827 1194.17 L550.054 1195.89 L550.282 1198.48 L550.51 1201.67 L550.738 1196.61 L550.965 1197.95 L551.193 1197.75 L551.421 1199.25 L551.649 1197.34 L551.877 1200.08 L552.104 1198.59 L552.332 1200.83 L552.56 1204.1 L552.788 1206.01 L553.015 1204.89 L553.243 1206.76 L553.471 1207.29 L553.699 1204.97 L553.926 1205.59 L554.154 1210.64 L554.382 1211.46 L554.61 1211.84 L554.838 1212.8 L555.065 1213.1 L555.293 1210.25 L555.521 1209.77 L555.749 1208.99 L555.976 1206.91 L556.204 1205.65 L556.432 1202.25 L556.66 1195.95 L556.888 1189.34 L557.115 1190.37 L557.343 1190.87 L557.571 1190.25 L557.799 1188.14 L558.026 1190.29 L558.254 1195.02 L558.482 1193.6 L558.71 1191.76 L558.937 1190.76 L559.165 1192.48 L559.393 1194.07 L559.621 1196.65 L559.849 1193.58 L560.076 1194.6 L560.304 1197.72 L560.532 1196.77 L560.76 1195.64 L560.987 1192.96 L561.215 1190.17 L561.443 1189.36 L561.671 1192.81 L561.898 1196.56 L562.126 1195.18 L562.354 1194.73 L562.582 1197.28 L562.81 1194.33 L563.037 1198.57 L563.265 1194.32 L563.493 1195.55 L563.721 1195.58 L563.948 1195.16 L564.176 1197.36 L564.404 1193.13 L564.632 1193.01 L564.859 1189.8 L565.087 1193.76 L565.315 1192.83 L565.543 1190.34 L565.771 1192.64 L565.998 1191.39 L566.226 1188.24 L566.454 1185.68 L566.682 1184.97 L566.909 1183.64 L567.137 1181.87 L567.365 1183.73 L567.593 1184.21 L567.821 1184.27 L568.048 1185.06 L568.276 1184.86 L568.504 1185.43 L568.732 1182.61 L568.959 1180.66 L569.187 1174.49 L569.415 1175.33 L569.643 1172.83 L569.87 1171.67 L570.098 1170.64 L570.326 1169.93 L570.554 1168.05 L570.782 1167.11 L571.009 1169.3 L571.237 1169.75 L571.465 1168.81 L571.693 1165.47 L571.92 1165.76 L572.148 1163.16 L572.376 1159.09 L572.604 1159.69 L572.831 1160.94 L573.059 1162.25 L573.287 1163.03 L573.515 1164.57 L573.743 1167.83 L573.97 1169.4 L574.198 1171.7 L574.426 1171.56 L574.654 1171.62 L574.881 1169.1 L575.109 1167.7 L575.337 1166.62 L575.565 1163.96 L575.792 1164.29 L576.02 1165.92 L576.248 1167.18 L576.476 1169.76 L576.704 1170.72 L576.931 1169.84 L577.159 1171.07 L577.387 1170.51 L577.615 1169.88 L577.842 1166.11 L578.07 1164.83 L578.298 1166.53 L578.526 1165.53 L578.754 1166.91 L578.981 1164.58 L579.209 1168.63 L579.437 1172.33 L579.665 1169.82 L579.892 1169.1 L580.12 1166.23 L580.348 1166.14 L580.576 1167.52 L580.803 1169.29 L581.031 1168.41 L581.259 1170.91 L581.487 1168.85 L581.715 1167.39 L581.942 1166.44 L582.17 1165.9 L582.398 1166 L582.626 1169.54 L582.853 1170.3 L583.081 1172.47 L583.309 1172.87 L583.537 1173.09 L583.764 1174.32 L583.992 1173.79 L584.22 1172.8 L584.448 1171.32 L584.676 1171.2 L584.903 1171.08 L585.131 1172.46 L585.359 1174.13 L585.587 1172.45 L585.814 1172.33 L586.042 1169.97 L586.27 1172.4 L586.498 1173.61 L586.725 1177.31 L586.953 1179.96 L587.181 1182.98 L587.409 1181.76 L587.637 1180.68 L587.864 1180.28 L588.092 1177.67 L588.32 1175.53 L588.548 1176.13 L588.775 1177.71 L589.003 1179.41 L589.231 1178.72 L589.459 1181.23 L589.687 1183.33 L589.914 1181.28 L590.142 1175.97 L590.37 1179.26 L590.598 1180.33 L590.825 1179.73 L591.053 1180.3 L591.281 1180.21 L591.509 1179.07 L591.736 1177.52 L591.964 1178.82 L592.192 1174.88 L592.42 1174.9 L592.648 1176.45 L592.875 1176.77 L593.103 1175.81 L593.331 1174.65 L593.559 1173.35 L593.786 1174.96 L594.014 1177.46 L594.242 1178.16 L594.47 1178.39 L594.697 1179.73 L594.925 1176.5 L595.153 1173.81 L595.381 1173.86 L595.609 1174.91 L595.836 1174.94 L596.064 1174.72 L596.292 1176.59 L596.52 1176.87 L596.747 1175.38 L596.975 1173.86 L597.203 1174.49 L597.431 1175.08 L597.658 1177.32 L597.886 1179.13 L598.114 1180.43 L598.342 1178.11 L598.57 1181.01 L598.797 1178.8 L599.025 1176.98 L599.253 1177.04 L599.481 1177.85 L599.708 1177.36 L599.936 1178.95 L600.164 1180.35 L600.392 1179.48 L600.62 1178.65 L600.847 1174.89 L601.075 1174.84 L601.303 1175.2 L601.531 1174 L601.758 1176 L601.986 1177 L602.214 1176 L602.442 1179.45 L602.669 1180.15 L602.897 1183.67 L603.125 1184.66 L603.353 1185.79 L603.581 1184.91 L603.808 1184.99 L604.036 1183.38 L604.264 1181.72 L604.492 1180.84 L604.719 1182.3 L604.947 1183.87 L605.175 1184.33 L605.403 1185.44 L605.63 1185.79 L605.858 1184.02 L606.086 1184.51 L606.314 1184.39 L606.542 1186.42 L606.769 1186.69 L606.997 1183.02 L607.225 1182.49 L607.453 1186.65 L607.68 1184.04 L607.908 1181.35 L608.136 1180.37 L608.364 1182.6 L608.591 1185.14 L608.819 1186.92 L609.047 1184.05 L609.275 1182.84 L609.503 1181.83 L609.73 1178.76 L609.958 1177.93 L610.186 1176.46 L610.414 1177.83 L610.641 1175.5 L610.869 1176.69 L611.097 1178.28 L611.325 1181.78 L611.553 1179.39 L611.78 1180.39 L612.008 1178.86 L612.236 1179.22 L612.464 1179.68 L612.691 1182.79 L612.919 1180.33 L613.147 1178.18 L613.375 1177.89 L613.602 1177.49 L613.83 1175.72 L614.058 1171.86 L614.286 1169.53 L614.514 1169.97 L614.741 1170.49 L614.969 1171.41 L615.197 1172.53 L615.425 1170.81 L615.652 1170.09 L615.88 1170.42 L616.108 1167.73 L616.336 1167.37 L616.563 1166.41 L616.791 1169.15 L617.019 1166.9 L617.247 1169.41 L617.475 1167.72 L617.702 1168.1 L617.93 1168.28 L618.158 1167.56 L618.386 1168.77 L618.613 1167.05 L618.841 1164.56 L619.069 1163.22 L619.297 1160.2 L619.524 1160.67 L619.752 1160.91 L619.98 1162.69 L620.208 1161.37 L620.436 1158.87 L620.663 1155.02 L620.891 1154.91 L621.119 1153.69 L621.347 1148.77 L621.574 1148.4 L621.802 1149.26 L622.03 1148.34 L622.258 1150.87 L622.486 1149.78 L622.713 1152.19 L622.941 1150.5 L623.169 1151.73 L623.397 1151.92 L623.624 1154.3 L623.852 1153.13 L624.08 1156.31 L624.308 1155.8 L624.535 1156.62 L624.763 1156.49 L624.991 1155.36 L625.219 1152.21 L625.447 1153.75 L625.674 1155.77 L625.902 1156.64 L626.13 1157.75 L626.358 1157.09 L626.585 1156.69 L626.813 1153.52 L627.041 1154.92 L627.269 1156.3 L627.496 1155.14 L627.724 1153.72 L627.952 1153.41 L628.18 1156.7 L628.408 1156.75 L628.635 1157.07 L628.863 1158.93 L629.091 1163.24 L629.319 1167.1 L629.546 1168.04 L629.774 1170.65 L630.002 1172.75 L630.23 1174.05 L630.457 1172.87 L630.685 1172.71 L630.913 1172.99 L631.141 1173.86 L631.369 1171.91 L631.596 1172.03 L631.824 1172.57 L632.052 1174.49 L632.28 1178.67 L632.507 1176.81 L632.735 1174.07 L632.963 1174.63 L633.191 1174.15 L633.419 1174.99 L633.646 1176.48 L633.874 1175.95 L634.102 1175.68 L634.33 1176.6 L634.557 1179.5 L634.785 1178.27 L635.013 1178.21 L635.241 1179.36 L635.468 1177.9 L635.696 1177.42 L635.924 1178.19 L636.152 1178.11 L636.38 1174.6 L636.607 1172.77 L636.835 1173.59 L637.063 1176.4 L637.291 1178.53 L637.518 1179.19 L637.746 1179 L637.974 1179.42 L638.202 1178.55 L638.429 1180.66 L638.657 1181.36 L638.885 1182.69 L639.113 1183.12 L639.341 1183.29 L639.568 1182.13 L639.796 1180.48 L640.024 1178.82 L640.252 1177.11 L640.479 1175.97 L640.707 1178.33 L640.935 1180.41 L641.163 1181.01 L641.39 1181.89 L641.618 1184.02 L641.846 1182.03 L642.074 1183.23 L642.302 1185.58 L642.529 1186.9 L642.757 1186.86 L642.985 1182.73 L643.213 1185.51 L643.44 1184.46 L643.668 1183.1 L643.896 1184.81 L644.124 1186.03 L644.352 1185.21 L644.579 1186.68 L644.807 1187.51 L645.035 1187.31 L645.263 1187.29 L645.49 1188.19 L645.718 1187.44 L645.946 1187.23 L646.174 1188.54 L646.401 1189.87 L646.629 1185.2 L646.857 1185.32 L647.085 1182.1 L647.313 1183.28 L647.54 1183.48 L647.768 1182.25 L647.996 1182.99 L648.224 1180.48 L648.451 1181.88 L648.679 1182.25 L648.907 1180.16 L649.135 1179.24 L649.362 1181.7 L649.59 1183.43 L649.818 1184.63 L650.046 1184.11 L650.274 1185.08 L650.501 1185.24 L650.729 1185.24 L650.957 1186.17 L651.185 1185.89 L651.412 1186.16 L651.64 1187.98 L651.868 1188.64 L652.096 1189.31 L652.323 1189.41 L652.551 1190.79 L652.779 1186.39 L653.007 1186.75 L653.235 1185.03 L653.462 1182.49 L653.69 1184.17 L653.918 1184.43 L654.146 1185.14 L654.373 1187.27 L654.601 1187.97 L654.829 1189.32 L655.057 1191.93 L655.285 1191.73 L655.512 1193.97 L655.74 1195.82 L655.968 1194.03 L656.196 1194.41 L656.423 1195.17 L656.651 1196.8 L656.879 1197.69 L657.107 1199 L657.334 1200.84 L657.562 1199.28 L657.79 1200.81 L658.018 1200.89 L658.246 1197.89 L658.473 1198.05 L658.701 1196.94 L658.929 1197.05 L659.157 1199.39 L659.384 1200.75 L659.612 1200.82 L659.84 1199.93 L660.068 1201.06 L660.295 1202.24 L660.523 1202.69 L660.751 1202.04 L660.979 1206.11 L661.207 1204.02 L661.434 1205.63 L661.662 1205.61 L661.89 1204.78 L662.118 1201.14 L662.345 1203.15 L662.573 1200.44 L662.801 1199.16 L663.029 1198.39 L663.256 1198.57 L663.484 1197.4 L663.712 1195.92 L663.94 1196.52 L664.168 1197.95 L664.395 1196.55 L664.623 1197.94 L664.851 1199.7 L665.079 1200.79 L665.306 1200.89 L665.534 1200.57 L665.762 1199.63 L665.99 1198.84 L666.218 1196.09 L666.445 1193.98 L666.673 1192.58 L666.901 1190.83 L667.129 1191.7 L667.356 1189.15 L667.584 1187.61 L667.812 1186.61 L668.04 1185.42 L668.267 1185.6 L668.495 1186.17 L668.723 1185.25 L668.951 1183.59 L669.179 1185.24 L669.406 1184.1 L669.634 1183.24 L669.862 1183.54 L670.09 1182.78 L670.317 1182.76 L670.545 1180.27 L670.773 1179.38 L671.001 1179.18 L671.228 1178.18 L671.456 1176.05 L671.684 1177.74 L671.912 1180.36 L672.14 1180.94 L672.367 1179.84 L672.595 1182.11 L672.823 1183.01 L673.051 1182.69 L673.278 1181.1 L673.506 1181.88 L673.734 1181.84 L673.962 1179.68 L674.189 1177.27 L674.417 1179.06 L674.645 1181.07 L674.873 1178.79 L675.101 1180.86 L675.328 1183.46 L675.556 1182.66 L675.784 1183.51 L676.012 1183.13 L676.239 1180.01 L676.467 1180.4 L676.695 1178.1 L676.923 1177.38 L677.151 1177.5 L677.378 1177.86 L677.606 1179.41 L677.834 1178.62 L678.062 1179.68 L678.289 1181.02 L678.517 1181.84 L678.745 1181.78 L678.973 1180.83 L679.2 1182.96 L679.428 1184.31 L679.656 1183.61 L679.884 1184.12 L680.112 1183.75 L680.339 1184.25 L680.567 1183.44 L680.795 1185.39 L681.023 1180.72 L681.25 1182.38 L681.478 1181.59 L681.706 1181.23 L681.934 1179.78 L682.161 1177.59 L682.389 1177.49 L682.617 1176.37 L682.845 1178.17 L683.073 1178.5 L683.3 1180.87 L683.528 1177.75 L683.756 1179.89 L683.984 1181.17 L684.211 1181.64 L684.439 1182.65 L684.667 1181.86 L684.895 1183.46 L685.122 1181.73 L685.35 1182.56 L685.578 1182.95 L685.806 1183.9 L686.034 1185.13 L686.261 1186.75 L686.489 1184.27 L686.717 1186.58 L686.945 1186.23 L687.172 1185.76 L687.4 1186.1 L687.628 1186.36 L687.856 1185.71 L688.084 1184.43 L688.311 1182.85 L688.539 1183.76 L688.767 1181.63 L688.995 1182.86 L689.222 1183.03 L689.45 1183.5 L689.678 1182.15 L689.906 1182.95 L690.133 1181.77 L690.361 1182.82 L690.589 1181.96 L690.817 1181.73 L691.045 1181.64 L691.272 1181.1 L691.5 1179.46 L691.728 1180.89 L691.956 1181.52 L692.183 1181.55 L692.411 1181.19 L692.639 1180.69 L692.867 1180.27 L693.094 1180.43 L693.322 1182.32 L693.55 1183.33 L693.778 1183.13 L694.006 1182.52 L694.233 1182.33 L694.461 1181.28 L694.689 1181.36 L694.917 1180.87 L695.144 1176.77 L695.372 1178.58 L695.6 1175.64 L695.828 1176.57 L696.055 1176.31 L696.283 1176.11 L696.511 1175.41 L696.739 1175.93 L696.967 1175.22 L697.194 1173.41 L697.422 1170.12 L697.65 1170.38 L697.878 1168.08 L698.105 1169.62 L698.333 1168 L698.561 1168.05 L698.789 1166.28 L699.017 1166.66 L699.244 1167.47 L699.472 1168.11 L699.7 1166.15 L699.928 1166.85 L700.155 1168.61 L700.383 1170.14 L700.611 1172.54 L700.839 1170.35 L701.066 1169.51 L701.294 1170.13 L701.522 1167.62 L701.75 1169.23 L701.978 1169.58 L702.205 1166.92 L702.433 1167.32 L702.661 1166.78 L702.889 1167.49 L703.116 1168.91 L703.344 1168.51 L703.572 1167.94 L703.8 1168.06 L704.027 1167.89 L704.255 1165.88 L704.483 1166.52 L704.711 1166.61 L704.939 1165.99 L705.166 1166.12 L705.394 1167.31 L705.622 1169.43 L705.85 1170.24 L706.077 1170.42 L706.305 1170.42 L706.533 1169.79 L706.761 1168.68 L706.988 1168.06 L707.216 1169.23 L707.444 1167.57 L707.672 1165.98 L707.9 1165.16 L708.127 1166.05 L708.355 1166.38 L708.583 1166.42 L708.811 1166.45 L709.038 1167.51 L709.266 1165.02 L709.494 1164.9 L709.722 1164.31 L709.95 1164.84 L710.177 1165.59 L710.405 1167.89 L710.633 1167.63 L710.861 1168.56 L711.088 1167.94 L711.316 1169.88 L711.544 1170.24 L711.772 1172.31 L711.999 1170.4 L712.227 1171.5 L712.455 1172.09 L712.683 1174.95 L712.911 1174.76 L713.138 1174.84 L713.366 1176.11 L713.594 1176.86 L713.822 1177.16 L714.049 1179.01 L714.277 1179.35 L714.505 1178.3 L714.733 1178.09 L714.96 1175.97 L715.188 1175.17 L715.416 1175.2 L715.644 1176.33 L715.872 1177.85 L716.099 1180.08 L716.327 1179.78 L716.555 1178.47 L716.783 1178.72 L717.01 1179.43 L717.238 1178.99 L717.466 1178.29 L717.694 1179.39 L717.921 1178.31 L718.149 1177.79 L718.377 1177.58 L718.605 1177.95 L718.833 1175.17 L719.06 1174.52 L719.288 1170.07 L719.516 1167.93 L719.744 1166.71 L719.971 1167.77 L720.199 1167.99 L720.427 1168.24 L720.655 1168.15 L720.883 1169.8 L721.11 1172.59 L721.338 1171.48 L721.566 1167.82 L721.794 1165.65 L722.021 1165.81 L722.249 1166.83 L722.477 1166.83 L722.705 1167.85 L722.932 1168.43 L723.16 1169.49 L723.388 1171.87 L723.616 1172.4 L723.844 1174.66 L724.071 1175.2 L724.299 1176.27 L724.527 1177.58 L724.755 1178.29 L724.982 1176.51 L725.21 1177.48 L725.438 1178.98 L725.666 1178.58 L725.893 1180.95 L726.121 1182.66 L726.349 1184.13 L726.577 1182.45 L726.805 1182.4 L727.032 1183.9 L727.26 1183.96 L727.488 1184.1 L727.716 1183.31 L727.943 1182.62 L728.171 1183.04 L728.399 1184.8 L728.627 1182.43 L728.854 1181.16 L729.082 1181.77 L729.31 1183.72 L729.538 1183.71 L729.766 1184.6 L729.993 1184.42 L730.221 1184.79 L730.449 1185.09 L730.677 1183.37 L730.904 1182.58 L731.132 1182.43 L731.36 1181.14 L731.588 1181.08 L731.816 1181.83 L732.043 1182.04 L732.271 1184.85 L732.499 1183.76 L732.727 1185.62 L732.954 1188.09 L733.182 1187.48 L733.41 1189.32 L733.638 1192.01 L733.865 1190.05 L734.093 1190.47 L734.321 1191.77 L734.549 1191.62 L734.777 1191.24 L735.004 1189.6 L735.232 1188.31 L735.46 1186.94 L735.688 1185.45 L735.915 1185.08 L736.143 1183.39 L736.371 1183.19 L736.599 1184.85 L736.826 1185.34 L737.054 1185.22 L737.282 1186.32 L737.51 1187.5 L737.738 1185.48 L737.965 1184.79 L738.193 1183.34 L738.421 1185.53 L738.649 1186 L738.876 1187.78 L739.104 1186.97 L739.332 1187.6 L739.56 1189.34 L739.787 1186.29 L740.015 1187.1 L740.243 1184.38 L740.471 1184.66 L740.699 1184.58 L740.926 1182.27 L741.154 1182.06 L741.382 1180.95 L741.61 1181.53 L741.837 1183.41 L742.065 1185.22 L742.293 1186.2 L742.521 1187.82 L742.749 1186.56 L742.976 1186.65 L743.204 1187.85 L743.432 1189.23 L743.66 1188.1 L743.887 1190.1 L744.115 1188.26 L744.343 1188.84 L744.571 1189.61 L744.798 1191.58 L745.026 1191.11 L745.254 1189.65 L745.482 1189.96 L745.71 1189.43 L745.937 1189.35 L746.165 1186.99 L746.393 1187.24 L746.621 1187.86 L746.848 1186.34 L747.076 1186.89 L747.304 1184.83 L747.532 1185.67 L747.759 1185.11 L747.987 1183.31 L748.215 1183.31 L748.443 1183.74 L748.671 1182.45 L748.898 1182.88 L749.126 1183.05 L749.354 1182.91 L749.582 1181.55 L749.809 1180.22 L750.037 1180.74 L750.265 1180.35 L750.493 1179.81 L750.72 1181.83 L750.948 1179.08 L751.176 1178.2 L751.404 1177.46 L751.632 1176.94 L751.859 1176.54 L752.087 1176.54 L752.315 1177.92 L752.543 1175.83 L752.77 1176.13 L752.998 1174.01 L753.226 1173.83 L753.454 1175.22 L753.682 1175.93 L753.909 1176.08 L754.137 1175.19 L754.365 1176.66 L754.593 1176.74 L754.82 1176.28 L755.048 1177.02 L755.276 1178.58 L755.504 1176.24 L755.731 1176.71 L755.959 1178.19 L756.187 1176.53 L756.415 1177.4 L756.643 1177.37 L756.87 1176.64 L757.098 1175.62 L757.326 1176.29 L757.554 1173.89 L757.781 1175.08 L758.009 1176.43 L758.237 1175.96 L758.465 1175.86 L758.692 1173.77 L758.92 1171.68 L759.148 1171.72 L759.376 1172.78 L759.604 1172.34 L759.831 1172.89 L760.059 1172.29 L760.287 1172.99 L760.515 1174.3 L760.742 1176.03 L760.97 1177.84 L761.198 1178.9 L761.426 1179.35 L761.653 1180.08 L761.881 1179.09 L762.109 1177.28 L762.337 1179.25 L762.565 1181.07 L762.792 1177.99 L763.02 1177.56 L763.248 1180.11 L763.476 1178.78 L763.703 1178.53 L763.931 1176.34 L764.159 1178.04 L764.387 1176.87 L764.615 1175.96 L764.842 1175.18 L765.07 1173.86 L765.298 1174.89 L765.526 1176.41 L765.753 1176.64 L765.981 1176.13 L766.209 1175.96 L766.437 1175.79 L766.664 1177.86 L766.892 1176.77 L767.12 1176.09 L767.348 1175.38 L767.576 1174.2 L767.803 1174.62 L768.031 1175.67 L768.259 1177.55 L768.487 1176.74 L768.714 1175.72 L768.942 1176.68 L769.17 1178.45 L769.398 1180.66 L769.625 1180.46 L769.853 1180.56 L770.081 1181.32 L770.309 1182.4 L770.537 1181.72 L770.764 1179.51 L770.992 1179.6 L771.22 1179.44 L771.448 1177.86 L771.675 1178.2 L771.903 1180.05 L772.131 1178.84 L772.359 1177.86 L772.586 1177.98 L772.814 1180.04 L773.042 1182.34 L773.27 1180.16 L773.498 1181.54 L773.725 1183.15 L773.953 1182.72 L774.181 1182.62 L774.409 1184.69 L774.636 1185.84 L774.864 1186.53 L775.092 1186.29 L775.32 1187.27 L775.548 1186.77 L775.775 1185.53 L776.003 1184.87 L776.231 1186.13 L776.459 1185.58 L776.686 1187.55 L776.914 1187.91 L777.142 1187.28 L777.37 1186.91 L777.597 1188.42 L777.825 1191.1 L778.053 1188.51 L778.281 1187.4 L778.509 1185.2 L778.736 1185.29 L778.964 1182.96 L779.192 1182.47 L779.42 1180.97 L779.647 1180.21 L779.875 1181.25 L780.103 1180.91 L780.331 1181.26 L780.558 1178.39 L780.786 1176.17 L781.014 1176.15 L781.242 1175.47 L781.47 1174.84 L781.697 1172.98 L781.925 1173.93 L782.153 1176.02 L782.381 1176.33 L782.608 1174.7 L782.836 1173.92 L783.064 1174.7 L783.292 1173.8 L783.519 1172.06 L783.747 1172.27 L783.975 1170.85 L784.203 1170.96 L784.431 1172.14 L784.658 1172.32 L784.886 1171.21 L785.114 1172.26 L785.342 1171.94 L785.569 1170.84 L785.797 1170.76 L786.025 1171.92 L786.253 1169.39 L786.481 1168.98 L786.708 1167.28 L786.936 1165.87 L787.164 1167.83 L787.392 1167.38 L787.619 1167.57 L787.847 1165.4 L788.075 1165.95 L788.303 1165.52 L788.53 1164.13 L788.758 1165.79 L788.986 1165.68 L789.214 1165.19 L789.442 1166.78 L789.669 1166.06 L789.897 1166.57 L790.125 1165.19 L790.353 1165.36 L790.58 1162.7 L790.808 1164.9 L791.036 1165.57 L791.264 1167.24 L791.491 1167.76 L791.719 1168.03 L791.947 1168.15 L792.175 1167.61 L792.403 1167.45 L792.63 1167.79 L792.858 1168.86 L793.086 1167.57 L793.314 1167.84 L793.541 1168.05 L793.769 1170.07 L793.997 1170.88 L794.225 1170.12 L794.452 1170.51 L794.68 1170.41 L794.908 1167.82 L795.136 1170.19 L795.364 1169.74 L795.591 1168.43 L795.819 1168.43 L796.047 1165.95 L796.275 1167.19 L796.502 1165.32 L796.73 1165.44 L796.958 1166.77 L797.186 1167.55 L797.413 1168.67 L797.641 1167.41 L797.869 1167.61 L798.097 1168.92 L798.325 1170.02 L798.552 1170.4 L798.78 1172.03 L799.008 1172.65 L799.236 1171.3 L799.463 1170.91 L799.691 1170.95 L799.919 1170.59 L800.147 1169.51 L800.375 1169.96 L800.602 1170.15 L800.83 1170.41 L801.058 1170.39 L801.286 1170.62 L801.513 1170.65 L801.741 1171.95 L801.969 1172.05 L802.197 1170.31 L802.424 1169.72 L802.652 1169.98 L802.88 1169.31 L803.108 1169.71 L803.336 1170.27 L803.563 1169.72 L803.791 1169.4 L804.019 1169.88 L804.247 1169.35 L804.474 1168.52 L804.702 1170.09 L804.93 1172.32 L805.158 1172.56 L805.385 1172.23 L805.613 1170.64 L805.841 1168.71 L806.069 1169.16 L806.297 1167.3 L806.524 1168.54 L806.752 1167.36 L806.98 1167.18 L807.208 1165.74 L807.435 1164.51 L807.663 1164.08 L807.891 1163.77 L808.119 1163.41 L808.346 1162.22 L808.574 1162.34 L808.802 1161.31 L809.03 1161.55 L809.258 1159.44 L809.485 1157.71 L809.713 1154.75 L809.941 1156.05 L810.169 1157 L810.396 1155.72 L810.624 1156.57 L810.852 1157.51 L811.08 1158.99 L811.308 1160.86 L811.535 1159.56 L811.763 1160.52 L811.991 1161.76 L812.219 1162.89 L812.446 1162.72 L812.674 1163.93 L812.902 1162.32 L813.13 1163.71 L813.357 1161.87 L813.585 1161.11 L813.813 1160.72 L814.041 1161.68 L814.269 1164.19 L814.496 1164.55 L814.724 1164.54 L814.952 1165.85 L815.18 1165.98 L815.407 1165.92 L815.635 1167.67 L815.863 1168.75 L816.091 1168.12 L816.318 1168.82 L816.546 1167.84 L816.774 1168.13 L817.002 1168.97 L817.23 1168.97 L817.457 1169.46 L817.685 1169.29 L817.913 1167.76 L818.141 1169 L818.368 1169.8 L818.596 1169.7 L818.824 1168.53 L819.052 1169.81 L819.279 1169.87 L819.507 1168.83 L819.735 1168.5 L819.963 1169.91 L820.191 1168.03 L820.418 1167.28 L820.646 1168.32 L820.874 1167.58 L821.102 1166.66 L821.329 1166.28 L821.557 1164.62 L821.785 1163.54 L822.013 1162.83 L822.241 1162.48 L822.468 1161.66 L822.696 1161.79 L822.924 1162.51 L823.152 1162.44 L823.379 1164.83 L823.607 1164.49 L823.835 1164.16 L824.063 1164.15 L824.29 1163.05 L824.518 1163.13 L824.746 1162.89 L824.974 1161.69 L825.202 1161.3 L825.429 1159.47 L825.657 1159.99 L825.885 1160.59 L826.113 1158.88 L826.34 1159.46 L826.568 1159.08 L826.796 1159.78 L827.024 1159.31 L827.251 1159.3 L827.479 1158.96 L827.707 1159.05 L827.935 1160.3 L828.163 1161.57 L828.39 1162.46 L828.618 1162.4 L828.846 1163.11 L829.074 1164.42 L829.301 1164.52 L829.529 1165.31 L829.757 1167.24 L829.985 1167.52 L830.212 1167.82 L830.44 1167.44 L830.668 1166.73 L830.896 1166.7 L831.124 1167.27 L831.351 1167.02 L831.579 1166.02 L831.807 1165.26 L832.035 1165.31 L832.262 1163.92 L832.49 1163.97 L832.718 1164.47 L832.946 1163.91 L833.174 1164.02 L833.401 1165.35 L833.629 1163.17 L833.857 1163.16 L834.085 1163.55 L834.312 1162.8 L834.54 1163.29 L834.768 1162.87 L834.996 1162.98 L835.223 1162.6 L835.451 1162.98 L835.679 1161.89 L835.907 1162.01 L836.135 1160.67 L836.362 1160.69 L836.59 1161.29 L836.818 1159.82 L837.046 1159.7 L837.273 1159.92 L837.501 1159.81 L837.729 1161.56 L837.957 1163.01 L838.184 1163.86 L838.412 1163.76 L838.64 1163.21 L838.868 1162.66 L839.096 1163.67 L839.323 1164.33 L839.551 1164.62 L839.779 1166.15 L840.007 1164.69 L840.234 1164.1 L840.462 1164.44 L840.69 1164.74 L840.918 1163.27 L841.145 1164.14 L841.373 1164.54 L841.601 1163.48 L841.829 1164.97 L842.057 1166.63 L842.284 1167.22 L842.512 1166.31 L842.74 1165.11 L842.968 1163.57 L843.195 1161.66 L843.423 1162.14 L843.651 1162.09 L843.879 1162.14 L844.107 1162.1 L844.334 1161.88 L844.562 1162.91 L844.79 1163.08 L845.018 1163.11 L845.245 1160.39 L845.473 1161.27 L845.701 1162.15 L845.929 1161.73 L846.156 1160.97 L846.384 1160.8 L846.612 1162.19 L846.84 1163.97 L847.068 1164.33 L847.295 1165.14 L847.523 1164.7 L847.751 1163.1 L847.979 1160.95 L848.206 1161.59 L848.434 1161.84 L848.662 1160.96 L848.89 1160.49 L849.117 1160.74 L849.345 1161.4 L849.573 1162.25 L849.801 1162.6 L850.029 1162.55 L850.256 1161.64 L850.484 1160.25 L850.712 1161.11 L850.94 1160.38 L851.167 1160.49 L851.395 1160.61 L851.623 1162.06 L851.851 1162.85 L852.078 1162.41 L852.306 1164.16 L852.534 1162.86 L852.762 1163.55 L852.99 1163.03 L853.217 1163.55 L853.445 1163.47 L853.673 1164.28 L853.901 1164.28 L854.128 1164.12 L854.356 1162.93 L854.584 1161.4 L854.812 1161.54 L855.04 1161.24 L855.267 1162.35 L855.495 1160.34 L855.723 1159.85 L855.951 1161.02 L856.178 1163.06 L856.406 1162.32 L856.634 1161.18 L856.862 1161.02 L857.089 1161.37 L857.317 1163.13 L857.545 1164.25 L857.773 1165 L858.001 1164.74 L858.228 1164.25 L858.456 1161.23 L858.684 1163.09 L858.912 1163.47 L859.139 1162.48 L859.367 1161.88 L859.595 1161.64 L859.823 1162.96 L860.05 1162 L860.278 1162.72 L860.506 1162.64 L860.734 1164.07 L860.962 1163.53 L861.189 1164.23 L861.417 1164.59 L861.645 1162.97 L861.873 1163.93 L862.1 1163.75 L862.328 1164.32 L862.556 1164.42 L862.784 1164.05 L863.011 1162.8 L863.239 1162.61 L863.467 1161.91 L863.695 1161.51 L863.923 1162.21 L864.15 1164.07 L864.378 1162.38 L864.606 1160.86 L864.834 1161.25 L865.061 1162.09 L865.289 1161.81 L865.517 1161.35 L865.745 1162.05 L865.973 1161.71 L866.2 1162.16 L866.428 1161.6 L866.656 1162.3 L866.884 1163.04 L867.111 1162.34 L867.339 1162.76 L867.567 1163.71 L867.795 1163.63 L868.022 1163.97 L868.25 1163.83 L868.478 1164.1 L868.706 1164.91 L868.934 1162.84 L869.161 1160.98 L869.389 1160.26 L869.617 1158.87 L869.845 1158.48 L870.072 1158.14 L870.3 1158.86 L870.528 1158.55 L870.756 1159.14 L870.983 1160.23 L871.211 1160.27 L871.439 1161.95 L871.667 1161.79 L871.895 1163.69 L872.122 1162.8 L872.35 1163.04 L872.578 1162.23 L872.806 1160.77 L873.033 1161.75 L873.261 1161.97 L873.489 1163.62 L873.717 1163.51 L873.944 1163.49 L874.172 1162.02 L874.4 1161.34 L874.628 1160.86 L874.856 1160.29 L875.083 1160.31 L875.311 1162.54 L875.539 1163.12 L875.767 1164.06 L875.994 1163.72 L876.222 1161.98 L876.45 1160.88 L876.678 1159.9 L876.906 1160.73 L877.133 1160.59 L877.361 1160.91 L877.589 1161.5 L877.817 1161.03 L878.044 1160.68 L878.272 1161.07 L878.5 1160.41 L878.728 1159.7 L878.955 1160.92 L879.183 1160.77 L879.411 1159.89 L879.639 1159.83 L879.867 1161.36 L880.094 1161.89 L880.322 1162.87 L880.55 1162.41 L880.778 1162.75 L881.005 1163.78 L881.233 1165.57 L881.461 1165.71 L881.689 1166.31 L881.916 1165.29 L882.144 1164.1 L882.372 1162.58 L882.6 1161.91 L882.828 1160.6 L883.055 1160.12 L883.283 1159.68 L883.511 1157.35 L883.739 1156.24 L883.966 1157.33 L884.194 1158.23 L884.422 1159.42 L884.65 1160.39 L884.877 1160.02 L885.105 1160.11 L885.333 1161.44 L885.561 1163.37 L885.789 1164.9 L886.016 1164.31 L886.244 1163.18 L886.472 1161.79 L886.7 1162.22 L886.927 1163.16 L887.155 1163.61 L887.383 1165.13 L887.611 1166.08 L887.839 1167.38 L888.066 1168.75 L888.294 1169.22 L888.522 1168.92 L888.75 1168.47 L888.977 1170.37 L889.205 1169.74 L889.433 1170.52 L889.661 1171.3 L889.888 1171.45 L890.116 1171.67 L890.344 1172.09 L890.572 1171.8 L890.8 1170.84 L891.027 1170.97 L891.255 1168.21 L891.483 1167.82 L891.711 1167.73 L891.938 1167.31 L892.166 1167.64 L892.394 1166.9 L892.622 1167.07 L892.849 1167.57 L893.077 1165.51 L893.305 1164.97 L893.533 1164.29 L893.761 1163.82 L893.988 1164.24 L894.216 1164.24 L894.444 1164.64 L894.672 1164.86 L894.899 1165.98 L895.127 1164.8 L895.355 1165.04 L895.583 1166.16 L895.81 1164.85 L896.038 1164.66 L896.266 1165.1 L896.494 1166.64 L896.722 1167.47 L896.949 1166.03 L897.177 1165.85 L897.405 1163.95 L897.633 1163.51 L897.86 1162.52 L898.088 1162.7 L898.316 1163.79 L898.544 1164.07 L898.772 1163.48 L898.999 1162.72 L899.227 1161.96 L899.455 1163.03 L899.683 1165.19 L899.91 1164.26 L900.138 1164.49 L900.366 1163.35 L900.594 1161.11 L900.821 1161.74 L901.049 1163.32 L901.277 1162.56 L901.505 1163.02 L901.733 1164.3 L901.96 1165.04 L902.188 1165.32 L902.416 1166.16 L902.644 1165.58 L902.871 1165.33 L903.099 1166.28 L903.327 1167.77 L903.555 1168.86 L903.782 1169.14 L904.01 1169.07 L904.238 1168.92 L904.466 1168.35 L904.694 1167.9 L904.921 1169.28 L905.149 1169.99 L905.377 1169.52 L905.605 1169.72 L905.832 1169.17 L906.06 1169.04 L906.288 1168.69 L906.516 1168.67 L906.743 1169.86 L906.971 1170.15 L907.199 1171.41 L907.427 1172.98 L907.655 1174.96 L907.882 1175.74 L908.11 1175.54 L908.338 1173.1 L908.566 1171.78 L908.793 1173.06 L909.021 1172.69 L909.249 1171.94 L909.477 1172.75 L909.705 1172.52 L909.932 1174.17 L910.16 1172.95 L910.388 1172.72 L910.616 1172.78 L910.843 1173.84 L911.071 1174.47 L911.299 1174.87 L911.527 1174.18 L911.754 1175.02 L911.982 1176.06 L912.21 1176.28 L912.438 1176.43 L912.666 1177.55 L912.893 1177.16 L913.121 1176.94 L913.349 1177.04 L913.577 1177.67 L913.804 1179 L914.032 1178.71 L914.26 1178.01 L914.488 1179.08 L914.715 1178.85 L914.943 1178.64 L915.171 1177.24 L915.399 1176.72 L915.627 1176.82 L915.854 1176.05 L916.082 1176.89 L916.31 1176.9 L916.538 1177.43 L916.765 1177.8 L916.993 1176.84 L917.221 1175.07 L917.449 1174.71 L917.676 1173.71 L917.904 1173.18 L918.132 1173.45 L918.36 1172.9 L918.588 1172.64 L918.815 1170.35 L919.043 1170.11 L919.271 1171.61 L919.499 1171.43 L919.726 1171.36 L919.954 1171.68 L920.182 1172.8 L920.41 1173.08 L920.638 1171.01 L920.865 1171.24 L921.093 1171.93 L921.321 1173.45 L921.549 1172.39 L921.776 1172.98 L922.004 1173.19 L922.232 1173.55 L922.46 1173.28 L922.687 1173.16 L922.915 1173.62 L923.143 1173.22 L923.371 1172.97 L923.599 1174.2 L923.826 1175.32 L924.054 1176.78 L924.282 1176.98 L924.51 1177.45 L924.737 1176.32 L924.965 1176.91 L925.193 1176.25 L925.421 1175.56 L925.648 1176.33 L925.876 1175.49 L926.104 1174.73 L926.332 1175.86 L926.56 1174.67 L926.787 1174.01 L927.015 1174.35 L927.243 1174.14 L927.471 1175.59 L927.698 1175.96 L927.926 1174.58 L928.154 1175.75 L928.382 1174.68 L928.609 1172.22 L928.837 1173.18 L929.065 1173.84 L929.293 1174.07 L929.521 1174.25 L929.748 1174.62 L929.976 1176.27 L930.204 1177.21 L930.432 1176.35 L930.659 1177.27 L930.887 1176.23 L931.115 1175.95 L931.343 1173.94 L931.571 1173.01 L931.798 1174.1 L932.026 1175.17 L932.254 1175.72 L932.482 1174.82 L932.709 1173.04 L932.937 1174.77 L933.165 1174.65 L933.393 1174.95 L933.62 1176.16 L933.848 1176.51 L934.076 1177.15 L934.304 1177.17 L934.532 1177.35 L934.759 1176.3 L934.987 1176.64 L935.215 1176.44 L935.443 1173.54 L935.67 1174.16 L935.898 1174.66 L936.126 1174.33 L936.354 1172.36 L936.581 1171.99 L936.809 1171.72 L937.037 1171.48 L937.265 1172.85 L937.493 1171.98 L937.72 1171.23 L937.948 1169.98 L938.176 1168.56 L938.404 1167.88 L938.631 1167.87 L938.859 1168.97 L939.087 1168.22 L939.315 1168.88 L939.542 1168.44 L939.77 1169.13 L939.998 1168.93 L940.226 1169.96 L940.454 1169.06 L940.681 1167.77 L940.909 1168.44 L941.137 1167.59 L941.365 1166.87 L941.592 1166.44 L941.82 1166.65 L942.048 1166.89 L942.276 1166.64 L942.504 1166.44 L942.731 1167.05 L942.959 1166.8 L943.187 1168.33 L943.415 1166.3 L943.642 1166.49 L943.87 1167.15 L944.098 1165.21 L944.326 1166.9 L944.553 1165.46 L944.781 1165.15 L945.009 1166.08 L945.237 1166.41 L945.465 1164.95 L945.692 1163.16 L945.92 1162.48 L946.148 1162.83 L946.376 1162.76 L946.603 1162.6 L946.831 1160.61 L947.059 1161.66 L947.287 1161.32 L947.514 1160.45 L947.742 1160.77 L947.97 1161.13 L948.198 1161.05 L948.426 1160.55 L948.653 1159.73 L948.881 1158.49 L949.109 1156 L949.337 1155.77 L949.564 1156.23 L949.792 1155.14 L950.02 1154.78 L950.248 1154.96 L950.475 1153.48 L950.703 1153.03 L950.931 1153.91 L951.159 1154.12 L951.387 1154.71 L951.614 1154.97 L951.842 1154.75 L952.07 1154.14 L952.298 1154.33 L952.525 1153.45 L952.753 1152.53 L952.981 1152.3 L953.209 1153.82 L953.437 1153.41 L953.664 1154.01 L953.892 1153.13 L954.12 1153.1 L954.348 1151.76 L954.575 1151.96 L954.803 1151.9 L955.031 1152.52 L955.259 1152.32 L955.486 1151.29 L955.714 1151.67 L955.942 1152.77 L956.17 1153.09 L956.398 1151.75 L956.625 1152.28 L956.853 1152.61 L957.081 1152.04 L957.309 1150.11 L957.536 1150.56 L957.764 1151.61 L957.992 1152.24 L958.22 1152.76 L958.447 1151.33 L958.675 1149.68 L958.903 1149.99 L959.131 1149.67 L959.359 1149.52 L959.586 1148.38 L959.814 1148.24 L960.042 1148.84 L960.27 1149.13 L960.497 1148.77 L960.725 1149.64 L960.953 1149.12 L961.181 1148.63 L961.408 1147.99 L961.636 1147.18 L961.864 1147.71 L962.092 1147.1 L962.32 1146.21 L962.547 1146.8 L962.775 1147.62 L963.003 1147.57 L963.231 1147.2 L963.458 1147.51 L963.686 1146.6 L963.914 1147.25 L964.142 1147.31 L964.37 1148.14 L964.597 1147.43 L964.825 1147.37 L965.053 1147.2 L965.281 1147.14 L965.508 1147.35 L965.736 1147.41 L965.964 1146.78 L966.192 1147.33 L966.419 1148.04 L966.647 1147.36 L966.875 1148.26 L967.103 1147.64 L967.331 1147.85 L967.558 1148.28 L967.786 1148.8 L968.014 1148.03 L968.242 1148.64 L968.469 1149.96 L968.697 1148.96 L968.925 1148.3 L969.153 1148.92 L969.38 1148.09 L969.608 1148.64 L969.836 1148.56 L970.064 1149.81 L970.292 1150.25 L970.519 1149.8 L970.747 1149.63 L970.975 1149.47 L971.203 1148.9 L971.43 1149.29 L971.658 1150.28 L971.886 1148.33 L972.114 1148.69 L972.341 1149.98 L972.569 1149.56 L972.797 1148.81 L973.025 1146.93 L973.253 1147.28 L973.48 1147.95 L973.708 1149.38 L973.936 1151.8 L974.164 1152.29 L974.391 1151.81 L974.619 1152.48 L974.847 1150.65 L975.075 1151.51 L975.303 1151.54 L975.53 1149.76 L975.758 1147.73 L975.986 1146.1 L976.214 1146.06 L976.441 1146 L976.669 1146.47 L976.897 1147.94 L977.125 1145.54 L977.352 1146.14 L977.58 1145.67 L977.808 1145.53 L978.036 1145.34 L978.264 1145.07 L978.491 1145.08 L978.719 1144.89 L978.947 1144.92 L979.175 1145.44 L979.402 1146.34 L979.63 1145.85 L979.858 1146.25 L980.086 1146.21 L980.313 1146.75 L980.541 1148.26 L980.769 1147.99 L980.997 1144.7 L981.225 1144.07 L981.452 1145.3 L981.68 1145.65 L981.908 1144.31 L982.136 1144.7 L982.363 1145.8 L982.591 1145.44 L982.819 1146.72 L983.047 1146.85 L983.274 1148.02 L983.502 1148.66 L983.73 1148.95 L983.958 1148.68 L984.186 1148.42 L984.413 1148.33 L984.641 1149.57 L984.869 1150.78 L985.097 1149.19 L985.324 1150.21 L985.552 1150.51 L985.78 1150.11 L986.008 1149.67 L986.236 1148.44 L986.463 1148 L986.691 1148.12 L986.919 1148.93 L987.147 1147.1 L987.374 1147.67 L987.602 1148.18 L987.83 1148.09 L988.058 1149.65 L988.285 1149.76 L988.513 1150.51 L988.741 1150.26 L988.969 1149.24 L989.197 1149.97 L989.424 1150.96 L989.652 1150.8 L989.88 1151.84 L990.108 1153.08 L990.335 1152.9 L990.563 1153.85 L990.791 1152.59 L991.019 1153.56 L991.246 1153.91 L991.474 1152.23 L991.702 1151.64 L991.93 1151.3 L992.158 1150.55 L992.385 1151.78 L992.613 1152.58 L992.841 1151.32 L993.069 1151.53 L993.296 1150.29 L993.524 1150.21 L993.752 1149.08 L993.98 1148.94 L994.207 1148.82 L994.435 1146.93 L994.663 1146.09 L994.891 1145.65 L995.119 1144.45 L995.346 1145.08 L995.574 1144.63 L995.802 1144.85 L996.03 1145.48 L996.257 1144.68 L996.485 1145.57 L996.713 1145.05 L996.941 1144.31 L997.169 1144.33 L997.396 1144.7 L997.624 1144.77 L997.852 1144.48 L998.08 1145.78 L998.307 1147.02 L998.535 1147.28 L998.763 1148.45 L998.991 1149.14 L999.218 1151.29 L999.446 1151.58 L999.674 1152.54 L999.902 1150.51 L1000.13 1150.8 L1000.36 1151.29 L1000.59 1151.29 L1000.81 1152.31 L1001.04 1152.28 L1001.27 1153.16 L1001.5 1154.79 L1001.72 1154.78 L1001.95 1155.54 L1002.18 1155.04 L1002.41 1155.91 L1002.64 1156.12 L1002.86 1154.6 L1003.09 1154.79 L1003.32 1155.38 L1003.55 1156.39 L1003.77 1156.8 L1004 1157.29 L1004.23 1156.83 L1004.46 1158.51 L1004.68 1159.52 L1004.91 1158.39 L1005.14 1158.84 L1005.37 1158.35 L1005.6 1157.6 L1005.82 1156.79 L1006.05 1157.41 L1006.28 1157 L1006.51 1156.73 L1006.73 1156.29 L1006.96 1156.77 L1007.19 1156.71 L1007.42 1156.2 L1007.65 1155.72 L1007.87 1156.22 L1008.1 1156.67 L1008.33 1156.62 L1008.56 1156.78 L1008.78 1158.19 L1009.01 1158.75 L1009.24 1158.31 L1009.47 1157.79 L1009.7 1158.23 L1009.92 1157.74 L1010.15 1157.58 L1010.38 1157.26 L1010.61 1158.25 L1010.83 1159 L1011.06 1159.54 L1011.29 1159.81 L1011.52 1159.77 L1011.75 1160.59 L1011.97 1161.69 L1012.2 1162.15 L1012.43 1163.22 L1012.66 1163.97 L1012.88 1163.82 L1013.11 1164.4 L1013.34 1163.77 L1013.57 1163.17 L1013.8 1161.02 L1014.02 1162.16 L1014.25 1162.09 L1014.48 1163.83 L1014.71 1163.47 L1014.93 1163.22 L1015.16 1163.71 L1015.39 1164.48 L1015.62 1164.6 L1015.85 1165 L1016.07 1164.68 L1016.3 1164.85 L1016.53 1165.01 L1016.76 1164.97 L1016.98 1164.9 L1017.21 1164.81 L1017.44 1164.86 L1017.67 1164.58 L1017.9 1164.57 L1018.12 1162.77 L1018.35 1162.39 L1018.58 1161.87 L1018.81 1162.52 L1019.03 1163.19 L1019.26 1162.26 L1019.49 1163.29 L1019.72 1164.12 L1019.95 1165.45 L1020.17 1166.42 L1020.4 1166.28 L1020.63 1166.6 L1020.86 1166.13 L1021.08 1165.08 L1021.31 1164.31 L1021.54 1165.84 L1021.77 1165 L1022 1164.36 L1022.22 1164.5 L1022.45 1164.68 L1022.68 1164.59 L1022.91 1164.77 L1023.13 1165.42 L1023.36 1165.36 L1023.59 1165.46 L1023.82 1166.01 L1024.05 1167.4 L1024.27 1166.66 L1024.5 1166.47 L1024.73 1166.15 L1024.96 1167.25 L1025.18 1166.58 L1025.41 1166.64 L1025.64 1166.19 L1025.87 1166.38 L1026.1 1167.76 L1026.32 1168.69 L1026.55 1168.56 L1026.78 1168.82 L1027.01 1169.63 L1027.23 1168 L1027.46 1168.2 L1027.69 1168.68 L1027.92 1169.13 L1028.15 1169.13 L1028.37 1168.95 L1028.6 1168.03 L1028.83 1169.27 L1029.06 1168.12 L1029.28 1167.68 L1029.51 1168 L1029.74 1168.21 L1029.97 1168.06 L1030.2 1167.09 L1030.42 1167.37 L1030.65 1167.92 L1030.88 1167.89 L1031.11 1168.33 L1031.33 1169.1 L1031.56 1169.55 L1031.79 1168.41 L1032.02 1168.1 L1032.25 1167.19 L1032.47 1167.15 L1032.7 1165.72 L1032.93 1164.96 L1033.16 1164.78 L1033.38 1164.37 L1033.61 1165.18 L1033.84 1165.72 L1034.07 1164.25 L1034.3 1165.47 L1034.52 1165.06 L1034.75 1165.29 L1034.98 1165.31 L1035.21 1166.64 L1035.43 1166.44 L1035.66 1165.25 L1035.89 1166.19 L1036.12 1165.37 L1036.35 1165.55 L1036.57 1164.08 L1036.8 1164.22 L1037.03 1162.81 L1037.26 1161.88 L1037.48 1160.67 L1037.71 1161.63 L1037.94 1162.46 L1038.17 1160.81 L1038.4 1160.2 L1038.62 1161.69 L1038.85 1159.74 L1039.08 1158.68 L1039.31 1158.43 L1039.53 1159.45 L1039.76 1158.76 L1039.99 1160.07 L1040.22 1159.67 L1040.44 1158.39 L1040.67 1157.63 L1040.9 1158.22 L1041.13 1158.51 L1041.36 1157.69 L1041.58 1158.11 L1041.81 1159.09 L1042.04 1159.44 L1042.27 1159.76 L1042.49 1160.8 L1042.72 1160.2 L1042.95 1160.31 L1043.18 1160.4 L1043.41 1161.39 L1043.63 1160.51 L1043.86 1159.59 L1044.09 1159.38 L1044.32 1159.26 L1044.54 1159.33 L1044.77 1159.84 L1045 1160.46 L1045.23 1160.82 L1045.46 1160.45 L1045.68 1160.89 L1045.91 1161.09 L1046.14 1162.46 L1046.37 1161.56 L1046.59 1159.26 L1046.82 1160.14 L1047.05 1160.8 L1047.28 1161.99 L1047.51 1162.14 L1047.73 1161.45 L1047.96 1162.78 L1048.19 1163.1 L1048.42 1162.39 L1048.64 1162.72 L1048.87 1163.27 L1049.1 1163.07 L1049.33 1160.85 L1049.56 1161.08 L1049.78 1160.91 L1050.01 1161.65 L1050.24 1161.98 L1050.47 1161.44 L1050.69 1161.74 L1050.92 1161.93 L1051.15 1162.26 L1051.38 1162.43 L1051.61 1163.56 L1051.83 1163.51 L1052.06 1163.3 L1052.29 1164.24 L1052.52 1163 L1052.74 1162.48 L1052.97 1162.89 L1053.2 1161.88 L1053.43 1161.38 L1053.66 1160.63 L1053.88 1158.47 L1054.11 1158.16 L1054.34 1159.13 L1054.57 1158.26 L1054.79 1158.51 L1055.02 1157.95 L1055.25 1158.14 L1055.48 1159.43 L1055.71 1159.45 L1055.93 1158.65 L1056.16 1158.75 L1056.39 1158.45 L1056.62 1157.52 L1056.84 1157.69 L1057.07 1157.79 L1057.3 1157.92 L1057.53 1158.46 L1057.76 1159.17 L1057.98 1159.64 L1058.21 1160.25 L1058.44 1160.98 L1058.67 1161.69 L1058.89 1161.79 L1059.12 1161.8 L1059.35 1161.56 L1059.58 1161.97 L1059.81 1161.46 L1060.03 1162.88 L1060.26 1163.17 L1060.49 1163.07 L1060.72 1163.24 L1060.94 1164.01 L1061.17 1164.09 L1061.4 1163.42 L1061.63 1163.21 L1061.86 1163.71 L1062.08 1165.36 L1062.31 1164.69 L1062.54 1164.9 L1062.77 1165.51 L1062.99 1166.43 L1063.22 1166.05 L1063.45 1166.74 L1063.68 1166.41 L1063.91 1166.68 L1064.13 1167.27 L1064.36 1165.65 L1064.59 1165.67 L1064.82 1166.63 L1065.04 1167.84 L1065.27 1167.73 L1065.5 1168.21 L1065.73 1167.76 L1065.96 1168.04 L1066.18 1167.7 L1066.41 1167.29 L1066.64 1168.09 L1066.87 1167.76 L1067.09 1166.82 L1067.32 1165.28 L1067.55 1166.08 L1067.78 1167.29 L1068.01 1167.54 L1068.23 1166.46 L1068.46 1164.78 L1068.69 1164.85 L1068.92 1166.19 L1069.14 1166.6 L1069.37 1166.27 L1069.6 1166.23 L1069.83 1165.55 L1070.06 1163.87 L1070.28 1162.28 L1070.51 1162.93 L1070.74 1162.54 L1070.97 1161.22 L1071.19 1161.52 L1071.42 1161.31 L1071.65 1161.61 L1071.88 1161.47 L1072.11 1161.84 L1072.33 1162.67 L1072.56 1162.49 L1072.79 1162.65 L1073.02 1162.36 L1073.24 1161.16 L1073.47 1162.13 L1073.7 1163.81 L1073.93 1163.56 L1074.16 1163.55 L1074.38 1164.89 L1074.61 1162.88 L1074.84 1164.15 L1075.07 1163.91 L1075.29 1164.17 L1075.52 1164.1 L1075.75 1164.96 L1075.98 1165.55 L1076.2 1166.28 L1076.43 1166.52 L1076.66 1166.49 L1076.89 1166.3 L1077.12 1166.06 L1077.34 1166.54 L1077.57 1166.94 L1077.8 1166.53 L1078.03 1166.51 L1078.25 1165.89 L1078.48 1166.66 L1078.71 1166.14 L1078.94 1165.85 L1079.17 1165.37 L1079.39 1164.77 L1079.62 1164.92 L1079.85 1165.45 L1080.08 1166.26 L1080.3 1166.85 L1080.53 1167.32 L1080.76 1167.98 L1080.99 1167.86 L1081.22 1166.52 L1081.44 1165.83 L1081.67 1165.32 L1081.9 1165.45 L1082.13 1164.01 L1082.35 1163.21 L1082.58 1163.65 L1082.81 1163.15 L1083.04 1162.42 L1083.27 1162.65 L1083.49 1160.72 L1083.72 1160.4 L1083.95 1159.74 L1084.18 1158.1 L1084.4 1158.7 L1084.63 1158.64 L1084.86 1158.14 L1085.09 1158.97 L1085.32 1159.99 L1085.54 1158.89 L1085.77 1159.78 L1086 1161.01 L1086.23 1162.15 L1086.45 1161.88 L1086.68 1163.03 L1086.91 1162.97 L1087.14 1162.93 L1087.37 1162.82 L1087.59 1163.33 L1087.82 1162.89 L1088.05 1163.61 L1088.28 1163.13 L1088.5 1163.8 L1088.73 1163.85 L1088.96 1163.86 L1089.19 1164.89 L1089.42 1164.22 L1089.64 1164.32 L1089.87 1164.19 L1090.1 1164.18 L1090.33 1164.77 L1090.55 1164.86 L1090.78 1165.26 L1091.01 1164.85 L1091.24 1165.12 L1091.47 1164.81 L1091.69 1166.21 L1091.92 1165.96 L1092.15 1164.98 L1092.38 1164.97 L1092.6 1164.5 L1092.83 1164.6 L1093.06 1166.21 L1093.29 1166.22 L1093.52 1166.9 L1093.74 1167.06 L1093.97 1166.4 L1094.2 1166.69 L1094.43 1166.13 L1094.65 1166.39 L1094.88 1167.38 L1095.11 1167.06 L1095.34 1166.46 L1095.57 1168.03 L1095.79 1166.98 L1096.02 1167.19 L1096.25 1168.03 L1096.48 1166.17 L1096.7 1163.61 L1096.93 1164.41 L1097.16 1165.25 L1097.39 1166.06 L1097.62 1166.33 L1097.84 1166.05 L1098.07 1165.79 L1098.3 1165.56 L1098.53 1165.73 L1098.75 1165.95 L1098.98 1165.38 L1099.21 1164.07 L1099.44 1165.12 L1099.67 1164.66 L1099.89 1165.89 L1100.12 1165.33 L1100.35 1163.99 L1100.58 1164.25 L1100.8 1164.13 L1101.03 1164.83 L1101.26 1164.29 L1101.49 1163.65 L1101.72 1163.67 L1101.94 1163.45 L1102.17 1162.98 L1102.4 1163.06 L1102.63 1163.59 L1102.85 1163 L1103.08 1163.43 L1103.31 1163.11 L1103.54 1162.94 L1103.77 1163.08 L1103.99 1163.08 L1104.22 1163.3 L1104.45 1161.45 L1104.68 1160.56 L1104.9 1159.7 L1105.13 1159.65 L1105.36 1161.03 L1105.59 1160.74 L1105.82 1160.56 L1106.04 1160.1 L1106.27 1158.99 L1106.5 1158.99 L1106.73 1159.54 L1106.95 1160.18 L1107.18 1160.61 L1107.41 1160.11 L1107.64 1160.13 L1107.87 1159.82 L1108.09 1159.45 L1108.32 1160.43 L1108.55 1161.4 L1108.78 1161.92 L1109 1162.48 L1109.23 1163.2 L1109.46 1162.71 L1109.69 1162.65 L1109.92 1163.61 L1110.14 1163.54 L1110.37 1162.76 L1110.6 1162.95 L1110.83 1163.13 L1111.05 1163.4 L1111.28 1164.57 L1111.51 1165.44 L1111.74 1165.03 L1111.97 1165.22 L1112.19 1165.9 L1112.42 1165.55 L1112.65 1165.97 L1112.88 1165.66 L1113.1 1165.49 L1113.33 1166.27 L1113.56 1167.47 L1113.79 1167.39 L1114.01 1167.99 L1114.24 1168.17 L1114.47 1167.05 L1114.7 1166.74 L1114.93 1167.7 L1115.15 1167.72 L1115.38 1168.21 L1115.61 1168.69 L1115.84 1169.43 L1116.06 1170.25 L1116.29 1169.92 L1116.52 1170.28 L1116.75 1170.7 L1116.98 1170.89 L1117.2 1170.83 L1117.43 1171.02 L1117.66 1170.09 L1117.89 1169.78 L1118.11 1168.39 L1118.34 1167.42 L1118.57 1168.5 L1118.8 1167.82 L1119.03 1167.01 L1119.25 1166.79 L1119.48 1165.68 L1119.71 1166.56 L1119.94 1167.09 L1120.16 1166.54 L1120.39 1166.34 L1120.62 1167.06 L1120.85 1168.27 L1121.08 1166.15 L1121.3 1165.81 L1121.53 1166.22 L1121.76 1167.46 L1121.99 1168.32 L1122.21 1167.57 L1122.44 1168.24 L1122.67 1169.71 L1122.9 1168.28 L1123.13 1168.23 L1123.35 1167.96 L1123.58 1167.75 L1123.81 1167.9 L1124.04 1167.13 L1124.26 1166.62 L1124.49 1166.23 L1124.72 1165.24 L1124.95 1165.18 L1125.18 1165.05 L1125.4 1165.29 L1125.63 1164.08 L1125.86 1165.17 L1126.09 1164.81 L1126.31 1165.63 L1126.54 1165.82 L1126.77 1166.14 L1127 1166.76 L1127.23 1166.2 L1127.45 1164.96 L1127.68 1163.85 L1127.91 1161.94 L1128.14 1162.58 L1128.36 1162.55 L1128.59 1163.73 L1128.82 1164.08 L1129.05 1164.85 L1129.28 1165.46 L1129.5 1162.66 L1129.73 1164.41 L1129.96 1164.98 L1130.19 1164.39 L1130.41 1163.57 L1130.64 1164.02 L1130.87 1164.19 L1131.1 1164.7 L1131.33 1164.56 L1131.55 1165.01 L1131.78 1164.76 L1132.01 1164.73 L1132.24 1162.28 L1132.46 1161.41 L1132.69 1161.09 L1132.92 1161.8 L1133.15 1162.4 L1133.38 1163.03 L1133.6 1163.8 L1133.83 1163.41 L1134.06 1163.01 L1134.29 1162.66 L1134.51 1162.25 L1134.74 1161.57 L1134.97 1161.91 L1135.2 1163.43 L1135.43 1164.6 L1135.65 1164 L1135.88 1164.4 L1136.11 1164.71 L1136.34 1164.74 L1136.56 1164.99 L1136.79 1164.98 L1137.02 1164.64 L1137.25 1165.65 L1137.48 1166.38 L1137.7 1166.42 L1137.93 1166.76 L1138.16 1166.74 L1138.39 1167.34 L1138.61 1167.17 L1138.84 1168.6 L1139.07 1170.1 L1139.3 1170.53 L1139.53 1169.33 L1139.75 1169.5 L1139.98 1170.4 L1140.21 1170.3 L1140.44 1169.33 L1140.66 1167.9 L1140.89 1167.93 L1141.12 1168.55 L1141.35 1168.69 L1141.58 1169.07 L1141.8 1168.34 L1142.03 1169.45 L1142.26 1168.81 L1142.49 1168.14 L1142.71 1168.6 L1142.94 1167.73 L1143.17 1168.27 L1143.4 1168.5 L1143.63 1168.11 L1143.85 1168.47 L1144.08 1167.33 L1144.31 1167.36 L1144.54 1166.62 L1144.76 1166.66 L1144.99 1167.23 L1145.22 1168.09 L1145.45 1167.3 L1145.68 1166.19 L1145.9 1164.42 L1146.13 1164.06 L1146.36 1164.83 L1146.59 1164.42 L1146.81 1163.87 L1147.04 1163.39 L1147.27 1163.09 L1147.5 1162.49 L1147.73 1162.57 L1147.95 1163.48 L1148.18 1161.32 L1148.41 1161.45 L1148.64 1162 L1148.86 1161.92 L1149.09 1163.07 L1149.32 1163.54 L1149.55 1162.8 L1149.77 1161.53 L1150 1159.91 L1150.23 1161.24 L1150.46 1162.34 L1150.69 1161.37 L1150.91 1160.94 L1151.14 1159.86 L1151.37 1160.02 L1151.6 1159.48 L1151.82 1157.48 L1152.05 1158.03 L1152.28 1158.6 L1152.51 1159.47 L1152.74 1160.14 L1152.96 1160.98 L1153.19 1161.33 L1153.42 1161.34 L1153.65 1162.4 L1153.87 1161.94 L1154.1 1160.33 L1154.33 1160.85 L1154.56 1160.57 L1154.79 1159.94 L1155.01 1160.69 L1155.24 1159.69 L1155.47 1160.1 L1155.7 1158.91 L1155.92 1159.16 L1156.15 1157.75 L1156.38 1158.35 L1156.61 1158.45 L1156.84 1157.93 L1157.06 1157.29 L1157.29 1157.04 L1157.52 1157.33 L1157.75 1157.83 L1157.97 1157.75 L1158.2 1157.85 L1158.43 1157.94 L1158.66 1156.89 L1158.89 1156.82 L1159.11 1156.39 L1159.34 1156.36 L1159.57 1156.33 L1159.8 1155.58 L1160.02 1155.92 L1160.25 1156.11 L1160.48 1156.5 L1160.71 1156.23 L1160.94 1155.91 L1161.16 1156.56 L1161.39 1155.14 L1161.62 1155 L1161.85 1155.09 L1162.07 1156.05 L1162.3 1156.07 L1162.53 1155.69 L1162.76 1156.58 L1162.99 1155.86 L1163.21 1156.76 L1163.44 1157.08 L1163.67 1155.09 L1163.9 1155.36 L1164.12 1154.2 L1164.35 1154.73 L1164.58 1154.92 L1164.81 1154.62 L1165.04 1152.61 L1165.26 1153.18 L1165.49 1152.97 L1165.72 1153.44 L1165.95 1153.6 L1166.17 1153.77 L1166.4 1152.5 L1166.63 1150.4 L1166.86 1149.78 L1167.09 1149.84 L1167.31 1148.88 L1167.54 1147.51 L1167.77 1147.19 L1168 1148.05 L1168.22 1148.13 L1168.45 1147.78 L1168.68 1148.99 L1168.91 1148.7 L1169.14 1148.85 L1169.36 1149.24 L1169.59 1149.67 L1169.82 1148.83 L1170.05 1149.46 L1170.27 1148.88 L1170.5 1148.36 L1170.73 1147.93 L1170.96 1148.23 L1171.19 1148.66 L1171.41 1148.95 L1171.64 1150.09 L1171.87 1149.49 L1172.1 1149.78 L1172.32 1150.13 L1172.55 1149.79 L1172.78 1150.25 L1173.01 1150.93 L1173.24 1151.01 L1173.46 1151.54 L1173.69 1152.22 L1173.92 1151.42 L1174.15 1152.49 L1174.37 1152.24 L1174.6 1152.32 L1174.83 1152.14 L1175.06 1150.83 L1175.29 1151.21 L1175.51 1152.25 L1175.74 1152.4 L1175.97 1153.49 L1176.2 1154.12 L1176.42 1153.23 L1176.65 1152.19 L1176.88 1151.73 L1177.11 1151.28 L1177.34 1150.78 L1177.56 1150.4 L1177.79 1150.2 L1178.02 1150.83 L1178.25 1150.44 L1178.47 1151.59 L1178.7 1151.29 L1178.93 1150.41 L1179.16 1150.89 L1179.39 1150.73 L1179.61 1150.53 L1179.84 1149.98 L1180.07 1149.72 L1180.3 1150.11 L1180.52 1150.76 L1180.75 1151.65 L1180.98 1152.17 L1181.21 1153.2 L1181.44 1152.39 L1181.66 1151.38 L1181.89 1151.3 L1182.12 1150.19 L1182.35 1150.35 L1182.57 1150.14 L1182.8 1150.09 L1183.03 1151.5 L1183.26 1151.2 L1183.49 1151.52 L1183.71 1151.88 L1183.94 1151.94 L1184.17 1151.15 L1184.4 1150.81 L1184.62 1150.9 L1184.85 1151.99 L1185.08 1151.91 L1185.31 1152.1 L1185.53 1152.01 L1185.76 1152.52 L1185.99 1151.78 L1186.22 1151.95 L1186.45 1151.45 L1186.67 1152.26 L1186.9 1152.28 L1187.13 1153.54 L1187.36 1154.91 L1187.58 1155 L1187.81 1155.1 L1188.04 1155.47 L1188.27 1155.58 L1188.5 1156.44 L1188.72 1156.46 L1188.95 1155.86 L1189.18 1155.35 L1189.41 1155.71 L1189.63 1155.48 L1189.86 1155.03 L1190.09 1156.17 L1190.32 1156.98 L1190.55 1157.18 L1190.77 1158.01 L1191 1155.92 L1191.23 1156.03 L1191.46 1156.52 L1191.68 1156.59 L1191.91 1156.32 L1192.14 1157.39 L1192.37 1157.67 L1192.6 1157.33 L1192.82 1157.69 L1193.05 1157.78 L1193.28 1156.99 L1193.51 1156.03 L1193.73 1155.87 L1193.96 1155.24 L1194.19 1154.77 L1194.42 1155.01 L1194.65 1155.99 L1194.87 1154.97 L1195.1 1156.35 L1195.33 1156.91 L1195.56 1156.38 L1195.78 1156.8 L1196.01 1156.4 L1196.24 1156.26 L1196.47 1155.16 L1196.7 1154.99 L1196.92 1155.65 L1197.15 1156.15 L1197.38 1155.37 L1197.61 1156.51 L1197.83 1156.06 L1198.06 1156.43 L1198.29 1155.51 L1198.52 1155.56 L1198.75 1155.17 L1198.97 1154.63 L1199.2 1155.29 L1199.43 1154.83 L1199.66 1154.35 L1199.88 1155.05 L1200.11 1155.12 L1200.34 1155.06 L1200.57 1155.35 L1200.8 1156.15 L1201.02 1155.98 L1201.25 1154.8 L1201.48 1154.26 L1201.71 1154.9 L1201.93 1155 L1202.16 1154.44 L1202.39 1154.3 L1202.62 1154.73 L1202.85 1155.61 L1203.07 1156.13 L1203.3 1155.71 L1203.53 1156.11 L1203.76 1156.81 L1203.98 1157.17 L1204.21 1156.47 L1204.44 1155.92 L1204.67 1155.16 L1204.9 1154.41 L1205.12 1154.55 L1205.35 1155.39 L1205.58 1155.17 L1205.81 1155.73 L1206.03 1157.1 L1206.26 1157.52 L1206.49 1157.64 L1206.72 1158.04 L1206.95 1158.78 L1207.17 1157.59 L1207.4 1157.87 L1207.63 1158.87 L1207.86 1159.64 L1208.08 1159.64 L1208.31 1160.36 L1208.54 1160.07 L1208.77 1160.62 L1209 1161.56 L1209.22 1161.69 L1209.45 1162.1 L1209.68 1162.87 L1209.91 1161.13 L1210.13 1161.58 L1210.36 1160.99 L1210.59 1160.54 L1210.82 1160.29 L1211.05 1158.8 L1211.27 1158.77 L1211.5 1158.82 L1211.73 1159 L1211.96 1160.25 L1212.18 1159.24 L1212.41 1159.09 L1212.64 1158.53 L1212.87 1158.65 L1213.1 1159.07 L1213.32 1159.59 L1213.55 1160.6 L1213.78 1159.89 L1214.01 1158.18 L1214.23 1158.01 L1214.46 1158.24 L1214.69 1156.52 L1214.92 1157.25 L1215.15 1158.21 L1215.37 1158.36 L1215.6 1159.22 L1215.83 1159.59 L1216.06 1159.13 L1216.28 1159.96 L1216.51 1159.94 L1216.74 1160.28 L1216.97 1161.05 L1217.2 1160.35 L1217.42 1159.34 L1217.65 1158.57 L1217.88 1158.18 L1218.11 1158.33 L1218.33 1158.55 L1218.56 1158.33 L1218.79 1158.27 L1219.02 1157.48 L1219.25 1156.8 L1219.47 1156.44 L1219.7 1156.16 L1219.93 1156.74 L1220.16 1156.02 L1220.38 1156.39 L1220.61 1157.13 L1220.84 1157.81 L1221.07 1157.58 L1221.29 1158.5 L1221.52 1157.64 L1221.75 1157.37 L1221.98 1156.28 L1222.21 1155.9 L1222.43 1156.82 L1222.66 1157.11 L1222.89 1157.31 L1223.12 1156.64 L1223.34 1155.76 L1223.57 1154.76 L1223.8 1155.22 L1224.03 1154.39 L1224.26 1154.34 L1224.48 1153.58 L1224.71 1153.33 L1224.94 1152.66 L1225.17 1153.83 L1225.39 1154.18 L1225.62 1154.21 L1225.85 1154.05 L1226.08 1154.01 L1226.31 1154.12 L1226.53 1154.23 L1226.76 1154 L1226.99 1154.14 L1227.22 1154.02 L1227.44 1154.8 L1227.67 1154.44 L1227.9 1154.85 L1228.13 1155.82 L1228.36 1155.48 L1228.58 1155.04 L1228.81 1154.35 L1229.04 1154.73 L1229.27 1154.75 L1229.49 1154.81 L1229.72 1154.98 L1229.95 1155.02 L1230.18 1154.66 L1230.41 1154.91 L1230.63 1154.01 L1230.86 1152.98 L1231.09 1154.01 L1231.32 1153.87 L1231.54 1154.85 L1231.77 1154.67 L1232 1154.75 L1232.23 1155 L1232.46 1154.15 L1232.68 1153.9 L1232.91 1153.65 L1233.14 1153.27 L1233.37 1153.01 L1233.59 1153.95 L1233.82 1154.53 L1234.05 1153.92 L1234.28 1153.82 L1234.51 1154.86 L1234.73 1154.89 L1234.96 1155.74 L1235.19 1156.12 L1235.42 1156.05 L1235.64 1155.77 L1235.87 1155.05 L1236.1 1154.63 L1236.33 1153.5 L1236.56 1154.38 L1236.78 1153.89 L1237.01 1154.46 L1237.24 1154.54 L1237.47 1155.19 L1237.69 1155.03 L1237.92 1155.9 L1238.15 1155.6 L1238.38 1156.74 L1238.61 1157.95 L1238.83 1158.6 L1239.06 1159.06 L1239.29 1158.96 L1239.52 1158.28 L1239.74 1158.24 L1239.97 1157.8 L1240.2 1157.26 L1240.43 1157.02 L1240.66 1157.76 L1240.88 1158.02 L1241.11 1156.72 L1241.34 1156.88 L1241.57 1156.9 L1241.79 1157.44 L1242.02 1156.76 L1242.25 1157.37 L1242.48 1157.24 L1242.71 1157.63 L1242.93 1158.07 L1243.16 1157.4 L1243.39 1156.63 L1243.62 1156.36 L1243.84 1156.19 L1244.07 1157.13 L1244.3 1157.2 L1244.53 1157.37 L1244.76 1156.9 L1244.98 1156.35 L1245.21 1156.28 L1245.44 1156.04 L1245.67 1157.08 L1245.89 1156.97 L1246.12 1158.2 L1246.35 1158.56 L1246.58 1158.73 L1246.81 1159.18 L1247.03 1158.99 L1247.26 1159.02 L1247.49 1158.86 L1247.72 1159.32 L1247.94 1159.25 L1248.17 1158.1 L1248.4 1157.39 L1248.63 1156.55 L1248.86 1156.12 L1249.08 1156.26 L1249.31 1156.69 L1249.54 1157.73 L1249.77 1158.04 L1249.99 1158.3 L1250.22 1157.94 L1250.45 1157.94 L1250.68 1158.65 L1250.91 1159.15 L1251.13 1158.67 L1251.36 1158.75 L1251.59 1159.1 L1251.82 1158.97 L1252.04 1158.66 L1252.27 1157.33 L1252.5 1158.05 L1252.73 1158.46 L1252.96 1156.74 L1253.18 1156.87 L1253.41 1156.34 L1253.64 1157.29 L1253.87 1157.25 L1254.09 1158.09 L1254.32 1158.16 L1254.55 1158.53 L1254.78 1158.7 L1255.01 1157.9 L1255.23 1156.32 L1255.46 1155.4 L1255.69 1154.61 L1255.92 1153.93 L1256.14 1153.43 L1256.37 1153.12 L1256.6 1153.22 L1256.83 1153.18 L1257.06 1153.48 L1257.28 1153.83 L1257.51 1154.06 L1257.74 1154.74 L1257.97 1155.69 L1258.19 1156.17 L1258.42 1155.48 L1258.65 1155.46 L1258.88 1156.14 L1259.1 1155.65 L1259.33 1155.88 L1259.56 1157.04 L1259.79 1157.29 L1260.02 1157.73 L1260.24 1156.83 L1260.47 1157.33 L1260.7 1158.08 L1260.93 1158.36 L1261.15 1158.49 L1261.38 1158.22 L1261.61 1158.33 L1261.84 1158.55 L1262.07 1157.68 L1262.29 1157.2 L1262.52 1157.28 L1262.75 1155.96 L1262.98 1156.32 L1263.2 1155.29 L1263.43 1154.14 L1263.66 1153.14 L1263.89 1152.77 L1264.12 1152.25 L1264.34 1152.1 L1264.57 1152.34 L1264.8 1153.54 L1265.03 1153.15 L1265.25 1153.44 L1265.48 1153.3 L1265.71 1154.54 L1265.94 1153.74 L1266.17 1152.8 L1266.39 1152.08 L1266.62 1151.76 L1266.85 1150.54 L1267.08 1150.92 L1267.3 1150.04 L1267.53 1150.16 L1267.76 1150.62 L1267.99 1150.42 L1268.22 1150.19 L1268.44 1150.28 L1268.67 1150.69 L1268.9 1150.66 L1269.13 1151.61 L1269.35 1152.36 L1269.58 1151.56 L1269.81 1152.12 L1270.04 1152.25 L1270.27 1152.13 L1270.49 1151.45 L1270.72 1151.68 L1270.95 1151.34 L1271.18 1151.58 L1271.4 1151.05 L1271.63 1151.52 L1271.86 1150.96 L1272.09 1150.57 L1272.32 1150.02 L1272.54 1150.4 L1272.77 1150.51 L1273 1150.57 L1273.23 1150.9 L1273.45 1150.81 L1273.68 1150.81 L1273.91 1149.77 L1274.14 1149.34 L1274.37 1149.22 L1274.59 1148.43 L1274.82 1147.99 L1275.05 1146.96 L1275.28 1146.14 L1275.5 1146.12 L1275.73 1145.67 L1275.96 1145.06 L1276.19 1145.39 L1276.42 1146.1 L1276.64 1145.59 L1276.87 1145.18 L1277.1 1145.49 L1277.33 1145.12 L1277.55 1144.55 L1277.78 1144.93 L1278.01 1144.35 L1278.24 1144.9 L1278.47 1143.76 L1278.69 1143.25 L1278.92 1143.14 L1279.15 1142.96 L1279.38 1142.47 L1279.6 1142.75 L1279.83 1143.32 L1280.06 1143.43 L1280.29 1143.76 L1280.52 1144.02 L1280.74 1145.22 L1280.97 1144.8 L1281.2 1144.06 L1281.43 1145.27 L1281.65 1145.74 L1281.88 1144.94 L1282.11 1144.39 L1282.34 1143.2 L1282.57 1144.44 L1282.79 1144.93 L1283.02 1144.47 L1283.25 1145.37 L1283.48 1144.75 L1283.7 1144.14 L1283.93 1144.65 L1284.16 1144.73 L1284.39 1144.67 L1284.62 1144.68 L1284.84 1144.22 L1285.07 1143.79 L1285.3 1144.25 L1285.53 1143.19 L1285.75 1142.97 L1285.98 1142.62 L1286.21 1141.61 L1286.44 1141.96 L1286.67 1141.79 L1286.89 1141.74 L1287.12 1141.95 L1287.35 1141.73 L1287.58 1141.76 L1287.8 1142.07 L1288.03 1142.03 L1288.26 1142.27 L1288.49 1142.35 L1288.72 1142.47 L1288.94 1142.69 L1289.17 1142.34 L1289.4 1142.54 L1289.63 1143.39 L1289.85 1143.3 L1290.08 1144.33 L1290.31 1144.53 L1290.54 1145.25 L1290.77 1144.52 L1290.99 1144.8 L1291.22 1144.04 L1291.45 1143.54 L1291.68 1143.18 L1291.9 1143.19 L1292.13 1143.54 L1292.36 1143.29 L1292.59 1143.03 L1292.82 1143.08 L1293.04 1141.69 L1293.27 1141.61 L1293.5 1142.08 L1293.73 1141.81 L1293.95 1141.8 L1294.18 1141.58 L1294.41 1141.06 L1294.64 1141.18 L1294.86 1140.84 L1295.09 1140.46 L1295.32 1140.6 L1295.55 1140.93 L1295.78 1141.43 L1296 1141.74 L1296.23 1142.38 L1296.46 1142.25 L1296.69 1143.47 L1296.91 1143.24 L1297.14 1143.36 L1297.37 1142.75 L1297.6 1143.13 L1297.83 1143.28 L1298.05 1142.87 L1298.28 1141.63 L1298.51 1141.21 L1298.74 1142.15 L1298.96 1140.92 L1299.19 1140.92 L1299.42 1140.87 L1299.65 1141.1 L1299.88 1141.94 L1300.1 1141.67 L1300.33 1141.39 L1300.56 1141.89 L1300.79 1141.54 L1301.01 1141.08 L1301.24 1140.69 L1301.47 1141.04 L1301.7 1141.01 L1301.93 1140.73 L1302.15 1141.36 L1302.38 1141.88 L1302.61 1141.25 L1302.84 1140.76 L1303.06 1140.69 L1303.29 1141.05 L1303.52 1141.23 L1303.75 1141.42 L1303.98 1141.03 L1304.2 1141.82 L1304.43 1141.56 L1304.66 1141.15 L1304.89 1141.1 L1305.11 1141.25 L1305.34 1140.89 L1305.57 1140.88 L1305.8 1141.14 L1306.03 1141.22 L1306.25 1141.46 L1306.48 1140.72 L1306.71 1140.96 L1306.94 1142.19 L1307.16 1141.18 L1307.39 1141.36 L1307.62 1141.19 L1307.85 1142.16 L1308.08 1142.39 L1308.3 1142.66 L1308.53 1141.94 L1308.76 1141.77 L1308.99 1141.32 L1309.21 1140.87 L1309.44 1141.24 L1309.67 1141.69 L1309.9 1142.15 L1310.13 1142.91 L1310.35 1142.63 L1310.58 1142.7 L1310.81 1143.12 L1311.04 1144.16 L1311.26 1144.82 L1311.49 1144.14 L1311.72 1143.85 L1311.95 1144.74 L1312.18 1143.81 L1312.4 1142.97 L1312.63 1141.94 L1312.86 1140.69 L1313.09 1139.88 L1313.31 1141.03 L1313.54 1140.56 L1313.77 1140.29 L1314 1140.64 L1314.23 1141.36 L1314.45 1142.1 L1314.68 1141 L1314.91 1141.6 L1315.14 1142.38 L1315.36 1142.27 L1315.59 1142.51 L1315.82 1142.08 L1316.05 1140.55 L1316.28 1140.67 L1316.5 1139.67 L1316.73 1139.89 L1316.96 1138.98 L1317.19 1139.72 L1317.41 1140.52 L1317.64 1139.68 L1317.87 1139.82 L1318.1 1140.6 L1318.33 1140.2 L1318.55 1139.78 L1318.78 1139.83 L1319.01 1140.27 L1319.24 1140.11 L1319.46 1140.54 L1319.69 1140.49 L1319.92 1140.64 L1320.15 1140.22 L1320.38 1139.7 L1320.6 1140.3 L1320.83 1141.21 L1321.06 1141.31 L1321.29 1141.49 L1321.51 1141.37 L1321.74 1140.63 L1321.97 1140.93 L1322.2 1140.17 L1322.43 1138.72 L1322.65 1138.72 L1322.88 1138.43 L1323.11 1138.4 L1323.34 1137.97 L1323.56 1138.08 L1323.79 1138.12 L1324.02 1138.72 L1324.25 1138.78 L1324.48 1139.57 L1324.7 1139.34 L1324.93 1139.4 L1325.16 1140.22 L1325.39 1140.51 L1325.61 1140.16 L1325.84 1139.92 L1326.07 1140.16 L1326.3 1139.9 L1326.53 1140.17 L1326.75 1139.75 L1326.98 1138.8 L1327.21 1138.84 L1327.44 1138.4 L1327.66 1138.41 L1327.89 1138.31 L1328.12 1139.25 L1328.35 1139.57 L1328.58 1139.1 L1328.8 1140.45 L1329.03 1140.04 L1329.26 1139.4 L1329.49 1139.08 L1329.71 1139.22 L1329.94 1139.51 L1330.17 1139.72 L1330.4 1137.99 L1330.62 1138.63 L1330.85 1138.22 L1331.08 1137.69 L1331.31 1137.11 L1331.54 1137.08 L1331.76 1136.85 L1331.99 1137.33 L1332.22 1137.69 L1332.45 1137.47 L1332.67 1136.67 L1332.9 1137.65 L1333.13 1137.74 L1333.36 1137.62 L1333.59 1137.57 L1333.81 1137.07 L1334.04 1137.28 L1334.27 1138.05 L1334.5 1138.53 L1334.72 1138.78 L1334.95 1137.65 L1335.18 1138.26 L1335.41 1138.62 L1335.64 1139.43 L1335.86 1139.62 L1336.09 1140.23 L1336.32 1140.75 L1336.55 1140.8 L1336.77 1140.14 L1337 1140.55 L1337.23 1140.52 L1337.46 1141.02 L1337.69 1141.23 L1337.91 1142.29 L1338.14 1142.43 L1338.37 1141.97 L1338.6 1140.82 L1338.82 1140.05 L1339.05 1140.9 L1339.28 1141.45 L1339.51 1140.89 L1339.74 1141.01 L1339.96 1141.29 L1340.19 1140.89 L1340.42 1141.05 L1340.65 1141.18 L1340.87 1140.69 L1341.1 1140.72 L1341.33 1141.37 L1341.56 1141.32 L1341.79 1140.7 L1342.01 1140.11 L1342.24 1141.05 L1342.47 1140.87 L1342.7 1140.91 L1342.92 1139.91 L1343.15 1140.04 L1343.38 1139.91 L1343.61 1139.57 L1343.84 1140.04 L1344.06 1139.78 L1344.29 1139.57 L1344.52 1139.42 L1344.75 1138.29 L1344.97 1137.69 L1345.2 1136.89 L1345.43 1136.68 L1345.66 1137.04 L1345.89 1137.23 L1346.11 1137.23 L1346.34 1138.04 L1346.57 1138.76 L1346.8 1138.84 L1347.02 1138.75 L1347.25 1139.41 L1347.48 1139.56 L1347.71 1140.32 L1347.94 1140 L1348.16 1140.95 L1348.39 1140.96 L1348.62 1141.8 L1348.85 1142.1 L1349.07 1142.99 L1349.3 1143.41 L1349.53 1144.14 L1349.76 1143.18 L1349.99 1142.84 L1350.21 1142.92 L1350.44 1142.88 L1350.67 1142.87 L1350.9 1142.6 L1351.12 1143.14 L1351.35 1142.99 L1351.58 1142.7 L1351.81 1141.73 L1352.04 1141.79 L1352.26 1141.61 L1352.49 1141.46 L1352.72 1141.66 L1352.95 1142.89 L1353.17 1143.32 L1353.4 1143.83 L1353.63 1143.82 L1353.86 1143.09 L1354.09 1144.15 L1354.31 1144.54 L1354.54 1144.55 L1354.77 1144.2 L1355 1143.61 L1355.22 1144.11 L1355.45 1144.06 L1355.68 1144.25 L1355.91 1144.21 L1356.14 1143.87 L1356.36 1144.76 L1356.59 1145.47 L1356.82 1145.86 L1357.05 1145.75 L1357.27 1145.26 L1357.5 1145.24 L1357.73 1145.2 L1357.96 1144.32 L1358.19 1143.52 L1358.41 1143.22 L1358.64 1142.9 L1358.87 1142.76 L1359.1 1143.02 L1359.32 1142.22 L1359.55 1141.42 L1359.78 1141.71 L1360.01 1142.05 L1360.24 1142.99 L1360.46 1143.16 L1360.69 1143.24 L1360.92 1142.37 L1361.15 1142.69 L1361.37 1141.99 L1361.6 1142.89 L1361.83 1143.08 L1362.06 1143.73 L1362.29 1144.49 L1362.51 1144.36 L1362.74 1144.44 L1362.97 1144.35 L1363.2 1144.62 L1363.42 1144.87 L1363.65 1144.67 L1363.88 1145.32 L1364.11 1145.98 L1364.34 1145.57 L1364.56 1144.4 L1364.79 1144.02 L1365.02 1144.49 L1365.25 1144.68 L1365.47 1145.44 L1365.7 1144.92 L1365.93 1144.9 L1366.16 1145.37 L1366.39 1145.65 L1366.61 1146.06 L1366.84 1146.57 L1367.07 1147.13 L1367.3 1147.21 L1367.52 1147.33 L1367.75 1147.11 L1367.98 1147.9 L1368.21 1148.02 L1368.43 1147.69 L1368.66 1147.15 L1368.89 1147.15 L1369.12 1147.06 L1369.35 1147.25 L1369.57 1146.85 L1369.8 1146.9 L1370.03 1147.29 L1370.26 1147.05 L1370.48 1146.48 L1370.71 1146.22 L1370.94 1146.75 L1371.17 1147.23 L1371.4 1146.4 L1371.62 1146.73 L1371.85 1147.21 L1372.08 1147.46 L1372.31 1147.12 L1372.53 1147.58 L1372.76 1147.92 L1372.99 1148.01 L1373.22 1148.42 L1373.45 1148.05 L1373.67 1148.44 L1373.9 1147.24 L1374.13 1147.99 L1374.36 1148.87 L1374.58 1148.48 L1374.81 1148.35 L1375.04 1148.95 L1375.27 1147.78 L1375.5 1148.38 L1375.72 1148.52 L1375.95 1148.38 L1376.18 1148.9 L1376.41 1148.72 L1376.63 1148.96 L1376.86 1148.94 L1377.09 1149.91 L1377.32 1149.16 L1377.55 1149.23 L1377.77 1149.9 L1378 1149.91 L1378.23 1150.06 L1378.46 1149.65 L1378.68 1149.93 L1378.91 1149.37 L1379.14 1149.45 L1379.37 1149.07 L1379.6 1149.56 L1379.82 1149.32 L1380.05 1149.28 L1380.28 1149.45 L1380.51 1149.95 L1380.73 1149.8 L1380.96 1150.51 L1381.19 1150.34 L1381.42 1150.17 L1381.65 1150.18 L1381.87 1150.31 L1382.1 1150.48 L1382.33 1150.72 L1382.56 1150.31 L1382.78 1150.1 L1383.01 1150.08 L1383.24 1150.57 L1383.47 1149.85 L1383.7 1149.24 L1383.92 1149.31 L1384.15 1150.14 L1384.38 1149.84 L1384.61 1149.97 L1384.83 1150.04 L1385.06 1149.53 L1385.29 1149.33 L1385.52 1148.74 L1385.75 1148.7 L1385.97 1148.13 L1386.2 1148.71 L1386.43 1148.1 L1386.66 1147.88 L1386.88 1148.03 L1387.11 1149.25 L1387.34 1148.74 L1387.57 1149.57 L1387.8 1149.55 L1388.02 1149.18 L1388.25 1148.47 L1388.48 1147.74 L1388.71 1147.75 L1388.93 1148.24 L1389.16 1148.85 L1389.39 1148.36 L1389.62 1148.44 L1389.85 1148.53 L1390.07 1147.65 L1390.3 1147.81 L1390.53 1148.13 L1390.76 1148.72 L1390.98 1149.37 L1391.21 1149.32 L1391.44 1148.7 L1391.67 1148.36 L1391.9 1148.29 L1392.12 1148.15 L1392.35 1148.46 L1392.58 1148.62 L1392.81 1148.57 L1393.03 1149.09 L1393.26 1148.91 L1393.49 1148.87 L1393.72 1148.86 L1393.95 1147.78 L1394.17 1146.97 L1394.4 1146.86 L1394.63 1147.28 L1394.86 1147.04 L1395.08 1147.63 L1395.31 1148.05 L1395.54 1148.04 L1395.77 1148.43 L1396 1148.4 L1396.22 1148.49 L1396.45 1148.15 L1396.68 1148.04 L1396.91 1147.01 L1397.13 1147.4 L1397.36 1146.64 L1397.59 1146.78 L1397.82 1147.26 L1398.05 1146.89 L1398.27 1146.47 L1398.5 1146.72 L1398.73 1146.31 L1398.96 1145.16 L1399.18 1144.76 L1399.41 1145.07 L1399.64 1145.89 L1399.87 1146.15 L1400.1 1145.37 L1400.32 1145.25 L1400.55 1145.02 L1400.78 1144.57 L1401.01 1143.78 L1401.23 1144.45 L1401.46 1144.6 L1401.69 1145.14 L1401.92 1145.03 L1402.15 1143.87 L1402.37 1144.01 L1402.6 1144.53 L1402.83 1144.76 L1403.06 1144.82 L1403.28 1145.36 L1403.51 1145.69 L1403.74 1145.42 L1403.97 1145.72 L1404.19 1146.52 L1404.42 1147.31 L1404.65 1148.23 L1404.88 1149.24 L1405.11 1149.46 L1405.33 1148.31 L1405.56 1147.71 L1405.79 1147.85 L1406.02 1148.56 L1406.24 1148.35 L1406.47 1148.95 L1406.7 1148.63 L1406.93 1147.95 L1407.16 1149.1 L1407.38 1148.83 L1407.61 1148.38 L1407.84 1148.56 L1408.07 1148.72 L1408.29 1147.55 L1408.52 1147.76 L1408.75 1147.54 L1408.98 1147.06 L1409.21 1147.34 L1409.43 1147.31 L1409.66 1148.31 L1409.89 1148.25 L1410.12 1148.74 L1410.34 1148.52 L1410.57 1148.88 L1410.8 1148.69 L1411.03 1148.38 L1411.26 1149.26 L1411.48 1148.99 L1411.71 1148.2 L1411.94 1147.68 L1412.17 1148.3 L1412.39 1148.18 L1412.62 1147.86 L1412.85 1148.18 L1413.08 1148.3 L1413.31 1147.81 L1413.53 1147.31 L1413.76 1146.68 L1413.99 1147.45 L1414.22 1147.28 L1414.44 1147.09 L1414.67 1146.85 L1414.9 1147.33 L1415.13 1148.1 L1415.36 1147.51 L1415.58 1147.68 L1415.81 1146.4 L1416.04 1146.05 L1416.27 1145.49 L1416.49 1145.45 L1416.72 1145.96 L1416.95 1145.86 L1417.18 1146.22 L1417.41 1145.79 L1417.63 1146.01 L1417.86 1146.13 L1418.09 1146.17 L1418.32 1145.84 L1418.54 1146.57 L1418.77 1146.82 L1419 1147.13 L1419.23 1145.9 L1419.46 1146.66 L1419.68 1146.33 L1419.91 1146.8 L1420.14 1147.39 L1420.37 1147.33 L1420.59 1147.08 L1420.82 1147.17 L1421.05 1147.31 L1421.28 1146.53 L1421.51 1146.73 L1421.73 1146.86 L1421.96 1147 L1422.19 1146.7 L1422.42 1146.6 L1422.64 1145.88 L1422.87 1146.06 L1423.1 1146.32 L1423.33 1146.18 L1423.56 1146.01 L1423.78 1146.05 L1424.01 1146.94 L1424.24 1147.87 L1424.47 1148.62 L1424.69 1147.69 L1424.92 1148.85 L1425.15 1148.71 L1425.38 1149.75 L1425.61 1149.87 L1425.83 1149.77 L1426.06 1149.75 L1426.29 1150.22 L1426.52 1150 L1426.74 1150.84 L1426.97 1151.36 L1427.2 1150.29 L1427.43 1150.22 L1427.66 1150.19 L1427.88 1150.44 L1428.11 1150.94 L1428.34 1151.4 L1428.57 1151.33 L1428.79 1151.75 L1429.02 1152 L1429.25 1151.46 L1429.48 1151.79 L1429.71 1151.81 L1429.93 1151.56 L1430.16 1151.89 L1430.39 1151.51 L1430.62 1151.73 L1430.84 1152.4 L1431.07 1152.8 L1431.3 1152.76 L1431.53 1151.64 L1431.76 1151.03 L1431.98 1151.79 L1432.21 1150.14 L1432.44 1149.54 L1432.67 1150.14 L1432.89 1150.68 L1433.12 1150.68 L1433.35 1150.57 L1433.58 1150.11 L1433.81 1150.41 L1434.03 1151.06 L1434.26 1151.78 L1434.49 1151.66 L1434.72 1152.07 L1434.94 1152.61 L1435.17 1152.81 L1435.4 1151.09 L1435.63 1151.02 L1435.86 1150.46 L1436.08 1149.97 L1436.31 1150.29 L1436.54 1150.84 L1436.77 1150.84 L1436.99 1150.08 L1437.22 1149.53 L1437.45 1149.7 L1437.68 1150.02 L1437.91 1150.19 L1438.13 1150.47 L1438.36 1150.57 L1438.59 1151.13 L1438.82 1150.9 L1439.04 1151.09 L1439.27 1150.11 L1439.5 1149.56 L1439.73 1150.44 L1439.95 1150.52 L1440.18 1149.76 L1440.41 1150.54 L1440.64 1150.31 L1440.87 1150.54 L1441.09 1149.99 L1441.32 1150 L1441.55 1150.35 L1441.78 1150.21 L1442 1149.83 L1442.23 1149.03 L1442.46 1148.74 L1442.69 1148.7 L1442.92 1148.63 L1443.14 1149.22 L1443.37 1148.37 L1443.6 1148.12 L1443.83 1147.52 L1444.05 1147.48 L1444.28 1147.39 L1444.51 1147.4 L1444.74 1147.41 L1444.97 1147.9 L1445.19 1148.04 L1445.42 1147.78 L1445.65 1147.22 L1445.88 1147.24 L1446.1 1147.75 L1446.33 1147.58 L1446.56 1146.99 L1446.79 1146.63 L1447.02 1146.99 L1447.24 1146.86 L1447.47 1146.08 L1447.7 1145.34 L1447.93 1145.29 L1448.15 1144.38 L1448.38 1144.36 L1448.61 1144.35 L1448.84 1143.96 L1449.07 1143.88 L1449.29 1143.55 L1449.52 1143.32 L1449.75 1143.36 L1449.98 1143.79 L1450.2 1143.65 L1450.43 1143.29 L1450.66 1144.21 L1450.89 1143.52 L1451.12 1143.47 L1451.34 1143.27 L1451.57 1144.02 L1451.8 1143.2 L1452.03 1143.32 L1452.25 1143.17 L1452.48 1142.15 L1452.71 1142.47 L1452.94 1143.01 L1453.17 1143.06 L1453.39 1143.28 L1453.62 1144.1 L1453.85 1144.34 L1454.08 1144.07 L1454.3 1143.73 L1454.53 1143.58 L1454.76 1144.02 L1454.99 1143.76 L1455.22 1144.75 L1455.44 1144.71 L1455.67 1144.33 L1455.9 1143.98 L1456.13 1144.72 L1456.35 1144.76 L1456.58 1144.83 L1456.81 1145.28 L1457.04 1144.62 L1457.27 1144.71 L1457.49 1144.15 L1457.72 1144.43 L1457.95 1143.91 L1458.18 1144.73 L1458.4 1145.22 L1458.63 1145.26 L1458.86 1144.93 L1459.09 1145.14 L1459.32 1145.85 L1459.54 1146.04 L1459.77 1146.41 L1460 1146.4 L1460.23 1146.53 L1460.45 1146 L1460.68 1146.43 L1460.91 1146.64 L1461.14 1146.69 L1461.37 1146.04 L1461.59 1145.75 L1461.82 1145.52 L1462.05 1145.02 L1462.28 1145.49 L1462.5 1146.09 L1462.73 1146.15 L1462.96 1145.58 L1463.19 1145.65 L1463.42 1146.36 L1463.64 1146.5 L1463.87 1145.97 L1464.1 1145.25 L1464.33 1145.43 L1464.55 1145.06 L1464.78 1145.07 L1465.01 1145.34 L1465.24 1145.42 L1465.47 1145.51 L1465.69 1145.87 L1465.92 1146.32 L1466.15 1146.82 L1466.38 1146.65 L1466.6 1147.21 L1466.83 1148.02 L1467.06 1148.62 L1467.29 1148.11 L1467.52 1148.26 L1467.74 1148.32 L1467.97 1148.07 L1468.2 1148.21 L1468.43 1147.84 L1468.65 1148.22 L1468.88 1148.54 L1469.11 1147.71 L1469.34 1147.4 L1469.57 1147.46 L1469.79 1147.82 L1470.02 1147.09 L1470.25 1146.72 L1470.48 1145.65 L1470.7 1145.83 L1470.93 1146.18 L1471.16 1146.1 L1471.39 1146.55 L1471.62 1146.65 L1471.84 1146.88 L1472.07 1147.83 L1472.3 1148.36 L1472.53 1148.84 L1472.75 1148.99 L1472.98 1149.3 L1473.21 1149.35 L1473.44 1149.45 L1473.67 1149.94 L1473.89 1150.46 L1474.12 1150.79 L1474.35 1150.92 L1474.58 1151.28 L1474.8 1151.42 L1475.03 1150.38 L1475.26 1150.54 L1475.49 1149.8 L1475.71 1150.51 L1475.94 1150.66 L1476.17 1150.8 L1476.4 1150.05 L1476.63 1150.4 L1476.85 1150.84 L1477.08 1150.91 L1477.31 1150.47 L1477.54 1151.22 L1477.76 1151.78 L1477.99 1150.79 L1478.22 1150.18 L1478.45 1150.47 L1478.68 1150.26 L1478.9 1150.13 L1479.13 1150.03 L1479.36 1150.06 L1479.59 1150.95 L1479.81 1150.45 L1480.04 1149.67 L1480.27 1150.37 L1480.5 1150.38 L1480.73 1150.73 L1480.95 1150.17 L1481.18 1150.94 L1481.41 1150.56 L1481.64 1150.41 L1481.86 1150.57 L1482.09 1149.89 L1482.32 1150.03 L1482.55 1150.28 L1482.78 1149.7 L1483 1149.46 L1483.23 1150.84 L1483.46 1150.97 L1483.69 1150.38 L1483.91 1150.42 L1484.14 1150.23 L1484.37 1150.25 L1484.6 1150.47 L1484.83 1150.47 L1485.05 1150.42 L1485.28 1151.5 L1485.51 1152.02 L1485.74 1151.62 L1485.96 1151.16 L1486.19 1151.06 L1486.42 1150.84 L1486.65 1151.15 L1486.88 1150.62 L1487.1 1150.37 L1487.33 1150.51 L1487.56 1150.08 L1487.79 1149.64 L1488.01 1150.01 L1488.24 1150 L1488.47 1150.39 L1488.7 1150.37 L1488.93 1150.97 L1489.15 1151.06 L1489.38 1150.95 L1489.61 1150.89 L1489.84 1150.95 L1490.06 1150.77 L1490.29 1150.78 L1490.52 1150.49 L1490.75 1150.52 L1490.98 1150.42 L1491.2 1150.3 L1491.43 1149.75 L1491.66 1149.28 L1491.89 1149.48 L1492.11 1149.4 L1492.34 1149.55 L1492.57 1149.18 L1492.8 1149.7 L1493.03 1148.91 L1493.25 1149.22 L1493.48 1149.07 L1493.71 1148.86 L1493.94 1149.39 L1494.16 1149.1 L1494.39 1149.35 L1494.62 1150.03 L1494.85 1150.41 L1495.08 1151.16 L1495.3 1150.8 L1495.53 1150.65 L1495.76 1150.29 L1495.99 1149.69 L1496.21 1150.18 L1496.44 1151.26 L1496.67 1151.66 L1496.9 1152.38 L1497.13 1151.69 L1497.35 1151.89 L1497.58 1151.36 L1497.81 1151.65 L1498.04 1152.19 L1498.26 1152.41 L1498.49 1152.88 L1498.72 1153.03 L1498.95 1153.1 L1499.18 1152.92 L1499.4 1152.21 L1499.63 1151.64 L1499.86 1152.36 L1500.09 1152.02 L1500.31 1151.56 L1500.54 1152.12 L1500.77 1152.86 L1501 1153.61 L1501.23 1153.03 L1501.45 1152.98 L1501.68 1153.71 L1501.91 1154.41 L1502.14 1154.03 L1502.36 1154.11 L1502.59 1154.57 L1502.82 1154.73 L1503.05 1154.82 L1503.28 1153.56 L1503.5 1153.45 L1503.73 1154.21 L1503.96 1153.27 L1504.19 1151.89 L1504.41 1152.78 L1504.64 1152.82 L1504.87 1152.79 L1505.1 1152.59 L1505.33 1151.99 L1505.55 1151.66 L1505.78 1152.58 L1506.01 1152.2 L1506.24 1152.79 L1506.46 1153.01 L1506.69 1153.23 L1506.92 1153.63 L1507.15 1152.53 L1507.38 1152.7 L1507.6 1152.4 L1507.83 1152.07 L1508.06 1151.72 L1508.29 1151.23 L1508.51 1152.08 L1508.74 1151.19 L1508.97 1151.07 L1509.2 1150.64 L1509.43 1150.37 L1509.65 1150.39 L1509.88 1151.08 L1510.11 1150.74 L1510.34 1150.96 L1510.56 1151.54 L1510.79 1151.51 L1511.02 1151.32 L1511.25 1151.51 L1511.48 1150.88 L1511.7 1151.14 L1511.93 1151.11 L1512.16 1150.71 L1512.39 1151.56 L1512.61 1152.02 L1512.84 1150.49 L1513.07 1150.78 L1513.3 1150.47 L1513.52 1150.94 L1513.75 1150.82 L1513.98 1151.47 L1514.21 1151.63 L1514.44 1151.54 L1514.66 1151.51 L1514.89 1151.73 L1515.12 1151.9 L1515.35 1151.74 L1515.57 1151.79 L1515.8 1151.95 L1516.03 1151.58 L1516.26 1151.83 L1516.49 1151.3 L1516.71 1151.31 L1516.94 1151.86 L1517.17 1152.05 L1517.4 1151.95 L1517.62 1151.35 L1517.85 1150.54 L1518.08 1151.42 L1518.31 1151.57 L1518.54 1151.46 L1518.76 1151.64 L1518.99 1152.11 L1519.22 1150.82 L1519.45 1150.22 L1519.67 1150.08 L1519.9 1150.58 L1520.13 1150.54 L1520.36 1150.77 L1520.59 1150.38 L1520.81 1150.23 L1521.04 1150.36 L1521.27 1150.72 L1521.5 1150.65 L1521.72 1151.02 L1521.95 1151.54 L1522.18 1151.6 L1522.41 1151.42 L1522.64 1150.58 L1522.86 1150.48 L1523.09 1151.38 L1523.32 1150.9 L1523.55 1149.66 L1523.77 1148.89 L1524 1148.25 L1524.23 1147.75 L1524.46 1147.71 L1524.69 1147.6 L1524.91 1147.43 L1525.14 1146.87 L1525.37 1147.23 L1525.6 1147.04 L1525.82 1146.93 L1526.05 1146.68 L1526.28 1147.41 L1526.51 1148.11 L1526.74 1147.89 L1526.96 1147.83 L1527.19 1148.14 L1527.42 1148.55 L1527.65 1149.37 L1527.87 1149.24 L1528.1 1148.5 L1528.33 1147.99 L1528.56 1147.88 L1528.79 1148.34 L1529.01 1148.01 L1529.24 1148.32 L1529.47 1148.83 L1529.7 1149.26 L1529.92 1148.69 L1530.15 1149.04 L1530.38 1149.76 L1530.61 1150.38 L1530.84 1150.63 L1531.06 1150.76 L1531.29 1150.5 L1531.52 1150.85 L1531.75 1150.83 L1531.97 1151.09 L1532.2 1150.52 L1532.43 1150.32 L1532.66 1151.09 L1532.89 1150.64 L1533.11 1150.45 L1533.34 1150.47 L1533.57 1150.79 L1533.8 1151.31 L1534.02 1151.69 L1534.25 1151.37 L1534.48 1151.98 L1534.71 1151.85 L1534.94 1151.2 L1535.16 1150.92 L1535.39 1150.74 L1535.62 1151.02 L1535.85 1151.1 L1536.07 1150.11 L1536.3 1150.16 L1536.53 1150.57 L1536.76 1149.98 L1536.99 1150.41 L1537.21 1150.23 L1537.44 1150.72 L1537.67 1149.71 L1537.9 1149.48 L1538.12 1149.83 L1538.35 1150.29 L1538.58 1149.02 L1538.81 1148.48 L1539.04 1148.38 L1539.26 1148.21 L1539.49 1147.81 L1539.72 1147.76 L1539.95 1147.72 L1540.17 1148.43 L1540.4 1148.44 L1540.63 1148.66 L1540.86 1148.56 L1541.09 1148.98 L1541.31 1148.74 L1541.54 1149.54 L1541.77 1149.38 L1542 1149.52 L1542.22 1149.9 L1542.45 1150.39 L1542.68 1150.16 L1542.91 1150.11 L1543.14 1150.01 L1543.36 1149.91 L1543.59 1150.3 L1543.82 1150.66 L1544.05 1150.71 L1544.27 1150.95 L1544.5 1151.42 L1544.73 1151.51 L1544.96 1151.88 L1545.19 1151.61 L1545.41 1151.18 L1545.64 1150.99 L1545.87 1150.29 L1546.1 1150.68 L1546.32 1150.6 L1546.55 1150.53 L1546.78 1149.79 L1547.01 1149.52 L1547.24 1149.09 L1547.46 1149.82 L1547.69 1149.99 L1547.92 1150.24 L1548.15 1150.21 L1548.37 1149.91 L1548.6 1150.17 L1548.83 1149.56 L1549.06 1149.48 L1549.28 1149.05 L1549.51 1149.49 L1549.74 1149.04 L1549.97 1149.92 L1550.2 1150.01 L1550.42 1149.55 L1550.65 1149.26 L1550.88 1149.53 L1551.11 1148.75 L1551.33 1149 L1551.56 1149.11 L1551.79 1149.03 L1552.02 1148.91 L1552.25 1148.06 L1552.47 1148.41 L1552.7 1148.13 L1552.93 1148.37 L1553.16 1148.24 L1553.38 1147.11 L1553.61 1147.17 L1553.84 1147.63 L1554.07 1147.02 L1554.3 1147.21 L1554.52 1147.6 L1554.75 1147.14 L1554.98 1146.9 L1555.21 1146.45 L1555.43 1146.99 L1555.66 1147.18 L1555.89 1147.77 L1556.12 1147.19 L1556.35 1147.56 L1556.57 1147.42 L1556.8 1147.53 L1557.03 1147.94 L1557.26 1147.68 L1557.48 1147.91 L1557.71 1147.63 L1557.94 1147.69 L1558.17 1148 L1558.4 1147.41 L1558.62 1147.04 L1558.85 1146.92 L1559.08 1146.3 L1559.31 1145.47 L1559.53 1145.66 L1559.76 1145.6 L1559.99 1145.61 L1560.22 1146.17 L1560.45 1145.47 L1560.67 1145.44 L1560.9 1145.65 L1561.13 1145.95 L1561.36 1146 L1561.58 1145.68 L1561.81 1145.93 L1562.04 1146.07 L1562.27 1145.79 L1562.5 1146.14 L1562.72 1146 L1562.95 1145.47 L1563.18 1145.51 L1563.41 1146.01 L1563.63 1146.14 L1563.86 1146.74 L1564.09 1146.11 L1564.32 1146.69 L1564.55 1146.45 L1564.77 1146.04 L1565 1146.63 L1565.23 1146.91 L1565.46 1147.21 L1565.68 1147.21 L1565.91 1146.21 L1566.14 1145.29 L1566.37 1145.94 L1566.6 1145.34 L1566.82 1145.66 L1567.05 1145.84 L1567.28 1145.05 L1567.51 1145.25 L1567.73 1145.14 L1567.96 1145.13 L1568.19 1145.32 L1568.42 1145.45 L1568.65 1145.59 L1568.87 1145.22 L1569.1 1145.49 L1569.33 1145.21 L1569.56 1145.51 L1569.78 1146.23 L1570.01 1146.29 L1570.24 1146.31 L1570.47 1146.48 L1570.7 1146.08 L1570.92 1146.19 L1571.15 1144.75 L1571.38 1145.03 L1571.61 1145.59 L1571.83 1145.23 L1572.06 1144.74 L1572.29 1144.71 L1572.52 1145.33 L1572.75 1145.09 L1572.97 1145.17 L1573.2 1145.98 L1573.43 1146.07 L1573.66 1145.64 L1573.88 1146.2 L1574.11 1146.53 L1574.34 1146.34 L1574.57 1145.36 L1574.8 1145.38 L1575.02 1144.76 L1575.25 1144.92 L1575.48 1145.36 L1575.71 1145.1 L1575.93 1144.24 L1576.16 1144.4 L1576.39 1144.4 L1576.62 1143.81 L1576.85 1143.57 L1577.07 1143.88 L1577.3 1143.29 L1577.53 1142.5 L1577.76 1141.71 L1577.98 1141.65 L1578.21 1141.59 L1578.44 1142.14 L1578.67 1142.38 L1578.9 1141.96 L1579.12 1142.11 L1579.35 1141.83 L1579.58 1142.28 L1579.81 1142.18 L1580.03 1142.57 L1580.26 1143.13 L1580.49 1143.09 L1580.72 1143.44 L1580.95 1143.65 L1581.17 1143.97 L1581.4 1144.55 L1581.63 1144.96 L1581.86 1145.09 L1582.08 1144.7 L1582.31 1145.07 L1582.54 1144.81 L1582.77 1144.68 L1583 1144.48 L1583.22 1145.12 L1583.45 1144.78 L1583.68 1144.13 L1583.91 1144.44 L1584.13 1144.05 L1584.36 1144.11 L1584.59 1142.96 L1584.82 1142.87 L1585.04 1142.33 L1585.27 1142.88 L1585.5 1143.51 L1585.73 1143.55 L1585.96 1144.07 L1586.18 1144.03 L1586.41 1144.59 L1586.64 1144.76 L1586.87 1145.26 L1587.09 1145.28 L1587.32 1145.2 L1587.55 1144.98 L1587.78 1145.14 L1588.01 1145.02 L1588.23 1145.61 L1588.46 1145.59 L1588.69 1145.42 L1588.92 1145.93 L1589.14 1146.46 L1589.37 1147.07 L1589.6 1146.74 L1589.83 1146.22 L1590.06 1145.65 L1590.28 1146.31 L1590.51 1146.86 L1590.74 1147.38 L1590.97 1147.17 L1591.19 1147.04 L1591.42 1145.89 L1591.65 1145.65 L1591.88 1145.43 L1592.11 1145.91 L1592.33 1145.48 L1592.56 1146.31 L1592.79 1146.19 L1593.02 1145.51 L1593.24 1145.7 L1593.47 1145.83 L1593.7 1146.33 L1593.93 1146.69 L1594.16 1146.98 L1594.38 1147.44 L1594.61 1147.72 L1594.84 1147.39 L1595.07 1146.38 L1595.29 1146.74 L1595.52 1146.77 L1595.75 1145.81 L1595.98 1145.39 L1596.21 1144.39 L1596.43 1144.22 L1596.66 1143.92 L1596.89 1143.79 L1597.12 1144.46 L1597.34 1144.14 L1597.57 1143.94 L1597.8 1144.79 L1598.03 1145.21 L1598.26 1145.3 L1598.48 1144.83 L1598.71 1144.79 L1598.94 1144.95 L1599.17 1144.35 L1599.39 1144.65 L1599.62 1144.35 L1599.85 1144.56 L1600.08 1143.95 L1600.31 1144.18 L1600.53 1145.1 L1600.76 1144.91 L1600.99 1144.63 L1601.22 1144.38 L1601.44 1144.28 L1601.67 1143.73 L1601.9 1144.17 L1602.13 1144.59 L1602.36 1144.81 L1602.58 1144.8 L1602.81 1144.36 L1603.04 1144.27 L1603.27 1144.49 L1603.49 1145.09 L1603.72 1145.55 L1603.95 1145.52 L1604.18 1145.77 L1604.41 1145.11 L1604.63 1145.46 L1604.86 1145.72 L1605.09 1146.13 L1605.32 1146.62 L1605.54 1146.55 L1605.77 1146.69 L1606 1146.03 L1606.23 1145.78 L1606.46 1145.94 L1606.68 1146.17 L1606.91 1146.45 L1607.14 1146.36 L1607.37 1146.21 L1607.59 1146.56 L1607.82 1147.04 L1608.05 1147.37 L1608.28 1147.23 L1608.51 1147.22 L1608.73 1146.62 L1608.96 1146.35 L1609.19 1145.93 L1609.42 1145.84 L1609.64 1146.09 L1609.87 1145.4 L1610.1 1145.55 L1610.33 1145.28 L1610.56 1146.05 L1610.78 1145.99 L1611.01 1146.33 L1611.24 1146.64 L1611.47 1146.89 L1611.69 1146.49 L1611.92 1146.87 L1612.15 1146.72 L1612.38 1147.52 L1612.61 1146.86 L1612.83 1147.3 L1613.06 1148.4 L1613.29 1148.78 L1613.52 1147.76 L1613.74 1147.36 L1613.97 1147.09 L1614.2 1147.47 L1614.43 1147.32 L1614.66 1147.04 L1614.88 1147.01 L1615.11 1146.29 L1615.34 1146.59 L1615.57 1147.32 L1615.79 1146.19 L1616.02 1146.42 L1616.25 1146.87 L1616.48 1146.88 L1616.71 1146.74 L1616.93 1147 L1617.16 1147.12 L1617.39 1146.12 L1617.62 1146.35 L1617.84 1146.4 L1618.07 1146.19 L1618.3 1145.66 L1618.53 1145.75 L1618.76 1146.06 L1618.98 1146.27 L1619.21 1146.1 L1619.44 1146.04 L1619.67 1145.84 L1619.89 1144.77 L1620.12 1145.16 L1620.35 1145.03 L1620.58 1145.1 L1620.81 1144.84 L1621.03 1145.04 L1621.26 1145.93 L1621.49 1146.25 L1621.72 1146.29 L1621.94 1146.02 L1622.17 1145.85 L1622.4 1145.36 L1622.63 1145.35 L1622.85 1145.5 L1623.08 1145.78 L1623.31 1145.83 L1623.54 1146.43 L1623.77 1146 L1623.99 1145.77 L1624.22 1146.25 L1624.45 1145.87 L1624.68 1145.42 L1624.9 1145.36 L1625.13 1145.55 L1625.36 1145.21 L1625.59 1145.42 L1625.82 1146.07 L1626.04 1145.49 L1626.27 1145.32 L1626.5 1144.76 L1626.73 1143.32 L1626.95 1143.69 L1627.18 1143.91 L1627.41 1143.5 L1627.64 1143.8 L1627.87 1144.52 L1628.09 1144.68 L1628.32 1144.63 L1628.55 1143.86 L1628.78 1144.05 L1629 1144.8 L1629.23 1144.55 L1629.46 1144.42 L1629.69 1144.43 L1629.92 1144.61 L1630.14 1144.77 L1630.37 1144.81 L1630.6 1144.55 L1630.83 1144.09 L1631.05 1143.62 L1631.28 1143.09 L1631.51 1144.1 L1631.74 1144.16 L1631.97 1144.39 L1632.19 1144.57 L1632.42 1144.96 L1632.65 1144.81 L1632.88 1144.24 L1633.1 1143.98 L1633.33 1143.75 L1633.56 1143.24 L1633.79 1143.42 L1634.02 1144 L1634.24 1143.01 L1634.47 1142.92 L1634.7 1143.37 L1634.93 1143.18 L1635.15 1143.49 L1635.38 1143.23 L1635.61 1143.56 L1635.84 1143.78 L1636.07 1143.98 L1636.29 1143.51 L1636.52 1143.57 L1636.75 1143.33 L1636.98 1143.5 L1637.2 1144.15 L1637.43 1144.05 L1637.66 1144.53 L1637.89 1144.46 L1638.12 1144.83 L1638.34 1144.46 L1638.57 1145.2 L1638.8 1146.03 L1639.03 1146.23 L1639.25 1146.52 L1639.48 1146.11 L1639.71 1146.64 L1639.94 1146.85 L1640.17 1147.25 L1640.39 1146.72 L1640.62 1147.01 L1640.85 1147.01 L1641.08 1146.96 L1641.3 1147.41 L1641.53 1147.26 L1641.76 1146.94 L1641.99 1147.72 L1642.22 1147.79 L1642.44 1147.89 L1642.67 1148.29 L1642.9 1148.48 L1643.13 1149.2 L1643.35 1150.03 L1643.58 1150.02 L1643.81 1149.78 L1644.04 1149.95 L1644.27 1149.79 L1644.49 1149.91 L1644.72 1150.31 L1644.95 1149.93 L1645.18 1149.47 L1645.4 1150.48 L1645.63 1151.35 L1645.86 1151.57 L1646.09 1151.56 L1646.32 1151.88 L1646.54 1151.99 L1646.77 1151.65 L1647 1152.39 L1647.23 1152.07 L1647.45 1151.56 L1647.68 1151.22 L1647.91 1151.45 L1648.14 1151.49 L1648.37 1151.64 L1648.59 1151.08 L1648.82 1151.26 L1649.05 1151.73 L1649.28 1152.51 L1649.5 1152.29 L1649.73 1153.06 L1649.96 1153.61 L1650.19 1153.11 L1650.42 1152.32 L1650.64 1151.83 L1650.87 1151.74 L1651.1 1151.9 L1651.33 1152.51 L1651.55 1152.29 L1651.78 1152.17 L1652.01 1152.35 L1652.24 1152.64 L1652.47 1153.16 L1652.69 1152.99 L1652.92 1152.39 L1653.15 1152.51 L1653.38 1152.62 L1653.6 1151.59 L1653.83 1151.81 L1654.06 1151.75 L1654.29 1151.28 L1654.52 1151.82 L1654.74 1152.87 L1654.97 1152.71 L1655.2 1152.28 L1655.43 1151.69 L1655.65 1152.53 L1655.88 1152.75 L1656.11 1152.98 L1656.34 1153.56 L1656.57 1153.62 L1656.79 1153.87 L1657.02 1154.29 L1657.25 1154.63 L1657.48 1154.73 L1657.7 1154.49 L1657.93 1154.86 L1658.16 1154.32 L1658.39 1153.99 L1658.61 1153.56 L1658.84 1153 L1659.07 1153.12 L1659.3 1153.51 L1659.53 1153.52 L1659.75 1153.15 L1659.98 1153.71 L1660.21 1154.23 L1660.44 1153.77 L1660.66 1153.53 L1660.89 1152.74 L1661.12 1152.73 L1661.35 1152.21 L1661.58 1151.53 L1661.8 1151.76 L1662.03 1151.89 L1662.26 1151.78 L1662.49 1151.52 L1662.71 1151.73 L1662.94 1151.91 L1663.17 1151.45 L1663.4 1151.57 L1663.63 1151.53 L1663.85 1151.99 L1664.08 1151.79 L1664.31 1151.49 L1664.54 1152.22 L1664.76 1152.51 L1664.99 1152.22 L1665.22 1152.96 L1665.45 1153.63 L1665.68 1153.79 L1665.9 1153.4 L1666.13 1153.6 L1666.36 1153.76 L1666.59 1153.97 L1666.81 1154.53 L1667.04 1154.34 L1667.27 1154.4 L1667.5 1154.89 L1667.73 1154.88 L1667.95 1154.97 L1668.18 1155.27 L1668.41 1155.78 L1668.64 1155.83 L1668.86 1155.36 L1669.09 1155.84 L1669.32 1155.78 L1669.55 1156.13 L1669.78 1156.13 L1670 1156.78 L1670.23 1157.23 L1670.46 1157.83 L1670.69 1158.38 L1670.91 1158.01 L1671.14 1158.18 L1671.37 1158.72 L1671.6 1158.77 L1671.83 1159.06 L1672.05 1159.5 L1672.28 1158.12 L1672.51 1158.36 L1672.74 1158.34 L1672.96 1158.33 L1673.19 1158.17 L1673.42 1158.21 L1673.65 1158.09 L1673.88 1158.4 L1674.1 1158.46 L1674.33 1158.58 L1674.56 1158.81 L1674.79 1158.99 L1675.01 1158.89 L1675.24 1158.35 L1675.47 1158.9 L1675.7 1158.33 L1675.93 1158.16 L1676.15 1158.17 L1676.38 1157.62 L1676.61 1158.11 L1676.84 1158.27 L1677.06 1158.16 L1677.29 1158.64 L1677.52 1158.96 L1677.75 1159.02 L1677.98 1159.36 L1678.2 1159.71 L1678.43 1159.21 L1678.66 1158.44 L1678.89 1157.49 L1679.11 1157.03 L1679.34 1156.81 L1679.57 1156.99 L1679.8 1156.6 L1680.03 1155.69 L1680.25 1155.67 L1680.48 1155.57 L1680.71 1155.02 L1680.94 1154.83 L1681.16 1155.17 L1681.39 1154.96 L1681.62 1155.24 L1681.85 1154.92 L1682.08 1154.71 L1682.3 1155.23 L1682.53 1154.75 L1682.76 1154.73 L1682.99 1154.69 L1683.21 1154.87 L1683.44 1154.75 L1683.67 1155.16 L1683.9 1154.17 L1684.13 1154.44 L1684.35 1154.15 L1684.58 1154.64 L1684.81 1153.78 L1685.04 1154.33 L1685.26 1154.15 L1685.49 1154.03 L1685.72 1154.18 L1685.95 1154.01 L1686.18 1154.14 L1686.4 1154.32 L1686.63 1155.04 L1686.86 1155.15 L1687.09 1155.55 L1687.31 1155.98 L1687.54 1156.19 L1687.77 1156.63 L1688 1156.44 L1688.23 1156.37 L1688.45 1156.36 L1688.68 1156.2 L1688.91 1156.73 L1689.14 1156.52 L1689.36 1156.14 L1689.59 1156.56 L1689.82 1156.74 L1690.05 1156.88 L1690.28 1156.96 L1690.5 1157.07 L1690.73 1157.14 L1690.96 1156.41 L1691.19 1156.53 L1691.41 1155.86 L1691.64 1155.99 L1691.87 1156.17 L1692.1 1155.52 L1692.33 1155.89 L1692.55 1156.15 L1692.78 1156.66 L1693.01 1157.04 L1693.24 1157.4 L1693.46 1157.04 L1693.69 1155.36 L1693.92 1155.3 L1694.15 1155.98 L1694.37 1155.86 L1694.6 1155.08 L1694.83 1155.43 L1695.06 1155.7 L1695.29 1155.79 L1695.51 1155.96 L1695.74 1155.28 L1695.97 1155.22 L1696.2 1155.54 L1696.42 1154.68 L1696.65 1155.12 L1696.88 1154.77 L1697.11 1155.07 L1697.34 1155.08 L1697.56 1154.72 L1697.79 1154.64 L1698.02 1154.43 L1698.25 1154.95 L1698.47 1155.22 L1698.7 1155.12 L1698.93 1155.16 L1699.16 1155.26 L1699.39 1155.24 L1699.61 1155.33 L1699.84 1155.08 L1700.07 1154.18 L1700.3 1153.88 L1700.52 1154.47 L1700.75 1154.94 L1700.98 1155.25 L1701.21 1155.99 L1701.44 1155.17 L1701.66 1155 L1701.89 1155.38 L1702.12 1155.38 L1702.35 1155.47 L1702.57 1156.05 L1702.8 1156.29 L1703.03 1156.8 L1703.26 1156.84 L1703.49 1157.06 L1703.71 1156.92 L1703.94 1156.22 L1704.17 1156.3 L1704.4 1156.23 L1704.62 1156.38 L1704.85 1156.25 L1705.08 1154.62 L1705.31 1155.26 L1705.54 1156.02 L1705.76 1156.29 L1705.99 1157.18 L1706.22 1157.14 L1706.45 1157.5 L1706.67 1157.69 L1706.9 1158.04 L1707.13 1158.47 L1707.36 1158.46 L1707.59 1157.87 L1707.81 1157.62 L1708.04 1158.38 L1708.27 1157.97 L1708.5 1158.35 L1708.72 1157.95 L1708.95 1157.72 L1709.18 1157.79 L1709.41 1157.82 L1709.64 1158.24 L1709.86 1158.55 L1710.09 1158.06 L1710.32 1158.3 L1710.55 1158.62 L1710.77 1157.69 L1711 1156.74 L1711.23 1157.62 L1711.46 1157.39 L1711.69 1157.36 L1711.91 1157.04 L1712.14 1156.98 L1712.37 1157.31 L1712.6 1156.61 L1712.82 1156.8 L1713.05 1156.69 L1713.28 1157.03 L1713.51 1156.85 L1713.74 1156.47 L1713.96 1156.28 L1714.19 1156.24 L1714.42 1156.37 L1714.65 1156.04 L1714.87 1156.21 L1715.1 1156.37 L1715.33 1155.84 L1715.56 1155.17 L1715.79 1155.36 L1716.01 1154.52 L1716.24 1154.02 L1716.47 1153.94 L1716.7 1153.39 L1716.92 1153.83 L1717.15 1154.07 L1717.38 1153.91 L1717.61 1154.08 L1717.84 1153.84 L1718.06 1154.1 L1718.29 1153.89 L1718.52 1153.91 L1718.75 1153.86 L1718.97 1154.25 L1719.2 1154.84 L1719.43 1154.47 L1719.66 1154.44 L1719.89 1154.22 L1720.11 1154.54 L1720.34 1154.83 L1720.57 1154.5 L1720.8 1154.67 L1721.02 1155.12 L1721.25 1154.39 L1721.48 1155.13 L1721.71 1155.36 L1721.94 1155.67 L1722.16 1156.08 L1722.39 1155.88 L1722.62 1156.04 L1722.85 1156.38 L1723.07 1156.03 L1723.3 1155.52 L1723.53 1155.62 L1723.76 1155.06 L1723.99 1155.14 L1724.21 1154.81 L1724.44 1154.83 L1724.67 1154.66 L1724.9 1154.37 L1725.12 1154.44 L1725.35 1153.89 L1725.58 1154.22 L1725.81 1154.62 L1726.04 1154.82 L1726.26 1154.64 L1726.49 1154.64 L1726.72 1155.12 L1726.95 1154.51 L1727.17 1154.86 L1727.4 1155.02 L1727.63 1155.18 L1727.86 1155.64 L1728.09 1155.36 L1728.31 1155.35 L1728.54 1155.69 L1728.77 1155.28 L1729 1155.43 L1729.22 1154.57 L1729.45 1154.7 L1729.68 1154.81 L1729.91 1155.18 L1730.14 1155.08 L1730.36 1154.78 L1730.59 1154.94 L1730.82 1155.34 L1731.05 1155.53 L1731.27 1155.81 L1731.5 1155.98 L1731.73 1156.5 L1731.96 1156.58 L1732.18 1156.39 L1732.41 1156.28 L1732.64 1156.11 L1732.87 1156.95 L1733.1 1156.96 L1733.32 1157.15 L1733.55 1157.59 L1733.78 1158.2 L1734.01 1158.1 L1734.23 1157.98 L1734.46 1158.44 L1734.69 1158.41 L1734.92 1158.33 L1735.15 1158.42 L1735.37 1158.27 L1735.6 1158.37 L1735.83 1158.35 L1736.06 1158.22 L1736.28 1158.41 L1736.51 1158.79 L1736.74 1158.63 L1736.97 1158.92 L1737.2 1158.45 L1737.42 1158.37 L1737.65 1158.46 L1737.88 1158.76 L1738.11 1158.05 L1738.33 1158 L1738.56 1158.03 L1738.79 1158.11 L1739.02 1158.68 L1739.25 1158.56 L1739.47 1157.9 L1739.7 1157.83 L1739.93 1157.86 L1740.16 1158.52 L1740.38 1158.92 L1740.61 1159.49 L1740.84 1159.41 L1741.07 1158.51 L1741.3 1158.29 L1741.52 1158.11 L1741.75 1157.38 L1741.98 1157.85 L1742.21 1157.86 L1742.43 1158.22 L1742.66 1158.63 L1742.89 1158.92 L1743.12 1159.53 L1743.35 1159.47 L1743.57 1159.42 L1743.8 1159.82 L1744.03 1160.06 L1744.26 1160.05 L1744.48 1160.03 L1744.71 1159.89 L1744.94 1160.19 L1745.17 1161.06 L1745.4 1160.28 L1745.62 1160.18 L1745.85 1160.2 L1746.08 1160.09 L1746.31 1160.8 L1746.53 1160.66 L1746.76 1160.52 L1746.99 1160.26 L1747.22 1160.35 L1747.45 1160.02 L1747.67 1159.87 L1747.9 1159.92 L1748.13 1159.74 L1748.36 1159.8 L1748.58 1159.56 L1748.81 1159.07 L1749.04 1159.5 L1749.27 1158.93 L1749.5 1159.06 L1749.72 1159.56 L1749.95 1159.32 L1750.18 1159.14 L1750.41 1159.41 L1750.63 1159.62 L1750.86 1159.77 L1751.09 1160.4 L1751.32 1160.52 L1751.55 1160.84 L1751.77 1161.69 L1752 1161.62 L1752.23 1160.9 L1752.46 1161.1 L1752.68 1161.18 L1752.91 1161.01 L1753.14 1161.24 L1753.37 1162.25 L1753.6 1161.72 L1753.82 1161.32 L1754.05 1161.25 L1754.28 1161.25 L1754.51 1161.1 L1754.73 1160.98 L1754.96 1161.13 L1755.19 1160.83 L1755.42 1160.37 L1755.65 1160.31 L1755.87 1160.31 L1756.1 1160.27 L1756.33 1159.99 L1756.56 1159.51 L1756.78 1159.06 L1757.01 1159.23 L1757.24 1159.16 L1757.47 1158.7 L1757.7 1159.02 L1757.92 1158.69 L1758.15 1158.39 L1758.38 1158.04 L1758.61 1158.65 L1758.83 1158.79 L1759.06 1158.67 L1759.29 1159.08 L1759.52 1159.71 L1759.75 1160.2 L1759.97 1159.67 L1760.2 1159.5 L1760.43 1159.3 L1760.66 1159.17 L1760.88 1159.36 L1761.11 1159.27 L1761.34 1159.12 L1761.57 1159.59 L1761.8 1159.64 L1762.02 1159.32 L1762.25 1159.46 L1762.48 1159.71 L1762.71 1159.34 L1762.93 1159.98 L1763.16 1159.88 L1763.39 1159.88 L1763.62 1158.74 L1763.85 1158.8 L1764.07 1158.69 L1764.3 1158.69 L1764.53 1158.69 L1764.76 1158.93 L1764.98 1159 L1765.21 1158.75 L1765.44 1158.28 L1765.67 1158.66 L1765.9 1158.82 L1766.12 1159.24 L1766.35 1159.86 L1766.58 1160.31 L1766.81 1159.6 L1767.03 1159 L1767.26 1159.16 L1767.49 1159.53 L1767.72 1159.54 L1767.94 1159.81 L1768.17 1159.79 L1768.4 1160.2 L1768.63 1160.31 L1768.86 1160.34 L1769.08 1160.77 L1769.31 1160.79 L1769.54 1160.87 L1769.77 1161.51 L1769.99 1161.56 L1770.22 1161.77 L1770.45 1162.01 L1770.68 1161.68 L1770.91 1161.33 L1771.13 1161.08 L1771.36 1161.52 L1771.59 1161.61 L1771.82 1161.18 L1772.04 1161.38 L1772.27 1160.62 L1772.5 1160.6 L1772.73 1161.19 L1772.96 1161.13 L1773.18 1161.14 L1773.41 1161.43 L1773.64 1160.82 L1773.87 1161.25 L1774.09 1161.17 L1774.32 1160.93 L1774.55 1161.15 L1774.78 1161.39 L1775.01 1160.54 L1775.23 1160.17 L1775.46 1160.08 L1775.69 1160.29 L1775.92 1160.78 L1776.14 1161 L1776.37 1161 L1776.6 1161.28 L1776.83 1161.6 L1777.06 1161.88 L1777.28 1161.45 L1777.51 1161.25 L1777.74 1161.42 L1777.97 1161.65 L1778.19 1161.67 L1778.42 1161.19 L1778.65 1161.08 L1778.88 1160.43 L1779.11 1160.56 L1779.33 1161.06 L1779.56 1161.27 L1779.79 1161.3 L1780.02 1160.87 L1780.24 1160.77 L1780.47 1161.12 L1780.7 1161.14 L1780.93 1160.41 L1781.16 1160.14 L1781.38 1160.16 L1781.61 1159.76 L1781.84 1159.57 L1782.07 1159.53 L1782.29 1159.52 L1782.52 1159.99 L1782.75 1158.07 L1782.98 1156.76 L1783.21 1157.18 L1783.43 1156.59 L1783.66 1156.68 L1783.89 1156.82 L1784.12 1156.55 L1784.34 1156.08 L1784.57 1156.26 L1784.8 1156.08 L1785.03 1156.44 L1785.26 1155.9 L1785.48 1156.58 L1785.71 1156.49 L1785.94 1155.75 L1786.17 1155.25 L1786.39 1155.93 L1786.62 1155.62 L1786.85 1155.62 L1787.08 1155.88 L1787.31 1156.2 L1787.53 1155.96 L1787.76 1156.46 L1787.99 1156.66 L1788.22 1155.99 L1788.44 1156.19 L1788.67 1156.33 L1788.9 1155.98 L1789.13 1155.82 L1789.36 1155.61 L1789.58 1154.99 L1789.81 1154.48 L1790.04 1154.92 L1790.27 1155.14 L1790.49 1155.32 L1790.72 1154.65 L1790.95 1154.7 L1791.18 1155.42 L1791.41 1155.32 L1791.63 1155.43 L1791.86 1154.84 L1792.09 1154.37 L1792.32 1154.94 L1792.54 1155.02 L1792.77 1154.66 L1793 1154.69 L1793.23 1154.58 L1793.46 1154.51 L1793.68 1154.98 L1793.91 1155.83 L1794.14 1156.3 L1794.37 1156.5 L1794.59 1156.4 L1794.82 1155.71 L1795.05 1155.29 L1795.28 1155.47 L1795.51 1154.74 L1795.73 1154.55 L1795.96 1154.89 L1796.19 1154.61 L1796.42 1155.32 L1796.64 1154.85 L1796.87 1155.28 L1797.1 1155.85 L1797.33 1155.91 L1797.56 1155.55 L1797.78 1155.73 L1798.01 1155.54 L1798.24 1155.97 L1798.47 1155.14 L1798.69 1155.4 L1798.92 1155.81 L1799.15 1155.93 L1799.38 1156.02 L1799.61 1156.65 L1799.83 1156.95 L1800.06 1156.69 L1800.29 1156.81 L1800.52 1156.68 L1800.74 1156.97 L1800.97 1156.2 L1801.2 1156.69 L1801.43 1156.84 L1801.66 1156.26 L1801.88 1156.6 L1802.11 1156.3 L1802.34 1156.43 L1802.57 1156.02 L1802.79 1155.98 L1803.02 1156.37 L1803.25 1156.71 L1803.48 1156.35 L1803.7 1156.61 L1803.93 1156.72 L1804.16 1156.36 L1804.39 1155.79 L1804.62 1156.15 L1804.84 1156.41 L1805.07 1155.92 L1805.3 1156.07 L1805.53 1155.84 L1805.75 1156.05 L1805.98 1155.79 L1806.21 1155.03 L1806.44 1155.66 L1806.67 1155.66 L1806.89 1155.79 L1807.12 1156.08 L1807.35 1156.12 L1807.58 1156.19 L1807.8 1156.81 L1808.03 1156.2 L1808.26 1155.94 L1808.49 1155.86 L1808.72 1155 L1808.94 1155.54 L1809.17 1155.75 L1809.4 1155.84 L1809.63 1155.04 L1809.85 1154.53 L1810.08 1154.69 L1810.31 1154.68 L1810.54 1154.6 L1810.77 1154.31 L1810.99 1154.76 L1811.22 1154.66 L1811.45 1154.88 L1811.68 1154.51 L1811.9 1154.38 L1812.13 1154.89 L1812.36 1155.18 L1812.59 1153.89 L1812.82 1154.37 L1813.04 1154.66 L1813.27 1154.77 L1813.5 1154.8 L1813.73 1154.59 L1813.95 1154.76 L1814.18 1155.09 L1814.41 1155.09 L1814.64 1154.97 L1814.87 1154.88 L1815.09 1154.8 L1815.32 1154.69 L1815.55 1154.56 L1815.78 1154.05 L1816 1153.96 L1816.23 1153.85 L1816.46 1153.73 L1816.69 1153.92 L1816.92 1154.17 L1817.14 1154.44 L1817.37 1154.11 L1817.6 1154.56 L1817.83 1154.54 L1818.05 1154.86 L1818.28 1154.09 L1818.51 1153.17 L1818.74 1152.76 L1818.97 1152.84 L1819.19 1152.56 L1819.42 1152.5 L1819.65 1151.77 L1819.88 1151.76 L1820.1 1152.12 L1820.33 1151.73 L1820.56 1152.22 L1820.79 1152.3 L1821.02 1152.25 L1821.24 1152.44 L1821.47 1151.87 L1821.7 1152.11 L1821.93 1152.08 L1822.15 1152.62 L1822.38 1152.22 L1822.61 1151.91 L1822.84 1152.05 L1823.07 1152 L1823.29 1151.97 L1823.52 1151.79 L1823.75 1151.23 L1823.98 1150.78 L1824.2 1150.52 L1824.43 1150.43 L1824.66 1150.4 L1824.89 1150.37 L1825.12 1150.47 L1825.34 1150.16 L1825.57 1150.71 L1825.8 1150.68 L1826.03 1151.16 L1826.25 1151.36 L1826.48 1151.68 L1826.71 1150.9 L1826.94 1150.76 L1827.17 1151.05 L1827.39 1151.08 L1827.62 1151.33 L1827.85 1151.95 L1828.08 1152.17 L1828.3 1152.89 L1828.53 1153.07 L1828.76 1152.09 L1828.99 1152.39 L1829.22 1152.41 L1829.44 1151.5 L1829.67 1152.1 L1829.9 1152.12 L1830.13 1151.16 L1830.35 1150.93 L1830.58 1150.91 L1830.81 1151.25 L1831.04 1151.73 L1831.27 1152 L1831.49 1152.37 L1831.72 1152.82 L1831.95 1152.84 L1832.18 1153.36 L1832.4 1152.94 L1832.63 1153 L1832.86 1153.57 L1833.09 1153.58 L1833.32 1154.27 L1833.54 1154.33 L1833.77 1154.61 L1834 1154.77 L1834.23 1154.81 L1834.45 1154.84 L1834.68 1154.55 L1834.91 1155.04 L1835.14 1155.51 L1835.37 1155.94 L1835.59 1155.9 L1835.82 1155.88 L1836.05 1155.8 L1836.28 1156.01 L1836.5 1156.07 L1836.73 1156.7 L1836.96 1156.62 L1837.19 1156.66 L1837.42 1156.54 L1837.64 1156.74 L1837.87 1157.13 L1838.1 1157.57 L1838.33 1157.48 L1838.55 1157.69 L1838.78 1157.23 L1839.01 1157.38 L1839.24 1157.16 L1839.46 1157.5 L1839.69 1157.33 L1839.92 1157.2 L1840.15 1157.38 L1840.38 1157.69 L1840.6 1157.86 L1840.83 1157.92 L1841.06 1158.14 L1841.29 1157.74 L1841.51 1157.65 L1841.74 1158.08 L1841.97 1158.27 L1842.2 1158.34 L1842.43 1158.8 L1842.65 1159.01 L1842.88 1159.7 L1843.11 1159.77 L1843.34 1159.31 L1843.56 1159.53 L1843.79 1159.39 L1844.02 1159.53 L1844.25 1159.89 L1844.48 1160.02 L1844.7 1160.16 L1844.93 1160.14 L1845.16 1160.26 L1845.39 1160.13 L1845.61 1159.87 L1845.84 1159.86 L1846.07 1159.85 L1846.3 1159.92 L1846.53 1159.87 L1846.75 1159.59 L1846.98 1160.37 L1847.21 1160.02 L1847.44 1160.12 L1847.66 1159.92 L1847.89 1159.95 L1848.12 1159.28 L1848.35 1159.05 L1848.58 1158.55 L1848.8 1158.18 L1849.03 1158.18 L1849.26 1157.8 L1849.49 1157.74 L1849.71 1157.33 L1849.94 1157.77 L1850.17 1157.87 L1850.4 1157.61 L1850.63 1157.95 L1850.85 1157.43 L1851.08 1158.15 L1851.31 1158.65 L1851.54 1158.54 L1851.76 1158.54 L1851.99 1158.32 L1852.22 1158.29 L1852.45 1157.94 L1852.68 1157.78 L1852.9 1157.37 L1853.13 1157.14 L1853.36 1156.75 L1853.59 1157.16 L1853.81 1157.13 L1854.04 1156.99 L1854.27 1156.85 L1854.5 1157.35 L1854.73 1157.59 L1854.95 1157.12 L1855.18 1156.71 L1855.41 1156.45 L1855.64 1156.17 L1855.86 1155.59 L1856.09 1155.41 L1856.32 1155.61 L1856.55 1154.93 L1856.78 1155.07 L1857 1154.98 L1857.23 1154.8 L1857.46 1155.25 L1857.69 1155.51 L1857.91 1155.5 L1858.14 1155.51 L1858.37 1154.88 L1858.6 1154.52 L1858.83 1154.74 L1859.05 1155.17 L1859.28 1154.69 L1859.51 1154.22 L1859.74 1154.09 L1859.96 1153.85 L1860.19 1153.98 L1860.42 1153.49 L1860.65 1154.49 L1860.88 1154.36 L1861.1 1154.2 L1861.33 1154.95 L1861.56 1154.73 L1861.79 1154.72 L1862.01 1154.64 L1862.24 1154.89 L1862.47 1155.55 L1862.7 1156.05 L1862.93 1155.84 L1863.15 1155.51 L1863.38 1155.48 L1863.61 1155.26 L1863.84 1155.29 L1864.06 1155.84 L1864.29 1155.89 L1864.52 1156.45 L1864.75 1156.74 L1864.98 1156.62 L1865.2 1157.19 L1865.43 1157.25 L1865.66 1157.68 L1865.89 1156.85 L1866.11 1156.89 L1866.34 1157.31 L1866.57 1157.96 L1866.8 1158.29 L1867.03 1158.76 L1867.25 1158.91 L1867.48 1158.03 L1867.71 1157.98 L1867.94 1157.97 L1868.16 1157.95 L1868.39 1158.29 L1868.62 1157.95 L1868.85 1157.76 L1869.08 1158.19 L1869.3 1158.48 L1869.53 1158.66 L1869.76 1158.95 L1869.99 1158.42 L1870.21 1157.95 L1870.44 1158.18 L1870.67 1158.33 L1870.9 1158.2 L1871.13 1158.01 L1871.35 1157.88 L1871.58 1157.74 L1871.81 1156.91 L1872.04 1157.2 L1872.26 1157.2 L1872.49 1156.91 L1872.72 1157.41 L1872.95 1157.55 L1873.18 1157.49 L1873.4 1157.57 L1873.63 1157.49 L1873.86 1157.31 L1874.09 1157.38 L1874.31 1157.64 L1874.54 1158.1 L1874.77 1157.54 L1875 1157.64 L1875.23 1157.71 L1875.45 1157.5 L1875.68 1158.01 L1875.91 1158.22 L1876.14 1158.51 L1876.36 1157.77 L1876.59 1157.92 L1876.82 1157.42 L1877.05 1157.95 L1877.27 1157.76 L1877.5 1158.01 L1877.73 1157.73 L1877.96 1157.61 L1878.19 1158.3 L1878.41 1158.24 L1878.64 1158.95 L1878.87 1159.23 L1879.1 1158.34 L1879.32 1158.38 L1879.55 1158.22 L1879.78 1158.31 L1880.01 1158.21 L1880.24 1158.68 L1880.46 1159.11 L1880.69 1159.09 L1880.92 1158.95 L1881.15 1159.45 L1881.37 1159.22 L1881.6 1159.82 L1881.83 1160.15 L1882.06 1160.51 L1882.29 1161.05 L1882.51 1161.09 L1882.74 1161.24 L1882.97 1161.71 L1883.2 1161.72 L1883.42 1162.07 L1883.65 1162.3 L1883.88 1162.57 L1884.11 1162.28 L1884.34 1162.74 L1884.56 1162.61 L1884.79 1162.44 L1885.02 1162.29 L1885.25 1161.72 L1885.47 1162.16 L1885.7 1162.77 L1885.93 1162.85 L1886.16 1163.28 L1886.39 1163.2 L1886.61 1163.2 L1886.84 1162.69 L1887.07 1162.63 L1887.3 1163.06 L1887.52 1162.69 L1887.75 1162.49 L1887.98 1162.06 L1888.21 1161.8 L1888.44 1161.97 L1888.66 1162.09 L1888.89 1162.1 L1889.12 1162.11 L1889.35 1162.44 L1889.57 1162.59 L1889.8 1162.56 L1890.03 1162.08 L1890.26 1161.25 L1890.49 1161.03 L1890.71 1160.43 L1890.94 1160.31 L1891.17 1159.91 L1891.4 1159.61 L1891.62 1158.86 L1891.85 1159.09 L1892.08 1158.8 L1892.31 1157.92 L1892.54 1157.86 L1892.76 1157.77 L1892.99 1158.13 L1893.22 1158.34 L1893.45 1158.19 L1893.67 1158.4 L1893.9 1158.76 L1894.13 1159.25 L1894.36 1159.67 L1894.59 1159.45 L1894.81 1159.56 L1895.04 1158.93 L1895.27 1158.34 L1895.5 1158.16 L1895.72 1158.28 L1895.95 1157.37 L1896.18 1157.09 L1896.41 1157.83 L1896.64 1157.34 L1896.86 1157.05 L1897.09 1157.41 L1897.32 1157.89 L1897.55 1157.23 L1897.77 1157.56 L1898 1157.19 L1898.23 1156.87 L1898.46 1156.64 L1898.69 1156.59 L1898.91 1156.69 L1899.14 1156.83 L1899.37 1156.61 L1899.6 1156.52 L1899.82 1157.15 L1900.05 1157.17 L1900.28 1156.59 L1900.51 1157.02 L1900.74 1157.34 L1900.96 1157.32 L1901.19 1157.83 L1901.42 1158.06 L1901.65 1158.28 L1901.87 1158.37 L1902.1 1158.95 L1902.33 1159.26 L1902.56 1159.2 L1902.79 1158.43 L1903.01 1158.39 L1903.24 1158.58 L1903.47 1159.05 L1903.7 1158.88 L1903.92 1159.13 L1904.15 1159.09 L1904.38 1159.38 L1904.61 1160.14 L1904.84 1160.38 L1905.06 1160.33 L1905.29 1160.91 L1905.52 1160.9 L1905.75 1160.24 L1905.97 1160.37 L1906.2 1160.51 L1906.43 1159.52 L1906.66 1159.45 L1906.89 1160.03 L1907.11 1160.81 L1907.34 1161.03 L1907.57 1160.27 L1907.8 1161.05 L1908.02 1160.7 L1908.25 1160.89 L1908.48 1160.7 L1908.71 1160.76 L1908.94 1160.86 L1909.16 1160.66 L1909.39 1160.37 L1909.62 1161.14 L1909.85 1160.59 L1910.07 1160.35 L1910.3 1160.26 L1910.53 1160.28 L1910.76 1160.23 L1910.99 1160.28 L1911.21 1159.67 L1911.44 1159.57 L1911.67 1159.63 L1911.9 1159.44 L1912.12 1158.92 L1912.35 1158.49 L1912.58 1158.94 L1912.81 1158.18 L1913.03 1157.51 L1913.26 1158.13 L1913.49 1157.87 L1913.72 1157.57 L1913.95 1157.9 L1914.17 1158.16 L1914.4 1158.6 L1914.63 1158.3 L1914.86 1158.12 L1915.08 1158.43 L1915.31 1157.99 L1915.54 1158.48 L1915.77 1157.89 L1916 1157.82 L1916.22 1157.66 L1916.45 1157.63 L1916.68 1157.36 L1916.91 1157.4 L1917.13 1157.53 L1917.36 1157.88 L1917.59 1158.14 L1917.82 1157.86 L1918.05 1157.74 L1918.27 1157.05 L1918.5 1156.82 L1918.73 1156.18 L1918.96 1154.83 L1919.18 1155.53 L1919.41 1154.43 L1919.64 1154.5 L1919.87 1155.09 L1920.1 1154.7 L1920.32 1154.76 L1920.55 1155.54 L1920.78 1155.41 L1921.01 1155.32 L1921.23 1155.72 L1921.46 1155.29 L1921.69 1155.24 L1921.92 1155.08 L1922.15 1155.35 L1922.37 1155.27 L1922.6 1155.32 L1922.83 1155.72 L1923.06 1155.99 L1923.28 1155.85 L1923.51 1156.42 L1923.74 1156.68 L1923.97 1157.1 L1924.2 1156.63 L1924.42 1156.07 L1924.65 1156.27 L1924.88 1156.38 L1925.11 1156.27 L1925.33 1156.59 L1925.56 1156.77 L1925.79 1156.19 L1926.02 1155.77 L1926.25 1156.3 L1926.47 1156.72 L1926.7 1156.85 L1926.93 1157.16 L1927.16 1156.82 L1927.38 1156.13 L1927.61 1156.64 L1927.84 1156.59 L1928.07 1156.66 L1928.3 1156.35 L1928.52 1156.25 L1928.75 1156.45 L1928.98 1156.52 L1929.21 1157.04 L1929.43 1157.6 L1929.66 1157.9 L1929.89 1157.88 L1930.12 1156.84 L1930.35 1156.98 L1930.57 1157.19 L1930.8 1156.68 L1931.03 1157 L1931.26 1157.04 L1931.48 1156.91 L1931.71 1157.07 L1931.94 1157.24 L1932.17 1157.44 L1932.4 1157.85 L1932.62 1158.53 L1932.85 1159.03 L1933.08 1159.37 L1933.31 1159.07 L1933.53 1158.65 L1933.76 1158.74 L1933.99 1159.3 L1934.22 1158.52 L1934.45 1158.53 L1934.67 1158.02 L1934.9 1157.63 L1935.13 1157.47 L1935.36 1157.38 L1935.58 1157.65 L1935.81 1158.13 L1936.04 1157.92 L1936.27 1157.74 L1936.5 1157.71 L1936.72 1157.83 L1936.95 1158.07 L1937.18 1157.83 L1937.41 1156.98 L1937.63 1157.57 L1937.86 1158.12 L1938.09 1158.53 L1938.32 1158.5 L1938.55 1158.62 L1938.77 1158.6 L1939 1158.87 L1939.23 1158.75 L1939.46 1158.73 L1939.68 1159.12 L1939.91 1158.99 L1940.14 1158.24 L1940.37 1158.3 L1940.6 1158.15 L1940.82 1158.27 L1941.05 1158.67 L1941.28 1159.1 L1941.51 1158.96 L1941.73 1159.14 L1941.96 1158.49 L1942.19 1158.56 L1942.42 1158.67 L1942.65 1158.35 L1942.87 1158.22 L1943.1 1157.92 L1943.33 1156.82 L1943.56 1156.86 L1943.78 1156.78 L1944.01 1157.14 L1944.24 1156.93 L1944.47 1156.79 L1944.7 1157.17 L1944.92 1157.25 L1945.15 1157.49 L1945.38 1157.04 L1945.61 1156.56 L1945.83 1156.45 L1946.06 1156.41 L1946.29 1155.84 L1946.52 1156.51 L1946.75 1156.95 L1946.97 1156.95 L1947.2 1157.02 L1947.43 1156.98 L1947.66 1156.96 L1947.88 1156.44 L1948.11 1156.91 L1948.34 1156.96 L1948.57 1157.34 L1948.79 1157.57 L1949.02 1157.24 L1949.25 1157.33 L1949.48 1157.23 L1949.71 1157 L1949.93 1156.6 L1950.16 1157.03 L1950.39 1157.21 L1950.62 1156.86 L1950.84 1156.49 L1951.07 1156.55 L1951.3 1156.92 L1951.53 1157.18 L1951.76 1157.37 L1951.98 1156.92 L1952.21 1156.97 L1952.44 1157.05 L1952.67 1156.96 L1952.89 1157.29 L1953.12 1157.94 L1953.35 1157.78 L1953.58 1158.35 L1953.81 1157.42 L1954.03 1157.68 L1954.26 1158.02 L1954.49 1157.63 L1954.72 1157.6 L1954.94 1157.92 L1955.17 1157.73 L1955.4 1157.73 L1955.63 1157.89 L1955.86 1158.24 L1956.08 1158.14 L1956.31 1158.15 L1956.54 1158.26 L1956.77 1158.46 L1956.99 1158.22 L1957.22 1158.61 L1957.45 1158.72 L1957.68 1158.58 L1957.91 1157.96 L1958.13 1157.77 L1958.36 1158.33 L1958.59 1157.97 L1958.82 1158.24 L1959.04 1158.59 L1959.27 1158.38 L1959.5 1158.38 L1959.73 1158.5 L1959.96 1158.87 L1960.18 1159.41 L1960.41 1158.57 L1960.64 1158.63 L1960.87 1158.33 L1961.09 1158.06 L1961.32 1157.62 L1961.55 1157.06 L1961.78 1156.65 L1962.01 1156.89 L1962.23 1157.23 L1962.46 1157.67 L1962.69 1157.9 L1962.92 1157.55 L1963.14 1157.76 L1963.37 1157.82 L1963.6 1158.4 L1963.83 1158.1 L1964.06 1158.26 L1964.28 1157.42 L1964.51 1156.85 L1964.74 1156.12 L1964.97 1156.21 L1965.19 1156.35 L1965.42 1156.23 L1965.65 1156.34 L1965.88 1156.25 L1966.11 1156.26 L1966.33 1156.04 L1966.56 1156.59 L1966.79 1156.83 L1967.02 1156.52 L1967.24 1156.85 L1967.47 1156.61 L1967.7 1157.13 L1967.93 1157.26 L1968.16 1157.18 L1968.38 1157.44 L1968.61 1157.5 L1968.84 1157.28 L1969.07 1157.35 L1969.29 1156.72 L1969.52 1157.12 L1969.75 1157.18 L1969.98 1157.1 L1970.21 1156.52 L1970.43 1156.89 L1970.66 1156.17 L1970.89 1156.53 L1971.12 1156.92 L1971.34 1156.91 L1971.57 1156.66 L1971.8 1156.69 L1972.03 1157.33 L1972.26 1157.61 L1972.48 1158.06 L1972.71 1158.28 L1972.94 1158.32 L1973.17 1158.85 L1973.39 1158.84 L1973.62 1158.89 L1973.85 1159.19 L1974.08 1159.3 L1974.31 1159.6 L1974.53 1160.02 L1974.76 1159.76 L1974.99 1160.15 L1975.22 1159.71 L1975.44 1159.93 L1975.67 1160.27 L1975.9 1160.53 L1976.13 1160.06 L1976.36 1160.29 L1976.58 1159.28 L1976.81 1159.52 L1977.04 1159.32 L1977.27 1159.39 L1977.49 1159.19 L1977.72 1159.04 L1977.95 1158.97 L1978.18 1158.89 L1978.41 1158.95 L1978.63 1159.21 L1978.86 1159 L1979.09 1158.76 L1979.32 1158.63 L1979.54 1158.54 L1979.77 1158.1 L1980 1158.03 L1980.23 1157.8 L1980.46 1158.01 L1980.68 1157.54 L1980.91 1157.78 L1981.14 1158.04 L1981.37 1157.97 L1981.59 1157.76 L1981.82 1158.07 L1982.05 1157.8 L1982.28 1157.41 L1982.51 1157.63 L1982.73 1158.28 L1982.96 1158.34 L1983.19 1158.1 L1983.42 1158.63 L1983.64 1158.81 L1983.87 1158.46 L1984.1 1158.84 L1984.33 1158.73 L1984.56 1159.04 L1984.78 1159.35 L1985.01 1158.98 L1985.24 1159.28 L1985.47 1159.29 L1985.69 1159.85 L1985.92 1159.67 L1986.15 1159.18 L1986.38 1159.19 L1986.6 1158.65 L1986.83 1158.83 L1987.06 1158.89 L1987.29 1158.09 L1987.52 1157.95 L1987.74 1158.13 L1987.97 1158.33 L1988.2 1158.32 L1988.43 1158.59 L1988.65 1158.48 L1988.88 1158.34 L1989.11 1158.34 L1989.34 1158.4 L1989.57 1158.52 L1989.79 1158.06 L1990.02 1157.46 L1990.25 1157.24 L1990.48 1157.38 L1990.7 1157.71 L1990.93 1157.12 L1991.16 1157.24 L1991.39 1157.25 L1991.62 1157.75 L1991.84 1158.19 L1992.07 1158.54 L1992.3 1158.83 L1992.53 1158.42 L1992.75 1158.63 L1992.98 1158.71 L1993.21 1158.68 L1993.44 1158.86 L1993.67 1158.99 L1993.89 1158.87 L1994.12 1159.13 L1994.35 1158.7 L1994.58 1158.61 L1994.8 1158.38 L1995.03 1158.22 L1995.26 1157.82 L1995.49 1158.02 L1995.72 1158.13 L1995.94 1157.67 L1996.17 1157.58 L1996.4 1156.88 L1996.63 1157.28 L1996.85 1157.61 L1997.08 1157.17 L1997.31 1156.72 L1997.54 1156.94 L1997.77 1157.02 L1997.99 1157.16 L1998.22 1156.44 L1998.45 1156.73 L1998.68 1157.19 L1998.9 1157.47 L1999.13 1157.21 L1999.36 1157.21 L1999.59 1157.35 L1999.82 1156.72 L2000.04 1156.54 L2000.27 1156.3 L2000.5 1156.52 L2000.73 1156.39 L2000.95 1156.54 L2001.18 1156.44 L2001.41 1156.41 L2001.64 1156.65 L2001.87 1156.56 L2002.09 1156.79 L2002.32 1156.78 L2002.55 1156.85 L2002.78 1156.38 L2003 1156.16 L2003.23 1156.64 L2003.46 1156.92 L2003.69 1156.68 L2003.92 1156.03 L2004.14 1156.2 L2004.37 1156.73 L2004.6 1156.19 L2004.83 1155.45 L2005.05 1155.08 L2005.28 1155.45 L2005.51 1155.57 L2005.74 1155.79 L2005.97 1155.68 L2006.19 1155.9 L2006.42 1156.15 L2006.65 1156.23 L2006.88 1155.95 L2007.1 1154.89 L2007.33 1154.48 L2007.56 1154.21 L2007.79 1153.81 L2008.02 1154.37 L2008.24 1154.24 L2008.47 1154.41 L2008.7 1154.19 L2008.93 1153.85 L2009.15 1154.27 L2009.38 1153.83 L2009.61 1154.12 L2009.84 1154.29 L2010.07 1154.25 L2010.29 1154.24 L2010.52 1154.58 L2010.75 1154.61 L2010.98 1154.77 L2011.2 1154.8 L2011.43 1155.56 L2011.66 1155.74 L2011.89 1155.88 L2012.12 1155.65 L2012.34 1155.28 L2012.57 1155.72 L2012.8 1155.36 L2013.03 1155.38 L2013.25 1155.36 L2013.48 1155.38 L2013.71 1155.13 L2013.94 1155.92 L2014.17 1155.61 L2014.39 1154.87 L2014.62 1154.97 L2014.85 1154.77 L2015.08 1154.96 L2015.3 1154.48 L2015.53 1154.5 L2015.76 1154.32 L2015.99 1154.43 L2016.22 1154.22 L2016.44 1154.51 L2016.67 1154.76 L2016.9 1154.88 L2017.13 1154.86 L2017.35 1154.53 L2017.58 1154.61 L2017.81 1154.53 L2018.04 1153.7 L2018.27 1153.74 L2018.49 1154.28 L2018.72 1154.52 L2018.95 1154.15 L2019.18 1153.59 L2019.4 1153.47 L2019.63 1153.65 L2019.86 1153.37 L2020.09 1153.84 L2020.32 1154.05 L2020.54 1153.61 L2020.77 1154.01 L2021 1154.13 L2021.23 1153.81 L2021.45 1153.94 L2021.68 1154.23 L2021.91 1154.07 L2022.14 1153.92 L2022.36 1153.69 L2022.59 1154.1 L2022.82 1153.8 L2023.05 1153.98 L2023.28 1153.45 L2023.5 1153.89 L2023.73 1153.5 L2023.96 1153.5 L2024.19 1153.15 L2024.41 1153.38 L2024.64 1153.72 L2024.87 1154.3 L2025.1 1153.77 L2025.33 1154.39 L2025.55 1154.2 L2025.78 1154.03 L2026.01 1154.49 L2026.24 1153.81 L2026.46 1154.27 L2026.69 1154.09 L2026.92 1153.35 L2027.15 1153.02 L2027.38 1153 L2027.6 1152.88 L2027.83 1152.96 L2028.06 1152.94 L2028.29 1153.19 L2028.51 1152.95 L2028.74 1152.93 L2028.97 1152.68 L2029.2 1152.39 L2029.43 1152.33 L2029.65 1152.1 L2029.88 1152.4 L2030.11 1152.21 L2030.34 1151.67 L2030.56 1151.31 L2030.79 1151.84 L2031.02 1152.12 L2031.25 1152.36 L2031.48 1152.35 L2031.7 1152.56 L2031.93 1152.92 L2032.16 1153.07 L2032.39 1153.03 L2032.61 1153.66 L2032.84 1153.78 L2033.07 1153.63 L2033.3 1152.97 L2033.53 1152.67 L2033.75 1152.56 L2033.98 1153.1 L2034.21 1153.14 L2034.44 1152.87 L2034.66 1152.62 L2034.89 1152.8 L2035.12 1152.85 L2035.35 1152.41 L2035.58 1152.68 L2035.8 1152.53 L2036.03 1153.05 L2036.26 1153.17 L2036.49 1153 L2036.71 1153.44 L2036.94 1153.49 L2037.17 1153.49 L2037.4 1153.44 L2037.63 1153.64 L2037.85 1153.07 L2038.08 1153.21 L2038.31 1153.02 L2038.54 1153.03 L2038.76 1152.82 L2038.99 1153.24 L2039.22 1153.24 L2039.45 1153.29 L2039.68 1153.54 L2039.9 1153.6 L2040.13 1153.39 L2040.36 1153.6 L2040.59 1153.23 L2040.81 1153.4 L2041.04 1153.03 L2041.27 1153.27 L2041.5 1153.59 L2041.73 1153.71 L2041.95 1153.49 L2042.18 1153.66 L2042.41 1153.52 L2042.64 1153.38 L2042.86 1153.87 L2043.09 1153.83 L2043.32 1153.63 L2043.55 1152.94 L2043.78 1152.83 L2044 1153.25 L2044.23 1153.34 L2044.46 1153.23 L2044.69 1153.24 L2044.91 1153.12 L2045.14 1153.16 L2045.37 1153.5 L2045.6 1153.58 L2045.83 1153.83 L2046.05 1153.7 L2046.28 1154 L2046.51 1153.43 L2046.74 1153.81 L2046.96 1154.43 L2047.19 1153.95 L2047.42 1153.77 L2047.65 1153.93 L2047.88 1154.21 L2048.1 1153.85 L2048.33 1154.07 L2048.56 1154.22 L2048.79 1154.27 L2049.01 1153.23 L2049.24 1153.15 L2049.47 1152.62 L2049.7 1152.87 L2049.93 1152.87 L2050.15 1152.82 L2050.38 1152.77 L2050.61 1153.02 L2050.84 1153.43 L2051.06 1152.84 L2051.29 1153.06 L2051.52 1153.29 L2051.75 1153.63 L2051.98 1153.72 L2052.2 1154.04 L2052.43 1153.49 L2052.66 1153.63 L2052.89 1153.28 L2053.11 1152.65 L2053.34 1152.6 L2053.57 1152.47 L2053.8 1151.95 L2054.03 1152.3 L2054.25 1152.48 L2054.48 1152.45 L2054.71 1152.82 L2054.94 1152.78 L2055.16 1152.55 L2055.39 1152.72 L2055.62 1152.71 L2055.85 1152.51 L2056.08 1152.55 L2056.3 1152.63 L2056.53 1152.5 L2056.76 1152.29 L2056.99 1151.79 L2057.21 1151.74 L2057.44 1152.34 L2057.67 1152.96 L2057.9 1152.7 L2058.12 1153 L2058.35 1152.62 L2058.58 1152.38 L2058.81 1152.06 L2059.04 1151.99 L2059.26 1152.56 L2059.49 1153.03 L2059.72 1153.34 L2059.95 1153.51 L2060.17 1153.98 L2060.4 1153.89 L2060.63 1154.44 L2060.86 1154.59 L2061.09 1154.44 L2061.31 1154.4 L2061.54 1154.59 L2061.77 1154.5 L2062 1154.28 L2062.22 1153.62 L2062.45 1154.09 L2062.68 1154.45 L2062.91 1154.7 L2063.14 1155.08 L2063.36 1155.29 L2063.59 1155.6 L2063.82 1155.37 L2064.05 1154.23 L2064.27 1154.55 L2064.5 1154.14 L2064.73 1153.98 L2064.96 1154.23 L2065.19 1154.09 L2065.41 1154.22 L2065.64 1153.64 L2065.87 1153.36 L2066.1 1153.19 L2066.32 1153.45 L2066.55 1153.99 L2066.78 1154.39 L2067.01 1154.35 L2067.24 1154.13 L2067.46 1154.58 L2067.69 1154.88 L2067.92 1154.5 L2068.15 1154.49 L2068.37 1153.83 L2068.6 1153.78 L2068.83 1153.59 L2069.06 1153.71 L2069.29 1154.38 L2069.51 1154.7 L2069.74 1154.85 L2069.97 1155.02 L2070.2 1154.39 L2070.42 1154.61 L2070.65 1154.42 L2070.88 1154.11 L2071.11 1154.08 L2071.34 1153.78 L2071.56 1154.06 L2071.79 1153.69 L2072.02 1154.38 L2072.25 1154.3 L2072.47 1154.09 L2072.7 1154.01 L2072.93 1154.21 L2073.16 1153.96 L2073.39 1153.45 L2073.61 1153.87 L2073.84 1153.66 L2074.07 1153.98 L2074.3 1153.53 L2074.52 1153.07 L2074.75 1152.93 L2074.98 1152.8 L2075.21 1152.68 L2075.44 1153.12 L2075.66 1153.21 L2075.89 1153.5 L2076.12 1154 L2076.35 1154.43 L2076.57 1154.23 L2076.8 1153.62 L2077.03 1153.97 L2077.26 1154.28 L2077.49 1154.68 L2077.71 1154.61 L2077.94 1154.87 L2078.17 1154.6 L2078.4 1155.14 L2078.62 1154.87 L2078.85 1155.29 L2079.08 1155.75 L2079.31 1155.81 L2079.54 1156.41 L2079.76 1157.18 L2079.99 1156.86 L2080.22 1157.07 L2080.45 1157.33 L2080.67 1157.43 L2080.9 1157.29 L2081.13 1157.31 L2081.36 1157.21 L2081.59 1156.83 L2081.81 1156.93 L2082.04 1156.91 L2082.27 1156.63 L2082.5 1156 L2082.72 1155.65 L2082.95 1156.19 L2083.18 1156.24 L2083.41 1155.8 L2083.64 1155.85 L2083.86 1155.59 L2084.09 1155.47 L2084.32 1155.52 L2084.55 1156.26 L2084.77 1156.54 L2085 1156.93 L2085.23 1157.29 L2085.46 1157.44 L2085.69 1156.85 L2085.91 1156.75 L2086.14 1156.78 L2086.37 1156.89 L2086.6 1156.43 L2086.82 1155.62 L2087.05 1155.52 L2087.28 1155.51 L2087.51 1155.32 L2087.74 1155.3 L2087.96 1154.71 L2088.19 1154.63 L2088.42 1154.9 L2088.65 1155.25 L2088.87 1155.65 L2089.1 1155.28 L2089.33 1155.12 L2089.56 1155.67 L2089.79 1155.87 L2090.01 1156.24 L2090.24 1155.81 L2090.47 1155.73 L2090.7 1155.67 L2090.92 1155.93 L2091.15 1156.32 L2091.38 1156.8 L2091.61 1157.03 L2091.84 1156.84 L2092.06 1156.72 L2092.29 1156.96 L2092.52 1156.73 L2092.75 1156.22 L2092.97 1156.68 L2093.2 1156.69 L2093.43 1156.75 L2093.66 1156.89 L2093.89 1157.14 L2094.11 1157.01 L2094.34 1156.79 L2094.57 1155.95 L2094.8 1155.94 L2095.02 1155.39 L2095.25 1155.31 L2095.48 1155.39 L2095.71 1155.43 L2095.93 1155.33 L2096.16 1155.86 L2096.39 1156.09 L2096.62 1156.11 L2096.85 1155.96 L2097.07 1156.22 L2097.3 1156.67 L2097.53 1156.99 L2097.76 1156.99 L2097.98 1157.08 L2098.21 1156.97 L2098.44 1157.24 L2098.67 1156.91 L2098.9 1157.39 L2099.12 1157.43 L2099.35 1156.93 L2099.58 1157.11 L2099.81 1157.04 L2100.03 1157.15 L2100.26 1157.82 L2100.49 1157.62 L2100.72 1157.46 L2100.95 1157.32 L2101.17 1157.21 L2101.4 1157.62 L2101.63 1157.48 L2101.86 1157.6 L2102.08 1157.66 L2102.31 1158.05 L2102.54 1158.05 L2102.77 1157.94 L2103 1158.32 L2103.22 1158.82 L2103.45 1159.42 L2103.68 1159.74 L2103.91 1159.67 L2104.13 1160.04 L2104.36 1159.86 L2104.59 1159.58 L2104.82 1159.45 L2105.05 1159.89 L2105.27 1159.75 L2105.5 1160.02 L2105.73 1159.37 L2105.96 1159.63 L2106.18 1160.13 L2106.41 1159.77 L2106.64 1159.93 L2106.87 1159.57 L2107.1 1159.11 L2107.32 1159.42 L2107.55 1159.81 L2107.78 1159.94 L2108.01 1160.39 L2108.23 1160.01 L2108.46 1159.11 L2108.69 1159.27 L2108.92 1159.6 L2109.15 1159.96 L2109.37 1159.97 L2109.6 1160.09 L2109.83 1160.69 L2110.06 1160.68 L2110.28 1160.32 L2110.51 1159.81 L2110.74 1159.79 L2110.97 1159.2 L2111.2 1158.63 L2111.42 1158.84 L2111.65 1159.01 L2111.88 1159.39 L2112.11 1159.67 L2112.33 1158.79 L2112.56 1158.87 L2112.79 1158.89 L2113.02 1158.87 L2113.25 1158.51 L2113.47 1158.34 L2113.7 1158.13 L2113.93 1157.96 L2114.16 1158.24 L2114.38 1158.4 L2114.61 1158.15 L2114.84 1157.29 L2115.07 1157.25 L2115.3 1157.48 L2115.52 1157.28 L2115.75 1157.38 L2115.98 1157.68 L2116.21 1158.01 L2116.43 1158.02 L2116.66 1157.42 L2116.89 1157.81 L2117.12 1158.06 L2117.35 1158.2 L2117.57 1158.07 L2117.8 1158.15 L2118.03 1158.19 L2118.26 1158.08 L2118.48 1158.6 L2118.71 1159.01 L2118.94 1159.07 L2119.17 1158.4 L2119.4 1158.52 L2119.62 1158.59 L2119.85 1158.42 L2120.08 1158.55 L2120.31 1158.7 L2120.53 1158.39 L2120.76 1157.92 L2120.99 1157.92 L2121.22 1158.23 L2121.45 1157.56 L2121.67 1157.86 L2121.9 1156.91 L2122.13 1157.3 L2122.36 1157.4 L2122.58 1157.27 L2122.81 1157.63 L2123.04 1157.73 L2123.27 1157.76 L2123.5 1157.96 L2123.72 1157.55 L2123.95 1157.62 L2124.18 1157.74 L2124.41 1157.59 L2124.63 1157.49 L2124.86 1157.51 L2125.09 1157.34 L2125.32 1157.61 L2125.55 1157.6 L2125.77 1157.61 L2126 1157.34 L2126.23 1157.55 L2126.46 1157.68 L2126.68 1157.71 L2126.91 1158.23 L2127.14 1158.67 L2127.37 1158.67 L2127.6 1158.85 L2127.82 1158.91 L2128.05 1159.29 L2128.28 1159.52 L2128.51 1159.8 L2128.73 1159.47 L2128.96 1159.38 L2129.19 1159.31 L2129.42 1159.34 L2129.65 1160.05 L2129.87 1160.42 L2130.1 1160.64 L2130.33 1160.86 L2130.56 1161.18 L2130.78 1160.41 L2131.01 1160.77 L2131.24 1160.57 L2131.47 1160.04 L2131.69 1160.24 L2131.92 1160.53 L2132.15 1160.44 L2132.38 1160.67 L2132.61 1160.97 L2132.83 1160.67 L2133.06 1161.15 L2133.29 1161.08 L2133.52 1161.13 L2133.74 1160.99 L2133.97 1160.9 L2134.2 1160.63 L2134.43 1160.89 L2134.66 1160.6 L2134.88 1160.53 L2135.11 1161.01 L2135.34 1160.82 L2135.57 1161.17 L2135.79 1161.02 L2136.02 1160.7 L2136.25 1160.85 L2136.48 1160.64 L2136.71 1160.35 L2136.93 1160.02 L2137.16 1160.41 L2137.39 1159.52 L2137.62 1159.55 L2137.84 1159.31 L2138.07 1159.38 L2138.3 1159.63 L2138.53 1160.36 L2138.76 1160.32 L2138.98 1159.87 L2139.21 1160.24 L2139.44 1160.2 L2139.67 1160.34 L2139.89 1160.53 L2140.12 1160.88 L2140.35 1161.1 L2140.58 1160.78 L2140.81 1161.25 L2141.03 1160.95 L2141.26 1160.96 L2141.49 1160.98 L2141.72 1161.29 L2141.94 1161.2 L2142.17 1161.2 L2142.4 1161.52 L2142.63 1161.33 L2142.86 1161.14 L2143.08 1160.44 L2143.31 1160.68 L2143.54 1160.88 L2143.77 1160.98 L2143.99 1160.9 L2144.22 1160.79 L2144.45 1160.73 L2144.68 1160.79 L2144.91 1160.69 L2145.13 1160.8 L2145.36 1160.76 L2145.59 1161.04 L2145.82 1160.69 L2146.04 1160.96 L2146.27 1160.89 L2146.5 1160.91 L2146.73 1160.56 L2146.96 1159.92 L2147.18 1160.17 L2147.41 1160.66 L2147.64 1160.37 L2147.87 1160.54 L2148.09 1160.49 L2148.32 1160.46 L2148.55 1161.3 L2148.78 1161.41 L2149.01 1161.32 L2149.23 1161.23 L2149.46 1161.29 L2149.69 1161.27 L2149.92 1161.02 L2150.14 1160.96 L2150.37 1160.03 L2150.6 1159.94 L2150.83 1160.16 L2151.06 1160.33 L2151.28 1159.8 L2151.51 1160.05 L2151.74 1159.78 L2151.97 1159.92 L2152.19 1159.56 L2152.42 1159.52 L2152.65 1159.6 L2152.88 1159.6 L2153.11 1159.79 L2153.33 1160.25 L2153.56 1160.94 L2153.79 1161.43 L2154.02 1161.59 L2154.24 1161.54 L2154.47 1160.93 L2154.7 1160.04 L2154.93 1159.68 L2155.16 1159.85 L2155.38 1160.31 L2155.61 1160.49 L2155.84 1160.61 L2156.07 1160.57 L2156.29 1160.57 L2156.52 1160.83 L2156.75 1160.31 L2156.98 1160.14 L2157.21 1160.19 L2157.43 1160.23 L2157.66 1160.12 L2157.89 1160.56 L2158.12 1160.93 L2158.34 1160.85 L2158.57 1160.66 L2158.8 1160.66 L2159.03 1161.08 L2159.26 1160.69 L2159.48 1160.84 L2159.71 1160.83 L2159.94 1160.37 L2160.17 1160.41 L2160.39 1160.29 L2160.62 1159.77 L2160.85 1159.43 L2161.08 1159.49 L2161.31 1159.88 L2161.53 1159.96 L2161.76 1160.05 L2161.99 1160.4 L2162.22 1160.43 L2162.44 1160.12 L2162.67 1160.69 L2162.9 1161.41 L2163.13 1161.15 L2163.36 1160.72 L2163.58 1160.31 L2163.81 1160.12 L2164.04 1160.3 L2164.27 1160.64 L2164.49 1160.5 L2164.72 1160.81 L2164.95 1161.1 L2165.18 1161.29 L2165.41 1161.01 L2165.63 1160.91 L2165.86 1160.99 L2166.09 1160.98 L2166.32 1160.65 L2166.54 1159.85 L2166.77 1159.42 L2167 1159.63 L2167.23 1160.03 L2167.45 1160.2 L2167.68 1159.35 L2167.91 1159.41 L2168.14 1159.64 L2168.37 1159.18 L2168.59 1158.88 L2168.82 1158.65 L2169.05 1158.75 L2169.28 1159 L2169.5 1159.27 L2169.73 1157.82 L2169.96 1157.54 L2170.19 1157.98 L2170.42 1157.97 L2170.64 1158.06 L2170.87 1158.63 L2171.1 1158.79 L2171.33 1159.16 L2171.55 1159.02 L2171.78 1158.6 L2172.01 1158.11 L2172.24 1158.25 L2172.47 1158.29 L2172.69 1158.23 L2172.92 1158.33 L2173.15 1158.36 L2173.38 1157.48 L2173.6 1157.6 L2173.83 1157.63 L2174.06 1157.77 L2174.29 1157.97 L2174.52 1157.81 L2174.74 1157.81 L2174.97 1158.21 L2175.2 1158.14 L2175.43 1158.26 L2175.65 1157.86 L2175.88 1158.21 L2176.11 1158.64 L2176.34 1158.44 L2176.57 1158.61 L2176.79 1158.57 L2177.02 1158.42 L2177.25 1158.38 L2177.48 1158.13 L2177.7 1157.8 L2177.93 1157.42 L2178.16 1157.31 L2178.39 1156.91 L2178.62 1156.8 L2178.84 1157.11 L2179.07 1157.07 L2179.3 1157.31 L2179.53 1156.83 L2179.75 1157.05 L2179.98 1157.07 L2180.21 1156.66 L2180.44 1157.05 L2180.67 1156.8 L2180.89 1156.57 L2181.12 1156.2 L2181.35 1156.8 L2181.58 1156.34 L2181.8 1156.45 L2182.03 1156.47 L2182.26 1157 L2182.49 1156.64 L2182.72 1156.66 L2182.94 1157.13 L2183.17 1157.42 L2183.4 1156.97 L2183.63 1156.77 L2183.85 1156.75 L2184.08 1156.9 L2184.31 1157.09 L2184.54 1157.14 L2184.77 1156.92 L2184.99 1156.33 L2185.22 1156.53 L2185.45 1157.02 L2185.68 1156.69 L2185.9 1156.8 L2186.13 1156.89 L2186.36 1156.51 L2186.59 1156.66 L2186.82 1156.95 L2187.04 1157.15 L2187.27 1157.19 L2187.5 1157.08 L2187.73 1156.69 L2187.95 1156.66 L2188.18 1156.82 L2188.41 1157.19 L2188.64 1157.68 L2188.87 1157.6 L2189.09 1157.45 L2189.32 1157.47 L2189.55 1156.83 L2189.78 1156.81 L2190 1156.78 L2190.23 1157.08 L2190.46 1157.12 L2190.69 1155.85 L2190.92 1156.15 L2191.14 1156.35 L2191.37 1156.13 L2191.6 1156.03 L2191.83 1155.9 L2192.05 1155.88 L2192.28 1155.98 L2192.51 1156.04 L2192.74 1156.15 L2192.97 1156.34 L2193.19 1155.79 L2193.42 1155.74 L2193.65 1155.63 L2193.88 1155.6 L2194.1 1156.05 L2194.33 1156.09 L2194.56 1156.61 L2194.79 1155.87 L2195.02 1155.33 L2195.24 1155.8 L2195.47 1155.85 L2195.7 1155.88 L2195.93 1156.36 L2196.15 1156.7 L2196.38 1156.74 L2196.61 1156.53 L2196.84 1156.65 L2197.07 1156.89 L2197.29 1157.29 L2197.52 1157.09 L2197.75 1157.13 L2197.98 1157.34 L2198.2 1157.73 L2198.43 1157.42 L2198.66 1157.55 L2198.89 1157.41 L2199.12 1157.05 L2199.34 1156.78 L2199.57 1156.71 L2199.8 1155.96 L2200.03 1155.55 L2200.25 1155.76 L2200.48 1155.88 L2200.71 1155.88 L2200.94 1156.03 L2201.17 1155.7 L2201.39 1156.13 L2201.62 1156.23 L2201.85 1155.93 L2202.08 1156.28 L2202.3 1155.78 L2202.53 1155.93 L2202.76 1155.95 L2202.99 1156.18 L2203.21 1155.76 L2203.44 1155.32 L2203.67 1155.36 L2203.9 1155.34 L2204.13 1155.3 L2204.35 1155.11 L2204.58 1155.08 L2204.81 1155.71 L2205.04 1155.87 L2205.26 1156.08 L2205.49 1155.94 L2205.72 1156.08 L2205.95 1156.16 L2206.18 1155.84 L2206.4 1155.48 L2206.63 1155.5 L2206.86 1155.58 L2207.09 1155.34 L2207.31 1155.81 L2207.54 1155.93 L2207.77 1155.58 L2208 1155.2 L2208.23 1154.55 L2208.45 1154.9 L2208.68 1154.82 L2208.91 1154.86 L2209.14 1154.86 L2209.36 1154.98 L2209.59 1154.95 L2209.82 1155.15 L2210.05 1155.25 L2210.28 1154.91 L2210.5 1155.1 L2210.73 1155.05 L2210.96 1154.99 L2211.19 1155.46 L2211.41 1155.06 L2211.64 1154.96 L2211.87 1154.85 L2212.1 1154.88 L2212.33 1155.1 L2212.55 1155.39 L2212.78 1155.38 L2213.01 1155.48 L2213.24 1155.68 L2213.46 1155.84 L2213.69 1156 L2213.92 1156.15 L2214.15 1155.73 L2214.38 1155.25 L2214.6 1154.69 L2214.83 1154.83 L2215.06 1154.46 L2215.29 1154.59 L2215.51 1154.5 L2215.74 1154.75 L2215.97 1154.4 L2216.2 1154.47 L2216.43 1154.67 L2216.65 1154.61 L2216.88 1154.06 L2217.11 1153.96 L2217.34 1154.48 L2217.56 1154.2 L2217.79 1154.29 L2218.02 1154.86 L2218.25 1154.57 L2218.48 1154.5 L2218.7 1154.16 L2218.93 1154.3 L2219.16 1154.59 L2219.39 1154.7 L2219.61 1154.81 L2219.84 1154.92 L2220.07 1155.27 L2220.3 1155.26 L2220.53 1155.51 L2220.75 1155.82 L2220.98 1155.74 L2221.21 1155.34 L2221.44 1155.08 L2221.66 1155.1 L2221.89 1154.69 L2222.12 1155.09 L2222.35 1155.01 L2222.58 1155.54 L2222.8 1156.06 L2223.03 1156.41 L2223.26 1156.13 L2223.49 1156.18 L2223.71 1155.47 L2223.94 1155.07 L2224.17 1155.08 L2224.4 1155.22 L2224.63 1155.12 L2224.85 1155.33 L2225.08 1155.36 L2225.31 1154.82 L2225.54 1155.42 L2225.76 1155.3 L2225.99 1155.2 L2226.22 1155.47 L2226.45 1154.65 L2226.68 1154.41 L2226.9 1154.4 L2227.13 1154.13 L2227.36 1154.58 L2227.59 1154.96 L2227.81 1155.37 L2228.04 1155.22 L2228.27 1155.26 L2228.5 1155.27 L2228.73 1155.29 L2228.95 1155.32 L2229.18 1155.22 L2229.41 1154.91 L2229.64 1155.12 L2229.86 1154.67 L2230.09 1154.91 L2230.32 1155.1 L2230.55 1155.31 L2230.78 1154.8 L2231 1154.46 L2231.23 1154.44 L2231.46 1154.74 L2231.69 1154.04 L2231.91 1154.1 L2232.14 1154.1 L2232.37 1153.92 L2232.6 1153.67 L2232.83 1153.14 L2233.05 1153.72 L2233.28 1153.89 L2233.51 1154.04 L2233.74 1154.4 L2233.96 1154.58 L2234.19 1154.67 L2234.42 1154.35 L2234.65 1154.03 L2234.88 1153.85 L2235.1 1154.01 L2235.33 1154.02 L2235.56 1153.88 L2235.79 1153.97 L2236.01 1154.33 L2236.24 1154.19 L2236.47 1153.92 L2236.7 1153.97 L2236.93 1154.28 L2237.15 1154.09 L2237.38 1154.17 L2237.61 1154.33 L2237.84 1154.13 L2238.06 1154.18 L2238.29 1153.73 L2238.52 1153.59 L2238.75 1154.02 L2238.98 1153.94 L2239.2 1154.17 L2239.43 1154.32 L2239.66 1154.59 L2239.89 1154.27 L2240.11 1153.62 L2240.34 1154.25 L2240.57 1154.41 L2240.8 1154.38 L2241.02 1154.31 L2241.25 1154.32 L2241.48 1154.37 L2241.71 1154.61 L2241.94 1154.6 L2242.16 1154.32 L2242.39 1154.42 L2242.62 1154.59 L2242.85 1154.91 L2243.07 1155.3 L2243.3 1155.77 L2243.53 1155.35 L2243.76 1155.67 L2243.99 1155.34 L2244.21 1155.39 L2244.44 1155.36 L2244.67 1155.79 L2244.9 1155.87 L2245.12 1155.91 L2245.35 1155.81 L2245.58 1156.4 L2245.81 1156.33 L2246.04 1156.33 L2246.26 1156.57 L2246.49 1156.35 L2246.72 1156.55 L2246.95 1156.35 L2247.17 1156.67 L2247.4 1156.79 L2247.63 1157.27 L2247.86 1157.18 L2248.09 1157.05 L2248.31 1156.89 L2248.54 1157.41 L2248.77 1156.65 L2249 1156.9 L2249.22 1157.07 L2249.45 1157.11 L2249.68 1157.43 L2249.91 1157.33 L2250.14 1157.55 L2250.36 1157.42 L2250.59 1157.68 L2250.82 1157.91 L2251.05 1158.33 L2251.27 1158.17 L2251.5 1157.86 L2251.73 1158.03 L2251.96 1158.11 L2252.19 1158.25 L2252.41 1158.58 L2252.64 1158.85 L2252.87 1158.56 L2253.1 1158.43 L2253.32 1159.18 L2253.55 1158.94 L2253.78 1159.18 L2254.01 1159.5 L2254.24 1159.66 L2254.46 1159.5 L2254.69 1159.45 L2254.92 1159.32 L2255.15 1159.53 L2255.37 1159.79 L2255.6 1159.88 L2255.83 1160.02 L2256.06 1160.14 L2256.29 1159.76 L2256.51 1159.49 L2256.74 1159.65 L2256.97 1159.76 L2257.2 1160.3 L2257.42 1160.24 L2257.65 1159.69 L2257.88 1159.52 L2258.11 1159.68 L2258.34 1159.76 L2258.56 1160.16 L2258.79 1160.07 L2259.02 1160.26 L2259.25 1160.65 L2259.47 1161.1 L2259.7 1160.86 L2259.93 1160.55 L2260.16 1160.4 L2260.39 1160.34 L2260.61 1160.04 L2260.84 1160.11 L2261.07 1159.97 L2261.3 1159.74 L2261.52 1159.38 L2261.75 1159.17 L2261.98 1159.09 L2262.21 1159.19 L2262.44 1159.01 L2262.66 1159.17 L2262.89 1159.4 L2263.12 1159.88 L2263.35 1159.8 L2263.57 1159.33 L2263.8 1158.94 L2264.03 1159.22 L2264.26 1159.46 L2264.49 1159.31 L2264.71 1159.52 L2264.94 1159.34 L2265.17 1158.97 L2265.4 1159.11 L2265.62 1159.52 L2265.85 1159.52 L2266.08 1159.4 L2266.31 1159.27 L2266.54 1158.56 L2266.76 1158.98 L2266.99 1159.19 L2267.22 1159.32 L2267.45 1159.32 L2267.67 1159.53 L2267.9 1159.55 L2268.13 1159.52 L2268.36 1159.04 L2268.59 1158.7 L2268.81 1159.03 L2269.04 1159.19 L2269.27 1159.3 L2269.5 1159.15 L2269.72 1158.66 L2269.95 1158.93 L2270.18 1158.93 L2270.41 1158.89 L2270.64 1159.38 L2270.86 1160.01 L2271.09 1159.48 L2271.32 1159.77 L2271.55 1159.45 L2271.77 1159.68 L2272 1159.91 L2272.23 1159.77 L2272.46 1159.54 L2272.69 1159.58 L2272.91 1158.18 L2273.14 1157.92 L2273.37 1157.42 L2273.6 1157.53 L2273.82 1157.4 L2274.05 1157.16 L2274.28 1157.38 L2274.51 1157.49 L2274.74 1157.61 L2274.96 1157.55 L2275.19 1157.52 L2275.42 1158 L2275.65 1157.84 L2275.87 1156.97 L2276.1 1157.03 L2276.33 1156.96 L2276.56 1156.6 L2276.78 1156.41 L2277.01 1155.87 L2277.24 1155.97 L2277.47 1156.28 L2277.7 1156.01 L2277.92 1156.16 L2278.15 1156.29 L2278.38 1156.5 L2278.61 1156.68 L2278.83 1156.47 L2279.06 1156.32 L2279.29 1156.04 L2279.52 1155.66 L2279.75 1155.27 L2279.97 1155.23 L2280.2 1155.38 L2280.43 1155.2 L2280.66 1155.54 L2280.88 1154.88 L2281.11 1154.87 L2281.34 1155.17 L2281.57 1155.38 L2281.8 1155.43 L2282.02 1154.51 L2282.25 1154.51 L2282.48 1154.42 L2282.71 1154.48 L2282.93 1153.99 L2283.16 1154.48 L2283.39 1154.68 L2283.62 1154.77 L2283.85 1154.91 L2284.07 1154.76 L2284.3 1154.44 L2284.53 1153.84 L2284.76 1153.28 L2284.98 1153.66 L2285.21 1153.74 L2285.44 1153.03 L2285.67 1153.08 L2285.9 1153.06 L2286.12 1152.78 L2286.35 1152.51 L2286.58 1152.66 L2286.81 1152.57 L2287.03 1152.69 L2287.26 1152.74 L2287.49 1153.01 L2287.72 1152.43 L2287.95 1152.34 L2288.17 1152.6 L2288.4 1152.47 L2288.63 1152.58 L2288.86 1152.45 L2289.08 1152.72 L2289.31 1152.84 L2289.54 1153 L2289.77 1153.04 L2290 1153.31 L2290.22 1153.78 L2290.45 1153.86 L2290.68 1153.87 L2290.91 1154.27 L2291.13 1154.29 L2291.36 1154.65 L2291.59 1154.92 L2291.82 1154.28 L2292.05 1154.52 L2292.27 1154.38 L2292.5 1154.83 L2292.73 1154.6 L2292.96 1154.55 L2293.18 1154.04 L2293.41 1154.28 L2293.64 1154.63 L2293.87 1154.88 L2294.1 1155 L2294.32 1155.28 L2294.55 1155.59 L2294.78 1155.49 L2295.01 1155.58 L2295.23 1155.94 L2295.46 1156.1 L2295.69 1155.74 L2295.92 1155.73 L2296.15 1155.68 L2296.37 1155.55 L2296.6 1155.24 L2296.83 1154.82 L2297.06 1155.17 L2297.28 1155.31 L2297.51 1155.59 L2297.74 1155.87 L2297.97 1156.18 L2298.2 1156.16 L2298.42 1155.79 L2298.65 1155.6 L2298.88 1155.52 L2299.11 1155.42 L2299.33 1155.47 L2299.56 1155.1 L2299.79 1154.83 L2300.02 1155.47 L2300.25 1155.3 L2300.47 1155.16 L2300.7 1155.08 L2300.93 1154.92 L2301.16 1154.03 L2301.38 1154.09 L2301.61 1154.21 L2301.84 1154.17 L2302.07 1154.45 L2302.3 1154.77 L2302.52 1154.62 L2302.75 1154.88 L2302.98 1154.53 L2303.21 1154.25 L2303.43 1154.04 L2303.66 1154.28 L2303.89 1153.79 L2304.12 1154.06 L2304.35 1154.3 L2304.57 1154.66 L2304.8 1154.35 L2305.03 1154.42 L2305.26 1154.65 L2305.48 1154.3 L2305.71 1154.5 L2305.94 1154.6 L2306.17 1154.11 L2306.4 1154.38 L2306.62 1154.38 L2306.85 1154.1 L2307.08 1153.7 L2307.31 1153.67 L2307.53 1153.07 L2307.76 1152.91 L2307.99 1153.23 L2308.22 1153.67 L2308.45 1153.53 L2308.67 1154.1 L2308.9 1153.8 L2309.13 1153.95 L2309.36 1153.75 L2309.58 1153.99 L2309.81 1154.22 L2310.04 1154.22 L2310.27 1154.38 L2310.5 1154.5 L2310.72 1154.11 L2310.95 1153.93 L2311.18 1153.54 L2311.41 1153.6 L2311.63 1153.62 L2311.86 1153.47 L2312.09 1153.9 L2312.32 1153.67 L2312.54 1153.96 L2312.77 1154.16 L2313 1154.4 L2313.23 1154.21 L2313.46 1154.19 L2313.68 1153.83 L2313.91 1154.04 L2314.14 1154.52 L2314.37 1154.59 L2314.59 1155.01 L2314.82 1155.27 L2315.05 1154.98 L2315.28 1155.12 L2315.51 1155.2 L2315.73 1155.25 L2315.96 1155.28 L2316.19 1155.05 L2316.42 1154.95 L2316.64 1155.06 L2316.87 1155.42 L2317.1 1155.47 L2317.33 1155.75 L2317.56 1155.58 L2317.78 1155.71 L2318.01 1155.8 L2318.24 1156.1 L2318.47 1156.08 L2318.69 1155.95 L2318.92 1156.15 L2319.15 1155.72 L2319.38 1154.96 L2319.61 1154.75 L2319.83 1154.95 L2320.06 1155.08 L2320.29 1155.04 L2320.52 1155.45 L2320.74 1155.84 L2320.97 1155.75 L2321.2 1155.9 L2321.43 1156.12 L2321.66 1156.41 L2321.88 1156.42 L2322.11 1156.75 L2322.34 1155.96 L2322.57 1156.23 L2322.79 1156.18 L2323.02 1156.43 L2323.25 1156.74 L2323.48 1156.72 L2323.71 1156.96 L2323.93 1156.6 L2324.16 1156.03 L2324.39 1156.18 L2324.62 1156.6 L2324.84 1156.84 L2325.07 1156.73 L2325.3 1156.68 L2325.53 1157.09 L2325.76 1156.94 L2325.98 1157.43 L2326.21 1157.61 L2326.44 1157.66 L2326.67 1157.38 L2326.89 1157.96 L2327.12 1157.69 L2327.35 1157.59 L2327.58 1157.84 L2327.81 1157.87 L2328.03 1157.79 L2328.26 1157.71 L2328.49 1157.59 L2328.72 1158 L2328.94 1158.01 L2329.17 1158.37 L2329.4 1158.43 L2329.63 1158.7 L2329.86 1158.53 L2330.08 1158.64 L2330.31 1159.12 L2330.54 1159.09 L2330.77 1159.02 L2330.99 1159.08 L2331.22 1158.49 L2331.45 1158.16 L2331.68 1158.47 L2331.91 1158.26 L2332.13 1158.61 L2332.36 1158.84 L2332.59 1159.14 L2332.82 1159.53 L2333.04 1159.04 L2333.27 1159.28 L2333.5 1159.36 L2333.73 1159.59 L2333.96 1159.56 L2334.18 1159.23 L2334.41 1159.16 L2334.64 1158.62 L2334.87 1158.15 L2335.09 1157.96 L2335.32 1157.96 L2335.55 1157.88 L2335.78 1157.96 L2336.01 1158.5 L2336.23 1158.85 L2336.46 1159.35 L2336.69 1159.49 L2336.92 1159.53 L2337.14 1159.06 L2337.37 1159.38 L2337.6 1159.81 L2337.83 1159.91 L2338.06 1160.05 L2338.28 1159.95 L2338.51 1159.93 L2338.74 1159.98 L2338.97 1159.92 L2339.19 1160.43 L2339.42 1160.28 L2339.65 1160.6 L2339.88 1160.46 L2340.11 1160.33 L2340.33 1160.12 L2340.56 1159.9 L2340.79 1159.6 L2341.02 1159.68 L2341.24 1159.83 L2341.47 1159.83 L2341.7 1159.74 L2341.93 1159.56 L2342.16 1159.48 L2342.38 1159.33 L2342.61 1159.7 L2342.84 1159.82 L2343.07 1160.07 L2343.29 1160.23 L2343.52 1160.39 L2343.75 1160.55 L2343.98 1160.48 L2344.21 1160.49 L2344.43 1160.29 L2344.66 1160.14 L2344.89 1160.14 L2345.12 1159.48 L2345.34 1159.14 L2345.57 1158.87 L2345.8 1159.28 L2346.03 1159.3 L2346.26 1159.29 L2346.48 1159.17 L2346.71 1159.72 L2346.94 1159.18 L2347.17 1159.03 L2347.39 1159.31 L2347.62 1159.8 L2347.85 1160.1 L2348.08 1160.2 L2348.31 1159.92 L2348.53 1159.97 L2348.76 1160.27 L2348.99 1160.67 L2349.22 1161.13 L2349.44 1161.33 L2349.67 1161.15 L2349.9 1161.39 L2350.13 1161.49 L2350.35 1161.98 L2350.58 1162.11 L2350.81 1161.96 L2351.04 1162.24 L2351.27 1162.21 L2351.49 1162.02 L2351.72 1162.13 L2351.95 1161.65 L2352.18 1161.59 L2352.4 1161.84 L2352.63 1161.82 L2352.86 1161.39 L2353.09 1161.54 L2353.32 1161.51 L2353.54 1161.93 L2353.77 1161.57 L2354 1161.4 L2354.23 1161.59 L2354.45 1162.11 L2354.68 1162.13 L2354.91 1162.29 L2355.14 1162.47 L2355.37 1162.92 L2355.59 1162.72 L2355.82 1162.75 L2356.05 1162.24 L2356.28 1162.04 L2356.5 1162.11 L2356.73 1161.84 L2356.96 1162.17 L2357.19 1161.76 L2357.42 1161.57 L2357.64 1161.88 L2357.87 1161.7 L2358.1 1162.07 L2358.33 1161.8 L2358.55 1161.87 L2358.78 1162.09 L2359.01 1162.25 L2359.24 1161.94 L2359.47 1161.98 L2359.69 1161.84 L2359.92 1161.9 L2360.15 1161.64 L2360.38 1161.11 L2360.6 1160.74 L2360.83 1160.69 L2361.06 1160.84 L2361.29 1160.73 L2361.52 1160.88 L2361.74 1161.3 L2361.97 1161.26 L2362.2 1161.37 L2362.43 1161.4 L2362.65 1161.51 L2362.88 1161.26 L2363.11 1161.23 L2363.34 1161.12 L2363.57 1160.83 L2363.79 1160.98 L2364.02 1160.97 L2364.25 1161.18 L2364.48 1161.45 L2364.7 1161.8 L2364.93 1161.76 L2365.16 1161.68 L2365.39 1161.66 L2365.62 1161.74 L2365.84 1161.89 L2366.07 1161.51 L2366.3 1161.34 L2366.53 1161.32 L2366.75 1161.84 L2366.98 1161.54 L2367.21 1161.69 L2367.44 1161.49 L2367.67 1161.22 L2367.89 1160.96 L2368.12 1160.65 L2368.35 1160.66 L2368.58 1160.46 L2368.8 1160.59 L2369.03 1159.99 L2369.26 1160.35 L2369.49 1160.35 L2369.72 1160.39 L2369.94 1159.7 L2370.17 1159.92 L2370.4 1159.96 L2370.63 1160.51 L2370.85 1160.55 L2371.08 1160.49 L2371.31 1160.02 L2371.54 1160.16 L2371.77 1160.49 L2371.99 1160.28 L2372.22 1160.23 L2372.45 1160.41 L2372.68 1160.31 L2372.9 1160.4 L2373.13 1160.05 L2373.36 1160.23 L2373.59 1160.47 L2373.82 1160.23 L2374.04 1159.96 L2374.27 1160.08 L2374.5 1159.53 L2374.73 1158.94 L2374.95 1158.39 L2375.18 1157.68 L2375.41 1157.1 L2375.64 1157.27 L2375.87 1157.12 L2376.09 1157.44 L2376.32 1157.62 L2376.55 1157.89 L2376.78 1158.28 L2377 1158.22 L2377.23 1157.88 L2377.46 1158.3 L2377.69 1158.35 L2377.92 1158.12 L2378.14 1157.68 L2378.37 1158.02 L2378.6 1157.59 L2378.83 1157.96 L2379.05 1157.2 L2379.28 1157.21 L2379.51 1156.6 L2379.74 1156.35 L2379.97 1156.68 L2380.19 1156.36 L2380.42 1156.48 L2380.65 1156.09 L2380.88 1155.91 L2381.1 1155.62 L2381.33 1155.76 L2381.56 1155.58 L2381.79 1155.3 L2382.02 1155.73 L2382.24 1155.88 L2382.47 1155.93 L2382.7 1156.05 L2382.93 1156.56 L2383.15 1156.44 L2383.38 1156.71 L2383.61 1157.28 L2383.84 1157.4 L2384.07 1157.71 L2384.29 1157.88 L2384.52 1158.13 L2384.75 1158.07 L2384.98 1158.2 L2385.2 1158.07 L2385.43 1158.46 L2385.66 1158.34 L2385.89 1158.57 L2386.11 1158.52 L2386.34 1158.42 L2386.57 1158.44 L2386.8 1158.11 L2387.03 1158.15 L2387.25 1157.85 L2387.48 1157.99 L2387.71 1157.98 L2387.94 1157.72 L2388.16 1157.99 L2388.39 1157.81 L2388.62 1157.5 L2388.85 1157.59 L2389.08 1157.75 L2389.3 1157.84 L2389.53 1157.41 L2389.76 1157.56 L2389.99 1157.51 L2390.21 1157.76 L2390.44 1157.83 L2390.67 1158.15 L2390.9 1157.96 L2391.13 1157.76 L2391.35 1157.69 L2391.58 1157.71 L2391.81 1157.79 L2392.04 1157.31 L2392.26 1157.37 L2392.49 1157.3 L2392.72 1157.48 L2392.95 1157.5 L2393.18 1157.97 L2393.4 1158.41 L2393.63 1158.42 L2393.86 1158.07 L2394.09 1158.12 L2394.31 1158.02 L2394.54 1158.4 L2394.77 1157.8 L2395 1158.05 L2395.23 1157.8 L2395.45 1157.81 L2395.68 1157.99 L2395.91 1158.18 L2396.14 1157.94 L2396.36 1158.33 L2396.59 1158.29 L2396.82 1158.48 L2397.05 1158.77 L2397.28 1158.37 L2397.5 1158.16 L2397.73 1158.5 L2397.96 1158.52 L2398.19 1158.66 L2398.41 1158.81 L2398.64 1158.87 L2398.87 1159.5 L2399.1 1159.65 L2399.33 1159.79 L2399.55 1159.89 L2399.78 1160.01 L2400.01 1160.46 L2400.24 1160.5 L2400.46 1160.33 L2400.69 1160.64 L2400.92 1160.56 L2401.15 1160.83 L2401.38 1160.81 L2401.6 1160.63 L2401.83 1160.79 L2402.06 1161.01 L2402.29 1161.16 L2402.51 1161.38 L2402.74 1161.33 L2402.97 1161.2 L2403.2 1161.33 L2403.43 1161.57 L2403.65 1161.28 L2403.88 1161.17 L2404.11 1160.94 L2404.34 1160.85 L2404.56 1160.6 L2404.79 1160.78 L2405.02 1160.91 L2405.25 1161.43 L2405.48 1160.98 L2405.7 1160.74 L2405.93 1160.97 L2406.16 1160.93 L2406.39 1160.96 L2406.61 1160.77 L2406.84 1160.7 L2407.07 1160.72 L2407.3 1160.63 L2407.53 1161.08 L2407.75 1161.22 L2407.98 1160.63 L2408.21 1160.51 L2408.44 1160.87 L2408.66 1161.32 L2408.89 1161.52 L2409.12 1161.78 L2409.35 1161.55 L2409.58 1161.35 L2409.8 1161.43 L2410.03 1161.49 L2410.26 1161.58 L2410.49 1161.77 L2410.71 1161.5 L2410.94 1161.57 L2411.17 1161.6 L2411.4 1161.18 L2411.63 1161.39 L2411.85 1161.4 L2412.08 1161.32 L2412.31 1161.13 L2412.54 1160.85 L2412.76 1160.79 L2412.99 1160.67 L2413.22 1160.55 L2413.45 1161.04 L2413.68 1160.53 L2413.9 1160.68 L2414.13 1160.48 L2414.36 1160.5 L2414.59 1159.62 L2414.81 1159.65 L2415.04 1159.92 L2415.27 1160.2 L2415.5 1160.62 L2415.73 1160.55 L2415.95 1160.68 L2416.18 1160.55 L2416.41 1160.25 L2416.64 1160.72 L2416.86 1160.59 L2417.09 1160.11 L2417.32 1159.79 L2417.55 1159.82 L2417.78 1159.41 L2418 1159.03 L2418.23 1159.41 L2418.46 1159.23 L2418.69 1159 L2418.91 1158.88 L2419.14 1158.64 L2419.37 1158.32 L2419.6 1158.21 L2419.83 1158.32 L2420.05 1158.39 L2420.28 1158.34 L2420.51 1158.14 L2420.74 1158.11 L2420.96 1158.15 L2421.19 1157.89 L2421.42 1157.95 L2421.65 1157.61 L2421.87 1157.33 L2422.1 1157.37 L2422.33 1157.05 L2422.56 1157.06 L2422.79 1157.17 L2423.01 1157.05 L2423.24 1157.06 L2423.47 1157.1 L2423.7 1156.92 L2423.92 1156.48 L2424.15 1156.72 L2424.38 1156.9 L2424.61 1157.1 L2424.84 1157.12 L2425.06 1157.24 L2425.29 1157.27 L2425.52 1157.23 L2425.75 1156.94 L2425.97 1157.25 L2426.2 1157.47 L2426.43 1157.54 L2426.66 1157.53 L2426.89 1157.82 L2427.11 1157.5 L2427.34 1157.54 L2427.57 1157.63 L2427.8 1157.53 L2428.02 1158.1 L2428.25 1157.46 L2428.48 1157.03 L2428.71 1157.46 L2428.94 1157.74 L2429.16 1157.71 L2429.39 1157.81 L2429.62 1158.14 L2429.85 1158.01 L2430.07 1157.87 L2430.3 1157.74 L2430.53 1157.68 L2430.76 1157.67 L2430.99 1157.81 L2431.21 1157.76 L2431.44 1157.68 L2431.67 1157.54 L2431.9 1157.24 L2432.12 1157.61 L2432.35 1157.72 L2432.58 1157.35 L2432.81 1157.42 L2433.04 1157.61 L2433.26 1157.94 L2433.49 1157.65 L2433.72 1157.71 L2433.95 1157.8 L2434.17 1157.46 L2434.4 1157.42 L2434.63 1157.33 L2434.86 1157.02 L2435.09 1157.17 L2435.31 1157.26 L2435.54 1157.31 L2435.77 1157.2 L2436 1156.77 L2436.22 1156.66 L2436.45 1156.22 L2436.68 1156.55 L2436.91 1156.86 L2437.14 1157.01 L2437.36 1157.51 L2437.59 1157.61 L2437.82 1157.68 L2438.05 1157.77 L2438.27 1157.53 L2438.5 1157.4 L2438.73 1156.79 L2438.96 1156.65 L2439.19 1156.76 L2439.41 1156.59 L2439.64 1156.45 L2439.87 1156.95 L2440.1 1156.93 L2440.32 1157.03 L2440.55 1156.98 L2440.78 1157.28 L2441.01 1156.86 L2441.24 1156.31 L2441.46 1156.53 L2441.69 1156.28 L2441.92 1156.59 L2442.15 1156.65 L2442.37 1156.79 L2442.6 1156.66 L2442.83 1156.58 L2443.06 1156.51 L2443.29 1156.1 L2443.51 1156.3 L2443.74 1156.33 L2443.97 1156.4 L2444.2 1156.66 L2444.42 1156.97 L2444.65 1156.47 L2444.88 1156.61 L2445.11 1156.89 L2445.34 1156.41 L2445.56 1156.8 L2445.79 1156.66 L2446.02 1156.75 L2446.25 1156.59 L2446.47 1156.42 L2446.7 1156.25 L2446.93 1156.24 L2447.16 1156.36 L2447.39 1156.66 L2447.61 1156.85 L2447.84 1156.95 L2448.07 1157.05 L2448.3 1157.36 L2448.52 1157.63 L2448.75 1157.4 L2448.98 1157.6 L2449.21 1157.36 L2449.44 1157.78 L2449.66 1157.86 L2449.89 1157.64 L2450.12 1157.16 L2450.35 1157.08 L2450.57 1157.12 L2450.8 1157.51 L2451.03 1157.68 L2451.26 1157.75 L2451.49 1157.66 L2451.71 1157.44 L2451.94 1157.39 L2452.17 1157.24 L2452.4 1157.25 L2452.62 1157.31 L2452.85 1157.39 L2453.08 1157.98 L2453.31 1158.29 L2453.54 1158.46 L2453.76 1158.27 L2453.99 1158.67 L2454.22 1158.53 L2454.45 1158.8 L2454.67 1158.96 L2454.9 1158.88 L2455.13 1158.48 L2455.36 1157.87 L2455.59 1157.77 L2455.81 1157.64 L2456.04 1157.33 L2456.27 1157.87 L2456.5 1157.55 L2456.72 1157.87 L2456.95 1158.36 L2457.18 1158.63 L2457.41 1158.38 L2457.64 1158.4 L2457.86 1158.73 L2458.09 1158.72 L2458.32 1158.03 L2458.55 1157.65 L2458.77 1158.01 L2459 1158.15 L2459.23 1158.39 L2459.46 1158.2 L2459.68 1158.31 L2459.91 1158 L2460.14 1158.14 L2460.37 1158.23 L2460.6 1157.91 L2460.82 1157.83 L2461.05 1157.95 L2461.28 1157.9 L2461.51 1157.68 L2461.73 1157.64 L2461.96 1157.7 L2462.19 1158.19 L2462.42 1158.06 L2462.65 1158.08 L2462.87 1157.71 L2463.1 1158.11 L2463.33 1157.89 L2463.56 1157.88 L2463.78 1158 L2464.01 1157.65 L2464.24 1157.72 L2464.47 1157.28 L2464.7 1157.46 L2464.92 1157.13 L2465.15 1157.55 L2465.38 1157.3 L2465.61 1157.59 L2465.83 1158.11 L2466.06 1158.09 L2466.29 1157.8 L2466.52 1157.73 L2466.75 1157.45 L2466.97 1157.35 L2467.2 1157.33 L2467.43 1157.49 L2467.66 1157.48 L2467.88 1157.24 L2468.11 1156.98 L2468.34 1156.77 L2468.57 1156.7 L2468.8 1156.78 L2469.02 1156.87 L2469.25 1156.66 L2469.48 1156.84 L2469.71 1156.65 L2469.93 1156.69 L2470.16 1156.82 L2470.39 1157.07 L2470.62 1157.31 L2470.85 1157.33 L2471.07 1157.56 L2471.3 1157.41 L2471.53 1157.49 L2471.76 1157.43 L2471.98 1157.17 L2472.21 1157 L2472.44 1156.31 L2472.67 1156.58 L2472.9 1156.55 L2473.12 1156.67 L2473.35 1156.38 L2473.58 1156.3 L2473.81 1156.84 L2474.03 1157.05 L2474.26 1156.81 L2474.49 1156.55 L2474.72 1156.86 L2474.95 1156.78 L2475.17 1156.78 L2475.4 1156.78 L2475.63 1156.7 L2475.86 1156.73 L2476.08 1157.08 L2476.31 1157.01 L2476.54 1156.73 L2476.77 1156.61 L2477 1156.46 L2477.22 1156.37 L2477.45 1156.39 L2477.68 1155.99 L2477.91 1155.81 L2478.13 1156.01 L2478.36 1156.04 L2478.59 1156.14 L2478.82 1156.07 L2479.05 1156.43 L2479.27 1156.36 L2479.5 1156.31 L2479.73 1156.44 L2479.96 1156.13 L2480.18 1155.97 L2480.41 1155.9 L2480.64 1155.87 L2480.87 1155.94 L2481.1 1156.1 L2481.32 1156.31 L2481.55 1156.03 L2481.78 1156.51 L2482.01 1156.52 L2482.23 1156.01 L2482.46 1156.53 L2482.69 1156.29 L2482.92 1156.56 L2483.15 1155.99 L2483.37 1156 L2483.6 1155.93 L2483.83 1155.76 L2484.06 1155.83 L2484.28 1156.09 L2484.51 1156.03 L2484.74 1155.87 L2484.97 1155.81 L2485.2 1155.85 L2485.42 1156.11 L2485.65 1155.85 L2485.88 1156.19 L2486.11 1156.03 L2486.33 1156.47 L2486.56 1156.77 L2486.79 1156.9 L2487.02 1157.02 L2487.25 1156.76 L2487.47 1156.4 L2487.7 1156.31 L2487.93 1156.73 L2488.16 1156.72 L2488.38 1157.06 L2488.61 1156.87 L2488.84 1156.66 L2489.07 1156.79 L2489.3 1155.93 L2489.52 1156.51 L2489.75 1156.56 L2489.98 1156.71 L2490.21 1157.02 L2490.43 1156.97 L2490.66 1157.37 L2490.89 1157.76 L2491.12 1157.89 L2491.35 1158.18 L2491.57 1158.42 L2491.8 1158.36 L2492.03 1158.24 L2492.26 1158.64 L2492.48 1158.9 L2492.71 1158.92 L2492.94 1158.99 L2493.17 1158.56 L2493.4 1157.91 L2493.62 1157.95 L2493.85 1157.98 L2494.08 1158.27 L2494.31 1158.26 L2494.53 1158.39 L2494.76 1158.06 L2494.99 1158.36 L2495.22 1158.42 L2495.44 1158.68 L2495.67 1158.69 L2495.9 1159.08 L2496.13 1159.36 L2496.36 1158.82 L2496.58 1158.92 L2496.81 1159.09 L2497.04 1159.35 L2497.27 1159.11 L2497.49 1159.05 L2497.72 1159.18 L2497.95 1159.27 L2498.18 1159.01 L2498.41 1159.29 L2498.63 1159.48 L2498.86 1159.31 L2499.09 1159.5 L2499.32 1159.39 L2499.54 1159.63 L2499.77 1159.99 L2500 1159.96 L2500.23 1159.84 L2500.46 1159.69 L2500.68 1159.79 L2500.91 1159.99 L2501.14 1160.1 L2501.37 1159.69 L2501.59 1159.94 L2501.82 1160.27 L2502.05 1160.27 L2502.28 1159.49 L2502.51 1159.25 L2502.73 1159.27 L2502.96 1158.94 L2503.19 1159.34 L2503.42 1159.34 L2503.64 1159.32 L2503.87 1158.79 L2504.1 1159.38 L2504.33 1159.7 L2504.56 1159.42 L2504.78 1159.44 L2505.01 1159.41 L2505.24 1159.71 L2505.47 1160.22 L2505.69 1160.6 L2505.92 1160.87 L2506.15 1160.85 L2506.38 1160.56 L2506.61 1160.56 L2506.83 1160.29 L2507.06 1160.72 L2507.29 1160.83 L2507.52 1160.83 L2507.74 1160.87 L2507.97 1161.24 L2508.2 1161.78 L2508.43 1161.43 L2508.66 1161.48 L2508.88 1161.36 L2509.11 1161.18 L2509.34 1161.12 L2509.57 1161.36 L2509.79 1161.09 L2510.02 1161.38 L2510.25 1161.46 L2510.48 1161.19 L2510.71 1160.84 L2510.93 1161.09 L2511.16 1160.84 L2511.39 1160.82 L2511.62 1160.93 L2511.84 1161.4 L2512.07 1161.62 L2512.3 1161.21 L2512.53 1161.15 L2512.76 1161.39 L2512.98 1161.37 L2513.21 1161.65 L2513.44 1161.4 L2513.67 1161.35 L2513.89 1161.59 L2514.12 1161.94 L2514.35 1162.31 L2514.58 1162.07 L2514.81 1161.83 L2515.03 1161.85 L2515.26 1161.67 L2515.49 1161.71 L2515.72 1161.58 L2515.94 1161.6 L2516.17 1161.67 L2516.4 1161.72 L2516.63 1161.96 L2516.86 1161.86 L2517.08 1161.91 L2517.31 1161.44 L2517.54 1161.47 L2517.77 1161.43 L2517.99 1161.67 L2518.22 1161.62 L2518.45 1161.78 L2518.68 1161.25 L2518.91 1160.82 L2519.13 1161.1 L2519.36 1161.21 L2519.59 1160.89 L2519.82 1160.87 L2520.04 1161.15 L2520.27 1160.8 L2520.5 1161.19 L2520.73 1161.24 L2520.96 1161.67 L2521.18 1161.91 L2521.41 1162.47 L2521.64 1162.44 L2521.87 1162.47 L2522.09 1162.54 L2522.32 1162.47 L2522.55 1162.96 L2522.78 1162.67 L2523.01 1162.73 L2523.23 1162.79 L2523.46 1162.73 L2523.69 1162.59 L2523.92 1162.55 L2524.14 1162.09 L2524.37 1162.41 L2524.6 1162.05 L2524.83 1161.45 L2525.06 1161.49 L2525.28 1161.34 L2525.51 1161.44 L2525.74 1161.01 L2525.97 1160.57 L2526.19 1160.52 L2526.42 1160.36 L2526.65 1160.61 L2526.88 1160.85 L2527.11 1161.13 L2527.33 1161.09 L2527.56 1161.41 L2527.79 1161.76 L2528.02 1161.89 L2528.24 1161.64 L2528.47 1162 L2528.7 1161.78 L2528.93 1161.53 L2529.16 1161.31 L2529.38 1161.17 L2529.61 1161.18 L2529.84 1161.39 L2530.07 1161.15 L2530.29 1161.31 L2530.52 1161.16 L2530.75 1160.9 L2530.98 1160.35 L2531.2 1160.22 L2531.43 1160.14 L2531.66 1160.17 L2531.89 1160.51 L2532.12 1160.26 L2532.34 1160.11 L2532.57 1159.76 L2532.8 1159.81 L2533.03 1159.93 L2533.25 1159.64 L2533.48 1159.52 L2533.71 1159.35 L2533.94 1159.09 L2534.17 1159.12 L2534.39 1158.93 L2534.62 1159.11 L2534.85 1159.2 L2535.08 1158.82 L2535.3 1158.63 L2535.53 1158.81 L2535.76 1158.48 L2535.99 1158.65 L2536.22 1158.43 L2536.44 1158.09 L2536.67 1158.26 L2536.9 1158.1 L2537.13 1157.83 L2537.35 1157.72 L2537.58 1158.01 L2537.81 1158.34 L2538.04 1158.7 L2538.27 1158.7 L2538.49 1158.41 L2538.72 1158.41 L2538.95 1158.61 L2539.18 1158.29 L2539.4 1158.38 L2539.63 1158.41 L2539.86 1158.64 L2540.09 1158.79 L2540.32 1158.91 L2540.54 1159.03 L2540.77 1158.83 L2541 1158.6 L2541.23 1158.38 L2541.45 1158.4 L2541.68 1158.79 L2541.91 1159.12 L2542.14 1158.97 L2542.37 1159.09 L2542.59 1158.75 L2542.82 1158.95 L2543.05 1158.95 L2543.28 1158.76 L2543.5 1158.66 L2543.73 1158.38 L2543.96 1158.67 L2544.19 1158.54 L2544.42 1158.43 L2544.64 1158.72 L2544.87 1159.07 L2545.1 1159.53 L2545.33 1159.03 L2545.55 1159.59 L2545.78 1159.4 L2546.01 1159.35 L2546.24 1159.03 L2546.47 1158.52 L2546.69 1158.67 L2546.92 1158.77 L2547.15 1159.09 L2547.38 1157.87 L2547.6 1158.18 L2547.83 1157.73 L2548.06 1157.63 L2548.29 1157.53 L2548.52 1157.21 L2548.74 1157.15 L2548.97 1157.25 L2549.2 1157.23 L2549.43 1157.03 L2549.65 1156.86 L2549.88 1157.35 L2550.11 1157.33 L2550.34 1157.38 L2550.57 1157.87 L2550.79 1157.86 L2551.02 1157.98 L2551.25 1158.58 L2551.48 1158.71 L2551.7 1158.75 L2551.93 1158.66 L2552.16 1158.28 L2552.39 1158.22 L2552.62 1158.27 L2552.84 1157.95 L2553.07 1157.64 L2553.3 1157.64 L2553.53 1157.66 L2553.75 1158.03 L2553.98 1157.47 L2554.21 1157.32 L2554.44 1157.33 L2554.67 1157.13 L2554.89 1156.89 L2555.12 1156.85 L2555.35 1156.78 L2555.58 1156.65 L2555.8 1156.33 L2556.03 1156.38 L2556.26 1156.55 L2556.49 1156.86 L2556.72 1156.78 L2556.94 1156.64 L2557.17 1156.48 L2557.4 1156.28 L2557.63 1156.35 L2557.85 1156.02 L2558.08 1155.84 L2558.31 1155.38 L2558.54 1155.2 L2558.77 1155.46 L2558.99 1155.8 L2559.22 1156.15 L2559.45 1155.48 L2559.68 1155.57 L2559.9 1155.85 L2560.13 1155.68 L2560.36 1155.48 L2560.59 1155.79 L2560.82 1155.91 L2561.04 1156.33 L2561.27 1156.22 L2561.5 1156.62 L2561.73 1156.27 L2561.95 1156.4 L2562.18 1156.4 L2562.41 1156.58 L2562.64 1156.66 L2562.87 1156.4 L2563.09 1156.31 L2563.32 1156.53 L2563.55 1157.2 L2563.78 1157.23 L2564 1157.41 L2564.23 1157.7 L2564.46 1157.88 L2564.69 1157.77 L2564.92 1157.76 L2565.14 1157.37 L2565.37 1157.13 L2565.6 1157.17 L2565.83 1157.17 L2566.05 1157.66 L2566.28 1157.53 L2566.51 1158.02 L2566.74 1158.09 L2566.96 1158.16 L2567.19 1158.14 L2567.42 1158.18 L2567.65 1157.81 L2567.88 1158.08 L2568.1 1157.57 L2568.33 1157.67 L2568.56 1158.06 L2568.79 1158.38 L2569.01 1158.01 L2569.24 1157.46 L2569.47 1157.79 L2569.7 1157.89 L2569.93 1158.02 L2570.15 1158.32 L2570.38 1157.78 L2570.61 1158.04 L2570.84 1157.54 L2571.06 1157.73 L2571.29 1157.91 L2571.52 1158.12 L2571.75 1157.92 L2571.98 1157.83 L2572.2 1157.54 L2572.43 1157.65 L2572.43 1054.93 L2572.2 1054.82 L2571.98 1055.11 L2571.75 1055.18 L2571.52 1055.37 L2571.29 1055.16 L2571.06 1054.97 L2570.84 1054.77 L2570.61 1055.29 L2570.38 1055.01 L2570.15 1055.57 L2569.93 1055.27 L2569.7 1055.12 L2569.47 1055.02 L2569.24 1054.69 L2569.01 1055.25 L2568.79 1055.62 L2568.56 1055.3 L2568.33 1054.91 L2568.1 1054.8 L2567.88 1055.32 L2567.65 1055.04 L2567.42 1055.41 L2567.19 1055.36 L2566.96 1055.37 L2566.74 1055.29 L2566.51 1055.22 L2566.28 1054.73 L2566.05 1054.85 L2565.83 1054.37 L2565.6 1054.36 L2565.37 1054.31 L2565.14 1054.54 L2564.92 1054.93 L2564.69 1054.93 L2564.46 1055.03 L2564.23 1054.84 L2564 1054.55 L2563.78 1054.36 L2563.55 1054.32 L2563.32 1053.68 L2563.09 1053.45 L2562.87 1053.53 L2562.64 1053.78 L2562.41 1053.69 L2562.18 1053.5 L2561.95 1053.49 L2561.73 1053.35 L2561.5 1053.7 L2561.27 1053.31 L2561.04 1053.41 L2560.82 1052.99 L2560.59 1052.86 L2560.36 1052.55 L2560.13 1052.74 L2559.9 1052.9 L2559.68 1052.61 L2559.45 1052.52 L2559.22 1053.21 L2558.99 1052.86 L2558.77 1052.52 L2558.54 1052.26 L2558.31 1052.42 L2558.08 1052.89 L2557.85 1053.06 L2557.63 1053.39 L2557.4 1053.31 L2557.17 1053.51 L2556.94 1053.65 L2556.72 1053.78 L2556.49 1053.85 L2556.26 1053.54 L2556.03 1053.36 L2555.8 1053.3 L2555.58 1053.62 L2555.35 1053.74 L2555.12 1053.8 L2554.89 1053.83 L2554.67 1054.06 L2554.44 1054.26 L2554.21 1054.24 L2553.98 1054.37 L2553.75 1054.95 L2553.53 1054.58 L2553.3 1054.55 L2553.07 1054.54 L2552.84 1054.85 L2552.62 1055.16 L2552.39 1055.1 L2552.16 1055.15 L2551.93 1055.53 L2551.7 1055.61 L2551.48 1055.57 L2551.25 1055.43 L2551.02 1054.84 L2550.79 1054.72 L2550.57 1054.71 L2550.34 1054.23 L2550.11 1054.17 L2549.88 1054.17 L2549.65 1053.7 L2549.43 1053.86 L2549.2 1054.05 L2548.97 1054.06 L2548.74 1053.95 L2548.52 1054 L2548.29 1054.31 L2548.06 1054.41 L2547.83 1054.5 L2547.6 1054.95 L2547.38 1054.64 L2547.15 1055.97 L2546.92 1055.65 L2546.69 1055.53 L2546.47 1055.38 L2546.24 1055.89 L2546.01 1056.21 L2545.78 1056.25 L2545.55 1056.44 L2545.33 1055.89 L2545.1 1056.39 L2544.87 1055.94 L2544.64 1055.59 L2544.42 1055.3 L2544.19 1055.4 L2543.96 1055.51 L2543.73 1055.22 L2543.5 1055.5 L2543.28 1055.59 L2543.05 1055.77 L2542.82 1055.76 L2542.59 1055.56 L2542.37 1055.89 L2542.14 1055.76 L2541.91 1055.9 L2541.68 1055.57 L2541.45 1055.18 L2541.23 1055.15 L2541 1055.36 L2540.77 1055.59 L2540.54 1055.78 L2540.32 1055.65 L2540.09 1055.52 L2539.86 1055.37 L2539.63 1055.12 L2539.4 1055.08 L2539.18 1054.98 L2538.95 1055.3 L2538.72 1055.09 L2538.49 1055.09 L2538.27 1055.37 L2538.04 1055.36 L2537.81 1055 L2537.58 1054.66 L2537.35 1054.37 L2537.13 1054.48 L2536.9 1054.73 L2536.67 1054.89 L2536.44 1054.71 L2536.22 1055.04 L2535.99 1055.26 L2535.76 1055.08 L2535.53 1055.41 L2535.3 1055.22 L2535.08 1055.41 L2534.85 1055.79 L2534.62 1055.68 L2534.39 1055.5 L2534.17 1055.67 L2533.94 1055.64 L2533.71 1055.9 L2533.48 1056.06 L2533.25 1056.17 L2533.03 1056.44 L2532.8 1056.32 L2532.57 1056.26 L2532.34 1056.61 L2532.12 1056.75 L2531.89 1056.99 L2531.66 1056.65 L2531.43 1056.61 L2531.2 1056.68 L2530.98 1056.8 L2530.75 1057.36 L2530.52 1057.62 L2530.29 1057.76 L2530.07 1057.59 L2529.84 1057.83 L2529.61 1057.61 L2529.38 1057.58 L2529.16 1057.72 L2528.93 1057.93 L2528.7 1058.18 L2528.47 1058.39 L2528.24 1058.03 L2528.02 1058.27 L2527.79 1058.13 L2527.56 1057.78 L2527.33 1057.46 L2527.11 1057.49 L2526.88 1057.2 L2526.65 1056.96 L2526.42 1056.69 L2526.19 1056.85 L2525.97 1056.9 L2525.74 1057.34 L2525.51 1057.77 L2525.28 1057.66 L2525.06 1057.8 L2524.83 1057.74 L2524.6 1058.37 L2524.37 1058.72 L2524.14 1058.4 L2523.92 1058.87 L2523.69 1058.9 L2523.46 1059.03 L2523.23 1059.08 L2523.01 1059.01 L2522.78 1058.94 L2522.55 1059.22 L2522.32 1058.74 L2522.09 1058.8 L2521.87 1058.72 L2521.64 1058.68 L2521.41 1058.7 L2521.18 1058.15 L2520.96 1057.91 L2520.73 1057.47 L2520.5 1057.41 L2520.27 1057.03 L2520.04 1057.38 L2519.82 1057.09 L2519.59 1057.1 L2519.36 1057.42 L2519.13 1057.3 L2518.91 1057.01 L2518.68 1057.44 L2518.45 1057.99 L2518.22 1057.82 L2517.99 1057.86 L2517.77 1057.61 L2517.54 1057.64 L2517.31 1057.59 L2517.08 1058.08 L2516.86 1058.01 L2516.63 1058.1 L2516.4 1057.86 L2516.17 1057.8 L2515.94 1057.72 L2515.72 1057.69 L2515.49 1057.81 L2515.26 1057.76 L2515.03 1057.93 L2514.81 1057.9 L2514.58 1058.13 L2514.35 1058.37 L2514.12 1057.99 L2513.89 1057.65 L2513.67 1057.4 L2513.44 1057.44 L2513.21 1057.69 L2512.98 1057.4 L2512.76 1057.41 L2512.53 1057.16 L2512.3 1057.21 L2512.07 1057.63 L2511.84 1057.4 L2511.62 1056.93 L2511.39 1056.81 L2511.16 1056.82 L2510.93 1057.06 L2510.71 1056.81 L2510.48 1057.16 L2510.25 1057.42 L2510.02 1057.33 L2509.79 1057.04 L2509.57 1057.3 L2509.34 1057.05 L2509.11 1057.11 L2508.88 1057.28 L2508.66 1057.38 L2508.43 1057.32 L2508.2 1057.67 L2507.97 1057.14 L2507.74 1056.77 L2507.52 1056.72 L2507.29 1056.71 L2507.06 1056.59 L2506.83 1056.16 L2506.61 1056.43 L2506.38 1056.42 L2506.15 1056.71 L2505.92 1056.71 L2505.69 1056.44 L2505.47 1056.06 L2505.24 1055.55 L2505.01 1055.25 L2504.78 1055.27 L2504.56 1055.24 L2504.33 1055.52 L2504.1 1055.2 L2503.87 1054.62 L2503.64 1055.15 L2503.42 1055.17 L2503.19 1055.16 L2502.96 1054.75 L2502.73 1055.09 L2502.51 1055.05 L2502.28 1055.29 L2502.05 1056.1 L2501.82 1056.09 L2501.59 1055.76 L2501.37 1055.5 L2501.14 1055.92 L2500.91 1055.8 L2500.68 1055.59 L2500.46 1055.48 L2500.23 1055.62 L2500 1055.73 L2499.77 1055.75 L2499.54 1055.39 L2499.32 1055.14 L2499.09 1055.24 L2498.86 1055.05 L2498.63 1055.21 L2498.41 1055.01 L2498.18 1054.72 L2497.95 1054.97 L2497.72 1054.87 L2497.49 1054.74 L2497.27 1054.79 L2497.04 1055.01 L2496.81 1054.76 L2496.58 1054.57 L2496.36 1054.46 L2496.13 1055.01 L2495.9 1054.73 L2495.67 1054.34 L2495.44 1054.32 L2495.22 1054.05 L2494.99 1053.99 L2494.76 1053.68 L2494.53 1054.01 L2494.31 1053.87 L2494.08 1053.87 L2493.85 1053.57 L2493.62 1053.53 L2493.4 1053.48 L2493.17 1054.15 L2492.94 1054.59 L2492.71 1054.51 L2492.48 1054.48 L2492.26 1054.21 L2492.03 1053.81 L2491.8 1053.92 L2491.57 1053.97 L2491.35 1053.72 L2491.12 1053.42 L2490.89 1053.29 L2490.66 1052.9 L2490.43 1052.5 L2490.21 1052.54 L2489.98 1052.22 L2489.75 1052.07 L2489.52 1052.01 L2489.3 1051.44 L2489.07 1052.34 L2488.84 1052.2 L2488.61 1052.4 L2488.38 1052.59 L2488.16 1052.25 L2487.93 1052.25 L2487.7 1051.83 L2487.47 1051.91 L2487.25 1052.26 L2487.02 1052.52 L2486.79 1052.39 L2486.56 1052.25 L2486.33 1051.95 L2486.11 1051.51 L2485.88 1051.66 L2485.65 1051.32 L2485.42 1051.57 L2485.2 1051.31 L2484.97 1051.25 L2484.74 1051.31 L2484.51 1051.45 L2484.28 1051.51 L2484.06 1051.24 L2483.83 1051.16 L2483.6 1051.32 L2483.37 1051.38 L2483.15 1051.36 L2482.92 1051.94 L2482.69 1051.67 L2482.46 1051.9 L2482.23 1051.39 L2482.01 1051.9 L2481.78 1051.88 L2481.55 1051.41 L2481.32 1051.68 L2481.1 1051.47 L2480.87 1051.3 L2480.64 1051.22 L2480.41 1051.24 L2480.18 1051.3 L2479.96 1051.45 L2479.73 1051.75 L2479.5 1051.62 L2479.27 1051.66 L2479.05 1051.71 L2478.82 1051.36 L2478.59 1051.42 L2478.36 1051.3 L2478.13 1051.26 L2477.91 1051.06 L2477.68 1051.22 L2477.45 1051.63 L2477.22 1051.59 L2477 1051.68 L2476.77 1051.82 L2476.54 1051.93 L2476.31 1052.2 L2476.08 1052.26 L2475.86 1051.91 L2475.63 1051.86 L2475.4 1051.94 L2475.17 1051.93 L2474.95 1051.91 L2474.72 1051.99 L2474.49 1051.67 L2474.26 1051.93 L2474.03 1052.15 L2473.81 1051.94 L2473.58 1051.41 L2473.35 1051.48 L2473.12 1051.76 L2472.9 1051.63 L2472.67 1051.65 L2472.44 1051.38 L2472.21 1052.09 L2471.98 1052.25 L2471.76 1052.51 L2471.53 1052.55 L2471.3 1052.47 L2471.07 1052.6 L2470.85 1052.37 L2470.62 1052.33 L2470.39 1052.09 L2470.16 1051.83 L2469.93 1051.7 L2469.71 1051.64 L2469.48 1051.82 L2469.25 1051.64 L2469.02 1051.84 L2468.8 1051.74 L2468.57 1051.65 L2468.34 1051.71 L2468.11 1051.91 L2467.88 1052.17 L2467.66 1052.39 L2467.43 1052.4 L2467.2 1052.22 L2466.97 1052.24 L2466.75 1052.33 L2466.52 1052.6 L2466.29 1052.66 L2466.06 1052.94 L2465.83 1052.96 L2465.61 1052.44 L2465.38 1052.15 L2465.15 1052.39 L2464.92 1051.98 L2464.7 1052.3 L2464.47 1052.11 L2464.24 1052.55 L2464.01 1052.47 L2463.78 1052.82 L2463.56 1052.69 L2463.33 1052.69 L2463.1 1052.9 L2462.87 1052.5 L2462.65 1052.87 L2462.42 1052.84 L2462.19 1052.96 L2461.96 1052.47 L2461.73 1052.4 L2461.51 1052.43 L2461.28 1052.65 L2461.05 1052.69 L2460.82 1052.55 L2460.6 1052.62 L2460.37 1052.94 L2460.14 1052.84 L2459.91 1052.69 L2459.68 1053 L2459.46 1052.87 L2459.23 1053.06 L2459 1052.81 L2458.77 1052.66 L2458.55 1052.3 L2458.32 1052.68 L2458.09 1053.39 L2457.86 1053.39 L2457.64 1053.06 L2457.41 1053.03 L2457.18 1053.27 L2456.95 1052.99 L2456.72 1052.51 L2456.5 1052.19 L2456.27 1052.5 L2456.04 1051.98 L2455.81 1052.28 L2455.59 1052.4 L2455.36 1052.49 L2455.13 1053.12 L2454.9 1053.52 L2454.67 1053.59 L2454.45 1053.42 L2454.22 1053.14 L2453.99 1053.27 L2453.76 1052.88 L2453.54 1053.06 L2453.31 1052.87 L2453.08 1052.56 L2452.85 1051.98 L2452.62 1051.9 L2452.4 1051.82 L2452.17 1051.8 L2451.94 1051.94 L2451.71 1051.98 L2451.49 1052.2 L2451.26 1052.28 L2451.03 1052.19 L2450.8 1052.02 L2450.57 1051.62 L2450.35 1051.58 L2450.12 1051.64 L2449.89 1052.12 L2449.66 1052.34 L2449.44 1052.25 L2449.21 1051.83 L2448.98 1052.07 L2448.75 1051.85 L2448.52 1052.07 L2448.3 1051.81 L2448.07 1051.49 L2447.84 1051.38 L2447.61 1051.27 L2447.39 1051.07 L2447.16 1050.77 L2446.93 1050.64 L2446.7 1050.63 L2446.47 1050.79 L2446.25 1050.95 L2446.02 1051.1 L2445.79 1051.01 L2445.56 1051.14 L2445.34 1050.74 L2445.11 1051.23 L2444.88 1050.94 L2444.65 1050.79 L2444.42 1051.31 L2444.2 1050.99 L2443.97 1050.73 L2443.74 1050.65 L2443.51 1050.6 L2443.29 1050.39 L2443.06 1050.81 L2442.83 1050.87 L2442.6 1050.94 L2442.37 1051.06 L2442.15 1050.91 L2441.92 1050.83 L2441.69 1050.52 L2441.46 1050.76 L2441.24 1050.53 L2441.01 1051.1 L2440.78 1051.52 L2440.55 1051.22 L2440.32 1051.25 L2440.1 1051.14 L2439.87 1051.15 L2439.64 1050.65 L2439.41 1050.79 L2439.19 1050.95 L2438.96 1050.83 L2438.73 1050.95 L2438.5 1051.59 L2438.27 1051.71 L2438.05 1051.93 L2437.82 1051.83 L2437.59 1051.76 L2437.36 1051.64 L2437.14 1051.15 L2436.91 1050.99 L2436.68 1050.68 L2436.45 1050.35 L2436.22 1050.79 L2436 1050.89 L2435.77 1051.32 L2435.54 1051.42 L2435.31 1051.35 L2435.09 1051.26 L2434.86 1051.1 L2434.63 1051.4 L2434.4 1051.49 L2434.17 1051.51 L2433.95 1051.85 L2433.72 1051.75 L2433.49 1051.67 L2433.26 1051.96 L2433.04 1051.63 L2432.81 1051.43 L2432.58 1051.35 L2432.35 1051.72 L2432.12 1051.6 L2431.9 1051.22 L2431.67 1051.52 L2431.44 1051.65 L2431.21 1051.72 L2430.99 1051.76 L2430.76 1051.61 L2430.53 1051.61 L2430.3 1051.66 L2430.07 1051.78 L2429.85 1051.91 L2429.62 1052.03 L2429.39 1051.69 L2429.16 1051.58 L2428.94 1051.6 L2428.71 1051.32 L2428.48 1050.89 L2428.25 1051.32 L2428.02 1051.98 L2427.8 1051.42 L2427.57 1051.51 L2427.34 1051.41 L2427.11 1051.36 L2426.89 1051.67 L2426.66 1051.38 L2426.43 1051.38 L2426.2 1051.3 L2425.97 1051.07 L2425.75 1050.76 L2425.52 1051.04 L2425.29 1051.07 L2425.06 1051.03 L2424.84 1050.89 L2424.61 1050.87 L2424.38 1050.65 L2424.15 1050.46 L2423.92 1050.22 L2423.7 1050.66 L2423.47 1050.83 L2423.24 1050.78 L2423.01 1050.77 L2422.79 1050.87 L2422.56 1050.75 L2422.33 1050.73 L2422.1 1051.04 L2421.87 1051 L2421.65 1051.27 L2421.42 1051.61 L2421.19 1051.53 L2420.96 1051.79 L2420.74 1051.74 L2420.51 1051.76 L2420.28 1051.94 L2420.05 1051.99 L2419.83 1051.91 L2419.6 1051.78 L2419.37 1051.88 L2419.14 1052.2 L2418.91 1052.44 L2418.69 1052.55 L2418.46 1052.77 L2418.23 1052.94 L2418 1052.56 L2417.78 1052.94 L2417.55 1053.35 L2417.32 1053.31 L2417.09 1053.61 L2416.86 1054.11 L2416.64 1054.22 L2416.41 1053.76 L2416.18 1054.06 L2415.95 1054.17 L2415.73 1054.03 L2415.5 1054.09 L2415.27 1053.67 L2415.04 1053.38 L2414.81 1053.11 L2414.59 1053.07 L2414.36 1054 L2414.13 1053.96 L2413.9 1054.15 L2413.68 1053.99 L2413.45 1054.52 L2413.22 1054.03 L2412.99 1054.14 L2412.76 1054.25 L2412.54 1054.3 L2412.31 1054.58 L2412.08 1054.76 L2411.85 1054.82 L2411.63 1054.81 L2411.4 1054.58 L2411.17 1055.01 L2410.94 1054.97 L2410.71 1054.88 L2410.49 1055.14 L2410.26 1054.95 L2410.03 1054.85 L2409.8 1054.77 L2409.58 1054.68 L2409.35 1054.87 L2409.12 1055.1 L2408.89 1054.83 L2408.66 1054.63 L2408.44 1054.18 L2408.21 1053.81 L2407.98 1053.92 L2407.75 1054.53 L2407.53 1054.38 L2407.3 1053.93 L2407.07 1054.01 L2406.84 1053.98 L2406.61 1054.03 L2406.39 1054.22 L2406.16 1054.18 L2405.93 1054.21 L2405.7 1053.97 L2405.48 1054.2 L2405.25 1054.65 L2405.02 1054.14 L2404.79 1054.01 L2404.56 1053.81 L2404.34 1054.05 L2404.11 1054.14 L2403.88 1054.36 L2403.65 1054.45 L2403.43 1054.74 L2403.2 1054.49 L2402.97 1054.35 L2402.74 1054.47 L2402.51 1054.51 L2402.29 1054.28 L2402.06 1054.13 L2401.83 1053.9 L2401.6 1053.73 L2401.38 1053.9 L2401.15 1053.91 L2400.92 1053.63 L2400.69 1053.69 L2400.46 1053.38 L2400.24 1053.54 L2400.01 1053.49 L2399.78 1053.04 L2399.55 1052.91 L2399.33 1052.8 L2399.1 1052.65 L2398.87 1052.49 L2398.64 1051.88 L2398.41 1051.8 L2398.19 1051.65 L2397.96 1051.49 L2397.73 1051.46 L2397.5 1051.12 L2397.28 1051.32 L2397.05 1051.72 L2396.82 1051.43 L2396.59 1051.23 L2396.36 1051.25 L2396.14 1050.86 L2395.91 1051.1 L2395.68 1050.9 L2395.45 1050.71 L2395.23 1050.69 L2395 1050.93 L2394.77 1050.68 L2394.54 1051.29 L2394.31 1050.91 L2394.09 1051 L2393.86 1050.94 L2393.63 1051.29 L2393.4 1051.26 L2393.18 1050.83 L2392.95 1050.36 L2392.72 1050.33 L2392.49 1050.13 L2392.26 1050.19 L2392.04 1050.12 L2391.81 1050.62 L2391.58 1050.52 L2391.35 1050.48 L2391.13 1050.55 L2390.9 1050.73 L2390.67 1050.91 L2390.44 1050.6 L2390.21 1050.51 L2389.99 1050.25 L2389.76 1050.29 L2389.53 1050.13 L2389.3 1050.57 L2389.08 1050.47 L2388.85 1050.29 L2388.62 1050.2 L2388.39 1050.5 L2388.16 1050.67 L2387.94 1050.4 L2387.71 1050.64 L2387.48 1050.64 L2387.25 1050.5 L2387.03 1050.79 L2386.8 1050.73 L2386.57 1051.06 L2386.34 1051.03 L2386.11 1051.12 L2385.89 1051.16 L2385.66 1050.92 L2385.43 1051.03 L2385.2 1050.64 L2384.98 1050.75 L2384.75 1050.61 L2384.52 1050.67 L2384.29 1050.41 L2384.07 1050.23 L2383.84 1049.92 L2383.61 1049.79 L2383.38 1049.23 L2383.15 1048.95 L2382.93 1049.06 L2382.7 1048.55 L2382.47 1048.43 L2382.24 1048.36 L2382.02 1048.2 L2381.79 1047.78 L2381.56 1048.05 L2381.33 1048.22 L2381.1 1048.07 L2380.88 1048.35 L2380.65 1048.53 L2380.42 1048.91 L2380.19 1048.78 L2379.97 1049.09 L2379.74 1048.76 L2379.51 1049.01 L2379.28 1049.63 L2379.05 1049.61 L2378.83 1050.4 L2378.6 1050.03 L2378.37 1050.46 L2378.14 1050.12 L2377.92 1050.56 L2377.69 1050.78 L2377.46 1050.72 L2377.23 1050.3 L2377 1050.64 L2376.78 1050.68 L2376.55 1050.29 L2376.32 1050.01 L2376.09 1049.82 L2375.87 1049.5 L2375.64 1049.64 L2375.41 1049.47 L2375.18 1050.06 L2374.95 1050.8 L2374.73 1051.35 L2374.5 1051.96 L2374.27 1052.52 L2374.04 1052.38 L2373.82 1052.65 L2373.59 1052.88 L2373.36 1052.64 L2373.13 1052.44 L2372.9 1052.79 L2372.68 1052.69 L2372.45 1052.78 L2372.22 1052.59 L2371.99 1052.63 L2371.77 1052.83 L2371.54 1052.5 L2371.31 1052.34 L2371.08 1052.82 L2370.85 1052.87 L2370.63 1052.81 L2370.4 1052.27 L2370.17 1052.23 L2369.94 1051.99 L2369.72 1052.71 L2369.49 1052.66 L2369.26 1052.64 L2369.03 1052.29 L2368.8 1052.9 L2368.58 1052.76 L2368.35 1052.95 L2368.12 1052.93 L2367.89 1053.24 L2367.67 1053.48 L2367.44 1053.75 L2367.21 1053.94 L2366.98 1053.78 L2366.75 1054.08 L2366.53 1053.56 L2366.3 1053.57 L2366.07 1053.73 L2365.84 1054.11 L2365.62 1053.95 L2365.39 1053.85 L2365.16 1053.87 L2364.93 1053.93 L2364.7 1053.96 L2364.48 1053.6 L2364.25 1053.33 L2364.02 1053.11 L2363.79 1053.11 L2363.57 1052.95 L2363.34 1053.23 L2363.11 1053.34 L2362.88 1053.35 L2362.65 1053.59 L2362.43 1053.47 L2362.2 1053.43 L2361.97 1053.3 L2361.74 1053.34 L2361.52 1052.92 L2361.29 1052.76 L2361.06 1052.85 L2360.83 1052.7 L2360.6 1052.73 L2360.38 1053.1 L2360.15 1053.64 L2359.92 1053.9 L2359.69 1053.82 L2359.47 1053.96 L2359.24 1053.9 L2359.01 1054.21 L2358.78 1054.04 L2358.55 1053.81 L2358.33 1053.72 L2358.1 1053.99 L2357.87 1053.61 L2357.64 1053.79 L2357.42 1053.47 L2357.19 1053.66 L2356.96 1054.07 L2356.73 1053.73 L2356.5 1053.99 L2356.28 1053.91 L2356.05 1054.1 L2355.82 1054.62 L2355.59 1054.57 L2355.37 1054.76 L2355.14 1054.32 L2354.91 1054.13 L2354.68 1053.96 L2354.45 1053.92 L2354.23 1053.41 L2354 1053.22 L2353.77 1053.37 L2353.54 1053.73 L2353.32 1053.31 L2353.09 1053.33 L2352.86 1053.17 L2352.63 1053.6 L2352.4 1053.61 L2352.18 1053.35 L2351.95 1053.4 L2351.72 1053.88 L2351.49 1053.76 L2351.27 1053.94 L2351.04 1053.96 L2350.81 1053.68 L2350.58 1053.81 L2350.35 1053.67 L2350.13 1053.19 L2349.9 1053.07 L2349.67 1052.83 L2349.44 1053 L2349.22 1052.79 L2348.99 1052.33 L2348.76 1051.93 L2348.53 1051.63 L2348.31 1051.56 L2348.08 1051.84 L2347.85 1051.73 L2347.62 1051.42 L2347.39 1050.94 L2347.17 1050.65 L2346.94 1050.8 L2346.71 1051.34 L2346.48 1050.8 L2346.26 1050.91 L2346.03 1050.91 L2345.8 1050.88 L2345.57 1050.46 L2345.34 1050.73 L2345.12 1051.07 L2344.89 1051.75 L2344.66 1051.73 L2344.43 1051.87 L2344.21 1052.06 L2343.98 1052.04 L2343.75 1052.09 L2343.52 1051.93 L2343.29 1051.76 L2343.07 1051.58 L2342.84 1051.33 L2342.61 1051.2 L2342.38 1050.82 L2342.16 1050.96 L2341.93 1051.03 L2341.7 1051.21 L2341.47 1051.28 L2341.24 1051.27 L2341.02 1051.11 L2340.79 1051.02 L2340.56 1051.31 L2340.33 1051.52 L2340.11 1051.72 L2339.88 1051.84 L2339.65 1051.98 L2339.42 1051.65 L2339.19 1051.78 L2338.97 1051.28 L2338.74 1051.33 L2338.51 1051.27 L2338.28 1051.28 L2338.06 1051.37 L2337.83 1051.21 L2337.6 1051.1 L2337.37 1050.68 L2337.14 1050.35 L2336.92 1050.82 L2336.69 1050.77 L2336.46 1050.62 L2336.23 1050.13 L2336.01 1049.77 L2335.78 1049.24 L2335.55 1049.15 L2335.32 1049.22 L2335.09 1049.2 L2334.87 1049.38 L2334.64 1049.85 L2334.41 1050.41 L2334.18 1050.47 L2333.96 1050.8 L2333.73 1050.81 L2333.5 1050.57 L2333.27 1050.48 L2333.04 1050.23 L2332.82 1050.72 L2332.59 1050.34 L2332.36 1050.03 L2332.13 1049.79 L2331.91 1049.44 L2331.68 1049.64 L2331.45 1049.32 L2331.22 1049.65 L2330.99 1050.26 L2330.77 1050.18 L2330.54 1050.23 L2330.31 1050.25 L2330.08 1049.78 L2329.86 1049.66 L2329.63 1049.82 L2329.4 1049.54 L2329.17 1049.47 L2328.94 1049.11 L2328.72 1049.08 L2328.49 1048.68 L2328.26 1048.78 L2328.03 1048.85 L2327.81 1048.92 L2327.58 1048.87 L2327.35 1048.62 L2327.12 1048.71 L2326.89 1048.97 L2326.67 1048.4 L2326.44 1048.67 L2326.21 1048.61 L2325.98 1048.42 L2325.76 1047.94 L2325.53 1048.08 L2325.3 1047.66 L2325.07 1047.71 L2324.84 1047.8 L2324.62 1047.56 L2324.39 1047.14 L2324.16 1046.97 L2323.93 1047.55 L2323.71 1047.91 L2323.48 1047.67 L2323.25 1047.68 L2323.02 1047.35 L2322.79 1047.1 L2322.57 1047.14 L2322.34 1046.86 L2322.11 1047.68 L2321.88 1047.34 L2321.66 1047.33 L2321.43 1047.03 L2321.2 1046.8 L2320.97 1046.64 L2320.74 1046.72 L2320.52 1046.33 L2320.29 1045.92 L2320.06 1045.94 L2319.83 1045.8 L2319.61 1045.59 L2319.38 1045.8 L2319.15 1046.58 L2318.92 1047.02 L2318.69 1046.81 L2318.47 1046.92 L2318.24 1046.93 L2318.01 1046.63 L2317.78 1046.52 L2317.56 1046.38 L2317.33 1046.54 L2317.1 1046.26 L2316.87 1046.19 L2316.64 1045.83 L2316.42 1045.71 L2316.19 1045.8 L2315.96 1046.02 L2315.73 1045.97 L2315.51 1045.92 L2315.28 1045.82 L2315.05 1045.68 L2314.82 1045.96 L2314.59 1045.68 L2314.37 1045.27 L2314.14 1045.19 L2313.91 1044.71 L2313.68 1044.49 L2313.46 1044.85 L2313.23 1044.85 L2313 1045.03 L2312.77 1044.79 L2312.54 1044.58 L2312.32 1044.28 L2312.09 1044.5 L2311.86 1044.08 L2311.63 1044.21 L2311.41 1044.18 L2311.18 1044.11 L2310.95 1044.5 L2310.72 1044.67 L2310.5 1045.05 L2310.27 1044.92 L2310.04 1044.75 L2309.81 1044.74 L2309.58 1044.5 L2309.36 1044.25 L2309.13 1044.44 L2308.9 1044.28 L2308.67 1044.58 L2308.45 1044.01 L2308.22 1044.15 L2307.99 1043.71 L2307.76 1043.38 L2307.53 1043.53 L2307.31 1044.14 L2307.08 1044.16 L2306.85 1044.57 L2306.62 1044.84 L2306.4 1044.82 L2306.17 1044.54 L2305.94 1045.04 L2305.71 1044.93 L2305.48 1044.72 L2305.26 1045.06 L2305.03 1044.83 L2304.8 1044.74 L2304.57 1045.04 L2304.35 1044.69 L2304.12 1044.44 L2303.89 1044.16 L2303.66 1044.65 L2303.43 1044.4 L2303.21 1044.61 L2302.98 1044.88 L2302.75 1045.23 L2302.52 1044.96 L2302.3 1045.1 L2302.07 1044.77 L2301.84 1044.49 L2301.61 1044.51 L2301.38 1044.38 L2301.16 1044.31 L2300.93 1045.25 L2300.7 1045.39 L2300.47 1045.47 L2300.25 1045.59 L2300.02 1045.75 L2299.79 1045.12 L2299.56 1045.39 L2299.33 1045.75 L2299.11 1045.7 L2298.88 1045.78 L2298.65 1045.85 L2298.42 1046.03 L2298.2 1046.4 L2297.97 1046.4 L2297.74 1046.09 L2297.51 1045.8 L2297.28 1045.51 L2297.06 1045.37 L2296.83 1045.01 L2296.6 1045.43 L2296.37 1045.73 L2296.15 1045.85 L2295.92 1045.89 L2295.69 1045.89 L2295.46 1046.24 L2295.23 1046.07 L2295.01 1045.7 L2294.78 1045.61 L2294.55 1045.69 L2294.32 1045.38 L2294.1 1045.09 L2293.87 1044.96 L2293.64 1044.7 L2293.41 1044.35 L2293.18 1044.1 L2292.96 1044.61 L2292.73 1044.65 L2292.5 1044.87 L2292.27 1044.42 L2292.05 1044.56 L2291.82 1044.31 L2291.59 1044.97 L2291.36 1044.69 L2291.13 1044.32 L2290.91 1044.29 L2290.68 1043.88 L2290.45 1043.87 L2290.22 1043.78 L2290 1043.31 L2289.77 1043.03 L2289.54 1042.98 L2289.31 1042.81 L2289.08 1042.67 L2288.86 1042.4 L2288.63 1042.52 L2288.4 1042.39 L2288.17 1042.51 L2287.95 1042.25 L2287.72 1042.32 L2287.49 1042.91 L2287.26 1042.64 L2287.03 1042.58 L2286.81 1042.44 L2286.58 1042.52 L2286.35 1042.36 L2286.12 1042.62 L2285.9 1042.9 L2285.67 1042.9 L2285.44 1042.84 L2285.21 1043.58 L2284.98 1043.48 L2284.76 1043.1 L2284.53 1043.67 L2284.3 1044.28 L2284.07 1044.6 L2283.85 1044.73 L2283.62 1044.59 L2283.39 1044.48 L2283.16 1044.28 L2282.93 1043.79 L2282.71 1044.28 L2282.48 1044.21 L2282.25 1044.29 L2282.02 1044.28 L2281.8 1045.24 L2281.57 1045.19 L2281.34 1044.96 L2281.11 1044.66 L2280.88 1044.66 L2280.66 1045.33 L2280.43 1044.98 L2280.2 1045.16 L2279.97 1044.99 L2279.75 1045.02 L2279.52 1045.41 L2279.29 1045.79 L2279.06 1046.06 L2278.83 1046.2 L2278.61 1046.4 L2278.38 1046.21 L2278.15 1045.99 L2277.92 1045.84 L2277.7 1045.69 L2277.47 1045.94 L2277.24 1045.63 L2277.01 1045.52 L2276.78 1046.07 L2276.56 1046.25 L2276.33 1046.61 L2276.1 1046.67 L2275.87 1046.59 L2275.65 1047.5 L2275.42 1047.66 L2275.19 1047.17 L2274.96 1047.19 L2274.74 1047.24 L2274.51 1047.11 L2274.28 1046.98 L2274.05 1046.76 L2273.82 1046.99 L2273.6 1047.11 L2273.37 1046.99 L2273.14 1047.49 L2272.91 1047.74 L2272.69 1049.28 L2272.46 1049.23 L2272.23 1049.45 L2272 1049.58 L2271.77 1049.33 L2271.55 1049.1 L2271.32 1049.42 L2271.09 1049.12 L2270.86 1049.65 L2270.64 1049.04 L2270.41 1048.55 L2270.18 1048.58 L2269.95 1048.57 L2269.72 1048.29 L2269.5 1048.78 L2269.27 1048.92 L2269.04 1048.8 L2268.81 1048.63 L2268.59 1048.29 L2268.36 1048.63 L2268.13 1049.12 L2267.9 1049.13 L2267.67 1049.09 L2267.45 1048.88 L2267.22 1048.86 L2266.99 1048.72 L2266.76 1048.51 L2266.54 1048.08 L2266.31 1048.82 L2266.08 1048.94 L2265.85 1049.04 L2265.62 1049.03 L2265.4 1048.62 L2265.17 1048.47 L2264.94 1048.84 L2264.71 1049.01 L2264.49 1048.78 L2264.26 1048.93 L2264.03 1048.67 L2263.8 1048.39 L2263.57 1048.77 L2263.35 1049.25 L2263.12 1049.32 L2262.89 1048.84 L2262.66 1048.6 L2262.44 1048.43 L2262.21 1048.6 L2261.98 1048.49 L2261.75 1048.55 L2261.52 1048.75 L2261.3 1049.11 L2261.07 1049.33 L2260.84 1049.46 L2260.61 1049.37 L2260.39 1049.67 L2260.16 1049.72 L2259.93 1049.85 L2259.7 1050.16 L2259.47 1050.39 L2259.25 1049.95 L2259.02 1049.55 L2258.79 1049.35 L2258.56 1049.43 L2258.34 1049.02 L2258.11 1048.93 L2257.88 1048.76 L2257.65 1048.92 L2257.42 1049.48 L2257.2 1049.52 L2256.97 1049 L2256.74 1048.87 L2256.51 1048.7 L2256.29 1048.97 L2256.06 1049.35 L2255.83 1049.21 L2255.6 1049.06 L2255.37 1048.96 L2255.15 1048.69 L2254.92 1048.47 L2254.69 1048.59 L2254.46 1048.63 L2254.24 1048.77 L2254.01 1048.61 L2253.78 1048.28 L2253.55 1048.03 L2253.32 1048.26 L2253.1 1047.53 L2252.87 1047.65 L2252.64 1047.94 L2252.41 1047.65 L2252.19 1047.32 L2251.96 1047.17 L2251.73 1047.08 L2251.5 1046.9 L2251.27 1047.2 L2251.05 1047.35 L2250.82 1046.93 L2250.59 1046.69 L2250.36 1046.42 L2250.14 1046.54 L2249.91 1046.31 L2249.68 1046.4 L2249.45 1046.07 L2249.22 1046.02 L2249 1045.84 L2248.77 1045.58 L2248.54 1046.37 L2248.31 1045.86 L2248.09 1046 L2247.86 1046.12 L2247.63 1046.2 L2247.4 1045.72 L2247.17 1045.59 L2246.95 1045.27 L2246.72 1045.45 L2246.49 1045.25 L2246.26 1045.45 L2246.04 1045.2 L2245.81 1045.19 L2245.58 1045.24 L2245.35 1044.66 L2245.12 1044.75 L2244.9 1044.7 L2244.67 1044.61 L2244.44 1044.18 L2244.21 1044.2 L2243.99 1044.14 L2243.76 1044.46 L2243.53 1044.14 L2243.3 1044.55 L2243.07 1044.08 L2242.85 1043.69 L2242.62 1043.37 L2242.39 1043.19 L2242.16 1043.08 L2241.94 1043.35 L2241.71 1043.34 L2241.48 1043.09 L2241.25 1043.03 L2241.02 1043.01 L2240.8 1043.06 L2240.57 1043.08 L2240.34 1042.91 L2240.11 1042.29 L2239.89 1042.96 L2239.66 1043.28 L2239.43 1043 L2239.2 1042.83 L2238.98 1042.6 L2238.75 1042.66 L2238.52 1042.24 L2238.29 1042.36 L2238.06 1042.82 L2237.84 1042.75 L2237.61 1042.94 L2237.38 1042.77 L2237.15 1042.68 L2236.93 1042.86 L2236.7 1042.54 L2236.47 1042.48 L2236.24 1042.74 L2236.01 1042.87 L2235.79 1042.5 L2235.56 1042.4 L2235.33 1042.53 L2235.1 1042.51 L2234.88 1042.33 L2234.65 1042.51 L2234.42 1042.82 L2234.19 1043.13 L2233.96 1043.03 L2233.74 1042.84 L2233.51 1042.47 L2233.28 1042.31 L2233.05 1042.13 L2232.83 1041.56 L2232.6 1042.1 L2232.37 1042.33 L2232.14 1042.51 L2231.91 1042.5 L2231.69 1042.42 L2231.46 1043.15 L2231.23 1042.84 L2231 1042.84 L2230.78 1043.18 L2230.55 1043.69 L2230.32 1043.48 L2230.09 1043.27 L2229.86 1043.02 L2229.64 1043.48 L2229.41 1043.26 L2229.18 1043.56 L2228.95 1043.65 L2228.73 1043.6 L2228.5 1043.57 L2228.27 1043.54 L2228.04 1043.49 L2227.81 1043.63 L2227.59 1043.22 L2227.36 1042.84 L2227.13 1042.38 L2226.9 1042.65 L2226.68 1042.65 L2226.45 1042.88 L2226.22 1043.73 L2225.99 1043.45 L2225.76 1043.54 L2225.54 1043.64 L2225.31 1043.06 L2225.08 1043.6 L2224.85 1043.56 L2224.63 1043.34 L2224.4 1043.43 L2224.17 1043.28 L2223.94 1043.26 L2223.71 1043.65 L2223.49 1044.39 L2223.26 1044.33 L2223.03 1044.6 L2222.8 1044.24 L2222.58 1043.72 L2222.35 1043.2 L2222.12 1043.26 L2221.89 1042.86 L2221.66 1043.27 L2221.44 1043.25 L2221.21 1043.49 L2220.98 1043.89 L2220.75 1043.96 L2220.53 1043.64 L2220.3 1043.38 L2220.07 1043.38 L2219.84 1043.03 L2219.61 1042.9 L2219.39 1042.78 L2219.16 1042.65 L2218.93 1042.36 L2218.7 1042.2 L2218.48 1042.54 L2218.25 1042.6 L2218.02 1042.88 L2217.79 1042.32 L2217.56 1042.21 L2217.34 1042.49 L2217.11 1041.98 L2216.88 1042.06 L2216.65 1042.62 L2216.43 1042.67 L2216.2 1042.46 L2215.97 1042.37 L2215.74 1042.72 L2215.51 1042.46 L2215.29 1042.53 L2215.06 1042.39 L2214.83 1042.76 L2214.6 1042.61 L2214.38 1043.17 L2214.15 1043.66 L2213.92 1044.08 L2213.69 1043.92 L2213.46 1043.75 L2213.24 1043.57 L2213.01 1043.37 L2212.78 1043.25 L2212.55 1043.25 L2212.33 1042.96 L2212.1 1042.72 L2211.87 1042.68 L2211.64 1042.78 L2211.41 1042.87 L2211.19 1043.26 L2210.96 1042.79 L2210.73 1042.84 L2210.5 1042.88 L2210.28 1042.68 L2210.05 1043.01 L2209.82 1042.9 L2209.59 1042.69 L2209.36 1042.7 L2209.14 1042.57 L2208.91 1042.55 L2208.68 1042.51 L2208.45 1042.57 L2208.23 1042.21 L2208 1042.88 L2207.77 1043.26 L2207.54 1043.61 L2207.31 1043.47 L2207.09 1043 L2206.86 1043.23 L2206.63 1043.14 L2206.4 1043.11 L2206.18 1043.46 L2205.95 1043.77 L2205.72 1043.68 L2205.49 1043.53 L2205.26 1043.66 L2205.04 1043.44 L2204.81 1043.26 L2204.58 1042.65 L2204.35 1042.67 L2204.13 1042.84 L2203.9 1042.87 L2203.67 1042.88 L2203.44 1042.82 L2203.21 1043.26 L2202.99 1043.69 L2202.76 1043.44 L2202.53 1043.41 L2202.3 1043.25 L2202.08 1043.76 L2201.85 1043.4 L2201.62 1043.69 L2201.39 1043.58 L2201.17 1043.15 L2200.94 1043.47 L2200.71 1043.31 L2200.48 1043.29 L2200.25 1043.17 L2200.03 1042.94 L2199.8 1043.35 L2199.57 1044.13 L2199.34 1044.18 L2199.12 1044.44 L2198.89 1044.8 L2198.66 1044.93 L2198.43 1044.78 L2198.2 1045.09 L2197.98 1044.7 L2197.75 1044.48 L2197.52 1044.42 L2197.29 1044.62 L2197.07 1044.21 L2196.84 1043.96 L2196.61 1043.83 L2196.38 1044.02 L2196.15 1043.97 L2195.93 1043.63 L2195.7 1043.15 L2195.47 1043.11 L2195.24 1043.04 L2195.02 1042.58 L2194.79 1043.12 L2194.56 1043.89 L2194.33 1043.37 L2194.1 1043.32 L2193.88 1042.87 L2193.65 1042.89 L2193.42 1042.98 L2193.19 1043.02 L2192.97 1043.58 L2192.74 1043.38 L2192.51 1043.25 L2192.28 1043.18 L2192.05 1043.07 L2191.83 1043.08 L2191.6 1043.19 L2191.37 1043.28 L2191.14 1043.49 L2190.92 1043.27 L2190.69 1042.97 L2190.46 1044.35 L2190.23 1044.29 L2190 1043.98 L2189.78 1044 L2189.55 1044.01 L2189.32 1044.66 L2189.09 1044.62 L2188.87 1044.77 L2188.64 1044.84 L2188.41 1044.35 L2188.18 1043.97 L2187.95 1043.8 L2187.73 1043.81 L2187.5 1044.2 L2187.27 1044.3 L2187.04 1044.25 L2186.82 1044.04 L2186.59 1043.74 L2186.36 1043.58 L2186.13 1043.95 L2185.9 1043.85 L2185.68 1043.73 L2185.45 1044.05 L2185.22 1043.56 L2184.99 1043.35 L2184.77 1043.95 L2184.54 1044.17 L2184.31 1044.1 L2184.08 1043.9 L2183.85 1043.73 L2183.63 1043.75 L2183.4 1043.93 L2183.17 1044.39 L2182.94 1044.09 L2182.72 1043.61 L2182.49 1043.59 L2182.26 1043.93 L2182.03 1043.41 L2181.8 1043.38 L2181.58 1043.26 L2181.35 1043.72 L2181.12 1043.13 L2180.89 1043.49 L2180.67 1043.71 L2180.44 1043.95 L2180.21 1043.56 L2179.98 1043.97 L2179.75 1043.93 L2179.53 1043.71 L2179.3 1044.18 L2179.07 1043.93 L2178.84 1043.96 L2178.62 1043.65 L2178.39 1043.74 L2178.16 1044.14 L2177.93 1044.24 L2177.7 1044.61 L2177.48 1044.93 L2177.25 1045.18 L2177.02 1045.21 L2176.79 1045.34 L2176.57 1045.36 L2176.34 1045.19 L2176.11 1045.38 L2175.88 1044.94 L2175.65 1044.59 L2175.43 1044.98 L2175.2 1044.85 L2174.97 1044.91 L2174.74 1044.51 L2174.52 1044.49 L2174.29 1044.63 L2174.06 1044.43 L2173.83 1044.27 L2173.6 1044.23 L2173.38 1044.1 L2173.15 1045.02 L2172.92 1044.97 L2172.69 1044.87 L2172.47 1044.91 L2172.24 1044.86 L2172.01 1044.7 L2171.78 1045.2 L2171.55 1045.61 L2171.33 1045.74 L2171.1 1045.37 L2170.87 1045.2 L2170.64 1044.64 L2170.42 1044.53 L2170.19 1044.53 L2169.96 1044.09 L2169.73 1044.36 L2169.5 1045.95 L2169.28 1045.67 L2169.05 1045.42 L2168.82 1045.3 L2168.59 1045.52 L2168.37 1045.82 L2168.14 1046.28 L2167.91 1046.03 L2167.68 1045.96 L2167.45 1046.84 L2167.23 1046.67 L2167 1046.26 L2166.77 1046.04 L2166.54 1046.47 L2166.32 1047.3 L2166.09 1047.63 L2165.86 1047.62 L2165.63 1047.53 L2165.41 1047.61 L2165.18 1047.89 L2164.95 1047.69 L2164.72 1047.38 L2164.49 1047.07 L2164.27 1047.2 L2164.04 1046.85 L2163.81 1046.66 L2163.58 1046.84 L2163.36 1047.24 L2163.13 1047.68 L2162.9 1047.92 L2162.67 1047.22 L2162.44 1046.67 L2162.22 1046.97 L2161.99 1046.92 L2161.76 1046.56 L2161.53 1046.46 L2161.31 1046.37 L2161.08 1045.98 L2160.85 1045.91 L2160.62 1046.23 L2160.39 1046.76 L2160.17 1046.87 L2159.94 1046.82 L2159.71 1047.27 L2159.48 1047.27 L2159.26 1047.11 L2159.03 1047.5 L2158.8 1047.08 L2158.57 1047.05 L2158.34 1047.24 L2158.12 1047.3 L2157.89 1046.93 L2157.66 1046.48 L2157.43 1046.59 L2157.21 1046.54 L2156.98 1046.47 L2156.75 1046.63 L2156.52 1047.15 L2156.29 1046.88 L2156.07 1046.87 L2155.84 1046.9 L2155.61 1046.76 L2155.38 1046.57 L2155.16 1046.11 L2154.93 1045.93 L2154.7 1046.28 L2154.47 1047.22 L2154.24 1047.83 L2154.02 1047.87 L2153.79 1047.7 L2153.56 1047.21 L2153.33 1046.54 L2153.11 1046.08 L2152.88 1045.87 L2152.65 1045.87 L2152.42 1045.78 L2152.19 1045.8 L2151.97 1046.15 L2151.74 1046 L2151.51 1046.26 L2151.28 1046.01 L2151.06 1046.53 L2150.83 1046.35 L2150.6 1046.12 L2150.37 1046.2 L2150.14 1047.18 L2149.92 1047.22 L2149.69 1047.46 L2149.46 1047.47 L2149.23 1047.4 L2149.01 1047.48 L2148.78 1047.55 L2148.55 1047.43 L2148.32 1046.62 L2148.09 1046.64 L2147.87 1046.67 L2147.64 1046.48 L2147.41 1046.77 L2147.18 1046.28 L2146.96 1046.02 L2146.73 1046.68 L2146.5 1047.02 L2146.27 1046.98 L2146.04 1047.04 L2145.82 1046.77 L2145.59 1047.11 L2145.36 1046.82 L2145.13 1046.85 L2144.91 1046.73 L2144.68 1046.81 L2144.45 1046.74 L2144.22 1046.78 L2143.99 1046.88 L2143.77 1046.94 L2143.54 1046.84 L2143.31 1046.62 L2143.08 1046.37 L2142.86 1047.09 L2142.63 1047.27 L2142.4 1047.45 L2142.17 1047.12 L2141.94 1047.11 L2141.72 1047.19 L2141.49 1046.86 L2141.26 1046.83 L2141.03 1046.81 L2140.81 1047.1 L2140.58 1046.63 L2140.35 1046.94 L2140.12 1046.71 L2139.89 1046.36 L2139.67 1046.15 L2139.44 1046 L2139.21 1046.02 L2138.98 1045.66 L2138.76 1046.1 L2138.53 1046.12 L2138.3 1045.42 L2138.07 1045.16 L2137.84 1045.08 L2137.62 1045.31 L2137.39 1045.26 L2137.16 1046.19 L2136.93 1045.8 L2136.71 1046.12 L2136.48 1046.4 L2136.25 1046.6 L2136.02 1046.44 L2135.79 1046.75 L2135.57 1046.89 L2135.34 1046.53 L2135.11 1046.71 L2134.88 1046.23 L2134.66 1046.29 L2134.43 1046.57 L2134.2 1046.3 L2133.97 1046.56 L2133.74 1046.64 L2133.52 1046.76 L2133.29 1046.7 L2133.06 1046.75 L2132.83 1046.27 L2132.61 1046.57 L2132.38 1046.26 L2132.15 1046.02 L2131.92 1046.09 L2131.69 1045.8 L2131.47 1045.59 L2131.24 1046.12 L2131.01 1046.31 L2130.78 1045.94 L2130.56 1046.74 L2130.33 1046.41 L2130.1 1046.18 L2129.87 1045.95 L2129.65 1045.58 L2129.42 1044.89 L2129.19 1044.84 L2128.96 1044.9 L2128.73 1044.97 L2128.51 1045.29 L2128.28 1045.01 L2128.05 1044.77 L2127.82 1044.38 L2127.6 1044.31 L2127.37 1044.12 L2127.14 1044.1 L2126.91 1043.66 L2126.68 1043.15 L2126.46 1043.1 L2126.23 1042.96 L2126 1042.73 L2125.77 1043 L2125.55 1042.97 L2125.32 1042.97 L2125.09 1042.68 L2124.86 1042.85 L2124.63 1042.82 L2124.41 1042.9 L2124.18 1043.04 L2123.95 1042.9 L2123.72 1042.81 L2123.5 1043.23 L2123.27 1043.02 L2123.04 1042.97 L2122.81 1042.86 L2122.58 1042.49 L2122.36 1042.61 L2122.13 1042.49 L2121.9 1042.1 L2121.67 1043.1 L2121.45 1042.8 L2121.22 1043.48 L2120.99 1043.16 L2120.76 1043.15 L2120.53 1043.62 L2120.31 1043.92 L2120.08 1043.76 L2119.85 1043.61 L2119.62 1043.77 L2119.4 1043.68 L2119.17 1043.55 L2118.94 1044.24 L2118.71 1044.16 L2118.48 1043.76 L2118.26 1043.23 L2118.03 1043.33 L2117.8 1043.28 L2117.57 1043.18 L2117.35 1043.3 L2117.12 1043.14 L2116.89 1042.89 L2116.66 1042.5 L2116.43 1043.1 L2116.21 1043.08 L2115.98 1042.74 L2115.75 1042.44 L2115.52 1042.33 L2115.3 1042.51 L2115.07 1042.27 L2114.84 1042.3 L2114.61 1043.19 L2114.38 1043.43 L2114.16 1043.26 L2113.93 1042.96 L2113.7 1043.12 L2113.47 1043.32 L2113.25 1043.48 L2113.02 1043.83 L2112.79 1043.84 L2112.56 1043.8 L2112.33 1043.72 L2112.11 1044.64 L2111.88 1044.34 L2111.65 1043.96 L2111.42 1043.78 L2111.2 1043.55 L2110.97 1044.13 L2110.74 1044.73 L2110.51 1044.74 L2110.28 1045.25 L2110.06 1045.6 L2109.83 1045.6 L2109.6 1045.01 L2109.37 1044.88 L2109.15 1044.85 L2108.92 1044.48 L2108.69 1044.15 L2108.46 1043.98 L2108.23 1044.92 L2108.01 1045.29 L2107.78 1044.84 L2107.55 1044.7 L2107.32 1044.31 L2107.1 1043.99 L2106.87 1044.45 L2106.64 1044.8 L2106.41 1044.63 L2106.18 1044.99 L2105.96 1044.49 L2105.73 1044.21 L2105.5 1044.88 L2105.27 1044.6 L2105.05 1044.73 L2104.82 1044.29 L2104.59 1044.4 L2104.36 1044.68 L2104.13 1044.84 L2103.91 1044.47 L2103.68 1044.52 L2103.45 1044.19 L2103.22 1043.6 L2103 1043.11 L2102.77 1042.72 L2102.54 1042.81 L2102.31 1042.81 L2102.08 1042.41 L2101.86 1042.33 L2101.63 1042.2 L2101.4 1042.32 L2101.17 1041.92 L2100.95 1042.01 L2100.72 1042.13 L2100.49 1042.29 L2100.26 1042.47 L2100.03 1041.81 L2099.81 1041.69 L2099.58 1041.75 L2099.35 1041.56 L2099.12 1042.06 L2098.9 1042 L2098.67 1041.52 L2098.44 1041.85 L2098.21 1041.57 L2097.98 1041.66 L2097.76 1041.56 L2097.53 1041.54 L2097.3 1041.22 L2097.07 1040.77 L2096.85 1040.49 L2096.62 1040.64 L2096.39 1040.6 L2096.16 1040.35 L2095.93 1039.83 L2095.71 1039.92 L2095.48 1039.86 L2095.25 1039.77 L2095.02 1039.84 L2094.8 1040.39 L2094.57 1040.38 L2094.34 1041.26 L2094.11 1041.47 L2093.89 1041.58 L2093.66 1041.33 L2093.43 1041.17 L2093.2 1041.09 L2092.97 1041.07 L2092.75 1040.61 L2092.52 1041.13 L2092.29 1041.34 L2092.06 1041.09 L2091.84 1041.2 L2091.61 1041.38 L2091.38 1041.14 L2091.15 1040.66 L2090.92 1040.26 L2090.7 1039.99 L2090.47 1040.03 L2090.24 1040.1 L2090.01 1040.53 L2089.79 1040.15 L2089.56 1039.95 L2089.33 1039.4 L2089.1 1039.55 L2088.87 1039.91 L2088.65 1039.5 L2088.42 1039.15 L2088.19 1038.87 L2087.96 1038.93 L2087.74 1039.54 L2087.51 1039.54 L2087.28 1039.72 L2087.05 1039.71 L2086.82 1039.8 L2086.6 1040.64 L2086.37 1041.1 L2086.14 1040.97 L2085.91 1040.93 L2085.69 1041.02 L2085.46 1041.61 L2085.23 1041.45 L2085 1041.09 L2084.77 1040.69 L2084.55 1040.4 L2084.32 1039.68 L2084.09 1039.62 L2083.86 1039.72 L2083.64 1039.97 L2083.41 1039.91 L2083.18 1040.34 L2082.95 1040.28 L2082.72 1039.75 L2082.5 1040.09 L2082.27 1040.73 L2082.04 1041 L2081.81 1041 L2081.59 1040.89 L2081.36 1041.27 L2081.13 1041.36 L2080.9 1041.31 L2080.67 1041.44 L2080.45 1041.33 L2080.22 1041.06 L2079.99 1040.84 L2079.76 1041.15 L2079.54 1040.41 L2079.31 1039.82 L2079.08 1039.74 L2078.85 1039.27 L2078.62 1038.86 L2078.4 1039.11 L2078.17 1038.58 L2077.94 1038.84 L2077.71 1038.57 L2077.49 1038.62 L2077.26 1038.22 L2077.03 1037.9 L2076.8 1037.54 L2076.57 1038.16 L2076.35 1038.35 L2076.12 1037.91 L2075.89 1037.42 L2075.66 1037.12 L2075.44 1037.02 L2075.21 1036.58 L2074.98 1036.68 L2074.75 1036.8 L2074.52 1036.92 L2074.3 1037.38 L2074.07 1037.83 L2073.84 1037.51 L2073.61 1037.71 L2073.39 1037.28 L2073.16 1037.79 L2072.93 1038.03 L2072.7 1037.81 L2072.47 1037.88 L2072.25 1038.08 L2072.02 1038.14 L2071.79 1037.48 L2071.56 1037.84 L2071.34 1037.55 L2071.11 1037.84 L2070.88 1037.86 L2070.65 1038.15 L2070.42 1038.33 L2070.2 1038.1 L2069.97 1038.74 L2069.74 1038.56 L2069.51 1038.39 L2069.29 1038.07 L2069.06 1037.41 L2068.83 1037.27 L2068.6 1037.46 L2068.37 1037.49 L2068.15 1038.17 L2067.92 1038.15 L2067.69 1038.53 L2067.46 1038.23 L2067.24 1037.77 L2067.01 1037.98 L2066.78 1038.01 L2066.55 1037.6 L2066.32 1037.07 L2066.1 1036.8 L2065.87 1036.95 L2065.64 1037.22 L2065.41 1037.81 L2065.19 1037.66 L2064.96 1037.8 L2064.73 1037.53 L2064.5 1037.67 L2064.27 1038.08 L2064.05 1037.76 L2063.82 1038.97 L2063.59 1039.19 L2063.36 1038.87 L2063.14 1038.65 L2062.91 1038.27 L2062.68 1038 L2062.45 1037.64 L2062.22 1037.17 L2062 1037.84 L2061.77 1038.05 L2061.54 1038.13 L2061.31 1037.92 L2061.09 1037.95 L2060.86 1038.09 L2060.63 1037.92 L2060.4 1037.37 L2060.17 1037.45 L2059.95 1036.98 L2059.72 1036.79 L2059.49 1036.48 L2059.26 1036 L2059.04 1035.45 L2058.81 1035.5 L2058.58 1035.81 L2058.35 1036.04 L2058.12 1036.42 L2057.9 1036.1 L2057.67 1036.36 L2057.44 1035.74 L2057.21 1035.15 L2056.99 1035.19 L2056.76 1035.69 L2056.53 1035.89 L2056.3 1036.01 L2056.08 1035.91 L2055.85 1035.85 L2055.62 1036.05 L2055.39 1036.04 L2055.16 1035.86 L2054.94 1036.07 L2054.71 1036.1 L2054.48 1035.72 L2054.25 1035.74 L2054.03 1035.54 L2053.8 1035.18 L2053.57 1035.71 L2053.34 1035.82 L2053.11 1035.86 L2052.89 1036.5 L2052.66 1036.84 L2052.43 1036.69 L2052.2 1037.25 L2051.98 1036.92 L2051.75 1036.81 L2051.52 1036.46 L2051.29 1036.23 L2051.06 1035.99 L2050.84 1036.59 L2050.61 1036.18 L2050.38 1035.91 L2050.15 1035.95 L2049.93 1035.98 L2049.7 1035.97 L2049.47 1035.71 L2049.24 1036.24 L2049.01 1036.3 L2048.79 1037.41 L2048.56 1037.34 L2048.33 1037.18 L2048.1 1036.95 L2047.88 1037.3 L2047.65 1037.02 L2047.42 1036.84 L2047.19 1037 L2046.96 1037.49 L2046.74 1036.87 L2046.51 1036.49 L2046.28 1037.07 L2046.05 1036.76 L2045.83 1036.88 L2045.6 1036.61 L2045.37 1036.52 L2045.14 1036.17 L2044.91 1036.12 L2044.69 1036.22 L2044.46 1036.19 L2044.23 1036.29 L2044 1036.18 L2043.78 1035.76 L2043.55 1035.85 L2043.32 1036.56 L2043.09 1036.76 L2042.86 1036.77 L2042.64 1036.29 L2042.41 1036.42 L2042.18 1036.54 L2041.95 1036.36 L2041.73 1036.57 L2041.5 1036.43 L2041.27 1036.1 L2041.04 1035.85 L2040.81 1036.21 L2040.59 1036.03 L2040.36 1036.4 L2040.13 1036.17 L2039.9 1036.37 L2039.68 1036.3 L2039.45 1036.03 L2039.22 1035.97 L2038.99 1035.95 L2038.76 1035.53 L2038.54 1035.73 L2038.31 1035.7 L2038.08 1035.88 L2037.85 1035.73 L2037.63 1036.3 L2037.4 1036.09 L2037.17 1036.12 L2036.94 1036.11 L2036.71 1036.04 L2036.49 1035.6 L2036.26 1035.76 L2036.03 1035.62 L2035.8 1035.11 L2035.58 1035.23 L2035.35 1034.96 L2035.12 1035.4 L2034.89 1035.33 L2034.66 1035.14 L2034.44 1035.37 L2034.21 1035.64 L2033.98 1035.58 L2033.75 1035.05 L2033.53 1035.14 L2033.3 1035.43 L2033.07 1036.1 L2032.84 1036.24 L2032.61 1036.1 L2032.39 1035.49 L2032.16 1035.5 L2031.93 1035.35 L2031.7 1034.98 L2031.48 1034.75 L2031.25 1034.75 L2031.02 1034.5 L2030.79 1034.21 L2030.56 1033.68 L2030.34 1034.03 L2030.11 1034.58 L2029.88 1034.76 L2029.65 1034.44 L2029.43 1034.67 L2029.2 1034.71 L2028.97 1034.99 L2028.74 1035.23 L2028.51 1035.23 L2028.29 1035.46 L2028.06 1035.2 L2027.83 1035.2 L2027.6 1035.11 L2027.38 1035.21 L2027.15 1035.22 L2026.92 1035.54 L2026.69 1036.3 L2026.46 1036.46 L2026.24 1036.01 L2026.01 1036.7 L2025.78 1036.24 L2025.55 1036.39 L2025.33 1036.57 L2025.1 1035.96 L2024.87 1036.49 L2024.64 1035.92 L2024.41 1035.57 L2024.19 1035.33 L2023.96 1035.67 L2023.73 1035.65 L2023.5 1036.04 L2023.28 1035.59 L2023.05 1036.13 L2022.82 1035.93 L2022.59 1036.23 L2022.36 1035.81 L2022.14 1036.03 L2021.91 1036.16 L2021.68 1036.31 L2021.45 1036.02 L2021.23 1035.86 L2021 1036.18 L2020.77 1036.05 L2020.54 1035.64 L2020.32 1036.08 L2020.09 1035.86 L2019.86 1035.38 L2019.63 1035.65 L2019.4 1035.46 L2019.18 1035.57 L2018.95 1036.13 L2018.72 1036.49 L2018.49 1036.24 L2018.27 1035.7 L2018.04 1035.65 L2017.81 1036.51 L2017.58 1036.57 L2017.35 1036.48 L2017.13 1036.8 L2016.9 1036.81 L2016.67 1036.67 L2016.44 1036.41 L2016.22 1036.11 L2015.99 1036.31 L2015.76 1036.18 L2015.53 1036.35 L2015.3 1036.32 L2015.08 1036.8 L2014.85 1036.59 L2014.62 1036.77 L2014.39 1036.66 L2014.17 1037.43 L2013.94 1037.72 L2013.71 1036.96 L2013.48 1037.19 L2013.25 1037.16 L2013.03 1037.16 L2012.8 1037.13 L2012.57 1037.48 L2012.34 1037.03 L2012.12 1037.4 L2011.89 1037.62 L2011.66 1037.46 L2011.43 1037.27 L2011.2 1036.53 L2010.98 1036.49 L2010.75 1036.31 L2010.52 1036.27 L2010.29 1035.91 L2010.07 1035.92 L2009.84 1035.93 L2009.61 1035.75 L2009.38 1035.45 L2009.15 1035.89 L2008.93 1035.47 L2008.7 1035.8 L2008.47 1036 L2008.24 1035.82 L2008.02 1035.94 L2007.79 1035.38 L2007.56 1035.78 L2007.33 1036.04 L2007.1 1036.44 L2006.88 1037.56 L2006.65 1037.83 L2006.42 1037.74 L2006.19 1037.48 L2005.97 1037.24 L2005.74 1037.33 L2005.51 1037.11 L2005.28 1036.97 L2005.05 1036.59 L2004.83 1036.96 L2004.6 1037.71 L2004.37 1038.26 L2004.14 1037.73 L2003.92 1037.55 L2003.69 1038.21 L2003.46 1038.44 L2003.23 1038.15 L2003 1037.67 L2002.78 1037.87 L2002.55 1038.34 L2002.32 1038.26 L2002.09 1038.25 L2001.87 1038 L2001.64 1038.09 L2001.41 1037.83 L2001.18 1037.85 L2000.95 1037.93 L2000.73 1037.77 L2000.5 1037.88 L2000.27 1037.65 L2000.04 1037.88 L1999.82 1038.04 L1999.59 1038.69 L1999.36 1038.53 L1999.13 1038.51 L1998.9 1038.76 L1998.68 1038.47 L1998.45 1038.01 L1998.22 1037.7 L1997.99 1038.44 L1997.77 1038.29 L1997.54 1038.2 L1997.31 1037.96 L1997.08 1038.41 L1996.85 1038.85 L1996.63 1038.51 L1996.4 1038.11 L1996.17 1038.82 L1995.94 1038.89 L1995.72 1039.36 L1995.49 1039.22 L1995.26 1039.01 L1995.03 1039.4 L1994.8 1039.56 L1994.58 1039.77 L1994.35 1039.84 L1994.12 1040.27 L1993.89 1040 L1993.67 1040.11 L1993.44 1039.96 L1993.21 1039.77 L1992.98 1039.78 L1992.75 1039.69 L1992.53 1039.46 L1992.3 1039.87 L1992.07 1039.56 L1991.84 1039.21 L1991.62 1038.76 L1991.39 1038.26 L1991.16 1038.24 L1990.93 1038.11 L1990.7 1038.7 L1990.48 1038.36 L1990.25 1038.21 L1990.02 1038.42 L1989.79 1039.03 L1989.57 1039.48 L1989.34 1039.35 L1989.11 1039.27 L1988.88 1039.25 L1988.65 1039.38 L1988.43 1039.47 L1988.2 1039.2 L1987.97 1039.19 L1987.74 1038.97 L1987.52 1038.78 L1987.29 1038.9 L1987.06 1039.74 L1986.83 1039.65 L1986.6 1039.46 L1986.38 1040 L1986.15 1039.98 L1985.92 1040.47 L1985.69 1040.64 L1985.47 1040.08 L1985.24 1040.05 L1985.01 1039.74 L1984.78 1040.11 L1984.56 1039.79 L1984.33 1039.47 L1984.1 1039.56 L1983.87 1039.18 L1983.64 1039.51 L1983.42 1039.32 L1983.19 1038.8 L1982.96 1039.02 L1982.73 1038.95 L1982.51 1038.31 L1982.28 1038.07 L1982.05 1038.46 L1981.82 1038.71 L1981.59 1038.4 L1981.37 1038.59 L1981.14 1038.65 L1980.91 1038.38 L1980.68 1038.12 L1980.46 1038.59 L1980.23 1038.36 L1980 1038.58 L1979.77 1038.63 L1979.54 1039.07 L1979.32 1039.14 L1979.09 1039.26 L1978.86 1039.49 L1978.63 1039.69 L1978.41 1039.42 L1978.18 1039.34 L1977.95 1039.4 L1977.72 1039.45 L1977.49 1039.59 L1977.27 1039.78 L1977.04 1039.69 L1976.81 1039.88 L1976.58 1039.62 L1976.36 1040.69 L1976.13 1040.45 L1975.9 1040.91 L1975.67 1040.65 L1975.44 1040.29 L1975.22 1040.06 L1974.99 1040.5 L1974.76 1040.1 L1974.53 1040.34 L1974.31 1039.92 L1974.08 1039.61 L1973.85 1039.49 L1973.62 1039.18 L1973.39 1039.12 L1973.17 1039.11 L1972.94 1038.58 L1972.71 1038.52 L1972.48 1038.29 L1972.26 1037.83 L1972.03 1037.54 L1971.8 1036.91 L1971.57 1036.87 L1971.34 1037.11 L1971.12 1037.1 L1970.89 1036.71 L1970.66 1036.33 L1970.43 1037.07 L1970.21 1036.7 L1969.98 1037.28 L1969.75 1037.34 L1969.52 1037.27 L1969.29 1036.86 L1969.07 1037.5 L1968.84 1037.42 L1968.61 1037.62 L1968.38 1037.55 L1968.16 1037.27 L1967.93 1037.34 L1967.7 1037.2 L1967.47 1036.68 L1967.24 1036.9 L1967.02 1036.56 L1966.79 1036.86 L1966.56 1036.61 L1966.33 1036.06 L1966.11 1036.27 L1965.88 1036.25 L1965.65 1036.32 L1965.42 1036.19 L1965.19 1036.3 L1964.97 1036.14 L1964.74 1036.04 L1964.51 1036.78 L1964.28 1037.35 L1964.06 1038.23 L1963.83 1038.05 L1963.6 1038.35 L1963.37 1037.76 L1963.14 1037.69 L1962.92 1037.47 L1962.69 1037.82 L1962.46 1037.57 L1962.23 1037.12 L1962.01 1036.78 L1961.78 1036.53 L1961.55 1036.93 L1961.32 1037.49 L1961.09 1037.92 L1960.87 1038.19 L1960.64 1038.47 L1960.41 1038.4 L1960.18 1039.27 L1959.96 1038.73 L1959.73 1038.35 L1959.5 1038.22 L1959.27 1038.2 L1959.04 1038.4 L1958.82 1038.04 L1958.59 1037.76 L1958.36 1038.11 L1958.13 1037.55 L1957.91 1037.73 L1957.68 1038.36 L1957.45 1038.48 L1957.22 1038.36 L1956.99 1037.96 L1956.77 1038.19 L1956.54 1037.98 L1956.31 1037.85 L1956.08 1037.82 L1955.86 1037.9 L1955.63 1037.54 L1955.4 1037.37 L1955.17 1037.36 L1954.94 1037.53 L1954.72 1037.2 L1954.49 1037.22 L1954.26 1037.6 L1954.03 1037.25 L1953.81 1036.97 L1953.58 1037.95 L1953.35 1037.38 L1953.12 1037.53 L1952.89 1036.89 L1952.67 1036.55 L1952.44 1036.62 L1952.21 1036.53 L1951.98 1036.46 L1951.76 1036.9 L1951.53 1036.7 L1951.3 1036.43 L1951.07 1036.05 L1950.84 1035.98 L1950.62 1036.33 L1950.39 1036.68 L1950.16 1036.48 L1949.93 1036.05 L1949.71 1036.45 L1949.48 1036.66 L1949.25 1036.74 L1949.02 1036.64 L1948.79 1036.96 L1948.57 1036.71 L1948.34 1036.32 L1948.11 1036.26 L1947.88 1035.79 L1947.66 1036.31 L1947.43 1036.31 L1947.2 1036.34 L1946.97 1036.25 L1946.75 1036.23 L1946.52 1035.79 L1946.29 1035.13 L1946.06 1035.7 L1945.83 1035.72 L1945.61 1035.82 L1945.38 1036.3 L1945.15 1036.75 L1944.92 1036.49 L1944.7 1036.4 L1944.47 1036.01 L1944.24 1036.14 L1944.01 1036.33 L1943.78 1035.96 L1943.56 1036.03 L1943.33 1035.96 L1943.1 1037.14 L1942.87 1037.42 L1942.65 1037.53 L1942.42 1037.84 L1942.19 1037.72 L1941.96 1037.63 L1941.73 1038.29 L1941.51 1038.1 L1941.28 1038.22 L1941.05 1037.79 L1940.82 1037.39 L1940.6 1037.25 L1940.37 1037.38 L1940.14 1037.3 L1939.91 1038.07 L1939.68 1038.19 L1939.46 1037.8 L1939.23 1037.8 L1939 1037.9 L1938.77 1037.61 L1938.55 1037.62 L1938.32 1037.48 L1938.09 1037.5 L1937.86 1037.08 L1937.63 1036.53 L1937.41 1035.95 L1937.18 1036.83 L1936.95 1037.06 L1936.72 1036.8 L1936.5 1036.67 L1936.27 1036.68 L1936.04 1036.85 L1935.81 1037.04 L1935.58 1036.56 L1935.36 1036.27 L1935.13 1036.35 L1934.9 1036.5 L1934.67 1036.88 L1934.45 1037.38 L1934.22 1037.36 L1933.99 1038.16 L1933.76 1037.61 L1933.53 1037.51 L1933.31 1037.91 L1933.08 1038.2 L1932.85 1037.86 L1932.62 1037.36 L1932.4 1036.69 L1932.17 1036.27 L1931.94 1036.05 L1931.71 1035.87 L1931.48 1035.7 L1931.26 1035.81 L1931.03 1035.75 L1930.8 1035.42 L1930.57 1035.93 L1930.35 1035.71 L1930.12 1035.55 L1929.89 1036.65 L1929.66 1036.65 L1929.43 1036.34 L1929.21 1035.79 L1928.98 1035.26 L1928.75 1035.17 L1928.52 1034.96 L1928.3 1035.04 L1928.07 1035.34 L1927.84 1035.26 L1927.61 1035.29 L1927.38 1034.78 L1927.16 1035.48 L1926.93 1035.81 L1926.7 1035.5 L1926.47 1035.35 L1926.25 1034.92 L1926.02 1034.4 L1925.79 1034.81 L1925.56 1035.39 L1925.33 1035.2 L1925.11 1034.87 L1924.88 1034.96 L1924.65 1034.83 L1924.42 1034.62 L1924.2 1035.18 L1923.97 1035.65 L1923.74 1035.23 L1923.51 1034.95 L1923.28 1034.38 L1923.06 1034.51 L1922.83 1034.23 L1922.6 1033.82 L1922.37 1033.76 L1922.15 1033.82 L1921.92 1033.54 L1921.69 1033.68 L1921.46 1033.72 L1921.23 1034.14 L1921.01 1033.73 L1920.78 1033.81 L1920.55 1033.91 L1920.32 1033.16 L1920.1 1033.08 L1919.87 1033.47 L1919.64 1032.88 L1919.41 1032.79 L1919.18 1033.96 L1918.96 1033.27 L1918.73 1034.73 L1918.5 1035.38 L1918.27 1035.59 L1918.05 1036.3 L1917.82 1036.4 L1917.59 1036.67 L1917.36 1036.39 L1917.13 1036.03 L1916.91 1035.89 L1916.68 1035.83 L1916.45 1036.09 L1916.22 1036.1 L1916 1036.24 L1915.77 1036.3 L1915.54 1036.9 L1915.31 1036.41 L1915.08 1036.84 L1914.86 1036.52 L1914.63 1036.69 L1914.4 1036.97 L1914.17 1036.52 L1913.95 1036.26 L1913.72 1035.92 L1913.49 1036.21 L1913.26 1036.46 L1913.03 1035.83 L1912.81 1036.52 L1912.58 1037.3 L1912.35 1036.85 L1912.12 1037.27 L1911.9 1037.79 L1911.67 1037.97 L1911.44 1037.89 L1911.21 1037.98 L1910.99 1038.59 L1910.76 1038.52 L1910.53 1038.55 L1910.3 1038.52 L1910.07 1038.59 L1909.85 1038.82 L1909.62 1039.37 L1909.39 1038.62 L1909.16 1038.9 L1908.94 1039.08 L1908.71 1038.97 L1908.48 1038.88 L1908.25 1039.06 L1908.02 1038.86 L1907.8 1039.2 L1907.57 1038.44 L1907.34 1039.22 L1907.11 1038.98 L1906.89 1038.23 L1906.66 1037.65 L1906.43 1037.7 L1906.2 1038.74 L1905.97 1038.59 L1905.75 1038.44 L1905.52 1039.11 L1905.29 1039.1 L1905.06 1038.53 L1904.84 1038.56 L1904.61 1038.31 L1904.38 1037.57 L1904.15 1037.26 L1903.92 1037.28 L1903.7 1037.02 L1903.47 1037.18 L1903.24 1036.71 L1903.01 1036.5 L1902.79 1036.52 L1902.56 1037.31 L1902.33 1037.35 L1902.1 1037.04 L1901.87 1036.45 L1901.65 1036.36 L1901.42 1036.12 L1901.19 1035.87 L1900.96 1035.36 L1900.74 1035.37 L1900.51 1035.04 L1900.28 1034.6 L1900.05 1035.18 L1899.82 1035.15 L1899.6 1034.52 L1899.37 1034.59 L1899.14 1034.8 L1898.91 1034.64 L1898.69 1034.53 L1898.46 1034.56 L1898.23 1034.78 L1898 1035.09 L1897.77 1035.45 L1897.55 1035.1 L1897.32 1035.78 L1897.09 1035.3 L1896.86 1034.92 L1896.64 1035.2 L1896.41 1035.69 L1896.18 1034.97 L1895.95 1035.24 L1895.72 1036.18 L1895.5 1036.04 L1895.27 1036.21 L1895.04 1036.8 L1894.81 1037.44 L1894.59 1037.32 L1894.36 1037.52 L1894.13 1037.1 L1893.9 1036.6 L1893.67 1036.23 L1893.45 1036.01 L1893.22 1036.14 L1892.99 1035.92 L1892.76 1035.55 L1892.54 1035.62 L1892.31 1035.67 L1892.08 1036.58 L1891.85 1036.86 L1891.62 1036.61 L1891.4 1037.38 L1891.17 1037.67 L1890.94 1038.06 L1890.71 1038.17 L1890.49 1038.77 L1890.26 1038.98 L1890.03 1039.83 L1889.8 1040.31 L1889.57 1040.33 L1889.35 1040.15 L1889.12 1039.82 L1888.89 1039.79 L1888.66 1039.76 L1888.44 1039.62 L1888.21 1039.44 L1887.98 1039.69 L1887.75 1040.11 L1887.52 1040.3 L1887.3 1040.65 L1887.07 1040.22 L1886.84 1040.26 L1886.61 1040.77 L1886.39 1040.75 L1886.16 1040.81 L1885.93 1040.38 L1885.7 1040.28 L1885.47 1039.68 L1885.25 1039.24 L1885.02 1039.8 L1884.79 1039.94 L1884.56 1040.09 L1884.34 1040.2 L1884.11 1039.74 L1883.88 1040.02 L1883.65 1039.74 L1883.42 1039.49 L1883.2 1039.13 L1882.97 1039.11 L1882.74 1038.63 L1882.51 1038.47 L1882.29 1038.41 L1882.06 1037.87 L1881.83 1037.5 L1881.6 1037.16 L1881.37 1036.57 L1881.15 1036.78 L1880.92 1036.28 L1880.69 1036.41 L1880.46 1036.41 L1880.24 1035.97 L1880.01 1035.49 L1879.78 1035.58 L1879.55 1035.48 L1879.32 1035.61 L1879.1 1035.56 L1878.87 1036.48 L1878.64 1036.19 L1878.41 1035.49 L1878.19 1035.53 L1877.96 1034.86 L1877.73 1034.96 L1877.5 1035.23 L1877.27 1034.97 L1877.05 1035.14 L1876.82 1034.6 L1876.59 1035.1 L1876.36 1034.93 L1876.14 1035.69 L1875.91 1035.39 L1875.68 1035.17 L1875.45 1034.66 L1875.23 1034.85 L1875 1034.77 L1874.77 1034.65 L1874.54 1035.21 L1874.31 1034.74 L1874.09 1034.47 L1873.86 1034.38 L1873.63 1034.55 L1873.4 1034.61 L1873.18 1034.52 L1872.95 1034.56 L1872.72 1034.4 L1872.49 1033.9 L1872.26 1034.18 L1872.04 1034.16 L1871.81 1033.86 L1871.58 1034.71 L1871.35 1034.83 L1871.13 1034.95 L1870.9 1035.12 L1870.67 1035.23 L1870.44 1035.07 L1870.21 1034.82 L1869.99 1035.3 L1869.76 1035.82 L1869.53 1035.51 L1869.3 1035.33 L1869.08 1035.02 L1868.85 1034.58 L1868.62 1034.76 L1868.39 1035.09 L1868.16 1034.74 L1867.94 1034.74 L1867.71 1034.74 L1867.48 1034.76 L1867.25 1035.68 L1867.03 1035.51 L1866.8 1035.04 L1866.57 1034.7 L1866.34 1034.05 L1866.11 1033.63 L1865.89 1033.57 L1865.66 1034.43 L1865.43 1033.99 L1865.2 1033.92 L1864.98 1033.35 L1864.75 1033.45 L1864.52 1033.14 L1864.29 1032.59 L1864.06 1032.52 L1863.84 1031.97 L1863.61 1031.93 L1863.38 1032.13 L1863.15 1032.14 L1862.93 1032.46 L1862.7 1032.65 L1862.47 1032.15 L1862.24 1031.5 L1862.01 1031.24 L1861.79 1031.3 L1861.56 1031.29 L1861.33 1031.49 L1861.1 1030.76 L1860.88 1030.9 L1860.65 1031.02 L1860.42 1030.06 L1860.19 1030.54 L1859.96 1030.4 L1859.74 1030.62 L1859.51 1030.74 L1859.28 1031.2 L1859.05 1031.68 L1858.83 1031.24 L1858.6 1031.01 L1858.37 1031.36 L1858.14 1031.99 L1857.91 1031.96 L1857.69 1031.96 L1857.46 1031.69 L1857.23 1031.23 L1857 1031.39 L1856.78 1031.46 L1856.55 1031.31 L1856.32 1032 L1856.09 1031.78 L1855.86 1031.94 L1855.64 1032.53 L1855.41 1032.8 L1855.18 1033.05 L1854.95 1033.45 L1854.73 1033.91 L1854.5 1033.66 L1854.27 1033.16 L1854.04 1033.28 L1853.81 1033.4 L1853.59 1033.42 L1853.36 1032.99 L1853.13 1033.37 L1852.9 1033.59 L1852.68 1034 L1852.45 1034.13 L1852.22 1034.48 L1851.99 1034.49 L1851.76 1034.69 L1851.54 1034.67 L1851.31 1034.77 L1851.08 1034.27 L1850.85 1033.56 L1850.63 1034.08 L1850.4 1033.73 L1850.17 1033.98 L1849.94 1033.86 L1849.71 1033.41 L1849.49 1033.81 L1849.26 1033.85 L1849.03 1034.23 L1848.8 1034.21 L1848.58 1034.57 L1848.35 1035.07 L1848.12 1035.27 L1847.89 1035.96 L1847.66 1035.92 L1847.44 1036.1 L1847.21 1035.98 L1846.98 1036.32 L1846.75 1035.56 L1846.53 1035.83 L1846.3 1035.85 L1846.07 1035.76 L1845.84 1035.76 L1845.61 1035.75 L1845.39 1035.99 L1845.16 1036.11 L1844.93 1035.97 L1844.7 1035.98 L1844.48 1035.81 L1844.25 1035.67 L1844.02 1035.3 L1843.79 1035.15 L1843.56 1035.27 L1843.34 1035.03 L1843.11 1035.49 L1842.88 1035.4 L1842.65 1034.72 L1842.43 1034.49 L1842.2 1034.03 L1841.97 1033.94 L1841.74 1033.74 L1841.51 1033.29 L1841.29 1033.36 L1841.06 1033.76 L1840.83 1033.52 L1840.6 1033.45 L1840.38 1033.26 L1840.15 1032.94 L1839.92 1032.74 L1839.69 1032.85 L1839.46 1033.01 L1839.24 1032.66 L1839.01 1032.86 L1838.78 1032.7 L1838.55 1033.15 L1838.33 1032.92 L1838.1 1032.99 L1837.87 1032.55 L1837.64 1032.15 L1837.42 1031.94 L1837.19 1032.04 L1836.96 1031.98 L1836.73 1032.04 L1836.5 1031.41 L1836.28 1031.34 L1836.05 1031.11 L1835.82 1031.17 L1835.59 1031.17 L1835.37 1031.19 L1835.14 1030.77 L1834.91 1030.29 L1834.68 1029.79 L1834.45 1030.07 L1834.23 1030.02 L1834 1029.96 L1833.77 1029.79 L1833.54 1029.5 L1833.32 1029.41 L1833.09 1028.73 L1832.86 1028.7 L1832.63 1028.14 L1832.4 1028.06 L1832.18 1028.47 L1831.95 1027.95 L1831.72 1027.91 L1831.49 1027.46 L1831.27 1027.08 L1831.04 1026.79 L1830.81 1026.31 L1830.58 1025.96 L1830.35 1025.95 L1830.13 1026.17 L1829.9 1027.18 L1829.67 1027.13 L1829.44 1026.54 L1829.22 1027.48 L1828.99 1027.45 L1828.76 1027.13 L1828.53 1028.15 L1828.3 1027.96 L1828.08 1027.25 L1827.85 1027.02 L1827.62 1026.39 L1827.39 1026.14 L1827.17 1026.08 L1826.94 1025.78 L1826.71 1025.91 L1826.48 1026.71 L1826.25 1026.37 L1826.03 1026.16 L1825.8 1025.67 L1825.57 1025.68 L1825.34 1025.13 L1825.12 1025.44 L1824.89 1025.31 L1824.66 1025.33 L1824.43 1025.34 L1824.2 1025.41 L1823.98 1025.66 L1823.75 1026.09 L1823.52 1026.65 L1823.29 1026.82 L1823.07 1026.83 L1822.84 1026.87 L1822.61 1026.7 L1822.38 1027 L1822.15 1027.4 L1821.93 1026.85 L1821.7 1026.87 L1821.47 1026.61 L1821.24 1027.19 L1821.02 1026.98 L1820.79 1027 L1820.56 1026.91 L1820.33 1026.42 L1820.1 1026.79 L1819.88 1026.43 L1819.65 1026.42 L1819.42 1027.16 L1819.19 1027.2 L1818.97 1027.47 L1818.74 1027.37 L1818.51 1027.77 L1818.28 1028.73 L1818.05 1029.52 L1817.83 1029.19 L1817.6 1029.18 L1817.37 1028.72 L1817.14 1029.04 L1816.92 1028.76 L1816.69 1028.5 L1816.46 1028.29 L1816.23 1028.39 L1816 1028.48 L1815.78 1028.55 L1815.55 1029.07 L1815.32 1029.18 L1815.09 1029.26 L1814.87 1029.32 L1814.64 1029.4 L1814.41 1029.5 L1814.18 1029.48 L1813.95 1029.14 L1813.73 1028.95 L1813.5 1029.15 L1813.27 1029.1 L1813.04 1028.97 L1812.82 1028.67 L1812.59 1028.18 L1812.36 1029.57 L1812.13 1029.26 L1811.9 1028.74 L1811.68 1028.86 L1811.45 1029.22 L1811.22 1028.98 L1810.99 1029.07 L1810.77 1028.61 L1810.54 1028.88 L1810.31 1028.95 L1810.08 1028.93 L1809.85 1028.76 L1809.63 1029.26 L1809.4 1030.09 L1809.17 1029.98 L1808.94 1029.76 L1808.72 1029.21 L1808.49 1030.1 L1808.26 1030.16 L1808.03 1030.41 L1807.8 1031.02 L1807.58 1030.41 L1807.35 1030.32 L1807.12 1030.25 L1806.89 1029.96 L1806.67 1029.81 L1806.44 1029.78 L1806.21 1029.16 L1805.98 1029.94 L1805.75 1030.19 L1805.53 1029.96 L1805.3 1030.17 L1805.07 1030 L1804.84 1030.5 L1804.62 1030.22 L1804.39 1029.85 L1804.16 1030.41 L1803.93 1030.76 L1803.7 1030.64 L1803.48 1030.36 L1803.25 1030.71 L1803.02 1030.35 L1802.79 1029.96 L1802.57 1029.98 L1802.34 1030.38 L1802.11 1030.23 L1801.88 1030.52 L1801.66 1030.17 L1801.43 1030.75 L1801.2 1030.58 L1800.97 1030.09 L1800.74 1030.87 L1800.52 1030.57 L1800.29 1030.68 L1800.06 1030.55 L1799.83 1030.79 L1799.61 1030.48 L1799.38 1029.85 L1799.15 1029.74 L1798.92 1029.6 L1798.69 1029.18 L1798.47 1028.92 L1798.24 1029.77 L1798.01 1029.33 L1797.78 1029.5 L1797.56 1029.31 L1797.33 1029.65 L1797.1 1029.58 L1796.87 1029.01 L1796.64 1028.57 L1796.42 1029.03 L1796.19 1028.33 L1795.96 1028.59 L1795.73 1028.25 L1795.51 1028.42 L1795.28 1029.16 L1795.05 1028.97 L1794.82 1029.37 L1794.59 1030.07 L1794.37 1030.16 L1794.14 1029.94 L1793.91 1029.46 L1793.68 1028.63 L1793.46 1028.16 L1793.23 1028.21 L1793 1028.3 L1792.77 1028.26 L1792.54 1028.61 L1792.32 1028.51 L1792.09 1027.93 L1791.86 1028.4 L1791.63 1028.99 L1791.41 1028.86 L1791.18 1028.94 L1790.95 1028.23 L1790.72 1028.17 L1790.49 1028.85 L1790.27 1028.65 L1790.04 1028.42 L1789.81 1027.97 L1789.58 1028.47 L1789.36 1029.09 L1789.13 1029.29 L1788.9 1029.43 L1788.67 1029.77 L1788.44 1029.61 L1788.22 1029.4 L1787.99 1030.07 L1787.76 1029.85 L1787.53 1029.36 L1787.31 1029.58 L1787.08 1029.24 L1786.85 1028.97 L1786.62 1028.95 L1786.39 1029.25 L1786.17 1028.57 L1785.94 1029.07 L1785.71 1029.83 L1785.48 1029.89 L1785.26 1029.22 L1785.03 1029.76 L1784.8 1029.39 L1784.57 1029.55 L1784.34 1029.36 L1784.12 1029.82 L1783.89 1030.07 L1783.66 1029.91 L1783.43 1029.81 L1783.21 1030.4 L1782.98 1029.97 L1782.75 1031.37 L1782.52 1033.53 L1782.29 1033.05 L1782.07 1033.04 L1781.84 1033.06 L1781.61 1033.23 L1781.38 1033.62 L1781.16 1033.59 L1780.93 1033.84 L1780.7 1034.58 L1780.47 1034.55 L1780.24 1034.18 L1780.02 1034.27 L1779.79 1034.69 L1779.56 1034.64 L1779.33 1034.41 L1779.11 1033.9 L1778.88 1033.76 L1778.65 1034.42 L1778.42 1034.5 L1778.19 1034.98 L1777.97 1034.94 L1777.74 1034.7 L1777.51 1034.51 L1777.28 1034.69 L1777.06 1035.12 L1776.83 1034.82 L1776.6 1034.49 L1776.37 1034.19 L1776.14 1034.17 L1775.92 1033.93 L1775.69 1033.44 L1775.46 1033.22 L1775.23 1033.28 L1775.01 1033.64 L1774.78 1034.52 L1774.55 1034.26 L1774.32 1034.02 L1774.09 1034.24 L1773.87 1034.31 L1773.64 1033.87 L1773.41 1034.48 L1773.18 1034.18 L1772.96 1034.16 L1772.73 1034.19 L1772.5 1033.6 L1772.27 1033.61 L1772.04 1034.38 L1771.82 1034.16 L1771.59 1034.58 L1771.36 1034.47 L1771.13 1034.02 L1770.91 1034.26 L1770.68 1034.59 L1770.45 1034.91 L1770.22 1034.66 L1769.99 1034.43 L1769.77 1034.36 L1769.54 1033.73 L1769.31 1033.63 L1769.08 1033.58 L1768.86 1033.15 L1768.63 1033.1 L1768.4 1032.97 L1768.17 1032.55 L1767.94 1032.55 L1767.72 1032.27 L1767.49 1032.24 L1767.26 1031.86 L1767.03 1031.68 L1766.81 1032.28 L1766.58 1033 L1766.35 1032.55 L1766.12 1031.93 L1765.9 1031.5 L1765.67 1031.33 L1765.44 1030.93 L1765.21 1031.39 L1764.98 1031.62 L1764.76 1031.54 L1764.53 1031.29 L1764.3 1031.27 L1764.07 1031.25 L1763.85 1031.34 L1763.62 1031.25 L1763.39 1032.46 L1763.16 1032.44 L1762.93 1032.52 L1762.71 1031.89 L1762.48 1032.24 L1762.25 1031.98 L1762.02 1031.82 L1761.8 1032.13 L1761.57 1032.06 L1761.34 1031.58 L1761.11 1031.71 L1760.88 1031.78 L1760.66 1031.57 L1760.43 1031.68 L1760.2 1031.87 L1759.97 1032.02 L1759.75 1032.55 L1759.52 1032.05 L1759.29 1031.42 L1759.06 1031 L1758.83 1031.1 L1758.61 1030.95 L1758.38 1030.34 L1758.15 1030.68 L1757.92 1030.96 L1757.7 1031.27 L1757.47 1030.95 L1757.24 1031.39 L1757.01 1031.45 L1756.78 1031.26 L1756.56 1031.7 L1756.33 1032.17 L1756.1 1032.44 L1755.87 1032.46 L1755.65 1032.44 L1755.42 1032.48 L1755.19 1032.94 L1754.96 1033.21 L1754.73 1033.05 L1754.51 1033.15 L1754.28 1033.28 L1754.05 1033.26 L1753.82 1033.31 L1753.6 1033.7 L1753.37 1034.23 L1753.14 1033.26 L1752.91 1033.01 L1752.68 1033.16 L1752.46 1033.07 L1752.23 1032.85 L1752 1033.57 L1751.77 1033.63 L1751.55 1032.8 L1751.32 1032.46 L1751.09 1032.32 L1750.86 1031.69 L1750.63 1031.53 L1750.41 1031.3 L1750.18 1031.02 L1749.95 1031.18 L1749.72 1031.4 L1749.5 1030.9 L1749.27 1030.74 L1749.04 1031.32 L1748.81 1030.88 L1748.58 1031.37 L1748.36 1031.58 L1748.13 1031.51 L1747.9 1031.67 L1747.67 1031.6 L1747.45 1031.73 L1747.22 1032.05 L1746.99 1031.93 L1746.76 1032.18 L1746.53 1032.3 L1746.31 1032.42 L1746.08 1031.72 L1745.85 1031.81 L1745.62 1031.77 L1745.4 1031.85 L1745.17 1032.65 L1744.94 1031.81 L1744.71 1031.49 L1744.48 1031.61 L1744.26 1031.61 L1744.03 1031.6 L1743.8 1031.35 L1743.57 1030.93 L1743.35 1030.96 L1743.12 1031 L1742.89 1030.39 L1742.66 1030.09 L1742.43 1029.66 L1742.21 1029.29 L1741.98 1029.27 L1741.75 1028.79 L1741.52 1029.54 L1741.3 1029.69 L1741.07 1029.9 L1740.84 1030.82 L1740.61 1030.88 L1740.38 1030.31 L1740.16 1029.9 L1739.93 1029.25 L1739.7 1029.2 L1739.47 1029.25 L1739.25 1029.92 L1739.02 1030.01 L1738.79 1029.44 L1738.56 1029.34 L1738.33 1029.3 L1738.11 1029.32 L1737.88 1030.04 L1737.65 1029.73 L1737.42 1029.62 L1737.2 1029.68 L1736.97 1030.14 L1736.74 1029.83 L1736.51 1029.98 L1736.28 1029.58 L1736.06 1029.38 L1735.83 1029.49 L1735.6 1029.49 L1735.37 1029.37 L1735.15 1029.49 L1734.92 1029.39 L1734.69 1029.45 L1734.46 1029.45 L1734.23 1028.99 L1734.01 1029.09 L1733.78 1029.17 L1733.55 1028.57 L1733.32 1028.11 L1733.1 1027.91 L1732.87 1027.88 L1732.64 1027.05 L1732.41 1027.21 L1732.18 1027.3 L1731.96 1027.47 L1731.73 1027.37 L1731.5 1026.85 L1731.27 1026.65 L1731.05 1026.36 L1730.82 1026.16 L1730.59 1025.74 L1730.36 1025.57 L1730.14 1025.84 L1729.91 1025.93 L1729.68 1025.55 L1729.45 1025.42 L1729.22 1025.27 L1729 1026.15 L1728.77 1025.98 L1728.54 1026.38 L1728.31 1026.03 L1728.09 1026.01 L1727.86 1026.29 L1727.63 1025.81 L1727.4 1025.63 L1727.17 1025.46 L1726.95 1025.09 L1726.72 1025.71 L1726.49 1025.22 L1726.26 1025.19 L1726.04 1025.36 L1725.81 1025.15 L1725.58 1024.73 L1725.35 1024.38 L1725.12 1024.94 L1724.9 1024.84 L1724.67 1025.12 L1724.44 1025.28 L1724.21 1025.23 L1723.99 1025.54 L1723.76 1025.44 L1723.53 1026.01 L1723.3 1025.88 L1723.07 1026.38 L1722.85 1026.73 L1722.62 1026.37 L1722.39 1026.19 L1722.16 1026.37 L1721.94 1025.96 L1721.71 1025.62 L1721.48 1025.38 L1721.25 1024.65 L1721.02 1025.39 L1720.8 1024.93 L1720.57 1024.74 L1720.34 1025.06 L1720.11 1024.76 L1719.89 1024.42 L1719.66 1024.62 L1719.43 1024.64 L1719.2 1024.99 L1718.97 1024.4 L1718.75 1024 L1718.52 1024.03 L1718.29 1023.99 L1718.06 1024.18 L1717.84 1023.9 L1717.61 1024.13 L1717.38 1023.93 L1717.15 1024.08 L1716.92 1023.81 L1716.7 1023.37 L1716.47 1023.92 L1716.24 1023.97 L1716.01 1024.47 L1715.79 1025.33 L1715.56 1025.12 L1715.33 1025.8 L1715.1 1026.32 L1714.87 1026.15 L1714.65 1025.96 L1714.42 1026.28 L1714.19 1026.12 L1713.96 1026.14 L1713.74 1026.32 L1713.51 1026.68 L1713.28 1026.84 L1713.05 1026.49 L1712.82 1026.58 L1712.6 1026.37 L1712.37 1027.07 L1712.14 1026.73 L1711.91 1026.77 L1711.69 1027.08 L1711.46 1027.08 L1711.23 1027.3 L1711 1026.44 L1710.77 1027.43 L1710.55 1028.38 L1710.32 1028.05 L1710.09 1027.8 L1709.86 1028.27 L1709.64 1027.95 L1709.41 1027.52 L1709.18 1027.47 L1708.95 1027.38 L1708.72 1027.59 L1708.5 1027.98 L1708.27 1027.59 L1708.04 1027.98 L1707.81 1027.24 L1707.59 1027.47 L1707.36 1028.06 L1707.13 1028.04 L1706.9 1027.61 L1706.67 1027.25 L1706.45 1027.04 L1706.22 1026.66 L1705.99 1026.68 L1705.76 1025.81 L1705.54 1025.53 L1705.31 1024.78 L1705.08 1024.15 L1704.85 1025.92 L1704.62 1026.04 L1704.4 1025.87 L1704.17 1025.92 L1703.94 1025.81 L1703.71 1026.52 L1703.49 1026.64 L1703.26 1026.4 L1703.03 1026.34 L1702.8 1025.83 L1702.57 1025.57 L1702.35 1024.99 L1702.12 1024.87 L1701.89 1024.85 L1701.66 1024.47 L1701.44 1024.62 L1701.21 1025.46 L1700.98 1024.73 L1700.75 1024.4 L1700.52 1023.92 L1700.3 1023.33 L1700.07 1023.62 L1699.84 1024.54 L1699.61 1024.77 L1699.39 1024.66 L1699.16 1024.66 L1698.93 1024.54 L1698.7 1024.48 L1698.47 1024.56 L1698.25 1024.27 L1698.02 1023.75 L1697.79 1023.94 L1697.56 1024 L1697.34 1024.34 L1697.11 1024.32 L1696.88 1024 L1696.65 1024.33 L1696.42 1023.88 L1696.2 1024.77 L1695.97 1024.43 L1695.74 1024.47 L1695.51 1025.16 L1695.29 1024.97 L1695.06 1024.86 L1694.83 1024.57 L1694.6 1024.21 L1694.37 1025 L1694.15 1025.1 L1693.92 1024.42 L1693.69 1024.47 L1693.46 1026.31 L1693.24 1026.65 L1693.01 1026.28 L1692.78 1025.89 L1692.55 1025.38 L1692.33 1025.09 L1692.1 1024.72 L1691.87 1025.36 L1691.64 1025.16 L1691.41 1025.01 L1691.19 1025.69 L1690.96 1025.55 L1690.73 1026.29 L1690.5 1026.2 L1690.28 1026.07 L1690.05 1025.97 L1689.82 1025.81 L1689.59 1025.61 L1689.36 1025.18 L1689.14 1025.55 L1688.91 1025.74 L1688.68 1025.2 L1688.45 1025.34 L1688.23 1025.32 L1688 1025.38 L1687.77 1025.55 L1687.54 1025.1 L1687.31 1024.87 L1687.09 1024.43 L1686.86 1024.02 L1686.63 1023.89 L1686.4 1023.18 L1686.18 1022.97 L1685.95 1022.82 L1685.72 1022.98 L1685.49 1022.81 L1685.26 1022.9 L1685.04 1023.06 L1684.81 1022.51 L1684.58 1023.4 L1684.35 1022.9 L1684.13 1023.17 L1683.9 1022.88 L1683.67 1023.91 L1683.44 1023.49 L1683.21 1023.59 L1682.99 1023.39 L1682.76 1023.41 L1682.53 1023.41 L1682.3 1023.88 L1682.08 1023.35 L1681.85 1023.55 L1681.62 1023.84 L1681.39 1023.55 L1681.16 1023.74 L1680.94 1023.39 L1680.71 1023.56 L1680.48 1024.11 L1680.25 1024.18 L1680.03 1024.18 L1679.8 1025.12 L1679.57 1025.5 L1679.34 1025.3 L1679.11 1025.5 L1678.89 1025.95 L1678.66 1026.93 L1678.43 1027.72 L1678.2 1028.2 L1677.98 1027.85 L1677.75 1027.49 L1677.52 1027.41 L1677.29 1027.08 L1677.06 1026.59 L1676.84 1026.67 L1676.61 1026.49 L1676.38 1026 L1676.15 1026.54 L1675.93 1026.51 L1675.7 1026.66 L1675.47 1027.23 L1675.24 1026.67 L1675.01 1027.21 L1674.79 1027.29 L1674.56 1027.09 L1674.33 1026.84 L1674.1 1026.7 L1673.88 1026.62 L1673.65 1026.29 L1673.42 1026.39 L1673.19 1026.33 L1672.96 1026.47 L1672.74 1026.46 L1672.51 1026.46 L1672.28 1026.19 L1672.05 1027.67 L1671.83 1027.22 L1671.6 1026.92 L1671.37 1026.85 L1671.14 1026.3 L1670.91 1026.11 L1670.69 1026.46 L1670.46 1025.91 L1670.23 1025.32 L1670 1024.85 L1669.78 1024.2 L1669.55 1024.18 L1669.32 1023.82 L1669.09 1023.85 L1668.86 1023.36 L1668.64 1023.82 L1668.41 1023.76 L1668.18 1023.24 L1667.95 1022.93 L1667.73 1022.82 L1667.5 1022.8 L1667.27 1022.3 L1667.04 1022.22 L1666.81 1022.39 L1666.59 1021.83 L1666.36 1021.6 L1666.13 1021.41 L1665.9 1021.2 L1665.68 1021.58 L1665.45 1021.4 L1665.22 1020.73 L1664.99 1019.99 L1664.76 1020.27 L1664.54 1019.96 L1664.31 1019.24 L1664.08 1019.52 L1663.85 1019.7 L1663.63 1019.24 L1663.4 1019.25 L1663.17 1019.12 L1662.94 1019.56 L1662.71 1019.36 L1662.49 1019.14 L1662.26 1019.37 L1662.03 1019.46 L1661.8 1019.31 L1661.58 1019.06 L1661.35 1019.75 L1661.12 1020.26 L1660.89 1020.25 L1660.66 1021.05 L1660.44 1021.27 L1660.21 1021.73 L1659.98 1021.2 L1659.75 1020.63 L1659.53 1020.99 L1659.3 1020.96 L1659.07 1020.56 L1658.84 1020.42 L1658.61 1020.97 L1658.39 1021.39 L1658.16 1021.71 L1657.93 1022.23 L1657.7 1021.85 L1657.48 1022.07 L1657.25 1021.95 L1657.02 1021.6 L1656.79 1021.17 L1656.57 1020.89 L1656.34 1020.82 L1656.11 1020.23 L1655.88 1019.99 L1655.65 1019.75 L1655.43 1018.92 L1655.2 1019.51 L1654.97 1019.94 L1654.74 1020.07 L1654.52 1019.06 L1654.29 1018.52 L1654.06 1018.97 L1653.83 1019.01 L1653.6 1018.77 L1653.38 1019.85 L1653.15 1019.72 L1652.92 1019.57 L1652.69 1020.17 L1652.47 1020.32 L1652.24 1019.8 L1652.01 1019.49 L1651.78 1019.29 L1651.55 1019.39 L1651.33 1019.58 L1651.1 1018.97 L1650.87 1018.8 L1650.64 1018.86 L1650.42 1019.34 L1650.19 1020.15 L1649.96 1020.64 L1649.73 1020.09 L1649.5 1019.33 L1649.28 1019.53 L1649.05 1018.75 L1648.82 1018.28 L1648.59 1018.08 L1648.37 1018.63 L1648.14 1018.46 L1647.91 1018.4 L1647.68 1018.15 L1647.45 1018.47 L1647.23 1018.98 L1647 1019.28 L1646.77 1018.55 L1646.54 1018.87 L1646.32 1018.74 L1646.09 1018.41 L1645.86 1018.39 L1645.63 1018.15 L1645.4 1017.31 L1645.18 1016.33 L1644.95 1016.78 L1644.72 1017.15 L1644.49 1016.74 L1644.27 1016.59 L1644.04 1016.73 L1643.81 1016.54 L1643.58 1016.76 L1643.35 1016.75 L1643.13 1015.93 L1642.9 1015.22 L1642.67 1015.01 L1642.44 1014.59 L1642.22 1014.47 L1641.99 1014.38 L1641.76 1013.61 L1641.53 1013.92 L1641.3 1014.05 L1641.08 1013.58 L1640.85 1013.61 L1640.62 1013.59 L1640.39 1013.28 L1640.17 1013.81 L1639.94 1013.39 L1639.71 1013.17 L1639.48 1012.62 L1639.25 1013.03 L1639.03 1012.71 L1638.8 1012.5 L1638.57 1011.69 L1638.34 1010.95 L1638.12 1011.31 L1637.89 1010.92 L1637.66 1010.97 L1637.43 1010.48 L1637.2 1010.56 L1636.98 1009.91 L1636.75 1009.72 L1636.52 1009.94 L1636.29 1009.86 L1636.07 1010.32 L1635.84 1010.09 L1635.61 1009.86 L1635.38 1009.51 L1635.15 1009.76 L1634.93 1009.43 L1634.7 1009.59 L1634.47 1009.13 L1634.24 1009.2 L1634.02 1010.22 L1633.79 1009.64 L1633.56 1009.45 L1633.33 1009.94 L1633.1 1010.15 L1632.88 1010.4 L1632.65 1010.96 L1632.42 1011.09 L1632.19 1010.68 L1631.97 1010.49 L1631.74 1010.23 L1631.51 1010.16 L1631.28 1009.18 L1631.05 1009.7 L1630.83 1010.15 L1630.6 1010.61 L1630.37 1010.85 L1630.14 1010.79 L1629.92 1010.61 L1629.69 1010.4 L1629.46 1010.38 L1629.23 1010.48 L1629 1010.71 L1628.78 1009.97 L1628.55 1009.76 L1628.32 1010.54 L1628.09 1010.56 L1627.87 1010.39 L1627.64 1009.67 L1627.41 1009.36 L1627.18 1009.75 L1626.95 1009.52 L1626.73 1009.13 L1626.5 1010.67 L1626.27 1011.23 L1626.04 1011.37 L1625.82 1011.96 L1625.59 1011.3 L1625.36 1011.07 L1625.13 1011.4 L1624.9 1011.19 L1624.68 1011.23 L1624.45 1011.66 L1624.22 1012.03 L1623.99 1011.54 L1623.77 1011.75 L1623.54 1012.16 L1623.31 1011.56 L1623.08 1011.49 L1622.85 1011.19 L1622.63 1011.02 L1622.4 1011 L1622.17 1011.48 L1621.94 1011.63 L1621.72 1011.89 L1621.49 1011.82 L1621.26 1011.48 L1621.03 1010.61 L1620.81 1010.4 L1620.58 1010.64 L1620.35 1010.55 L1620.12 1010.66 L1619.89 1010.25 L1619.67 1011.36 L1619.44 1011.54 L1619.21 1011.58 L1618.98 1011.73 L1618.76 1011.5 L1618.53 1011.17 L1618.3 1011.06 L1618.07 1011.58 L1617.84 1011.77 L1617.62 1011.7 L1617.39 1011.44 L1617.16 1012.48 L1616.93 1012.34 L1616.71 1012.06 L1616.48 1012.18 L1616.25 1012.15 L1616.02 1011.69 L1615.79 1011.44 L1615.57 1012.61 L1615.34 1011.89 L1615.11 1011.58 L1614.88 1012.31 L1614.66 1012.31 L1614.43 1012.57 L1614.2 1012.7 L1613.97 1012.3 L1613.74 1012.56 L1613.52 1012.93 L1613.29 1013.99 L1613.06 1013.6 L1612.83 1012.54 L1612.61 1012.09 L1612.38 1012.76 L1612.15 1011.97 L1611.92 1012.1 L1611.69 1011.7 L1611.47 1012.09 L1611.24 1011.82 L1611.01 1011.49 L1610.78 1011.13 L1610.56 1011.17 L1610.33 1010.41 L1610.1 1010.66 L1609.87 1010.49 L1609.64 1011.18 L1609.42 1010.91 L1609.19 1010.98 L1608.96 1011.39 L1608.73 1011.63 L1608.51 1012.24 L1608.28 1012.22 L1608.05 1012.34 L1607.82 1011.99 L1607.59 1011.5 L1607.37 1011.14 L1607.14 1011.27 L1606.91 1011.33 L1606.68 1011.03 L1606.46 1010.78 L1606.23 1010.6 L1606 1010.83 L1605.77 1011.49 L1605.54 1011.33 L1605.32 1011.38 L1605.09 1010.87 L1604.86 1010.46 L1604.63 1010.17 L1604.41 1009.81 L1604.18 1010.47 L1603.95 1010.2 L1603.72 1010.2 L1603.49 1009.73 L1603.27 1009.13 L1603.04 1008.89 L1602.81 1008.96 L1602.58 1009.38 L1602.36 1009.37 L1602.13 1009.13 L1601.9 1008.69 L1601.67 1008.25 L1601.44 1008.78 L1601.22 1008.86 L1600.99 1009.09 L1600.76 1009.36 L1600.53 1009.52 L1600.31 1008.63 L1600.08 1008.38 L1599.85 1008.99 L1599.62 1008.75 L1599.39 1009.03 L1599.17 1008.71 L1598.94 1009.3 L1598.71 1009.13 L1598.48 1009.14 L1598.26 1009.6 L1598.03 1009.49 L1597.8 1009.06 L1597.57 1008.22 L1597.34 1008.4 L1597.12 1008.7 L1596.89 1008.03 L1596.66 1008.14 L1596.43 1008.42 L1596.21 1008.57 L1595.98 1009.6 L1595.75 1010.01 L1595.52 1011 L1595.29 1010.95 L1595.07 1010.57 L1594.84 1011.62 L1594.61 1011.93 L1594.38 1011.63 L1594.16 1011.16 L1593.93 1010.85 L1593.7 1010.47 L1593.47 1009.97 L1593.24 1009.82 L1593.02 1009.6 L1592.79 1010.28 L1592.56 1010.38 L1592.33 1009.56 L1592.11 1009.98 L1591.88 1009.48 L1591.65 1009.69 L1591.42 1009.91 L1591.19 1011.11 L1590.97 1011.22 L1590.74 1011.41 L1590.51 1010.88 L1590.28 1010.32 L1590.06 1009.66 L1589.83 1010.22 L1589.6 1010.73 L1589.37 1011.05 L1589.14 1010.44 L1588.92 1009.9 L1588.69 1009.38 L1588.46 1009.53 L1588.23 1009.52 L1588.01 1008.93 L1587.78 1009.03 L1587.55 1008.85 L1587.32 1009.04 L1587.09 1009.1 L1586.87 1009.06 L1586.64 1008.55 L1586.41 1008.36 L1586.18 1007.78 L1585.96 1007.8 L1585.73 1007.27 L1585.5 1007.21 L1585.27 1006.58 L1585.04 1006.02 L1584.82 1006.56 L1584.59 1006.62 L1584.36 1007.82 L1584.13 1007.74 L1583.91 1008.12 L1583.68 1007.79 L1583.45 1008.43 L1583.22 1008.76 L1583 1008.12 L1582.77 1008.3 L1582.54 1008.4 L1582.31 1008.64 L1582.08 1008.26 L1581.86 1008.63 L1581.63 1008.47 L1581.4 1008.05 L1581.17 1007.47 L1580.95 1007.13 L1580.72 1006.9 L1580.49 1006.53 L1580.26 1006.54 L1580.03 1005.98 L1579.81 1005.58 L1579.58 1005.65 L1579.35 1005.19 L1579.12 1005.44 L1578.9 1005.27 L1578.67 1005.68 L1578.44 1005.42 L1578.21 1004.87 L1577.98 1004.9 L1577.76 1004.93 L1577.53 1005.73 L1577.3 1006.53 L1577.07 1007.13 L1576.85 1006.79 L1576.62 1007.01 L1576.39 1007.6 L1576.16 1007.58 L1575.93 1007.39 L1575.71 1008.27 L1575.48 1008.51 L1575.25 1008.05 L1575.02 1007.87 L1574.8 1008.48 L1574.57 1008.45 L1574.34 1009.46 L1574.11 1009.62 L1573.88 1009.27 L1573.66 1008.71 L1573.43 1009.13 L1573.2 1009.01 L1572.97 1008.21 L1572.75 1008.11 L1572.52 1008.33 L1572.29 1007.71 L1572.06 1007.71 L1571.83 1008.19 L1571.61 1008.53 L1571.38 1007.97 L1571.15 1007.67 L1570.92 1009.2 L1570.7 1009.07 L1570.47 1009.46 L1570.24 1009.26 L1570.01 1009.22 L1569.78 1009.13 L1569.56 1008.42 L1569.33 1008.1 L1569.1 1008.36 L1568.87 1008.06 L1568.65 1008.42 L1568.42 1008.25 L1568.19 1008.1 L1567.96 1007.89 L1567.73 1007.88 L1567.51 1007.97 L1567.28 1007.75 L1567.05 1008.54 L1566.82 1008.34 L1566.6 1008 L1566.37 1008.6 L1566.14 1007.94 L1565.91 1008.89 L1565.68 1009.93 L1565.46 1009.9 L1565.23 1009.58 L1565 1009.28 L1564.77 1008.69 L1564.55 1009.07 L1564.32 1009.29 L1564.09 1008.71 L1563.86 1009.34 L1563.63 1008.73 L1563.41 1008.58 L1563.18 1008.07 L1562.95 1008.01 L1562.72 1008.52 L1562.5 1008.64 L1562.27 1008.28 L1562.04 1008.54 L1561.81 1008.36 L1561.58 1008.1 L1561.36 1008.4 L1561.13 1008.32 L1560.9 1008 L1560.67 1007.78 L1560.45 1007.78 L1560.22 1008.48 L1559.99 1007.91 L1559.76 1007.88 L1559.53 1007.91 L1559.31 1007.7 L1559.08 1008.55 L1558.85 1009.16 L1558.62 1009.26 L1558.4 1009.61 L1558.17 1010.19 L1557.94 1009.87 L1557.71 1009.78 L1557.48 1010.04 L1557.26 1009.79 L1557.03 1010.03 L1556.8 1009.61 L1556.57 1009.47 L1556.35 1009.58 L1556.12 1009.2 L1555.89 1009.77 L1555.66 1009.18 L1555.43 1008.96 L1555.21 1008.42 L1554.98 1008.84 L1554.75 1009.07 L1554.52 1009.52 L1554.3 1009.11 L1554.07 1008.89 L1553.84 1009.5 L1553.61 1009.03 L1553.38 1008.94 L1553.16 1010.12 L1552.93 1010.22 L1552.7 1009.96 L1552.47 1010.22 L1552.25 1009.85 L1552.02 1010.72 L1551.79 1010.82 L1551.56 1010.88 L1551.33 1010.74 L1551.11 1010.47 L1550.88 1011.26 L1550.65 1010.97 L1550.42 1011.24 L1550.2 1011.68 L1549.97 1011.57 L1549.74 1010.71 L1549.51 1011.14 L1549.28 1010.68 L1549.06 1011.1 L1548.83 1011.15 L1548.6 1011.76 L1548.37 1011.48 L1548.15 1011.76 L1547.92 1011.76 L1547.69 1011.5 L1547.46 1011.3 L1547.24 1010.58 L1547.01 1010.99 L1546.78 1011.23 L1546.55 1011.98 L1546.32 1012.03 L1546.1 1012.08 L1545.87 1011.67 L1545.64 1012.38 L1545.41 1012.54 L1545.19 1012.96 L1544.96 1013.2 L1544.73 1012.82 L1544.5 1012.71 L1544.27 1012.23 L1544.05 1011.96 L1543.82 1011.89 L1543.59 1011.51 L1543.36 1011.1 L1543.14 1011.17 L1542.91 1011.25 L1542.68 1011.28 L1542.45 1011.48 L1542.22 1010.98 L1542 1010.58 L1541.77 1010.42 L1541.54 1010.56 L1541.31 1009.77 L1541.09 1009.99 L1540.86 1009.54 L1540.63 1009.62 L1540.4 1009.38 L1540.17 1009.34 L1539.95 1008.64 L1539.72 1008.66 L1539.49 1008.68 L1539.26 1009.06 L1539.04 1009.2 L1538.81 1009.29 L1538.58 1009.81 L1538.35 1011.15 L1538.12 1010.68 L1537.9 1010.31 L1537.67 1010.52 L1537.44 1011.56 L1537.21 1011.06 L1536.99 1011.21 L1536.76 1010.76 L1536.53 1011.35 L1536.3 1010.92 L1536.07 1010.84 L1535.85 1011.87 L1535.62 1011.76 L1535.39 1011.47 L1535.16 1011.62 L1534.94 1011.87 L1534.71 1012.52 L1534.48 1012.63 L1534.25 1012.01 L1534.02 1012.31 L1533.8 1011.92 L1533.57 1011.39 L1533.34 1011.04 L1533.11 1011 L1532.89 1011.17 L1532.66 1011.6 L1532.43 1010.84 L1532.2 1011.01 L1531.97 1011.58 L1531.75 1011.29 L1531.52 1011.3 L1531.29 1010.93 L1531.06 1011.16 L1530.84 1011.01 L1530.61 1010.74 L1530.38 1010.11 L1530.15 1009.39 L1529.92 1009.02 L1529.7 1009.58 L1529.47 1009.14 L1529.24 1008.62 L1529.01 1008.29 L1528.79 1008.59 L1528.56 1008.12 L1528.33 1008.21 L1528.1 1008.71 L1527.87 1009.45 L1527.65 1009.55 L1527.42 1008.75 L1527.19 1008.32 L1526.96 1007.99 L1526.74 1008.02 L1526.51 1008.22 L1526.28 1007.52 L1526.05 1006.8 L1525.82 1007.02 L1525.6 1007.11 L1525.37 1007.28 L1525.14 1006.89 L1524.91 1007.44 L1524.69 1007.59 L1524.46 1007.67 L1524.23 1007.69 L1524 1008.17 L1523.77 1008.81 L1523.55 1009.59 L1523.32 1010.89 L1523.09 1011.36 L1522.86 1010.47 L1522.64 1010.55 L1522.41 1011.4 L1522.18 1011.57 L1521.95 1011.48 L1521.72 1010.95 L1521.5 1010.55 L1521.27 1010.6 L1521.04 1010.22 L1520.81 1010.07 L1520.59 1010.19 L1520.36 1010.56 L1520.13 1010.31 L1519.9 1010.32 L1519.67 1009.81 L1519.45 1009.93 L1519.22 1010.52 L1518.99 1011.88 L1518.76 1011.4 L1518.54 1011.19 L1518.31 1011.28 L1518.08 1011.11 L1517.85 1010.24 L1517.62 1011.06 L1517.4 1011.66 L1517.17 1011.73 L1516.94 1011.52 L1516.71 1010.95 L1516.49 1010.92 L1516.26 1011.44 L1516.03 1011.16 L1515.8 1011.52 L1515.57 1011.33 L1515.35 1011.25 L1515.12 1011.39 L1514.89 1011.19 L1514.66 1010.95 L1514.44 1010.96 L1514.21 1011.02 L1513.98 1010.83 L1513.75 1010.18 L1513.52 1010.27 L1513.3 1009.79 L1513.07 1010.08 L1512.84 1009.77 L1512.61 1011.41 L1512.39 1010.93 L1512.16 1010.1 L1511.93 1010.48 L1511.7 1010.49 L1511.48 1010.2 L1511.25 1010.83 L1511.02 1010.62 L1510.79 1010.78 L1510.56 1010.78 L1510.34 1010.19 L1510.11 1009.95 L1509.88 1010.27 L1509.65 1009.58 L1509.43 1009.53 L1509.2 1009.78 L1508.97 1010.2 L1508.74 1010.29 L1508.51 1011.2 L1508.29 1010.36 L1508.06 1010.83 L1507.83 1011.16 L1507.6 1011.48 L1507.38 1011.75 L1507.15 1011.55 L1506.92 1012.7 L1506.69 1012.28 L1506.46 1012.04 L1506.24 1011.79 L1506.01 1011.2 L1505.78 1011.56 L1505.55 1010.66 L1505.33 1010.96 L1505.1 1011.56 L1504.87 1011.73 L1504.64 1011.74 L1504.41 1011.67 L1504.19 1010.8 L1503.96 1012.26 L1503.73 1013.22 L1503.5 1012.47 L1503.28 1012.55 L1503.05 1013.87 L1502.82 1013.76 L1502.59 1013.57 L1502.36 1013.1 L1502.14 1012.99 L1501.91 1013.36 L1501.68 1012.65 L1501.45 1011.93 L1501.23 1011.95 L1501 1012.52 L1500.77 1011.77 L1500.54 1011.04 L1500.31 1010.47 L1500.09 1010.91 L1499.86 1011.24 L1499.63 1010.52 L1499.4 1011.08 L1499.18 1011.79 L1498.95 1011.94 L1498.72 1011.84 L1498.49 1011.67 L1498.26 1011.19 L1498.04 1010.95 L1497.81 1010.39 L1497.58 1010.08 L1497.35 1010.6 L1497.13 1010.38 L1496.9 1011.06 L1496.67 1010.34 L1496.44 1009.93 L1496.21 1008.88 L1495.99 1008.38 L1495.76 1008.97 L1495.53 1009.31 L1495.3 1009.44 L1495.08 1009.77 L1494.85 1009.02 L1494.62 1008.63 L1494.39 1007.95 L1494.16 1007.67 L1493.94 1007.94 L1493.71 1007.4 L1493.48 1007.59 L1493.25 1007.7 L1493.03 1007.37 L1492.8 1008.18 L1492.57 1007.64 L1492.34 1007.99 L1492.11 1007.82 L1491.89 1007.87 L1491.66 1007.64 L1491.43 1008.1 L1491.2 1008.64 L1490.98 1008.73 L1490.75 1008.81 L1490.52 1008.75 L1490.29 1009.02 L1490.06 1008.98 L1489.84 1009.13 L1489.61 1009.05 L1489.38 1009.08 L1489.15 1009.16 L1488.93 1009.05 L1488.7 1008.44 L1488.47 1008.44 L1488.24 1008.02 L1488.01 1008 L1487.79 1007.62 L1487.56 1008.04 L1487.33 1008.45 L1487.1 1008.28 L1486.88 1008.52 L1486.65 1009.03 L1486.42 1008.7 L1486.19 1008.89 L1485.96 1008.96 L1485.74 1009.41 L1485.51 1009.8 L1485.28 1009.26 L1485.05 1008.22 L1484.83 1008.23 L1484.6 1008.21 L1484.37 1007.97 L1484.14 1007.91 L1483.91 1008.08 L1483.69 1008.01 L1483.46 1008.6 L1483.23 1008.44 L1483 1007.14 L1482.78 1007.34 L1482.55 1007.92 L1482.32 1007.64 L1482.09 1007.48 L1481.86 1008.16 L1481.64 1007.97 L1481.41 1008.09 L1481.18 1008.46 L1480.95 1007.69 L1480.73 1008.24 L1480.5 1007.86 L1480.27 1007.83 L1480.04 1007.12 L1479.81 1007.92 L1479.59 1008.4 L1479.36 1007.53 L1479.13 1007.47 L1478.9 1007.54 L1478.68 1007.64 L1478.45 1007.83 L1478.22 1007.52 L1477.99 1008.12 L1477.76 1009.14 L1477.54 1008.56 L1477.31 1007.82 L1477.08 1008.25 L1476.85 1008.14 L1476.63 1007.69 L1476.4 1007.32 L1476.17 1008.07 L1475.94 1007.9 L1475.71 1007.73 L1475.49 1007.02 L1475.26 1007.76 L1475.03 1007.57 L1474.8 1008.64 L1474.58 1008.48 L1474.35 1008.1 L1474.12 1007.95 L1473.89 1007.59 L1473.67 1007.05 L1473.44 1006.55 L1473.21 1006.43 L1472.98 1006.35 L1472.75 1006.02 L1472.53 1005.85 L1472.3 1005.35 L1472.07 1004.8 L1471.84 1003.87 L1471.62 1003.62 L1471.39 1003.49 L1471.16 1003.02 L1470.93 1003.08 L1470.7 1002.7 L1470.48 1002.51 L1470.25 1003.61 L1470.02 1003.96 L1469.79 1004.69 L1469.57 1004.31 L1469.34 1004.22 L1469.11 1004.51 L1468.88 1005.35 L1468.65 1005.01 L1468.43 1004.61 L1468.2 1004.95 L1467.97 1004.79 L1467.74 1005.01 L1467.52 1004.93 L1467.29 1004.75 L1467.06 1005.25 L1466.83 1004.63 L1466.6 1003.83 L1466.38 1003.26 L1466.15 1003.4 L1465.92 1002.89 L1465.69 1002.42 L1465.47 1002.04 L1465.24 1001.93 L1465.01 1001.82 L1464.78 1001.53 L1464.55 1001.49 L1464.33 1001.84 L1464.1 1001.63 L1463.87 1002.35 L1463.64 1002.87 L1463.42 1002.7 L1463.19 1001.99 L1462.96 1001.9 L1462.73 1002.45 L1462.5 1002.36 L1462.28 1001.75 L1462.05 1001.27 L1461.82 1001.76 L1461.59 1001.96 L1461.37 1002.22 L1461.14 1002.87 L1460.91 1002.79 L1460.68 1002.56 L1460.45 1002.1 L1460.23 1002.63 L1460 1002.47 L1459.77 1002.45 L1459.54 1002.06 L1459.32 1001.85 L1459.09 1001.13 L1458.86 1000.9 L1458.63 1001.21 L1458.4 1001.13 L1458.18 1000.63 L1457.95 999.815 L1457.72 1000.33 L1457.49 1000.02 L1457.27 1000.56 L1457.04 1000.45 L1456.81 1001.1 L1456.58 1000.64 L1456.35 1000.54 L1456.13 1000.47 L1455.9 999.731 L1455.67 1000.06 L1455.44 1000.42 L1455.22 1000.43 L1454.99 999.47 L1454.76 999.704 L1454.53 999.245 L1454.3 999.364 L1454.08 999.686 L1453.85 999.925 L1453.62 999.665 L1453.39 998.848 L1453.17 998.605 L1452.94 998.521 L1452.71 997.969 L1452.48 997.626 L1452.25 998.676 L1452.03 998.802 L1451.8 998.653 L1451.57 999.48 L1451.34 998.733 L1451.12 998.903 L1450.89 998.924 L1450.66 999.62 L1450.43 998.711 L1450.2 999.048 L1449.98 999.167 L1449.75 998.717 L1449.52 998.648 L1449.29 998.85 L1449.07 999.155 L1448.84 999.207 L1448.61 999.58 L1448.38 999.557 L1448.15 999.549 L1447.93 1000.48 L1447.7 1000.49 L1447.47 1001.24 L1447.24 1002.03 L1447.02 1002.12 L1446.79 1001.74 L1446.56 1002.08 L1446.33 1002.66 L1446.1 1002.8 L1445.88 1002.28 L1445.65 1002.23 L1445.42 1002.78 L1445.19 1003.01 L1444.97 1002.84 L1444.74 1002.34 L1444.51 1002.3 L1444.28 1002.26 L1444.05 1002.33 L1443.83 1002.33 L1443.6 1002.92 L1443.37 1003.15 L1443.14 1004.01 L1442.92 1003.41 L1442.69 1003.45 L1442.46 1003.46 L1442.23 1003.73 L1442 1004.53 L1441.78 1004.89 L1441.55 1005 L1441.32 1004.63 L1441.09 1004.59 L1440.87 1005.13 L1440.64 1004.87 L1440.41 1005.08 L1440.18 1004.3 L1439.95 1005.06 L1439.73 1004.95 L1439.5 1004.09 L1439.27 1004.62 L1439.04 1005.62 L1438.82 1005.4 L1438.59 1005.6 L1438.36 1005.04 L1438.13 1004.91 L1437.91 1004.6 L1437.68 1004.41 L1437.45 1004.06 L1437.22 1003.87 L1436.99 1004.4 L1436.77 1005.16 L1436.54 1005.13 L1436.31 1004.57 L1436.08 1004.23 L1435.86 1004.7 L1435.63 1005.25 L1435.4 1005.29 L1435.17 1007.15 L1434.94 1006.92 L1434.72 1006.37 L1434.49 1005.94 L1434.26 1006.03 L1434.03 1005.31 L1433.81 1004.66 L1433.58 1004.33 L1433.35 1004.77 L1433.12 1004.85 L1432.89 1004.83 L1432.67 1004.27 L1432.44 1003.66 L1432.21 1004.25 L1431.98 1006.02 L1431.76 1005.26 L1431.53 1005.86 L1431.3 1007.03 L1431.07 1007.04 L1430.84 1006.61 L1430.62 1005.94 L1430.39 1005.69 L1430.16 1006.05 L1429.93 1005.7 L1429.71 1005.92 L1429.48 1005.88 L1429.25 1005.52 L1429.02 1006.04 L1428.79 1005.77 L1428.57 1005.33 L1428.34 1005.37 L1428.11 1004.89 L1427.88 1004.38 L1427.66 1004.1 L1427.43 1004.11 L1427.2 1004.14 L1426.97 1005.25 L1426.74 1004.71 L1426.52 1003.88 L1426.29 1004.07 L1426.06 1003.59 L1425.83 1003.58 L1425.61 1003.65 L1425.38 1003.5 L1425.15 1002.48 L1424.92 1002.59 L1424.69 1001.47 L1424.47 1002.42 L1424.24 1001.67 L1424.01 1000.76 L1423.78 999.874 L1423.56 999.803 L1423.33 999.946 L1423.1 1000.06 L1422.87 999.779 L1422.64 999.567 L1422.42 1000.28 L1422.19 1000.36 L1421.96 1000.64 L1421.73 1000.46 L1421.51 1000.31 L1421.28 1000.08 L1421.05 1000.86 L1420.82 1000.69 L1420.59 1000.57 L1420.37 1000.79 L1420.14 1000.83 L1419.91 1000.22 L1419.68 999.732 L1419.46 1000.04 L1419.23 999.282 L1419 1000.57 L1418.77 1000.23 L1418.54 999.951 L1418.32 999.219 L1418.09 999.53 L1417.86 999.465 L1417.63 999.307 L1417.41 999.067 L1417.18 999.474 L1416.95 999.094 L1416.72 999.158 L1416.49 998.637 L1416.27 998.646 L1416.04 999.188 L1415.81 999.516 L1415.58 1000.86 L1415.36 1000.66 L1415.13 1001.24 L1414.9 1000.47 L1414.67 999.975 L1414.44 1000.18 L1414.22 1000.34 L1413.99 1000.48 L1413.76 999.716 L1413.53 1000.34 L1413.31 1000.82 L1413.08 1001.3 L1412.85 1001.14 L1412.62 1000.8 L1412.39 1001.09 L1412.17 1001.19 L1411.94 1000.56 L1411.71 1001.06 L1411.48 1001.86 L1411.26 1002.09 L1411.03 1001.23 L1410.8 1001.51 L1410.57 1001.67 L1410.34 1001.29 L1410.12 1001.49 L1409.89 1000.97 L1409.66 1001.01 L1409.43 1000.03 L1409.21 1000.03 L1408.98 999.725 L1408.75 1000.18 L1408.52 1000.37 L1408.29 1000.13 L1408.07 1001.35 L1407.84 1001.17 L1407.61 1000.95 L1407.38 1001.38 L1407.16 1001.63 L1406.93 1000.51 L1406.7 1001.19 L1406.47 1001.49 L1406.24 1000.87 L1406.02 1001.06 L1405.79 1000.34 L1405.56 1000.17 L1405.33 1000.76 L1405.11 1001.95 L1404.88 1001.7 L1404.65 1000.72 L1404.42 999.803 L1404.19 999.022 L1403.97 998.225 L1403.74 997.896 L1403.51 998.142 L1403.28 997.788 L1403.06 997.229 L1402.83 997.143 L1402.6 996.88 L1402.37 996.349 L1402.15 996.175 L1401.92 997.38 L1401.69 997.463 L1401.46 996.904 L1401.23 996.729 L1401.01 996.048 L1400.78 996.844 L1400.55 997.271 L1400.32 997.48 L1400.1 997.571 L1399.87 998.342 L1399.64 998.061 L1399.41 997.241 L1399.18 996.906 L1398.96 997.286 L1398.73 998.48 L1398.5 998.868 L1398.27 998.587 L1398.05 998.983 L1397.82 999.336 L1397.59 998.837 L1397.36 998.664 L1397.13 999.429 L1396.91 999.015 L1396.68 1000.07 L1396.45 1000.14 L1396.22 1000.46 L1396 1000.34 L1395.77 1000.35 L1395.54 999.934 L1395.31 999.91 L1395.08 999.471 L1394.86 998.864 L1394.63 999.076 L1394.4 998.642 L1394.17 998.713 L1393.95 999.532 L1393.72 1000.65 L1393.49 1000.62 L1393.26 1000.63 L1393.03 1000.79 L1392.81 1000.24 L1392.58 1000.26 L1392.35 1000.07 L1392.12 999.74 L1391.9 999.849 L1391.67 999.891 L1391.44 1000.21 L1391.21 1000.82 L1390.98 1000.83 L1390.76 1000.18 L1390.53 999.569 L1390.3 999.225 L1390.07 999.034 L1389.85 999.923 L1389.62 999.812 L1389.39 999.697 L1389.16 1000.17 L1388.93 999.545 L1388.71 999.042 L1388.48 999 L1388.25 999.719 L1388.02 1000.43 L1387.8 1000.77 L1387.57 1000.77 L1387.34 999.937 L1387.11 1000.43 L1386.88 999.253 L1386.66 999.076 L1386.43 999.261 L1386.2 999.866 L1385.97 999.272 L1385.75 999.82 L1385.52 999.834 L1385.29 1000.41 L1385.06 1000.58 L1384.83 1001.07 L1384.61 1000.97 L1384.38 1000.81 L1384.15 1001.08 L1383.92 1000.25 L1383.7 1000.15 L1383.47 1000.75 L1383.24 1001.47 L1383.01 1000.96 L1382.78 1000.95 L1382.56 1001.13 L1382.33 1001.52 L1382.1 1001.26 L1381.87 1001.05 L1381.65 1000.89 L1381.42 1000.85 L1381.19 1000.99 L1380.96 1001.13 L1380.73 1000.42 L1380.51 1000.54 L1380.28 1000.02 L1380.05 999.82 L1379.82 999.825 L1379.6 1000.04 L1379.37 999.528 L1379.14 999.881 L1378.91 999.776 L1378.68 1000.32 L1378.46 1000.01 L1378.23 1000.4 L1378 1000.21 L1377.77 1000.18 L1377.55 999.498 L1377.32 999.395 L1377.09 1000.14 L1376.86 999.191 L1376.63 999.18 L1376.41 998.908 L1376.18 999.06 L1375.95 998.521 L1375.72 998.634 L1375.5 998.457 L1375.27 997.844 L1375.04 999.055 L1374.81 998.447 L1374.58 998.549 L1374.36 998.91 L1374.13 998.04 L1373.9 997.284 L1373.67 998.53 L1373.45 998.113 L1373.22 998.46 L1372.99 998.027 L1372.76 997.905 L1372.53 997.541 L1372.31 997.068 L1372.08 997.38 L1371.85 997.095 L1371.62 996.602 L1371.4 996.238 L1371.17 997.072 L1370.94 996.573 L1370.71 996.03 L1370.48 996.258 L1370.26 996.817 L1370.03 997.031 L1369.8 996.614 L1369.57 996.53 L1369.35 996.912 L1369.12 996.684 L1368.89 996.749 L1368.66 996.718 L1368.43 997.237 L1368.21 997.536 L1367.98 997.393 L1367.75 996.596 L1367.52 996.794 L1367.3 996.637 L1367.07 996.524 L1366.84 995.952 L1366.61 995.425 L1366.39 994.992 L1366.16 994.678 L1365.93 994.195 L1365.7 994.178 L1365.47 994.677 L1365.25 993.92 L1365.02 993.697 L1364.79 993.212 L1364.56 993.565 L1364.34 994.769 L1364.11 995.154 L1363.88 994.488 L1363.65 993.825 L1363.42 993.995 L1363.2 993.715 L1362.97 993.422 L1362.74 993.483 L1362.51 993.371 L1362.29 993.469 L1362.06 992.699 L1361.83 992.04 L1361.6 991.82 L1361.37 990.931 L1361.15 991.62 L1360.92 991.274 L1360.69 992.151 L1360.46 992.039 L1360.24 991.839 L1360.01 990.916 L1359.78 990.544 L1359.55 990.229 L1359.32 991.025 L1359.1 991.827 L1358.87 991.543 L1358.64 991.651 L1358.41 991.943 L1358.19 992.217 L1357.96 993.01 L1357.73 993.898 L1357.5 993.912 L1357.27 993.891 L1357.05 994.364 L1356.82 994.44 L1356.59 994.03 L1356.36 993.316 L1356.14 992.424 L1355.91 992.747 L1355.68 992.75 L1355.45 992.526 L1355.22 992.549 L1355 992.031 L1354.77 992.601 L1354.54 992.921 L1354.31 992.888 L1354.09 992.466 L1353.86 991.431 L1353.63 992.153 L1353.4 992.137 L1353.17 991.605 L1352.95 991.151 L1352.72 989.961 L1352.49 989.73 L1352.26 989.855 L1352.04 989.999 L1351.81 989.909 L1351.58 990.896 L1351.35 991.157 L1351.12 991.269 L1350.9 990.712 L1350.67 990.955 L1350.44 990.935 L1350.21 990.944 L1349.99 990.827 L1349.76 991.136 L1349.53 992.112 L1349.3 991.378 L1349.07 990.93 L1348.85 990.055 L1348.62 989.722 L1348.39 988.882 L1348.16 988.845 L1347.94 987.909 L1347.71 988.196 L1347.48 987.436 L1347.25 987.254 L1347.02 986.581 L1346.8 986.642 L1346.57 986.525 L1346.34 985.797 L1346.11 984.988 L1345.89 984.959 L1345.66 984.739 L1345.43 984.345 L1345.2 984.533 L1344.97 985.323 L1344.75 985.914 L1344.52 987.072 L1344.29 987.196 L1344.06 987.371 L1343.84 987.604 L1343.61 987.107 L1343.38 987.421 L1343.15 987.524 L1342.92 987.357 L1342.7 988.382 L1342.47 988.308 L1342.24 988.456 L1342.01 987.519 L1341.79 988.102 L1341.56 988.709 L1341.33 988.718 L1341.1 988.057 L1340.87 987.999 L1340.65 988.463 L1340.42 988.3 L1340.19 988.109 L1339.96 988.48 L1339.74 988.171 L1339.51 988.02 L1339.28 988.563 L1339.05 988.002 L1338.82 987.155 L1338.6 987.918 L1338.37 989.103 L1338.14 989.535 L1337.91 989.368 L1337.69 988.332 L1337.46 988.088 L1337.23 987.562 L1337 987.565 L1336.77 987.131 L1336.55 987.78 L1336.32 987.697 L1336.09 987.155 L1335.86 986.53 L1335.64 986.306 L1335.41 985.495 L1335.18 985.107 L1334.95 984.486 L1334.72 985.647 L1334.5 985.369 L1334.27 984.868 L1334.04 984.096 L1333.81 983.855 L1333.59 984.336 L1333.36 984.346 L1333.13 984.432 L1332.9 984.308 L1332.67 983.348 L1332.45 984.143 L1332.22 984.336 L1331.99 983.94 L1331.76 983.439 L1331.54 983.645 L1331.31 983.639 L1331.08 984.196 L1330.85 984.711 L1330.62 985.098 L1330.4 984.442 L1330.17 986.294 L1329.94 986.062 L1329.71 985.738 L1329.49 985.565 L1329.26 985.858 L1329.03 986.484 L1328.8 986.864 L1328.58 985.567 L1328.35 986.017 L1328.12 985.665 L1327.89 984.741 L1327.66 984.802 L1327.44 984.758 L1327.21 985.179 L1326.98 985.104 L1326.75 986.063 L1326.53 986.46 L1326.3 986.159 L1326.07 986.39 L1325.84 986.122 L1325.61 986.326 L1325.39 986.644 L1325.16 986.331 L1324.93 985.504 L1324.7 985.418 L1324.48 985.613 L1324.25 984.824 L1324.02 984.726 L1323.79 984.115 L1323.56 984.041 L1323.34 983.892 L1323.11 984.302 L1322.88 984.291 L1322.65 984.554 L1322.43 984.517 L1322.2 986.048 L1321.97 986.795 L1321.74 986.466 L1321.51 987.205 L1321.29 987.291 L1321.06 987.074 L1320.83 986.94 L1320.6 986.037 L1320.38 985.428 L1320.15 985.92 L1319.92 986.318 L1319.69 986.138 L1319.46 986.148 L1319.24 985.696 L1319.01 985.821 L1318.78 985.356 L1318.55 985.273 L1318.33 985.669 L1318.1 986.039 L1317.87 985.258 L1317.64 985.082 L1317.41 985.924 L1317.19 985.124 L1316.96 984.373 L1316.73 985.292 L1316.5 985.044 L1316.28 986.055 L1316.05 985.902 L1315.82 987.524 L1315.59 987.93 L1315.36 987.656 L1315.14 987.728 L1314.91 986.941 L1314.68 986.326 L1314.45 987.452 L1314.23 986.709 L1314 985.98 L1313.77 985.605 L1313.54 985.845 L1313.31 986.283 L1313.09 985.163 L1312.86 985.973 L1312.63 987.274 L1312.4 988.32 L1312.18 989.157 L1311.95 990.104 L1311.72 989.213 L1311.49 989.478 L1311.26 990.141 L1311.04 989.468 L1310.81 988.443 L1310.58 988.001 L1310.35 987.893 L1310.13 988.146 L1309.9 987.379 L1309.67 986.895 L1309.44 986.425 L1309.21 986.024 L1308.99 986.452 L1308.76 986.875 L1308.53 987.014 L1308.3 987.719 L1308.08 987.424 L1307.85 987.162 L1307.62 986.196 L1307.39 986.338 L1307.16 986.124 L1306.94 987.152 L1306.71 985.954 L1306.48 985.686 L1306.25 986.423 L1306.03 986.148 L1305.8 986.029 L1305.57 985.741 L1305.34 985.718 L1305.11 986.044 L1304.89 985.866 L1304.66 985.88 L1304.43 986.259 L1304.2 986.489 L1303.98 985.694 L1303.75 986.062 L1303.52 985.832 L1303.29 985.626 L1303.06 985.237 L1302.84 985.27 L1302.61 985.735 L1302.38 986.346 L1302.15 985.809 L1301.93 985.16 L1301.7 985.406 L1301.47 985.402 L1301.24 985.028 L1301.01 985.391 L1300.79 985.821 L1300.56 986.139 L1300.33 985.616 L1300.1 985.871 L1299.88 986.102 L1299.65 985.27 L1299.42 984.999 L1299.19 985.018 L1298.96 984.984 L1298.74 986.254 L1298.51 985.325 L1298.28 985.715 L1298.05 987.002 L1297.83 987.382 L1297.6 987.198 L1297.37 986.79 L1297.14 987.376 L1296.91 987.23 L1296.69 987.421 L1296.46 986.241 L1296.23 986.33 L1296 985.681 L1295.78 985.333 L1295.55 984.815 L1295.32 984.458 L1295.09 984.282 L1294.86 984.628 L1294.64 984.939 L1294.41 984.786 L1294.18 985.282 L1293.95 985.474 L1293.73 985.441 L1293.5 985.682 L1293.27 985.188 L1293.04 985.227 L1292.82 986.69 L1292.59 986.599 L1292.36 986.832 L1292.13 987.05 L1291.9 986.666 L1291.68 986.62 L1291.45 986.951 L1291.22 987.426 L1290.99 988.181 L1290.77 987.871 L1290.54 988.589 L1290.31 987.859 L1290.08 987.619 L1289.85 986.612 L1289.63 986.659 L1289.4 985.815 L1289.17 985.582 L1288.94 985.897 L1288.72 985.648 L1288.49 985.49 L1288.26 985.37 L1288.03 985.103 L1287.8 985.105 L1287.58 984.761 L1287.35 984.694 L1287.12 984.881 L1286.89 984.643 L1286.67 984.652 L1286.44 984.788 L1286.21 984.407 L1285.98 985.435 L1285.75 985.749 L1285.53 985.939 L1285.3 987.021 L1285.07 986.532 L1284.84 986.939 L1284.62 987.374 L1284.39 987.325 L1284.16 987.352 L1283.93 987.236 L1283.7 986.701 L1283.48 987.294 L1283.25 987.89 L1283.02 986.992 L1282.79 987.43 L1282.57 986.918 L1282.34 985.714 L1282.11 986.937 L1281.88 987.466 L1281.65 988.263 L1281.43 987.761 L1281.2 986.589 L1280.97 987.319 L1280.74 987.709 L1280.52 986.539 L1280.29 986.247 L1280.06 985.891 L1279.83 985.743 L1279.6 985.155 L1279.38 984.842 L1279.15 985.303 L1278.92 985.45 L1278.69 985.519 L1278.47 986.008 L1278.24 987.176 L1278.01 986.601 L1277.78 987.161 L1277.55 986.752 L1277.33 987.305 L1277.1 987.642 L1276.87 987.305 L1276.64 987.684 L1276.42 988.166 L1276.19 987.451 L1275.96 987.085 L1275.73 987.677 L1275.5 988.097 L1275.28 988.082 L1275.05 988.902 L1274.82 989.947 L1274.59 990.361 L1274.37 991.141 L1274.14 991.221 L1273.91 991.624 L1273.68 992.679 L1273.45 992.647 L1273.23 992.695 L1273 992.342 L1272.77 992.238 L1272.54 992.092 L1272.32 991.682 L1272.09 992.21 L1271.86 992.577 L1271.63 993.115 L1271.4 992.616 L1271.18 993.123 L1270.95 992.844 L1270.72 993.156 L1270.49 992.894 L1270.27 993.559 L1270.04 993.642 L1269.81 993.471 L1269.58 992.895 L1269.35 993.688 L1269.13 992.927 L1268.9 991.987 L1268.67 991.974 L1268.44 991.541 L1268.22 991.414 L1267.99 991.604 L1267.76 991.774 L1267.53 991.287 L1267.3 991.128 L1267.08 992.004 L1266.85 991.601 L1266.62 992.85 L1266.39 993.136 L1266.17 993.851 L1265.94 994.797 L1265.71 995.591 L1265.48 994.381 L1265.25 994.483 L1265.03 994.163 L1264.8 994.524 L1264.57 993.358 L1264.34 993.083 L1264.12 993.188 L1263.89 993.688 L1263.66 994.022 L1263.43 995.042 L1263.2 996.214 L1262.98 997.263 L1262.75 996.874 L1262.52 998.243 L1262.29 998.126 L1262.07 998.575 L1261.84 999.446 L1261.61 999.191 L1261.38 999.043 L1261.15 999.276 L1260.93 999.112 L1260.7 998.799 L1260.47 998.042 L1260.24 997.514 L1260.02 998.411 L1259.79 997.945 L1259.56 997.666 L1259.33 996.528 L1259.1 996.262 L1258.88 996.728 L1258.65 996.036 L1258.42 996.019 L1258.19 996.693 L1257.97 996.188 L1257.74 995.237 L1257.51 994.541 L1257.28 994.277 L1257.06 993.9 L1256.83 993.567 L1256.6 993.572 L1256.37 993.435 L1256.14 993.711 L1255.92 994.178 L1255.69 994.84 L1255.46 995.627 L1255.23 996.551 L1255.01 998.217 L1254.78 999.007 L1254.55 998.804 L1254.32 998.4 L1254.09 998.292 L1253.87 997.452 L1253.64 997.458 L1253.41 996.506 L1253.18 997.016 L1252.96 996.851 L1252.73 998.686 L1252.5 998.241 L1252.27 997.51 L1252.04 998.887 L1251.82 999.17 L1251.59 999.262 L1251.36 998.874 L1251.13 998.76 L1250.91 999.21 L1250.68 998.689 L1250.45 997.964 L1250.22 997.929 L1249.99 998.249 L1249.77 997.96 L1249.54 997.62 L1249.31 996.585 L1249.08 996.126 L1248.86 995.952 L1248.63 996.349 L1248.4 997.184 L1248.17 997.88 L1247.94 999.057 L1247.72 999.096 L1247.49 998.601 L1247.26 998.725 L1247.03 998.663 L1246.81 998.812 L1246.58 998.331 L1246.35 998.129 L1246.12 997.735 L1245.89 996.537 L1245.67 996.611 L1245.44 995.583 L1245.21 995.784 L1244.98 995.822 L1244.76 996.341 L1244.53 996.789 L1244.3 996.58 L1244.07 996.465 L1243.84 995.534 L1243.62 995.667 L1243.39 995.899 L1243.16 996.661 L1242.93 997.317 L1242.71 996.842 L1242.48 996.419 L1242.25 996.516 L1242.02 995.882 L1241.79 996.548 L1241.57 995.98 L1241.34 995.925 L1241.11 995.729 L1240.88 997.076 L1240.66 996.782 L1240.43 996.026 L1240.2 996.225 L1239.97 996.749 L1239.74 997.159 L1239.52 997.157 L1239.29 997.822 L1239.06 997.882 L1238.83 997.387 L1238.61 996.726 L1238.38 995.546 L1238.15 994.429 L1237.92 994.689 L1237.69 993.82 L1237.47 993.94 L1237.24 993.269 L1237.01 993.149 L1236.78 992.556 L1236.56 993.02 L1236.33 992.139 L1236.1 993.289 L1235.87 993.684 L1235.64 994.385 L1235.42 994.631 L1235.19 994.656 L1234.96 994.249 L1234.73 993.396 L1234.51 993.324 L1234.28 992.297 L1234.05 992.36 L1233.82 992.941 L1233.59 992.341 L1233.37 991.405 L1233.14 991.624 L1232.91 991.971 L1232.68 992.191 L1232.46 992.4 L1232.23 993.25 L1232 992.959 L1231.77 992.839 L1231.54 992.979 L1231.32 992.01 L1231.09 992.107 L1230.86 991.088 L1230.63 992.134 L1230.41 993.03 L1230.18 992.746 L1229.95 993.071 L1229.72 992.989 L1229.49 992.786 L1229.27 992.69 L1229.04 992.625 L1228.81 992.214 L1228.58 992.889 L1228.36 993.296 L1228.13 993.596 L1227.9 992.639 L1227.67 992.191 L1227.44 992.521 L1227.22 991.729 L1226.99 991.808 L1226.76 991.631 L1226.53 991.828 L1226.31 991.673 L1226.08 991.528 L1225.85 991.531 L1225.62 991.644 L1225.39 991.574 L1225.17 991.193 L1224.94 990.044 L1224.71 990.697 L1224.48 990.91 L1224.26 991.655 L1224.03 991.671 L1223.8 992.496 L1223.57 992.004 L1223.34 993.005 L1223.12 993.882 L1222.89 994.538 L1222.66 994.294 L1222.43 993.972 L1222.21 993.058 L1221.98 993.403 L1221.75 994.505 L1221.52 994.74 L1221.29 995.59 L1221.07 994.669 L1220.84 994.865 L1220.61 994.169 L1220.38 993.412 L1220.16 993.008 L1219.93 993.714 L1219.7 993.107 L1219.47 993.357 L1219.25 993.677 L1219.02 994.343 L1218.79 995.12 L1218.56 995.142 L1218.33 995.317 L1218.11 995.066 L1217.88 994.873 L1217.65 995.228 L1217.42 995.989 L1217.2 997.002 L1216.97 997.683 L1216.74 996.909 L1216.51 996.527 L1216.28 996.507 L1216.06 995.673 L1215.83 996.106 L1215.6 995.697 L1215.37 994.833 L1215.15 994.648 L1214.92 993.689 L1214.69 992.939 L1214.46 994.774 L1214.23 994.506 L1214.01 994.638 L1213.78 996.453 L1213.55 997.142 L1213.32 996.136 L1213.1 995.587 L1212.87 995.141 L1212.64 994.98 L1212.41 995.512 L1212.18 995.628 L1211.96 996.638 L1211.73 995.423 L1211.5 995.205 L1211.27 995.107 L1211.05 995.101 L1210.82 996.654 L1210.59 996.874 L1210.36 997.289 L1210.13 997.859 L1209.91 997.377 L1209.68 999.227 L1209.45 998.443 L1209.22 998.002 L1209 997.831 L1208.77 996.892 L1208.54 996.313 L1208.31 996.57 L1208.08 995.828 L1207.86 995.787 L1207.63 995.009 L1207.4 994.018 L1207.17 993.696 L1206.95 994.91 L1206.72 994.155 L1206.49 993.729 L1206.26 993.57 L1206.03 993.11 L1205.81 991.786 L1205.58 991.206 L1205.35 991.386 L1205.12 990.533 L1204.9 990.358 L1204.67 991.089 L1204.44 991.832 L1204.21 992.357 L1203.98 993.037 L1203.76 992.645 L1203.53 991.93 L1203.3 991.492 L1203.07 991.876 L1202.85 991.336 L1202.62 990.444 L1202.39 989.984 L1202.16 990.084 L1201.93 990.619 L1201.71 990.476 L1201.48 989.816 L1201.25 990.325 L1201.02 991.536 L1200.8 991.665 L1200.57 990.851 L1200.34 990.528 L1200.11 990.539 L1199.88 990.432 L1199.66 989.712 L1199.43 990.158 L1199.2 990.586 L1198.97 989.913 L1198.75 990.419 L1198.52 990.779 L1198.29 990.689 L1198.06 991.6 L1197.83 991.2 L1197.61 991.618 L1197.38 990.49 L1197.15 991.254 L1196.92 990.728 L1196.7 990.046 L1196.47 990.174 L1196.24 991.289 L1196.01 991.392 L1195.78 991.757 L1195.56 991.304 L1195.33 991.804 L1195.1 991.22 L1194.87 989.883 L1194.65 990.913 L1194.42 989.934 L1194.19 989.652 L1193.96 990.089 L1193.73 990.697 L1193.51 990.815 L1193.28 991.78 L1193.05 992.553 L1192.82 992.421 L1192.6 992.033 L1192.37 992.329 L1192.14 992.01 L1191.91 990.952 L1191.68 991.189 L1191.46 991.074 L1191.23 990.551 L1191 990.401 L1190.77 992.665 L1190.55 991.83 L1190.32 991.586 L1190.09 990.769 L1189.86 989.65 L1189.63 990.065 L1189.41 990.253 L1189.18 989.858 L1188.95 990.337 L1188.72 990.907 L1188.5 990.845 L1188.27 989.982 L1188.04 989.832 L1187.81 989.424 L1187.58 989.283 L1187.36 989.148 L1187.13 987.823 L1186.9 986.594 L1186.67 986.531 L1186.45 985.703 L1186.22 986.179 L1185.99 985.965 L1185.76 986.692 L1185.53 986.147 L1185.31 986.195 L1185.08 985.97 L1184.85 985.999 L1184.62 984.924 L1184.4 984.795 L1184.17 985.098 L1183.94 985.873 L1183.71 985.77 L1183.49 985.371 L1183.26 985.018 L1183.03 985.274 L1182.8 983.91 L1182.57 983.921 L1182.35 984.087 L1182.12 983.891 L1181.89 985.013 L1181.66 985.047 L1181.44 986.064 L1181.21 986.865 L1180.98 985.84 L1180.75 985.286 L1180.52 984.391 L1180.3 983.722 L1180.07 983.294 L1179.84 983.511 L1179.61 984.036 L1179.39 984.199 L1179.16 984.316 L1178.93 983.805 L1178.7 984.673 L1178.47 984.929 L1178.25 983.799 L1178.02 984.155 L1177.79 983.495 L1177.56 983.656 L1177.34 984.004 L1177.11 984.465 L1176.88 984.88 L1176.65 985.311 L1176.42 986.354 L1176.2 987.238 L1175.97 986.582 L1175.74 985.51 L1175.51 985.316 L1175.29 984.286 L1175.06 983.867 L1174.83 985.21 L1174.6 985.347 L1174.37 985.224 L1174.15 985.436 L1173.92 984.374 L1173.69 985.16 L1173.46 984.457 L1173.24 983.902 L1173.01 983.78 L1172.78 983.072 L1172.55 982.581 L1172.32 982.882 L1172.1 982.497 L1171.87 982.167 L1171.64 982.745 L1171.41 981.616 L1171.19 981.29 L1170.96 980.824 L1170.73 980.48 L1170.5 980.878 L1170.27 981.372 L1170.05 981.919 L1169.82 981.266 L1169.59 982.095 L1169.36 981.631 L1169.14 981.197 L1168.91 981.01 L1168.68 981.263 L1168.45 980.067 L1168.22 980.382 L1168 980.257 L1167.77 979.388 L1167.54 979.674 L1167.31 981.089 L1167.09 982.042 L1166.86 981.938 L1166.63 982.537 L1166.4 984.811 L1166.17 986.113 L1165.95 985.899 L1165.72 985.693 L1165.49 985.194 L1165.26 985.359 L1165.04 984.764 L1164.81 986.927 L1164.58 987.191 L1164.35 986.957 L1164.12 986.395 L1163.9 987.573 L1163.67 987.262 L1163.44 989.407 L1163.21 989.052 L1162.99 988.139 L1162.76 988.838 L1162.53 987.946 L1162.3 988.286 L1162.07 988.223 L1161.85 987.259 L1161.62 987.126 L1161.39 987.22 L1161.16 988.699 L1160.94 988.024 L1160.71 988.297 L1160.48 988.526 L1160.25 988.097 L1160.02 987.872 L1159.8 987.488 L1159.57 988.224 L1159.34 988.206 L1159.11 988.195 L1158.89 988.586 L1158.66 988.609 L1158.43 989.673 L1158.2 989.534 L1157.97 989.391 L1157.75 989.431 L1157.52 988.9 L1157.29 988.563 L1157.06 988.769 L1156.84 989.382 L1156.61 989.871 L1156.38 989.728 L1156.15 989.106 L1155.92 990.562 L1155.7 990.273 L1155.47 991.479 L1155.24 991.032 L1155.01 992.041 L1154.79 991.269 L1154.56 991.874 L1154.33 992.107 L1154.1 991.555 L1153.87 993.247 L1153.65 993.673 L1153.42 992.621 L1153.19 992.563 L1152.96 992.176 L1152.74 991.325 L1152.51 990.631 L1152.28 989.744 L1152.05 989.152 L1151.82 988.571 L1151.6 990.717 L1151.37 991.225 L1151.14 991.024 L1150.91 992.109 L1150.69 992.507 L1150.46 993.472 L1150.23 992.385 L1150 991.085 L1149.77 992.779 L1149.55 994.084 L1149.32 994.809 L1149.09 994.296 L1148.86 993.162 L1148.64 993.204 L1148.41 992.616 L1148.18 992.446 L1147.95 994.791 L1147.73 993.875 L1147.5 993.752 L1147.27 994.316 L1147.04 994.578 L1146.81 995.025 L1146.59 995.546 L1146.36 995.916 L1146.13 995.13 L1145.9 995.444 L1145.68 997.319 L1145.45 998.449 L1145.22 999.211 L1144.99 998.346 L1144.76 997.74 L1144.54 997.656 L1144.31 998.38 L1144.08 998.305 L1143.85 999.456 L1143.63 999.063 L1143.4 999.406 L1143.17 999.14 L1142.94 998.57 L1142.71 999.424 L1142.49 998.932 L1142.26 999.572 L1142.03 1000.18 L1141.8 999.08 L1141.58 999.791 L1141.35 999.373 L1141.12 999.188 L1140.89 998.542 L1140.66 998.465 L1140.44 999.947 L1140.21 1000.91 L1139.98 1000.96 L1139.75 1000.06 L1139.53 999.845 L1139.3 1001.06 L1139.07 1000.6 L1138.84 999.153 L1138.61 997.77 L1138.39 997.893 L1138.16 997.261 L1137.93 997.24 L1137.7 996.861 L1137.48 996.772 L1137.25 996.02 L1137.02 995.009 L1136.79 995.313 L1136.56 995.274 L1136.34 994.98 L1136.11 994.902 L1135.88 994.551 L1135.65 994.119 L1135.43 994.68 L1135.2 993.529 L1134.97 992.061 L1134.74 991.684 L1134.51 992.338 L1134.29 992.71 L1134.06 993.015 L1133.83 993.383 L1133.6 993.723 L1133.38 992.941 L1133.15 992.278 L1132.92 991.648 L1132.69 990.92 L1132.46 991.196 L1132.24 992.059 L1132.01 994.759 L1131.78 994.739 L1131.55 994.951 L1131.33 994.462 L1131.1 994.555 L1130.87 994.014 L1130.64 993.8 L1130.41 993.314 L1130.19 994.115 L1129.96 994.674 L1129.73 994.073 L1129.5 992.405 L1129.28 995.561 L1129.05 994.924 L1128.82 994.136 L1128.59 993.739 L1128.36 992.579 L1128.14 992.56 L1127.91 991.899 L1127.68 993.938 L1127.45 995.059 L1127.23 996.318 L1127 996.848 L1126.77 996.199 L1126.54 995.834 L1126.31 995.597 L1126.09 994.763 L1125.86 995.087 L1125.63 994.003 L1125.4 995.231 L1125.18 994.944 L1124.95 995.03 L1124.72 995.048 L1124.49 996.031 L1124.26 996.385 L1124.04 996.859 L1123.81 997.602 L1123.58 997.412 L1123.35 997.578 L1123.13 997.802 L1122.9 997.8 L1122.67 999.277 L1122.44 997.861 L1122.21 997.161 L1121.99 997.889 L1121.76 997.015 L1121.53 995.798 L1121.3 995.346 L1121.08 995.646 L1120.85 997.938 L1120.62 996.749 L1120.39 996.009 L1120.16 996.161 L1119.94 996.678 L1119.71 996.111 L1119.48 995.221 L1119.25 996.341 L1119.03 996.513 L1118.8 997.305 L1118.57 997.956 L1118.34 996.885 L1118.11 997.846 L1117.89 999.283 L1117.66 999.543 L1117.43 1000.46 L1117.2 1000.23 L1116.98 1000.24 L1116.75 1000.01 L1116.52 999.555 L1116.29 999.155 L1116.06 999.433 L1115.84 998.604 L1115.61 997.837 L1115.38 997.32 L1115.15 996.795 L1114.93 996.727 L1114.7 995.758 L1114.47 996.03 L1114.24 997.156 L1114.01 996.934 L1113.79 996.299 L1113.56 996.333 L1113.33 995.15 L1113.1 994.351 L1112.88 994.47 L1112.65 994.745 L1112.42 994.286 L1112.19 994.589 L1111.97 993.883 L1111.74 993.644 L1111.51 994.015 L1111.28 993.134 L1111.05 991.973 L1110.83 991.659 L1110.6 991.432 L1110.37 991.205 L1110.14 991.953 L1109.92 991.978 L1109.69 991.018 L1109.46 991.023 L1109.23 991.48 L1109 990.73 L1108.78 990.141 L1108.55 989.583 L1108.32 988.606 L1108.09 987.626 L1107.87 987.954 L1107.64 988.213 L1107.41 988.153 L1107.18 988.615 L1106.95 988.137 L1106.73 987.475 L1106.5 986.893 L1106.27 986.837 L1106.04 987.956 L1105.82 988.381 L1105.59 988.512 L1105.36 988.755 L1105.13 987.406 L1104.9 987.412 L1104.68 988.261 L1104.45 989.129 L1104.22 991.093 L1103.99 990.825 L1103.77 990.778 L1103.54 990.592 L1103.31 990.714 L1103.08 990.996 L1102.85 990.525 L1102.63 991.073 L1102.4 990.508 L1102.17 990.385 L1101.94 990.815 L1101.72 990.993 L1101.49 990.924 L1101.26 991.527 L1101.03 992.028 L1100.8 991.305 L1100.58 991.381 L1100.35 991.07 L1100.12 992.448 L1099.89 992.968 L1099.67 991.751 L1099.44 992.18 L1099.21 991.125 L1098.98 992.467 L1098.75 992.998 L1098.53 992.733 L1098.3 992.515 L1098.07 992.695 L1097.84 992.909 L1097.62 993.148 L1097.39 992.827 L1097.16 992.001 L1096.93 991.141 L1096.7 990.322 L1096.48 993.156 L1096.25 995.133 L1096.02 994.271 L1095.79 994.017 L1095.57 995.074 L1095.34 993.554 L1095.11 994.126 L1094.88 994.4 L1094.65 993.398 L1094.43 993.097 L1094.2 993.617 L1093.97 993.289 L1093.74 993.914 L1093.52 993.703 L1093.29 993 L1093.06 992.935 L1092.83 991.388 L1092.6 991.24 L1092.38 991.675 L1092.15 991.633 L1091.92 992.603 L1091.69 992.807 L1091.47 991.439 L1091.24 991.701 L1091.01 991.394 L1090.78 991.756 L1090.55 991.319 L1090.33 991.172 L1090.1 990.555 L1089.87 990.511 L1089.64 990.597 L1089.42 990.442 L1089.19 991.081 L1088.96 990.052 L1088.73 989.988 L1088.5 989.893 L1088.28 989.19 L1088.05 989.629 L1087.82 988.886 L1087.59 989.281 L1087.37 988.734 L1087.14 988.794 L1086.91 988.788 L1086.68 988.792 L1086.45 987.653 L1086.23 987.873 L1086 986.741 L1085.77 985.528 L1085.54 984.623 L1085.32 985.721 L1085.09 984.697 L1084.86 983.852 L1084.63 984.316 L1084.4 984.324 L1084.18 983.686 L1083.95 985.403 L1083.72 986.031 L1083.49 986.303 L1083.27 988.362 L1083.04 988.08 L1082.81 988.781 L1082.58 989.242 L1082.35 988.765 L1082.13 989.535 L1081.9 991.022 L1081.67 990.84 L1081.44 991.308 L1081.22 991.969 L1080.99 993.346 L1080.76 993.41 L1080.53 992.723 L1080.3 992.211 L1080.08 991.587 L1079.85 990.757 L1079.62 990.192 L1079.39 989.989 L1079.17 990.556 L1078.94 990.993 L1078.71 991.236 L1078.48 991.715 L1078.25 990.922 L1078.03 991.505 L1077.8 991.478 L1077.57 991.838 L1077.34 991.396 L1077.12 990.873 L1076.89 991.064 L1076.66 991.208 L1076.43 991.184 L1076.2 990.902 L1075.98 990.143 L1075.75 989.519 L1075.52 988.641 L1075.29 988.662 L1075.07 988.348 L1074.84 988.539 L1074.61 987.292 L1074.38 989.435 L1074.16 988.123 L1073.93 988.08 L1073.7 988.279 L1073.47 986.667 L1073.24 985.687 L1073.02 986.901 L1072.79 987.141 L1072.56 986.933 L1072.33 987.064 L1072.11 986.213 L1071.88 985.797 L1071.65 985.884 L1071.42 985.545 L1071.19 985.704 L1070.97 985.353 L1070.74 986.697 L1070.51 987.047 L1070.28 986.36 L1070.06 988.009 L1069.83 989.771 L1069.6 990.42 L1069.37 990.406 L1069.14 990.695 L1068.92 990.239 L1068.69 988.921 L1068.46 988.803 L1068.23 990.561 L1068.01 991.635 L1067.78 991.334 L1067.55 990.141 L1067.32 989.317 L1067.09 990.908 L1066.87 991.841 L1066.64 992.124 L1066.41 991.293 L1066.18 991.659 L1065.96 991.951 L1065.73 991.625 L1065.5 992.035 L1065.27 991.512 L1065.04 991.57 L1064.82 990.37 L1064.59 989.4 L1064.36 989.327 L1064.13 991.012 L1063.91 990.385 L1063.68 990.074 L1063.45 990.353 L1063.22 989.632 L1062.99 989.965 L1062.77 989.027 L1062.54 988.381 L1062.31 988.126 L1062.08 988.76 L1061.86 987.175 L1061.63 986.628 L1061.4 986.785 L1061.17 987.429 L1060.94 987.293 L1060.72 986.494 L1060.49 986.275 L1060.26 986.329 L1060.03 985.984 L1059.81 984.597 L1059.58 985.064 L1059.35 984.617 L1059.12 984.803 L1058.89 984.739 L1058.67 984.591 L1058.44 983.847 L1058.21 983.084 L1057.98 982.446 L1057.76 981.926 L1057.53 981.186 L1057.3 980.608 L1057.07 980.43 L1056.84 980.275 L1056.62 980.052 L1056.39 980.97 L1056.16 981.22 L1055.93 981.07 L1055.71 981.84 L1055.48 981.767 L1055.25 980.494 L1055.02 980.25 L1054.79 980.775 L1054.57 980.477 L1054.34 981.325 L1054.11 980.339 L1053.88 980.604 L1053.66 982.923 L1053.43 983.644 L1053.2 984.098 L1052.97 985.105 L1052.74 984.648 L1052.52 985.127 L1052.29 986.379 L1052.06 985.426 L1051.83 985.582 L1051.61 985.574 L1051.38 984.443 L1051.15 984.23 L1050.92 983.846 L1050.69 983.607 L1050.47 983.261 L1050.24 983.753 L1050.01 983.38 L1049.78 982.61 L1049.56 982.724 L1049.33 982.44 L1049.1 984.843 L1048.87 984.992 L1048.64 984.394 L1048.42 984.019 L1048.19 984.694 L1047.96 984.33 L1047.73 983.016 L1047.51 983.676 L1047.28 983.469 L1047.05 982.292 L1046.82 981.589 L1046.59 980.69 L1046.37 983.186 L1046.14 984.071 L1045.91 982.722 L1045.68 982.467 L1045.46 981.989 L1045.23 982.304 L1045 981.904 L1044.77 981.246 L1044.54 980.691 L1044.32 980.568 L1044.09 980.633 L1043.86 980.785 L1043.63 981.688 L1043.41 982.551 L1043.18 981.55 L1042.95 981.401 L1042.72 981.241 L1042.49 981.804 L1042.27 980.754 L1042.04 980.388 L1041.81 979.981 L1041.58 978.988 L1041.36 978.53 L1041.13 979.316 L1040.9 978.973 L1040.67 978.347 L1040.44 979.083 L1040.22 980.378 L1039.99 980.721 L1039.76 979.435 L1039.53 980.091 L1039.31 979.061 L1039.08 979.255 L1038.85 980.311 L1038.62 982.374 L1038.4 980.923 L1038.17 981.495 L1037.94 983.204 L1037.71 982.358 L1037.48 981.382 L1037.26 982.597 L1037.03 983.511 L1036.8 984.955 L1036.57 984.754 L1036.35 986.267 L1036.12 986.032 L1035.89 986.822 L1035.66 985.872 L1035.43 987.063 L1035.21 987.211 L1034.98 985.904 L1034.75 985.823 L1034.52 985.542 L1034.3 985.9 L1034.07 984.692 L1033.84 986.199 L1033.61 985.614 L1033.38 984.782 L1033.16 985.146 L1032.93 985.263 L1032.7 985.998 L1032.47 987.46 L1032.25 987.443 L1032.02 988.338 L1031.79 988.587 L1031.56 989.728 L1031.33 989.236 L1031.11 988.438 L1030.88 987.946 L1030.65 987.923 L1030.42 987.333 L1030.2 986.998 L1029.97 987.95 L1029.74 988.051 L1029.51 987.791 L1029.28 987.411 L1029.06 987.81 L1028.83 988.955 L1028.6 987.727 L1028.37 988.626 L1028.15 988.747 L1027.92 988.695 L1027.69 988.197 L1027.46 987.67 L1027.23 987.421 L1027.01 989.112 L1026.78 988.275 L1026.55 987.96 L1026.32 988.035 L1026.1 987.086 L1025.87 985.725 L1025.64 985.488 L1025.41 985.891 L1025.18 985.773 L1024.96 986.397 L1024.73 985.299 L1024.5 985.563 L1024.27 985.701 L1024.05 986.403 L1023.82 985.034 L1023.59 984.448 L1023.36 984.285 L1023.13 984.287 L1022.91 983.604 L1022.68 983.369 L1022.45 983.402 L1022.22 983.172 L1022 982.973 L1021.77 983.573 L1021.54 984.38 L1021.31 982.891 L1021.08 983.631 L1020.86 984.664 L1020.63 985.089 L1020.4 984.723 L1020.17 984.798 L1019.95 983.81 L1019.72 982.505 L1019.49 981.646 L1019.26 980.604 L1019.03 981.508 L1018.81 980.801 L1018.58 980.116 L1018.35 980.583 L1018.12 980.913 L1017.9 982.808 L1017.67 982.752 L1017.44 982.985 L1017.21 982.875 L1016.98 982.911 L1016.76 982.924 L1016.53 982.899 L1016.3 982.684 L1016.07 982.458 L1015.85 982.728 L1015.62 982.275 L1015.39 982.102 L1015.16 981.298 L1014.93 980.763 L1014.71 980.957 L1014.48 981.266 L1014.25 979.591 L1014.02 979.599 L1013.8 978.459 L1013.57 980.754 L1013.34 981.309 L1013.11 981.905 L1012.88 981.279 L1012.66 981.372 L1012.43 980.585 L1012.2 979.51 L1011.97 978.999 L1011.75 977.897 L1011.52 977.042 L1011.29 977.023 L1011.06 976.699 L1010.83 976.119 L1010.61 975.338 L1010.38 974.33 L1010.15 974.594 L1009.92 974.701 L1009.7 975.132 L1009.47 974.642 L1009.24 975.121 L1009.01 975.506 L1008.78 974.899 L1008.56 973.521 L1008.33 973.302 L1008.1 973.291 L1007.87 972.792 L1007.65 972.246 L1007.42 972.677 L1007.19 973.137 L1006.96 973.14 L1006.73 972.614 L1006.51 972.995 L1006.28 973.214 L1006.05 973.566 L1005.82 972.908 L1005.6 973.686 L1005.37 974.4 L1005.14 974.837 L1004.91 974.34 L1004.68 975.462 L1004.46 974.431 L1004.23 972.806 L1004 973.222 L1003.77 972.684 L1003.55 972.22 L1003.32 971.197 L1003.09 970.559 L1002.86 970.313 L1002.64 971.875 L1002.41 971.611 L1002.18 970.706 L1001.95 971.161 L1001.72 970.368 L1001.5 970.313 L1001.27 968.738 L1001.04 967.826 L1000.81 967.799 L1000.59 966.766 L1000.36 966.705 L1000.13 966.167 L999.902 965.814 L999.674 967.972 L999.446 966.989 L999.218 966.644 L998.991 964.62 L998.763 963.89 L998.535 962.718 L998.307 962.405 L998.08 961.166 L997.852 959.877 L997.624 960.109 L997.396 959.978 L997.169 959.553 L996.941 959.476 L996.713 960.178 L996.485 960.643 L996.257 959.732 L996.03 960.5 L995.802 959.82 L995.574 959.548 L995.346 959.944 L995.119 959.27 L994.891 960.471 L994.663 960.859 L994.435 961.67 L994.207 963.652 L993.98 963.71 L993.752 963.787 L993.524 964.911 L993.296 964.933 L993.069 966.174 L992.841 965.907 L992.613 967.173 L992.385 966.341 L992.158 965.109 L991.93 965.828 L991.702 966.108 L991.474 966.65 L991.246 968.393 L991.019 967.983 L990.791 966.999 L990.563 968.261 L990.335 967.291 L990.108 967.406 L989.88 966.171 L989.652 965.118 L989.424 965.209 L989.197 964.199 L988.969 963.436 L988.741 964.435 L988.513 964.624 L988.285 963.844 L988.058 963.669 L987.83 962.144 L987.602 962.178 L987.374 961.616 L987.147 961.005 L986.919 962.912 L986.691 962.072 L986.463 961.895 L986.236 962.278 L986.008 963.504 L985.78 963.887 L985.552 964.239 L985.324 963.876 L985.097 962.839 L984.869 964.477 L984.641 963.268 L984.413 962.033 L984.186 962.056 L983.958 962.26 L983.73 962.472 L983.502 962.118 L983.274 961.439 L983.047 960.264 L982.819 960.069 L982.591 958.797 L982.363 959.1 L982.136 957.987 L981.908 957.541 L981.68 958.891 L981.452 958.493 L981.225 957.263 L980.997 957.839 L980.769 961.582 L980.541 961.792 L980.313 960.311 L980.086 959.725 L979.858 959.697 L979.63 959.249 L979.402 959.677 L979.175 958.755 L978.947 958.179 L978.719 958.087 L978.491 958.224 L978.264 958.152 L978.036 958.361 L977.808 958.485 L977.58 958.562 L977.352 958.984 L977.125 958.336 L976.897 960.932 L976.669 959.483 L976.441 958.959 L976.214 958.958 L975.986 958.937 L975.758 960.618 L975.53 962.76 L975.303 964.619 L975.075 964.522 L974.847 963.633 L974.619 965.544 L974.391 964.833 L974.164 965.252 L973.936 964.711 L973.708 962.455 L973.48 961.047 L973.253 960.33 L973.025 959.923 L972.797 961.892 L972.569 962.602 L972.341 962.968 L972.114 961.679 L971.886 961.269 L971.658 963.319 L971.43 962.304 L971.203 961.865 L970.975 962.383 L970.747 962.481 L970.519 962.581 L970.292 962.977 L970.064 962.487 L969.836 961.237 L969.608 961.252 L969.38 960.649 L969.153 961.445 L968.925 960.782 L968.697 961.393 L968.469 962.369 L968.242 961.059 L968.014 960.396 L967.786 961.128 L967.558 960.559 L967.331 960.077 L967.103 959.807 L966.875 960.376 L966.647 959.443 L966.419 960.08 L966.192 959.329 L965.964 958.725 L965.736 959.306 L965.508 959.187 L965.281 958.917 L965.053 958.906 L964.825 959.018 L964.597 959.008 L964.37 959.672 L964.142 958.807 L963.914 958.682 L963.686 957.993 L963.458 958.865 L963.231 958.499 L963.003 958.809 L962.775 958.794 L962.547 957.943 L962.32 957.3 L962.092 958.156 L961.864 958.715 L961.636 958.133 L961.408 958.905 L961.181 959.495 L960.953 959.93 L960.725 960.393 L960.497 959.496 L960.27 959.788 L960.042 959.443 L959.814 958.791 L959.586 958.862 L959.359 959.991 L959.131 960.076 L958.903 960.34 L958.675 959.967 L958.447 961.662 L958.22 963.115 L957.992 962.543 L957.764 961.861 L957.536 960.79 L957.309 960.29 L957.081 962.307 L956.853 962.827 L956.625 962.436 L956.398 961.851 L956.17 963.208 L955.942 962.822 L955.714 961.711 L955.486 961.265 L955.259 962.274 L955.031 962.408 L954.803 961.738 L954.575 961.732 L954.348 961.473 L954.12 962.821 L953.892 962.786 L953.664 963.627 L953.437 962.974 L953.209 963.33 L952.981 961.839 L952.753 962.003 L952.525 962.891 L952.298 963.726 L952.07 963.475 L951.842 964.031 L951.614 964.185 L951.387 963.862 L951.159 963.223 L950.931 962.948 L950.703 962.037 L950.475 962.427 L950.248 963.932 L950.02 963.686 L949.792 963.987 L949.564 965.057 L949.337 964.54 L949.109 964.699 L948.881 967.401 L948.653 968.63 L948.426 969.414 L948.198 969.858 L947.97 969.868 L947.742 969.448 L947.514 969.066 L947.287 969.899 L947.059 970.169 L946.831 969.101 L946.603 971.195 L946.376 971.283 L946.148 971.285 L945.92 970.88 L945.692 971.51 L945.465 973.368 L945.237 974.848 L945.009 974.452 L944.781 973.49 L944.553 973.739 L944.326 975.192 L944.098 973.544 L943.87 975.58 L943.642 974.867 L943.415 974.616 L943.187 976.752 L942.959 975.245 L942.731 975.437 L942.504 974.768 L942.276 974.906 L942.048 975.089 L941.82 974.782 L941.592 974.508 L941.365 974.876 L941.137 975.547 L940.909 976.366 L940.681 975.644 L940.454 976.931 L940.226 977.794 L939.998 976.736 L939.77 976.877 L939.542 976.141 L939.315 976.519 L939.087 975.806 L938.859 976.513 L938.631 975.395 L938.404 975.329 L938.176 975.964 L937.948 977.395 L937.72 978.633 L937.493 979.338 L937.265 980.169 L937.037 978.807 L936.809 978.98 L936.581 979.186 L936.354 979.486 L936.126 981.556 L935.898 981.818 L935.67 981.264 L935.443 980.594 L935.215 983.796 L934.987 983.925 L934.759 983.524 L934.532 984.551 L934.304 984.303 L934.076 984.214 L933.848 983.524 L933.62 983.105 L933.393 981.883 L933.165 981.526 L932.937 981.573 L932.709 979.888 L932.482 981.738 L932.254 982.594 L932.026 981.993 L931.798 980.9 L931.571 979.787 L931.343 980.679 L931.115 982.79 L930.887 983 L930.659 984.014 L930.432 983.062 L930.204 983.879 L929.976 982.91 L929.748 981.29 L929.521 980.859 L929.293 980.611 L929.065 980.321 L928.837 979.606 L928.609 978.618 L928.382 981.262 L928.154 982.303 L927.926 981.121 L927.698 982.505 L927.471 982.073 L927.243 980.638 L927.015 980.777 L926.787 980.379 L926.56 980.98 L926.332 982.157 L926.104 981.012 L925.876 981.714 L925.648 982.519 L925.421 981.706 L925.193 982.334 L924.965 982.944 L924.737 982.297 L924.51 983.404 L924.282 982.881 L924.054 982.605 L923.826 981.163 L923.599 980.023 L923.371 978.782 L923.143 978.963 L922.915 979.3 L922.687 978.775 L922.46 978.824 L922.232 979.029 L922.004 978.609 L921.776 978.33 L921.549 977.679 L921.321 978.711 L921.093 977.213 L920.865 976.476 L920.638 976.17 L920.41 978.353 L920.182 978.002 L919.954 976.862 L919.726 976.476 L919.499 976.478 L919.271 976.588 L919.043 975.107 L918.815 975.27 L918.588 977.719 L918.36 977.906 L918.132 978.395 L917.904 978.055 L917.676 978.521 L917.449 979.488 L917.221 979.784 L916.993 981.61 L916.765 982.538 L916.538 982.103 L916.31 981.508 L916.082 981.426 L915.854 980.549 L915.627 981.266 L915.399 981.091 L915.171 981.552 L914.943 982.953 L914.715 983.097 L914.488 983.255 L914.26 982.157 L914.032 982.803 L913.804 983.031 L913.577 981.699 L913.349 981.011 L913.121 980.839 L912.893 980.991 L912.666 981.305 L912.438 980.168 L912.21 979.941 L911.982 979.659 L911.754 978.585 L911.527 977.7 L911.299 978.341 L911.071 977.875 L910.843 977.187 L910.616 976.098 L910.388 975.968 L910.16 976.121 L909.932 977.334 L909.705 975.711 L909.477 975.87 L909.249 975.019 L909.021 975.709 L908.793 976.015 L908.566 974.723 L908.338 976.042 L908.11 978.658 L907.882 978.786 L907.655 977.96 L907.427 976.048 L907.199 974.499 L906.971 973.234 L906.743 972.868 L906.516 971.661 L906.288 971.615 L906.06 971.892 L905.832 971.948 L905.605 972.436 L905.377 972.162 L905.149 972.573 L904.921 971.81 L904.694 970.428 L904.466 970.812 L904.238 971.314 L904.01 971.392 L903.782 971.392 L903.555 971.041 L903.327 969.919 L903.099 968.448 L902.871 967.455 L902.644 967.63 L902.416 968.155 L902.188 967.267 L901.96 966.916 L901.733 966.12 L901.505 964.833 L901.277 964.306 L901.049 965.016 L900.821 963.458 L900.594 962.773 L900.366 965.138 L900.138 966.258 L899.91 965.953 L899.683 966.844 L899.455 964.78 L899.227 963.678 L898.999 964.389 L898.772 965.088 L898.544 965.614 L898.316 965.265 L898.088 964.148 L897.86 963.898 L897.633 964.851 L897.405 965.222 L897.177 967.19 L896.949 967.294 L896.722 968.744 L896.494 967.863 L896.266 966.339 L896.038 965.832 L895.81 965.951 L895.583 967.253 L895.355 966.106 L895.127 965.793 L894.899 966.943 L894.672 965.8 L894.444 965.507 L894.216 965.036 L893.988 964.967 L893.761 964.477 L893.533 964.873 L893.305 965.502 L893.077 965.973 L892.849 968.122 L892.622 967.558 L892.394 967.316 L892.166 967.999 L891.938 967.595 L891.711 967.947 L891.483 967.958 L891.255 968.283 L891.027 971.287 L890.8 971.081 L890.572 971.994 L890.344 972.211 L890.116 971.728 L889.888 971.433 L889.661 971.206 L889.433 970.374 L889.205 969.543 L888.977 970.11 L888.75 968.263 L888.522 968.648 L888.294 968.872 L888.066 968.331 L887.839 966.96 L887.611 965.645 L887.383 964.659 L887.155 963.145 L886.927 962.63 L886.7 961.646 L886.472 961.154 L886.244 962.534 L886.016 963.633 L885.789 964.164 L885.561 962.642 L885.333 960.773 L885.105 959.431 L884.877 959.263 L884.65 959.562 L884.422 958.555 L884.194 957.34 L883.966 956.396 L883.739 955.276 L883.511 956.354 L883.283 958.822 L883.055 959.196 L882.828 959.602 L882.6 960.9 L882.372 961.509 L882.144 963.042 L881.916 964.207 L881.689 965.188 L881.461 964.521 L881.233 964.302 L881.005 962.556 L880.778 961.493 L880.55 961.079 L880.322 961.469 L880.094 960.443 L879.867 959.854 L879.639 958.334 L879.411 958.31 L879.183 959.138 L878.955 959.213 L878.728 957.968 L878.5 958.618 L878.272 959.213 L878.044 958.759 L877.817 959.029 L877.589 959.427 L877.361 958.774 L877.133 958.377 L876.906 958.438 L876.678 957.563 L876.45 958.498 L876.222 959.563 L875.994 961.34 L875.767 961.605 L875.539 960.618 L875.311 959.974 L875.083 957.843 L874.856 957.744 L874.628 958.243 L874.4 958.651 L874.172 959.271 L873.944 960.737 L873.717 960.68 L873.489 960.707 L873.261 959.082 L873.033 958.785 L872.806 957.767 L872.578 959.228 L872.35 959.975 L872.122 959.657 L871.895 960.502 L871.667 958.653 L871.439 958.734 L871.211 957.08 L870.983 956.959 L870.756 955.828 L870.528 955.177 L870.3 955.404 L870.072 954.625 L869.845 954.894 L869.617 955.206 L869.389 956.592 L869.161 957.241 L868.934 959.157 L868.706 961.317 L868.478 960.456 L868.25 960.11 L868.022 960.162 L867.795 959.753 L867.567 959.751 L867.339 958.754 L867.111 958.261 L866.884 958.898 L866.656 958.098 L866.428 957.332 L866.2 957.829 L865.973 957.298 L865.745 957.568 L865.517 956.801 L865.289 957.192 L865.061 957.392 L864.834 956.498 L864.606 956.028 L864.378 957.553 L864.15 959.269 L863.923 957.452 L863.695 956.693 L863.467 957.021 L863.239 957.655 L863.011 957.762 L862.784 958.989 L862.556 959.273 L862.328 959.093 L862.1 958.462 L861.873 958.56 L861.645 957.552 L861.417 959.184 L861.189 958.749 L860.962 957.99 L860.734 958.457 L860.506 957.014 L860.278 957.018 L860.05 956.235 L859.823 957.146 L859.595 955.802 L859.367 955.968 L859.139 956.497 L858.912 957.433 L858.684 956.975 L858.456 955.159 L858.228 958.469 L858.001 958.887 L857.773 959.069 L857.545 958.257 L857.317 957.102 L857.089 955.368 L856.862 954.944 L856.634 955.021 L856.406 956.127 L856.178 956.793 L855.951 954.826 L855.723 953.623 L855.495 954.038 L855.267 956.119 L855.04 954.968 L854.812 955.192 L854.584 954.967 L854.356 956.502 L854.128 957.658 L853.901 957.729 L853.673 957.651 L853.445 956.779 L853.217 956.78 L852.99 956.185 L852.762 956.625 L852.534 955.878 L852.306 957.149 L852.078 955.43 L851.851 955.786 L851.623 954.939 L851.395 953.478 L851.167 953.279 L850.94 953.085 L850.712 953.742 L850.484 952.835 L850.256 954.208 L850.029 955.058 L849.801 955.028 L849.573 954.593 L849.345 953.691 L849.117 952.96 L848.89 952.632 L848.662 953.018 L848.434 953.84 L848.206 953.504 L847.979 952.803 L847.751 955.042 L847.523 956.655 L847.295 957.014 L847.068 956.138 L846.84 955.706 L846.612 953.954 L846.384 952.543 L846.156 952.629 L845.929 953.328 L845.701 953.67 L845.473 952.725 L845.245 951.792 L845.018 954.719 L844.79 954.603 L844.562 954.346 L844.334 953.279 L844.107 953.406 L843.879 953.367 L843.651 953.224 L843.423 953.192 L843.195 952.636 L842.968 954.589 L842.74 956.138 L842.512 957.295 L842.284 958.148 L842.057 957.486 L841.829 955.837 L841.601 954.345 L841.373 955.354 L841.145 954.88 L840.918 953.952 L840.69 955.409 L840.462 955.029 L840.234 954.605 L840.007 955.114 L839.779 956.569 L839.551 955.033 L839.323 954.667 L839.096 953.934 L838.868 952.879 L838.64 953.348 L838.412 953.816 L838.184 953.831 L837.957 952.917 L837.729 951.46 L837.501 949.734 L837.273 949.754 L837.046 949.446 L836.818 949.48 L836.59 950.935 L836.362 950.269 L836.135 950.153 L835.907 951.471 L835.679 951.27 L835.451 952.307 L835.223 951.841 L834.996 952.138 L834.768 951.945 L834.54 952.275 L834.312 951.705 L834.085 952.388 L833.857 951.914 L833.629 951.841 L833.401 954.113 L833.174 952.758 L832.946 952.553 L832.718 953.042 L832.49 952.454 L832.262 952.315 L832.035 953.685 L831.807 953.549 L831.579 954.242 L831.351 955.186 L831.124 955.342 L830.896 954.702 L830.668 954.637 L830.44 955.273 L830.212 955.567 L829.985 955.183 L829.757 954.814 L829.529 952.929 L829.301 952.069 L829.074 951.88 L828.846 950.54 L828.618 949.759 L828.39 949.735 L828.163 948.783 L827.935 947.479 L827.707 946.197 L827.479 946.018 L827.251 946.271 L827.024 946.192 L826.796 946.57 L826.568 945.8 L826.34 946.096 L826.113 945.441 L825.885 947.165 L825.657 946.486 L825.429 945.885 L825.202 947.743 L824.974 948.049 L824.746 949.203 L824.518 949.35 L824.29 949.184 L824.063 950.237 L823.835 950.154 L823.607 950.386 L823.379 950.64 L823.152 948.353 L822.924 948.335 L822.696 947.543 L822.468 947.317 L822.241 948.075 L822.013 948.334 L821.785 948.964 L821.557 949.996 L821.329 951.663 L821.102 951.947 L820.874 952.807 L820.646 953.47 L820.418 952.379 L820.191 953.048 L819.963 954.97 L819.735 953.54 L819.507 953.781 L819.279 954.756 L819.052 954.61 L818.824 953.291 L818.596 954.418 L818.368 954.419 L818.141 953.551 L817.913 952.278 L817.685 953.793 L817.457 953.875 L817.23 953.302 L817.002 953.199 L816.774 952.298 L816.546 951.913 L816.318 952.838 L816.091 952.064 L815.863 952.611 L815.635 951.48 L815.407 949.738 L815.18 949.705 L814.952 949.482 L814.724 948.141 L814.496 948.051 L814.269 947.602 L814.041 945.21 L813.813 944.197 L813.585 944.494 L813.357 945.181 L813.13 947.037 L812.902 945.627 L812.674 947.236 L812.446 945.98 L812.219 946.053 L811.991 944.873 L811.763 943.598 L811.535 942.573 L811.308 943.837 L811.08 941.999 L810.852 940.502 L810.624 939.497 L810.396 938.583 L810.169 939.814 L809.941 938.806 L809.713 937.469 L809.485 940.67 L809.258 942.41 L809.03 944.586 L808.802 944.255 L808.574 945.223 L808.346 945.013 L808.119 946.152 L807.891 946.421 L807.663 946.63 L807.435 946.973 L807.208 948.161 L806.98 949.574 L806.752 949.652 L806.524 950.786 L806.297 949.502 L806.069 951.398 L805.841 950.851 L805.613 952.825 L805.385 954.407 L805.158 954.637 L804.93 954.304 L804.702 952.146 L804.474 950.566 L804.247 951.326 L804.019 951.765 L803.791 951.197 L803.563 951.425 L803.336 951.884 L803.108 951.232 L802.88 950.744 L802.652 951.336 L802.424 950.974 L802.197 951.478 L801.969 953.233 L801.741 953.035 L801.513 951.692 L801.286 951.565 L801.058 951.241 L800.83 951.161 L800.602 950.802 L800.375 950.516 L800.147 949.982 L799.919 950.995 L799.691 951.257 L799.463 951.122 L799.236 951.421 L799.008 952.728 L798.78 952.023 L798.552 950.388 L798.325 949.918 L798.097 948.762 L797.869 947.417 L797.641 947.114 L797.413 948.326 L797.186 947.153 L796.958 946.298 L796.73 944.93 L796.502 944.709 L796.275 946.603 L796.047 945.325 L795.819 947.92 L795.591 947.821 L795.364 949.088 L795.136 949.448 L794.908 947.169 L794.68 949.903 L794.452 949.903 L794.225 949.419 L793.997 950.088 L793.769 949.204 L793.541 947.228 L793.314 946.913 L793.086 946.549 L792.858 947.792 L792.63 946.667 L792.403 946.223 L792.175 946.289 L791.947 946.735 L791.719 946.512 L791.491 946.149 L791.264 945.536 L791.036 943.858 L790.808 943.111 L790.58 940.964 L790.353 943.792 L790.125 943.52 L789.897 944.856 L789.669 944.263 L789.442 944.895 L789.214 943.293 L788.986 943.687 L788.758 943.689 L788.53 942.027 L788.303 943.381 L788.075 943.708 L787.847 943.066 L787.619 945.304 L787.392 945.009 L787.164 945.364 L786.936 943.431 L786.708 944.804 L786.481 946.502 L786.253 946.813 L786.025 949.474 L785.797 948.264 L785.569 948.238 L785.342 949.275 L785.114 949.49 L784.886 948.382 L784.658 949.427 L784.431 949.145 L784.203 947.906 L783.975 947.7 L783.747 949.081 L783.519 948.766 L783.292 950.51 L783.064 951.335 L782.836 950.47 L782.608 951.163 L782.381 952.782 L782.153 952.375 L781.925 950.325 L781.697 949.305 L781.47 951.18 L781.242 951.711 L781.014 952.305 L780.786 952.222 L780.558 954.513 L780.331 957.572 L780.103 957.129 L779.875 957.367 L779.647 956.256 L779.42 956.926 L779.192 958.406 L778.964 958.796 L778.736 961.211 L778.509 961.017 L778.281 963.276 L778.053 964.326 L777.825 967.051 L777.597 964.495 L777.37 962.962 L777.142 963.231 L776.914 963.772 L776.686 963.309 L776.459 961.36 L776.231 961.818 L776.003 960.508 L775.775 961.078 L775.548 962.259 L775.32 962.653 L775.092 961.605 L774.864 961.739 L774.636 960.965 L774.409 959.75 L774.181 957.717 L773.953 957.709 L773.725 958.038 L773.498 956.409 L773.27 954.993 L773.042 957.234 L772.814 955 L772.586 952.98 L772.359 952.749 L772.131 953.649 L771.903 954.804 L771.675 952.958 L771.448 952.524 L771.22 954.077 L770.992 954.125 L770.764 953.932 L770.537 956.206 L770.309 956.787 L770.081 955.648 L769.853 954.793 L769.625 954.594 L769.398 954.683 L769.17 952.523 L768.942 950.752 L768.714 949.722 L768.487 950.663 L768.259 951.387 L768.031 949.518 L767.803 948.396 L767.576 947.874 L767.348 948.987 L767.12 949.606 L766.892 950.19 L766.664 951.212 L766.437 949.171 L766.209 949.234 L765.981 949.299 L765.753 949.703 L765.526 949.367 L765.298 947.816 L765.07 946.717 L764.842 947.984 L764.615 948.663 L764.387 949.497 L764.159 950.598 L763.931 948.883 L763.703 951.136 L763.476 951.276 L763.248 952.554 L763.02 950.105 L762.792 950.42 L762.565 953.736 L762.337 951.924 L762.109 949.968 L761.881 951.779 L761.653 952.688 L761.426 951.869 L761.198 951.318 L760.97 950.183 L760.742 948.373 L760.515 946.633 L760.287 945.276 L760.059 944.479 L759.831 944.976 L759.604 944.33 L759.376 944.662 L759.148 943.529 L758.92 943.378 L758.692 945.51 L758.465 947.639 L758.237 947.625 L758.009 947.983 L757.781 946.584 L757.554 945.333 L757.326 947.823 L757.098 947.064 L756.87 948.002 L756.643 948.634 L756.415 948.553 L756.187 947.594 L755.959 949.229 L755.731 947.716 L755.504 947.137 L755.276 949.555 L755.048 947.969 L754.82 947.135 L754.593 947.48 L754.365 947.291 L754.137 945.781 L753.909 946.583 L753.682 946.324 L753.454 945.519 L753.226 944.078 L752.998 944.149 L752.77 946.303 L752.543 945.896 L752.315 948.017 L752.087 946.592 L751.859 946.476 L751.632 946.759 L751.404 947.177 L751.176 947.819 L750.948 948.602 L750.72 951.51 L750.493 949.505 L750.265 949.943 L750.037 950.215 L749.809 949.595 L749.582 950.867 L749.354 952.163 L749.126 952.192 L748.898 951.904 L748.671 951.37 L748.443 952.601 L748.215 952.061 L747.987 951.946 L747.759 953.727 L747.532 954.186 L747.304 953.257 L747.076 955.334 L746.848 954.687 L746.621 956.165 L746.393 955.441 L746.165 955.077 L745.937 957.514 L745.71 957.469 L745.482 957.891 L745.254 957.468 L745.026 958.879 L744.798 959.237 L744.571 957.276 L744.343 956.409 L744.115 955.727 L743.887 957.557 L743.66 955.577 L743.432 956.627 L743.204 955.193 L742.976 953.929 L742.749 953.72 L742.521 954.904 L742.293 953.258 L742.065 952.196 L741.837 950.372 L741.61 948.493 L741.382 947.809 L741.154 948.837 L740.926 948.924 L740.699 951.299 L740.471 951.262 L740.243 950.861 L740.015 953.721 L739.787 952.816 L739.56 956.074 L739.332 954.313 L739.104 953.575 L738.876 954.286 L738.649 952.498 L738.421 951.916 L738.193 949.754 L737.965 951.159 L737.738 951.74 L737.51 953.769 L737.282 952.518 L737.054 951.347 L736.826 951.346 L736.599 950.743 L736.371 949.05 L736.143 949.136 L735.915 950.8 L735.688 951.05 L735.46 952.483 L735.232 953.798 L735.004 955.016 L734.777 956.614 L734.549 956.885 L734.321 956.912 L734.093 955.541 L733.865 955.012 L733.638 956.972 L733.41 954.389 L733.182 952.538 L732.954 953.035 L732.727 950.636 L732.499 948.773 L732.271 949.769 L732.043 947.084 L731.816 946.762 L731.588 945.905 L731.36 945.847 L731.132 947.063 L730.904 947.089 L730.677 947.774 L730.449 949.467 L730.221 949.046 L729.993 948.557 L729.766 948.618 L729.538 947.634 L729.31 947.513 L729.082 945.566 L728.854 944.848 L728.627 946.042 L728.399 948.476 L728.171 946.694 L727.943 946.161 L727.716 946.738 L727.488 947.413 L727.26 947.157 L727.032 946.966 L726.805 945.424 L726.577 945.349 L726.349 946.987 L726.121 945.466 L725.893 943.725 L725.666 941.409 L725.438 941.69 L725.21 940.14 L724.982 939.076 L724.755 940.827 L724.527 940.009 L724.299 938.636 L724.071 937.476 L723.844 936.823 L723.616 934.602 L723.388 933.959 L723.16 931.631 L722.932 930.477 L722.705 929.786 L722.477 928.674 L722.249 928.55 L722.021 927.439 L721.794 927.156 L721.566 929.346 L721.338 933.362 L721.11 934.378 L720.883 931.701 L720.655 930.01 L720.427 929.981 L720.199 929.605 L719.971 929.259 L719.744 928.105 L719.516 929.242 L719.288 931.409 L719.06 936.457 L718.833 936.991 L718.605 939.902 L718.377 939.409 L718.149 939.492 L717.921 939.894 L717.694 940.885 L717.466 939.692 L717.238 940.277 L717.01 940.59 L716.783 939.772 L716.555 939.396 L716.327 940.634 L716.099 940.807 L715.872 938.603 L715.644 937.026 L715.416 935.818 L715.188 935.659 L714.96 936.343 L714.733 938.481 L714.505 938.564 L714.277 939.515 L714.049 939.047 L713.822 937.172 L713.594 936.754 L713.366 935.888 L713.138 934.547 L712.911 934.333 L712.683 934.391 L712.455 931.65 L712.227 930.941 L711.999 929.757 L711.772 931.65 L711.544 929.586 L711.316 929.1 L711.088 927.15 L710.861 927.645 L710.633 926.614 L710.405 926.747 L710.177 924.481 L709.95 923.615 L709.722 922.968 L709.494 923.433 L709.266 923.42 L709.038 925.983 L708.811 924.828 L708.583 924.659 L708.355 924.485 L708.127 924.036 L707.9 923.041 L707.672 923.74 L707.444 925.269 L707.216 926.888 L706.988 925.626 L706.761 926.126 L706.533 927.138 L706.305 927.639 L706.077 927.503 L705.85 927.199 L705.622 926.277 L705.394 924.162 L705.166 922.881 L704.939 922.62 L704.711 923.113 L704.483 922.893 L704.255 922.134 L704.027 924.128 L703.8 924.169 L703.572 923.908 L703.344 924.348 L703.116 924.618 L702.889 923.127 L702.661 922.304 L702.433 922.711 L702.205 922.181 L701.978 924.941 L701.75 924.452 L701.522 922.793 L701.294 925.362 L701.066 924.623 L700.839 925.344 L700.611 927.552 L700.383 925.189 L700.155 923.598 L699.928 921.805 L699.7 920.985 L699.472 922.918 L699.244 922.162 L699.017 921.231 L698.789 920.725 L698.561 922.45 L698.333 922.261 L698.105 923.817 L697.878 922.223 L697.65 924.544 L697.422 924.148 L697.194 927.664 L696.967 929.434 L696.739 930.021 L696.511 929.37 L696.283 929.944 L696.055 930.006 L695.828 930.126 L695.6 929.085 L695.372 932.172 L695.144 930.325 L694.917 934.86 L694.689 935.22 L694.461 934.998 L694.233 935.941 L694.006 935.987 L693.778 936.467 L693.55 936.524 L693.322 935.414 L693.094 933.496 L692.867 933.189 L692.639 933.481 L692.411 933.843 L692.183 934.059 L691.956 933.886 L691.728 933.133 L691.5 931.631 L691.272 933.205 L691.045 933.611 L690.817 933.56 L690.589 933.646 L690.361 934.381 L690.133 933.226 L689.906 934.303 L689.678 933.385 L689.45 934.641 L689.222 934.037 L688.995 933.724 L688.767 932.402 L688.539 934.529 L688.311 933.495 L688.084 935.005 L687.856 936.189 L687.628 936.707 L687.4 936.304 L687.172 935.828 L686.945 936.15 L686.717 936.361 L686.489 934.072 L686.261 936.599 L686.034 934.911 L685.806 933.589 L685.578 932.528 L685.35 931.993 L685.122 931.039 L684.895 932.718 L684.667 931.046 L684.439 931.709 L684.211 930.586 L683.984 929.984 L683.756 928.61 L683.528 926.461 L683.3 929.745 L683.073 927.401 L682.845 926.929 L682.617 925.079 L682.389 926.092 L682.161 926.041 L681.934 928.226 L681.706 929.597 L681.478 929.804 L681.25 930.464 L681.023 928.746 L680.795 934.016 L680.567 932.035 L680.339 932.715 L680.112 932.081 L679.884 932.298 L679.656 931.656 L679.428 932.218 L679.2 930.778 L678.973 928.634 L678.745 929.457 L678.517 929.367 L678.289 928.423 L678.062 926.991 L677.834 925.821 L677.606 926.475 L677.378 924.851 L677.151 924.343 L676.923 924.078 L676.695 924.655 L676.467 926.971 L676.239 926.439 L676.012 929.709 L675.784 929.946 L675.556 928.97 L675.328 929.635 L675.101 927.084 L674.873 924.989 L674.645 927.27 L674.417 925.238 L674.189 923.39 L673.962 925.834 L673.734 927.973 L673.506 927.863 L673.278 926.951 L673.051 928.464 L672.823 928.634 L672.595 927.608 L672.367 925.337 L672.14 926.32 L671.912 925.593 L671.684 923.029 L671.456 921.271 L671.228 923.384 L671.001 924.251 L670.773 924.304 L670.545 925.055 L670.317 927.585 L670.09 927.45 L669.862 928.069 L669.634 927.618 L669.406 928.333 L669.179 929.36 L668.951 927.635 L668.723 929.218 L668.495 930.004 L668.267 929.296 L668.04 928.956 L667.812 930.031 L667.584 930.898 L667.356 932.352 L667.129 934.938 L666.901 933.938 L666.673 935.625 L666.445 936.92 L666.218 939.005 L665.99 941.823 L665.762 942.476 L665.534 943.274 L665.306 943.445 L665.079 943.188 L664.851 941.976 L664.623 940.151 L664.395 938.662 L664.168 939.958 L663.94 938.44 L663.712 937.693 L663.484 939.074 L663.256 940.118 L663.029 939.776 L662.801 940.405 L662.573 941.573 L662.345 944.343 L662.118 942.294 L661.89 946.196 L661.662 946.883 L661.434 946.743 L661.207 945.049 L660.979 947.111 L660.751 943.332 L660.523 943.836 L660.295 943.235 L660.068 941.938 L659.84 940.683 L659.612 941.438 L659.384 941.2 L659.157 939.737 L658.929 937.398 L658.701 937.126 L658.473 938.112 L658.246 937.788 L658.018 940.9 L657.79 940.658 L657.562 939.036 L657.334 940.501 L657.107 938.602 L656.879 937.179 L656.651 936.152 L656.423 934.444 L656.196 933.539 L655.968 933 L655.74 934.721 L655.512 932.804 L655.285 930.551 L655.057 930.583 L654.829 928.007 L654.601 926.548 L654.373 925.699 L654.146 923.538 L653.918 922.681 L653.69 922.267 L653.462 920.503 L653.235 923.064 L653.007 924.699 L652.779 924.185 L652.551 929.038 L652.323 927.553 L652.096 927.293 L651.868 926.471 L651.64 925.656 L651.412 923.772 L651.185 923.341 L650.957 923.454 L650.729 922.378 L650.501 922.209 L650.274 921.891 L650.046 920.778 L649.818 921.133 L649.59 919.815 L649.362 918.004 L649.135 915.551 L648.907 916.318 L648.679 918.366 L648.451 917.835 L648.224 916.321 L647.996 918.852 L647.768 917.958 L647.54 919.054 L647.313 918.693 L647.085 917.379 L646.857 920.75 L646.629 920.454 L646.401 925.669 L646.174 924.225 L645.946 922.797 L645.718 922.835 L645.49 923.423 L645.263 922.385 L645.035 922.23 L644.807 922.256 L644.579 921.277 L644.352 919.705 L644.124 920.363 L643.896 919.018 L643.668 917.219 L643.44 918.456 L643.213 919.357 L642.985 916.624 L642.757 921.124 L642.529 920.992 L642.302 919.55 L642.074 917.186 L641.846 915.853 L641.618 917.78 L641.39 915.612 L641.163 914.58 L640.935 913.817 L640.707 911.696 L640.479 909.321 L640.252 910.317 L640.024 911.933 L639.796 913.489 L639.568 915.039 L639.341 916.052 L639.113 915.706 L638.885 915.104 L638.657 913.656 L638.429 912.797 L638.202 910.635 L637.974 911.354 L637.746 910.764 L637.518 910.769 L637.291 909.944 L637.063 907.77 L636.835 905.006 L636.607 904.026 L636.38 905.778 L636.152 909.468 L635.924 909.37 L635.696 908.445 L635.468 908.753 L635.241 910.089 L635.013 908.793 L634.785 908.679 L634.557 909.769 L634.33 906.919 L634.102 905.843 L633.874 905.939 L633.646 906.294 L633.419 904.685 L633.191 903.693 L632.963 903.991 L632.735 903.264 L632.507 906.028 L632.28 907.802 L632.052 903.902 L631.824 901.912 L631.596 901.2 L631.369 900.896 L631.141 902.77 L630.913 901.738 L630.685 901.28 L630.457 901.258 L630.23 902.279 L630.002 900.853 L629.774 898.694 L629.546 896.089 L629.319 894.993 L629.091 891.351 L628.863 887.342 L628.635 885.4 L628.408 884.901 L628.18 884.665 L627.952 881.485 L627.724 881.614 L627.496 882.89 L627.269 883.903 L627.041 882.395 L626.813 880.863 L626.585 884.143 L626.358 884.359 L626.13 884.841 L625.902 883.582 L625.674 882.549 L625.447 880.46 L625.219 878.8 L624.991 882.054 L624.763 883.027 L624.535 882.965 L624.308 881.984 L624.08 882.31 L623.852 879.215 L623.624 880.232 L623.397 877.816 L623.169 877.442 L622.941 876.071 L622.713 877.647 L622.486 875.205 L622.258 876.138 L622.03 873.59 L621.802 874.342 L621.574 873.321 L621.347 873.497 L621.119 878.988 L620.891 880.049 L620.663 879.961 L620.436 884.065 L620.208 886.551 L619.98 887.713 L619.752 885.836 L619.524 885.407 L619.297 884.755 L619.069 887.843 L618.841 889.031 L618.613 891.5 L618.386 893.106 L618.158 891.746 L617.93 892.277 L617.702 891.909 L617.475 891.341 L617.247 892.908 L617.019 890.381 L616.791 892.572 L616.563 889.846 L616.336 890.626 L616.108 890.789 L615.88 893.488 L615.652 892.968 L615.425 893.5 L615.197 895.099 L614.969 893.82 L614.741 892.729 L614.514 892.026 L614.286 891.391 L614.058 893.673 L613.83 897.779 L613.602 899.427 L613.375 899.627 L613.147 899.722 L612.919 901.793 L612.691 904.225 L612.464 901.178 L612.236 900.519 L612.008 899.971 L611.78 901.359 L611.553 900.187 L611.325 902.537 L611.097 899.156 L610.869 897.446 L610.641 896.091 L610.414 898.374 L610.186 896.858 L609.958 898.181 L609.73 898.82 L609.503 901.948 L609.275 902.785 L609.047 903.819 L608.819 906.718 L608.591 904.826 L608.364 902.262 L608.136 899.972 L607.908 900.77 L607.68 903.45 L607.453 906.036 L607.225 902.121 L606.997 902.452 L606.769 906.309 L606.542 905.837 L606.314 903.719 L606.086 903.632 L605.858 902.943 L605.63 904.595 L605.403 904.036 L605.175 902.762 L604.947 902.104 L604.719 900.405 L604.492 898.799 L604.264 899.488 L604.036 901.013 L603.808 902.477 L603.581 902.189 L603.353 902.873 L603.125 901.571 L602.897 900.409 L602.669 897.004 L602.442 896.109 L602.214 892.762 L601.986 893.576 L601.758 892.398 L601.531 890.299 L601.303 891.323 L601.075 890.757 L600.847 890.596 L600.62 894.553 L600.392 895.187 L600.164 895.861 L599.936 894.304 L599.708 892.571 L599.481 892.852 L599.253 891.856 L599.025 891.587 L598.797 893.27 L598.57 895.388 L598.342 892.503 L598.114 894.751 L597.886 893.284 L597.658 891.352 L597.431 889.037 L597.203 888.248 L596.975 887.419 L596.747 888.78 L596.52 890.102 L596.292 889.606 L596.064 887.618 L595.836 887.621 L595.609 887.374 L595.381 886.141 L595.153 885.878 L594.925 888.542 L594.697 891.833 L594.47 890.332 L594.242 889.883 L594.014 888.984 L593.786 886.432 L593.559 884.676 L593.331 885.799 L593.103 886.763 L592.875 887.523 L592.648 886.987 L592.42 885.284 L592.192 885.045 L591.964 889.204 L591.736 887.729 L591.509 889.108 L591.281 890.064 L591.053 889.928 L590.825 889.147 L590.598 889.531 L590.37 888.27 L590.142 885.033 L589.914 890.955 L589.687 892.884 L589.459 890.69 L589.231 888.125 L589.003 888.596 L588.775 886.749 L588.548 885.018 L588.32 884.21 L588.092 886.236 L587.864 888.794 L587.637 888.97 L587.409 889.848 L587.181 890.878 L586.953 887.867 L586.725 885.175 L586.498 881.593 L586.27 880.199 L586.042 877.708 L585.814 879.978 L585.587 879.867 L585.359 881.386 L585.131 879.565 L584.903 878.014 L584.676 877.904 L584.448 877.792 L584.22 879.085 L583.992 879.87 L583.764 880.168 L583.537 878.759 L583.309 878.305 L583.081 877.683 L582.853 875.407 L582.626 874.435 L582.398 870.978 L582.17 870.648 L581.942 870.957 L581.715 871.692 L581.487 872.962 L581.259 874.887 L581.031 872.317 L580.803 872.979 L580.576 871.063 L580.348 869.505 L580.12 869.358 L579.892 872.197 L579.665 872.692 L579.437 875.126 L579.209 871.526 L578.981 867.646 L578.754 869.876 L578.526 868.317 L578.298 869.095 L578.07 867.236 L577.842 868.315 L577.615 872.223 L577.387 872.625 L577.159 872.949 L576.931 871.525 L576.704 872.176 L576.476 871 L576.248 868.353 L576.02 866.904 L575.792 865.104 L575.565 864.541 L575.337 867.134 L575.109 867.991 L574.881 869.191 L574.654 871.628 L574.426 871.32 L574.198 871.219 L573.97 868.814 L573.743 867.069 L573.515 863.833 L573.287 862.119 L573.059 861.11 L572.831 859.599 L572.604 858.152 L572.376 857.321 L572.148 861.576 L571.92 864.098 L571.693 863.567 L571.465 866.944 L571.237 867.649 L571.009 866.961 L570.782 864.654 L570.554 865.355 L570.326 867.074 L570.098 867.534 L569.87 868.334 L569.643 869.277 L569.415 871.676 L569.187 870.609 L568.959 877.628 L568.732 879.416 L568.504 882.179 L568.276 881.372 L568.048 881.31 L567.821 880.293 L567.593 879.982 L567.365 879.253 L567.137 877.234 L566.909 878.825 L566.682 879.936 L566.454 880.399 L566.226 882.859 L565.998 885.995 L565.771 887.027 L565.543 884.612 L565.315 886.986 L565.087 887.677 L564.859 883.843 L564.632 887.046 L564.404 886.917 L564.176 891.353 L563.948 889.024 L563.721 889.187 L563.493 888.898 L563.265 887.456 L563.037 891.919 L562.81 887.848 L562.582 890.757 L562.354 888.118 L562.126 888.309 L561.898 889.459 L561.671 885.792 L561.443 882.381 L561.215 882.94 L560.987 885.654 L560.76 888.245 L560.532 889.137 L560.304 889.838 L560.076 886.702 L559.849 885.444 L559.621 888.485 L559.393 885.808 L559.165 884.026 L558.937 882.124 L558.71 882.876 L558.482 884.529 L558.254 885.719 L558.026 881.245 L557.799 878.956 L557.571 880.903 L557.343 881.26 L557.115 880.493 L556.888 879.226 L556.66 886.825 L556.432 893.98 L556.204 897.407 L555.976 898.429 L555.749 900.344 L555.521 900.866 L555.293 901.074 L555.065 903.845 L554.838 903.288 L554.61 902.078 L554.382 901.438 L554.154 900.363 L553.926 895.634 L553.699 894.758 L553.471 896.924 L553.243 896.139 L553.015 894.083 L552.788 894.957 L552.56 892.871 L552.332 889.588 L552.104 887.208 L551.877 888.474 L551.649 885.646 L551.421 887.36 L551.193 885.643 L550.965 885.569 L550.738 884.005 L550.51 889.466 L550.282 886.253 L550.054 883.55 L549.827 881.638 L549.599 882.149 L549.371 883.202 L549.143 880.938 L548.916 880.507 L548.688 883.681 L548.46 883.28 L548.232 882.987 L548.004 879.462 L547.777 880.26 L547.549 880.221 L547.321 881.513 L547.093 877.113 L546.866 874.377 L546.638 878.673 L546.41 882.692 L546.182 885.346 L545.955 886.152 L545.727 885.128 L545.499 891.971 L545.271 895.465 L545.043 892.458 L544.816 893.796 L544.588 894.861 L544.36 892.928 L544.132 893.44 L543.905 891.346 L543.677 890.09 L543.449 887.795 L543.221 897.473 L542.993 894.966 L542.766 896.559 L542.538 895.328 L542.31 892.965 L542.082 894.629 L541.855 896.478 L541.627 897.639 L541.399 898.141 L541.171 901.936 L540.944 903.244 L540.716 902.02 L540.488 901.034 L540.26 896.276 L540.032 897.26 L539.805 900.441 L539.577 899.032 L539.349 897.576 L539.121 895.226 L538.894 897.133 L538.666 891.551 L538.438 893.976 L538.21 891.701 L537.983 893.594 L537.755 899.865 L537.527 896.27 L537.299 894.915 L537.071 893.966 L536.844 889.129 L536.616 888.636 L536.388 886.999 L536.16 888.01 L535.933 885.675 L535.705 883.089 L535.477 885.466 L535.249 884.994 L535.022 883.903 L534.794 883.324 L534.566 882.04 L534.338 881.948 L534.11 881.464 L533.883 880.845 L533.655 879.186 L533.427 880.209 L533.199 879.079 L532.972 877.632 L532.744 874.784 L532.516 869.403 L532.288 866.826 L532.06 866.437 L531.833 863.639 L531.605 862.028 L531.377 860.778 L531.149 857.416 L530.922 855.523 L530.694 851.899 L530.466 846.948 L530.238 848.037 L530.011 845.263 L529.783 842.794 L529.555 841.615 L529.327 842.998 L529.099 840.373 L528.872 841.666 L528.644 842.679 L528.416 844.134 L528.188 841.594 L527.961 838.126 L527.733 837.103 L527.505 834.101 L527.277 834.547 L527.05 831.398 L526.822 836.25 L526.594 840.665 L526.366 841.471 L526.138 844.238 L525.911 843.025 L525.683 843.776 L525.455 842.438 L525.227 839.229 L525 837.998 L524.772 838.211 L524.544 836.318 L524.316 834.271 L524.089 832.479 L523.861 833.075 L523.633 832.557 L523.405 837.649 L523.177 837.395 L522.95 838.167 L522.722 843.844 L522.494 843.678 L522.266 841.319 L522.039 844.683 L521.811 845.194 L521.583 844.424 L521.355 849.393 L521.127 844.494 L520.9 844.229 L520.672 843.548 L520.444 844.84 L520.216 845.774 L519.989 844.949 L519.761 841.913 L519.533 839.938 L519.305 842.237 L519.078 842.124 L518.85 842.187 L518.622 841.964 L518.394 839 L518.166 837.756 L517.939 836.242 L517.711 835.909 L517.483 834.188 L517.255 845.691 L517.028 843.815 L516.8 842.261 L516.572 841.834 L516.344 840.525 L516.117 837.218 L515.889 836.533 L515.661 837.115 L515.433 835.168 L515.205 834.807 L514.978 833.497 L514.75 831.19 L514.522 834.045 L514.294 830.47 L514.067 833.532 L513.839 833.05 L513.611 834.246 L513.383 833.318 L513.156 833.82 L512.928 836.417 L512.7 835.003 L512.472 831.631 L512.244 827.263 L512.017 826.024 L511.789 825.51 L511.561 825.599 L511.333 824.329 L511.106 825.17 L510.878 824.737 L510.65 821.493 L510.422 823.86 L510.194 819.261 L509.967 818.67 L509.739 819.053 L509.511 824.394 L509.283 823.323 L509.056 823.951 L508.828 824.124 L508.6 822.824 L508.372 828.238 L508.145 829.428 L507.917 835.537 L507.689 834.194 L507.461 834.341 L507.233 832.796 L507.006 828.888 L506.778 828.734 L506.55 828.227 L506.322 826.896 L506.095 833.948 L505.867 834.574 L505.639 832.072 L505.411 832.036 L505.184 834.426 L504.956 832.256 L504.728 831.173 L504.5 831.443 L504.272 828.923 L504.045 831.055 L503.817 832.559 L503.589 827.312 L503.361 826.021 L503.134 823.49 L502.906 825.079 L502.678 828.611 L502.45 829.675 L502.223 827.009 L501.995 824.053 L501.767 824.376 L501.539 824.193 L501.311 822.165 L501.084 821.706 L500.856 823.756 L500.628 825.901 L500.4 825.774 L500.173 827.017 L499.945 828.451 L499.717 827.689 L499.489 825.537 L499.261 822.82 L499.034 822.77 L498.806 820.65 L498.578 823.421 L498.35 822.841 L498.123 823.383 L497.895 822.363 L497.667 819.454 L497.439 817.287 L497.212 816.583 L496.984 815.841 L496.756 818.506 L496.528 817.662 L496.3 819.693 L496.073 817 L495.845 815.335 L495.617 814.638 L495.389 815.25 L495.162 816.442 L494.934 819.18 L494.706 818.716 L494.478 816.953 L494.251 817.086 L494.023 814.431 L493.795 811.585 L493.567 810.88 L493.339 813.429 L493.112 817.446 L492.884 819.473 L492.656 825.254 L492.428 822.01 L492.201 821.382 L491.973 824.427 L491.745 824.259 L491.517 830.338 L491.29 829.408 L491.062 832.15 L490.834 830.605 L490.606 826.766 L490.378 829.597 L490.151 828.889 L489.923 830.048 L489.695 825.329 L489.467 828.996 L489.24 829.683 L489.012 829.227 L488.784 826.09 L488.556 824.229 L488.328 821.884 L488.101 819.949 L487.873 816.68 L487.645 816.771 L487.417 816.761 L487.19 818.764 L486.962 821.728 L486.734 821.681 L486.506 819.985 L486.279 815.841 L486.051 813.328 L485.823 812.923 L485.595 810.091 L485.367 809.624 L485.14 813.423 L484.912 810.875 L484.684 810.285 L484.456 807.528 L484.229 807.36 L484.001 801.776 L483.773 804.639 L483.545 804.518 L483.318 803.138 L483.09 800.182 L482.862 797.987 L482.634 807.274 L482.406 803.22 L482.179 798.89 L481.951 796.417 L481.723 804.275 L481.495 804.255 L481.268 801.135 L481.04 801.486 L480.812 799.221 L480.584 796.178 L480.357 799.021 L480.129 798.642 L479.901 794.209 L479.673 794.48 L479.445 796.826 L479.218 797.53 L478.99 794.995 L478.762 789.271 L478.534 782.469 L478.307 782.366 L478.079 787.884 L477.851 785.93 L477.623 789.769 L477.395 790.911 L477.168 790.986 L476.94 789.526 L476.712 789.638 L476.484 787.569 L476.257 785.298 L476.029 781.764 L475.801 781.288 L475.573 780.283 L475.346 784.607 L475.118 786.51 L474.89 787.084 L474.662 786.502 L474.434 787.348 L474.207 785.062 L473.979 782.185 L473.751 794.542 L473.523 801.812 L473.296 809.363 L473.068 805.772 L472.84 804.03 L472.612 803.452 L472.385 800.843 L472.157 799.424 L471.929 794.22 L471.701 803.84 L471.473 798.311 L471.246 792.526 L471.018 796.459 L470.79 797.144 L470.562 793.169 L470.335 791.055 L470.107 796.157 L469.879 798.32 L469.651 799.996 L469.424 801.212 L469.196 796.755 L468.968 798.83 L468.74 794.85 L468.512 800.602 L468.285 796.966 L468.057 797.994 L467.829 800.372 L467.601 800.883 L467.374 806.313 L467.146 808.206 L466.918 807.063 L466.69 814.088 L466.462 821.04 L466.235 816.817 L466.007 819.184 L465.779 822.974 L465.551 821.38 L465.324 825.124 L465.096 826.111 L464.868 824.865 L464.64 824.104 L464.413 822.274 L464.185 818.073 L463.957 821.647 L463.729 821.206 L463.501 821.973 L463.274 825.664 L463.046 824.022 L462.818 825.659 L462.59 831.97 L462.363 826.273 L462.135 825.77 L461.907 832.319 L461.679 838.321 L461.452 841.964 L461.224 840.747 L460.996 834.456 L460.768 839.011 L460.54 838.524 L460.313 839.932 L460.085 843.05 L459.857 839.893 L459.629 836.981 L459.402 835.833 L459.174 833.111 L458.946 828.575 L458.718 824.568 L458.491 828.954 L458.263 828.329 L458.035 832.222 L457.807 837.974 L457.579 832.1 L457.352 831.235 L457.124 832.047 L456.896 829.222 L456.668 829.981 L456.441 831.136 L456.213 827.926 L455.985 824.528 L455.757 823.916 L455.53 821.243 L455.302 822.025 L455.074 823.18 L454.846 818.111 L454.618 815.647 L454.391 817.673 L454.163 822.403 L453.935 821.976 L453.707 818.869 L453.48 830.936 L453.252 838.08 L453.024 834.14 L452.796 832.483 L452.568 827.378 L452.341 827.531 L452.113 828.023 L451.885 830.129 L451.657 826.607 L451.43 821.956 L451.202 824.536 L450.974 822.293 L450.746 824.29 L450.519 827.793 L450.291 827.545 L450.063 825.447 L449.835 828.09 L449.607 831.197 L449.38 827.164 L449.152 827.431 L448.924 829.705 L448.696 831.949 L448.469 826.826 L448.241 829.127 L448.013 823.794 L447.785 818.882 L447.558 824.179 L447.33 820.883 L447.102 824.547 L446.874 825.097 L446.646 825.978 L446.419 824.486 L446.191 824.115 L445.963 820.883 L445.735 820.142 L445.508 816.742 L445.28 813.018 L445.052 812.757 L444.824 815.171 L444.597 810.665 L444.369 807.022 L444.141 812.856 L443.913 812.691 L443.685 810.155 L443.458 808.714 L443.23 807.719 L443.002 809.964 L442.774 808.526 L442.547 816.598 L442.319 816.875 L442.091 815.969 L441.863 816.943 L441.635 810.928 L441.408 810.58 L441.18 820.583 L440.952 816.026 L440.724 812.205 L440.497 816.206 L440.269 811.595 L440.041 810.604 L439.813 811.96 L439.586 813.157 L439.358 812.419 L439.13 803.636 L438.902 800.996 L438.674 801.306 L438.447 798.2 L438.219 800.476 L437.991 795.84 L437.763 798.989 L437.536 797.099 L437.308 795.358 L437.08 802.672 L436.852 809.145 L436.625 810.88 L436.397 814.195 L436.169 818.925 L435.941 819.953 L435.713 816.782 L435.486 814.803 L435.258 826.944 L435.03 832.537 L434.802 832.225 L434.575 829.388 L434.347 833.112 L434.119 837.831 L433.891 841.015 L433.664 837.331 L433.436 832.252 L433.208 834.678 L432.98 833.504 L432.752 842.443 L432.525 840.475 L432.297 838.741 L432.069 835.071 L431.841 848.735 L431.614 843.054 L431.386 837.573 L431.158 835.895 L430.93 835.962 L430.702 840.998 L430.475 839.019 L430.247 847.404 L430.019 846.604 L429.791 842.34 L429.564 845.012 L429.336 841.092 L429.108 848.573 L428.88 847.801 L428.653 845.867 L428.425 847.536 L428.197 846.231 L427.969 852.495 L427.741 850.047 L427.514 851.248 L427.286 845.734 L427.058 847.245 L426.83 840.986 L426.603 840.287 L426.375 837.341 L426.147 843.056 L425.919 846.031 L425.692 850.963 L425.464 868.627 L425.236 863.717 L425.008 875.76 L424.78 871.032 L424.553 882.555 L424.325 885.965 L424.097 880.401 L423.869 887.633 L423.642 888.449 L423.414 881.973 L423.186 889.562 L422.958 892.65 L422.731 886.898 L422.503 890.724 L422.275 889.747 L422.047 884.671 L421.819 878.378 L421.592 878.974 L421.364 889.445 L421.136 891.836 L420.908 900.472 L420.681 897.214 L420.453 895.181 L420.225 891.83 L419.997 889.366 L419.769 890.58 L419.542 883.1 L419.314 888.036 L419.086 887.407 L418.858 888.149 L418.631 891.666 L418.403 891.265 L418.175 890.529 L417.947 886.051 L417.72 880.296 L417.492 871.583 L417.264 864.652 L417.036 867.871 L416.808 869.691 L416.581 863.56 L416.353 855.293 L416.125 853.98 L415.897 857.886 L415.67 863.853 L415.442 868.667 L415.214 863.4 L414.986 863.842 L414.759 865.796 L414.531 875.034 L414.303 873.05 L414.075 877.774 L413.847 880.548 L413.62 876.552 L413.392 876.267 L413.164 879.817 L412.936 882.584 L412.709 888.375 L412.481 893.358 L412.253 890.765 L412.025 889.045 L411.798 899.291 L411.57 909.127 L411.342 903.27 L411.114 899.712 L410.886 893.66 L410.659 904.997 L410.431 907.501 L410.203 914.481 L409.975 914.708 L409.748 908.179 L409.52 900.807 L409.292 894.961 L409.064 898.181 L408.836 897.943 L408.609 894.509 L408.381 886.07 L408.153 874.128 L407.925 864.407 L407.698 862.215 L407.47 870.847 L407.242 868.972 L407.014 866.56 L406.787 866.055 L406.559 857.885 L406.331 854.828 L406.103 850.858 L405.875 845.256 L405.648 846.793 L405.42 848.734 L405.192 840.056 L404.964 837.263 L404.737 834.097 L404.509 833.682 L404.281 834.939 L404.053 832.305 L403.826 827.931 L403.598 819.426 L403.37 817.688 L403.142 815.957 L402.914 815.496 L402.687 807.82 L402.459 804.631 L402.231 814.185 L402.003 816.377 L401.776 806.858 L401.548 803.486 L401.32 803.448 L401.092 811.633 L400.865 806.115 L400.637 797.671 L400.409 797.467 L400.181 795.14 L399.953 786.75 L399.726 789.176 L399.498 784.808 L399.27 774.568 L399.042 776.19 L398.815 784.839 L398.587 791.046 L398.359 783.625 L398.131 774.743 L397.903 769.618 L397.676 764.263 L397.448 759.431 L397.22 764.137 L396.992 762.485 L396.765 776.638 L396.537 775.518 L396.309 772.811 L396.081 777.344 L395.854 783.546 L395.626 777.068 L395.398 775.368 L395.17 773.045 L394.942 765.578 L394.715 759.194 L394.487 748.987 L394.259 744.986 L394.031 738.223 L393.804 731.556 L393.576 733.283 L393.348 736.894 L393.12 745.265 L392.893 743.064 L392.665 743.729 L392.437 733.959 L392.209 728.605 L391.981 720.659 L391.754 717.614 L391.526 727.227 L391.298 724.417 L391.07 720.388 L390.843 720.679 L390.615 721.2 L390.387 715.527 L390.159 709.845 L389.932 707.06 L389.704 702.198 L389.476 704.931 L389.248 722.293 L389.02 719.435 L388.793 720.828 L388.565 716.777 L388.337 722.157 L388.109 715.89 L387.882 708.434 L387.654 706.327 L387.426 715.991 L387.198 721.012 L386.97 720.084 L386.743 720.419 L386.515 713.725 L386.287 702.301 L386.059 695.773 L385.832 705.776 L385.604 707.318 L385.376 709.135 L385.148 714.731 L384.921 716.481 L384.693 714.069 L384.465 705.645 L384.237 705.479 L384.009 713.785 L383.782 705.847 L383.554 699.166 L383.326 698.5 L383.098 720.203 L382.871 733.306 L382.643 721.066 L382.415 712.269 L382.187 705.064 L381.96 706.719 L381.732 701.778 L381.504 689.666 L381.276 682.673 L381.048 673.708 L380.821 668.452 L380.593 660.762 L380.365 660.406 L380.137 658.515 L379.91 655.879 L379.682 647.045 L379.454 653.929 L379.226 648.907 L378.999 643.498 L378.771 640.486 L378.543 644.891 L378.315 643.986 L378.087 651.882 L377.86 655.25 L377.632 650.859 L377.404 646.497 L377.176 645.784 L376.949 643.429 L376.721 648.229 L376.493 657.44 L376.265 690.456 L376.037 690.424 L375.81 690.68 L375.582 684.211 L375.354 680.859 L375.126 690.935 L374.899 702.822 L374.671 704.199 L374.443 701.859 L374.215 702.327 L373.988 692.109 L373.76 695.881 L373.532 697.749 L373.304 714.237 L373.076 734.751 L372.849 730.783 L372.621 732.751 L372.393 725.266 L372.165 715.644 L371.938 744.429 L371.71 746.863 L371.482 748.769 L371.254 783.693 L371.027 785.436 L370.799 782.653 L370.571 777.266 L370.343 779.781 L370.115 791.261 L369.888 795.486 L369.66 787.96 L369.432 781.598 L369.204 783.625 L368.977 784.882 L368.749 806.21 L368.521 801.631 L368.293 800.478 L368.066 807.942 L367.838 803.162 L367.61 808.138 L367.382 805.695 L367.154 808.891 L366.927 808.382 L366.699 799.377 L366.471 793.892 L366.243 795.95 L366.016 794.452 L365.788 790.557 L365.56 793.242 L365.332 788.8 L365.104 797.16 L364.877 789.976 L364.649 783.127 L364.421 772.248 L364.193 765.943 L363.966 758.395 L363.738 765.639 L363.51 763.153 L363.282 779.046 L363.055 774.577 L362.827 782.6 L362.599 790.184 L362.371 782.408 L362.143 777.114 L361.916 767.201 L361.688 769.175 L361.46 765.219 L361.232 768.409 L361.005 757.217 L360.777 760.66 L360.549 747.887 L360.321 744.79 L360.094 745.284 L359.866 733.614 L359.638 742.044 L359.41 725.852 L359.182 722.354 L358.955 713.66 L358.727 711.922 L358.499 718.491 L358.271 721.597 L358.044 705.516 L357.816 719.603 L357.588 717.565 L357.36 726.758 L357.133 726.68 L356.905 734.879 L356.677 736.742 L356.449 737.261 L356.221 735.871 L355.994 738.616 L355.766 730.095 L355.538 735.844 L355.31 732.701 L355.083 758.491 L354.855 754.676 L354.627 762.704 L354.399 769.524 L354.171 758.344 L353.944 745.235 L353.716 743.447 L353.488 736.031 L353.26 746.855 L353.033 768.256 L352.805 779.872 L352.577 796.053 L352.349 804.319 L352.122 798.21 L351.894 804.737 L351.666 813.269 L351.438 811.51 L351.21 804.488 L350.983 794.597 L350.755 784.145 L350.527 770.769 L350.299 763.717 L350.072 761.395 L349.844 756.921 L349.616 746.953 L349.388 735.632 L349.161 730.35 L348.933 723.543 L348.705 713.667 L348.477 701.147 L348.249 697.222 L348.022 697.319 L347.794 708.884 L347.566 704.042 L347.338 703.473 L347.111 694.476 L346.883 686.462 L346.655 686.475 L346.427 693.789 L346.2 682.338 L345.972 680.213 L345.744 697.676 L345.516 687.093 L345.288 674.038 L345.061 695.118 L344.833 699.205 L344.605 695.857 L344.377 697.814 L344.15 676.675 L343.922 672.176 L343.694 674.133 L343.466 663.198 L343.238 645.946 L343.011 637.737 L342.783 627.441 L342.555 632.167 L342.327 614.534 L342.1 607.665 L341.872 589.499 L341.644 580.778 L341.416 558.187 L341.189 541.123 L340.961 512.469 L340.733 495.536 L340.505 480.703 L340.277 480.16 L340.05 456.785 L339.822 454.73 L339.594 457.309 L339.366 457.311 L339.139 433.585 L338.911 417.285 L338.683 429.543 L338.455 425.896 L338.228 422.364 L338 443.831 L337.772 432.663 L337.544 431.938 L337.316 415.236 L337.089 453.772 L336.861 447.985 L336.633 436.097 L336.405 431.135 L336.178 426.316 L335.95 435.605 L335.722 420.292 L335.494 427.247 L335.267 437.179 L335.039 482.933 L334.811 480.019 L334.583 479.139 L334.355 482.097 L334.128 472.361 L333.9 461.976 L333.672 437.604 L333.444 417.538 L333.217 429.947 L332.989 424.263 L332.761 418.825 L332.533 392.378 L332.305 395.801 L332.078 381.557 L331.85 373.959 L331.622 421.104 L331.394 418.419 L331.167 389.181 L330.939 377.725 L330.711 379.313 L330.483 370.83 L330.256 393.242 L330.028 412.284 L329.8 401.854 L329.572 388.632 L329.344 373.713 L329.117 359.956 L328.889 389.824 L328.661 380.676 L328.433 370.52 L328.206 370.766 L327.978 344.781 L327.75 362.047 L327.522 339.625 L327.295 334.821 L327.067 325.47 L326.839 305.542 L326.611 311.219 L326.383 278.97 L326.156 279.855 L325.928 274.942 L325.7 279.65 L325.472 271.786 L325.245 283.032 L325.017 283.581 L324.789 285.399 L324.561 263.821 L324.334 254.773 L324.106 266.253 L323.878 258.118 L323.65 233.601 L323.422 262.996 L323.195 237.371 L322.967 267.507 L322.739 284.668 L322.511 254.335 L322.284 234.601 L322.056 219.335 L321.828 202.563 L321.6 211.285 L321.372 184.04 L321.145 206.911 L320.917 238.328 L320.689 212.203 L320.461 244.653 L320.234 225.286 L320.006 209.764 L319.778 201.34 L319.55 209.874 L319.323 191.504 L319.095 218.442 L318.867 205.381 L318.639 197.165 L318.411 157.733 L318.184 165.311 L317.956 253.046 L317.728 269.414 L317.5 276.318 L317.273 244.746 L317.045 258.609 L316.817 236.769 L316.589 196.95 L316.362 188.319 L316.134 136.607 L315.906 124.28 L315.678 88.9395 L315.45 57.1588 L315.223 21.2059 L314.995 29.2452 L314.767 9.61019 L314.539 -7.136 L314.312 -3.41916 L314.084 -31.9148 L313.856 -39.6945 L313.628 -49.1219 L313.401 -103.259 L313.173 -112.929 L312.945 -101.01 L312.717 -107.712 L312.489 -137.291 L312.262 -87.1239 L312.034 -94.9787 L311.806 -114.066 L311.578 -93.7329 L311.351 -45.9835 L311.123 40.8872 L310.895 -7.55798 L310.667 -66.5665 L310.439 -22.6127 L310.212 -3.21669 L309.984 -18.6207 L309.756 -87.7616 L309.528 -124.35 L309.301 -174.052 L309.073 -170.788 L308.845 -119.384 L308.617 -37.6085 L308.39 -27.3396 L308.162 -27.4683 L307.934 -0.0411252 L307.706 29.5688 L307.478 -19.4118 L307.251 -19.2567 L307.023 -26.726 L306.795 -71.7352 L306.567 22.3127 L306.34 85.8578 L306.112 142.72 L305.884 139.856 L305.656 158.721 L305.429 248.067 L305.201 185.563 L304.973 188.699 L304.745 115.374 L304.517 219.506 L304.29 306.838 L304.062 382.073 L303.834 307.417 L303.606 265.002 L303.379 363.458 L303.151 291.383 L302.923 206.804 L302.695 183.112 L302.468 116.002 L302.24 65.2665 L302.012 86.5168 L301.784 15.969 L301.556 -55.9738 L301.329 -157.661 L301.101 -227.338 L300.873 -329.706 L300.645 -334.108 L300.418 -375.474 L300.19 -409.057 L299.962 -490.524 L299.734 -456.883 L299.506 -318.087 L299.279 -450.307 L299.051 -331.295 L298.823 -311.062 L298.595 36.7992 L298.368 -60.7436 L298.14 -105.1 L297.912 -82.2542 L297.684 -22.0193 L297.457 -55.2212 L297.229 -206.482 L297.001 -378.11 L296.773 -599.126 L296.545 -906.027 L296.318 -1216.38 L296.09 -1506.61 L295.862 -1704.4 L295.634 -1858.32 L295.407 -3074.82 L295.179 -5856.78 L294.951 2632.57  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="0.5"/>
<polyline clip-path="url(#clip532)" style="stroke:#009af9; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="294.951,2632.57 295.179,-235.454 295.407,404.846 295.634,647.848 295.862,332.746 296.09,182.556 296.318,212.434 296.545,368.702 296.773,679.71 297.001,783.751 297.229,853.253 297.457,921.341 297.684,879.944 297.912,776.885 298.14,706.519 298.368,698.615 298.595,760.25 298.823,518.946 299.051,461.771 299.279,339.379 299.506,486.508 299.734,353.41 299.962,292.799 300.19,348.949 300.418,351.599 300.645,364.93 300.873,344.172 301.101,461.626 301.329,515.436 301.556,635.712 301.784,694.981 302.012,754.101 302.24,716.49 302.468,751.496 302.695,811.168 302.923,817.283 303.151,915.653 303.379,986.933 303.606,897.223 303.834,926.251 304.062,1005.09 304.29,931.65 304.517,850.17 304.745,757.662 304.973,834.244 305.201,817.746 305.429,877.724 305.656,795.728 305.884,766.375 306.112,756.863 306.34,698.581 306.567,635.663 306.795,551.918 307.023,589.744 307.251,585.946 307.478,575.255 307.706,620.362 307.934,584.676 308.162,551.025 308.39,541.722 308.617,523.221 308.845,450.459 309.073,398.953 309.301,387.189 309.528,436.619 309.756,468.071 309.984,554.411 310.212,561.496 310.439,536.081 310.667,491.1 310.895,557.305 311.123,605.952 311.351,530.55 311.578,482.653 311.806,456.83 312.034,468.848 312.262,469.333 312.489,420.322 312.717,444.902 312.945,444.653 313.173,427.058 313.401,430.136 313.628,491.251 313.856,494.239 314.084,495.708 314.312,520.245 314.539,510.805 314.767,522.111 314.995,536.768 315.223,523.71 315.45,558.865 315.678,588.565 315.906,623.124 316.134,630.316 316.362,691.142 316.589,694.553 316.817,735.81 317.045,753.787 317.273,736.118 317.5,766.347 317.728,755.07 317.956,735.452 318.184,663.431 318.411,651.586 318.639,692.973 318.867,696.539 319.095,705.29 319.323,677 319.55,691.765 319.778,679.44 320.006,683.608 320.234,695.454 320.461,711.733 320.689,679.467 320.917,704 321.145,672.628 321.372,648.229 321.6,674.366 321.828,662.341 322.056,676.057 322.284,688.153 322.511,705.427 322.739,736.097 322.967,716.865 323.195,686.957 323.422,711.609 323.65,682.348 323.878,705.69 324.106,710.439 324.334,696.388 324.561,702.2 324.789,722.208 325.017,717.205 325.245,713.47 325.472,699.856 325.7,704.665 325.928,697.103 326.156,698.962 326.383,695.119 326.611,730.005 326.839,721.668 327.067,740.259 327.295,746.904 327.522,748.855 327.75,770.692 327.978,752.228 328.206,778.759 328.433,775.789 328.661,783.499 328.889,790.166 329.117,761.569 329.344,773.341 329.572,786.473 329.8,797.717 330.028,805.913 330.256,786.237 330.483,763.785 330.711,769.96 330.939,765.986 331.167,775.454 331.394,807.339 331.622,807.616 331.85,766.468 332.078,771.805 332.305,784.47 332.533,778.856 332.761,806.891 332.989,810.063 333.217,813.517 333.444,799.793 333.672,819.593 333.9,844.985 334.128,853.517 334.355,861.368 334.583,856.364 334.811,855.103 335.039,855.895 335.267,816.138 335.494,804.744 335.722,796.071 335.95,810.348 336.178,799.59 336.405,802.405 336.633,805.39 336.861,815.849 337.089,819.731 337.316,785.401 337.544,801.474 337.772,800.245 338,809.971 338.228,788.913 338.455,790.544 338.683,792.313 338.911,779.108 339.139,794.872 339.366,820.147 339.594,818.317 339.822,814.009 340.05,814.254 340.277,839.176 340.505,837.937 340.733,852.075 340.961,868.75 341.189,901.372 341.416,918.187 341.644,942.071 341.872,949.324 342.1,967.529 342.327,972.822 342.555,990.408 342.783,984.188 343.011,993.24 343.238,1000.03 343.466,1017.23 343.694,1027.04 343.922,1023.51 344.15,1026.43 344.377,1048.69 344.605,1045.19 344.833,1046.96 345.061,1041.47 345.288,1021.12 345.516,1033.46 345.744,1043 345.972,1025.67 346.2,1026.27 346.427,1036.83 346.655,1028.4 346.883,1026.91 347.111,1033.7 347.338,1041.58 347.566,1040.69 347.794,1044.15 348.022,1032 348.249,1030.49 348.477,1033.04 348.705,1044.98 348.933,1053.93 349.161,1059.53 349.388,1063.53 349.616,1074.15 349.844,1083.26 350.072,1086.44 350.299,1087.44 350.527,1093.37 350.755,1106.46 350.983,1116.17 351.21,1125.25 351.438,1131.2 351.666,1131.68 351.894,1122.39 352.122,1114.94 352.349,1119.95 352.577,1110.93 352.805,1095.06 353.033,1083.08 353.26,1062.93 353.488,1051.65 353.716,1058.1 353.944,1058.68 354.171,1071.61 354.399,1082.3 354.627,1074.65 354.855,1065.91 355.083,1068.59 355.31,1045.06 355.538,1047.05 355.766,1040.41 355.994,1048.15 356.221,1044.35 356.449,1044.59 356.677,1042.96 356.905,1040.03 357.133,1031.2 357.36,1030.18 357.588,1020.48 357.816,1021.43 358.044,1007.49 358.271,1024.29 358.499,1020.2 358.727,1012.9 358.955,1013.57 359.182,1021.62 359.41,1024.1 359.638,1041.11 359.866,1032.15 360.094,1043.63 360.321,1042.11 360.549,1044.2 360.777,1057.02 361.005,1052.68 361.232,1063.64 361.46,1059.55 361.688,1062.56 361.916,1059.65 362.143,1069.17 362.371,1073.61 362.599,1080.74 362.827,1072.62 363.055,1064.11 363.282,1067.7 363.51,1052.41 363.738,1053.95 363.966,1046.16 364.193,1053.09 364.421,1058.66 364.649,1069.38 364.877,1075.55 365.104,1082.1 365.332,1073.34 365.56,1076.95 365.788,1073.44 366.016,1076.49 366.243,1077.09 366.471,1074.2 366.699,1078.95 366.927,1087.59 367.154,1087.22 367.382,1083.26 367.61,1084.85 367.838,1079.23 368.066,1083.26 368.293,1075.38 368.521,1075.68 368.749,1079.52 368.977,1060.02 369.204,1057.95 369.432,1055.14 369.66,1060.91 369.888,1067.95 370.115,1063.07 370.343,1051.69 370.571,1048.43 370.799,1053.16 371.027,1055.17 371.254,1052.67 371.482,1023.12 371.71,1020.45 371.938,1017.28 372.165,992.126 372.393,1001.55 372.621,1008.57 372.849,1005.85 373.076,1009.09 373.304,990.261 373.532,974.666 373.76,972.047 373.988,967.617 374.215,977.763 374.443,976.518 374.671,978.092 374.899,975.968 375.126,964.269 375.354,954.141 375.582,956.771 375.81,962.724 376.037,961.712 376.265,960.989 376.493,932.813 376.721,923.448 376.949,918.086 377.176,919.7 377.404,919.658 377.632,923.364 377.86,927.107 378.087,923.108 378.315,914.946 378.543,915.113 378.771,910.15 378.999,912.471 379.226,917.323 379.454,921.764 379.682,914.536 379.91,923.192 380.137,925.143 380.365,926.333 380.593,925.984 380.821,933.367 381.048,938.084 381.276,946.921 381.504,953.539 381.732,966.11 381.96,970.5 382.187,968.197 382.415,975.062 382.643,983.727 382.871,996.476 383.098,983.857 383.326,964.264 383.554,964.248 383.782,970.537 384.009,978.23 384.237,969.77 384.465,969.267 384.693,977.519 384.921,979.287 385.148,976.916 385.376,970.924 385.604,968.494 385.832,966.333 386.059,956.406 386.287,962.564 386.515,974.371 386.743,980.717 386.97,979.745 387.198,980.034 387.426,974.595 387.654,964.984 387.882,966.474 388.109,973.687 388.337,979.586 388.565,973.828 388.793,977.349 389.02,975.366 389.248,977.645 389.476,961.602 389.704,958.333 389.932,962.727 390.159,964.937 390.387,970.222 390.615,975.499 390.843,974.385 391.07,973.499 391.298,977.026 391.526,979.279 391.754,969.764 391.981,972.266 392.209,980.091 392.437,985.046 392.665,994.989 392.893,993.751 393.12,995.392 393.348,986.986 393.576,982.926 393.804,980.663 394.031,987.075 394.259,993.598 394.487,997.127 394.715,1007.62 394.942,1013.73 395.17,1021.05 395.398,1022.83 395.626,1023.98 395.854,1030.21 396.081,1023.77 396.309,1018.87 396.537,1021.07 396.765,1021.64 396.992,1008.33 397.22,1009.44 397.448,1004.39 397.676,1008.84 397.903,1013.85 398.131,1018.62 398.359,1027.6 398.587,1034.91 398.815,1028.49 399.042,1019.89 399.27,1017.77 399.498,1028.37 399.726,1032.33 399.953,1029.45 400.181,1037.87 400.409,1039.7 400.637,1039.39 400.865,1047.89 401.092,1053.1 401.32,1044.93 401.548,1044.46 401.776,1047.38 402.003,1057.16 402.231,1054.51 402.459,1045.15 402.687,1047.89 402.914,1055.53 403.142,1055.48 403.37,1056.72 403.598,1057.97 403.826,1066.58 404.053,1070.58 404.281,1072.76 404.509,1071.03 404.737,1070.95 404.964,1073.69 405.192,1076.04 405.42,1084.87 405.648,1082.49 405.875,1080.5 406.103,1085.85 406.331,1089.45 406.559,1092.08 406.787,1100.34 407.014,1100.37 407.242,1102.34 407.47,1103.76 407.698,1095.25 407.925,1097 408.153,1107.1 408.381,1119.93 408.609,1128.5 408.836,1131.54 409.064,1131.32 409.292,1127.74 409.52,1133.39 409.748,1140.74 409.975,1147.15 410.203,1146.47 410.431,1139.44 410.659,1136.55 410.886,1125.73 411.114,1131.62 411.342,1134.8 411.57,1140.48 411.798,1130.95 412.025,1121.06 412.253,1122.34 412.481,1124.53 412.709,1119.32 412.936,1113.38 413.164,1110.25 413.392,1106.38 413.62,1106.23 413.847,1109.9 414.075,1106.77 414.303,1101.81 414.531,1103.38 414.759,1094.39 414.986,1092.05 415.214,1091.18 415.442,1096.23 415.67,1091.21 415.897,1085.12 416.125,1080.94 416.353,1081.83 416.581,1090.28 416.808,1096.3 417.036,1094.1 417.264,1090.56 417.492,1097.48 417.72,1106.46 417.947,1112.06 418.175,1116.28 418.403,1116.6 418.631,1116.58 418.858,1112.78 419.086,1111.63 419.314,1111.85 419.542,1106.73 419.769,1114.3 419.997,1112.7 420.225,1114.79 420.453,1117.82 420.681,1119.47 420.908,1122.4 421.136,1113.98 421.364,1111.26 421.592,1101.25 421.819,1100.27 422.047,1106.5 422.275,1111.39 422.503,1111.97 422.731,1107.89 422.958,1113.53 423.186,1110.15 423.414,1102.66 423.642,1109.11 423.869,1107.91 424.097,1100.74 424.325,1106.18 424.553,1102.5 424.78,1091.62 425.008,1096.14 425.236,1084.83 425.464,1089.55 425.692,1073.68 425.919,1068.58 426.147,1065.32 426.375,1059.51 426.603,1062.13 426.83,1062.45 427.058,1068.66 427.286,1066.8 427.514,1072.18 427.741,1070.63 427.969,1072.74 428.197,1066.44 428.425,1067.38 428.653,1065.37 428.88,1066.95 429.108,1067.35 429.336,1059.98 429.564,1063.65 429.791,1060.69 430.019,1064.73 430.247,1065.16 430.475,1057 430.702,1058.64 430.93,1053.47 431.158,1053.04 431.386,1054.37 431.614,1059.75 431.841,1065.35 432.069,1052.73 432.297,1056.15 432.525,1057.54 432.752,1059.17 432.98,1050.53 433.208,1051.36 433.436,1048.64 433.664,1053.58 433.891,1057.02 434.119,1053.58 434.347,1048.72 434.575,1044.79 434.802,1047.33 435.03,1047.29 435.258,1041.64 435.486,1030.3 435.713,1031.95 435.941,1034.85 436.169,1033.49 436.397,1028.63 436.625,1025.08 436.852,1023.04 437.08,1016.6 437.308,1009.41 437.536,1010.83 437.763,1012.4 437.991,1009.01 438.219,1013.49 438.447,1010.94 438.674,1013.78 438.902,1013.14 439.13,1015.49 439.358,1024.71 439.586,1025.12 439.813,1023.61 440.041,1021.95 440.269,1022.61 440.497,1027.07 440.724,1022.9 440.952,1026.51 441.18,1030.92 441.408,1021.41 441.635,1021.43 441.863,1027.45 442.091,1026.17 442.319,1026.75 442.547,1026.16 442.774,1018.33 443.002,1019.45 443.23,1016.94 443.458,1017.62 443.685,1018.76 443.913,1021.02 444.141,1020.87 444.369,1015.03 444.597,1018.47 444.824,1022.83 445.052,1020.17 445.28,1020.12 445.508,1023.64 445.735,1026.83 445.963,1027.26 446.191,1030.26 446.419,1030.32 446.646,1031.52 446.874,1030.34 447.102,1029.49 447.33,1025.66 447.558,1028.74 447.785,1023.41 448.013,1028.23 448.241,1033.53 448.469,1030.98 448.696,1036.05 448.924,1033.56 449.152,1031.04 449.38,1030.48 449.607,1034.36 449.835,1031.05 450.063,1028.19 450.291,1030.02 450.519,1029.97 450.746,1026.3 450.974,1024.06 451.202,1026.05 451.43,1023.25 451.657,1027.8 451.885,1031.14 452.113,1028.8 452.341,1028.02 452.568,1027.58 452.796,1032.64 453.024,1034.03 453.252,1037.82 453.48,1030.86 453.707,1019.7 453.935,1022.6 454.163,1022.74 454.391,1017.94 454.618,1015.68 454.846,1017.91 455.074,1022.94 455.302,1021.52 455.53,1020.46 455.757,1022.91 455.985,1023.24 456.213,1026.46 456.441,1029.48 456.668,1028.06 456.896,1027.04 457.124,1029.65 457.352,1028.57 457.579,1029.17 457.807,1035.11 458.035,1029.4 458.263,1025.39 458.491,1025.74 458.718,1021.28 458.946,1025.16 459.174,1029.61 459.402,1032.13 459.629,1033.01 459.857,1035.72 460.085,1038.7 460.313,1035.42 460.54,1033.77 460.768,1033.99 460.996,1029.37 461.224,1035.81 461.452,1036.77 461.679,1033 461.907,1027.08 462.135,1020.67 462.363,1020.9 462.59,1026.67 462.818,1020.47 463.046,1018.6 463.274,1020 463.501,1016.19 463.729,1015.17 463.957,1015.35 464.185,1011.65 464.413,1015.76 464.64,1017.36 464.868,1017.86 465.096,1018.86 465.324,1017.63 465.551,1013.78 465.779,1015.13 466.007,1011.24 466.235,1008.68 466.462,1012.82 466.69,1006.07 466.918,999.26 467.146,1000.16 467.374,998.054 467.601,992.662 467.829,991.905 468.057,989.342 468.285,988.078 468.512,991.587 468.74,985.91 468.968,989.791 469.196,987.519 469.424,991.922 469.651,990.479 469.879,988.591 470.107,986.237 470.335,981.149 470.562,983.054 470.79,986.935 471.018,986.014 471.246,981.999 471.473,987.893 471.701,993.496 471.929,984.458 472.157,989.694 472.385,990.884 472.612,993.309 472.84,993.645 473.068,995.169 473.296,998.639 473.523,991.382 473.751,984.369 473.979,973.062 474.207,975.77 474.434,977.858 474.662,976.783 474.89,977.126 475.118,976.317 475.346,974.219 475.573,969.851 475.801,970.623 476.029,970.862 476.257,974.277 476.484,976.354 476.712,978.222 476.94,977.876 477.168,979.116 477.395,978.807 477.623,977.45 477.851,973.534 478.079,975.287 478.307,969.841 478.534,969.712 478.762,976.796 478.99,982.638 479.218,984.998 479.445,984.071 479.673,981.56 479.901,981.061 480.129,985.464 480.357,985.614 480.584,982.633 480.812,985.535 481.04,987.617 481.268,987.042 481.495,990.027 481.723,989.821 481.951,982.319 482.179,984.621 482.406,988.917 482.634,992.91 482.862,984.185 483.09,986.197 483.318,989.01 483.545,990.18 483.773,990.077 484.001,987.083 484.229,992.782 484.456,992.727 484.684,995.332 484.912,995.702 485.14,998.087 485.367,994.224 485.595,994.47 485.823,997.159 486.051,997.344 486.279,999.696 486.506,1003.8 486.734,1005.3 486.962,1005.13 487.19,1002.05 487.417,999.876 487.645,999.67 487.873,999.365 488.101,1002.52 488.328,1004.28 488.556,1006.46 488.784,1008.13 489.012,1011.15 489.24,1011.4 489.467,1010.5 489.695,1006.77 489.923,1011.52 490.151,1010.17 490.378,1010.67 490.606,1007.72 490.834,1011.5 491.062,1012.85 491.29,1009.99 491.517,1010.72 491.745,1004.8 491.973,1004.76 492.201,1001.61 492.428,1002.03 492.656,1005.18 492.884,999.53 493.112,997.347 493.339,993.298 493.567,990.62 493.795,991.122 494.023,993.842 494.251,996.36 494.478,996.023 494.706,997.609 494.934,997.871 495.162,995.017 495.389,993.642 495.617,992.835 495.845,993.333 496.073,994.82 496.3,997.383 496.528,995.203 496.756,995.851 496.984,993.07 497.212,993.616 497.439,994.124 497.667,996.136 497.895,998.932 498.123,999.762 498.35,999.028 498.578,999.413 498.806,996.536 499.034,998.501 499.261,998.356 499.489,1000.95 499.717,1002.95 499.945,1003.52 500.173,1001.92 500.4,1000.51 500.628,1000.44 500.856,998.161 501.084,995.972 501.311,996.239 501.539,998.114 501.767,998.106 501.995,997.595 502.223,1000.45 502.45,1003 502.678,1001.76 502.906,998.182 503.134,996.438 503.361,998.843 503.589,999.96 503.817,1005.34 504.045,1003.68 504.272,1001.42 504.5,1003.81 504.728,1003.36 504.956,1004.26 505.184,1006.29 505.411,1003.79 505.639,1003.64 505.867,1006.02 506.095,1005.22 506.322,998.487 506.55,999.648 506.778,999.973 507.006,999.943 507.233,1003.84 507.461,1005.22 507.689,1004.89 507.917,1006.07 508.145,1000.17 508.372,998.818 508.6,993.536 508.828,994.669 509.056,994.315 509.283,993.514 509.511,994.414 509.739,989.199 509.967,988.64 510.194,989.054 510.422,993.718 510.65,991.243 510.878,994.422 511.106,994.677 511.333,993.668 511.561,994.775 511.789,994.509 512.017,994.847 512.244,995.923 512.472,1000.34 512.7,1003.66 512.928,1004.91 513.156,1002.22 513.383,1001.55 513.611,1002.31 513.839,1000.96 514.067,1001.27 514.294,998.144 514.522,1001.69 514.75,998.759 514.978,1000.95 515.205,1002.1 515.433,1002.29 515.661,1004.1 515.889,1003.35 516.117,1003.87 516.344,1007.13 516.572,1008.28 516.8,1008.54 517.028,1009.94 517.255,1011.69 517.483,1001.27 517.711,1002.84 517.939,1003.01 518.166,1004.37 518.394,1005.46 518.622,1008.35 518.85,1008.41 519.078,1008.17 519.305,1008.12 519.533,1005.72 519.761,1007.57 519.989,1010.54 520.216,1011.2 520.444,1010.11 520.672,1008.68 520.9,1009.19 521.127,1009.29 521.355,1014.32 521.583,1009.46 521.811,1010.07 522.039,1009.4 522.266,1006 522.494,1008.26 522.722,1008.26 522.95,1002.77 523.177,1001.84 523.405,1001.93 523.633,996.968 523.861,997.324 524.089,996.571 524.316,998.233 524.544,1000.16 524.772,1001.93 525,1001.56 525.227,1002.64 525.455,1005.8 525.683,1007 525.911,1006.09 526.138,1007.16 526.366,1004.33 526.594,1003.37 526.822,999.022 527.05,994.275 527.277,997.378 527.505,996.777 527.733,999.723 527.961,1000.6 528.188,1004.05 528.416,1006.5 528.644,1004.92 528.872,1003.76 529.099,1002.33 529.327,1004.88 529.555,1003.37 529.783,1004.4 530.011,1006.78 530.238,1009.48 530.466,1008.26 530.694,1013.36 530.922,1016.99 531.149,1018.76 531.377,1022.1 531.605,1023.21 531.833,1024.69 532.06,1027.43 532.288,1027.66 532.516,1030.16 532.744,1035.76 532.972,1038.55 533.199,1039.86 533.427,1040.85 533.655,1039.69 533.883,1041.22 534.11,1041.69 534.338,1042.03 534.566,1041.97 534.794,1043.11 535.022,1043.54 535.249,1044.49 535.477,1044.82 535.705,1042.36 535.933,1044.87 536.16,1047.12 536.388,1045.97 536.616,1047.49 536.844,1047.83 537.071,1052.82 537.299,1053.63 537.527,1054.85 537.755,1058.46 537.983,1052.47 538.21,1050.48 538.438,1052.66 538.666,1050.17 538.894,1056.01 539.121,1054.01 539.349,1056.27 539.577,1057.6 539.805,1058.88 540.032,1055.68 540.26,1054.56 540.488,1059.47 540.716,1060.31 540.944,1061.41 541.171,1059.98 541.399,1056.21 541.627,1055.56 541.855,1054.28 542.082,1052.33 542.31,1050.56 542.538,1052.84 542.766,1053.94 542.993,1052.24 543.221,1054.67 543.449,1045.81 543.677,1048.02 543.905,1049.15 544.132,1051.15 544.36,1050.5 544.588,1052.33 544.816,1051.14 545.043,1049.68 545.271,1052.65 545.499,1049.16 545.727,1042.69 545.955,1043.58 546.182,1042.64 546.41,1039.94 546.638,1035.97 546.866,1031.75 547.093,1034.43 547.321,1038.93 547.549,1037.52 547.777,1037.42 548.004,1036.49 548.232,1040.03 548.46,1040.18 548.688,1040.44 548.916,1037.25 549.143,1037.54 549.371,1039.73 549.599,1038.55 549.827,1037.91 550.054,1039.72 550.282,1042.37 550.51,1045.57 550.738,1040.31 550.965,1041.76 551.193,1041.7 551.421,1043.31 551.649,1041.49 551.877,1044.28 552.104,1042.9 552.332,1045.21 552.56,1048.48 552.788,1050.48 553.015,1049.48 553.243,1051.45 553.471,1052.1 553.699,1049.86 553.926,1050.61 554.154,1055.5 554.382,1056.45 554.61,1056.96 554.838,1058.05 555.065,1058.47 555.293,1055.66 555.521,1055.32 555.749,1054.67 555.976,1052.67 556.204,1051.53 556.432,1048.11 556.66,1041.39 556.888,1034.28 557.115,1035.43 557.343,1036.07 557.571,1035.58 557.799,1033.55 558.026,1035.77 558.254,1040.37 558.482,1039.07 558.71,1037.32 558.937,1036.44 559.165,1038.25 559.393,1039.94 559.621,1042.57 559.849,1039.51 560.076,1040.65 560.304,1043.78 560.532,1042.95 560.76,1041.94 560.987,1039.31 561.215,1036.56 561.443,1035.87 561.671,1039.3 561.898,1043.01 562.126,1041.75 562.354,1041.43 562.582,1044.02 562.81,1041.09 563.037,1045.25 563.265,1040.89 563.493,1042.22 563.721,1042.38 563.948,1042.09 564.176,1044.36 564.404,1040.03 564.632,1040.03 564.859,1036.82 565.087,1040.72 565.315,1039.91 565.543,1037.48 565.771,1039.83 565.998,1038.69 566.226,1035.55 566.454,1033.04 566.682,1032.45 566.909,1031.24 567.137,1029.55 567.365,1031.49 567.593,1032.1 567.821,1032.28 568.048,1033.18 568.276,1033.12 568.504,1033.81 568.732,1031.01 568.959,1029.14 569.187,1022.55 569.415,1023.5 569.643,1021.05 569.87,1020 570.098,1019.08 570.326,1018.5 570.554,1016.7 570.782,1015.88 571.009,1018.13 571.237,1018.7 571.465,1017.88 571.693,1014.52 571.92,1014.93 572.148,1012.37 572.376,1008.21 572.604,1008.92 572.831,1010.27 573.059,1011.68 573.287,1012.58 573.515,1014.2 573.743,1017.45 573.97,1019.11 574.198,1021.46 574.426,1021.44 574.654,1021.63 574.881,1019.15 575.109,1017.85 575.337,1016.88 575.565,1014.25 575.792,1014.7 576.02,1016.41 576.248,1017.76 576.476,1020.38 576.704,1021.45 576.931,1020.68 577.159,1022.01 577.387,1021.57 577.615,1021.05 577.842,1017.21 578.07,1016.03 578.298,1017.81 578.526,1016.93 578.754,1018.39 578.981,1016.11 579.209,1020.08 579.437,1023.73 579.665,1021.26 579.892,1020.65 580.12,1017.8 580.348,1017.82 580.576,1019.29 580.803,1021.13 581.031,1020.36 581.259,1022.9 581.487,1020.91 581.715,1019.54 581.942,1018.7 582.17,1018.28 582.398,1018.49 582.626,1021.99 582.853,1022.85 583.081,1025.08 583.309,1025.59 583.537,1025.93 583.764,1027.24 583.992,1026.83 584.22,1025.94 584.448,1024.56 584.676,1024.55 584.903,1024.55 585.131,1026.01 585.359,1027.76 585.587,1026.16 585.814,1026.15 586.042,1023.84 586.27,1026.3 586.498,1027.6 586.725,1031.24 586.953,1033.91 587.181,1036.93 587.409,1035.8 587.637,1034.83 587.864,1034.54 588.092,1031.95 588.32,1029.87 588.548,1030.57 588.775,1032.23 589.003,1034 589.231,1033.42 589.459,1035.96 589.687,1038.1 589.914,1036.12 590.142,1030.5 590.37,1033.77 590.598,1034.93 590.825,1034.44 591.053,1035.12 591.281,1035.14 591.509,1034.09 591.736,1032.62 591.964,1034.01 592.192,1029.96 592.42,1030.09 592.648,1031.72 592.875,1032.15 593.103,1031.29 593.331,1030.23 593.559,1029.01 593.786,1030.7 594.014,1033.22 594.242,1034.02 594.47,1034.36 594.697,1035.78 594.925,1032.52 595.153,1029.84 595.381,1030 595.609,1031.14 595.836,1031.28 596.064,1031.17 596.292,1033.1 596.52,1033.49 596.747,1032.08 596.975,1030.64 597.203,1031.37 597.431,1032.06 597.658,1034.33 597.886,1036.21 598.114,1037.59 598.342,1035.31 598.57,1038.2 598.797,1036.03 599.025,1034.29 599.253,1034.45 599.481,1035.35 599.708,1034.96 599.936,1036.63 600.164,1038.11 600.392,1037.34 600.62,1036.6 600.847,1032.74 601.075,1032.8 601.303,1033.26 601.531,1032.15 601.758,1034.2 601.986,1035.29 602.214,1034.38 602.442,1037.78 602.669,1038.58 602.897,1042.04 603.125,1043.11 603.353,1044.33 603.581,1043.55 603.808,1043.74 604.036,1042.2 604.264,1040.6 604.492,1039.82 604.719,1041.35 604.947,1042.99 605.175,1043.55 605.403,1044.74 605.63,1045.19 605.858,1043.48 606.086,1044.07 606.314,1044.06 606.542,1046.13 606.769,1046.5 606.997,1042.74 607.225,1042.3 607.453,1046.34 607.68,1043.74 607.908,1041.06 608.136,1040.17 608.364,1042.43 608.591,1044.98 608.819,1046.82 609.047,1043.93 609.275,1042.81 609.503,1041.89 609.73,1038.79 609.958,1038.06 610.186,1036.66 610.414,1038.1 610.641,1035.79 610.869,1037.07 611.097,1038.72 611.325,1042.16 611.553,1039.79 611.78,1040.88 612.008,1039.42 612.236,1039.87 612.464,1040.43 612.691,1043.51 612.919,1041.06 613.147,1038.95 613.375,1038.76 613.602,1038.46 613.83,1036.75 614.058,1032.77 614.286,1030.46 614.514,1031 614.741,1031.61 614.969,1032.61 615.197,1033.81 615.425,1032.15 615.652,1031.53 615.88,1031.95 616.108,1029.26 616.336,1029 616.563,1028.13 616.791,1030.86 617.019,1028.64 617.247,1031.16 617.475,1029.53 617.702,1030 617.93,1030.28 618.158,1029.65 618.386,1030.94 618.613,1029.27 618.841,1026.79 619.069,1025.53 619.297,1022.48 619.524,1023.04 619.752,1023.37 619.98,1025.2 620.208,1023.96 620.436,1021.47 620.663,1017.49 620.891,1017.48 621.119,1016.34 621.347,1011.13 621.574,1010.86 621.802,1011.8 622.03,1010.96 622.258,1013.51 622.486,1012.49 622.713,1014.92 622.941,1013.29 623.169,1014.59 623.397,1014.87 623.624,1017.27 623.852,1016.17 624.08,1019.31 624.308,1018.89 624.535,1019.79 624.763,1019.76 624.991,1018.71 625.219,1015.5 625.447,1017.1 625.674,1019.16 625.902,1020.11 626.13,1021.3 626.358,1020.72 626.585,1020.42 626.813,1017.19 627.041,1018.66 627.269,1020.1 627.496,1019.01 627.724,1017.67 627.952,1017.45 628.18,1020.68 628.408,1020.83 628.635,1021.23 628.863,1023.14 629.091,1027.3 629.319,1031.04 629.546,1032.06 629.774,1034.67 630.002,1036.8 630.23,1038.16 630.457,1037.07 630.685,1037 630.913,1037.36 631.141,1038.32 631.369,1036.4 631.596,1036.62 631.824,1037.24 632.052,1039.19 632.28,1043.23 632.507,1041.42 632.735,1038.67 632.963,1039.31 633.191,1038.92 633.419,1039.84 633.646,1041.39 633.874,1040.95 634.102,1040.76 634.33,1041.76 634.557,1044.64 634.785,1043.48 635.013,1043.5 635.241,1044.73 635.468,1043.33 635.696,1042.93 635.924,1043.78 636.152,1043.79 636.38,1040.19 636.607,1038.4 636.835,1039.3 637.063,1042.08 637.291,1044.23 637.518,1044.98 637.746,1044.88 637.974,1045.39 638.202,1044.59 638.429,1046.73 638.657,1047.51 638.885,1048.89 639.113,1049.41 639.341,1049.67 639.568,1048.59 639.796,1046.99 640.024,1045.38 640.252,1043.71 640.479,1042.65 640.707,1045.01 640.935,1047.11 641.163,1047.79 641.39,1048.75 641.618,1050.9 641.846,1048.94 642.074,1050.21 642.302,1052.57 642.529,1053.95 642.757,1053.99 642.985,1049.68 643.213,1052.43 643.44,1051.46 643.668,1050.16 643.896,1051.91 644.124,1053.2 644.352,1052.46 644.579,1053.98 644.807,1054.88 645.035,1054.77 645.263,1054.84 645.49,1055.81 645.718,1055.14 645.946,1055.01 646.174,1056.38 646.401,1057.77 646.629,1052.82 646.857,1053.04 647.085,1049.74 647.313,1050.99 647.54,1051.27 647.768,1050.1 647.996,1050.92 648.224,1048.4 648.451,1049.86 648.679,1050.31 648.907,1048.24 649.135,1047.4 649.362,1049.85 649.59,1051.62 649.818,1052.88 650.046,1052.44 650.274,1053.49 650.501,1053.72 650.729,1053.81 650.957,1054.81 651.185,1054.62 651.412,1054.97 651.64,1056.82 651.868,1057.56 652.096,1058.3 652.323,1058.48 652.551,1059.91 652.779,1055.29 653.007,1055.72 653.235,1054.05 653.462,1051.5 653.69,1053.22 653.918,1053.55 654.146,1054.34 654.373,1056.48 654.601,1057.26 654.829,1058.66 655.057,1061.25 655.285,1061.14 655.512,1063.39 655.74,1065.27 655.968,1063.51 656.196,1063.97 656.423,1064.81 656.651,1066.47 656.879,1067.43 657.107,1068.8 657.334,1070.67 657.562,1069.16 657.79,1070.73 658.018,1070.9 658.246,1067.84 658.473,1068.08 658.701,1067.03 658.929,1067.22 659.157,1069.56 659.384,1070.97 659.612,1071.13 659.84,1070.31 660.068,1071.5 660.295,1072.74 660.523,1073.26 660.751,1072.68 660.979,1076.61 661.207,1074.53 661.434,1076.19 661.662,1076.25 661.89,1075.49 662.118,1071.72 662.345,1073.75 662.573,1071.01 662.801,1069.78 663.029,1069.08 663.256,1069.34 663.484,1068.24 663.712,1066.81 663.94,1067.48 664.168,1068.95 664.395,1067.6 664.623,1069.05 664.851,1070.84 665.079,1071.99 665.306,1072.17 665.534,1071.92 665.762,1071.05 665.99,1070.33 666.218,1067.55 666.445,1065.45 666.673,1064.11 666.901,1062.38 667.129,1063.32 667.356,1060.75 667.584,1059.25 667.812,1058.32 668.04,1057.19 668.267,1057.45 668.495,1058.09 668.723,1057.23 668.951,1055.61 669.179,1057.3 669.406,1056.21 669.634,1055.43 669.862,1055.81 670.09,1055.12 670.317,1055.17 670.545,1052.66 670.773,1051.84 671.001,1051.71 671.228,1050.78 671.456,1048.66 671.684,1050.39 671.912,1052.97 672.14,1053.63 672.367,1052.59 672.595,1054.86 672.823,1055.82 673.051,1055.58 673.278,1054.03 673.506,1054.87 673.734,1054.9 673.962,1052.76 674.189,1050.33 674.417,1052.15 674.645,1054.17 674.873,1051.89 675.101,1053.97 675.328,1056.55 675.556,1055.82 675.784,1056.73 676.012,1056.42 676.239,1053.23 676.467,1053.69 676.695,1051.38 676.923,1050.73 677.151,1050.92 677.378,1051.36 677.606,1052.94 677.834,1052.22 678.062,1053.33 678.289,1054.72 678.517,1055.6 678.745,1055.62 678.973,1054.73 679.2,1056.87 679.428,1058.27 679.656,1057.63 679.884,1058.21 680.112,1057.92 680.339,1058.48 680.567,1057.74 680.795,1059.7 681.023,1054.73 681.25,1056.42 681.478,1055.7 681.706,1055.42 681.934,1054 682.161,1051.82 682.389,1051.79 682.617,1050.72 682.845,1052.55 683.073,1052.95 683.3,1055.31 683.528,1052.1 683.756,1054.25 683.984,1055.58 684.211,1056.11 684.439,1057.18 684.667,1056.45 684.895,1058.09 685.122,1056.38 685.35,1057.28 685.578,1057.74 685.806,1058.75 686.034,1060.02 686.261,1061.68 686.489,1059.17 686.717,1061.47 686.945,1061.19 687.172,1060.8 687.4,1061.2 687.628,1061.53 687.856,1060.95 688.084,1059.72 688.311,1058.17 688.539,1059.15 688.767,1057.02 688.995,1058.29 689.222,1058.53 689.45,1059.07 689.678,1057.77 689.906,1058.63 690.133,1057.5 690.361,1058.6 690.589,1057.8 690.817,1057.65 691.045,1057.63 691.272,1057.15 691.5,1055.55 691.728,1057.01 691.956,1057.7 692.183,1057.8 692.411,1057.52 692.639,1057.09 692.867,1056.73 693.094,1056.96 693.322,1058.87 693.55,1059.92 693.778,1059.8 694.006,1059.25 694.233,1059.14 694.461,1058.14 694.689,1058.29 694.917,1057.86 695.144,1053.55 695.372,1055.38 695.6,1052.36 695.828,1053.35 696.055,1053.16 696.283,1053.03 696.511,1052.39 696.739,1052.98 696.967,1052.33 697.194,1050.54 697.422,1047.13 697.65,1047.46 697.878,1045.15 698.105,1046.72 698.333,1045.13 698.561,1045.25 698.789,1043.5 699.017,1043.94 699.244,1044.82 699.472,1045.51 699.7,1043.57 699.928,1044.33 700.155,1046.1 700.383,1047.66 700.611,1050.05 700.839,1047.85 701.066,1047.07 701.294,1047.74 701.522,1045.21 701.75,1046.84 701.978,1047.26 702.205,1044.55 702.433,1045.01 702.661,1044.54 702.889,1045.31 703.116,1046.76 703.344,1046.43 703.572,1045.92 703.8,1046.12 704.027,1046.01 704.255,1044.01 704.483,1044.71 704.711,1044.86 704.939,1044.3 705.166,1044.5 705.394,1045.74 705.622,1047.86 705.85,1048.72 706.077,1048.96 706.305,1049.03 706.533,1048.46 706.761,1047.4 706.988,1046.84 707.216,1048.06 707.444,1046.42 707.672,1044.86 707.9,1044.1 708.127,1045.05 708.355,1045.43 708.583,1045.54 708.811,1045.64 709.038,1046.75 709.266,1044.22 709.494,1044.17 709.722,1043.64 709.95,1044.23 710.177,1045.04 710.405,1047.32 710.633,1047.12 710.861,1048.1 711.088,1047.55 711.316,1049.49 711.544,1049.91 711.772,1051.98 711.999,1050.08 712.227,1051.22 712.455,1051.87 712.683,1054.67 712.911,1054.55 713.138,1054.7 713.366,1056 713.594,1056.81 713.822,1057.16 714.049,1059.03 714.277,1059.44 714.505,1058.43 714.733,1058.29 714.96,1056.16 715.188,1055.42 715.416,1055.51 715.644,1056.68 715.872,1058.23 716.099,1060.45 716.327,1060.21 716.555,1058.93 716.783,1059.24 717.01,1060.01 717.238,1059.63 717.466,1058.99 717.694,1060.14 717.921,1059.1 718.149,1058.64 718.377,1058.49 718.605,1058.92 718.833,1056.08 719.06,1055.49 719.288,1050.74 719.516,1048.58 719.744,1047.41 719.971,1048.51 720.199,1048.8 720.427,1049.11 720.655,1049.08 720.883,1050.75 721.11,1053.48 721.338,1052.42 721.566,1048.58 721.794,1046.4 722.021,1046.62 722.249,1047.69 722.477,1047.75 722.705,1048.82 722.932,1049.45 723.16,1050.56 723.388,1052.92 723.616,1053.5 723.844,1055.74 724.071,1056.34 724.299,1057.45 724.527,1058.79 724.755,1059.56 724.982,1057.79 725.21,1058.81 725.438,1060.34 725.666,1059.99 725.893,1062.34 726.121,1064.06 726.349,1065.56 726.577,1063.9 726.805,1063.91 727.032,1065.43 727.26,1065.56 727.488,1065.75 727.716,1065.03 727.943,1064.39 728.171,1064.87 728.399,1066.64 728.627,1064.24 728.854,1063 729.082,1063.67 729.31,1065.61 729.538,1065.67 729.766,1066.61 729.993,1066.49 730.221,1066.92 730.449,1067.28 730.677,1065.57 730.904,1064.83 731.132,1064.75 731.36,1063.5 731.588,1063.49 731.816,1064.3 732.043,1064.56 732.271,1067.31 732.499,1066.27 732.727,1068.13 732.954,1070.56 733.182,1070.01 733.41,1071.85 733.638,1074.49 733.865,1072.53 734.093,1073 734.321,1074.34 734.549,1074.25 734.777,1073.92 735.004,1072.31 735.232,1071.06 735.46,1069.71 735.688,1068.25 735.915,1067.94 736.143,1066.26 736.371,1066.12 736.599,1067.8 736.826,1068.34 737.054,1068.29 737.282,1069.42 737.51,1070.63 737.738,1068.61 737.965,1067.98 738.193,1066.54 738.421,1068.73 738.649,1069.25 738.876,1071.03 739.104,1070.27 739.332,1070.96 739.56,1072.71 739.787,1069.55 740.015,1070.41 740.243,1067.62 740.471,1067.96 740.699,1067.94 740.926,1065.6 741.154,1065.45 741.382,1064.38 741.61,1065.01 741.837,1066.89 742.065,1068.71 742.293,1069.73 742.521,1071.36 742.749,1070.14 742.976,1070.29 743.204,1071.52 743.432,1072.93 743.66,1071.84 743.887,1073.83 744.115,1071.99 744.343,1072.62 744.571,1073.44 744.798,1075.41 745.026,1074.99 745.254,1073.56 745.482,1073.93 745.71,1073.45 745.937,1073.43 746.165,1071.03 746.393,1071.34 746.621,1072.01 746.848,1070.52 747.076,1071.11 747.304,1069.05 747.532,1069.93 747.759,1069.42 747.987,1067.63 748.215,1067.69 748.443,1068.17 748.671,1066.91 748.898,1067.39 749.126,1067.62 749.354,1067.53 749.582,1066.21 749.809,1064.91 750.037,1065.48 750.265,1065.15 750.493,1064.66 750.72,1066.67 750.948,1063.84 751.176,1063.01 751.404,1062.32 751.632,1061.85 751.859,1061.51 752.087,1061.57 752.315,1062.97 752.543,1060.86 752.77,1061.22 752.998,1059.08 753.226,1058.95 753.454,1060.37 753.682,1061.13 753.909,1061.33 754.137,1060.48 754.365,1061.98 754.593,1062.11 754.82,1061.71 755.048,1062.5 755.276,1064.07 755.504,1061.69 755.731,1062.21 755.959,1063.71 756.187,1062.06 756.415,1062.98 756.643,1063 756.87,1062.32 757.098,1061.34 757.326,1062.06 757.554,1059.61 757.781,1060.83 758.009,1062.2 758.237,1061.79 758.465,1061.75 758.692,1059.64 758.92,1057.53 759.148,1057.62 759.376,1058.72 759.604,1058.33 759.831,1058.93 760.059,1058.38 760.287,1059.13 760.515,1060.47 760.742,1062.2 760.97,1064.01 761.198,1065.11 761.426,1065.61 761.653,1066.38 761.881,1065.43 762.109,1063.62 762.337,1065.59 762.565,1067.4 762.792,1064.2 763.02,1063.83 763.248,1066.33 763.476,1065.03 763.703,1064.84 763.931,1062.61 764.159,1064.32 764.387,1063.19 764.615,1062.31 764.842,1061.58 765.07,1060.29 765.298,1061.35 765.526,1062.89 765.753,1063.17 765.981,1062.72 766.209,1062.6 766.437,1062.48 766.664,1064.54 766.892,1063.48 767.12,1062.85 767.348,1062.18 767.576,1061.04 767.803,1061.51 768.031,1062.6 768.259,1064.47 768.487,1063.7 768.714,1062.72 768.942,1063.71 769.17,1065.48 769.398,1067.67 769.625,1067.53 769.853,1067.67 770.081,1068.49 770.309,1069.59 770.537,1068.96 770.764,1066.72 770.992,1066.86 771.22,1066.76 771.448,1065.19 771.675,1065.58 771.903,1067.43 772.131,1066.24 772.359,1065.3 772.586,1065.48 772.814,1067.52 773.042,1069.79 773.27,1067.58 773.498,1068.97 773.725,1070.59 773.953,1070.21 774.181,1070.17 774.409,1072.22 774.636,1073.4 774.864,1074.13 775.092,1073.95 775.32,1074.96 775.548,1074.52 775.775,1073.31 776.003,1072.69 776.231,1073.98 776.459,1073.47 776.686,1075.43 776.914,1075.84 777.142,1075.25 777.37,1074.93 777.597,1076.46 777.825,1079.08 778.053,1076.42 778.281,1075.34 778.509,1073.11 778.736,1073.25 778.964,1070.88 779.192,1070.44 779.42,1068.95 779.647,1068.23 779.875,1069.31 780.103,1069.02 780.331,1069.42 780.558,1066.45 780.786,1064.2 781.014,1064.23 781.242,1063.59 781.47,1063.01 781.697,1061.14 781.925,1062.13 782.153,1064.2 782.381,1064.56 782.608,1062.93 782.836,1062.2 783.064,1063.02 783.292,1062.16 783.519,1060.41 783.747,1060.68 783.975,1059.28 784.203,1059.43 784.431,1060.64 784.658,1060.87 784.886,1059.8 785.114,1060.87 785.342,1060.61 785.569,1059.54 785.797,1059.51 786.025,1060.7 786.253,1058.1 786.481,1057.74 786.708,1056.04 786.936,1054.65 787.164,1056.6 787.392,1056.19 787.619,1056.44 787.847,1054.23 788.075,1054.83 788.303,1054.45 788.53,1053.08 788.758,1054.74 788.986,1054.69 789.214,1054.24 789.442,1055.84 789.669,1055.16 789.897,1055.71 790.125,1054.36 790.353,1054.58 790.58,1051.83 790.808,1054.01 791.036,1054.71 791.264,1056.39 791.491,1056.96 791.719,1057.27 791.947,1057.44 792.175,1056.95 792.403,1056.83 792.63,1057.23 792.858,1058.33 793.086,1057.06 793.314,1057.37 793.541,1057.64 793.769,1059.64 793.997,1060.48 794.225,1059.77 794.452,1060.21 794.68,1060.16 794.908,1057.5 795.136,1059.82 795.364,1059.41 795.591,1058.12 795.819,1058.17 796.047,1055.64 796.275,1056.9 796.502,1055.01 796.73,1055.18 796.958,1056.53 797.186,1057.35 797.413,1058.5 797.641,1057.26 797.869,1057.52 798.097,1058.84 798.325,1059.97 798.552,1060.39 798.78,1062.02 799.008,1062.69 799.236,1061.36 799.463,1061.02 799.691,1061.1 799.919,1060.79 800.147,1059.75 800.375,1060.24 800.602,1060.47 800.83,1060.79 801.058,1060.82 801.286,1061.09 801.513,1061.17 801.741,1062.49 801.969,1062.64 802.197,1060.89 802.424,1060.34 802.652,1060.66 802.88,1060.03 803.108,1060.47 803.336,1061.08 803.563,1060.57 803.791,1060.3 804.019,1060.82 804.247,1060.34 804.474,1059.54 804.702,1061.12 804.93,1063.31 805.158,1063.6 805.385,1063.32 805.613,1061.73 805.841,1059.78 806.069,1060.28 806.297,1058.4 806.524,1059.66 806.752,1058.5 806.98,1058.38 807.208,1056.95 807.435,1055.74 807.663,1055.35 807.891,1055.1 808.119,1054.78 808.346,1053.62 808.574,1053.78 808.802,1052.78 809.03,1053.07 809.258,1050.92 809.485,1049.19 809.713,1046.11 809.941,1047.43 810.169,1048.4 810.396,1047.15 810.624,1048.03 810.852,1049.01 811.08,1050.5 811.308,1052.35 811.535,1051.06 811.763,1052.06 811.991,1053.31 812.219,1054.47 812.446,1054.35 812.674,1055.58 812.902,1053.97 813.13,1055.37 813.357,1053.53 813.585,1052.8 813.813,1052.46 814.041,1053.44 814.269,1055.89 814.496,1056.3 814.724,1056.34 814.952,1057.67 815.18,1057.84 815.407,1057.83 815.635,1059.58 815.863,1060.68 816.091,1060.09 816.318,1060.83 816.546,1059.87 816.774,1060.21 817.002,1061.08 817.23,1061.14 817.457,1061.67 817.685,1061.54 817.913,1060.02 818.141,1061.27 818.368,1062.11 818.596,1062.06 818.824,1060.91 819.052,1062.21 819.279,1062.31 819.507,1061.31 819.735,1061.02 819.963,1062.44 820.191,1060.54 820.418,1059.83 820.646,1060.89 820.874,1060.2 821.102,1059.3 821.329,1058.97 821.557,1057.31 821.785,1056.25 822.013,1055.58 822.241,1055.28 822.468,1054.49 822.696,1054.67 822.924,1055.42 823.152,1055.4 823.379,1057.73 823.607,1057.44 823.835,1057.16 824.063,1057.2 824.29,1056.12 824.518,1056.24 824.746,1056.05 824.974,1054.87 825.202,1054.52 825.429,1052.68 825.657,1053.24 825.885,1053.88 826.113,1052.16 826.34,1052.78 826.568,1052.44 826.796,1053.17 827.024,1052.75 827.251,1052.79 827.479,1052.49 827.707,1052.62 827.935,1053.89 828.163,1055.18 828.39,1056.1 828.618,1056.08 828.846,1056.82 829.074,1058.15 829.301,1058.29 829.529,1059.12 829.757,1061.03 829.985,1061.35 830.212,1061.69 830.44,1061.36 830.668,1060.68 830.896,1060.7 831.124,1061.3 831.351,1061.1 831.579,1060.13 831.807,1059.4 832.035,1059.5 832.262,1058.12 832.49,1058.21 832.718,1058.76 832.946,1058.23 833.174,1058.39 833.401,1059.73 833.629,1057.51 833.857,1057.54 834.085,1057.97 834.312,1057.25 834.54,1057.78 834.768,1057.41 834.996,1057.56 835.223,1057.22 835.451,1057.64 835.679,1056.58 835.907,1056.74 836.135,1055.41 836.362,1055.48 836.59,1056.11 836.818,1054.65 837.046,1054.57 837.273,1054.84 837.501,1054.77 837.729,1056.51 837.957,1057.96 838.184,1058.85 838.412,1058.79 838.64,1058.28 838.868,1057.77 839.096,1058.8 839.323,1059.5 839.551,1059.82 839.779,1061.36 840.007,1059.9 840.234,1059.35 840.462,1059.74 840.69,1060.07 840.918,1058.61 841.145,1059.51 841.373,1059.95 841.601,1058.91 841.829,1060.4 842.057,1062.06 842.284,1062.68 842.512,1061.8 842.74,1060.63 842.968,1059.08 843.195,1057.15 843.423,1057.67 843.651,1057.66 843.879,1057.76 844.107,1057.75 844.334,1057.58 844.562,1058.63 844.79,1058.84 845.018,1058.91 845.245,1056.09 845.473,1057 845.701,1057.91 845.929,1057.53 846.156,1056.8 846.384,1056.67 846.612,1058.07 846.84,1059.84 847.068,1060.23 847.295,1061.08 847.523,1060.68 847.751,1059.07 847.979,1056.88 848.206,1057.54 848.434,1057.84 848.662,1056.99 848.89,1056.56 849.117,1056.85 849.345,1057.55 849.573,1058.42 849.801,1058.82 850.029,1058.8 850.256,1057.92 850.484,1056.54 850.712,1057.42 850.94,1056.73 851.167,1056.89 851.395,1057.04 851.623,1058.5 851.851,1059.32 852.078,1058.92 852.306,1060.65 852.534,1059.37 852.762,1060.09 852.99,1059.61 853.217,1060.17 853.445,1060.12 853.673,1060.97 853.901,1061 854.128,1060.89 854.356,1059.72 854.584,1058.18 854.812,1058.37 855.04,1058.1 855.267,1059.24 855.495,1057.19 855.723,1056.74 855.951,1057.93 856.178,1059.92 856.406,1059.23 856.634,1058.1 856.862,1057.98 857.089,1058.37 857.317,1060.12 857.545,1061.26 857.773,1062.04 858.001,1061.81 858.228,1061.36 858.456,1058.19 858.684,1060.03 858.912,1060.45 859.139,1059.49 859.367,1058.93 859.595,1058.72 859.823,1060.05 860.05,1059.12 860.278,1059.87 860.506,1059.82 860.734,1061.27 860.962,1060.76 861.189,1061.49 861.417,1061.89 861.645,1060.26 861.873,1061.25 862.1,1061.11 862.328,1061.7 862.556,1061.84 862.784,1061.52 863.011,1060.28 863.239,1060.13 863.467,1059.47 863.695,1059.1 863.923,1059.83 864.15,1061.67 864.378,1059.97 864.606,1058.44 864.834,1058.88 865.061,1059.74 865.289,1059.5 865.517,1059.07 865.745,1059.81 865.973,1059.5 866.2,1060 866.428,1059.46 866.656,1060.2 866.884,1060.97 867.111,1060.3 867.339,1060.76 867.567,1061.73 867.795,1061.69 868.022,1062.06 868.25,1061.97 868.478,1062.28 868.706,1063.11 868.934,1061 869.161,1059.11 869.389,1058.43 869.617,1057.04 869.845,1056.69 870.072,1056.38 870.3,1057.13 870.528,1056.86 870.756,1057.48 870.983,1058.6 871.211,1058.68 871.439,1060.34 871.667,1060.22 871.895,1062.1 872.122,1061.23 872.35,1061.51 872.578,1060.73 872.806,1059.27 873.033,1060.27 873.261,1060.53 873.489,1062.16 873.717,1062.09 873.944,1062.11 874.172,1060.65 874.4,1059.99 874.628,1059.55 874.856,1059.02 875.083,1059.08 875.311,1061.26 875.539,1061.87 875.767,1062.83 875.994,1062.53 876.222,1060.77 876.45,1059.69 876.678,1058.73 876.906,1059.58 877.133,1059.48 877.361,1059.84 877.589,1060.46 877.817,1060.03 878.044,1059.72 878.272,1060.14 878.5,1059.51 878.728,1058.83 878.955,1060.07 879.183,1059.95 879.411,1059.1 879.639,1059.08 879.867,1060.61 880.094,1061.16 880.322,1062.17 880.55,1061.75 880.778,1062.12 881.005,1063.17 881.233,1064.93 881.461,1065.11 881.689,1065.75 881.916,1064.75 882.144,1063.57 882.372,1062.04 882.6,1061.4 882.828,1060.1 883.055,1059.66 883.283,1059.25 883.511,1056.85 883.739,1055.76 883.966,1056.86 884.194,1057.78 884.422,1058.99 884.65,1059.97 884.877,1059.64 885.105,1059.77 885.333,1061.11 885.561,1063.01 885.789,1064.53 886.016,1063.97 886.244,1062.86 886.472,1061.47 886.7,1061.93 886.927,1062.89 887.155,1063.38 887.383,1064.9 887.611,1065.86 887.839,1067.17 888.066,1068.54 888.294,1069.05 888.522,1068.78 888.75,1068.36 888.977,1070.24 889.205,1069.64 889.433,1070.45 889.661,1071.25 889.888,1071.44 890.116,1071.7 890.344,1072.15 890.572,1071.9 890.8,1070.96 891.027,1071.13 891.255,1068.25 891.483,1067.89 891.711,1067.84 891.938,1067.45 892.166,1067.82 892.394,1067.11 892.622,1067.31 892.849,1067.84 893.077,1065.74 893.305,1065.24 893.533,1064.58 893.761,1064.15 893.988,1064.6 894.216,1064.64 894.444,1065.07 894.672,1065.33 894.899,1066.46 895.127,1065.3 895.355,1065.57 895.583,1066.71 895.81,1065.4 896.038,1065.25 896.266,1065.72 896.494,1067.25 896.722,1068.11 896.949,1066.66 897.177,1066.52 897.405,1064.59 897.633,1064.18 897.86,1063.21 898.088,1063.42 898.316,1064.53 898.544,1064.84 898.772,1064.28 898.999,1063.56 899.227,1062.82 899.455,1063.91 899.683,1066.02 899.91,1065.11 900.138,1065.38 900.366,1064.24 900.594,1061.94 900.821,1062.6 901.049,1064.17 901.277,1063.43 901.505,1063.93 901.733,1065.21 901.96,1065.98 902.188,1066.29 902.416,1067.16 902.644,1066.6 902.871,1066.39 903.099,1067.37 903.327,1068.84 903.555,1069.95 903.782,1070.27 904.01,1070.23 904.238,1070.12 904.466,1069.58 904.694,1069.16 904.921,1070.55 905.149,1071.28 905.377,1070.84 905.605,1071.08 905.832,1070.56 906.06,1070.47 906.288,1070.15 906.516,1070.16 906.743,1071.36 906.971,1071.69 907.199,1072.95 907.427,1074.51 907.655,1076.46 907.882,1077.26 908.11,1077.1 908.338,1074.57 908.566,1073.25 908.793,1074.54 909.021,1074.2 909.249,1073.48 909.477,1074.31 909.705,1074.11 909.932,1075.75 910.16,1074.53 910.388,1074.34 910.616,1074.44 910.843,1075.51 911.071,1076.17 911.299,1076.61 911.527,1075.94 911.754,1076.8 911.982,1077.86 912.21,1078.11 912.438,1078.3 912.666,1079.43 912.893,1079.08 913.121,1078.89 913.349,1079.03 913.577,1079.69 913.804,1081.02 914.032,1080.75 914.26,1080.08 914.488,1081.17 914.715,1080.97 914.943,1080.79 915.171,1079.4 915.399,1078.9 915.627,1079.04 915.854,1078.3 916.082,1079.16 916.31,1079.2 916.538,1079.77 916.765,1080.17 916.993,1079.22 917.221,1077.43 917.449,1077.1 917.676,1076.11 917.904,1075.62 918.132,1075.92 918.36,1075.4 918.588,1075.18 918.815,1072.81 919.043,1072.61 919.271,1074.1 919.499,1073.95 919.726,1073.92 919.954,1074.27 920.182,1075.4 920.41,1075.72 920.638,1073.59 920.865,1073.86 921.093,1074.57 921.321,1076.08 921.549,1075.03 921.776,1075.66 922.004,1075.9 922.232,1076.29 922.46,1076.05 922.687,1075.97 922.915,1076.46 923.143,1076.09 923.371,1075.88 923.599,1077.11 923.826,1078.24 924.054,1079.69 924.282,1079.93 924.51,1080.43 924.737,1079.31 924.965,1079.93 925.193,1079.29 925.421,1078.64 925.648,1079.43 925.876,1078.6 926.104,1077.87 926.332,1079.01 926.56,1077.82 926.787,1077.2 927.015,1077.56 927.243,1077.39 927.471,1078.83 927.698,1079.23 927.926,1077.85 928.154,1079.02 928.382,1077.97 928.609,1075.42 928.837,1076.39 929.065,1077.08 929.293,1077.34 929.521,1077.55 929.748,1077.95 929.976,1079.59 930.204,1080.54 930.432,1079.71 930.659,1080.64 930.887,1079.61 931.115,1079.37 931.343,1077.31 931.571,1076.4 931.798,1077.5 932.026,1078.58 932.254,1079.16 932.482,1078.28 932.709,1076.46 932.937,1078.17 933.165,1078.09 933.393,1078.41 933.62,1079.63 933.848,1080.02 934.076,1080.68 934.304,1080.74 934.532,1080.95 934.759,1079.91 934.987,1080.28 935.215,1080.12 935.443,1077.07 935.67,1077.71 935.898,1078.24 936.126,1077.94 936.354,1075.92 936.581,1075.59 936.809,1075.35 937.037,1075.14 937.265,1076.51 937.493,1075.66 937.72,1074.93 937.948,1073.69 938.176,1072.26 938.404,1071.6 938.631,1071.64 938.859,1072.74 939.087,1072.01 939.315,1072.7 939.542,1072.29 939.77,1073 939.998,1072.83 940.226,1073.88 940.454,1072.99 940.681,1071.71 940.909,1072.41 941.137,1071.57 941.365,1070.87 941.592,1070.47 941.82,1070.71 942.048,1070.99 942.276,1070.77 942.504,1070.6 942.731,1071.25 942.959,1071.02 943.187,1072.54 943.415,1070.46 943.642,1070.68 943.87,1071.36 944.098,1069.37 944.326,1071.04 944.553,1069.6 944.781,1069.32 945.009,1070.27 945.237,1070.63 945.465,1069.16 945.692,1067.34 945.92,1066.68 946.148,1067.06 946.376,1067.02 946.603,1066.9 946.831,1064.86 947.059,1065.91 947.287,1065.61 947.514,1064.76 947.742,1065.11 947.97,1065.5 948.198,1065.46 948.426,1064.98 948.653,1064.18 948.881,1062.95 949.109,1060.35 949.337,1060.16 949.564,1060.64 949.792,1059.56 950.02,1059.23 950.248,1059.45 950.475,1057.95 950.703,1057.53 950.931,1058.43 951.159,1058.67 951.387,1059.28 951.614,1059.58 951.842,1059.39 952.07,1058.81 952.298,1059.03 952.525,1058.17 952.753,1057.27 952.981,1057.07 953.209,1058.58 953.437,1058.19 953.664,1058.82 953.892,1057.96 954.12,1057.96 954.348,1056.62 954.575,1056.84 954.803,1056.82 955.031,1057.46 955.259,1057.3 955.486,1056.28 955.714,1056.69 955.942,1057.79 956.17,1058.15 956.398,1056.8 956.625,1057.36 956.853,1057.72 957.081,1057.17 957.309,1055.2 957.536,1055.67 957.764,1056.73 957.992,1057.39 958.22,1057.94 958.447,1056.49 958.675,1054.82 958.903,1055.17 959.131,1054.87 959.359,1054.75 959.586,1053.62 959.814,1053.52 960.042,1054.14 960.27,1054.46 960.497,1054.14 960.725,1055.02 960.953,1054.53 961.181,1054.06 961.408,1053.45 961.636,1052.66 961.864,1053.21 962.092,1052.63 962.32,1051.75 962.547,1052.37 962.775,1053.21 963.003,1053.19 963.231,1052.85 963.458,1053.19 963.686,1052.3 963.914,1052.96 964.142,1053.06 964.37,1053.9 964.597,1053.22 964.825,1053.2 965.053,1053.05 965.281,1053.03 965.508,1053.27 965.736,1053.36 965.964,1052.75 966.192,1053.33 966.419,1054.06 966.647,1053.4 966.875,1054.32 967.103,1053.73 967.331,1053.97 967.558,1054.42 967.786,1054.96 968.014,1054.21 968.242,1054.85 968.469,1056.16 968.697,1055.18 968.925,1054.54 969.153,1055.18 969.38,1054.37 969.608,1054.95 969.836,1054.9 970.064,1056.15 970.292,1056.61 970.519,1056.19 970.747,1056.06 970.975,1055.93 971.203,1055.39 971.43,1055.8 971.658,1056.8 971.886,1054.8 972.114,1055.18 972.341,1056.47 972.569,1056.08 972.797,1055.35 973.025,1053.43 973.253,1053.8 973.48,1054.5 973.708,1055.92 973.936,1058.25 974.164,1058.77 974.391,1058.32 974.619,1059.01 974.847,1057.14 975.075,1058.02 975.303,1058.08 975.53,1056.26 975.758,1054.17 975.986,1052.52 976.214,1052.51 976.441,1052.48 976.669,1052.98 976.897,1054.44 977.125,1051.94 977.352,1052.56 977.58,1052.11 977.808,1052.01 978.036,1051.85 978.264,1051.61 978.491,1051.65 978.719,1051.49 978.947,1051.55 979.175,1052.1 979.402,1053.01 979.63,1052.55 979.858,1052.97 980.086,1052.97 980.313,1053.53 980.541,1055.03 980.769,1054.79 980.997,1051.27 981.225,1050.67 981.452,1051.9 981.68,1052.27 981.908,1050.92 982.136,1051.34 982.363,1052.45 982.591,1052.12 982.819,1053.39 983.047,1053.56 983.274,1054.73 983.502,1055.39 983.73,1055.71 983.958,1055.47 984.186,1055.24 984.413,1055.18 984.641,1056.42 984.869,1057.63 985.097,1056.01 985.324,1057.04 985.552,1057.38 985.78,1057 986.008,1056.59 986.236,1055.36 986.463,1054.95 986.691,1055.1 986.919,1055.92 987.147,1054.05 987.374,1054.64 987.602,1055.18 987.83,1055.12 988.058,1056.66 988.285,1056.8 988.513,1057.56 988.741,1057.35 988.969,1056.34 989.197,1057.08 989.424,1058.08 989.652,1057.96 989.88,1059.01 990.108,1060.24 990.335,1060.1 990.563,1061.06 990.791,1059.8 991.019,1060.77 991.246,1061.15 991.474,1059.44 991.702,1058.87 991.93,1058.57 992.158,1057.83 992.385,1059.06 992.613,1059.88 992.841,1058.61 993.069,1058.85 993.296,1057.61 993.524,1057.56 993.752,1056.43 993.98,1056.32 994.207,1056.24 994.435,1054.3 994.663,1053.48 994.891,1053.06 995.119,1051.86 995.346,1052.51 995.574,1052.09 995.802,1052.33 996.03,1052.99 996.257,1052.21 996.485,1053.1 996.713,1052.62 996.941,1051.89 997.169,1051.94 997.396,1052.34 997.624,1052.44 997.852,1052.18 998.08,1053.47 998.307,1054.72 998.535,1055 998.763,1056.17 998.991,1056.88 999.218,1058.97 999.446,1059.28 999.674,1060.26 999.902,1058.16 1000.13,1058.49 1000.36,1059 1000.59,1059.03 1000.81,1060.05 1001.04,1060.05 1001.27,1060.95 1001.5,1062.55 1001.72,1062.58 1001.95,1063.35 1002.18,1062.87 1002.41,1063.76 1002.64,1064 1002.86,1062.46 1003.09,1062.67 1003.32,1063.29 1003.55,1064.31 1003.77,1064.74 1004,1065.26 1004.23,1064.82 1004.46,1066.47 1004.68,1067.49 1004.91,1066.37 1005.14,1066.84 1005.37,1066.38 1005.6,1065.64 1005.82,1064.85 1006.05,1065.49 1006.28,1065.11 1006.51,1064.86 1006.73,1064.45 1006.96,1064.96 1007.19,1064.92 1007.42,1064.44 1007.65,1063.98 1007.87,1064.51 1008.1,1064.98 1008.33,1064.96 1008.56,1065.15 1008.78,1066.54 1009.01,1067.13 1009.24,1066.72 1009.47,1066.21 1009.7,1066.68 1009.92,1066.22 1010.15,1066.09 1010.38,1065.8 1010.61,1066.8 1010.83,1067.56 1011.06,1068.12 1011.29,1068.41 1011.52,1068.4 1011.75,1069.25 1011.97,1070.34 1012.2,1070.83 1012.43,1071.9 1012.66,1072.67 1012.88,1072.55 1013.11,1073.15 1013.34,1072.54 1013.57,1071.96 1013.8,1069.74 1014.02,1070.88 1014.25,1070.84 1014.48,1072.55 1014.71,1072.21 1014.93,1071.99 1015.16,1072.5 1015.39,1073.29 1015.62,1073.44 1015.85,1073.86 1016.07,1073.57 1016.3,1073.77 1016.53,1073.95 1016.76,1073.95 1016.98,1073.91 1017.21,1073.84 1017.44,1073.92 1017.67,1073.66 1017.9,1073.69 1018.12,1071.84 1018.35,1071.48 1018.58,1070.99 1018.81,1071.66 1019.03,1072.35 1019.26,1071.43 1019.49,1072.47 1019.72,1073.32 1019.95,1074.63 1020.17,1075.61 1020.4,1075.5 1020.63,1075.84 1020.86,1075.39 1021.08,1074.36 1021.31,1073.6 1021.54,1075.11 1021.77,1074.29 1022,1073.67 1022.22,1073.84 1022.45,1074.04 1022.68,1073.98 1022.91,1074.19 1023.13,1074.85 1023.36,1074.82 1023.59,1074.96 1023.82,1075.52 1024.05,1076.9 1024.27,1076.18 1024.5,1076.02 1024.73,1075.73 1024.96,1076.82 1025.18,1076.18 1025.41,1076.27 1025.64,1075.84 1025.87,1076.05 1026.1,1077.42 1026.32,1078.36 1026.55,1078.26 1026.78,1078.55 1027.01,1079.37 1027.23,1077.71 1027.46,1077.93 1027.69,1078.44 1027.92,1078.91 1028.15,1078.94 1028.37,1078.79 1028.6,1077.88 1028.83,1079.11 1029.06,1077.97 1029.28,1077.54 1029.51,1077.9 1029.74,1078.13 1029.97,1078 1030.2,1077.04 1030.42,1077.35 1030.65,1077.92 1030.88,1077.92 1031.11,1078.39 1031.33,1079.17 1031.56,1079.64 1031.79,1078.5 1032.02,1078.22 1032.25,1077.32 1032.47,1077.3 1032.7,1075.86 1032.93,1075.11 1033.16,1074.96 1033.38,1074.58 1033.61,1075.39 1033.84,1075.96 1034.07,1074.47 1034.3,1075.68 1034.52,1075.3 1034.75,1075.56 1034.98,1075.61 1035.21,1076.93 1035.43,1076.75 1035.66,1075.56 1035.89,1076.5 1036.12,1075.7 1036.35,1075.91 1036.57,1074.42 1036.8,1074.59 1037.03,1073.16 1037.26,1072.24 1037.48,1071.03 1037.71,1072 1037.94,1072.83 1038.17,1071.15 1038.4,1070.56 1038.62,1072.03 1038.85,1070.03 1039.08,1068.97 1039.31,1068.75 1039.53,1069.77 1039.76,1069.1 1039.99,1070.39 1040.22,1070.03 1040.44,1068.74 1040.67,1067.99 1040.9,1068.59 1041.13,1068.91 1041.36,1068.11 1041.58,1068.55 1041.81,1069.53 1042.04,1069.92 1042.27,1070.26 1042.49,1071.3 1042.72,1070.72 1042.95,1070.85 1043.18,1070.98 1043.41,1071.97 1043.63,1071.1 1043.86,1070.19 1044.09,1070.01 1044.32,1069.91 1044.54,1070.01 1044.77,1070.54 1045,1071.18 1045.23,1071.56 1045.46,1071.22 1045.68,1071.68 1045.91,1071.9 1046.14,1073.27 1046.37,1072.37 1046.59,1069.97 1046.82,1070.86 1047.05,1071.55 1047.28,1072.73 1047.51,1072.91 1047.73,1072.23 1047.96,1073.56 1048.19,1073.9 1048.42,1073.2 1048.64,1073.56 1048.87,1074.13 1049.1,1073.96 1049.33,1071.64 1049.56,1071.9 1049.78,1071.76 1050.01,1072.52 1050.24,1072.87 1050.47,1072.35 1050.69,1072.67 1050.92,1072.89 1051.15,1073.25 1051.38,1073.43 1051.61,1074.57 1051.83,1074.55 1052.06,1074.36 1052.29,1075.31 1052.52,1074.07 1052.74,1073.57 1052.97,1074 1053.2,1072.99 1053.43,1072.51 1053.66,1071.78 1053.88,1069.54 1054.11,1069.25 1054.34,1070.23 1054.57,1069.37 1054.79,1069.64 1055.02,1069.1 1055.25,1069.32 1055.48,1070.6 1055.71,1070.64 1055.93,1069.86 1056.16,1069.99 1056.39,1069.71 1056.62,1068.78 1056.84,1068.98 1057.07,1069.11 1057.3,1069.26 1057.53,1069.82 1057.76,1070.55 1057.98,1071.04 1058.21,1071.66 1058.44,1072.41 1058.67,1073.14 1058.89,1073.26 1059.12,1073.3 1059.35,1073.09 1059.58,1073.51 1059.81,1073.03 1060.03,1074.43 1060.26,1074.75 1060.49,1074.67 1060.72,1074.86 1060.94,1075.65 1061.17,1075.76 1061.4,1075.1 1061.63,1074.92 1061.86,1075.44 1062.08,1077.06 1062.31,1076.41 1062.54,1076.64 1062.77,1077.27 1062.99,1078.2 1063.22,1077.84 1063.45,1078.55 1063.68,1078.24 1063.91,1078.53 1064.13,1079.14 1064.36,1077.49 1064.59,1077.54 1064.82,1078.5 1065.04,1079.7 1065.27,1079.62 1065.5,1080.12 1065.73,1079.69 1065.96,1079.99 1066.18,1079.68 1066.41,1079.29 1066.64,1080.11 1066.87,1079.8 1067.09,1078.86 1067.32,1077.3 1067.55,1078.11 1067.78,1079.31 1068.01,1079.59 1068.23,1078.51 1068.46,1076.79 1068.69,1076.88 1068.92,1078.22 1069.14,1078.65 1069.37,1078.34 1069.6,1078.32 1069.83,1077.66 1070.06,1075.94 1070.28,1074.32 1070.51,1074.99 1070.74,1074.62 1070.97,1073.29 1071.19,1073.61 1071.42,1073.43 1071.65,1073.75 1071.88,1073.63 1072.11,1074.03 1072.33,1074.87 1072.56,1074.71 1072.79,1074.89 1073.02,1074.63 1073.24,1073.42 1073.47,1074.4 1073.7,1076.04 1073.93,1075.82 1074.16,1075.84 1074.38,1077.16 1074.61,1075.09 1074.84,1076.34 1075.07,1076.13 1075.29,1076.42 1075.52,1076.37 1075.75,1077.24 1075.98,1077.85 1076.2,1078.59 1076.43,1078.85 1076.66,1078.85 1076.89,1078.68 1077.12,1078.46 1077.34,1078.97 1077.57,1079.39 1077.8,1079.01 1078.03,1079.01 1078.25,1078.41 1078.48,1079.19 1078.71,1078.69 1078.94,1078.42 1079.17,1077.96 1079.39,1077.38 1079.62,1077.56 1079.85,1078.1 1080.08,1078.92 1080.3,1079.53 1080.53,1080.02 1080.76,1080.69 1080.99,1080.6 1081.22,1079.24 1081.44,1078.57 1081.67,1078.08 1081.9,1078.24 1082.13,1076.77 1082.35,1075.99 1082.58,1076.45 1082.81,1075.96 1083.04,1075.25 1083.27,1075.51 1083.49,1073.51 1083.72,1073.22 1083.95,1072.57 1084.18,1070.89 1084.4,1071.51 1084.63,1071.48 1084.86,1071 1085.09,1071.83 1085.32,1072.85 1085.54,1071.76 1085.77,1072.65 1086,1073.88 1086.23,1075.01 1086.45,1074.77 1086.68,1075.91 1086.91,1075.88 1087.14,1075.86 1087.37,1075.78 1087.59,1076.31 1087.82,1075.89 1088.05,1076.62 1088.28,1076.16 1088.5,1076.85 1088.73,1076.92 1088.96,1076.96 1089.19,1077.98 1089.42,1077.33 1089.64,1077.46 1089.87,1077.35 1090.1,1077.37 1090.33,1077.97 1090.55,1078.09 1090.78,1078.51 1091.01,1078.12 1091.24,1078.41 1091.47,1078.12 1091.69,1079.51 1091.92,1079.28 1092.15,1078.31 1092.38,1078.32 1092.6,1077.87 1092.83,1077.99 1093.06,1079.57 1093.29,1079.61 1093.52,1080.3 1093.74,1080.49 1093.97,1079.85 1094.2,1080.15 1094.43,1079.61 1094.65,1079.89 1094.88,1080.89 1095.11,1080.59 1095.34,1080.01 1095.57,1081.55 1095.79,1080.5 1096.02,1080.73 1096.25,1081.58 1096.48,1079.66 1096.7,1076.96 1096.93,1077.77 1097.16,1078.63 1097.39,1079.44 1097.62,1079.74 1097.84,1079.48 1098.07,1079.24 1098.3,1079.04 1098.53,1079.23 1098.75,1079.47 1098.98,1078.92 1099.21,1077.6 1099.44,1078.65 1099.67,1078.2 1099.89,1079.43 1100.12,1078.89 1100.35,1077.53 1100.58,1077.82 1100.8,1077.72 1101.03,1078.43 1101.26,1077.91 1101.49,1077.29 1101.72,1077.33 1101.94,1077.13 1102.17,1076.68 1102.4,1076.78 1102.63,1077.33 1102.85,1076.76 1103.08,1077.21 1103.31,1076.91 1103.54,1076.76 1103.77,1076.93 1103.99,1076.95 1104.22,1077.2 1104.45,1075.29 1104.68,1074.41 1104.9,1073.56 1105.13,1073.53 1105.36,1074.89 1105.59,1074.63 1105.82,1074.47 1106.04,1074.03 1106.27,1072.91 1106.5,1072.94 1106.73,1073.51 1106.95,1074.16 1107.18,1074.61 1107.41,1074.13 1107.64,1074.17 1107.87,1073.89 1108.09,1073.54 1108.32,1074.52 1108.55,1075.49 1108.78,1076.03 1109,1076.6 1109.23,1077.34 1109.46,1076.87 1109.69,1076.84 1109.92,1077.79 1110.14,1077.74 1110.37,1076.98 1110.6,1077.19 1110.83,1077.39 1111.05,1077.68 1111.28,1078.85 1111.51,1079.73 1111.74,1079.34 1111.97,1079.55 1112.19,1080.24 1112.42,1079.92 1112.65,1080.36 1112.88,1080.06 1113.1,1079.92 1113.33,1080.71 1113.56,1081.9 1113.79,1081.84 1114.01,1082.46 1114.24,1082.66 1114.47,1081.54 1114.7,1081.25 1114.93,1082.21 1115.15,1082.26 1115.38,1082.76 1115.61,1083.26 1115.84,1084.02 1116.06,1084.84 1116.29,1084.54 1116.52,1084.92 1116.75,1085.35 1116.98,1085.57 1117.2,1085.53 1117.43,1085.74 1117.66,1084.81 1117.89,1084.53 1118.11,1083.12 1118.34,1082.15 1118.57,1083.23 1118.8,1082.56 1119.03,1081.76 1119.25,1081.57 1119.48,1080.45 1119.71,1081.34 1119.94,1081.89 1120.16,1081.35 1120.39,1081.18 1120.62,1081.91 1120.85,1083.1 1121.08,1080.9 1121.3,1080.58 1121.53,1081.01 1121.76,1082.24 1121.99,1083.1 1122.21,1082.36 1122.44,1083.05 1122.67,1084.49 1122.9,1083.04 1123.13,1083.02 1123.35,1082.77 1123.58,1082.58 1123.81,1082.75 1124.04,1082 1124.26,1081.5 1124.49,1081.13 1124.72,1080.15 1124.95,1080.1 1125.18,1080 1125.4,1080.26 1125.63,1079.04 1125.86,1080.13 1126.09,1079.79 1126.31,1080.61 1126.54,1080.83 1126.77,1081.17 1127,1081.81 1127.23,1081.26 1127.45,1080.01 1127.68,1078.9 1127.91,1076.92 1128.14,1077.57 1128.36,1077.56 1128.59,1078.73 1128.82,1079.11 1129.05,1079.89 1129.28,1080.51 1129.5,1077.53 1129.73,1079.24 1129.96,1079.83 1130.19,1079.25 1130.41,1078.44 1130.64,1078.91 1130.87,1079.1 1131.1,1079.63 1131.33,1079.51 1131.55,1079.98 1131.78,1079.75 1132.01,1079.74 1132.24,1077.17 1132.46,1076.3 1132.69,1076.01 1132.92,1076.72 1133.15,1077.34 1133.38,1077.99 1133.6,1078.76 1133.83,1078.4 1134.06,1078.01 1134.29,1077.69 1134.51,1077.29 1134.74,1076.63 1134.97,1076.98 1135.2,1078.48 1135.43,1079.64 1135.65,1079.06 1135.88,1079.47 1136.11,1079.8 1136.34,1079.86 1136.56,1080.13 1136.79,1080.15 1137.02,1079.82 1137.25,1080.83 1137.48,1081.58 1137.7,1081.64 1137.93,1082 1138.16,1082 1138.39,1082.62 1138.61,1082.47 1138.84,1083.88 1139.07,1085.35 1139.3,1085.8 1139.53,1084.59 1139.75,1084.78 1139.98,1085.68 1140.21,1085.6 1140.44,1084.64 1140.66,1083.18 1140.89,1083.24 1141.12,1083.87 1141.35,1084.03 1141.58,1084.43 1141.8,1083.71 1142.03,1084.82 1142.26,1084.19 1142.49,1083.54 1142.71,1084.01 1142.94,1083.15 1143.17,1083.71 1143.4,1083.95 1143.63,1083.59 1143.85,1083.96 1144.08,1082.82 1144.31,1082.87 1144.54,1082.14 1144.76,1082.2 1144.99,1082.79 1145.22,1083.65 1145.45,1082.88 1145.68,1081.75 1145.9,1079.93 1146.13,1079.6 1146.36,1080.37 1146.59,1079.99 1146.81,1079.45 1147.04,1078.98 1147.27,1078.7 1147.5,1078.12 1147.73,1078.22 1147.95,1079.13 1148.18,1076.88 1148.41,1077.03 1148.64,1077.6 1148.86,1077.54 1149.09,1078.68 1149.32,1079.18 1149.55,1078.44 1149.77,1077.15 1150,1075.5 1150.23,1076.81 1150.46,1077.91 1150.69,1076.94 1150.91,1076.52 1151.14,1075.44 1151.37,1075.62 1151.6,1075.1 1151.82,1073.03 1152.05,1073.59 1152.28,1074.17 1152.51,1075.05 1152.74,1075.73 1152.96,1076.58 1153.19,1076.95 1153.42,1076.98 1153.65,1078.04 1153.87,1077.59 1154.1,1075.94 1154.33,1076.48 1154.56,1076.22 1154.79,1075.6 1155.01,1076.37 1155.24,1075.36 1155.47,1075.79 1155.7,1074.59 1155.92,1074.86 1156.15,1073.43 1156.38,1074.04 1156.61,1074.16 1156.84,1073.65 1157.06,1073.03 1157.29,1072.8 1157.52,1073.12 1157.75,1073.63 1157.97,1073.57 1158.2,1073.69 1158.43,1073.81 1158.66,1072.75 1158.89,1072.7 1159.11,1072.29 1159.34,1072.28 1159.57,1072.28 1159.8,1071.53 1160.02,1071.9 1160.25,1072.1 1160.48,1072.51 1160.71,1072.26 1160.94,1071.97 1161.16,1072.63 1161.39,1071.18 1161.62,1071.06 1161.85,1071.17 1162.07,1072.14 1162.3,1072.18 1162.53,1071.82 1162.76,1072.71 1162.99,1072 1163.21,1072.91 1163.44,1073.24 1163.67,1071.18 1163.9,1071.47 1164.12,1070.3 1164.35,1070.84 1164.58,1071.06 1164.81,1070.77 1165.04,1068.69 1165.26,1069.27 1165.49,1069.08 1165.72,1069.57 1165.95,1069.75 1166.17,1069.94 1166.4,1068.66 1166.63,1066.47 1166.86,1065.86 1167.09,1065.94 1167.31,1064.99 1167.54,1063.59 1167.77,1063.29 1168,1064.15 1168.22,1064.25 1168.45,1063.92 1168.68,1065.13 1168.91,1064.85 1169.14,1065.02 1169.36,1065.44 1169.59,1065.88 1169.82,1065.05 1170.05,1065.69 1170.27,1065.13 1170.5,1064.62 1170.73,1064.2 1170.96,1064.53 1171.19,1064.98 1171.41,1065.28 1171.64,1066.42 1171.87,1065.83 1172.1,1066.14 1172.32,1066.5 1172.55,1066.18 1172.78,1066.66 1173.01,1067.36 1173.24,1067.46 1173.46,1068 1173.69,1068.69 1173.92,1067.9 1174.15,1068.96 1174.37,1068.73 1174.6,1068.83 1174.83,1068.67 1175.06,1067.35 1175.29,1067.75 1175.51,1068.78 1175.74,1068.96 1175.97,1070.03 1176.2,1070.68 1176.42,1069.79 1176.65,1068.75 1176.88,1068.3 1177.11,1067.87 1177.34,1067.39 1177.56,1067.03 1177.79,1066.85 1178.02,1067.49 1178.25,1067.12 1178.47,1068.26 1178.7,1067.98 1178.93,1067.11 1179.16,1067.6 1179.39,1067.47 1179.61,1067.28 1179.84,1066.74 1180.07,1066.51 1180.3,1066.92 1180.52,1067.57 1180.75,1068.47 1180.98,1069.01 1181.21,1070.03 1181.44,1069.23 1181.66,1068.21 1181.89,1068.16 1182.12,1067.04 1182.35,1067.22 1182.57,1067.03 1182.8,1067 1183.03,1068.39 1183.26,1068.11 1183.49,1068.44 1183.71,1068.83 1183.94,1068.91 1184.17,1068.13 1184.4,1067.8 1184.62,1067.91 1184.85,1068.99 1185.08,1068.94 1185.31,1069.15 1185.53,1069.08 1185.76,1069.61 1185.99,1068.87 1186.22,1069.07 1186.45,1068.57 1186.67,1069.4 1186.9,1069.44 1187.13,1070.68 1187.36,1072.03 1187.58,1072.14 1187.81,1072.26 1188.04,1072.65 1188.27,1072.78 1188.5,1073.64 1188.72,1073.68 1188.95,1073.1 1189.18,1072.61 1189.41,1072.98 1189.63,1072.77 1189.86,1072.34 1190.09,1073.47 1190.32,1074.28 1190.55,1074.51 1190.77,1075.34 1191,1073.16 1191.23,1073.29 1191.46,1073.8 1191.68,1073.89 1191.91,1073.64 1192.14,1074.7 1192.37,1075 1192.6,1074.68 1192.82,1075.05 1193.05,1075.17 1193.28,1074.39 1193.51,1073.42 1193.73,1073.28 1193.96,1072.66 1194.19,1072.21 1194.42,1072.47 1194.65,1073.45 1194.87,1072.43 1195.1,1073.79 1195.33,1074.36 1195.56,1073.84 1195.78,1074.28 1196.01,1073.9 1196.24,1073.77 1196.47,1072.67 1196.7,1072.52 1196.92,1073.19 1197.15,1073.7 1197.38,1072.93 1197.61,1074.07 1197.83,1073.63 1198.06,1074.01 1198.29,1073.1 1198.52,1073.17 1198.75,1072.79 1198.97,1072.27 1199.2,1072.94 1199.43,1072.49 1199.66,1072.03 1199.88,1072.74 1200.11,1072.83 1200.34,1072.8 1200.57,1073.1 1200.8,1073.91 1201.02,1073.76 1201.25,1072.56 1201.48,1072.04 1201.71,1072.69 1201.93,1072.81 1202.16,1072.26 1202.39,1072.14 1202.62,1072.58 1202.85,1073.48 1203.07,1074 1203.3,1073.6 1203.53,1074.02 1203.76,1074.73 1203.98,1075.1 1204.21,1074.41 1204.44,1073.87 1204.67,1073.12 1204.9,1072.38 1205.12,1072.54 1205.35,1073.39 1205.58,1073.19 1205.81,1073.76 1206.03,1075.1 1206.26,1075.55 1206.49,1075.69 1206.72,1076.1 1206.95,1076.84 1207.17,1075.64 1207.4,1075.95 1207.63,1076.94 1207.86,1077.71 1208.08,1077.73 1208.31,1078.47 1208.54,1078.19 1208.77,1078.76 1209,1079.7 1209.22,1079.85 1209.45,1080.27 1209.68,1081.05 1209.91,1079.26 1210.13,1079.72 1210.36,1079.14 1210.59,1078.71 1210.82,1078.47 1211.05,1076.95 1211.27,1076.94 1211.5,1077.01 1211.73,1077.21 1211.96,1078.44 1212.18,1077.44 1212.41,1077.3 1212.64,1076.75 1212.87,1076.9 1213.1,1077.33 1213.32,1077.86 1213.55,1078.87 1213.78,1078.17 1214.01,1076.41 1214.23,1076.26 1214.46,1076.51 1214.69,1074.73 1214.92,1075.47 1215.15,1076.43 1215.37,1076.6 1215.6,1077.46 1215.83,1077.85 1216.06,1077.4 1216.28,1078.23 1216.51,1078.23 1216.74,1078.6 1216.97,1079.36 1217.2,1078.67 1217.42,1077.67 1217.65,1076.9 1217.88,1076.53 1218.11,1076.7 1218.33,1076.93 1218.56,1076.74 1218.79,1076.69 1219.02,1075.91 1219.25,1075.24 1219.47,1074.9 1219.7,1074.63 1219.93,1075.23 1220.16,1074.51 1220.38,1074.9 1220.61,1075.65 1220.84,1076.34 1221.07,1076.12 1221.29,1077.04 1221.52,1076.19 1221.75,1075.94 1221.98,1074.84 1222.21,1074.48 1222.43,1075.4 1222.66,1075.7 1222.89,1075.92 1223.12,1075.26 1223.34,1074.38 1223.57,1073.38 1223.8,1073.86 1224.03,1073.03 1224.26,1073 1224.48,1072.24 1224.71,1072.01 1224.94,1071.35 1225.17,1072.51 1225.39,1072.87 1225.62,1072.93 1225.85,1072.79 1226.08,1072.77 1226.31,1072.9 1226.53,1073.03 1226.76,1072.81 1226.99,1072.97 1227.22,1072.87 1227.44,1073.66 1227.67,1073.31 1227.9,1073.75 1228.13,1074.71 1228.36,1074.39 1228.58,1073.97 1228.81,1073.28 1229.04,1073.68 1229.27,1073.72 1229.49,1073.8 1229.72,1073.98 1229.95,1074.05 1230.18,1073.7 1230.41,1073.97 1230.63,1073.07 1230.86,1072.03 1231.09,1073.06 1231.32,1072.94 1231.54,1073.91 1231.77,1073.75 1232,1073.85 1232.23,1074.13 1232.46,1073.27 1232.68,1073.05 1232.91,1072.81 1233.14,1072.45 1233.37,1072.21 1233.59,1073.15 1233.82,1073.73 1234.05,1073.14 1234.28,1073.06 1234.51,1074.09 1234.73,1074.14 1234.96,1074.99 1235.19,1075.39 1235.42,1075.34 1235.64,1075.08 1235.87,1074.37 1236.1,1073.96 1236.33,1072.82 1236.56,1073.7 1236.78,1073.22 1237.01,1073.8 1237.24,1073.9 1237.47,1074.57 1237.69,1074.43 1237.92,1075.29 1238.15,1075.02 1238.38,1076.14 1238.61,1077.34 1238.83,1077.99 1239.06,1078.47 1239.29,1078.39 1239.52,1077.72 1239.74,1077.7 1239.97,1077.28 1240.2,1076.74 1240.43,1076.52 1240.66,1077.27 1240.88,1077.55 1241.11,1076.23 1241.34,1076.4 1241.57,1076.44 1241.79,1076.99 1242.02,1076.32 1242.25,1076.94 1242.48,1076.83 1242.71,1077.24 1242.93,1077.7 1243.16,1077.03 1243.39,1076.26 1243.62,1076.01 1243.84,1075.86 1244.07,1076.8 1244.3,1076.89 1244.53,1077.08 1244.76,1076.62 1244.98,1076.09 1245.21,1076.03 1245.44,1075.81 1245.67,1076.85 1245.89,1076.75 1246.12,1077.97 1246.35,1078.35 1246.58,1078.53 1246.81,1079 1247.03,1078.83 1247.26,1078.87 1247.49,1078.73 1247.72,1079.21 1247.94,1079.15 1248.17,1077.99 1248.4,1077.29 1248.63,1076.45 1248.86,1076.04 1249.08,1076.19 1249.31,1076.64 1249.54,1077.68 1249.77,1078 1249.99,1078.27 1250.22,1077.94 1250.45,1077.95 1250.68,1078.67 1250.91,1079.18 1251.13,1078.71 1251.36,1078.81 1251.59,1079.18 1251.82,1079.07 1252.04,1078.77 1252.27,1077.42 1252.5,1078.14 1252.73,1078.58 1252.96,1076.8 1253.18,1076.94 1253.41,1076.42 1253.64,1077.38 1253.87,1077.35 1254.09,1078.19 1254.32,1078.28 1254.55,1078.67 1254.78,1078.85 1255.01,1078.06 1255.23,1076.44 1255.46,1075.51 1255.69,1074.72 1255.92,1074.05 1256.14,1073.57 1256.37,1073.28 1256.6,1073.4 1256.83,1073.37 1257.06,1073.69 1257.28,1074.05 1257.51,1074.3 1257.74,1074.99 1257.97,1075.94 1258.19,1076.43 1258.42,1075.75 1258.65,1075.75 1258.88,1076.43 1259.1,1075.95 1259.33,1076.2 1259.56,1077.35 1259.79,1077.62 1260.02,1078.07 1260.24,1077.17 1260.47,1077.69 1260.7,1078.44 1260.93,1078.74 1261.15,1078.88 1261.38,1078.63 1261.61,1078.76 1261.84,1079 1262.07,1078.13 1262.29,1077.66 1262.52,1077.76 1262.75,1076.42 1262.98,1076.79 1263.2,1075.75 1263.43,1074.59 1263.66,1073.58 1263.89,1073.23 1264.12,1072.72 1264.34,1072.59 1264.57,1072.85 1264.8,1074.03 1265.03,1073.66 1265.25,1073.96 1265.48,1073.84 1265.71,1075.07 1265.94,1074.27 1266.17,1073.33 1266.39,1072.61 1266.62,1072.3 1266.85,1071.07 1267.08,1071.46 1267.3,1070.58 1267.53,1070.73 1267.76,1071.2 1267.99,1071.01 1268.22,1070.8 1268.44,1070.91 1268.67,1071.33 1268.9,1071.33 1269.13,1072.27 1269.35,1073.02 1269.58,1072.23 1269.81,1072.79 1270.04,1072.95 1270.27,1072.85 1270.49,1072.17 1270.72,1072.42 1270.95,1072.09 1271.18,1072.35 1271.4,1071.83 1271.63,1072.32 1271.86,1071.77 1272.09,1071.39 1272.32,1070.85 1272.54,1071.24 1272.77,1071.37 1273,1071.46 1273.23,1071.8 1273.45,1071.73 1273.68,1071.74 1273.91,1070.7 1274.14,1070.28 1274.37,1070.18 1274.59,1069.4 1274.82,1068.97 1275.05,1067.93 1275.28,1067.11 1275.5,1067.11 1275.73,1066.67 1275.96,1066.07 1276.19,1066.42 1276.42,1067.13 1276.64,1066.64 1276.87,1066.24 1277.1,1066.56 1277.33,1066.21 1277.55,1065.65 1277.78,1066.04 1278.01,1065.47 1278.24,1066.04 1278.47,1064.88 1278.69,1064.38 1278.92,1064.3 1279.15,1064.13 1279.38,1063.66 1279.6,1063.95 1279.83,1064.53 1280.06,1064.66 1280.29,1065 1280.52,1065.28 1280.74,1066.46 1280.97,1066.06 1281.2,1065.33 1281.43,1066.51 1281.65,1067 1281.88,1066.2 1282.11,1065.66 1282.34,1064.46 1282.57,1065.68 1282.79,1066.18 1283.02,1065.73 1283.25,1066.63 1283.48,1066.02 1283.7,1065.42 1283.93,1065.94 1284.16,1066.04 1284.39,1066 1284.62,1066.03 1284.84,1065.58 1285.07,1065.16 1285.3,1065.64 1285.53,1064.56 1285.75,1064.36 1285.98,1064.03 1286.21,1063.01 1286.44,1063.37 1286.67,1063.22 1286.89,1063.19 1287.12,1063.41 1287.35,1063.21 1287.58,1063.26 1287.8,1063.59 1288.03,1063.57 1288.26,1063.82 1288.49,1063.92 1288.72,1064.06 1288.94,1064.29 1289.17,1063.96 1289.4,1064.18 1289.63,1065.02 1289.85,1064.96 1290.08,1065.97 1290.31,1066.2 1290.54,1066.92 1290.77,1066.2 1290.99,1066.49 1291.22,1065.73 1291.45,1065.25 1291.68,1064.9 1291.9,1064.93 1292.13,1065.3 1292.36,1065.06 1292.59,1064.81 1292.82,1064.89 1293.04,1063.46 1293.27,1063.4 1293.5,1063.88 1293.73,1063.62 1293.95,1063.64 1294.18,1063.43 1294.41,1062.92 1294.64,1063.06 1294.86,1062.73 1295.09,1062.37 1295.32,1062.53 1295.55,1062.87 1295.78,1063.38 1296,1063.71 1296.23,1064.35 1296.46,1064.25 1296.69,1065.45 1296.91,1065.24 1297.14,1065.37 1297.37,1064.77 1297.6,1065.17 1297.83,1065.33 1298.05,1064.94 1298.28,1063.67 1298.51,1063.27 1298.74,1064.2 1298.96,1062.95 1299.19,1062.97 1299.42,1062.93 1299.65,1063.19 1299.88,1064.02 1300.1,1063.77 1300.33,1063.5 1300.56,1064.01 1300.79,1063.68 1301.01,1063.24 1301.24,1062.86 1301.47,1063.22 1301.7,1063.21 1301.93,1062.94 1302.15,1063.59 1302.38,1064.11 1302.61,1063.49 1302.84,1063.02 1303.06,1062.97 1303.29,1063.34 1303.52,1063.53 1303.75,1063.74 1303.98,1063.36 1304.2,1064.15 1304.43,1063.91 1304.66,1063.52 1304.89,1063.48 1305.11,1063.65 1305.34,1063.3 1305.57,1063.31 1305.8,1063.58 1306.03,1063.68 1306.25,1063.94 1306.48,1063.2 1306.71,1063.45 1306.94,1064.67 1307.16,1063.65 1307.39,1063.85 1307.62,1063.69 1307.85,1064.66 1308.08,1064.91 1308.3,1065.19 1308.53,1064.48 1308.76,1064.32 1308.99,1063.89 1309.21,1063.45 1309.44,1063.83 1309.67,1064.29 1309.9,1064.76 1310.13,1065.53 1310.35,1065.26 1310.58,1065.35 1310.81,1065.78 1311.04,1066.81 1311.26,1067.48 1311.49,1066.81 1311.72,1066.53 1311.95,1067.42 1312.18,1066.48 1312.4,1065.65 1312.63,1064.61 1312.86,1063.33 1313.09,1062.52 1313.31,1063.66 1313.54,1063.2 1313.77,1062.95 1314,1063.31 1314.23,1064.03 1314.45,1064.77 1314.68,1063.66 1314.91,1064.27 1315.14,1065.05 1315.36,1064.96 1315.59,1065.22 1315.82,1064.8 1316.05,1063.23 1316.28,1063.36 1316.5,1062.36 1316.73,1062.59 1316.96,1061.68 1317.19,1062.42 1317.41,1063.22 1317.64,1062.38 1317.87,1062.54 1318.1,1063.32 1318.33,1062.94 1318.55,1062.53 1318.78,1062.59 1319.01,1063.05 1319.24,1062.9 1319.46,1063.34 1319.69,1063.32 1319.92,1063.48 1320.15,1063.07 1320.38,1062.57 1320.6,1063.17 1320.83,1064.07 1321.06,1064.19 1321.29,1064.39 1321.51,1064.29 1321.74,1063.55 1321.97,1063.86 1322.2,1063.11 1322.43,1061.62 1322.65,1061.64 1322.88,1061.36 1323.11,1061.35 1323.34,1060.93 1323.56,1061.06 1323.79,1061.12 1324.02,1061.72 1324.25,1061.8 1324.48,1062.59 1324.7,1062.38 1324.93,1062.45 1325.16,1063.28 1325.39,1063.58 1325.61,1063.24 1325.84,1063.02 1326.07,1063.28 1326.3,1063.03 1326.53,1063.32 1326.75,1062.91 1326.98,1061.95 1327.21,1062.01 1327.44,1061.58 1327.66,1061.6 1327.89,1061.53 1328.12,1062.45 1328.35,1062.79 1328.58,1062.33 1328.8,1063.66 1329.03,1063.26 1329.26,1062.63 1329.49,1062.32 1329.71,1062.48 1329.94,1062.79 1330.17,1063 1330.4,1061.22 1330.62,1061.87 1330.85,1061.47 1331.08,1060.94 1331.31,1060.37 1331.54,1060.36 1331.76,1060.14 1331.99,1060.63 1332.22,1061.02 1332.45,1060.81 1332.67,1060.01 1332.9,1060.98 1333.13,1061.08 1333.36,1060.98 1333.59,1060.96 1333.81,1060.46 1334.04,1060.69 1334.27,1061.46 1334.5,1061.95 1334.72,1062.21 1334.95,1061.07 1335.18,1061.68 1335.41,1062.06 1335.64,1062.87 1335.86,1063.07 1336.09,1063.69 1336.32,1064.23 1336.55,1064.29 1336.77,1063.64 1337,1064.06 1337.23,1064.04 1337.46,1064.55 1337.69,1064.78 1337.91,1065.83 1338.14,1065.98 1338.37,1065.54 1338.6,1064.37 1338.82,1063.6 1339.05,1064.45 1339.28,1065 1339.51,1064.45 1339.74,1064.59 1339.96,1064.88 1340.19,1064.5 1340.42,1064.67 1340.65,1064.82 1340.87,1064.35 1341.1,1064.39 1341.33,1065.04 1341.56,1065.02 1341.79,1064.4 1342.01,1063.81 1342.24,1064.75 1342.47,1064.59 1342.7,1064.65 1342.92,1063.63 1343.15,1063.78 1343.38,1063.66 1343.61,1063.34 1343.84,1063.82 1344.06,1063.57 1344.29,1063.38 1344.52,1063.24 1344.75,1062.1 1344.97,1061.5 1345.2,1060.71 1345.43,1060.51 1345.66,1060.89 1345.89,1061.1 1346.11,1061.11 1346.34,1061.92 1346.57,1062.64 1346.8,1062.74 1347.02,1062.67 1347.25,1063.33 1347.48,1063.5 1347.71,1064.26 1347.94,1063.96 1348.16,1064.9 1348.39,1064.92 1348.62,1065.76 1348.85,1066.08 1349.07,1066.96 1349.3,1067.39 1349.53,1068.12 1349.76,1067.16 1349.99,1066.83 1350.21,1066.93 1350.44,1066.91 1350.67,1066.91 1350.9,1066.65 1351.12,1067.2 1351.35,1067.08 1351.58,1066.8 1351.81,1065.82 1352.04,1065.89 1352.26,1065.73 1352.49,1065.59 1352.72,1065.81 1352.95,1067.02 1353.17,1067.46 1353.4,1067.99 1353.63,1067.98 1353.86,1067.26 1354.09,1068.31 1354.31,1068.72 1354.54,1068.73 1354.77,1068.4 1355,1067.82 1355.22,1068.33 1355.45,1068.29 1355.68,1068.5 1355.91,1068.48 1356.14,1068.14 1356.36,1069.04 1356.59,1069.75 1356.82,1070.15 1357.05,1070.06 1357.27,1069.57 1357.5,1069.58 1357.73,1069.55 1357.96,1068.66 1358.19,1067.87 1358.41,1067.58 1358.64,1067.28 1358.87,1067.15 1359.1,1067.42 1359.32,1066.62 1359.55,1065.82 1359.78,1066.13 1360.01,1066.48 1360.24,1067.41 1360.46,1067.6 1360.69,1067.69 1360.92,1066.82 1361.15,1067.15 1361.37,1066.46 1361.6,1067.35 1361.83,1067.56 1362.06,1068.21 1362.29,1068.98 1362.51,1068.87 1362.74,1068.96 1362.97,1068.89 1363.2,1069.17 1363.42,1069.43 1363.65,1069.25 1363.88,1069.9 1364.11,1070.56 1364.34,1070.17 1364.56,1068.98 1364.79,1068.62 1365.02,1069.09 1365.25,1069.3 1365.47,1070.06 1365.7,1069.55 1365.93,1069.55 1366.16,1070.02 1366.39,1070.32 1366.61,1070.74 1366.84,1071.26 1367.07,1071.83 1367.3,1071.92 1367.52,1072.06 1367.75,1071.85 1367.98,1072.65 1368.21,1072.78 1368.43,1072.46 1368.66,1071.94 1368.89,1071.95 1369.12,1071.87 1369.35,1072.08 1369.57,1071.69 1369.8,1071.76 1370.03,1072.16 1370.26,1071.93 1370.48,1071.37 1370.71,1071.12 1370.94,1071.66 1371.17,1072.15 1371.4,1071.32 1371.62,1071.67 1371.85,1072.15 1372.08,1072.42 1372.31,1072.1 1372.53,1072.56 1372.76,1072.91 1372.99,1073.02 1373.22,1073.44 1373.45,1073.08 1373.67,1073.49 1373.9,1072.26 1374.13,1073.02 1374.36,1073.89 1374.58,1073.52 1374.81,1073.4 1375.04,1074 1375.27,1072.81 1375.5,1073.42 1375.72,1073.58 1375.95,1073.45 1376.18,1073.98 1376.41,1073.81 1376.63,1074.07 1376.86,1074.07 1377.09,1075.03 1377.32,1074.28 1377.55,1074.36 1377.77,1075.04 1378,1075.06 1378.23,1075.23 1378.46,1074.83 1378.68,1075.13 1378.91,1074.57 1379.14,1074.66 1379.37,1074.3 1379.6,1074.8 1379.82,1074.57 1380.05,1074.55 1380.28,1074.73 1380.51,1075.24 1380.73,1075.11 1380.96,1075.82 1381.19,1075.66 1381.42,1075.51 1381.65,1075.54 1381.87,1075.68 1382.1,1075.87 1382.33,1076.12 1382.56,1075.72 1382.78,1075.52 1383.01,1075.52 1383.24,1076.02 1383.47,1075.3 1383.7,1074.7 1383.92,1074.78 1384.15,1075.61 1384.38,1075.32 1384.61,1075.47 1384.83,1075.55 1385.06,1075.06 1385.29,1074.87 1385.52,1074.29 1385.75,1074.26 1385.97,1073.7 1386.2,1074.29 1386.43,1073.68 1386.66,1073.48 1386.88,1073.64 1387.11,1074.84 1387.34,1074.34 1387.57,1075.17 1387.8,1075.16 1388.02,1074.8 1388.25,1074.09 1388.48,1073.37 1388.71,1073.4 1388.93,1073.89 1389.16,1074.51 1389.39,1074.03 1389.62,1074.13 1389.85,1074.22 1390.07,1073.34 1390.3,1073.52 1390.53,1073.85 1390.76,1074.45 1390.98,1075.1 1391.21,1075.07 1391.44,1074.45 1391.67,1074.13 1391.9,1074.07 1392.12,1073.94 1392.35,1074.27 1392.58,1074.44 1392.81,1074.41 1393.03,1074.94 1393.26,1074.77 1393.49,1074.74 1393.72,1074.75 1393.95,1073.66 1394.17,1072.84 1394.4,1072.75 1394.63,1073.18 1394.86,1072.95 1395.08,1073.55 1395.31,1073.98 1395.54,1073.99 1395.77,1074.39 1396,1074.37 1396.22,1074.48 1396.45,1074.15 1396.68,1074.05 1396.91,1073.01 1397.13,1073.42 1397.36,1072.65 1397.59,1072.81 1397.82,1073.3 1398.05,1072.93 1398.27,1072.53 1398.5,1072.8 1398.73,1072.4 1398.96,1071.22 1399.18,1070.83 1399.41,1071.15 1399.64,1071.98 1399.87,1072.24 1400.1,1071.47 1400.32,1071.37 1400.55,1071.14 1400.78,1070.71 1401.01,1069.91 1401.23,1070.59 1401.46,1070.75 1401.69,1071.3 1401.92,1071.2 1402.15,1070.02 1402.37,1070.18 1402.6,1070.7 1402.83,1070.95 1403.06,1071.02 1403.28,1071.58 1403.51,1071.92 1403.74,1071.66 1403.97,1071.97 1404.19,1072.77 1404.42,1073.55 1404.65,1074.47 1404.88,1075.47 1405.11,1075.7 1405.33,1074.54 1405.56,1073.94 1405.79,1074.1 1406.02,1074.81 1406.24,1074.61 1406.47,1075.22 1406.7,1074.91 1406.93,1074.23 1407.16,1075.37 1407.38,1075.11 1407.61,1074.66 1407.84,1074.86 1408.07,1075.04 1408.29,1073.84 1408.52,1074.06 1408.75,1073.86 1408.98,1073.4 1409.21,1073.68 1409.43,1073.67 1409.66,1074.66 1409.89,1074.61 1410.12,1075.11 1410.34,1074.9 1410.57,1075.28 1410.8,1075.1 1411.03,1074.8 1411.26,1075.68 1411.48,1075.43 1411.71,1074.63 1411.94,1074.12 1412.17,1074.75 1412.39,1074.64 1412.62,1074.33 1412.85,1074.66 1413.08,1074.8 1413.31,1074.31 1413.53,1073.82 1413.76,1073.2 1413.99,1073.97 1414.22,1073.81 1414.44,1073.64 1414.67,1073.41 1414.9,1073.9 1415.13,1074.67 1415.36,1074.09 1415.58,1074.27 1415.81,1072.96 1416.04,1072.62 1416.27,1072.07 1416.49,1072.04 1416.72,1072.56 1416.95,1072.48 1417.18,1072.85 1417.41,1072.43 1417.63,1072.66 1417.86,1072.8 1418.09,1072.85 1418.32,1072.53 1418.54,1073.26 1418.77,1073.52 1419,1073.85 1419.23,1072.59 1419.46,1073.35 1419.68,1073.03 1419.91,1073.51 1420.14,1074.11 1420.37,1074.06 1420.59,1073.82 1420.82,1073.93 1421.05,1074.08 1421.28,1073.31 1421.51,1073.52 1421.73,1073.66 1421.96,1073.82 1422.19,1073.53 1422.42,1073.44 1422.64,1072.72 1422.87,1072.92 1423.1,1073.19 1423.33,1073.06 1423.56,1072.9 1423.78,1072.96 1424.01,1073.85 1424.24,1074.77 1424.47,1075.52 1424.69,1074.58 1424.92,1075.72 1425.15,1075.59 1425.38,1076.62 1425.61,1076.76 1425.83,1076.67 1426.06,1076.67 1426.29,1077.14 1426.52,1076.94 1426.74,1077.78 1426.97,1078.3 1427.2,1077.22 1427.43,1077.17 1427.66,1077.15 1427.88,1077.41 1428.11,1077.91 1428.34,1078.38 1428.57,1078.33 1428.79,1078.76 1429.02,1079.02 1429.25,1078.49 1429.48,1078.83 1429.71,1078.87 1429.93,1078.63 1430.16,1078.97 1430.39,1078.6 1430.62,1078.83 1430.84,1079.5 1431.07,1079.92 1431.3,1079.89 1431.53,1078.75 1431.76,1078.15 1431.98,1078.9 1432.21,1077.2 1432.44,1076.6 1432.67,1077.2 1432.89,1077.75 1433.12,1077.77 1433.35,1077.67 1433.58,1077.22 1433.81,1077.53 1434.03,1078.18 1434.26,1078.9 1434.49,1078.8 1434.72,1079.22 1434.94,1079.76 1435.17,1079.98 1435.4,1078.19 1435.63,1078.14 1435.86,1077.58 1436.08,1077.1 1436.31,1077.43 1436.54,1077.99 1436.77,1078 1436.99,1077.24 1437.22,1076.7 1437.45,1076.88 1437.68,1077.22 1437.91,1077.4 1438.13,1077.69 1438.36,1077.8 1438.59,1078.37 1438.82,1078.15 1439.04,1078.36 1439.27,1077.36 1439.5,1076.83 1439.73,1077.7 1439.95,1077.79 1440.18,1077.03 1440.41,1077.81 1440.64,1077.59 1440.87,1077.83 1441.09,1077.29 1441.32,1077.31 1441.55,1077.68 1441.78,1077.55 1442,1077.18 1442.23,1076.38 1442.46,1076.1 1442.69,1076.08 1442.92,1076.02 1443.14,1076.62 1443.37,1075.76 1443.6,1075.52 1443.83,1074.92 1444.05,1074.91 1444.28,1074.83 1444.51,1074.85 1444.74,1074.87 1444.97,1075.37 1445.19,1075.52 1445.42,1075.28 1445.65,1074.73 1445.88,1074.76 1446.1,1075.28 1446.33,1075.12 1446.56,1074.54 1446.79,1074.19 1447.02,1074.56 1447.24,1074.44 1447.47,1073.66 1447.7,1072.91 1447.93,1072.88 1448.15,1071.96 1448.38,1071.96 1448.61,1071.97 1448.84,1071.58 1449.07,1071.52 1449.29,1071.2 1449.52,1070.99 1449.75,1071.04 1449.98,1071.48 1450.2,1071.35 1450.43,1071 1450.66,1071.92 1450.89,1071.22 1451.12,1071.18 1451.34,1071 1451.57,1071.75 1451.8,1070.93 1452.03,1071.06 1452.25,1070.92 1452.48,1069.89 1452.71,1070.22 1452.94,1070.76 1453.17,1070.83 1453.39,1071.06 1453.62,1071.88 1453.85,1072.13 1454.08,1071.88 1454.3,1071.55 1454.53,1071.41 1454.76,1071.86 1454.99,1071.62 1455.22,1072.59 1455.44,1072.57 1455.67,1072.2 1455.9,1071.85 1456.13,1072.59 1456.35,1072.65 1456.58,1072.74 1456.81,1073.19 1457.04,1072.53 1457.27,1072.64 1457.49,1072.09 1457.72,1072.38 1457.95,1071.86 1458.18,1072.68 1458.4,1073.18 1458.63,1073.24 1458.86,1072.92 1459.09,1073.14 1459.32,1073.85 1459.54,1074.05 1459.77,1074.43 1460,1074.43 1460.23,1074.58 1460.45,1074.05 1460.68,1074.5 1460.91,1074.72 1461.14,1074.78 1461.37,1074.13 1461.59,1073.86 1461.82,1073.64 1462.05,1073.15 1462.28,1073.62 1462.5,1074.23 1462.73,1074.3 1462.96,1073.74 1463.19,1073.82 1463.42,1074.53 1463.64,1074.68 1463.87,1074.16 1464.1,1073.44 1464.33,1073.64 1464.55,1073.28 1464.78,1073.3 1465.01,1073.58 1465.24,1073.67 1465.47,1073.77 1465.69,1074.14 1465.92,1074.61 1466.15,1075.11 1466.38,1074.96 1466.6,1075.52 1466.83,1076.32 1467.06,1076.93 1467.29,1076.43 1467.52,1076.59 1467.74,1076.67 1467.97,1076.43 1468.2,1076.58 1468.43,1076.23 1468.65,1076.61 1468.88,1076.94 1469.11,1076.11 1469.34,1075.81 1469.57,1075.89 1469.79,1076.26 1470.02,1075.52 1470.25,1075.16 1470.48,1074.08 1470.7,1074.27 1470.93,1074.63 1471.16,1074.56 1471.39,1075.02 1471.62,1075.13 1471.84,1075.37 1472.07,1076.32 1472.3,1076.85 1472.53,1077.34 1472.75,1077.5 1472.98,1077.83 1473.21,1077.89 1473.44,1078 1473.67,1078.49 1473.89,1079.03 1474.12,1079.37 1474.35,1079.51 1474.58,1079.88 1474.8,1080.03 1475.03,1078.98 1475.26,1079.15 1475.49,1078.41 1475.71,1079.12 1475.94,1079.28 1476.17,1079.43 1476.4,1078.68 1476.63,1079.04 1476.85,1079.49 1477.08,1079.58 1477.31,1079.14 1477.54,1079.89 1477.76,1080.46 1477.99,1079.46 1478.22,1078.85 1478.45,1079.15 1478.68,1078.95 1478.9,1078.84 1479.13,1078.75 1479.36,1078.79 1479.59,1079.67 1479.81,1079.19 1480.04,1078.39 1480.27,1079.1 1480.5,1079.12 1480.73,1079.49 1480.95,1078.93 1481.18,1079.7 1481.41,1079.33 1481.64,1079.19 1481.86,1079.36 1482.09,1078.68 1482.32,1078.83 1482.55,1079.1 1482.78,1078.52 1483,1078.3 1483.23,1079.64 1483.46,1079.79 1483.69,1079.2 1483.91,1079.25 1484.14,1079.07 1484.37,1079.11 1484.6,1079.34 1484.83,1079.35 1485.05,1079.32 1485.28,1080.38 1485.51,1080.91 1485.74,1080.51 1485.96,1080.06 1486.19,1079.97 1486.42,1079.77 1486.65,1080.09 1486.88,1079.57 1487.1,1079.32 1487.33,1079.48 1487.56,1079.06 1487.79,1078.63 1488.01,1079 1488.24,1079.01 1488.47,1079.41 1488.7,1079.4 1488.93,1080.01 1489.15,1080.11 1489.38,1080.01 1489.61,1079.97 1489.84,1080.04 1490.06,1079.88 1490.29,1079.9 1490.52,1079.62 1490.75,1079.66 1490.98,1079.58 1491.2,1079.47 1491.43,1078.93 1491.66,1078.46 1491.89,1078.67 1492.11,1078.61 1492.34,1078.77 1492.57,1078.41 1492.8,1078.94 1493.03,1078.14 1493.25,1078.46 1493.48,1078.33 1493.71,1078.13 1493.94,1078.66 1494.16,1078.38 1494.39,1078.65 1494.62,1079.33 1494.85,1079.71 1495.08,1080.46 1495.3,1080.12 1495.53,1079.98 1495.76,1079.63 1495.99,1079.03 1496.21,1079.53 1496.44,1080.59 1496.67,1081 1496.9,1081.72 1497.13,1081.04 1497.35,1081.25 1497.58,1080.72 1497.81,1081.02 1498.04,1081.57 1498.26,1081.8 1498.49,1082.27 1498.72,1082.44 1498.95,1082.52 1499.18,1082.35 1499.4,1081.64 1499.63,1081.08 1499.86,1081.8 1500.09,1081.46 1500.31,1081.01 1500.54,1081.58 1500.77,1082.31 1501,1083.07 1501.23,1082.49 1501.45,1082.45 1501.68,1083.18 1501.91,1083.89 1502.14,1083.51 1502.36,1083.6 1502.59,1084.07 1502.82,1084.25 1503.05,1084.34 1503.28,1083.05 1503.5,1082.96 1503.73,1083.71 1503.96,1082.77 1504.19,1081.35 1504.41,1082.23 1504.64,1082.28 1504.87,1082.26 1505.1,1082.08 1505.33,1081.48 1505.55,1081.16 1505.78,1082.07 1506.01,1081.7 1506.24,1082.29 1506.46,1082.52 1506.69,1082.75 1506.92,1083.16 1507.15,1082.04 1507.38,1082.22 1507.6,1081.94 1507.83,1081.62 1508.06,1081.27 1508.29,1080.8 1508.51,1081.64 1508.74,1080.74 1508.97,1080.64 1509.2,1080.21 1509.43,1079.95 1509.65,1079.98 1509.88,1080.68 1510.11,1080.35 1510.34,1080.57 1510.56,1081.16 1510.79,1081.14 1511.02,1080.97 1511.25,1081.17 1511.48,1080.54 1511.7,1080.82 1511.93,1080.8 1512.16,1080.41 1512.39,1081.25 1512.61,1081.72 1512.84,1080.13 1513.07,1080.43 1513.3,1080.13 1513.52,1080.6 1513.75,1080.5 1513.98,1081.15 1514.21,1081.32 1514.44,1081.25 1514.66,1081.23 1514.89,1081.46 1515.12,1081.64 1515.35,1081.5 1515.57,1081.56 1515.8,1081.74 1516.03,1081.37 1516.26,1081.63 1516.49,1081.11 1516.71,1081.13 1516.94,1081.69 1517.17,1081.89 1517.4,1081.81 1517.62,1081.2 1517.85,1080.39 1518.08,1081.26 1518.31,1081.43 1518.54,1081.33 1518.76,1081.52 1518.99,1081.99 1519.22,1080.67 1519.45,1080.07 1519.67,1079.95 1519.9,1080.45 1520.13,1080.43 1520.36,1080.67 1520.59,1080.29 1520.81,1080.15 1521.04,1080.29 1521.27,1080.66 1521.5,1080.6 1521.72,1080.98 1521.95,1081.51 1522.18,1081.58 1522.41,1081.41 1522.64,1080.57 1522.86,1080.47 1523.09,1081.37 1523.32,1080.9 1523.55,1079.63 1523.77,1078.85 1524,1078.21 1524.23,1077.72 1524.46,1077.69 1524.69,1077.59 1524.91,1077.44 1525.14,1076.88 1525.37,1077.25 1525.6,1077.08 1525.82,1076.97 1526.05,1076.74 1526.28,1077.47 1526.51,1078.17 1526.74,1077.96 1526.96,1077.91 1527.19,1078.23 1527.42,1078.65 1527.65,1079.46 1527.87,1079.34 1528.1,1078.6 1528.33,1078.1 1528.56,1078 1528.79,1078.47 1529.01,1078.15 1529.24,1078.47 1529.47,1078.98 1529.7,1079.42 1529.92,1078.85 1530.15,1079.22 1530.38,1079.93 1530.61,1080.56 1530.84,1080.82 1531.06,1080.96 1531.29,1080.71 1531.52,1081.08 1531.75,1081.06 1531.97,1081.34 1532.2,1080.76 1532.43,1080.58 1532.66,1081.34 1532.89,1080.9 1533.11,1080.72 1533.34,1080.75 1533.57,1081.09 1533.8,1081.62 1534.02,1082 1534.25,1081.69 1534.48,1082.3 1534.71,1082.19 1534.94,1081.53 1535.16,1081.27 1535.39,1081.1 1535.62,1081.39 1535.85,1081.48 1536.07,1080.48 1536.3,1080.54 1536.53,1080.96 1536.76,1080.37 1536.99,1080.81 1537.21,1080.65 1537.44,1081.14 1537.67,1080.11 1537.9,1079.89 1538.12,1080.26 1538.35,1080.72 1538.58,1079.41 1538.81,1078.89 1539.04,1078.79 1539.26,1078.64 1539.49,1078.25 1539.72,1078.21 1539.95,1078.18 1540.17,1078.88 1540.4,1078.91 1540.63,1079.14 1540.86,1079.05 1541.09,1079.49 1541.31,1079.26 1541.54,1080.05 1541.77,1079.9 1542,1080.05 1542.22,1080.44 1542.45,1080.94 1542.68,1080.72 1542.91,1080.68 1543.14,1080.59 1543.36,1080.5 1543.59,1080.91 1543.82,1081.27 1544.05,1081.33 1544.27,1081.59 1544.5,1082.07 1544.73,1082.17 1544.96,1082.54 1545.19,1082.28 1545.41,1081.86 1545.64,1081.69 1545.87,1080.98 1546.1,1081.38 1546.32,1081.31 1546.55,1081.26 1546.78,1080.51 1547.01,1080.26 1547.24,1079.84 1547.46,1080.56 1547.69,1080.75 1547.92,1081 1548.15,1080.98 1548.37,1080.69 1548.6,1080.97 1548.83,1080.36 1549.06,1080.29 1549.28,1079.87 1549.51,1080.32 1549.74,1079.87 1549.97,1080.75 1550.2,1080.84 1550.42,1080.4 1550.65,1080.12 1550.88,1080.39 1551.11,1079.61 1551.33,1079.87 1551.56,1080 1551.79,1079.93 1552.02,1079.82 1552.25,1078.96 1552.47,1079.32 1552.7,1079.05 1552.93,1079.3 1553.16,1079.18 1553.38,1078.03 1553.61,1078.1 1553.84,1078.56 1554.07,1077.96 1554.3,1078.16 1554.52,1078.56 1554.75,1078.1 1554.98,1077.87 1555.21,1077.44 1555.43,1077.98 1555.66,1078.18 1555.89,1078.77 1556.12,1078.19 1556.35,1078.57 1556.57,1078.45 1556.8,1078.57 1557.03,1078.99 1557.26,1078.73 1557.48,1078.98 1557.71,1078.71 1557.94,1078.78 1558.17,1079.1 1558.4,1078.51 1558.62,1078.15 1558.85,1078.04 1559.08,1077.42 1559.31,1076.58 1559.53,1076.78 1559.76,1076.74 1559.99,1076.76 1560.22,1077.32 1560.45,1076.63 1560.67,1076.61 1560.9,1076.82 1561.13,1077.14 1561.36,1077.2 1561.58,1076.89 1561.81,1077.14 1562.04,1077.3 1562.27,1077.03 1562.5,1077.39 1562.72,1077.26 1562.95,1076.74 1563.18,1076.79 1563.41,1077.3 1563.63,1077.43 1563.86,1078.04 1564.09,1077.41 1564.32,1077.99 1564.55,1077.76 1564.77,1077.37 1565,1077.95 1565.23,1078.24 1565.46,1078.55 1565.68,1078.57 1565.91,1077.55 1566.14,1076.62 1566.37,1077.27 1566.6,1076.67 1566.82,1077 1567.05,1077.19 1567.28,1076.4 1567.51,1076.61 1567.73,1076.51 1567.96,1076.51 1568.19,1076.71 1568.42,1076.85 1568.65,1077 1568.87,1076.64 1569.1,1076.93 1569.33,1076.66 1569.56,1076.96 1569.78,1077.68 1570.01,1077.75 1570.24,1077.79 1570.47,1077.97 1570.7,1077.57 1570.92,1077.69 1571.15,1076.21 1571.38,1076.5 1571.61,1077.06 1571.83,1076.71 1572.06,1076.23 1572.29,1076.21 1572.52,1076.83 1572.75,1076.6 1572.97,1076.69 1573.2,1077.49 1573.43,1077.6 1573.66,1077.17 1573.88,1077.73 1574.11,1078.07 1574.34,1077.9 1574.57,1076.9 1574.8,1076.93 1575.02,1076.31 1575.25,1076.48 1575.48,1076.93 1575.71,1076.68 1575.93,1075.81 1576.16,1075.99 1576.39,1076 1576.62,1075.41 1576.85,1075.18 1577.07,1075.5 1577.3,1074.91 1577.53,1074.11 1577.76,1073.32 1577.98,1073.27 1578.21,1073.23 1578.44,1073.78 1578.67,1074.03 1578.9,1073.61 1579.12,1073.77 1579.35,1073.51 1579.58,1073.96 1579.81,1073.88 1580.03,1074.27 1580.26,1074.84 1580.49,1074.81 1580.72,1075.17 1580.95,1075.39 1581.17,1075.72 1581.4,1076.3 1581.63,1076.71 1581.86,1076.86 1582.08,1076.48 1582.31,1076.86 1582.54,1076.61 1582.77,1076.49 1583,1076.3 1583.22,1076.94 1583.45,1076.6 1583.68,1075.96 1583.91,1076.28 1584.13,1075.89 1584.36,1075.97 1584.59,1074.79 1584.82,1074.72 1585.04,1074.17 1585.27,1074.73 1585.5,1075.36 1585.73,1075.41 1585.96,1075.94 1586.18,1075.91 1586.41,1076.47 1586.64,1076.65 1586.87,1077.16 1587.09,1077.19 1587.32,1077.12 1587.55,1076.92 1587.78,1077.09 1588.01,1076.97 1588.23,1077.57 1588.46,1077.56 1588.69,1077.4 1588.92,1077.91 1589.14,1078.45 1589.37,1079.06 1589.6,1078.74 1589.83,1078.22 1590.06,1077.65 1590.28,1078.32 1590.51,1078.87 1590.74,1079.4 1590.97,1079.2 1591.19,1079.08 1591.42,1077.9 1591.65,1077.67 1591.88,1077.45 1592.11,1077.94 1592.33,1077.52 1592.56,1078.34 1592.79,1078.24 1593.02,1077.56 1593.24,1077.76 1593.47,1077.9 1593.7,1078.4 1593.93,1078.77 1594.16,1079.07 1594.38,1079.53 1594.61,1079.82 1594.84,1079.51 1595.07,1078.47 1595.29,1078.84 1595.52,1078.89 1595.75,1077.91 1595.98,1077.5 1596.21,1076.48 1596.43,1076.32 1596.66,1076.03 1596.89,1075.91 1597.12,1076.58 1597.34,1076.27 1597.57,1076.08 1597.8,1076.93 1598.03,1077.35 1598.26,1077.45 1598.48,1076.99 1598.71,1076.96 1598.94,1077.12 1599.17,1076.53 1599.39,1076.84 1599.62,1076.55 1599.85,1076.77 1600.08,1076.16 1600.31,1076.41 1600.53,1077.31 1600.76,1077.13 1600.99,1076.86 1601.22,1076.62 1601.44,1076.53 1601.67,1075.99 1601.9,1076.43 1602.13,1076.86 1602.36,1077.09 1602.58,1077.09 1602.81,1076.66 1603.04,1076.58 1603.27,1076.81 1603.49,1077.41 1603.72,1077.87 1603.95,1077.86 1604.18,1078.12 1604.41,1077.46 1604.63,1077.82 1604.86,1078.09 1605.09,1078.5 1605.32,1079 1605.54,1078.94 1605.77,1079.09 1606,1078.43 1606.23,1078.19 1606.46,1078.36 1606.68,1078.6 1606.91,1078.89 1607.14,1078.81 1607.37,1078.67 1607.59,1079.03 1607.82,1079.51 1608.05,1079.86 1608.28,1079.73 1608.51,1079.73 1608.73,1079.12 1608.96,1078.87 1609.19,1078.45 1609.42,1078.37 1609.64,1078.63 1609.87,1077.94 1610.1,1078.1 1610.33,1077.84 1610.56,1078.61 1610.78,1078.56 1611.01,1078.91 1611.24,1079.23 1611.47,1079.49 1611.69,1079.1 1611.92,1079.48 1612.15,1079.34 1612.38,1080.14 1612.61,1079.47 1612.83,1079.92 1613.06,1081 1613.29,1081.39 1613.52,1080.34 1613.74,1079.96 1613.97,1079.7 1614.2,1080.08 1614.43,1079.95 1614.66,1079.67 1614.88,1079.66 1615.11,1078.94 1615.34,1079.24 1615.57,1079.96 1615.79,1078.81 1616.02,1079.05 1616.25,1079.51 1616.48,1079.53 1616.71,1079.4 1616.93,1079.67 1617.16,1079.8 1617.39,1078.78 1617.62,1079.02 1617.84,1079.09 1618.07,1078.88 1618.3,1078.36 1618.53,1078.46 1618.76,1078.78 1618.98,1079 1619.21,1078.84 1619.44,1078.79 1619.67,1078.6 1619.89,1077.51 1620.12,1077.91 1620.35,1077.79 1620.58,1077.87 1620.81,1077.62 1621.03,1077.83 1621.26,1078.71 1621.49,1079.04 1621.72,1079.09 1621.94,1078.83 1622.17,1078.66 1622.4,1078.18 1622.63,1078.18 1622.85,1078.35 1623.08,1078.64 1623.31,1078.7 1623.54,1079.3 1623.77,1078.87 1623.99,1078.65 1624.22,1079.14 1624.45,1078.76 1624.68,1078.32 1624.9,1078.27 1625.13,1078.48 1625.36,1078.14 1625.59,1078.36 1625.82,1079.01 1626.04,1078.43 1626.27,1078.28 1626.5,1077.72 1626.73,1076.23 1626.95,1076.6 1627.18,1076.83 1627.41,1076.43 1627.64,1076.73 1627.87,1077.46 1628.09,1077.62 1628.32,1077.59 1628.55,1076.81 1628.78,1077.01 1629,1077.76 1629.23,1077.51 1629.46,1077.4 1629.69,1077.42 1629.92,1077.61 1630.14,1077.78 1630.37,1077.83 1630.6,1077.58 1630.83,1077.12 1631.05,1076.66 1631.28,1076.13 1631.51,1077.13 1631.74,1077.2 1631.97,1077.44 1632.19,1077.62 1632.42,1078.03 1632.65,1077.88 1632.88,1077.32 1633.1,1077.06 1633.33,1076.84 1633.56,1076.35 1633.79,1076.53 1634.02,1077.11 1634.24,1076.1 1634.47,1076.03 1634.7,1076.48 1634.93,1076.31 1635.15,1076.62 1635.38,1076.37 1635.61,1076.71 1635.84,1076.93 1636.07,1077.15 1636.29,1076.68 1636.52,1076.75 1636.75,1076.52 1636.98,1076.71 1637.2,1077.36 1637.43,1077.26 1637.66,1077.75 1637.89,1077.69 1638.12,1078.07 1638.34,1077.7 1638.57,1078.44 1638.8,1079.26 1639.03,1079.47 1639.25,1079.78 1639.48,1079.37 1639.71,1079.91 1639.94,1080.12 1640.17,1080.53 1640.39,1080 1640.62,1080.3 1640.85,1080.31 1641.08,1080.27 1641.3,1080.73 1641.53,1080.59 1641.76,1080.28 1641.99,1081.05 1642.22,1081.13 1642.44,1081.24 1642.67,1081.65 1642.9,1081.85 1643.13,1082.57 1643.35,1083.39 1643.58,1083.39 1643.81,1083.16 1644.04,1083.34 1644.27,1083.19 1644.49,1083.32 1644.72,1083.73 1644.95,1083.36 1645.18,1082.9 1645.4,1083.89 1645.63,1084.75 1645.86,1084.98 1646.09,1084.99 1646.32,1085.31 1646.54,1085.43 1646.77,1085.1 1647,1085.84 1647.23,1085.52 1647.45,1085.02 1647.68,1084.68 1647.91,1084.92 1648.14,1084.97 1648.37,1085.13 1648.59,1084.58 1648.82,1084.77 1649.05,1085.24 1649.28,1086.02 1649.5,1085.81 1649.73,1086.58 1649.96,1087.13 1650.19,1086.63 1650.42,1085.83 1650.64,1085.35 1650.87,1085.27 1651.1,1085.43 1651.33,1086.05 1651.55,1085.84 1651.78,1085.73 1652.01,1085.92 1652.24,1086.22 1652.47,1086.74 1652.69,1086.58 1652.92,1085.98 1653.15,1086.11 1653.38,1086.24 1653.6,1085.18 1653.83,1085.41 1654.06,1085.36 1654.29,1084.9 1654.52,1085.44 1654.74,1086.47 1654.97,1086.33 1655.2,1085.9 1655.43,1085.3 1655.65,1086.14 1655.88,1086.37 1656.11,1086.6 1656.34,1087.19 1656.57,1087.25 1656.79,1087.52 1657.02,1087.95 1657.25,1088.29 1657.48,1088.4 1657.7,1088.17 1657.93,1088.55 1658.16,1088.02 1658.39,1087.69 1658.61,1087.27 1658.84,1086.71 1659.07,1086.84 1659.3,1087.24 1659.53,1087.26 1659.75,1086.89 1659.98,1087.46 1660.21,1087.98 1660.44,1087.52 1660.66,1087.29 1660.89,1086.5 1661.12,1086.5 1661.35,1085.98 1661.58,1085.29 1661.8,1085.53 1662.03,1085.68 1662.26,1085.57 1662.49,1085.33 1662.71,1085.55 1662.94,1085.73 1663.17,1085.28 1663.4,1085.41 1663.63,1085.38 1663.85,1085.85 1664.08,1085.65 1664.31,1085.36 1664.54,1086.09 1664.76,1086.39 1664.99,1086.1 1665.22,1086.85 1665.45,1087.51 1665.68,1087.68 1665.9,1087.3 1666.13,1087.5 1666.36,1087.68 1666.59,1087.9 1666.81,1088.46 1667.04,1088.28 1667.27,1088.35 1667.5,1088.84 1667.73,1088.85 1667.95,1088.95 1668.18,1089.25 1668.41,1089.77 1668.64,1089.83 1668.86,1089.36 1669.09,1089.85 1669.32,1089.8 1669.55,1090.15 1669.78,1090.17 1670,1090.81 1670.23,1091.27 1670.46,1091.87 1670.69,1092.42 1670.91,1092.06 1671.14,1092.24 1671.37,1092.79 1671.6,1092.84 1671.83,1093.14 1672.05,1093.58 1672.28,1092.16 1672.51,1092.41 1672.74,1092.4 1672.96,1092.4 1673.19,1092.25 1673.42,1092.3 1673.65,1092.19 1673.88,1092.51 1674.1,1092.58 1674.33,1092.71 1674.56,1092.95 1674.79,1093.14 1675.01,1093.05 1675.24,1092.51 1675.47,1093.07 1675.7,1092.49 1675.93,1092.34 1676.15,1092.36 1676.38,1091.81 1676.61,1092.3 1676.84,1092.47 1677.06,1092.38 1677.29,1092.86 1677.52,1093.19 1677.75,1093.26 1677.98,1093.6 1678.2,1093.96 1678.43,1093.46 1678.66,1092.68 1678.89,1091.72 1679.11,1091.27 1679.34,1091.06 1679.57,1091.25 1679.8,1090.86 1680.03,1089.93 1680.25,1089.93 1680.48,1089.84 1680.71,1089.29 1680.94,1089.11 1681.16,1089.46 1681.39,1089.26 1681.62,1089.54 1681.85,1089.24 1682.08,1089.03 1682.3,1089.56 1682.53,1089.08 1682.76,1089.07 1682.99,1089.04 1683.21,1089.23 1683.44,1089.12 1683.67,1089.53 1683.9,1088.52 1684.13,1088.81 1684.35,1088.52 1684.58,1089.02 1684.81,1088.15 1685.04,1088.69 1685.26,1088.53 1685.49,1088.42 1685.72,1088.58 1685.95,1088.41 1686.18,1088.56 1686.4,1088.75 1686.63,1089.46 1686.86,1089.58 1687.09,1089.99 1687.31,1090.43 1687.54,1090.64 1687.77,1091.09 1688,1090.91 1688.23,1090.85 1688.45,1090.85 1688.68,1090.7 1688.91,1091.24 1689.14,1091.03 1689.36,1090.66 1689.59,1091.08 1689.82,1091.27 1690.05,1091.42 1690.28,1091.52 1690.5,1091.63 1690.73,1091.72 1690.96,1090.98 1691.19,1091.11 1691.41,1090.44 1691.64,1090.57 1691.87,1090.77 1692.1,1090.12 1692.33,1090.49 1692.55,1090.76 1692.78,1091.27 1693.01,1091.66 1693.24,1092.03 1693.46,1091.68 1693.69,1089.92 1693.92,1089.86 1694.15,1090.54 1694.37,1090.43 1694.6,1089.64 1694.83,1090 1695.06,1090.28 1695.29,1090.38 1695.51,1090.56 1695.74,1089.87 1695.97,1089.82 1696.2,1090.16 1696.42,1089.28 1696.65,1089.73 1696.88,1089.38 1697.11,1089.69 1697.34,1089.71 1697.56,1089.36 1697.79,1089.29 1698.02,1089.09 1698.25,1089.61 1698.47,1089.89 1698.7,1089.8 1698.93,1089.85 1699.16,1089.96 1699.39,1089.95 1699.61,1090.05 1699.84,1089.81 1700.07,1088.9 1700.3,1088.61 1700.52,1089.2 1700.75,1089.67 1700.98,1089.99 1701.21,1090.72 1701.44,1089.9 1701.66,1089.73 1701.89,1090.12 1702.12,1090.12 1702.35,1090.23 1702.57,1090.81 1702.8,1091.06 1703.03,1091.57 1703.26,1091.62 1703.49,1091.85 1703.71,1091.72 1703.94,1091.01 1704.17,1091.11 1704.4,1091.05 1704.62,1091.21 1704.85,1091.09 1705.08,1089.38 1705.31,1090.02 1705.54,1090.77 1705.76,1091.05 1705.99,1091.93 1706.22,1091.9 1706.45,1092.27 1706.67,1092.47 1706.9,1092.82 1707.13,1093.26 1707.36,1093.26 1707.59,1092.67 1707.81,1092.43 1708.04,1093.18 1708.27,1092.78 1708.5,1093.16 1708.72,1092.77 1708.95,1092.55 1709.18,1092.63 1709.41,1092.67 1709.64,1093.09 1709.86,1093.41 1710.09,1092.93 1710.32,1093.17 1710.55,1093.5 1710.77,1092.56 1711,1091.59 1711.23,1092.46 1711.46,1092.23 1711.69,1092.22 1711.91,1091.91 1712.14,1091.85 1712.37,1092.19 1712.6,1091.49 1712.82,1091.69 1713.05,1091.59 1713.28,1091.93 1713.51,1091.76 1713.74,1091.4 1713.96,1091.21 1714.19,1091.18 1714.42,1091.33 1714.65,1091 1714.87,1091.18 1715.1,1091.35 1715.33,1090.82 1715.56,1090.14 1715.79,1090.35 1716.01,1089.5 1716.24,1089 1716.47,1088.93 1716.7,1088.38 1716.92,1088.82 1717.15,1089.08 1717.38,1088.92 1717.61,1089.11 1717.84,1088.87 1718.06,1089.14 1718.29,1088.94 1718.52,1088.97 1718.75,1088.93 1718.97,1089.32 1719.2,1089.92 1719.43,1089.56 1719.66,1089.53 1719.89,1089.32 1720.11,1089.65 1720.34,1089.94 1720.57,1089.62 1720.8,1089.8 1721.02,1090.25 1721.25,1089.52 1721.48,1090.26 1721.71,1090.49 1721.94,1090.82 1722.16,1091.23 1722.39,1091.04 1722.62,1091.21 1722.85,1091.56 1723.07,1091.21 1723.3,1090.7 1723.53,1090.82 1723.76,1090.25 1723.99,1090.34 1724.21,1090.02 1724.44,1090.06 1724.67,1089.89 1724.9,1089.6 1725.12,1089.69 1725.35,1089.13 1725.58,1089.47 1725.81,1089.89 1726.04,1090.09 1726.26,1089.91 1726.49,1089.93 1726.72,1090.42 1726.95,1089.8 1727.17,1090.16 1727.4,1090.33 1727.63,1090.5 1727.86,1090.96 1728.09,1090.69 1728.31,1090.69 1728.54,1091.04 1728.77,1090.63 1729,1090.79 1729.22,1089.92 1729.45,1090.06 1729.68,1090.18 1729.91,1090.56 1730.14,1090.46 1730.36,1090.17 1730.59,1090.34 1730.82,1090.75 1731.05,1090.94 1731.27,1091.23 1731.5,1091.41 1731.73,1091.94 1731.96,1092.03 1732.18,1091.84 1732.41,1091.74 1732.64,1091.58 1732.87,1092.42 1733.1,1092.44 1733.32,1092.63 1733.55,1093.08 1733.78,1093.69 1734.01,1093.59 1734.23,1093.49 1734.46,1093.94 1734.69,1093.93 1734.92,1093.86 1735.15,1093.96 1735.37,1093.82 1735.6,1093.93 1735.83,1093.92 1736.06,1093.8 1736.28,1094 1736.51,1094.39 1736.74,1094.23 1736.97,1094.53 1737.2,1094.07 1737.42,1093.99 1737.65,1094.09 1737.88,1094.4 1738.11,1093.68 1738.33,1093.65 1738.56,1093.68 1738.79,1093.77 1739.02,1094.34 1739.25,1094.24 1739.47,1093.58 1739.7,1093.52 1739.93,1093.55 1740.16,1094.21 1740.38,1094.61 1740.61,1095.18 1740.84,1095.12 1741.07,1094.21 1741.3,1093.99 1741.52,1093.82 1741.75,1093.09 1741.98,1093.56 1742.21,1093.58 1742.43,1093.94 1742.66,1094.36 1742.89,1094.66 1743.12,1095.26 1743.35,1095.22 1743.57,1095.17 1743.8,1095.58 1744.03,1095.83 1744.26,1095.83 1744.48,1095.82 1744.71,1095.69 1744.94,1096 1745.17,1096.85 1745.4,1096.07 1745.62,1095.97 1745.85,1096 1746.08,1095.91 1746.31,1096.61 1746.53,1096.48 1746.76,1096.35 1746.99,1096.1 1747.22,1096.2 1747.45,1095.88 1747.67,1095.74 1747.9,1095.8 1748.13,1095.63 1748.36,1095.69 1748.58,1095.47 1748.81,1094.97 1749.04,1095.41 1749.27,1094.84 1749.5,1094.98 1749.72,1095.48 1749.95,1095.25 1750.18,1095.08 1750.41,1095.35 1750.63,1095.57 1750.86,1095.73 1751.09,1096.36 1751.32,1096.49 1751.55,1096.82 1751.77,1097.66 1752,1097.59 1752.23,1096.88 1752.46,1097.08 1752.68,1097.17 1752.91,1097.01 1753.14,1097.25 1753.37,1098.24 1753.6,1097.71 1753.82,1097.31 1754.05,1097.25 1754.28,1097.26 1754.51,1097.13 1754.73,1097.02 1754.96,1097.17 1755.19,1096.88 1755.42,1096.42 1755.65,1096.37 1755.87,1096.38 1756.1,1096.35 1756.33,1096.08 1756.56,1095.61 1756.78,1095.16 1757.01,1095.34 1757.24,1095.27 1757.47,1094.83 1757.7,1095.14 1757.92,1094.82 1758.15,1094.54 1758.38,1094.19 1758.61,1094.8 1758.83,1094.94 1759.06,1094.83 1759.29,1095.25 1759.52,1095.88 1759.75,1096.38 1759.97,1095.84 1760.2,1095.68 1760.43,1095.49 1760.66,1095.37 1760.88,1095.57 1761.11,1095.49 1761.34,1095.35 1761.57,1095.83 1761.8,1095.88 1762.02,1095.57 1762.25,1095.72 1762.48,1095.97 1762.71,1095.61 1762.93,1096.25 1763.16,1096.16 1763.39,1096.17 1763.62,1094.99 1763.85,1095.07 1764.07,1094.97 1764.3,1094.98 1764.53,1094.99 1764.76,1095.23 1764.98,1095.31 1765.21,1095.07 1765.44,1094.6 1765.67,1095 1765.9,1095.16 1766.12,1095.59 1766.35,1096.21 1766.58,1096.65 1766.81,1095.94 1767.03,1095.34 1767.26,1095.51 1767.49,1095.89 1767.72,1095.9 1767.94,1096.18 1768.17,1096.17 1768.4,1096.59 1768.63,1096.7 1768.86,1096.74 1769.08,1097.18 1769.31,1097.21 1769.54,1097.3 1769.77,1097.93 1769.99,1097.99 1770.22,1098.21 1770.45,1098.46 1770.68,1098.13 1770.91,1097.8 1771.13,1097.55 1771.36,1098 1771.59,1098.09 1771.82,1097.67 1772.04,1097.88 1772.27,1097.11 1772.5,1097.1 1772.73,1097.69 1772.96,1097.64 1773.18,1097.66 1773.41,1097.96 1773.64,1097.35 1773.87,1097.78 1774.09,1097.7 1774.32,1097.48 1774.55,1097.71 1774.78,1097.95 1775.01,1097.09 1775.23,1096.73 1775.46,1096.65 1775.69,1096.86 1775.92,1097.36 1776.14,1097.58 1776.37,1097.6 1776.6,1097.88 1776.83,1098.21 1777.06,1098.5 1777.28,1098.07 1777.51,1097.88 1777.74,1098.06 1777.97,1098.3 1778.19,1098.32 1778.42,1097.84 1778.65,1097.75 1778.88,1097.09 1779.11,1097.23 1779.33,1097.74 1779.56,1097.95 1779.79,1097.99 1780.02,1097.57 1780.24,1097.47 1780.47,1097.84 1780.7,1097.86 1780.93,1097.13 1781.16,1096.86 1781.38,1096.89 1781.61,1096.5 1781.84,1096.32 1782.07,1096.29 1782.29,1096.28 1782.52,1096.76 1782.75,1094.72 1782.98,1093.36 1783.21,1093.79 1783.43,1093.2 1783.66,1093.29 1783.89,1093.45 1784.12,1093.19 1784.34,1092.72 1784.57,1092.91 1784.8,1092.74 1785.03,1093.1 1785.26,1092.56 1785.48,1093.23 1785.71,1093.16 1785.94,1092.41 1786.17,1091.91 1786.39,1092.59 1786.62,1092.28 1786.85,1092.29 1787.08,1092.56 1787.31,1092.89 1787.53,1092.66 1787.76,1093.16 1787.99,1093.37 1788.22,1092.7 1788.44,1092.9 1788.67,1093.05 1788.9,1092.71 1789.13,1092.56 1789.36,1092.35 1789.58,1091.73 1789.81,1091.22 1790.04,1091.67 1790.27,1091.9 1790.49,1092.09 1790.72,1091.41 1790.95,1091.46 1791.18,1092.18 1791.41,1092.09 1791.63,1092.21 1791.86,1091.62 1792.09,1091.15 1792.32,1091.72 1792.54,1091.81 1792.77,1091.46 1793,1091.5 1793.23,1091.39 1793.46,1091.33 1793.68,1091.81 1793.91,1092.64 1794.14,1093.12 1794.37,1093.33 1794.59,1093.24 1794.82,1092.54 1795.05,1092.13 1795.28,1092.31 1795.51,1091.58 1795.73,1091.4 1795.96,1091.74 1796.19,1091.47 1796.42,1092.17 1796.64,1091.71 1796.87,1092.14 1797.1,1092.72 1797.33,1092.78 1797.56,1092.43 1797.78,1092.62 1798.01,1092.44 1798.24,1092.87 1798.47,1092.03 1798.69,1092.29 1798.92,1092.71 1799.15,1092.84 1799.38,1092.93 1799.61,1093.57 1799.83,1093.87 1800.06,1093.62 1800.29,1093.75 1800.52,1093.63 1800.74,1093.92 1800.97,1093.14 1801.2,1093.63 1801.43,1093.8 1801.66,1093.21 1801.88,1093.56 1802.11,1093.27 1802.34,1093.4 1802.57,1093 1802.79,1092.97 1803.02,1093.36 1803.25,1093.71 1803.48,1093.36 1803.7,1093.62 1803.93,1093.74 1804.16,1093.39 1804.39,1092.82 1804.62,1093.18 1804.84,1093.45 1805.07,1092.96 1805.3,1093.12 1805.53,1092.9 1805.75,1093.12 1805.98,1092.87 1806.21,1092.1 1806.44,1092.72 1806.67,1092.73 1806.89,1092.88 1807.12,1093.17 1807.35,1093.22 1807.58,1093.3 1807.8,1093.91 1808.03,1093.31 1808.26,1093.05 1808.49,1092.98 1808.72,1092.1 1808.94,1092.65 1809.17,1092.86 1809.4,1092.97 1809.63,1092.15 1809.85,1091.64 1810.08,1091.81 1810.31,1091.81 1810.54,1091.74 1810.77,1091.46 1810.99,1091.92 1811.22,1091.82 1811.45,1092.05 1811.68,1091.68 1811.9,1091.56 1812.13,1092.07 1812.36,1092.37 1812.59,1091.04 1812.82,1091.52 1813.04,1091.82 1813.27,1091.94 1813.5,1091.97 1813.73,1091.77 1813.95,1091.95 1814.18,1092.29 1814.41,1092.3 1814.64,1092.19 1814.87,1092.1 1815.09,1092.03 1815.32,1091.94 1815.55,1091.81 1815.78,1091.3 1816,1091.22 1816.23,1091.12 1816.46,1091.01 1816.69,1091.21 1816.92,1091.46 1817.14,1091.74 1817.37,1091.42 1817.6,1091.87 1817.83,1091.87 1818.05,1092.19 1818.28,1091.41 1818.51,1090.47 1818.74,1090.07 1818.97,1090.15 1819.19,1089.88 1819.42,1089.83 1819.65,1089.1 1819.88,1089.09 1820.1,1089.46 1820.33,1089.07 1820.56,1089.56 1820.79,1089.65 1821.02,1089.62 1821.24,1089.81 1821.47,1089.24 1821.7,1089.49 1821.93,1089.47 1822.15,1090.01 1822.38,1089.61 1822.61,1089.31 1822.84,1089.46 1823.07,1089.42 1823.29,1089.39 1823.52,1089.22 1823.75,1088.66 1823.98,1088.22 1824.2,1087.96 1824.43,1087.89 1824.66,1087.87 1824.89,1087.84 1825.12,1087.95 1825.34,1087.65 1825.57,1088.19 1825.8,1088.17 1826.03,1088.66 1826.25,1088.87 1826.48,1089.19 1826.71,1088.4 1826.94,1088.27 1827.17,1088.57 1827.39,1088.61 1827.62,1088.86 1827.85,1089.48 1828.08,1089.71 1828.3,1090.42 1828.53,1090.61 1828.76,1089.61 1828.99,1089.92 1829.22,1089.94 1829.44,1089.02 1829.67,1089.61 1829.9,1089.65 1830.13,1088.67 1830.35,1088.44 1830.58,1088.44 1830.81,1088.78 1831.04,1089.26 1831.27,1089.54 1831.49,1089.91 1831.72,1090.36 1831.95,1090.39 1832.18,1090.92 1832.4,1090.5 1832.63,1090.57 1832.86,1091.13 1833.09,1091.15 1833.32,1091.84 1833.54,1091.91 1833.77,1092.2 1834,1092.37 1834.23,1092.42 1834.45,1092.45 1834.68,1092.17 1834.91,1092.66 1835.14,1093.14 1835.37,1093.57 1835.59,1093.53 1835.82,1093.53 1836.05,1093.46 1836.28,1093.68 1836.5,1093.74 1836.73,1094.37 1836.96,1094.3 1837.19,1094.35 1837.42,1094.24 1837.64,1094.44 1837.87,1094.84 1838.1,1095.28 1838.33,1095.2 1838.55,1095.42 1838.78,1094.97 1839.01,1095.12 1839.24,1094.91 1839.46,1095.26 1839.69,1095.09 1839.92,1094.97 1840.15,1095.16 1840.38,1095.48 1840.6,1095.65 1840.83,1095.72 1841.06,1095.95 1841.29,1095.55 1841.51,1095.47 1841.74,1095.91 1841.97,1096.1 1842.2,1096.19 1842.43,1096.64 1842.65,1096.87 1842.88,1097.55 1843.11,1097.63 1843.34,1097.17 1843.56,1097.4 1843.79,1097.27 1844.02,1097.41 1844.25,1097.78 1844.48,1097.91 1844.7,1098.07 1844.93,1098.05 1845.16,1098.19 1845.39,1098.06 1845.61,1097.81 1845.84,1097.81 1846.07,1097.81 1846.3,1097.88 1846.53,1097.85 1846.75,1097.57 1846.98,1098.34 1847.21,1098 1847.44,1098.11 1847.66,1097.92 1847.89,1097.96 1848.12,1097.27 1848.35,1097.06 1848.58,1096.56 1848.8,1096.19 1849.03,1096.2 1849.26,1095.83 1849.49,1095.78 1849.71,1095.37 1849.94,1095.81 1850.17,1095.93 1850.4,1095.67 1850.63,1096.01 1850.85,1095.49 1851.08,1096.21 1851.31,1096.71 1851.54,1096.61 1851.76,1096.61 1851.99,1096.41 1852.22,1096.39 1852.45,1096.03 1852.68,1095.89 1852.9,1095.48 1853.13,1095.26 1853.36,1094.87 1853.59,1095.29 1853.81,1095.27 1854.04,1095.14 1854.27,1095 1854.5,1095.5 1854.73,1095.75 1854.95,1095.29 1855.18,1094.88 1855.41,1094.63 1855.64,1094.35 1855.86,1093.77 1856.09,1093.59 1856.32,1093.8 1856.55,1093.12 1856.78,1093.26 1857,1093.18 1857.23,1093.01 1857.46,1093.47 1857.69,1093.74 1857.91,1093.73 1858.14,1093.75 1858.37,1093.12 1858.6,1092.77 1858.83,1092.99 1859.05,1093.43 1859.28,1092.94 1859.51,1092.48 1859.74,1092.35 1859.96,1092.12 1860.19,1092.26 1860.42,1091.78 1860.65,1092.75 1860.88,1092.63 1861.1,1092.48 1861.33,1093.22 1861.56,1093.01 1861.79,1093.01 1862.01,1092.94 1862.24,1093.19 1862.47,1093.85 1862.7,1094.35 1862.93,1094.15 1863.15,1093.82 1863.38,1093.8 1863.61,1093.59 1863.84,1093.63 1864.06,1094.18 1864.29,1094.24 1864.52,1094.8 1864.75,1095.1 1864.98,1094.99 1865.2,1095.55 1865.43,1095.62 1865.66,1096.06 1865.89,1095.21 1866.11,1095.26 1866.34,1095.68 1866.57,1096.33 1866.8,1096.67 1867.03,1097.14 1867.25,1097.29 1867.48,1096.39 1867.71,1096.36 1867.94,1096.35 1868.16,1096.34 1868.39,1096.69 1868.62,1096.35 1868.85,1096.17 1869.08,1096.61 1869.3,1096.9 1869.53,1097.09 1869.76,1097.39 1869.99,1096.86 1870.21,1096.38 1870.44,1096.62 1870.67,1096.78 1870.9,1096.66 1871.13,1096.48 1871.35,1096.36 1871.58,1096.23 1871.81,1095.39 1872.04,1095.68 1872.26,1095.69 1872.49,1095.4 1872.72,1095.9 1872.95,1096.06 1873.18,1096.01 1873.4,1096.09 1873.63,1096.02 1873.86,1095.84 1874.09,1095.93 1874.31,1096.19 1874.54,1096.66 1874.77,1096.1 1875,1096.2 1875.23,1096.28 1875.45,1096.08 1875.68,1096.59 1875.91,1096.8 1876.14,1097.1 1876.36,1096.35 1876.59,1096.51 1876.82,1096.01 1877.05,1096.54 1877.27,1096.37 1877.5,1096.62 1877.73,1096.34 1877.96,1096.24 1878.19,1096.91 1878.41,1096.86 1878.64,1097.57 1878.87,1097.86 1879.1,1096.95 1879.32,1097 1879.55,1096.85 1879.78,1096.94 1880.01,1096.85 1880.24,1097.33 1880.46,1097.76 1880.69,1097.75 1880.92,1097.61 1881.15,1098.12 1881.37,1097.9 1881.6,1098.49 1881.83,1098.82 1882.06,1099.19 1882.29,1099.73 1882.51,1099.78 1882.74,1099.93 1882.97,1100.41 1883.2,1100.42 1883.42,1100.78 1883.65,1101.02 1883.88,1101.29 1884.11,1101.01 1884.34,1101.47 1884.56,1101.35 1884.79,1101.19 1885.02,1101.05 1885.25,1100.48 1885.47,1100.92 1885.7,1101.53 1885.93,1101.62 1886.16,1102.04 1886.39,1101.98 1886.61,1101.99 1886.84,1101.47 1887.07,1101.42 1887.3,1101.86 1887.52,1101.5 1887.75,1101.3 1887.98,1100.87 1888.21,1100.62 1888.44,1100.79 1888.66,1100.93 1888.89,1100.95 1889.12,1100.96 1889.35,1101.3 1889.57,1101.46 1889.8,1101.43 1890.03,1100.96 1890.26,1100.11 1890.49,1099.9 1890.71,1099.3 1890.94,1099.19 1891.17,1098.79 1891.4,1098.49 1891.62,1097.73 1891.85,1097.97 1892.08,1097.69 1892.31,1096.79 1892.54,1096.74 1892.76,1096.66 1892.99,1097.02 1893.22,1097.24 1893.45,1097.1 1893.67,1097.31 1893.9,1097.68 1894.13,1098.17 1894.36,1098.6 1894.59,1098.38 1894.81,1098.5 1895.04,1097.87 1895.27,1097.27 1895.5,1097.1 1895.72,1097.23 1895.95,1096.3 1896.18,1096.03 1896.41,1096.76 1896.64,1096.27 1896.86,1095.99 1897.09,1096.36 1897.32,1096.84 1897.55,1096.16 1897.77,1096.51 1898,1096.14 1898.23,1095.82 1898.46,1095.6 1898.69,1095.56 1898.91,1095.67 1899.14,1095.82 1899.37,1095.6 1899.6,1095.52 1899.82,1096.15 1900.05,1096.17 1900.28,1095.6 1900.51,1096.03 1900.74,1096.35 1900.96,1096.34 1901.19,1096.85 1901.42,1097.09 1901.65,1097.32 1901.87,1097.41 1902.1,1098 1902.33,1098.31 1902.56,1098.26 1902.79,1097.48 1903.01,1097.44 1903.24,1097.65 1903.47,1098.11 1903.7,1097.95 1903.92,1098.2 1904.15,1098.17 1904.38,1098.48 1904.61,1099.22 1904.84,1099.47 1905.06,1099.43 1905.29,1100 1905.52,1100.01 1905.75,1099.34 1905.97,1099.48 1906.2,1099.63 1906.43,1098.61 1906.66,1098.55 1906.89,1099.13 1907.11,1099.9 1907.34,1100.12 1907.57,1099.35 1907.8,1100.13 1908.02,1099.78 1908.25,1099.98 1908.48,1099.79 1908.71,1099.87 1908.94,1099.97 1909.16,1099.78 1909.39,1099.49 1909.62,1100.25 1909.85,1099.71 1910.07,1099.47 1910.3,1099.39 1910.53,1099.42 1910.76,1099.38 1910.99,1099.44 1911.21,1098.82 1911.44,1098.73 1911.67,1098.8 1911.9,1098.61 1912.12,1098.09 1912.35,1097.67 1912.58,1098.12 1912.81,1097.35 1913.03,1096.67 1913.26,1097.3 1913.49,1097.04 1913.72,1096.74 1913.95,1097.08 1914.17,1097.34 1914.4,1097.79 1914.63,1097.5 1914.86,1097.32 1915.08,1097.64 1915.31,1097.2 1915.54,1097.69 1915.77,1097.09 1916,1097.03 1916.22,1096.88 1916.45,1096.86 1916.68,1096.59 1916.91,1096.64 1917.13,1096.78 1917.36,1097.14 1917.59,1097.4 1917.82,1097.13 1918.05,1097.02 1918.27,1096.32 1918.5,1096.1 1918.73,1095.45 1918.96,1094.05 1919.18,1094.74 1919.41,1093.61 1919.64,1093.69 1919.87,1094.28 1920.1,1093.89 1920.32,1093.96 1920.55,1094.72 1920.78,1094.61 1921.01,1094.52 1921.23,1094.93 1921.46,1094.51 1921.69,1094.46 1921.92,1094.31 1922.15,1094.59 1922.37,1094.52 1922.6,1094.57 1922.83,1094.98 1923.06,1095.25 1923.28,1095.11 1923.51,1095.68 1923.74,1095.95 1923.97,1096.38 1924.2,1095.91 1924.42,1095.34 1924.65,1095.55 1924.88,1095.67 1925.11,1095.57 1925.33,1095.9 1925.56,1096.08 1925.79,1095.5 1926.02,1095.09 1926.25,1095.61 1926.47,1096.04 1926.7,1096.17 1926.93,1096.49 1927.16,1096.15 1927.38,1095.46 1927.61,1095.97 1927.84,1095.92 1928.07,1096 1928.3,1095.7 1928.52,1095.61 1928.75,1095.81 1928.98,1095.89 1929.21,1096.41 1929.43,1096.97 1929.66,1097.27 1929.89,1097.26 1930.12,1096.19 1930.35,1096.35 1930.57,1096.56 1930.8,1096.05 1931.03,1096.38 1931.26,1096.42 1931.48,1096.3 1931.71,1096.47 1931.94,1096.64 1932.17,1096.85 1932.4,1097.27 1932.62,1097.94 1932.85,1098.44 1933.08,1098.78 1933.31,1098.49 1933.53,1098.08 1933.76,1098.17 1933.99,1098.73 1934.22,1097.94 1934.45,1097.96 1934.67,1097.45 1934.9,1097.06 1935.13,1096.91 1935.36,1096.82 1935.58,1097.1 1935.81,1097.58 1936.04,1097.39 1936.27,1097.21 1936.5,1097.19 1936.72,1097.32 1936.95,1097.57 1937.18,1097.33 1937.41,1096.47 1937.63,1097.05 1937.86,1097.6 1938.09,1098.01 1938.32,1097.99 1938.55,1098.12 1938.77,1098.1 1939,1098.38 1939.23,1098.27 1939.46,1098.27 1939.68,1098.66 1939.91,1098.53 1940.14,1097.77 1940.37,1097.84 1940.6,1097.7 1940.82,1097.83 1941.05,1098.23 1941.28,1098.66 1941.51,1098.53 1941.73,1098.71 1941.96,1098.06 1942.19,1098.14 1942.42,1098.26 1942.65,1097.94 1942.87,1097.82 1943.1,1097.53 1943.33,1096.39 1943.56,1096.45 1943.78,1096.37 1944.01,1096.74 1944.24,1096.54 1944.47,1096.4 1944.7,1096.78 1944.92,1096.87 1945.15,1097.12 1945.38,1096.67 1945.61,1096.19 1945.83,1096.08 1946.06,1096.05 1946.29,1095.48 1946.52,1096.15 1946.75,1096.59 1946.97,1096.6 1947.2,1096.68 1947.43,1096.65 1947.66,1096.63 1947.88,1096.11 1948.11,1096.58 1948.34,1096.64 1948.57,1097.02 1948.79,1097.26 1949.02,1096.94 1949.25,1097.03 1949.48,1096.95 1949.71,1096.73 1949.93,1096.33 1950.16,1096.76 1950.39,1096.95 1950.62,1096.59 1950.84,1096.24 1951.07,1096.3 1951.3,1096.67 1951.53,1096.94 1951.76,1097.14 1951.98,1096.69 1952.21,1096.75 1952.44,1096.84 1952.67,1096.76 1952.89,1097.09 1953.12,1097.73 1953.35,1097.58 1953.58,1098.15 1953.81,1097.19 1954.03,1097.47 1954.26,1097.81 1954.49,1097.43 1954.72,1097.4 1954.94,1097.72 1955.17,1097.54 1955.4,1097.55 1955.63,1097.72 1955.86,1098.07 1956.08,1097.98 1956.31,1098 1956.54,1098.12 1956.77,1098.32 1956.99,1098.09 1957.22,1098.49 1957.45,1098.6 1957.68,1098.47 1957.91,1097.84 1958.13,1097.66 1958.36,1098.22 1958.59,1097.86 1958.82,1098.14 1959.04,1098.5 1959.27,1098.29 1959.5,1098.3 1959.73,1098.42 1959.96,1098.8 1960.18,1099.34 1960.41,1098.48 1960.64,1098.55 1960.87,1098.26 1961.09,1097.99 1961.32,1097.55 1961.55,1096.99 1961.78,1096.59 1962.01,1096.83 1962.23,1097.18 1962.46,1097.62 1962.69,1097.86 1962.92,1097.51 1963.14,1097.73 1963.37,1097.79 1963.6,1098.38 1963.83,1098.08 1964.06,1098.25 1964.28,1097.38 1964.51,1096.82 1964.74,1096.08 1964.97,1096.18 1965.19,1096.32 1965.42,1096.21 1965.65,1096.33 1965.88,1096.25 1966.11,1096.27 1966.33,1096.05 1966.56,1096.6 1966.79,1096.84 1967.02,1096.54 1967.24,1096.87 1967.47,1096.64 1967.7,1097.17 1967.93,1097.3 1968.16,1097.23 1968.38,1097.49 1968.61,1097.56 1968.84,1097.35 1969.07,1097.43 1969.29,1096.79 1969.52,1097.19 1969.75,1097.26 1969.98,1097.19 1970.21,1096.61 1970.43,1096.98 1970.66,1096.25 1970.89,1096.62 1971.12,1097.01 1971.34,1097.01 1971.57,1096.77 1971.8,1096.8 1972.03,1097.44 1972.26,1097.72 1972.48,1098.17 1972.71,1098.4 1972.94,1098.45 1973.17,1098.98 1973.39,1098.98 1973.62,1099.04 1973.85,1099.34 1974.08,1099.46 1974.31,1099.76 1974.53,1100.18 1974.76,1099.93 1974.99,1100.32 1975.22,1099.88 1975.44,1100.11 1975.67,1100.46 1975.9,1100.72 1976.13,1100.26 1976.36,1100.49 1976.58,1099.45 1976.81,1099.7 1977.04,1099.5 1977.27,1099.59 1977.49,1099.39 1977.72,1099.24 1977.95,1099.19 1978.18,1099.11 1978.41,1099.18 1978.63,1099.45 1978.86,1099.25 1979.09,1099.01 1979.32,1098.89 1979.54,1098.81 1979.77,1098.37 1980,1098.31 1980.23,1098.08 1980.46,1098.3 1980.68,1097.83 1980.91,1098.08 1981.14,1098.35 1981.37,1098.28 1981.59,1098.08 1981.82,1098.39 1982.05,1098.13 1982.28,1097.74 1982.51,1097.97 1982.73,1098.62 1982.96,1098.68 1983.19,1098.45 1983.42,1098.98 1983.64,1099.16 1983.87,1098.82 1984.1,1099.2 1984.33,1099.1 1984.56,1099.42 1984.78,1099.73 1985.01,1099.36 1985.24,1099.66 1985.47,1099.68 1985.69,1100.25 1985.92,1100.07 1986.15,1099.58 1986.38,1099.6 1986.6,1099.06 1986.83,1099.24 1987.06,1099.32 1987.29,1098.49 1987.52,1098.36 1987.74,1098.55 1987.97,1098.76 1988.2,1098.76 1988.43,1099.03 1988.65,1098.93 1988.88,1098.8 1989.11,1098.81 1989.34,1098.87 1989.57,1099 1989.79,1098.54 1990.02,1097.94 1990.25,1097.72 1990.48,1097.87 1990.7,1098.21 1990.93,1097.62 1991.16,1097.74 1991.39,1097.75 1991.62,1098.25 1991.84,1098.7 1992.07,1099.05 1992.3,1099.35 1992.53,1098.94 1992.75,1099.16 1992.98,1099.24 1993.21,1099.23 1993.44,1099.41 1993.67,1099.55 1993.89,1099.43 1994.12,1099.7 1994.35,1099.27 1994.58,1099.19 1994.8,1098.97 1995.03,1098.81 1995.26,1098.42 1995.49,1098.62 1995.72,1098.75 1995.94,1098.28 1996.17,1098.2 1996.4,1097.49 1996.63,1097.89 1996.85,1098.23 1997.08,1097.79 1997.31,1097.34 1997.54,1097.57 1997.77,1097.65 1997.99,1097.8 1998.22,1097.07 1998.45,1097.37 1998.68,1097.83 1998.9,1098.12 1999.13,1097.86 1999.36,1097.87 1999.59,1098.02 1999.82,1097.38 2000.04,1097.21 2000.27,1096.97 2000.5,1097.2 2000.73,1097.08 2000.95,1097.23 2001.18,1097.14 2001.41,1097.12 2001.64,1097.37 2001.87,1097.28 2002.09,1097.52 2002.32,1097.52 2002.55,1097.59 2002.78,1097.12 2003,1096.92 2003.23,1097.4 2003.46,1097.68 2003.69,1097.44 2003.92,1096.79 2004.14,1096.97 2004.37,1097.49 2004.6,1096.95 2004.83,1096.2 2005.05,1095.84 2005.28,1096.21 2005.51,1096.34 2005.74,1096.56 2005.97,1096.46 2006.19,1096.69 2006.42,1096.94 2006.65,1097.03 2006.88,1096.76 2007.1,1095.66 2007.33,1095.26 2007.56,1095 2007.79,1094.6 2008.02,1095.16 2008.24,1095.03 2008.47,1095.2 2008.7,1094.99 2008.93,1094.66 2009.15,1095.08 2009.38,1094.64 2009.61,1094.94 2009.84,1095.11 2010.07,1095.08 2010.29,1095.07 2010.52,1095.43 2010.75,1095.46 2010.98,1095.63 2011.2,1095.67 2011.43,1096.42 2011.66,1096.6 2011.89,1096.75 2012.12,1096.52 2012.34,1096.15 2012.57,1096.6 2012.8,1096.24 2013.03,1096.27 2013.25,1096.26 2013.48,1096.29 2013.71,1096.04 2013.94,1096.82 2014.17,1096.52 2014.39,1095.77 2014.62,1095.87 2014.85,1095.68 2015.08,1095.88 2015.3,1095.4 2015.53,1095.42 2015.76,1095.25 2015.99,1095.37 2016.22,1095.16 2016.44,1095.46 2016.67,1095.71 2016.9,1095.84 2017.13,1095.83 2017.35,1095.51 2017.58,1095.59 2017.81,1095.52 2018.04,1094.67 2018.27,1094.72 2018.49,1095.26 2018.72,1095.51 2018.95,1095.14 2019.18,1094.58 2019.4,1094.47 2019.63,1094.65 2019.86,1094.38 2020.09,1094.85 2020.32,1095.07 2020.54,1094.62 2020.77,1095.03 2021,1095.16 2021.23,1094.84 2021.45,1094.98 2021.68,1095.27 2021.91,1095.12 2022.14,1094.97 2022.36,1094.75 2022.59,1095.16 2022.82,1094.86 2023.05,1095.05 2023.28,1094.52 2023.5,1094.96 2023.73,1094.58 2023.96,1094.58 2024.19,1094.24 2024.41,1094.48 2024.64,1094.82 2024.87,1095.39 2025.1,1094.87 2025.33,1095.48 2025.55,1095.29 2025.78,1095.14 2026.01,1095.6 2026.24,1094.91 2026.46,1095.37 2026.69,1095.19 2026.92,1094.44 2027.15,1094.12 2027.38,1094.11 2027.6,1094 2027.83,1094.08 2028.06,1094.07 2028.29,1094.32 2028.51,1094.09 2028.74,1094.08 2028.97,1093.83 2029.2,1093.55 2029.43,1093.5 2029.65,1093.27 2029.88,1093.58 2030.11,1093.4 2030.34,1092.85 2030.56,1092.49 2030.79,1093.02 2031.02,1093.31 2031.25,1093.55 2031.48,1093.55 2031.7,1093.77 2031.93,1094.14 2032.16,1094.28 2032.39,1094.26 2032.61,1094.88 2032.84,1095.01 2033.07,1094.86 2033.3,1094.2 2033.53,1093.91 2033.75,1093.81 2033.98,1094.34 2034.21,1094.39 2034.44,1094.12 2034.66,1093.88 2034.89,1094.06 2035.12,1094.13 2035.35,1093.69 2035.58,1093.95 2035.8,1093.82 2036.03,1094.33 2036.26,1094.47 2036.49,1094.3 2036.71,1094.74 2036.94,1094.8 2037.17,1094.81 2037.4,1094.77 2037.63,1094.97 2037.85,1094.4 2038.08,1094.55 2038.31,1094.36 2038.54,1094.38 2038.76,1094.17 2038.99,1094.6 2039.22,1094.6 2039.45,1094.66 2039.68,1094.92 2039.9,1094.99 2040.13,1094.78 2040.36,1095 2040.59,1094.63 2040.81,1094.81 2041.04,1094.44 2041.27,1094.68 2041.5,1095.01 2041.73,1095.14 2041.95,1094.93 2042.18,1095.1 2042.41,1094.97 2042.64,1094.83 2042.86,1095.32 2043.09,1095.29 2043.32,1095.1 2043.55,1094.39 2043.78,1094.3 2044,1094.72 2044.23,1094.82 2044.46,1094.71 2044.69,1094.73 2044.91,1094.62 2045.14,1094.66 2045.37,1095.01 2045.6,1095.1 2045.83,1095.36 2046.05,1095.23 2046.28,1095.53 2046.51,1094.96 2046.74,1095.34 2046.96,1095.96 2047.19,1095.47 2047.42,1095.31 2047.65,1095.48 2047.88,1095.76 2048.1,1095.4 2048.33,1095.63 2048.56,1095.78 2048.79,1095.84 2049.01,1094.77 2049.24,1094.69 2049.47,1094.16 2049.7,1094.42 2049.93,1094.43 2050.15,1094.38 2050.38,1094.34 2050.61,1094.6 2050.84,1095.01 2051.06,1094.42 2051.29,1094.64 2051.52,1094.87 2051.75,1095.22 2051.98,1095.32 2052.2,1095.65 2052.43,1095.09 2052.66,1095.24 2052.89,1094.89 2053.11,1094.25 2053.34,1094.21 2053.57,1094.09 2053.8,1093.57 2054.03,1093.92 2054.25,1094.11 2054.48,1094.08 2054.71,1094.46 2054.94,1094.43 2055.16,1094.21 2055.39,1094.38 2055.62,1094.38 2055.85,1094.18 2056.08,1094.23 2056.3,1094.32 2056.53,1094.2 2056.76,1093.99 2056.99,1093.49 2057.21,1093.44 2057.44,1094.04 2057.67,1094.66 2057.9,1094.4 2058.12,1094.71 2058.35,1094.33 2058.58,1094.09 2058.81,1093.78 2059.04,1093.72 2059.26,1094.28 2059.49,1094.76 2059.72,1095.07 2059.95,1095.24 2060.17,1095.71 2060.4,1095.63 2060.63,1096.18 2060.86,1096.34 2061.09,1096.2 2061.31,1096.16 2061.54,1096.36 2061.77,1096.27 2062,1096.06 2062.22,1095.4 2062.45,1095.87 2062.68,1096.22 2062.91,1096.48 2063.14,1096.87 2063.36,1097.08 2063.59,1097.4 2063.82,1097.17 2064.05,1095.99 2064.27,1096.31 2064.5,1095.91 2064.73,1095.76 2064.96,1096.01 2065.19,1095.88 2065.41,1096.01 2065.64,1095.43 2065.87,1095.16 2066.1,1095 2066.32,1095.26 2066.55,1095.8 2066.78,1096.2 2067.01,1096.16 2067.24,1095.95 2067.46,1096.4 2067.69,1096.7 2067.92,1096.33 2068.15,1096.33 2068.37,1095.66 2068.6,1095.62 2068.83,1095.43 2069.06,1095.56 2069.29,1096.22 2069.51,1096.54 2069.74,1096.7 2069.97,1096.88 2070.2,1096.24 2070.42,1096.47 2070.65,1096.29 2070.88,1095.99 2071.11,1095.96 2071.34,1095.67 2071.56,1095.95 2071.79,1095.58 2072.02,1096.26 2072.25,1096.19 2072.47,1095.99 2072.7,1095.91 2072.93,1096.12 2073.16,1095.88 2073.39,1095.37 2073.61,1095.79 2073.84,1095.58 2074.07,1095.9 2074.3,1095.45 2074.52,1094.99 2074.75,1094.87 2074.98,1094.74 2075.21,1094.63 2075.44,1095.07 2075.66,1095.16 2075.89,1095.46 2076.12,1095.96 2076.35,1096.39 2076.57,1096.19 2076.8,1095.58 2077.03,1095.94 2077.26,1096.25 2077.49,1096.65 2077.71,1096.59 2077.94,1096.85 2078.17,1096.59 2078.4,1097.13 2078.62,1096.86 2078.85,1097.28 2079.08,1097.74 2079.31,1097.81 2079.54,1098.41 2079.76,1099.17 2079.99,1098.85 2080.22,1099.06 2080.45,1099.33 2080.67,1099.43 2080.9,1099.3 2081.13,1099.34 2081.36,1099.24 2081.59,1098.86 2081.81,1098.96 2082.04,1098.95 2082.27,1098.68 2082.5,1098.05 2082.72,1097.7 2082.95,1098.24 2083.18,1098.29 2083.41,1097.85 2083.64,1097.91 2083.86,1097.65 2084.09,1097.54 2084.32,1097.6 2084.55,1098.33 2084.77,1098.61 2085,1099.01 2085.23,1099.37 2085.46,1099.52 2085.69,1098.93 2085.91,1098.84 2086.14,1098.87 2086.37,1099 2086.6,1098.53 2086.82,1097.71 2087.05,1097.61 2087.28,1097.62 2087.51,1097.43 2087.74,1097.42 2087.96,1096.82 2088.19,1096.75 2088.42,1097.02 2088.65,1097.38 2088.87,1097.78 2089.1,1097.42 2089.33,1097.26 2089.56,1097.81 2089.79,1098.01 2090.01,1098.38 2090.24,1097.95 2090.47,1097.88 2090.7,1097.83 2090.92,1098.09 2091.15,1098.49 2091.38,1098.97 2091.61,1099.21 2091.84,1099.02 2092.06,1098.91 2092.29,1099.15 2092.52,1098.93 2092.75,1098.41 2092.97,1098.87 2093.2,1098.89 2093.43,1098.96 2093.66,1099.11 2093.89,1099.36 2094.11,1099.24 2094.34,1099.03 2094.57,1098.16 2094.8,1098.16 2095.02,1097.62 2095.25,1097.54 2095.48,1097.63 2095.71,1097.68 2095.93,1097.58 2096.16,1098.11 2096.39,1098.35 2096.62,1098.38 2096.85,1098.22 2097.07,1098.49 2097.3,1098.95 2097.53,1099.26 2097.76,1099.27 2097.98,1099.37 2098.21,1099.27 2098.44,1099.54 2098.67,1099.21 2098.9,1099.7 2099.12,1099.75 2099.35,1099.25 2099.58,1099.43 2099.81,1099.36 2100.03,1099.48 2100.26,1100.15 2100.49,1099.96 2100.72,1099.79 2100.95,1099.67 2101.17,1099.56 2101.4,1099.97 2101.63,1099.84 2101.86,1099.97 2102.08,1100.04 2102.31,1100.43 2102.54,1100.43 2102.77,1100.33 2103,1100.72 2103.22,1101.21 2103.45,1101.81 2103.68,1102.13 2103.91,1102.07 2104.13,1102.44 2104.36,1102.27 2104.59,1101.99 2104.82,1101.87 2105.05,1102.31 2105.27,1102.18 2105.5,1102.45 2105.73,1101.79 2105.96,1102.06 2106.18,1102.56 2106.41,1102.2 2106.64,1102.36 2106.87,1102.01 2107.1,1101.55 2107.32,1101.87 2107.55,1102.25 2107.78,1102.39 2108.01,1102.84 2108.23,1102.47 2108.46,1101.54 2108.69,1101.71 2108.92,1102.04 2109.15,1102.4 2109.37,1102.42 2109.6,1102.55 2109.83,1103.14 2110.06,1103.14 2110.28,1102.79 2110.51,1102.28 2110.74,1102.26 2110.97,1101.66 2111.2,1101.09 2111.42,1101.31 2111.65,1101.49 2111.88,1101.87 2112.11,1102.16 2112.33,1101.25 2112.56,1101.34 2112.79,1101.37 2113.02,1101.35 2113.25,1101 2113.47,1100.83 2113.7,1100.62 2113.93,1100.46 2114.16,1100.75 2114.38,1100.92 2114.61,1100.67 2114.84,1099.8 2115.07,1099.76 2115.3,1099.99 2115.52,1099.8 2115.75,1099.91 2115.98,1100.21 2116.21,1100.55 2116.43,1100.56 2116.66,1099.96 2116.89,1100.35 2117.12,1100.6 2117.35,1100.75 2117.57,1100.62 2117.8,1100.72 2118.03,1100.76 2118.26,1100.66 2118.48,1101.18 2118.71,1101.59 2118.94,1101.65 2119.17,1100.97 2119.4,1101.1 2119.62,1101.18 2119.85,1101.02 2120.08,1101.16 2120.31,1101.31 2120.53,1101 2120.76,1100.53 2120.99,1100.54 2121.22,1100.85 2121.45,1100.18 2121.67,1100.48 2121.9,1099.5 2122.13,1099.9 2122.36,1100.01 2122.58,1099.88 2122.81,1100.25 2123.04,1100.35 2123.27,1100.39 2123.5,1100.6 2123.72,1100.18 2123.95,1100.26 2124.18,1100.39 2124.41,1100.24 2124.63,1100.15 2124.86,1100.18 2125.09,1100.01 2125.32,1100.29 2125.55,1100.28 2125.77,1100.3 2126,1100.04 2126.23,1100.26 2126.46,1100.39 2126.68,1100.43 2126.91,1100.94 2127.14,1101.38 2127.37,1101.39 2127.6,1101.58 2127.82,1101.65 2128.05,1102.03 2128.28,1102.26 2128.51,1102.55 2128.73,1102.22 2128.96,1102.14 2129.19,1102.07 2129.42,1102.12 2129.65,1102.81 2129.87,1103.19 2130.1,1103.41 2130.33,1103.64 2130.56,1103.96 2130.78,1103.18 2131.01,1103.54 2131.24,1103.35 2131.47,1102.81 2131.69,1103.02 2131.92,1103.31 2132.15,1103.23 2132.38,1103.46 2132.61,1103.77 2132.83,1103.47 2133.06,1103.95 2133.29,1103.89 2133.52,1103.94 2133.74,1103.81 2133.97,1103.73 2134.2,1103.47 2134.43,1103.73 2134.66,1103.44 2134.88,1103.38 2135.11,1103.86 2135.34,1103.67 2135.57,1104.03 2135.79,1103.89 2136.02,1103.57 2136.25,1103.73 2136.48,1103.52 2136.71,1103.23 2136.93,1102.91 2137.16,1103.3 2137.39,1102.39 2137.62,1102.43 2137.84,1102.19 2138.07,1102.27 2138.3,1102.52 2138.53,1103.24 2138.76,1103.21 2138.98,1102.76 2139.21,1103.13 2139.44,1103.1 2139.67,1103.24 2139.89,1103.44 2140.12,1103.79 2140.35,1104.02 2140.58,1103.71 2140.81,1104.17 2141.03,1103.88 2141.26,1103.89 2141.49,1103.92 2141.72,1104.24 2141.94,1104.15 2142.17,1104.16 2142.4,1104.49 2142.63,1104.3 2142.86,1104.12 2143.08,1103.4 2143.31,1103.65 2143.54,1103.86 2143.77,1103.96 2143.99,1103.89 2144.22,1103.79 2144.45,1103.74 2144.68,1103.8 2144.91,1103.71 2145.13,1103.83 2145.36,1103.79 2145.59,1104.08 2145.82,1103.73 2146.04,1104 2146.27,1103.94 2146.5,1103.97 2146.73,1103.62 2146.96,1102.97 2147.18,1103.23 2147.41,1103.71 2147.64,1103.43 2147.87,1103.61 2148.09,1103.57 2148.32,1103.54 2148.55,1104.36 2148.78,1104.48 2149.01,1104.4 2149.23,1104.31 2149.46,1104.38 2149.69,1104.36 2149.92,1104.12 2150.14,1104.07 2150.37,1103.11 2150.6,1103.03 2150.83,1103.25 2151.06,1103.43 2151.28,1102.91 2151.51,1103.15 2151.74,1102.89 2151.97,1103.03 2152.19,1102.68 2152.42,1102.65 2152.65,1102.74 2152.88,1102.74 2153.11,1102.93 2153.33,1103.39 2153.56,1104.07 2153.79,1104.57 2154.02,1104.73 2154.24,1104.68 2154.47,1104.07 2154.7,1103.16 2154.93,1102.81 2155.16,1102.98 2155.38,1103.44 2155.61,1103.63 2155.84,1103.76 2156.07,1103.72 2156.29,1103.72 2156.52,1103.99 2156.75,1103.47 2156.98,1103.31 2157.21,1103.36 2157.43,1103.41 2157.66,1103.3 2157.89,1103.75 2158.12,1104.11 2158.34,1104.04 2158.57,1103.85 2158.8,1103.87 2159.03,1104.29 2159.26,1103.9 2159.48,1104.05 2159.71,1104.05 2159.94,1103.59 2160.17,1103.64 2160.39,1103.52 2160.62,1103 2160.85,1102.67 2161.08,1102.74 2161.31,1103.12 2161.53,1103.21 2161.76,1103.31 2161.99,1103.66 2162.22,1103.7 2162.44,1103.4 2162.67,1103.96 2162.9,1104.66 2163.13,1104.42 2163.36,1103.98 2163.58,1103.57 2163.81,1103.39 2164.04,1103.58 2164.27,1103.92 2164.49,1103.78 2164.72,1104.1 2164.95,1104.39 2165.18,1104.59 2165.41,1104.31 2165.63,1104.22 2165.86,1104.31 2166.09,1104.31 2166.32,1103.98 2166.54,1103.16 2166.77,1102.73 2167,1102.94 2167.23,1103.35 2167.45,1103.52 2167.68,1102.66 2167.91,1102.72 2168.14,1102.96 2168.37,1102.5 2168.59,1102.2 2168.82,1101.97 2169.05,1102.08 2169.28,1102.34 2169.5,1102.61 2169.73,1101.09 2169.96,1100.82 2170.19,1101.25 2170.42,1101.25 2170.64,1101.35 2170.87,1101.92 2171.1,1102.08 2171.33,1102.45 2171.55,1102.32 2171.78,1101.9 2172.01,1101.4 2172.24,1101.56 2172.47,1101.6 2172.69,1101.55 2172.92,1101.65 2173.15,1101.69 2173.38,1100.79 2173.6,1100.91 2173.83,1100.95 2174.06,1101.1 2174.29,1101.3 2174.52,1101.15 2174.74,1101.16 2174.97,1101.56 2175.2,1101.49 2175.43,1101.62 2175.65,1101.22 2175.88,1101.57 2176.11,1102.01 2176.34,1101.82 2176.57,1101.98 2176.79,1101.96 2177.02,1101.82 2177.25,1101.78 2177.48,1101.53 2177.7,1101.21 2177.93,1100.83 2178.16,1100.72 2178.39,1100.33 2178.62,1100.23 2178.84,1100.53 2179.07,1100.5 2179.3,1100.75 2179.53,1100.27 2179.75,1100.49 2179.98,1100.52 2180.21,1100.11 2180.44,1100.5 2180.67,1100.25 2180.89,1100.03 2181.12,1099.67 2181.35,1100.26 2181.58,1099.8 2181.8,1099.91 2182.03,1099.94 2182.26,1100.46 2182.49,1100.12 2182.72,1100.13 2182.94,1100.61 2183.17,1100.91 2183.4,1100.45 2183.63,1100.26 2183.85,1100.24 2184.08,1100.4 2184.31,1100.59 2184.54,1100.65 2184.77,1100.43 2184.99,1099.84 2185.22,1100.04 2185.45,1100.54 2185.68,1100.21 2185.9,1100.33 2186.13,1100.42 2186.36,1100.05 2186.59,1100.2 2186.82,1100.49 2187.04,1100.7 2187.27,1100.74 2187.5,1100.64 2187.73,1100.25 2187.95,1100.23 2188.18,1100.4 2188.41,1100.77 2188.64,1101.26 2188.87,1101.19 2189.09,1101.03 2189.32,1101.07 2189.55,1100.42 2189.78,1100.4 2190,1100.38 2190.23,1100.68 2190.46,1100.73 2190.69,1099.41 2190.92,1099.71 2191.14,1099.92 2191.37,1099.7 2191.6,1099.61 2191.83,1099.49 2192.05,1099.48 2192.28,1099.58 2192.51,1099.65 2192.74,1099.77 2192.97,1099.96 2193.19,1099.4 2193.42,1099.36 2193.65,1099.26 2193.88,1099.24 2194.1,1099.68 2194.33,1099.73 2194.56,1100.25 2194.79,1099.5 2195.02,1098.95 2195.24,1099.42 2195.47,1099.48 2195.7,1099.51 2195.93,1099.99 2196.15,1100.33 2196.38,1100.38 2196.61,1100.18 2196.84,1100.3 2197.07,1100.55 2197.29,1100.95 2197.52,1100.76 2197.75,1100.81 2197.98,1101.02 2198.2,1101.41 2198.43,1101.1 2198.66,1101.24 2198.89,1101.11 2199.12,1100.75 2199.34,1100.48 2199.57,1100.42 2199.8,1099.66 2200.03,1099.25 2200.25,1099.46 2200.48,1099.59 2200.71,1099.59 2200.94,1099.75 2201.17,1099.42 2201.39,1099.85 2201.62,1099.96 2201.85,1099.66 2202.08,1100.02 2202.3,1099.52 2202.53,1099.67 2202.76,1099.7 2202.99,1099.94 2203.21,1099.51 2203.44,1099.07 2203.67,1099.12 2203.9,1099.11 2204.13,1099.07 2204.35,1098.89 2204.58,1098.86 2204.81,1099.49 2205.04,1099.65 2205.26,1099.87 2205.49,1099.73 2205.72,1099.88 2205.95,1099.96 2206.18,1099.65 2206.4,1099.3 2206.63,1099.32 2206.86,1099.4 2207.09,1099.17 2207.31,1099.64 2207.54,1099.77 2207.77,1099.42 2208,1099.04 2208.23,1098.38 2208.45,1098.73 2208.68,1098.66 2208.91,1098.7 2209.14,1098.71 2209.36,1098.84 2209.59,1098.82 2209.82,1099.03 2210.05,1099.13 2210.28,1098.8 2210.5,1098.99 2210.73,1098.95 2210.96,1098.89 2211.19,1099.36 2211.41,1098.97 2211.64,1098.87 2211.87,1098.76 2212.1,1098.8 2212.33,1099.03 2212.55,1099.32 2212.78,1099.31 2213.01,1099.42 2213.24,1099.63 2213.46,1099.8 2213.69,1099.96 2213.92,1100.12 2214.15,1099.7 2214.38,1099.21 2214.6,1098.65 2214.83,1098.79 2215.06,1098.42 2215.29,1098.56 2215.51,1098.48 2215.74,1098.73 2215.97,1098.39 2216.2,1098.46 2216.43,1098.67 2216.65,1098.61 2216.88,1098.06 2217.11,1097.97 2217.34,1098.49 2217.56,1098.2 2217.79,1098.3 2218.02,1098.87 2218.25,1098.59 2218.48,1098.52 2218.7,1098.18 2218.93,1098.33 2219.16,1098.62 2219.39,1098.74 2219.61,1098.85 2219.84,1098.97 2220.07,1099.32 2220.3,1099.32 2220.53,1099.57 2220.75,1099.89 2220.98,1099.81 2221.21,1099.42 2221.44,1099.16 2221.66,1099.18 2221.89,1098.77 2222.12,1099.17 2222.35,1099.1 2222.58,1099.63 2222.8,1100.15 2223.03,1100.5 2223.26,1100.23 2223.49,1100.28 2223.71,1099.56 2223.94,1099.17 2224.17,1099.18 2224.4,1099.33 2224.63,1099.23 2224.85,1099.45 2225.08,1099.48 2225.31,1098.94 2225.54,1099.53 2225.76,1099.42 2225.99,1099.33 2226.22,1099.6 2226.45,1098.77 2226.68,1098.53 2226.9,1098.53 2227.13,1098.25 2227.36,1098.71 2227.59,1099.09 2227.81,1099.5 2228.04,1099.35 2228.27,1099.4 2228.5,1099.42 2228.73,1099.44 2228.95,1099.48 2229.18,1099.39 2229.41,1099.09 2229.64,1099.3 2229.86,1098.85 2230.09,1099.09 2230.32,1099.29 2230.55,1099.5 2230.78,1098.99 2231,1098.65 2231.23,1098.64 2231.46,1098.94 2231.69,1098.23 2231.91,1098.3 2232.14,1098.3 2232.37,1098.12 2232.6,1097.89 2232.83,1097.35 2233.05,1097.93 2233.28,1098.1 2233.51,1098.26 2233.74,1098.62 2233.96,1098.81 2234.19,1098.9 2234.42,1098.59 2234.65,1098.27 2234.88,1098.09 2235.1,1098.26 2235.33,1098.28 2235.56,1098.14 2235.79,1098.23 2236.01,1098.6 2236.24,1098.46 2236.47,1098.2 2236.7,1098.26 2236.93,1098.57 2237.15,1098.38 2237.38,1098.47 2237.61,1098.64 2237.84,1098.44 2238.06,1098.5 2238.29,1098.04 2238.52,1097.92 2238.75,1098.34 2238.98,1098.27 2239.2,1098.5 2239.43,1098.66 2239.66,1098.94 2239.89,1098.62 2240.11,1097.95 2240.34,1098.58 2240.57,1098.75 2240.8,1098.72 2241.02,1098.66 2241.25,1098.67 2241.48,1098.73 2241.71,1098.97 2241.94,1098.97 2242.16,1098.7 2242.39,1098.8 2242.62,1098.98 2242.85,1099.3 2243.07,1099.69 2243.3,1100.16 2243.53,1099.74 2243.76,1100.06 2243.99,1099.74 2244.21,1099.8 2244.44,1099.77 2244.67,1100.2 2244.9,1100.28 2245.12,1100.33 2245.35,1100.24 2245.58,1100.82 2245.81,1100.76 2246.04,1100.76 2246.26,1101.01 2246.49,1100.8 2246.72,1101 2246.95,1100.81 2247.17,1101.13 2247.4,1101.26 2247.63,1101.73 2247.86,1101.65 2248.09,1101.52 2248.31,1101.38 2248.54,1101.89 2248.77,1101.12 2249,1101.37 2249.22,1101.55 2249.45,1101.59 2249.68,1101.92 2249.91,1101.82 2250.14,1102.04 2250.36,1101.92 2250.59,1102.19 2250.82,1102.42 2251.05,1102.84 2251.27,1102.68 2251.5,1102.38 2251.73,1102.55 2251.96,1102.64 2252.19,1102.79 2252.41,1103.12 2252.64,1103.39 2252.87,1103.1 2253.1,1102.98 2253.32,1103.72 2253.55,1103.48 2253.78,1103.73 2254.01,1104.05 2254.24,1104.22 2254.46,1104.07 2254.69,1104.02 2254.92,1103.9 2255.15,1104.11 2255.37,1104.38 2255.6,1104.47 2255.83,1104.62 2256.06,1104.74 2256.29,1104.37 2256.51,1104.09 2256.74,1104.26 2256.97,1104.38 2257.2,1104.91 2257.42,1104.86 2257.65,1104.3 2257.88,1104.14 2258.11,1104.31 2258.34,1104.39 2258.56,1104.8 2258.79,1104.71 2259.02,1104.9 2259.25,1105.3 2259.47,1105.74 2259.7,1105.51 2259.93,1105.2 2260.16,1105.06 2260.39,1105 2260.61,1104.7 2260.84,1104.78 2261.07,1104.65 2261.3,1104.42 2261.52,1104.07 2261.75,1103.86 2261.98,1103.79 2262.21,1103.9 2262.44,1103.72 2262.66,1103.89 2262.89,1104.12 2263.12,1104.6 2263.35,1104.53 2263.57,1104.05 2263.8,1103.67 2264.03,1103.95 2264.26,1104.2 2264.49,1104.04 2264.71,1104.26 2264.94,1104.09 2265.17,1103.72 2265.4,1103.86 2265.62,1104.27 2265.85,1104.28 2266.08,1104.17 2266.31,1104.05 2266.54,1103.32 2266.76,1103.75 2266.99,1103.95 2267.22,1104.09 2267.45,1104.1 2267.67,1104.31 2267.9,1104.34 2268.13,1104.32 2268.36,1103.84 2268.59,1103.49 2268.81,1103.83 2269.04,1103.99 2269.27,1104.11 2269.5,1103.97 2269.72,1103.48 2269.95,1103.75 2270.18,1103.76 2270.41,1103.72 2270.64,1104.21 2270.86,1104.83 2271.09,1104.3 2271.32,1104.59 2271.55,1104.27 2271.77,1104.51 2272,1104.74 2272.23,1104.61 2272.46,1104.39 2272.69,1104.43 2272.91,1102.96 2273.14,1102.7 2273.37,1102.21 2273.6,1102.32 2273.82,1102.2 2274.05,1101.96 2274.28,1102.18 2274.51,1102.3 2274.74,1102.43 2274.96,1102.37 2275.19,1102.34 2275.42,1102.83 2275.65,1102.67 2275.87,1101.78 2276.1,1101.85 2276.33,1101.79 2276.56,1101.42 2276.78,1101.24 2277.01,1100.7 2277.24,1100.8 2277.47,1101.11 2277.7,1100.85 2277.92,1101 2278.15,1101.14 2278.38,1101.35 2278.61,1101.54 2278.83,1101.34 2279.06,1101.19 2279.29,1100.91 2279.52,1100.53 2279.75,1100.15 2279.97,1100.11 2280.2,1100.27 2280.43,1100.09 2280.66,1100.43 2280.88,1099.77 2281.11,1099.76 2281.34,1100.07 2281.57,1100.29 2281.8,1100.34 2282.02,1099.39 2282.25,1099.4 2282.48,1099.32 2282.71,1099.38 2282.93,1098.89 2283.16,1099.38 2283.39,1099.58 2283.62,1099.68 2283.85,1099.82 2284.07,1099.68 2284.3,1099.36 2284.53,1098.75 2284.76,1098.19 2284.98,1098.57 2285.21,1098.66 2285.44,1097.94 2285.67,1097.99 2285.9,1097.98 2286.12,1097.7 2286.35,1097.43 2286.58,1097.59 2286.81,1097.5 2287.03,1097.64 2287.26,1097.69 2287.49,1097.96 2287.72,1097.37 2287.95,1097.3 2288.17,1097.56 2288.4,1097.43 2288.63,1097.55 2288.86,1097.43 2289.08,1097.69 2289.31,1097.82 2289.54,1097.99 2289.77,1098.04 2290,1098.31 2290.22,1098.78 2290.45,1098.87 2290.68,1098.88 2290.91,1099.28 2291.13,1099.31 2291.36,1099.67 2291.59,1099.94 2291.82,1099.29 2292.05,1099.54 2292.27,1099.4 2292.5,1099.85 2292.73,1099.62 2292.96,1099.58 2293.18,1099.07 2293.41,1099.31 2293.64,1099.67 2293.87,1099.92 2294.1,1100.04 2294.32,1100.33 2294.55,1100.64 2294.78,1100.55 2295.01,1100.64 2295.23,1101 2295.46,1101.17 2295.69,1100.82 2295.92,1100.81 2296.15,1100.76 2296.37,1100.64 2296.6,1100.33 2296.83,1099.91 2297.06,1100.27 2297.28,1100.41 2297.51,1100.7 2297.74,1100.98 2297.97,1101.29 2298.2,1101.28 2298.42,1100.91 2298.65,1100.73 2298.88,1100.65 2299.11,1100.56 2299.33,1100.61 2299.56,1100.25 2299.79,1099.97 2300.02,1100.61 2300.25,1100.44 2300.47,1100.31 2300.7,1100.23 2300.93,1100.08 2301.16,1099.17 2301.38,1099.24 2301.61,1099.36 2301.84,1099.33 2302.07,1099.61 2302.3,1099.93 2302.52,1099.79 2302.75,1100.05 2302.98,1099.71 2303.21,1099.43 2303.43,1099.22 2303.66,1099.46 2303.89,1098.97 2304.12,1099.25 2304.35,1099.5 2304.57,1099.85 2304.8,1099.54 2305.03,1099.63 2305.26,1099.86 2305.48,1099.51 2305.71,1099.72 2305.94,1099.82 2306.17,1099.32 2306.4,1099.6 2306.62,1099.61 2306.85,1099.34 2307.08,1098.93 2307.31,1098.9 2307.53,1098.3 2307.76,1098.14 2307.99,1098.47 2308.22,1098.91 2308.45,1098.77 2308.67,1099.34 2308.9,1099.04 2309.13,1099.2 2309.36,1099 2309.58,1099.24 2309.81,1099.48 2310.04,1099.48 2310.27,1099.65 2310.5,1099.77 2310.72,1099.39 2310.95,1099.21 2311.18,1098.82 2311.41,1098.89 2311.63,1098.91 2311.86,1098.78 2312.09,1099.2 2312.32,1098.98 2312.54,1099.27 2312.77,1099.47 2313,1099.71 2313.23,1099.53 2313.46,1099.52 2313.68,1099.16 2313.91,1099.37 2314.14,1099.86 2314.37,1099.93 2314.59,1100.34 2314.82,1100.61 2315.05,1100.33 2315.28,1100.47 2315.51,1100.56 2315.73,1100.61 2315.96,1100.65 2316.19,1100.42 2316.42,1100.33 2316.64,1100.45 2316.87,1100.8 2317.1,1100.87 2317.33,1101.14 2317.56,1100.98 2317.78,1101.11 2318.01,1101.22 2318.24,1101.51 2318.47,1101.5 2318.69,1101.38 2318.92,1101.58 2319.15,1101.15 2319.38,1100.38 2319.61,1100.17 2319.83,1100.37 2320.06,1100.51 2320.29,1100.48 2320.52,1100.89 2320.74,1101.28 2320.97,1101.19 2321.2,1101.35 2321.43,1101.57 2321.66,1101.87 2321.88,1101.88 2322.11,1102.21 2322.34,1101.41 2322.57,1101.69 2322.79,1101.64 2323.02,1101.89 2323.25,1102.21 2323.48,1102.19 2323.71,1102.43 2323.93,1102.07 2324.16,1101.5 2324.39,1101.66 2324.62,1102.08 2324.84,1102.32 2325.07,1102.22 2325.3,1102.17 2325.53,1102.58 2325.76,1102.44 2325.98,1102.93 2326.21,1103.11 2326.44,1103.17 2326.67,1102.89 2326.89,1103.47 2327.12,1103.2 2327.35,1103.1 2327.58,1103.36 2327.81,1103.4 2328.03,1103.32 2328.26,1103.25 2328.49,1103.14 2328.72,1103.54 2328.94,1103.56 2329.17,1103.92 2329.4,1103.98 2329.63,1104.26 2329.86,1104.1 2330.08,1104.21 2330.31,1104.68 2330.54,1104.66 2330.77,1104.6 2330.99,1104.67 2331.22,1104.07 2331.45,1103.74 2331.68,1104.06 2331.91,1103.85 2332.13,1104.2 2332.36,1104.44 2332.59,1104.74 2332.82,1105.12 2333.04,1104.64 2333.27,1104.88 2333.5,1104.96 2333.73,1105.2 2333.96,1105.18 2334.18,1104.85 2334.41,1104.78 2334.64,1104.24 2334.87,1103.76 2335.09,1103.58 2335.32,1103.59 2335.55,1103.51 2335.78,1103.6 2336.01,1104.13 2336.23,1104.49 2336.46,1104.98 2336.69,1105.13 2336.92,1105.17 2337.14,1104.7 2337.37,1105.03 2337.6,1105.46 2337.83,1105.56 2338.06,1105.71 2338.28,1105.62 2338.51,1105.6 2338.74,1105.66 2338.97,1105.6 2339.19,1106.11 2339.42,1105.96 2339.65,1106.29 2339.88,1106.15 2340.11,1106.03 2340.33,1105.82 2340.56,1105.6 2340.79,1105.31 2341.02,1105.39 2341.24,1105.55 2341.47,1105.56 2341.7,1105.47 2341.93,1105.29 2342.16,1105.22 2342.38,1105.08 2342.61,1105.45 2342.84,1105.57 2343.07,1105.82 2343.29,1105.99 2343.52,1106.16 2343.75,1106.32 2343.98,1106.26 2344.21,1106.28 2344.43,1106.08 2344.66,1105.94 2344.89,1105.94 2345.12,1105.28 2345.34,1104.93 2345.57,1104.67 2345.8,1105.08 2346.03,1105.11 2346.26,1105.1 2346.48,1104.98 2346.71,1105.53 2346.94,1104.99 2347.17,1104.84 2347.39,1105.13 2347.62,1105.61 2347.85,1105.92 2348.08,1106.02 2348.31,1105.74 2348.53,1105.8 2348.76,1106.1 2348.99,1106.5 2349.22,1106.96 2349.44,1107.16 2349.67,1106.99 2349.9,1107.23 2350.13,1107.34 2350.35,1107.82 2350.58,1107.96 2350.81,1107.82 2351.04,1108.1 2351.27,1108.07 2351.49,1107.89 2351.72,1108.01 2351.95,1107.53 2352.18,1107.47 2352.4,1107.72 2352.63,1107.71 2352.86,1107.28 2353.09,1107.44 2353.32,1107.41 2353.54,1107.83 2353.77,1107.47 2354,1107.31 2354.23,1107.5 2354.45,1108.02 2354.68,1108.05 2354.91,1108.21 2355.14,1108.39 2355.37,1108.84 2355.59,1108.65 2355.82,1108.68 2356.05,1108.17 2356.28,1107.97 2356.5,1108.05 2356.73,1107.78 2356.96,1108.12 2357.19,1107.71 2357.42,1107.52 2357.64,1107.84 2357.87,1107.65 2358.1,1108.03 2358.33,1107.76 2358.55,1107.84 2358.78,1108.07 2359.01,1108.23 2359.24,1107.92 2359.47,1107.97 2359.69,1107.83 2359.92,1107.9 2360.15,1107.64 2360.38,1107.1 2360.6,1106.74 2360.83,1106.7 2361.06,1106.85 2361.29,1106.74 2361.52,1106.9 2361.74,1107.32 2361.97,1107.28 2362.2,1107.4 2362.43,1107.44 2362.65,1107.55 2362.88,1107.31 2363.11,1107.29 2363.34,1107.18 2363.57,1106.89 2363.79,1107.05 2364.02,1107.04 2364.25,1107.26 2364.48,1107.52 2364.7,1107.88 2364.93,1107.84 2365.16,1107.77 2365.39,1107.76 2365.62,1107.84 2365.84,1108 2366.07,1107.62 2366.3,1107.45 2366.53,1107.44 2366.75,1107.96 2366.98,1107.66 2367.21,1107.82 2367.44,1107.62 2367.67,1107.35 2367.89,1107.1 2368.12,1106.79 2368.35,1106.81 2368.58,1106.61 2368.8,1106.74 2369.03,1106.14 2369.26,1106.5 2369.49,1106.5 2369.72,1106.55 2369.94,1105.84 2370.17,1106.08 2370.4,1106.11 2370.63,1106.66 2370.85,1106.71 2371.08,1106.65 2371.31,1106.18 2371.54,1106.33 2371.77,1106.66 2371.99,1106.46 2372.22,1106.41 2372.45,1106.59 2372.68,1106.5 2372.9,1106.6 2373.13,1106.24 2373.36,1106.44 2373.59,1106.68 2373.82,1106.44 2374.04,1106.17 2374.27,1106.3 2374.5,1105.75 2374.73,1105.14 2374.95,1104.59 2375.18,1103.87 2375.41,1103.28 2375.64,1103.46 2375.87,1103.31 2376.09,1103.63 2376.32,1103.82 2376.55,1104.09 2376.78,1104.48 2377,1104.43 2377.23,1104.09 2377.46,1104.51 2377.69,1104.57 2377.92,1104.34 2378.14,1103.9 2378.37,1104.24 2378.6,1103.81 2378.83,1104.18 2379.05,1103.41 2379.28,1103.42 2379.51,1102.8 2379.74,1102.55 2379.97,1102.88 2380.19,1102.57 2380.42,1102.7 2380.65,1102.31 2380.88,1102.13 2381.1,1101.85 2381.33,1101.99 2381.56,1101.82 2381.79,1101.54 2382.02,1101.96 2382.24,1102.12 2382.47,1102.18 2382.7,1102.3 2382.93,1102.81 2383.15,1102.69 2383.38,1102.97 2383.61,1103.53 2383.84,1103.66 2384.07,1103.97 2384.29,1104.15 2384.52,1104.4 2384.75,1104.34 2384.98,1104.48 2385.2,1104.36 2385.43,1104.75 2385.66,1104.63 2385.89,1104.86 2386.11,1104.82 2386.34,1104.73 2386.57,1104.75 2386.8,1104.42 2387.03,1104.47 2387.25,1104.17 2387.48,1104.32 2387.71,1104.31 2387.94,1104.06 2388.16,1104.33 2388.39,1104.15 2388.62,1103.85 2388.85,1103.94 2389.08,1104.11 2389.3,1104.2 2389.53,1103.77 2389.76,1103.93 2389.99,1103.88 2390.21,1104.13 2390.44,1104.22 2390.67,1104.53 2390.9,1104.35 2391.13,1104.15 2391.35,1104.09 2391.58,1104.11 2391.81,1104.2 2392.04,1103.72 2392.26,1103.78 2392.49,1103.72 2392.72,1103.9 2392.95,1103.93 2393.18,1104.4 2393.4,1104.83 2393.63,1104.85 2393.86,1104.51 2394.09,1104.56 2394.31,1104.47 2394.54,1104.85 2394.77,1104.24 2395,1104.49 2395.23,1104.24 2395.45,1104.26 2395.68,1104.44 2395.91,1104.64 2396.14,1104.4 2396.36,1104.79 2396.59,1104.76 2396.82,1104.95 2397.05,1105.25 2397.28,1104.85 2397.5,1104.64 2397.73,1104.98 2397.96,1105.01 2398.19,1105.16 2398.41,1105.3 2398.64,1105.37 2398.87,1105.99 2399.1,1106.15 2399.33,1106.29 2399.55,1106.4 2399.78,1106.52 2400.01,1106.97 2400.24,1107.02 2400.46,1106.86 2400.69,1107.16 2400.92,1107.1 2401.15,1107.37 2401.38,1107.35 2401.6,1107.18 2401.83,1107.34 2402.06,1107.57 2402.29,1107.72 2402.51,1107.94 2402.74,1107.9 2402.97,1107.77 2403.2,1107.91 2403.43,1108.16 2403.65,1107.87 2403.88,1107.77 2404.11,1107.54 2404.34,1107.45 2404.56,1107.21 2404.79,1107.39 2405.02,1107.53 2405.25,1108.04 2405.48,1107.59 2405.7,1107.35 2405.93,1107.59 2406.16,1107.55 2406.39,1107.59 2406.61,1107.4 2406.84,1107.34 2407.07,1107.37 2407.3,1107.28 2407.53,1107.73 2407.75,1107.87 2407.98,1107.28 2408.21,1107.16 2408.44,1107.52 2408.66,1107.97 2408.89,1108.17 2409.12,1108.44 2409.35,1108.21 2409.58,1108.01 2409.8,1108.1 2410.03,1108.17 2410.26,1108.27 2410.49,1108.45 2410.71,1108.19 2410.94,1108.27 2411.17,1108.31 2411.4,1107.88 2411.63,1108.1 2411.85,1108.11 2412.08,1108.04 2412.31,1107.85 2412.54,1107.57 2412.76,1107.52 2412.99,1107.4 2413.22,1107.29 2413.45,1107.78 2413.68,1107.26 2413.9,1107.42 2414.13,1107.22 2414.36,1107.25 2414.59,1106.35 2414.81,1106.38 2415.04,1106.65 2415.27,1106.94 2415.5,1107.36 2415.73,1107.29 2415.95,1107.42 2416.18,1107.3 2416.41,1107 2416.64,1107.47 2416.86,1107.35 2417.09,1106.86 2417.32,1106.55 2417.55,1106.58 2417.78,1106.17 2418,1105.8 2418.23,1106.18 2418.46,1106 2418.69,1105.77 2418.91,1105.66 2419.14,1105.42 2419.37,1105.1 2419.6,1104.99 2419.83,1105.11 2420.05,1105.19 2420.28,1105.14 2420.51,1104.95 2420.74,1104.93 2420.96,1104.97 2421.19,1104.71 2421.42,1104.78 2421.65,1104.44 2421.87,1104.16 2422.1,1104.2 2422.33,1103.89 2422.56,1103.9 2422.79,1104.02 2423.01,1103.91 2423.24,1103.92 2423.47,1103.96 2423.7,1103.79 2423.92,1103.35 2424.15,1103.59 2424.38,1103.78 2424.61,1103.99 2424.84,1104 2425.06,1104.13 2425.29,1104.17 2425.52,1104.14 2425.75,1103.85 2425.97,1104.16 2426.2,1104.39 2426.43,1104.46 2426.66,1104.46 2426.89,1104.75 2427.11,1104.43 2427.34,1104.48 2427.57,1104.57 2427.8,1104.48 2428.02,1105.04 2428.25,1104.39 2428.48,1103.96 2428.71,1104.39 2428.94,1104.67 2429.16,1104.64 2429.39,1104.75 2429.62,1105.08 2429.85,1104.96 2430.07,1104.82 2430.3,1104.7 2430.53,1104.65 2430.76,1104.64 2430.99,1104.78 2431.21,1104.74 2431.44,1104.67 2431.67,1104.53 2431.9,1104.23 2432.12,1104.6 2432.35,1104.72 2432.58,1104.35 2432.81,1104.43 2433.04,1104.62 2433.26,1104.95 2433.49,1104.66 2433.72,1104.73 2433.95,1104.82 2434.17,1104.49 2434.4,1104.46 2434.63,1104.36 2434.86,1104.06 2435.09,1104.22 2435.31,1104.31 2435.54,1104.36 2435.77,1104.26 2436,1103.83 2436.22,1103.73 2436.45,1103.29 2436.68,1103.61 2436.91,1103.92 2437.14,1104.08 2437.36,1104.58 2437.59,1104.69 2437.82,1104.76 2438.05,1104.85 2438.27,1104.62 2438.5,1104.49 2438.73,1103.87 2438.96,1103.74 2439.19,1103.86 2439.41,1103.69 2439.64,1103.55 2439.87,1104.05 2440.1,1104.03 2440.32,1104.14 2440.55,1104.1 2440.78,1104.4 2441.01,1103.98 2441.24,1103.42 2441.46,1103.65 2441.69,1103.4 2441.92,1103.71 2442.15,1103.78 2442.37,1103.93 2442.6,1103.8 2442.83,1103.73 2443.06,1103.66 2443.29,1103.25 2443.51,1103.45 2443.74,1103.49 2443.97,1103.57 2444.2,1103.83 2444.42,1104.14 2444.65,1103.63 2444.88,1103.78 2445.11,1104.06 2445.34,1103.57 2445.56,1103.97 2445.79,1103.83 2446.02,1103.93 2446.25,1103.77 2446.47,1103.6 2446.7,1103.44 2446.93,1103.44 2447.16,1103.57 2447.39,1103.86 2447.61,1104.06 2447.84,1104.16 2448.07,1104.27 2448.3,1104.58 2448.52,1104.85 2448.75,1104.63 2448.98,1104.84 2449.21,1104.6 2449.44,1105.01 2449.66,1105.1 2449.89,1104.88 2450.12,1104.4 2450.35,1104.33 2450.57,1104.37 2450.8,1104.77 2451.03,1104.93 2451.26,1105.01 2451.49,1104.93 2451.71,1104.71 2451.94,1104.67 2452.17,1104.52 2452.4,1104.53 2452.62,1104.6 2452.85,1104.69 2453.08,1105.27 2453.31,1105.58 2453.54,1105.76 2453.76,1105.58 2453.99,1105.97 2454.22,1105.84 2454.45,1106.11 2454.67,1106.27 2454.9,1106.2 2455.13,1105.8 2455.36,1105.18 2455.59,1105.08 2455.81,1104.96 2456.04,1104.66 2456.27,1105.18 2456.5,1104.87 2456.72,1105.19 2456.95,1105.67 2457.18,1105.95 2457.41,1105.7 2457.64,1105.73 2457.86,1106.06 2458.09,1106.06 2458.32,1105.36 2458.55,1104.97 2458.77,1105.33 2459,1105.48 2459.23,1105.73 2459.46,1105.53 2459.68,1105.66 2459.91,1105.35 2460.14,1105.49 2460.37,1105.58 2460.6,1105.26 2460.82,1105.19 2461.05,1105.32 2461.28,1105.27 2461.51,1105.06 2461.73,1105.02 2461.96,1105.09 2462.19,1105.58 2462.42,1105.45 2462.65,1105.47 2462.87,1105.11 2463.1,1105.5 2463.33,1105.29 2463.56,1105.29 2463.78,1105.41 2464.01,1105.06 2464.24,1105.13 2464.47,1104.69 2464.7,1104.88 2464.92,1104.56 2465.15,1104.97 2465.38,1104.73 2465.61,1105.02 2465.83,1105.54 2466.06,1105.51 2466.29,1105.23 2466.52,1105.16 2466.75,1104.89 2466.97,1104.8 2467.2,1104.77 2467.43,1104.94 2467.66,1104.94 2467.88,1104.71 2468.11,1104.45 2468.34,1104.24 2468.57,1104.18 2468.8,1104.26 2469.02,1104.36 2469.25,1104.15 2469.48,1104.33 2469.71,1104.15 2469.93,1104.19 2470.16,1104.32 2470.39,1104.58 2470.62,1104.82 2470.85,1104.85 2471.07,1105.08 2471.3,1104.94 2471.53,1105.02 2471.76,1104.97 2471.98,1104.71 2472.21,1104.54 2472.44,1103.85 2472.67,1104.12 2472.9,1104.09 2473.12,1104.22 2473.35,1103.93 2473.58,1103.86 2473.81,1104.39 2474.03,1104.6 2474.26,1104.37 2474.49,1104.11 2474.72,1104.42 2474.95,1104.35 2475.17,1104.36 2475.4,1104.36 2475.63,1104.28 2475.86,1104.32 2476.08,1104.67 2476.31,1104.61 2476.54,1104.33 2476.77,1104.22 2477,1104.07 2477.22,1103.98 2477.45,1104.01 2477.68,1103.61 2477.91,1103.44 2478.13,1103.64 2478.36,1103.67 2478.59,1103.78 2478.82,1103.71 2479.05,1104.07 2479.27,1104.01 2479.5,1103.96 2479.73,1104.1 2479.96,1103.79 2480.18,1103.64 2480.41,1103.57 2480.64,1103.54 2480.87,1103.62 2481.1,1103.78 2481.32,1104 2481.55,1103.72 2481.78,1104.2 2482.01,1104.21 2482.23,1103.7 2482.46,1104.22 2482.69,1103.98 2482.92,1104.25 2483.15,1103.68 2483.37,1103.69 2483.6,1103.62 2483.83,1103.46 2484.06,1103.53 2484.28,1103.8 2484.51,1103.74 2484.74,1103.59 2484.97,1103.53 2485.2,1103.58 2485.42,1103.84 2485.65,1103.58 2485.88,1103.93 2486.11,1103.77 2486.33,1104.21 2486.56,1104.51 2486.79,1104.65 2487.02,1104.77 2487.25,1104.51 2487.47,1104.16 2487.7,1104.07 2487.93,1104.49 2488.16,1104.48 2488.38,1104.83 2488.61,1104.63 2488.84,1104.43 2489.07,1104.56 2489.3,1103.68 2489.52,1104.26 2489.75,1104.31 2489.98,1104.46 2490.21,1104.78 2490.43,1104.74 2490.66,1105.13 2490.89,1105.52 2491.12,1105.65 2491.35,1105.95 2491.57,1106.19 2491.8,1106.14 2492.03,1106.02 2492.26,1106.42 2492.48,1106.69 2492.71,1106.71 2492.94,1106.79 2493.17,1106.35 2493.4,1105.69 2493.62,1105.74 2493.85,1105.78 2494.08,1106.07 2494.31,1106.06 2494.53,1106.2 2494.76,1105.87 2494.99,1106.17 2495.22,1106.23 2495.44,1106.5 2495.67,1106.51 2495.9,1106.91 2496.13,1107.18 2496.36,1106.64 2496.58,1106.75 2496.81,1106.93 2497.04,1107.18 2497.27,1106.95 2497.49,1106.89 2497.72,1107.03 2497.95,1107.12 2498.18,1106.87 2498.41,1107.15 2498.63,1107.35 2498.86,1107.18 2499.09,1107.37 2499.32,1107.26 2499.54,1107.51 2499.77,1107.87 2500,1107.84 2500.23,1107.73 2500.46,1107.59 2500.68,1107.69 2500.91,1107.9 2501.14,1108.01 2501.37,1107.59 2501.59,1107.85 2501.82,1108.18 2502.05,1108.18 2502.28,1107.39 2502.51,1107.15 2502.73,1107.18 2502.96,1106.84 2503.19,1107.25 2503.42,1107.25 2503.64,1107.24 2503.87,1106.71 2504.1,1107.29 2504.33,1107.61 2504.56,1107.33 2504.78,1107.36 2505.01,1107.33 2505.24,1107.63 2505.47,1108.14 2505.69,1108.52 2505.92,1108.79 2506.15,1108.78 2506.38,1108.49 2506.61,1108.49 2506.83,1108.23 2507.06,1108.66 2507.29,1108.77 2507.52,1108.77 2507.74,1108.82 2507.97,1109.19 2508.2,1109.73 2508.43,1109.38 2508.66,1109.43 2508.88,1109.32 2509.11,1109.14 2509.34,1109.08 2509.57,1109.33 2509.79,1109.07 2510.02,1109.35 2510.25,1109.44 2510.48,1109.17 2510.71,1108.83 2510.93,1109.08 2511.16,1108.83 2511.39,1108.81 2511.62,1108.93 2511.84,1109.4 2512.07,1109.63 2512.3,1109.21 2512.53,1109.16 2512.76,1109.4 2512.98,1109.38 2513.21,1109.67 2513.44,1109.42 2513.67,1109.37 2513.89,1109.62 2514.12,1109.96 2514.35,1110.34 2514.58,1110.1 2514.81,1109.86 2515.03,1109.89 2515.26,1109.71 2515.49,1109.76 2515.72,1109.64 2515.94,1109.66 2516.17,1109.74 2516.4,1109.79 2516.63,1110.03 2516.86,1109.94 2517.08,1110 2517.31,1109.52 2517.54,1109.56 2517.77,1109.52 2517.99,1109.77 2518.22,1109.72 2518.45,1109.89 2518.68,1109.35 2518.91,1108.92 2519.13,1109.2 2519.36,1109.31 2519.59,1108.99 2519.82,1108.98 2520.04,1109.26 2520.27,1108.92 2520.5,1109.3 2520.73,1109.35 2520.96,1109.79 2521.18,1110.03 2521.41,1110.59 2521.64,1110.56 2521.87,1110.59 2522.09,1110.67 2522.32,1110.6 2522.55,1111.09 2522.78,1110.81 2523.01,1110.87 2523.23,1110.93 2523.46,1110.88 2523.69,1110.75 2523.92,1110.71 2524.14,1110.24 2524.37,1110.57 2524.6,1110.21 2524.83,1109.6 2525.06,1109.65 2525.28,1109.5 2525.51,1109.61 2525.74,1109.18 2525.97,1108.74 2526.19,1108.68 2526.42,1108.53 2526.65,1108.79 2526.88,1109.03 2527.11,1109.31 2527.33,1109.28 2527.56,1109.59 2527.79,1109.95 2528.02,1110.08 2528.24,1109.83 2528.47,1110.19 2528.7,1109.98 2528.93,1109.73 2529.16,1109.52 2529.38,1109.37 2529.61,1109.39 2529.84,1109.61 2530.07,1109.37 2530.29,1109.54 2530.52,1109.39 2530.75,1109.13 2530.98,1108.58 2531.2,1108.45 2531.43,1108.37 2531.66,1108.41 2531.89,1108.75 2532.12,1108.51 2532.34,1108.36 2532.57,1108.01 2532.8,1108.07 2533.03,1108.18 2533.25,1107.91 2533.48,1107.79 2533.71,1107.63 2533.94,1107.37 2534.17,1107.39 2534.39,1107.22 2534.62,1107.4 2534.85,1107.49 2535.08,1107.12 2535.3,1106.93 2535.53,1107.11 2535.76,1106.78 2535.99,1106.95 2536.22,1106.73 2536.44,1106.4 2536.67,1106.57 2536.9,1106.42 2537.13,1106.16 2537.35,1106.05 2537.58,1106.34 2537.81,1106.67 2538.04,1107.03 2538.27,1107.04 2538.49,1106.75 2538.72,1106.75 2538.95,1106.95 2539.18,1106.63 2539.4,1106.73 2539.63,1106.77 2539.86,1107.01 2540.09,1107.15 2540.32,1107.28 2540.54,1107.4 2540.77,1107.21 2541,1106.98 2541.23,1106.77 2541.45,1106.79 2541.68,1107.18 2541.91,1107.51 2542.14,1107.37 2542.37,1107.49 2542.59,1107.15 2542.82,1107.35 2543.05,1107.36 2543.28,1107.18 2543.5,1107.08 2543.73,1106.8 2543.96,1107.09 2544.19,1106.97 2544.42,1106.86 2544.64,1107.15 2544.87,1107.51 2545.1,1107.96 2545.33,1107.46 2545.55,1108.02 2545.78,1107.83 2546.01,1107.78 2546.24,1107.46 2546.47,1106.95 2546.69,1107.1 2546.92,1107.21 2547.15,1107.53 2547.38,1106.25 2547.6,1106.57 2547.83,1106.11 2548.06,1106.02 2548.29,1105.92 2548.52,1105.61 2548.74,1105.55 2548.97,1105.65 2549.2,1105.64 2549.43,1105.44 2549.65,1105.28 2549.88,1105.76 2550.11,1105.75 2550.34,1105.8 2550.57,1106.29 2550.79,1106.29 2551.02,1106.41 2551.25,1107 2551.48,1107.14 2551.7,1107.18 2551.93,1107.1 2552.16,1106.71 2552.39,1106.66 2552.62,1106.71 2552.84,1106.4 2553.07,1106.09 2553.3,1106.09 2553.53,1106.12 2553.75,1106.49 2553.98,1105.92 2554.21,1105.78 2554.44,1105.8 2554.67,1105.6 2554.89,1105.36 2555.12,1105.32 2555.35,1105.26 2555.58,1105.13 2555.8,1104.81 2556.03,1104.87 2556.26,1105.05 2556.49,1105.36 2556.72,1105.28 2556.94,1105.15 2557.17,1104.99 2557.4,1104.79 2557.63,1104.87 2557.85,1104.54 2558.08,1104.37 2558.31,1103.9 2558.54,1103.73 2558.77,1103.99 2558.99,1104.33 2559.22,1104.68 2559.45,1104 2559.68,1104.09 2559.9,1104.37 2560.13,1104.21 2560.36,1104.02 2560.59,1104.32 2560.82,1104.45 2561.04,1104.87 2561.27,1104.76 2561.5,1105.16 2561.73,1104.81 2561.95,1104.94 2562.18,1104.95 2562.41,1105.13 2562.64,1105.22 2562.87,1104.96 2563.09,1104.88 2563.32,1105.11 2563.55,1105.76 2563.78,1105.8 2564,1105.98 2564.23,1106.27 2564.46,1106.46 2564.69,1106.35 2564.92,1106.35 2565.14,1105.95 2565.37,1105.72 2565.6,1105.77 2565.83,1105.77 2566.05,1106.25 2566.28,1106.13 2566.51,1106.62 2566.74,1106.69 2566.96,1106.76 2567.19,1106.75 2567.42,1106.79 2567.65,1106.43 2567.88,1106.7 2568.1,1106.19 2568.33,1106.29 2568.56,1106.68 2568.79,1107 2569.01,1106.63 2569.24,1106.07 2569.47,1106.4 2569.7,1106.5 2569.93,1106.65 2570.15,1106.94 2570.38,1106.4 2570.61,1106.67 2570.84,1106.15 2571.06,1106.35 2571.29,1106.53 2571.52,1106.75 2571.75,1106.55 2571.98,1106.47 2572.2,1106.18 2572.43,1106.29 "/>
</svg>

Figure 4: Monte Carlo mean and 95% confidence interval of the simulated
temperature in 2100 as the number of samples increases.

1.  Here we can see the benefits of setting the variable `n_samples`
    above; if we had hard-coded the value and needed to change it, we
    would also have needed to change it in all three of these lines.
2.  There are more efficient ways to do this updating, for example
    `T_est[i] = (T_est[i-1] * (i-1) + T_est[i]) / i`, which avoids
    re-computing the average for the first `i-1` simulations, but in
    this case it’s fast enough.
3.  We can’t compute the standard deviation for a single value, so we
    just start with `i == 2`.

If we wanted tighter confidence intervals, we could run the simulation
longer (though recall the $1/\sqrt{n}$ error law, which means to get an
order of magnitude reduction, we’d need approximately 100,000 samples),
but it looks from <a href="#fig-mc-est" class="quarto-xref">Figure 4</a>
as though the estimate had stabilized around 8,000-9,000 samples, though
ultimately it depends on the desired level of precision in both the
estimate and the confidence interval. In this case, regardless, we’re
looking at an estimated temperature anomaly of around $4.7^\circ C$ once
we exceed 5,000 samples, and it isn’t clear what more precision
necessarily would get us.

### Problem 3 (10 points)

**This problem is only required for students in BEE 5750**.

A factory is planning a third wastewater discharge into the river
downstream of the second plant. This discharge would consist of 5
m<sup>3</sup>/day of wastewater with a dissolved oxygen content of 4.5
mg/L and CBOD and NBOD levels of 50 and 45 mg/L, respectively.

**In this problem**:

-   Assume that the treatment plan for waste stream 2 that you
    identified in Problem 1 is still in place for the existing
    discharges. If the third discharge will not be treated, under the
    original inflow conditions (7.5 mg/L DO), how far downstream from
    the second discharge does this third discharge need to be placed to
    keep the river concentration from dropping below 4 mg/L?

#### Solution

Based on how we solved Problem 1, we need to modify our solution to
include a third discharge with an unknown distance from the second
discharge.

> **Tip**
>
> We could have solved Problem 1 in a more modular way, which included
> multiple discharges (using a vector of tuples instead of explicitly
> specifying `waste1` and `waste2`, which would have made this solution
> simpler).

``` julia
# do_simulate: function to simulate dissolved oxygen concentrations over the entire river
# inputs:
#   - d: distance between waste stream 2 and waste stream 3
#   - inflow: tuple with inflow properties: (Volume, DO, CBOD, NBOD)
#   - waste1: tuple with waste stream 1 properties: (Volume, DO, CBOD, NBOD)
#   - waste2: tuple with waste stream 2 properties: (Volume, DO, CBOD, NBOD)
#   - waste3: tuple with waste stream 3 properties: (Volume, DO, CBOD, NBOD)
#   - C₀, B₀, N₀: initial conditions for DO, CBOD, and NBOD, respectively (mg/L)
#   - U: velocity of river (km/d)
#   - Cₛ = saturation oxygen concentration (mg/L)
#   - ka, kc, kn: reaeration, CBOD decay, and NBOD decay rates, respectively (d^{-1})
function do_simulate2(d, inflow, waste1, waste2, waste3, U, Cₛ, ka, kb, kn)
    # set up ranges for each box/segment
    x₁ = 0:1:15
    x₂ = (15:1:15+d) .- 15 
    x₃ = (15+d:1:50) .- (15 + d)


    V_inflow, C_inflow, B_inflow, N_inflow = inflow
    V_ws1, C_ws1, B_ws1, N_ws1 = waste1
    V_ws2, C_ws2, B_ws2, N_ws2 = waste2
    V_ws3, C_ws3, B_ws3, N_ws3 = waste2

    # initialize storage for final C, B, and N
    # need to store d=1 so the total length should be d+1
    C = zeros(51)
    B = zeros(51)
    N = zeros(51)

    # compute initial conditions for first segment
    C₀ = mix_concentration(V_inflow, C_inflow, V_ws1, C_ws1)
    B₀ = mix_concentration(V_inflow, B_inflow, V_ws1, B_ws1)
    N₀ = mix_concentration(V_inflow, N_inflow, V_ws1, N_ws1)

    # conduct first segment simulation
    (C₁, B₁, N₁) = dissolved_oxygen(x₁, C₀, B₀, N₀, U, Cₛ, ka, kb, kn)
    C[1:15] = C₁[1:end-1] 
    B[1:15] = B₁[1:end-1]
    N[1:15] = N₁[1:end-1]

    # compute initial conditions for second segment
    C₀₁ = mix_concentration(V_inflow + V_ws1, C₁[end], V_ws2, C_ws2)
    B₀₁ = mix_concentration(V_inflow + V_ws1, B₁[end], V_ws2, B_ws2)
    N₀₁ = mix_concentration(V_inflow + V_ws1, N₁[end], V_ws2, N_ws2)

    # conduct second segment simulation
    (C₂, B₂, N₂) = dissolved_oxygen(x₂, C₀₁, B₀₁, N₀₁, U, Cₛ, ka, kb, kn)
    C[16:16+d] = C₂ 
    B[16:16+d] = B₂
    N[16:16+d] = N₂

    # compute initial conditions for third segment
    C₀₂ = mix_concentration(V_inflow + V_ws1 + V_ws2, C₂[end], V_ws3, C_ws3)
    B₀₂ = mix_concentration(V_inflow + V_ws1 + V_ws2, B₂[end], V_ws3, B_ws3)
    N₀₂ = mix_concentration(V_inflow + V_ws1 + V_ws2, N₂[end], V_ws3, N_ws3)

    # conduct second segment simulation
    (C₃, B₃, N₃) = dissolved_oxygen(x₃, C₀₂, B₀₂, N₀₂, U, Cₛ, ka, kb, kn)
    C[16+d:end] = C₃ 
    B[16+d:end] = B₃
    N[16+d:end] = N₃


    return (C, B, N)
end

# set up treatment level for waste stream 2 based on solution for Problem 1
B_ws2 = (1 - 0.08) * 45.0
N_ws2 = (1 - 0.08) * 35.0
waste2 = (V_ws2, C_ws2, B_ws2, N_ws2)

# set parameters for waste stream 3
C_ws3 = 4.5
B_ws3 = 50.0
N_ws3 = 45.0
V_ws3 = 5.0 * 1_000 # convert from m^3/day to L/day
waste3 = (V_ws3, C_ws3, B_ws3, N_ws3)
```

    (5000.0, 4.5, 50.0, 45.0)

Now we can evaluate `do_simulate2` over the different values of `d` to
find the minimum value of `C`.

``` julia
# three_discharge_minC: function to find the minimum DO concentration based on the three discharge model
# inputs:
#   - d: distance between waste stream 2 and waste stream 3
#   - inflow: tuple with inflow properties: (Volume, DO, CBOD, NBOD)
#   - waste1: tuple with waste stream 1 properties: (Volume, DO, CBOD, NBOD)
#   - waste2: tuple with waste stream 2 properties: (Volume, DO, CBOD, NBOD)
#   - waste3: tuple with waste stream 3 properties: (Volume, DO, CBOD, NBOD)
#   - C₀, B₀, N₀: initial conditions for DO, CBOD, and NBOD, respectively (mg/L)
#   - U: velocity of river (km/d)
#   - Cₛ = saturation oxygen concentration (mg/L)
#   - ka, kc, kn: reaeration, CBOD decay, and NBOD decay rates, respectively (d^{-1})
function three_discharge_minC(d, inflow, waste1, waste2, waste3, U, Cₛ, ka, kb, kn)
    C, B, N = do_simulate2(d, inflow, waste1, waste2, waste3, U, Cₛ, ka, kb, kn)
    return minimum(C)
end

# evaluate over all distances from 1 to 35 m downstream using broadcasting and anonymous functions
d = 1:1:35
minC_d = (dist -> three_discharge_minC(dist, inflow, waste1, waste2, waste3, U, Cₛ, ka, kb, kn)).(d)
idx = findfirst(minC_d .>= 4.0)
min_dist = d[idx]
@show min_dist;
```

    min_dist = 14

So, without treatment, the third discharge should be placed 14 m
downstream from the second discharge.

## References

List any external references consulted, including classmates.